# Homework 3 Solutions

**Name**:

**ID**:

> **Due Date**
>
> Thursday, 10/03/23, 9:00pm

## Overview

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [1]:
using Random
using CSV
using DataFrames
using Plots
using LaTeXStrings
using Distributions

## Problems (Total: 50/60 Points)

### Problem 1 (30 points)

The first step is to write function(s) to implement the dissolved oxygen
simulation. With two releases, we can turn this into a two-box model,
with the first box from the initial waste release ($x=0\ \text{km}$) to
the second release ($x=15\ \text{km}$), and the second from the second
release to the end of the domain ($x=50\ \text{km}$). As a result, our
lives will be easiest if we write a function to simulate each box with
appropriate initial conditions, which we can then call for each river
segment. An example of how this might look is below. Note that we need
to compute B and N as well to get the appropriate initial conditions at
the transition between boxes (and this might also help with debugging).

``` julia
# mix_concentration: function to compute initial conditions by mixing inflow and new waste stream concentrations
# inputs:
#   - arguments ending in "_in" are inflow, those ending in "_st" are from the stream
#   - V is the volume (L/d), Q is the relevant concentration (mg/L); these should be Floats
# outputs:
#   - mixed concentration (a Float) in mg/L
function mix_concentration(V_in, Q_in, V_st, Q_st)
    Q_mix = ((V_in * Q_in) + (V_st * Q_st)) / (V_in + V_st)
    return Q_mix
end

# dissolved_oxygen: function to simulate dissolved oxygen concentrations for a given segment
# inputs:
#   - x: vector or range of downstream distances to simulate over
#   - C₀, B₀, N₀: initial conditions for DO, CBOD, and NBOD, respectively (mg/L)
#   - U: velocity of river (km/d)
#   - Cₛ = saturation oxygen concentration (mg/L)
#   - ka, kc, kn: reaeration, CBOD decay, and NBOD decay rates, respectively (d^{-1})
function dissolved_oxygen(x, C₀, B₀, N₀, U, Cₛ, ka, kc, kn)

    # initialize vectors for C, B, and N
    # the zeros function lets us define a vector of the appropriate length with values set to zero
    C = zeros(length(x))
    B = zeros(length(x))
    N = zeros(length(x))

    # compute α values for the simulation
    α₁ = exp.(-ka * x / U)
    α₂ = (kc / (ka - kc)) * (exp.(-kc * x / U) .- α₁)
    α₃ = (kn / (ka - kn)) * (exp.(-kn * x / U) .- α₁)

    # loop over values in x to calculate B, N, and C
    for (i, d) in pairs(x)
        B[i] = B₀ * exp(-kc * x[i] / U)
        N[i] = N₀ * exp(-kn * x[i] / U)
        C[i] = Cₛ * (1 - α₁[i]) + (C₀ * α₁[i]) - (B₀ * α₂[i]) - (N₀ * α₃[i])
    end

    return (C, B, N)
end
```

    dissolved_oxygen (generic function with 1 method)

1.  These will be vectors due to the broadcasting of `exp` and `-` over
    the `x` vector. We could also have computed the `α` values in the
    loop below for each value of `x`.
2.  `pairs(x)` lets us directly iterate over indices (`i`) and values
    (`d`) in the vector `x`, rather than only iterating over indices and
    needing to look up the values `d=x[i]`.
3.  While we don’t need `B` and `N` for this solution, returning this
    tuple can be useful for debugging.

Next, let’s simulate the concentrations. Hopefully this is intuitive,
but one critical thing is that we need to compute the initial segment
from $x=0$ to $15\ \text{km}$, not just to 14, as $x=15$ is the inflow
for the initial condition of the segment after the second waste stream.

> **Note**
>
> I initially wrote this out as a script to debug, but then reformulated
> it as a function with an optional parameters for treatment of the two
> waste streams to solve Problems 2 and 3, which means I didn’t have to
> copy and paste everything, possibly introducing new bugs.

``` julia
# do_simulate: function to simulate dissolved oxygen concentrations over the entire river
# inputs:
#   - inflow: tuple with inflow properties: (Volume, DO, CBOD, NBOD)
#   - waste1: tuple with waste stream 1 properties: (Volume, DO, CBOD, NBOD)
#   - waste2: tuple with waste stream 2 properties: (Volume, DO, CBOD, NBOD)
#   - U: velocity of river (km/d)
#   - Cₛ = saturation oxygen concentration (mg/L)
#   - ka, kc, kn: reaeration, CBOD decay, and NBOD decay rates, respectively (d^{-1})
function do_simulate(inflow, waste1, waste2, U, Cₛ, ka, kc, kn)
    # set up ranges for each box/segment
    x₁ = 0:1:15
    x₂ = (15:1:50) .- 15

    V_inflow, C_inflow, B_inflow, N_inflow = inflow
    V_ws1, C_ws1, B_ws1, N_ws1 = waste1
    V_ws2, C_ws2, B_ws2, N_ws2 = waste2

    # initialize storage for final C, B, and N
    # need to store d=0 so the total length should be d+1
    C = zeros(51)
    B = zeros(51)
    N = zeros(51)

    # compute initial conditions for first segment
    C₀ = mix_concentration(V_inflow, C_inflow, V_ws1, C_ws1)
    B₀ = mix_concentration(V_inflow, B_inflow, V_ws1, B_ws1)
    N₀ = mix_concentration(V_inflow, N_inflow, V_ws1, N_ws1)

    # conduct first segment simulation
    (C₁, B₁, N₁) = dissolved_oxygen(x₁, C₀, B₀, N₀, U, Cₛ, ka, kc, kn)
    C[1:15] = C₁[1:end-1]
    B[1:15] = B₁[1:end-1]
    N[1:15] = N₁[1:end-1]

    # compute initial conditions for second segment
    C₀₁ = mix_concentration(V_inflow + V_ws1, C₁[end], V_ws2, C_ws2)
    B₀₁ = mix_concentration(V_inflow + V_ws1, B₁[end], V_ws2, B_ws2)
    N₀₁ = mix_concentration(V_inflow + V_ws1, N₁[end], V_ws2, N_ws2)

    # conduct second segment simulation
    (C₂, B₂, N₂) = dissolved_oxygen(x₂, C₀₁, B₀₁, N₀₁, U, Cₛ, ka, kc, kn)
    C[16:end] = C₂
    B[16:end] = B₂
    N[16:end] = N₂

    return (C, B, N)
end

# set variables for river dynamics
U = 6
Cₛ = 10
ka = 0.55
kc = 0.35
kn = 0.25

# set initial parameters
C_inflow = 7.5 # DO concentration
B_inflow = 5.0 # CBOD
N_inflow = 5.0 # NBOD
V_inflow = 100 * 1_000 # volume converted to L
inflow = (V_inflow, C_inflow, B_inflow, N_inflow)

# set waste stream parameters
C_ws1 = 5.0
B_ws1 = 50.0
N_ws1 = 35.0
V_ws1 = 10 * 1_000
waste1 = (V_ws1, C_ws1, B_ws1, N_ws1)

C_ws2 = 4.0
B_ws2 = 45.0
N_ws2 = 35.0
V_ws2 = 15 * 1_000
waste2 = (V_ws2, C_ws2, B_ws2, N_ws2)

C, B, N = do_simulate(inflow, waste1, waste2, U, Cₛ, ka, kc, kn)
```

    ([7.2727272727272725, 6.718366940001292, 6.252736478441488, 5.865982487427475, 5.549224625451668, 5.294464471079079, 5.094502688374758, 4.942863751725384, 4.8337275512230455, 4.761867260163488  …  5.965177734979926, 6.111409254214727, 6.254932119301499, 6.395478206784325, 6.532828783895545, 6.66680882593059, 6.797281903682192, 6.924145587090053, 7.047327316192408, 7.166780694952493], [9.090909090909092, 8.575776817031748, 8.089834281709308, 7.63142746153825, 7.198996057607107, 6.791068184640297, 6.4062553610792135, 6.043247783048131, 5.700809866118693, 5.377776039698319  …  1.916804331604075, 1.8081894764731885, 1.7057292332453498, 1.6090748536058688, 1.5178973509064155, 1.4318863803790853, 1.350749182802631, 1.274209588025372, 1.2020070749530312, 1.1338958848019383], [7.7272727272727275, 7.411918532206977, 7.109434113044771, 6.819294247244602, 6.540995146882018, 6.27405358389127, 6.018006051006311, 5.772407956944252, 5.536832854433826, 5.3108716997484215  …  2.653493696687388, 2.5452031783680957, 2.4413320548913466, 2.3416999683543676, 2.246133921358312, 2.1544679766220987, 2.0665429688551744, 1.9822062283889013, 1.9013113160867028, 1.8237177690726656])

1.  The colon syntax sets up the range using the syntax
    `initial_value:stepsize:end_value`. In general a stepsize of 1 is
    implicit, but I’ve made it explicit here for illustration.
2.  This starts at 0 because we care about the distance from the initial
    condition, not the “absolute” distance.
3.  Tuples (including multiple outputs from functions) can be unpacked
    into multiple variables this way to make the subsequent code more
    readable (versus just relying on indexing).
4.  We don’t need to store the last value because that occurs at the
    point of mixing with the next waste stream. We will use it to
    compute the mixed concentration at that point.

Now we can plot the dissolved oxygen concentration, shown in
<a href="#fig-do-untreated" class="quarto-xref">Figure 1</a>.

``` julia
# create plot axis with labels, etc
p = plot(; xlabel="Distance Downriver (km)", ylabel="Dissolved Oxygen Concentration (mg/L)", legend=:top)
plot!(p, 0:50, C, label="Simulated DO")
hline!([4], label="Regulatory Standard")
```

<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="672" height="480" viewBox="0 0 2688 1920">
<defs>
  <clipPath id="clip820">
    <rect x="0" y="0" width="2688" height="1920"/>
  </clipPath>
</defs>
<path clip-path="url(#clip820)" d="M0 1920 L2688 1920 L2688 -4.26326e-14 L0 -4.26326e-14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip821">
    <rect x="537" y="0" width="1883" height="1883"/>
  </clipPath>
</defs>
<path clip-path="url(#clip820)" d="M226.465 1734.12 L2640.76 1734.12 L2640.76 47.2441 L226.465 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip822">
    <rect x="226" y="47" width="2415" height="1688"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip822)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="294.794,1734.12 294.794,47.2441 "/>
<polyline clip-path="url(#clip822)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="750.32,1734.12 750.32,47.2441 "/>
<polyline clip-path="url(#clip822)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1205.85,1734.12 1205.85,47.2441 "/>
<polyline clip-path="url(#clip822)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1661.37,1734.12 1661.37,47.2441 "/>
<polyline clip-path="url(#clip822)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2116.9,1734.12 2116.9,47.2441 "/>
<polyline clip-path="url(#clip822)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2572.43,1734.12 2572.43,47.2441 "/>
<polyline clip-path="url(#clip820)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.465,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip820)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="294.794,1734.12 294.794,1715.22 "/>
<polyline clip-path="url(#clip820)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="750.32,1734.12 750.32,1715.22 "/>
<polyline clip-path="url(#clip820)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1205.85,1734.12 1205.85,1715.22 "/>
<polyline clip-path="url(#clip820)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1661.37,1734.12 1661.37,1715.22 "/>
<polyline clip-path="url(#clip820)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2116.9,1734.12 2116.9,1715.22 "/>
<polyline clip-path="url(#clip820)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2572.43,1734.12 2572.43,1715.22 "/>
<path clip-path="url(#clip820)" d="M294.794 1767.34 Q291.183 1767.34 289.354 1770.9 Q287.548 1774.45 287.548 1781.58 Q287.548 1788.68 289.354 1792.25 Q291.183 1795.79 294.794 1795.79 Q298.428 1795.79 300.233 1792.25 Q302.062 1788.68 302.062 1781.58 Q302.062 1774.45 300.233 1770.9 Q298.428 1767.34 294.794 1767.34 M294.794 1763.64 Q300.604 1763.64 303.659 1768.24 Q306.738 1772.83 306.738 1781.58 Q306.738 1790.3 303.659 1794.91 Q300.604 1799.49 294.794 1799.49 Q288.983 1799.49 285.905 1794.91 Q282.849 1790.3 282.849 1781.58 Q282.849 1772.83 285.905 1768.24 Q288.983 1763.64 294.794 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M725.008 1794.89 L732.647 1794.89 L732.647 1768.52 L724.337 1770.19 L724.337 1765.93 L732.6 1764.26 L737.276 1764.26 L737.276 1794.89 L744.915 1794.89 L744.915 1798.82 L725.008 1798.82 L725.008 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M764.36 1767.34 Q760.748 1767.34 758.92 1770.9 Q757.114 1774.45 757.114 1781.58 Q757.114 1788.68 758.92 1792.25 Q760.748 1795.79 764.36 1795.79 Q767.994 1795.79 769.799 1792.25 Q771.628 1788.68 771.628 1781.58 Q771.628 1774.45 769.799 1770.9 Q767.994 1767.34 764.36 1767.34 M764.36 1763.64 Q770.17 1763.64 773.225 1768.24 Q776.304 1772.83 776.304 1781.58 Q776.304 1790.3 773.225 1794.91 Q770.17 1799.49 764.36 1799.49 Q758.549 1799.49 755.471 1794.91 Q752.415 1790.3 752.415 1781.58 Q752.415 1772.83 755.471 1768.24 Q758.549 1763.64 764.36 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1184.62 1794.89 L1200.94 1794.89 L1200.94 1798.82 L1179 1798.82 L1179 1794.89 Q1181.66 1792.13 1186.24 1787.5 Q1190.85 1782.85 1192.03 1781.51 Q1194.27 1778.98 1195.15 1777.25 Q1196.06 1775.49 1196.06 1773.8 Q1196.06 1771.04 1194.11 1769.31 Q1192.19 1767.57 1189.09 1767.57 Q1186.89 1767.57 1184.44 1768.34 Q1182 1769.1 1179.23 1770.65 L1179.23 1765.93 Q1182.05 1764.79 1184.5 1764.21 Q1186.96 1763.64 1189 1763.64 Q1194.37 1763.64 1197.56 1766.32 Q1200.75 1769.01 1200.75 1773.5 Q1200.75 1775.63 1199.94 1777.55 Q1199.16 1779.45 1197.05 1782.04 Q1196.47 1782.71 1193.37 1785.93 Q1190.27 1789.12 1184.62 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1220.75 1767.34 Q1217.14 1767.34 1215.31 1770.9 Q1213.51 1774.45 1213.51 1781.58 Q1213.51 1788.68 1215.31 1792.25 Q1217.14 1795.79 1220.75 1795.79 Q1224.39 1795.79 1226.19 1792.25 Q1228.02 1788.68 1228.02 1781.58 Q1228.02 1774.45 1226.19 1770.9 Q1224.39 1767.34 1220.75 1767.34 M1220.75 1763.64 Q1226.56 1763.64 1229.62 1768.24 Q1232.7 1772.83 1232.7 1781.58 Q1232.7 1790.3 1229.62 1794.91 Q1226.56 1799.49 1220.75 1799.49 Q1214.94 1799.49 1211.87 1794.91 Q1208.81 1790.3 1208.81 1781.58 Q1208.81 1772.83 1211.87 1768.24 Q1214.94 1763.64 1220.75 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1650.22 1780.19 Q1653.57 1780.9 1655.45 1783.17 Q1657.35 1785.44 1657.35 1788.77 Q1657.35 1793.89 1653.83 1796.69 Q1650.31 1799.49 1643.83 1799.49 Q1641.65 1799.49 1639.34 1799.05 Q1637.05 1798.64 1634.59 1797.78 L1634.59 1793.27 Q1636.54 1794.4 1638.85 1794.98 Q1641.17 1795.56 1643.69 1795.56 Q1648.09 1795.56 1650.38 1793.82 Q1652.69 1792.08 1652.69 1788.77 Q1652.69 1785.72 1650.54 1784.01 Q1648.41 1782.27 1644.59 1782.27 L1640.56 1782.27 L1640.56 1778.43 L1644.78 1778.43 Q1648.23 1778.43 1650.05 1777.06 Q1651.88 1775.67 1651.88 1773.08 Q1651.88 1770.42 1649.98 1769.01 Q1648.11 1767.57 1644.59 1767.57 Q1642.67 1767.57 1640.47 1767.99 Q1638.27 1768.4 1635.63 1769.28 L1635.63 1765.12 Q1638.3 1764.38 1640.61 1764.01 Q1642.95 1763.64 1645.01 1763.64 Q1650.33 1763.64 1653.43 1766.07 Q1656.54 1768.47 1656.54 1772.59 Q1656.54 1775.46 1654.89 1777.46 Q1653.25 1779.42 1650.22 1780.19 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1676.21 1767.34 Q1672.6 1767.34 1670.77 1770.9 Q1668.97 1774.45 1668.97 1781.58 Q1668.97 1788.68 1670.77 1792.25 Q1672.6 1795.79 1676.21 1795.79 Q1679.85 1795.79 1681.65 1792.25 Q1683.48 1788.68 1683.48 1781.58 Q1683.48 1774.45 1681.65 1770.9 Q1679.85 1767.34 1676.21 1767.34 M1676.21 1763.64 Q1682.02 1763.64 1685.08 1768.24 Q1688.16 1772.83 1688.16 1781.58 Q1688.16 1790.3 1685.08 1794.91 Q1682.02 1799.49 1676.21 1799.49 Q1670.4 1799.49 1667.32 1794.91 Q1664.27 1790.3 1664.27 1781.58 Q1664.27 1772.83 1667.32 1768.24 Q1670.4 1763.64 1676.21 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M2105.07 1768.34 L2093.27 1786.78 L2105.07 1786.78 L2105.07 1768.34 M2103.84 1764.26 L2109.72 1764.26 L2109.72 1786.78 L2114.65 1786.78 L2114.65 1790.67 L2109.72 1790.67 L2109.72 1798.82 L2105.07 1798.82 L2105.07 1790.67 L2089.47 1790.67 L2089.47 1786.16 L2103.84 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M2132.39 1767.34 Q2128.78 1767.34 2126.95 1770.9 Q2125.14 1774.45 2125.14 1781.58 Q2125.14 1788.68 2126.95 1792.25 Q2128.78 1795.79 2132.39 1795.79 Q2136.02 1795.79 2137.83 1792.25 Q2139.65 1788.68 2139.65 1781.58 Q2139.65 1774.45 2137.83 1770.9 Q2136.02 1767.34 2132.39 1767.34 M2132.39 1763.64 Q2138.2 1763.64 2141.25 1768.24 Q2144.33 1772.83 2144.33 1781.58 Q2144.33 1790.3 2141.25 1794.91 Q2138.2 1799.49 2132.39 1799.49 Q2126.58 1799.49 2123.5 1794.91 Q2120.44 1790.3 2120.44 1781.58 Q2120.44 1772.83 2123.5 1768.24 Q2126.58 1763.64 2132.39 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M2547.13 1764.26 L2565.48 1764.26 L2565.48 1768.2 L2551.41 1768.2 L2551.41 1776.67 Q2552.43 1776.32 2553.45 1776.16 Q2554.46 1775.97 2555.48 1775.97 Q2561.27 1775.97 2564.65 1779.15 Q2568.03 1782.32 2568.03 1787.73 Q2568.03 1793.31 2564.56 1796.41 Q2561.08 1799.49 2554.76 1799.49 Q2552.59 1799.49 2550.32 1799.12 Q2548.08 1798.75 2545.67 1798.01 L2545.67 1793.31 Q2547.75 1794.45 2549.97 1795 Q2552.2 1795.56 2554.67 1795.56 Q2558.68 1795.56 2561.01 1793.45 Q2563.35 1791.34 2563.35 1787.73 Q2563.35 1784.12 2561.01 1782.02 Q2558.68 1779.91 2554.67 1779.91 Q2552.8 1779.91 2550.92 1780.33 Q2549.07 1780.74 2547.13 1781.62 L2547.13 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M2587.24 1767.34 Q2583.63 1767.34 2581.8 1770.9 Q2580 1774.45 2580 1781.58 Q2580 1788.68 2581.8 1792.25 Q2583.63 1795.79 2587.24 1795.79 Q2590.88 1795.79 2592.68 1792.25 Q2594.51 1788.68 2594.51 1781.58 Q2594.51 1774.45 2592.68 1770.9 Q2590.88 1767.34 2587.24 1767.34 M2587.24 1763.64 Q2593.05 1763.64 2596.11 1768.24 Q2599.19 1772.83 2599.19 1781.58 Q2599.19 1790.3 2596.11 1794.91 Q2593.05 1799.49 2587.24 1799.49 Q2581.43 1799.49 2578.35 1794.91 Q2575.3 1790.3 2575.3 1781.58 Q2575.3 1772.83 2578.35 1768.24 Q2581.43 1763.64 2587.24 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1039.21 1841.06 L1039.21 1878.02 L1046.97 1878.02 Q1056.81 1878.02 1061.36 1873.56 Q1065.94 1869.11 1065.94 1859.49 Q1065.94 1849.94 1061.36 1845.52 Q1056.81 1841.06 1046.97 1841.06 L1039.21 1841.06 M1032.78 1835.78 L1045.99 1835.78 Q1059.8 1835.78 1066.26 1841.54 Q1072.72 1847.27 1072.72 1859.49 Q1072.72 1871.78 1066.23 1877.54 Q1059.74 1883.3 1045.99 1883.3 L1032.78 1883.3 L1032.78 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1082.72 1847.65 L1088.57 1847.65 L1088.57 1883.3 L1082.72 1883.3 L1082.72 1847.65 M1082.72 1833.78 L1088.57 1833.78 L1088.57 1841.19 L1082.72 1841.19 L1082.72 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1123.55 1848.7 L1123.55 1854.24 Q1121.07 1852.97 1118.4 1852.33 Q1115.72 1851.7 1112.86 1851.7 Q1108.5 1851.7 1106.3 1853.03 Q1104.14 1854.37 1104.14 1857.04 Q1104.14 1859.08 1105.7 1860.26 Q1107.26 1861.4 1111.97 1862.45 L1113.97 1862.9 Q1120.21 1864.24 1122.82 1866.69 Q1125.46 1869.11 1125.46 1873.47 Q1125.46 1878.43 1121.52 1881.33 Q1117.6 1884.22 1110.73 1884.22 Q1107.86 1884.22 1104.74 1883.65 Q1101.65 1883.11 1098.22 1882 L1098.22 1875.95 Q1101.46 1877.64 1104.61 1878.49 Q1107.77 1879.32 1110.85 1879.32 Q1114.99 1879.32 1117.22 1877.92 Q1119.45 1876.49 1119.45 1873.91 Q1119.45 1871.52 1117.82 1870.25 Q1116.23 1868.98 1110.79 1867.8 L1108.75 1867.32 Q1103.31 1866.18 1100.89 1863.82 Q1098.47 1861.43 1098.47 1857.3 Q1098.47 1852.27 1102.04 1849.53 Q1105.6 1846.79 1112.16 1846.79 Q1115.4 1846.79 1118.27 1847.27 Q1121.13 1847.75 1123.55 1848.7 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1140.58 1837.53 L1140.58 1847.65 L1152.64 1847.65 L1152.64 1852.2 L1140.58 1852.2 L1140.58 1871.56 Q1140.58 1875.92 1141.76 1877.16 Q1142.97 1878.4 1146.63 1878.4 L1152.64 1878.4 L1152.64 1883.3 L1146.63 1883.3 Q1139.85 1883.3 1137.27 1880.79 Q1134.69 1878.24 1134.69 1871.56 L1134.69 1852.2 L1130.4 1852.2 L1130.4 1847.65 L1134.69 1847.65 L1134.69 1837.53 L1140.58 1837.53 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1176.55 1865.38 Q1169.45 1865.38 1166.71 1867 Q1163.97 1868.63 1163.97 1872.54 Q1163.97 1875.66 1166.01 1877.51 Q1168.08 1879.32 1171.61 1879.32 Q1176.48 1879.32 1179.41 1875.88 Q1182.37 1872.42 1182.37 1866.69 L1182.37 1865.38 L1176.55 1865.38 M1188.23 1862.96 L1188.23 1883.3 L1182.37 1883.3 L1182.37 1877.89 Q1180.37 1881.14 1177.37 1882.7 Q1174.38 1884.22 1170.05 1884.22 Q1164.58 1884.22 1161.33 1881.17 Q1158.12 1878.08 1158.12 1872.92 Q1158.12 1866.91 1162.13 1863.85 Q1166.17 1860.8 1174.16 1860.8 L1182.37 1860.8 L1182.37 1860.23 Q1182.37 1856.18 1179.7 1853.99 Q1177.06 1851.76 1172.25 1851.76 Q1169.19 1851.76 1166.3 1852.49 Q1163.4 1853.22 1160.73 1854.69 L1160.73 1849.28 Q1163.94 1848.03 1166.97 1847.43 Q1169.99 1846.79 1172.86 1846.79 Q1180.59 1846.79 1184.41 1850.8 Q1188.23 1854.81 1188.23 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1229.92 1861.78 L1229.92 1883.3 L1224.07 1883.3 L1224.07 1861.98 Q1224.07 1856.92 1222.09 1854.4 Q1220.12 1851.89 1216.17 1851.89 Q1211.43 1851.89 1208.69 1854.91 Q1205.96 1857.93 1205.96 1863.15 L1205.96 1883.3 L1200.07 1883.3 L1200.07 1847.65 L1205.96 1847.65 L1205.96 1853.19 Q1208.06 1849.98 1210.89 1848.39 Q1213.75 1846.79 1217.48 1846.79 Q1223.62 1846.79 1226.77 1850.61 Q1229.92 1854.4 1229.92 1861.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1267.26 1849.02 L1267.26 1854.5 Q1264.78 1853.13 1262.26 1852.46 Q1259.78 1851.76 1257.23 1851.76 Q1251.54 1851.76 1248.38 1855.39 Q1245.23 1858.98 1245.23 1865.51 Q1245.23 1872.03 1248.38 1875.66 Q1251.54 1879.26 1257.23 1879.26 Q1259.78 1879.26 1262.26 1878.59 Q1264.78 1877.89 1267.26 1876.52 L1267.26 1881.93 Q1264.81 1883.08 1262.17 1883.65 Q1259.56 1884.22 1256.6 1884.22 Q1248.54 1884.22 1243.8 1879.16 Q1239.06 1874.1 1239.06 1865.51 Q1239.06 1856.79 1243.83 1851.79 Q1248.64 1846.79 1256.98 1846.79 Q1259.68 1846.79 1262.26 1847.37 Q1264.84 1847.91 1267.26 1849.02 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1307.94 1864.01 L1307.94 1866.88 L1281.01 1866.88 Q1281.39 1872.92 1284.64 1876.11 Q1287.92 1879.26 1293.74 1879.26 Q1297.11 1879.26 1300.26 1878.43 Q1303.45 1877.6 1306.57 1875.95 L1306.57 1881.49 Q1303.42 1882.82 1300.11 1883.52 Q1296.8 1884.22 1293.39 1884.22 Q1284.86 1884.22 1279.86 1879.26 Q1274.9 1874.29 1274.9 1865.83 Q1274.9 1857.07 1279.61 1851.95 Q1284.35 1846.79 1292.37 1846.79 Q1299.56 1846.79 1303.73 1851.44 Q1307.94 1856.06 1307.94 1864.01 M1302.08 1862.29 Q1302.02 1857.49 1299.37 1854.62 Q1296.76 1851.76 1292.43 1851.76 Q1287.53 1851.76 1284.57 1854.53 Q1281.64 1857.3 1281.2 1862.33 L1302.08 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1344.95 1841.06 L1344.95 1878.02 L1352.72 1878.02 Q1362.55 1878.02 1367.1 1873.56 Q1371.69 1869.11 1371.69 1859.49 Q1371.69 1849.94 1367.1 1845.52 Q1362.55 1841.06 1352.72 1841.06 L1344.95 1841.06 M1338.52 1835.78 L1351.73 1835.78 Q1365.54 1835.78 1372.01 1841.54 Q1378.47 1847.27 1378.47 1859.49 Q1378.47 1871.78 1371.97 1877.54 Q1365.48 1883.3 1351.73 1883.3 L1338.52 1883.3 L1338.52 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1402.28 1851.76 Q1397.56 1851.76 1394.83 1855.45 Q1392.09 1859.11 1392.09 1865.51 Q1392.09 1871.91 1394.8 1875.6 Q1397.53 1879.26 1402.28 1879.26 Q1406.95 1879.26 1409.69 1875.57 Q1412.43 1871.87 1412.43 1865.51 Q1412.43 1859.17 1409.69 1855.48 Q1406.95 1851.76 1402.28 1851.76 M1402.28 1846.79 Q1409.91 1846.79 1414.27 1851.76 Q1418.63 1856.72 1418.63 1865.51 Q1418.63 1874.26 1414.27 1879.26 Q1409.91 1884.22 1402.28 1884.22 Q1394.6 1884.22 1390.24 1879.26 Q1385.92 1874.26 1385.92 1865.51 Q1385.92 1856.72 1390.24 1851.76 Q1394.6 1846.79 1402.28 1846.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1424.94 1847.65 L1430.79 1847.65 L1438.11 1875.47 L1445.4 1847.65 L1452.31 1847.65 L1459.63 1875.47 L1466.92 1847.65 L1472.78 1847.65 L1463.45 1883.3 L1456.54 1883.3 L1448.87 1854.08 L1441.17 1883.3 L1434.26 1883.3 L1424.94 1847.65 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1511.29 1861.78 L1511.29 1883.3 L1505.43 1883.3 L1505.43 1861.98 Q1505.43 1856.92 1503.46 1854.4 Q1501.48 1851.89 1497.54 1851.89 Q1492.8 1851.89 1490.06 1854.91 Q1487.32 1857.93 1487.32 1863.15 L1487.32 1883.3 L1481.43 1883.3 L1481.43 1847.65 L1487.32 1847.65 L1487.32 1853.19 Q1489.42 1849.98 1492.25 1848.39 Q1495.12 1846.79 1498.84 1846.79 Q1504.99 1846.79 1508.14 1850.61 Q1511.29 1854.4 1511.29 1861.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1543.63 1853.13 Q1542.64 1852.55 1541.46 1852.3 Q1540.32 1852.01 1538.91 1852.01 Q1533.95 1852.01 1531.28 1855.26 Q1528.63 1858.47 1528.63 1864.52 L1528.63 1883.3 L1522.75 1883.3 L1522.75 1847.65 L1528.63 1847.65 L1528.63 1853.19 Q1530.48 1849.94 1533.44 1848.39 Q1536.4 1846.79 1540.63 1846.79 Q1541.24 1846.79 1541.97 1846.89 Q1542.7 1846.95 1543.59 1847.11 L1543.63 1853.13 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1549.77 1847.65 L1555.62 1847.65 L1555.62 1883.3 L1549.77 1883.3 L1549.77 1847.65 M1549.77 1833.78 L1555.62 1833.78 L1555.62 1841.19 L1549.77 1841.19 L1549.77 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1563.68 1847.65 L1569.88 1847.65 L1581.02 1877.57 L1592.16 1847.65 L1598.37 1847.65 L1585 1883.3 L1577.05 1883.3 L1563.68 1847.65 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1636.95 1864.01 L1636.95 1866.88 L1610.02 1866.88 Q1610.4 1872.92 1613.65 1876.11 Q1616.93 1879.26 1622.75 1879.26 Q1626.13 1879.26 1629.28 1878.43 Q1632.46 1877.6 1635.58 1875.95 L1635.58 1881.49 Q1632.43 1882.82 1629.12 1883.52 Q1625.81 1884.22 1622.4 1884.22 Q1613.87 1884.22 1608.87 1879.26 Q1603.91 1874.29 1603.91 1865.83 Q1603.91 1857.07 1608.62 1851.95 Q1613.36 1846.79 1621.38 1846.79 Q1628.58 1846.79 1632.75 1851.44 Q1636.95 1856.06 1636.95 1864.01 M1631.09 1862.29 Q1631.03 1857.49 1628.38 1854.62 Q1625.78 1851.76 1621.45 1851.76 Q1616.54 1851.76 1613.58 1854.53 Q1610.66 1857.3 1610.21 1862.33 L1631.09 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1667.22 1853.13 Q1666.23 1852.55 1665.05 1852.3 Q1663.91 1852.01 1662.51 1852.01 Q1657.54 1852.01 1654.87 1855.26 Q1652.22 1858.47 1652.22 1864.52 L1652.22 1883.3 L1646.34 1883.3 L1646.34 1847.65 L1652.22 1847.65 L1652.22 1853.19 Q1654.07 1849.94 1657.03 1848.39 Q1659.99 1846.79 1664.22 1846.79 Q1664.83 1846.79 1665.56 1846.89 Q1666.29 1846.95 1667.18 1847.11 L1667.22 1853.13 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1708.15 1833.84 Q1703.88 1841.16 1701.81 1848.32 Q1699.74 1855.48 1699.74 1862.84 Q1699.74 1870.19 1701.81 1877.41 Q1703.91 1884.61 1708.15 1891.89 L1703.05 1891.89 Q1698.28 1884.41 1695.89 1877.19 Q1693.54 1869.96 1693.54 1862.84 Q1693.54 1855.74 1695.89 1848.54 Q1698.25 1841.35 1703.05 1833.84 L1708.15 1833.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1719.29 1833.78 L1725.18 1833.78 L1725.18 1863.03 L1742.65 1847.65 L1750.13 1847.65 L1731.22 1864.33 L1750.92 1883.3 L1743.29 1883.3 L1725.18 1865.89 L1725.18 1883.3 L1719.29 1883.3 L1719.29 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1785.01 1854.5 Q1787.21 1850.55 1790.26 1848.67 Q1793.32 1846.79 1797.46 1846.79 Q1803.03 1846.79 1806.05 1850.71 Q1809.08 1854.59 1809.08 1861.78 L1809.08 1883.3 L1803.19 1883.3 L1803.19 1861.98 Q1803.19 1856.85 1801.37 1854.37 Q1799.56 1851.89 1795.83 1851.89 Q1791.28 1851.89 1788.64 1854.91 Q1786 1857.93 1786 1863.15 L1786 1883.3 L1780.11 1883.3 L1780.11 1861.98 Q1780.11 1856.82 1778.3 1854.37 Q1776.48 1851.89 1772.7 1851.89 Q1768.21 1851.89 1765.57 1854.94 Q1762.92 1857.97 1762.92 1863.15 L1762.92 1883.3 L1757.04 1883.3 L1757.04 1847.65 L1762.92 1847.65 L1762.92 1853.19 Q1764.93 1849.91 1767.73 1848.35 Q1770.53 1846.79 1774.38 1846.79 Q1778.27 1846.79 1780.97 1848.77 Q1783.71 1850.74 1785.01 1854.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1819.83 1833.84 L1824.93 1833.84 Q1829.7 1841.35 1832.06 1848.54 Q1834.44 1855.74 1834.44 1862.84 Q1834.44 1869.96 1832.06 1877.19 Q1829.7 1884.41 1824.93 1891.89 L1819.83 1891.89 Q1824.07 1884.61 1826.14 1877.41 Q1828.24 1870.19 1828.24 1862.84 Q1828.24 1855.48 1826.14 1848.32 Q1824.07 1841.16 1819.83 1833.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip822)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.465,1550.43 2640.76,1550.43 "/>
<polyline clip-path="url(#clip822)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.465,1328.07 2640.76,1328.07 "/>
<polyline clip-path="url(#clip822)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.465,1105.71 2640.76,1105.71 "/>
<polyline clip-path="url(#clip822)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.465,883.351 2640.76,883.351 "/>
<polyline clip-path="url(#clip822)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.465,660.992 2640.76,660.992 "/>
<polyline clip-path="url(#clip822)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.465,438.632 2640.76,438.632 "/>
<polyline clip-path="url(#clip822)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.465,216.273 2640.76,216.273 "/>
<polyline clip-path="url(#clip820)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.465,1734.12 226.465,47.2441 "/>
<polyline clip-path="url(#clip820)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.465,1550.43 245.362,1550.43 "/>
<polyline clip-path="url(#clip820)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.465,1328.07 245.362,1328.07 "/>
<polyline clip-path="url(#clip820)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.465,1105.71 245.362,1105.71 "/>
<polyline clip-path="url(#clip820)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.465,883.351 245.362,883.351 "/>
<polyline clip-path="url(#clip820)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.465,660.992 245.362,660.992 "/>
<polyline clip-path="url(#clip820)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.465,438.632 245.362,438.632 "/>
<polyline clip-path="url(#clip820)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.465,216.273 245.362,216.273 "/>
<path clip-path="url(#clip820)" d="M131.816 1537.22 L120.011 1555.67 L131.816 1555.67 L131.816 1537.22 M130.589 1533.15 L136.469 1533.15 L136.469 1555.67 L141.399 1555.67 L141.399 1559.56 L136.469 1559.56 L136.469 1567.71 L131.816 1567.71 L131.816 1559.56 L116.214 1559.56 L116.214 1555.05 L130.589 1533.15 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M149.131 1561.83 L154.015 1561.83 L154.015 1567.71 L149.131 1567.71 L149.131 1561.83 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M174.2 1536.23 Q170.589 1536.23 168.76 1539.79 Q166.955 1543.34 166.955 1550.46 Q166.955 1557.57 168.76 1561.14 Q170.589 1564.68 174.2 1564.68 Q177.834 1564.68 179.64 1561.14 Q181.469 1557.57 181.469 1550.46 Q181.469 1543.34 179.64 1539.79 Q177.834 1536.23 174.2 1536.23 M174.2 1532.53 Q180.01 1532.53 183.066 1537.13 Q186.145 1541.71 186.145 1550.46 Q186.145 1559.19 183.066 1563.8 Q180.01 1568.38 174.2 1568.38 Q168.39 1568.38 165.311 1563.8 Q162.256 1559.19 162.256 1550.46 Q162.256 1541.71 165.311 1537.13 Q168.39 1532.53 174.2 1532.53 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M132.812 1314.86 L121.006 1333.31 L132.812 1333.31 L132.812 1314.86 M131.585 1310.79 L137.464 1310.79 L137.464 1333.31 L142.395 1333.31 L142.395 1337.2 L137.464 1337.2 L137.464 1345.35 L132.812 1345.35 L132.812 1337.2 L117.21 1337.2 L117.21 1332.69 L131.585 1310.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M150.126 1339.47 L155.011 1339.47 L155.011 1345.35 L150.126 1345.35 L150.126 1339.47 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M165.242 1310.79 L183.598 1310.79 L183.598 1314.73 L169.524 1314.73 L169.524 1323.2 Q170.543 1322.85 171.561 1322.69 Q172.58 1322.5 173.598 1322.5 Q179.385 1322.5 182.765 1325.67 Q186.145 1328.85 186.145 1334.26 Q186.145 1339.84 182.672 1342.94 Q179.2 1346.02 172.881 1346.02 Q170.705 1346.02 168.436 1345.65 Q166.191 1345.28 163.784 1344.54 L163.784 1339.84 Q165.867 1340.98 168.089 1341.53 Q170.311 1342.09 172.788 1342.09 Q176.793 1342.09 179.131 1339.98 Q181.469 1337.87 181.469 1334.26 Q181.469 1330.65 179.131 1328.55 Q176.793 1326.44 172.788 1326.44 Q170.913 1326.44 169.038 1326.86 Q167.186 1327.27 165.242 1328.15 L165.242 1310.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M119.015 1088.43 L137.372 1088.43 L137.372 1092.37 L123.298 1092.37 L123.298 1100.84 Q124.316 1100.49 125.335 1100.33 Q126.353 1100.14 127.372 1100.14 Q133.159 1100.14 136.538 1103.32 Q139.918 1106.49 139.918 1111.9 Q139.918 1117.48 136.446 1120.58 Q132.974 1123.66 126.654 1123.66 Q124.478 1123.66 122.21 1123.29 Q119.964 1122.92 117.557 1122.18 L117.557 1117.48 Q119.64 1118.62 121.863 1119.17 Q124.085 1119.73 126.562 1119.73 Q130.566 1119.73 132.904 1117.62 Q135.242 1115.51 135.242 1111.9 Q135.242 1108.29 132.904 1106.19 Q130.566 1104.08 126.562 1104.08 Q124.687 1104.08 122.812 1104.5 Q120.96 1104.91 119.015 1105.79 L119.015 1088.43 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M149.131 1117.11 L154.015 1117.11 L154.015 1122.99 L149.131 1122.99 L149.131 1117.11 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M174.2 1091.51 Q170.589 1091.51 168.76 1095.07 Q166.955 1098.62 166.955 1105.75 Q166.955 1112.85 168.76 1116.42 Q170.589 1119.96 174.2 1119.96 Q177.834 1119.96 179.64 1116.42 Q181.469 1112.85 181.469 1105.75 Q181.469 1098.62 179.64 1095.07 Q177.834 1091.51 174.2 1091.51 M174.2 1087.81 Q180.01 1087.81 183.066 1092.41 Q186.145 1097 186.145 1105.75 Q186.145 1114.47 183.066 1119.08 Q180.01 1123.66 174.2 1123.66 Q168.39 1123.66 165.311 1119.08 Q162.256 1114.47 162.256 1105.75 Q162.256 1097 165.311 1092.41 Q168.39 1087.81 174.2 1087.81 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M120.011 866.071 L138.367 866.071 L138.367 870.007 L124.293 870.007 L124.293 878.479 Q125.312 878.132 126.33 877.97 Q127.349 877.784 128.367 877.784 Q134.154 877.784 137.534 880.956 Q140.913 884.127 140.913 889.544 Q140.913 895.122 137.441 898.224 Q133.969 901.303 127.65 901.303 Q125.474 901.303 123.205 900.932 Q120.96 900.562 118.552 899.821 L118.552 895.122 Q120.636 896.256 122.858 896.812 Q125.08 897.368 127.557 897.368 Q131.562 897.368 133.9 895.261 Q136.237 893.155 136.237 889.544 Q136.237 885.932 133.9 883.826 Q131.562 881.719 127.557 881.719 Q125.682 881.719 123.807 882.136 Q121.955 882.553 120.011 883.432 L120.011 866.071 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M150.126 894.752 L155.011 894.752 L155.011 900.631 L150.126 900.631 L150.126 894.752 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M165.242 866.071 L183.598 866.071 L183.598 870.007 L169.524 870.007 L169.524 878.479 Q170.543 878.132 171.561 877.97 Q172.58 877.784 173.598 877.784 Q179.385 877.784 182.765 880.956 Q186.145 884.127 186.145 889.544 Q186.145 895.122 182.672 898.224 Q179.2 901.303 172.881 901.303 Q170.705 901.303 168.436 900.932 Q166.191 900.562 163.784 899.821 L163.784 895.122 Q165.867 896.256 168.089 896.812 Q170.311 897.368 172.788 897.368 Q176.793 897.368 179.131 895.261 Q181.469 893.155 181.469 889.544 Q181.469 885.932 179.131 883.826 Q176.793 881.719 172.788 881.719 Q170.913 881.719 169.038 882.136 Q167.186 882.553 165.242 883.432 L165.242 866.071 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M129.548 659.128 Q126.4 659.128 124.548 661.281 Q122.719 663.434 122.719 667.184 Q122.719 670.911 124.548 673.087 Q126.4 675.239 129.548 675.239 Q132.696 675.239 134.525 673.087 Q136.376 670.911 136.376 667.184 Q136.376 663.434 134.525 661.281 Q132.696 659.128 129.548 659.128 M138.83 644.476 L138.83 648.735 Q137.071 647.902 135.265 647.462 Q133.483 647.022 131.724 647.022 Q127.094 647.022 124.64 650.147 Q122.21 653.272 121.863 659.591 Q123.228 657.578 125.288 656.513 Q127.349 655.425 129.825 655.425 Q135.034 655.425 138.043 658.596 Q141.075 661.744 141.075 667.184 Q141.075 672.508 137.927 675.726 Q134.779 678.943 129.548 678.943 Q123.552 678.943 120.381 674.36 Q117.21 669.753 117.21 661.027 Q117.21 652.832 121.099 647.971 Q124.988 643.087 131.538 643.087 Q133.298 643.087 135.08 643.434 Q136.886 643.781 138.83 644.476 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M149.131 672.392 L154.015 672.392 L154.015 678.272 L149.131 678.272 L149.131 672.392 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M174.2 646.791 Q170.589 646.791 168.76 650.355 Q166.955 653.897 166.955 661.027 Q166.955 668.133 168.76 671.698 Q170.589 675.239 174.2 675.239 Q177.834 675.239 179.64 671.698 Q181.469 668.133 181.469 661.027 Q181.469 653.897 179.64 650.355 Q177.834 646.791 174.2 646.791 M174.2 643.087 Q180.01 643.087 183.066 647.693 Q186.145 652.277 186.145 661.027 Q186.145 669.753 183.066 674.36 Q180.01 678.943 174.2 678.943 Q168.39 678.943 165.311 674.36 Q162.256 669.753 162.256 661.027 Q162.256 652.277 165.311 647.693 Q168.39 643.087 174.2 643.087 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M130.543 436.769 Q127.395 436.769 125.543 438.922 Q123.714 441.074 123.714 444.824 Q123.714 448.551 125.543 450.727 Q127.395 452.88 130.543 452.88 Q133.691 452.88 135.52 450.727 Q137.372 448.551 137.372 444.824 Q137.372 441.074 135.52 438.922 Q133.691 436.769 130.543 436.769 M139.825 422.116 L139.825 426.375 Q138.066 425.542 136.261 425.102 Q134.478 424.663 132.719 424.663 Q128.089 424.663 125.636 427.787 Q123.205 430.912 122.858 437.232 Q124.224 435.218 126.284 434.153 Q128.344 433.065 130.821 433.065 Q136.029 433.065 139.038 436.237 Q142.071 439.385 142.071 444.824 Q142.071 450.148 138.923 453.366 Q135.775 456.584 130.543 456.584 Q124.548 456.584 121.376 452 Q118.205 447.394 118.205 438.667 Q118.205 430.473 122.094 425.612 Q125.983 420.727 132.534 420.727 Q134.293 420.727 136.075 421.075 Q137.881 421.422 139.825 422.116 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M150.126 450.033 L155.011 450.033 L155.011 455.912 L150.126 455.912 L150.126 450.033 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M165.242 421.352 L183.598 421.352 L183.598 425.288 L169.524 425.288 L169.524 433.76 Q170.543 433.412 171.561 433.25 Q172.58 433.065 173.598 433.065 Q179.385 433.065 182.765 436.237 Q186.145 439.408 186.145 444.824 Q186.145 450.403 182.672 453.505 Q179.2 456.584 172.881 456.584 Q170.705 456.584 168.436 456.213 Q166.191 455.843 163.784 455.102 L163.784 450.403 Q165.867 451.537 168.089 452.093 Q170.311 452.648 172.788 452.648 Q176.793 452.648 179.131 450.542 Q181.469 448.436 181.469 444.824 Q181.469 441.213 179.131 439.107 Q176.793 437 172.788 437 Q170.913 437 169.038 437.417 Q167.186 437.834 165.242 438.713 L165.242 421.352 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M117.789 198.993 L140.011 198.993 L140.011 200.984 L127.464 233.553 L122.58 233.553 L134.386 202.928 L117.789 202.928 L117.789 198.993 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M149.131 227.673 L154.015 227.673 L154.015 233.553 L149.131 233.553 L149.131 227.673 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M174.2 202.071 Q170.589 202.071 168.76 205.636 Q166.955 209.178 166.955 216.308 Q166.955 223.414 168.76 226.979 Q170.589 230.52 174.2 230.52 Q177.834 230.52 179.64 226.979 Q181.469 223.414 181.469 216.308 Q181.469 209.178 179.64 205.636 Q177.834 202.071 174.2 202.071 M174.2 198.368 Q180.01 198.368 183.066 202.974 Q186.145 207.558 186.145 216.308 Q186.145 225.034 183.066 229.641 Q180.01 234.224 174.2 234.224 Q168.39 234.224 165.311 229.641 Q162.256 225.034 162.256 216.308 Q162.256 207.558 165.311 202.974 Q168.39 198.368 174.2 198.368 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M19.402 1529.67 L56.3549 1529.67 L56.3549 1521.9 Q56.3549 1512.07 51.8989 1507.52 Q47.4429 1502.93 37.8307 1502.93 Q28.2821 1502.93 23.858 1507.52 Q19.402 1512.07 19.402 1521.9 L19.402 1529.67 M14.1184 1536.1 L14.1184 1522.89 Q14.1184 1509.08 19.8794 1502.62 Q25.6085 1496.15 37.8307 1496.15 Q50.1165 1496.15 55.8775 1502.65 Q61.6384 1509.14 61.6384 1522.89 L61.6384 1536.1 L14.1184 1536.1 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M25.9905 1486.16 L25.9905 1480.3 L61.6384 1480.3 L61.6384 1486.16 L25.9905 1486.16 M12.1132 1486.16 L12.1132 1480.3 L19.5293 1480.3 L19.5293 1486.16 L12.1132 1486.16 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M27.0408 1445.32 L32.579 1445.32 Q31.3059 1447.81 30.6693 1450.48 Q30.0327 1453.15 30.0327 1456.02 Q30.0327 1460.38 31.3695 1462.58 Q32.7063 1464.74 35.3799 1464.74 Q37.4169 1464.74 38.5946 1463.18 Q39.7404 1461.62 40.7908 1456.91 L41.2364 1454.9 Q42.5732 1448.67 45.0239 1446.06 Q47.4429 1443.41 51.8034 1443.41 Q56.7687 1443.41 59.6651 1447.36 Q62.5615 1451.28 62.5615 1458.15 Q62.5615 1461.02 61.9886 1464.14 Q61.4475 1467.22 60.3335 1470.66 L54.2861 1470.66 Q55.973 1467.41 56.8323 1464.26 Q57.6599 1461.11 57.6599 1458.02 Q57.6599 1453.89 56.2594 1451.66 Q54.8271 1449.43 52.249 1449.43 Q49.8619 1449.43 48.5887 1451.05 Q47.3156 1452.65 46.1379 1458.09 L45.6605 1460.12 Q44.5147 1465.57 42.1594 1467.99 Q39.7722 1470.41 35.6345 1470.41 Q30.6056 1470.41 27.8684 1466.84 Q25.1311 1463.28 25.1311 1456.72 Q25.1311 1453.47 25.6085 1450.61 Q26.086 1447.74 27.0408 1445.32 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M27.0408 1411.36 L32.579 1411.36 Q31.3059 1413.85 30.6693 1416.52 Q30.0327 1419.19 30.0327 1422.06 Q30.0327 1426.42 31.3695 1428.61 Q32.7063 1430.78 35.3799 1430.78 Q37.4169 1430.78 38.5946 1429.22 Q39.7404 1427.66 40.7908 1422.95 L41.2364 1420.94 Q42.5732 1414.71 45.0239 1412.1 Q47.4429 1409.45 51.8034 1409.45 Q56.7687 1409.45 59.6651 1413.4 Q62.5615 1417.32 62.5615 1424.19 Q62.5615 1427.05 61.9886 1430.17 Q61.4475 1433.26 60.3335 1436.7 L54.2861 1436.7 Q55.973 1433.45 56.8323 1430.3 Q57.6599 1427.15 57.6599 1424.06 Q57.6599 1419.93 56.2594 1417.7 Q54.8271 1415.47 52.249 1415.47 Q49.8619 1415.47 48.5887 1417.09 Q47.3156 1418.68 46.1379 1424.13 L45.6605 1426.16 Q44.5147 1431.61 42.1594 1434.03 Q39.7722 1436.44 35.6345 1436.44 Q30.6056 1436.44 27.8684 1432.88 Q25.1311 1429.31 25.1311 1422.76 Q25.1311 1419.51 25.6085 1416.65 Q26.086 1413.78 27.0408 1411.36 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M30.0964 1386.31 Q30.0964 1391.03 33.7885 1393.76 Q37.4488 1396.5 43.8463 1396.5 Q50.2438 1396.5 53.9359 1393.79 Q57.5962 1391.06 57.5962 1386.31 Q57.5962 1381.64 53.9041 1378.9 Q50.212 1376.16 43.8463 1376.16 Q37.5124 1376.16 33.8203 1378.9 Q30.0964 1381.64 30.0964 1386.31 M25.1311 1386.31 Q25.1311 1378.68 30.0964 1374.32 Q35.0616 1369.95 43.8463 1369.95 Q52.5991 1369.95 57.5962 1374.32 Q62.5615 1378.68 62.5615 1386.31 Q62.5615 1393.99 57.5962 1398.35 Q52.5991 1402.67 43.8463 1402.67 Q35.0616 1402.67 30.0964 1398.35 Q25.1311 1393.99 25.1311 1386.31 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M12.1132 1360.25 L12.1132 1354.39 L61.6384 1354.39 L61.6384 1360.25 L12.1132 1360.25 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M25.9905 1346.34 L25.9905 1340.13 L55.9093 1328.99 L25.9905 1317.85 L25.9905 1311.64 L61.6384 1325.01 L61.6384 1332.97 L25.9905 1346.34 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M42.3504 1273.07 L45.2149 1273.07 L45.2149 1300 Q51.2623 1299.61 54.4452 1296.37 Q57.5962 1293.09 57.5962 1287.26 Q57.5962 1283.89 56.7687 1280.74 Q55.9411 1277.56 54.2861 1274.44 L59.8242 1274.44 Q61.161 1277.59 61.8612 1280.9 Q62.5615 1284.21 62.5615 1287.61 Q62.5615 1296.14 57.5962 1301.14 Q52.631 1306.11 44.1646 1306.11 Q35.4117 1306.11 30.2873 1301.4 Q25.1311 1296.65 25.1311 1288.63 Q25.1311 1281.44 29.7781 1277.27 Q34.3932 1273.07 42.3504 1273.07 M40.6316 1278.92 Q35.8255 1278.99 32.9609 1281.63 Q30.0964 1284.24 30.0964 1288.57 Q30.0964 1293.47 32.8654 1296.43 Q35.6345 1299.36 40.6634 1299.8 L40.6316 1278.92 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M31.4013 1240 L12.1132 1240 L12.1132 1234.14 L61.6384 1234.14 L61.6384 1240 L56.2912 1240 Q59.4741 1241.84 61.0337 1244.68 Q62.5615 1247.48 62.5615 1251.43 Q62.5615 1257.89 57.4052 1261.96 Q52.249 1266 43.8463 1266 Q35.4436 1266 30.2873 1261.96 Q25.1311 1257.89 25.1311 1251.43 Q25.1311 1247.48 26.6907 1244.68 Q28.2185 1241.84 31.4013 1240 M43.8463 1259.96 Q50.3075 1259.96 53.9996 1257.31 Q57.6599 1254.64 57.6599 1249.99 Q57.6599 1245.35 53.9996 1242.67 Q50.3075 1240 43.8463 1240 Q37.3851 1240 33.7248 1242.67 Q30.0327 1245.35 30.0327 1249.99 Q30.0327 1254.64 33.7248 1257.31 Q37.3851 1259.96 43.8463 1259.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M18.479 1181.82 Q18.479 1188.82 23.6988 1192.96 Q28.9187 1197.06 37.9262 1197.06 Q46.9018 1197.06 52.1217 1192.96 Q57.3416 1188.82 57.3416 1181.82 Q57.3416 1174.81 52.1217 1170.74 Q46.9018 1166.63 37.9262 1166.63 Q28.9187 1166.63 23.6988 1170.74 Q18.479 1174.81 18.479 1181.82 M13.2591 1181.82 Q13.2591 1171.82 19.9749 1165.84 Q26.6589 1159.85 37.9262 1159.85 Q49.1617 1159.85 55.8775 1165.84 Q62.5615 1171.82 62.5615 1181.82 Q62.5615 1191.84 55.8775 1197.86 Q49.1935 1203.84 37.9262 1203.84 Q26.6589 1203.84 19.9749 1197.86 Q13.2591 1191.84 13.2591 1181.82 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M25.9905 1120.42 L43.337 1133.31 L61.6384 1119.75 L61.6384 1126.66 L47.6339 1137.03 L61.6384 1147.41 L61.6384 1154.32 L42.9869 1140.47 L25.9905 1153.14 L25.9905 1146.23 L38.6901 1136.78 L25.9905 1127.33 L25.9905 1120.42 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M64.9486 1096.64 Q71.3143 1099.13 73.2559 1101.48 Q75.1974 1103.84 75.1974 1107.78 L75.1974 1112.46 L70.2958 1112.46 L70.2958 1109.02 Q70.2958 1106.61 69.15 1105.27 Q68.0041 1103.93 63.7391 1102.31 L61.0655 1101.26 L25.9905 1115.68 L25.9905 1109.47 L53.8723 1098.33 L25.9905 1087.19 L25.9905 1080.98 L64.9486 1096.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M43.4007 1049.44 Q37.035 1049.44 33.5338 1052.08 Q30.0327 1054.69 30.0327 1059.44 Q30.0327 1064.15 33.5338 1066.79 Q37.035 1069.4 43.4007 1069.4 Q49.7346 1069.4 53.2357 1066.79 Q56.7368 1064.15 56.7368 1059.44 Q56.7368 1054.69 53.2357 1052.08 Q49.7346 1049.44 43.4007 1049.44 M57.2143 1043.58 Q66.3172 1043.58 70.7414 1047.63 Q75.1974 1051.67 75.1974 1060.01 Q75.1974 1063.1 74.72 1065.83 Q74.2744 1068.57 73.3195 1071.15 L67.6222 1071.15 Q69.0227 1068.57 69.6911 1066.06 Q70.3595 1063.54 70.3595 1060.93 Q70.3595 1055.17 67.3358 1052.31 Q64.3439 1049.44 58.2646 1049.44 L55.3682 1049.44 Q58.5192 1051.26 60.0788 1054.09 Q61.6384 1056.92 61.6384 1060.87 Q61.6384 1067.42 56.6414 1071.43 Q51.6443 1075.45 43.4007 1075.45 Q35.1253 1075.45 30.1282 1071.43 Q25.1311 1067.42 25.1311 1060.87 Q25.1311 1056.92 26.6907 1054.09 Q28.2503 1051.26 31.4013 1049.44 L25.9905 1049.44 L25.9905 1043.58 L57.2143 1043.58 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M42.3504 1001.03 L45.2149 1001.03 L45.2149 1027.96 Q51.2623 1027.58 54.4452 1024.33 Q57.5962 1021.05 57.5962 1015.23 Q57.5962 1011.85 56.7687 1008.7 Q55.9411 1005.52 54.2861 1002.4 L59.8242 1002.4 Q61.161 1005.55 61.8612 1008.86 Q62.5615 1012.17 62.5615 1015.58 Q62.5615 1024.11 57.5962 1029.1 Q52.631 1034.07 44.1646 1034.07 Q35.4117 1034.07 30.2873 1029.36 Q25.1311 1024.62 25.1311 1016.59 Q25.1311 1009.4 29.7781 1005.23 Q34.3932 1001.03 42.3504 1001.03 M40.6316 1006.89 Q35.8255 1006.95 32.9609 1009.59 Q30.0964 1012.2 30.0964 1016.53 Q30.0964 1021.43 32.8654 1024.39 Q35.6345 1027.32 40.6634 1027.77 L40.6316 1006.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M40.1224 961.785 L61.6384 961.785 L61.6384 967.642 L40.3133 967.642 Q35.2526 967.642 32.7381 969.615 Q30.2237 971.589 30.2237 975.535 Q30.2237 980.278 33.2474 983.015 Q36.2711 985.752 41.491 985.752 L61.6384 985.752 L61.6384 991.641 L25.9905 991.641 L25.9905 985.752 L31.5287 985.752 Q28.314 983.652 26.7225 980.819 Q25.1311 977.954 25.1311 974.23 Q25.1311 968.088 28.9505 964.937 Q32.7381 961.785 40.1224 961.785 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M17.7787 893.545 L24.5582 893.545 Q21.5345 896.792 20.0386 900.484 Q18.5426 904.144 18.5426 908.282 Q18.5426 916.43 23.5397 920.759 Q28.5049 925.087 37.9262 925.087 Q47.3156 925.087 52.3127 920.759 Q57.2779 916.43 57.2779 908.282 Q57.2779 904.144 55.782 900.484 Q54.2861 896.792 51.2623 893.545 L57.9782 893.545 Q60.2698 896.919 61.4156 900.707 Q62.5615 904.462 62.5615 908.664 Q62.5615 919.454 55.973 925.66 Q49.3526 931.867 37.9262 931.867 Q26.4679 931.867 19.8794 925.66 Q13.2591 919.454 13.2591 908.664 Q13.2591 904.399 14.4049 900.643 Q15.5189 896.855 17.7787 893.545 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M30.0964 870.056 Q30.0964 874.766 33.7885 877.504 Q37.4488 880.241 43.8463 880.241 Q50.2438 880.241 53.9359 877.535 Q57.5962 874.798 57.5962 870.056 Q57.5962 865.377 53.9041 862.64 Q50.212 859.902 43.8463 859.902 Q37.5124 859.902 33.8203 862.64 Q30.0964 865.377 30.0964 870.056 M25.1311 870.056 Q25.1311 862.417 30.0964 858.056 Q35.0616 853.696 43.8463 853.696 Q52.5991 853.696 57.5962 858.056 Q62.5615 862.417 62.5615 870.056 Q62.5615 877.726 57.5962 882.087 Q52.5991 886.416 43.8463 886.416 Q35.0616 886.416 30.0964 882.087 Q25.1311 877.726 25.1311 870.056 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M40.1224 814.356 L61.6384 814.356 L61.6384 820.212 L40.3133 820.212 Q35.2526 820.212 32.7381 822.186 Q30.2237 824.159 30.2237 828.106 Q30.2237 832.848 33.2474 835.585 Q36.2711 838.323 41.491 838.323 L61.6384 838.323 L61.6384 844.211 L25.9905 844.211 L25.9905 838.323 L31.5287 838.323 Q28.314 836.222 26.7225 833.389 Q25.1311 830.525 25.1311 826.801 Q25.1311 820.658 28.9505 817.507 Q32.7381 814.356 40.1224 814.356 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M27.3591 777.021 L32.8336 777.021 Q31.465 779.503 30.7966 782.018 Q30.0964 784.501 30.0964 787.047 Q30.0964 792.744 33.7248 795.895 Q37.3214 799.046 43.8463 799.046 Q50.3711 799.046 53.9996 795.895 Q57.5962 792.744 57.5962 787.047 Q57.5962 784.501 56.9278 782.018 Q56.2276 779.503 54.859 777.021 L60.2698 777.021 Q61.4156 779.472 61.9886 782.113 Q62.5615 784.723 62.5615 787.683 Q62.5615 795.736 57.5007 800.478 Q52.44 805.221 43.8463 805.221 Q35.1253 805.221 30.1282 800.447 Q25.1311 795.641 25.1311 787.301 Q25.1311 784.596 25.704 782.018 Q26.2451 779.44 27.3591 777.021 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M42.3504 736.344 L45.2149 736.344 L45.2149 763.271 Q51.2623 762.889 54.4452 759.642 Q57.5962 756.364 57.5962 750.54 Q57.5962 747.166 56.7687 744.015 Q55.9411 740.832 54.2861 737.713 L59.8242 737.713 Q61.161 740.864 61.8612 744.174 Q62.5615 747.484 62.5615 750.89 Q62.5615 759.42 57.5962 764.417 Q52.631 769.382 44.1646 769.382 Q35.4117 769.382 30.2873 764.671 Q25.1311 759.929 25.1311 751.908 Q25.1311 744.715 29.7781 740.545 Q34.3932 736.344 42.3504 736.344 M40.6316 742.2 Q35.8255 742.264 32.9609 744.906 Q30.0964 747.516 30.0964 751.844 Q30.0964 756.746 32.8654 759.706 Q35.6345 762.634 40.6634 763.08 L40.6316 742.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M40.1224 697.099 L61.6384 697.099 L61.6384 702.956 L40.3133 702.956 Q35.2526 702.956 32.7381 704.929 Q30.2237 706.903 30.2237 710.849 Q30.2237 715.592 33.2474 718.329 Q36.2711 721.066 41.491 721.066 L61.6384 721.066 L61.6384 726.955 L25.9905 726.955 L25.9905 721.066 L31.5287 721.066 Q28.314 718.966 26.7225 716.133 Q25.1311 713.268 25.1311 709.544 Q25.1311 703.401 28.9505 700.25 Q32.7381 697.099 40.1224 697.099 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M15.869 679.626 L25.9905 679.626 L25.9905 667.563 L30.542 667.563 L30.542 679.626 L49.8937 679.626 Q54.2542 679.626 55.4955 678.448 Q56.7368 677.238 56.7368 673.578 L56.7368 667.563 L61.6384 667.563 L61.6384 673.578 Q61.6384 680.358 59.124 682.936 Q56.5777 685.514 49.8937 685.514 L30.542 685.514 L30.542 689.811 L25.9905 689.811 L25.9905 685.514 L15.869 685.514 L15.869 679.626 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M31.465 639.203 Q30.8921 640.19 30.6375 641.368 Q30.351 642.513 30.351 643.914 Q30.351 648.879 33.5975 651.553 Q36.8122 654.195 42.8596 654.195 L61.6384 654.195 L61.6384 660.083 L25.9905 660.083 L25.9905 654.195 L31.5287 654.195 Q28.2821 652.348 26.7225 649.388 Q25.1311 646.428 25.1311 642.195 Q25.1311 641.59 25.2266 640.858 Q25.2903 640.126 25.4494 639.235 L31.465 639.203 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M43.719 616.86 Q43.719 623.957 45.3422 626.695 Q46.9655 629.432 50.8804 629.432 Q53.9996 629.432 55.8457 627.395 Q57.6599 625.326 57.6599 621.793 Q57.6599 616.923 54.2224 613.995 Q50.7531 611.035 45.0239 611.035 L43.719 611.035 L43.719 616.86 M41.3 605.179 L61.6384 605.179 L61.6384 611.035 L56.2276 611.035 Q59.4741 613.04 61.0337 616.032 Q62.5615 619.024 62.5615 623.353 Q62.5615 628.827 59.5059 632.074 Q56.4186 635.288 51.2623 635.288 Q45.2467 635.288 42.1912 631.278 Q39.1357 627.236 39.1357 619.247 L39.1357 611.035 L38.5628 611.035 Q34.5205 611.035 32.3244 613.709 Q30.0964 616.35 30.0964 621.157 Q30.0964 624.212 30.8284 627.108 Q31.5605 630.005 33.0246 632.678 L27.6137 632.678 Q26.3724 629.464 25.7677 626.44 Q25.1311 623.416 25.1311 620.552 Q25.1311 612.817 29.1415 608.998 Q33.1519 605.179 41.3 605.179 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M15.869 587.323 L25.9905 587.323 L25.9905 575.26 L30.542 575.26 L30.542 587.323 L49.8937 587.323 Q54.2542 587.323 55.4955 586.145 Q56.7368 584.936 56.7368 581.275 L56.7368 575.26 L61.6384 575.26 L61.6384 581.275 Q61.6384 588.055 59.124 590.633 Q56.5777 593.211 49.8937 593.211 L30.542 593.211 L30.542 597.508 L25.9905 597.508 L25.9905 593.211 L15.869 593.211 L15.869 587.323 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M25.9905 567.557 L25.9905 561.701 L61.6384 561.701 L61.6384 567.557 L25.9905 567.557 M12.1132 567.557 L12.1132 561.701 L19.5293 561.701 L19.5293 567.557 L12.1132 567.557 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M30.0964 535.633 Q30.0964 540.344 33.7885 543.081 Q37.4488 545.818 43.8463 545.818 Q50.2438 545.818 53.9359 543.113 Q57.5962 540.376 57.5962 535.633 Q57.5962 530.954 53.9041 528.217 Q50.212 525.48 43.8463 525.48 Q37.5124 525.48 33.8203 528.217 Q30.0964 530.954 30.0964 535.633 M25.1311 535.633 Q25.1311 527.994 30.0964 523.634 Q35.0616 519.273 43.8463 519.273 Q52.5991 519.273 57.5962 523.634 Q62.5615 527.994 62.5615 535.633 Q62.5615 543.304 57.5962 547.664 Q52.5991 551.993 43.8463 551.993 Q35.0616 551.993 30.0964 547.664 Q25.1311 543.304 25.1311 535.633 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M40.1224 479.933 L61.6384 479.933 L61.6384 485.79 L40.3133 485.79 Q35.2526 485.79 32.7381 487.763 Q30.2237 489.737 30.2237 493.683 Q30.2237 498.426 33.2474 501.163 Q36.2711 503.9 41.491 503.9 L61.6384 503.9 L61.6384 509.788 L25.9905 509.788 L25.9905 503.9 L31.5287 503.9 Q28.314 501.8 26.7225 498.967 Q25.1311 496.102 25.1311 492.378 Q25.1311 486.235 28.9505 483.084 Q32.7381 479.933 40.1224 479.933 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M12.1769 433.464 Q19.4975 437.729 26.6589 439.798 Q33.8203 441.866 41.1727 441.866 Q48.5251 441.866 55.7502 439.798 Q62.9434 437.697 70.2321 433.464 L70.2321 438.556 Q62.7524 443.331 55.5274 445.718 Q48.3023 448.073 41.1727 448.073 Q34.0749 448.073 26.8817 445.718 Q19.6884 443.362 12.1769 438.556 L12.1769 433.464 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M32.8336 394.346 Q28.8869 392.15 27.009 389.095 Q25.1311 386.039 25.1311 381.901 Q25.1311 376.331 29.046 373.308 Q32.9291 370.284 40.1224 370.284 L61.6384 370.284 L61.6384 376.172 L40.3133 376.172 Q35.1889 376.172 32.7063 377.987 Q30.2237 379.801 30.2237 383.525 Q30.2237 388.076 33.2474 390.718 Q36.2711 393.36 41.491 393.36 L61.6384 393.36 L61.6384 399.248 L40.3133 399.248 Q35.1571 399.248 32.7063 401.062 Q30.2237 402.876 30.2237 406.664 Q30.2237 411.152 33.2792 413.794 Q36.3029 416.435 41.491 416.435 L61.6384 416.435 L61.6384 422.324 L25.9905 422.324 L25.9905 416.435 L31.5287 416.435 Q28.2503 414.43 26.6907 411.629 Q25.1311 408.828 25.1311 404.977 Q25.1311 401.094 27.1045 398.389 Q29.0779 395.651 32.8336 394.346 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M43.4007 335.145 Q37.035 335.145 33.5338 337.787 Q30.0327 340.397 30.0327 345.139 Q30.0327 349.85 33.5338 352.492 Q37.035 355.102 43.4007 355.102 Q49.7346 355.102 53.2357 352.492 Q56.7368 349.85 56.7368 345.139 Q56.7368 340.397 53.2357 337.787 Q49.7346 335.145 43.4007 335.145 M57.2143 329.289 Q66.3172 329.289 70.7414 333.331 Q75.1974 337.373 75.1974 345.712 Q75.1974 348.8 74.72 351.537 Q74.2744 354.274 73.3195 356.852 L67.6222 356.852 Q69.0227 354.274 69.6911 351.76 Q70.3595 349.245 70.3595 346.635 Q70.3595 340.874 67.3358 338.01 Q64.3439 335.145 58.2646 335.145 L55.3682 335.145 Q58.5192 336.96 60.0788 339.792 Q61.6384 342.625 61.6384 346.572 Q61.6384 353.128 56.6414 357.139 Q51.6443 361.149 43.4007 361.149 Q35.1253 361.149 30.1282 357.139 Q25.1311 353.128 25.1311 346.572 Q25.1311 342.625 26.6907 339.792 Q28.2503 336.96 31.4013 335.145 L25.9905 335.145 L25.9905 329.289 L57.2143 329.289 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M14.1184 306.818 L14.1184 301.407 L67.6859 317.958 L67.6859 323.369 L14.1184 306.818 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M14.1184 295.01 L14.1184 288.58 L56.2276 288.58 L56.2276 265.441 L61.6384 265.441 L61.6384 295.01 L14.1184 295.01 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M12.1769 259.871 L12.1769 254.778 Q19.6884 250.004 26.8817 247.649 Q34.0749 245.262 41.1727 245.262 Q48.3023 245.262 55.5274 247.649 Q62.7524 250.004 70.2321 254.778 L70.2321 259.871 Q62.9434 255.638 55.7502 253.569 Q48.5251 251.468 41.1727 251.468 Q33.8203 251.468 26.6589 253.569 Q19.4975 255.638 12.1769 259.871 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip822)" style="stroke:#009af9; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="294.794,94.9858 340.346,341.52 385.899,548.595 431.452,720.592 477.004,861.46 522.557,974.757 568.11,1063.68 613.662,1131.12 659.215,1179.66 704.768,1211.61 750.32,1229.08 795.873,1233.92 841.426,1227.82 886.978,1212.27 932.531,1188.6 978.084,1205.11 1023.64,1347.79 1069.19,1460.52 1114.74,1546.81 1160.29,1609.83 1205.85,1652.45 1251.4,1677.2 1296.95,1686.38 1342.5,1682.04 1388.06,1666.01 1433.61,1639.95 1479.16,1605.31 1524.72,1563.4 1570.27,1515.38 1615.82,1462.29 1661.37,1405.04 1706.93,1344.45 1752.48,1281.22 1798.03,1216.01 1843.58,1149.35 1889.14,1081.74 1934.69,1013.61 1980.24,945.332 2025.79,877.218 2071.35,809.552 2116.9,742.571 2162.45,676.478 2208.01,611.446 2253.56,547.619 2299.11,485.115 2344.66,424.033 2390.22,364.449 2435.77,306.425 2481.32,250.007 2526.87,195.225 2572.43,142.102 "/>
<polyline clip-path="url(#clip822)" style="stroke:#e26f46; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="-2187.83,1550.43 5055.05,1550.43 "/>
<path clip-path="url(#clip820)" d="M1073.27 258.993 L1793.95 258.993 L1793.95 103.473 L1073.27 103.473  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip820)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1073.27,258.993 1793.95,258.993 1793.95,103.473 1073.27,103.473 1073.27,258.993 "/>
<polyline clip-path="url(#clip820)" style="stroke:#009af9; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1100.09,155.313 1261.05,155.313 "/>
<path clip-path="url(#clip820)" d="M1310.12 139.167 L1310.12 143.728 Q1307.45 142.454 1305.09 141.829 Q1302.73 141.204 1300.53 141.204 Q1296.71 141.204 1294.63 142.686 Q1292.57 144.167 1292.57 146.899 Q1292.57 149.191 1293.94 150.371 Q1295.32 151.528 1299.17 152.246 L1301.99 152.825 Q1307.22 153.82 1309.7 156.343 Q1312.2 158.843 1312.2 163.056 Q1312.2 168.079 1308.82 170.672 Q1305.46 173.264 1298.96 173.264 Q1296.51 173.264 1293.73 172.709 Q1290.97 172.153 1288.01 171.065 L1288.01 166.251 Q1290.86 167.848 1293.59 168.658 Q1296.32 169.468 1298.96 169.468 Q1302.96 169.468 1305.14 167.894 Q1307.32 166.32 1307.32 163.403 Q1307.32 160.857 1305.74 159.422 Q1304.19 157.987 1300.63 157.269 L1297.78 156.714 Q1292.55 155.672 1290.21 153.45 Q1287.87 151.228 1287.87 147.269 Q1287.87 142.686 1291.09 140.047 Q1294.33 137.408 1300 137.408 Q1302.43 137.408 1304.95 137.848 Q1307.48 138.288 1310.12 139.167 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1319.31 146.667 L1323.57 146.667 L1323.57 172.593 L1319.31 172.593 L1319.31 146.667 M1319.31 136.575 L1323.57 136.575 L1323.57 141.968 L1319.31 141.968 L1319.31 136.575 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1352.66 151.644 Q1354.26 148.774 1356.48 147.408 Q1358.7 146.042 1361.71 146.042 Q1365.76 146.042 1367.96 148.89 Q1370.16 151.714 1370.16 156.945 L1370.16 172.593 L1365.88 172.593 L1365.88 157.084 Q1365.88 153.357 1364.56 151.552 Q1363.24 149.746 1360.53 149.746 Q1357.22 149.746 1355.3 151.945 Q1353.38 154.144 1353.38 157.94 L1353.38 172.593 L1349.1 172.593 L1349.1 157.084 Q1349.1 153.334 1347.78 151.552 Q1346.46 149.746 1343.7 149.746 Q1340.44 149.746 1338.52 151.968 Q1336.6 154.167 1336.6 157.94 L1336.6 172.593 L1332.32 172.593 L1332.32 146.667 L1336.6 146.667 L1336.6 150.695 Q1338.06 148.311 1340.09 147.177 Q1342.13 146.042 1344.93 146.042 Q1347.75 146.042 1349.72 147.478 Q1351.71 148.913 1352.66 151.644 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1378.22 162.362 L1378.22 146.667 L1382.48 146.667 L1382.48 162.2 Q1382.48 165.88 1383.91 167.732 Q1385.35 169.561 1388.22 169.561 Q1391.67 169.561 1393.66 167.362 Q1395.67 165.163 1395.67 161.366 L1395.67 146.667 L1399.93 146.667 L1399.93 172.593 L1395.67 172.593 L1395.67 168.612 Q1394.12 170.973 1392.06 172.13 Q1390.02 173.264 1387.31 173.264 Q1382.85 173.264 1380.53 170.487 Q1378.22 167.709 1378.22 162.362 M1388.94 146.042 L1388.94 146.042 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1408.7 136.575 L1412.96 136.575 L1412.96 172.593 L1408.7 172.593 L1408.7 136.575 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1433.66 159.561 Q1428.5 159.561 1426.5 160.741 Q1424.51 161.922 1424.51 164.769 Q1424.51 167.038 1426 168.38 Q1427.5 169.7 1430.07 169.7 Q1433.61 169.7 1435.74 167.2 Q1437.89 164.677 1437.89 160.51 L1437.89 159.561 L1433.66 159.561 M1442.15 157.802 L1442.15 172.593 L1437.89 172.593 L1437.89 168.658 Q1436.44 171.019 1434.26 172.153 Q1432.08 173.264 1428.94 173.264 Q1424.95 173.264 1422.59 171.042 Q1420.25 168.797 1420.25 165.047 Q1420.25 160.672 1423.17 158.45 Q1426.11 156.228 1431.92 156.228 L1437.89 156.228 L1437.89 155.811 Q1437.89 152.871 1435.95 151.274 Q1434.03 149.653 1430.53 149.653 Q1428.31 149.653 1426.2 150.186 Q1424.1 150.718 1422.15 151.783 L1422.15 147.848 Q1424.49 146.945 1426.69 146.505 Q1428.89 146.042 1430.97 146.042 Q1436.6 146.042 1439.37 148.959 Q1442.15 151.876 1442.15 157.802 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1455.14 139.306 L1455.14 146.667 L1463.91 146.667 L1463.91 149.978 L1455.14 149.978 L1455.14 164.052 Q1455.14 167.223 1456 168.126 Q1456.87 169.028 1459.54 169.028 L1463.91 169.028 L1463.91 172.593 L1459.54 172.593 Q1454.61 172.593 1452.73 170.764 Q1450.86 168.913 1450.86 164.052 L1450.86 149.978 L1447.73 149.978 L1447.73 146.667 L1450.86 146.667 L1450.86 139.306 L1455.14 139.306 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1491.69 158.565 L1491.69 160.649 L1472.11 160.649 Q1472.38 165.047 1474.75 167.362 Q1477.13 169.653 1481.37 169.653 Q1483.82 169.653 1486.11 169.052 Q1488.43 168.45 1490.69 167.246 L1490.69 171.274 Q1488.4 172.246 1485.99 172.755 Q1483.59 173.264 1481.11 173.264 Q1474.91 173.264 1471.27 169.653 Q1467.66 166.042 1467.66 159.885 Q1467.66 153.519 1471.09 149.792 Q1474.54 146.042 1480.37 146.042 Q1485.6 146.042 1488.63 149.422 Q1491.69 152.778 1491.69 158.565 M1487.43 157.315 Q1487.38 153.82 1485.46 151.737 Q1483.56 149.653 1480.42 149.653 Q1476.85 149.653 1474.7 151.667 Q1472.57 153.681 1472.25 157.339 L1487.43 157.315 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1515.74 150.603 L1515.74 136.575 L1520 136.575 L1520 172.593 L1515.74 172.593 L1515.74 168.704 Q1514.4 171.019 1512.34 172.153 Q1510.3 173.264 1507.43 173.264 Q1502.73 173.264 1499.77 169.515 Q1496.83 165.765 1496.83 159.653 Q1496.83 153.542 1499.77 149.792 Q1502.73 146.042 1507.43 146.042 Q1510.3 146.042 1512.34 147.177 Q1514.4 148.288 1515.74 150.603 M1501.23 159.653 Q1501.23 164.352 1503.15 167.038 Q1505.09 169.7 1508.47 169.7 Q1511.85 169.7 1513.8 167.038 Q1515.74 164.352 1515.74 159.653 Q1515.74 154.954 1513.8 152.292 Q1511.85 149.607 1508.47 149.607 Q1505.09 149.607 1503.15 152.292 Q1501.23 154.954 1501.23 159.653 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1548.7 141.876 L1548.7 168.751 L1554.35 168.751 Q1561.5 168.751 1564.81 165.51 Q1568.15 162.269 1568.15 155.278 Q1568.15 148.334 1564.81 145.116 Q1561.5 141.876 1554.35 141.876 L1548.7 141.876 M1544.03 138.033 L1553.63 138.033 Q1563.68 138.033 1568.38 142.223 Q1573.08 146.39 1573.08 155.278 Q1573.08 164.214 1568.36 168.403 Q1563.63 172.593 1553.63 172.593 L1544.03 172.593 L1544.03 138.033 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1594.56 141.204 Q1589.47 141.204 1586.46 145.001 Q1583.47 148.797 1583.47 155.348 Q1583.47 161.876 1586.46 165.672 Q1589.47 169.468 1594.56 169.468 Q1599.65 169.468 1602.61 165.672 Q1605.6 161.876 1605.6 155.348 Q1605.6 148.797 1602.61 145.001 Q1599.65 141.204 1594.56 141.204 M1594.56 137.408 Q1601.83 137.408 1606.18 142.292 Q1610.53 147.154 1610.53 155.348 Q1610.53 163.519 1606.18 168.403 Q1601.83 173.264 1594.56 173.264 Q1587.27 173.264 1582.89 168.403 Q1578.54 163.542 1578.54 155.348 Q1578.54 147.154 1582.89 142.292 Q1587.27 137.408 1594.56 137.408 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip820)" style="stroke:#e26f46; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1100.09,207.153 1261.05,207.153 "/>
<path clip-path="url(#clip820)" d="M1304.26 208.23 Q1305.76 208.739 1307.18 210.405 Q1308.61 212.072 1310.05 214.989 L1314.79 224.433 L1309.77 224.433 L1305.35 215.567 Q1303.63 212.095 1302.01 210.961 Q1300.42 209.827 1297.64 209.827 L1292.55 209.827 L1292.55 224.433 L1287.87 224.433 L1287.87 189.873 L1298.43 189.873 Q1304.35 189.873 1307.27 192.35 Q1310.19 194.827 1310.19 199.827 Q1310.19 203.091 1308.66 205.243 Q1307.15 207.396 1304.26 208.23 M1292.55 193.716 L1292.55 205.984 L1298.43 205.984 Q1301.81 205.984 1303.52 204.433 Q1305.26 202.859 1305.26 199.827 Q1305.26 196.794 1303.52 195.267 Q1301.81 193.716 1298.43 193.716 L1292.55 193.716 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1340.67 210.405 L1340.67 212.489 L1321.09 212.489 Q1321.37 216.887 1323.73 219.202 Q1326.11 221.493 1330.35 221.493 Q1332.8 221.493 1335.09 220.892 Q1337.41 220.29 1339.68 219.086 L1339.68 223.114 Q1337.38 224.086 1334.98 224.595 Q1332.57 225.104 1330.09 225.104 Q1323.89 225.104 1320.26 221.493 Q1316.64 217.882 1316.64 211.725 Q1316.64 205.359 1320.07 201.632 Q1323.52 197.882 1329.35 197.882 Q1334.58 197.882 1337.62 201.262 Q1340.67 204.618 1340.67 210.405 M1336.41 209.155 Q1336.37 205.66 1334.44 203.577 Q1332.55 201.493 1329.4 201.493 Q1325.83 201.493 1323.68 203.507 Q1321.55 205.521 1321.23 209.179 L1336.41 209.155 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1364.72 211.169 Q1364.72 206.54 1362.8 203.993 Q1360.9 201.447 1357.45 201.447 Q1354.03 201.447 1352.11 203.993 Q1350.21 206.54 1350.21 211.169 Q1350.21 215.776 1352.11 218.322 Q1354.03 220.868 1357.45 220.868 Q1360.9 220.868 1362.8 218.322 Q1364.72 215.776 1364.72 211.169 M1368.98 221.216 Q1368.98 227.836 1366.04 231.054 Q1363.1 234.294 1357.04 234.294 Q1354.79 234.294 1352.8 233.947 Q1350.81 233.623 1348.94 232.929 L1348.94 228.785 Q1350.81 229.804 1352.64 230.29 Q1354.47 230.776 1356.37 230.776 Q1360.56 230.776 1362.64 228.577 Q1364.72 226.401 1364.72 221.98 L1364.72 219.873 Q1363.4 222.165 1361.34 223.299 Q1359.28 224.433 1356.41 224.433 Q1351.64 224.433 1348.73 220.799 Q1345.81 217.165 1345.81 211.169 Q1345.81 205.151 1348.73 201.517 Q1351.64 197.882 1356.41 197.882 Q1359.28 197.882 1361.34 199.017 Q1363.4 200.151 1364.72 202.443 L1364.72 198.507 L1368.98 198.507 L1368.98 221.216 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1377.31 214.202 L1377.31 198.507 L1381.57 198.507 L1381.57 214.04 Q1381.57 217.72 1383.01 219.572 Q1384.44 221.401 1387.31 221.401 Q1390.76 221.401 1392.75 219.202 Q1394.77 217.003 1394.77 213.206 L1394.77 198.507 L1399.03 198.507 L1399.03 224.433 L1394.77 224.433 L1394.77 220.452 Q1393.22 222.813 1391.16 223.97 Q1389.12 225.104 1386.41 225.104 Q1381.94 225.104 1379.63 222.327 Q1377.31 219.549 1377.31 214.202 M1388.03 197.882 L1388.03 197.882 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1407.8 188.415 L1412.06 188.415 L1412.06 224.433 L1407.8 224.433 L1407.8 188.415 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1432.75 211.401 Q1427.59 211.401 1425.6 212.581 Q1423.61 213.762 1423.61 216.609 Q1423.61 218.878 1425.09 220.22 Q1426.6 221.54 1429.17 221.54 Q1432.71 221.54 1434.84 219.04 Q1436.99 216.517 1436.99 212.35 L1436.99 211.401 L1432.75 211.401 M1441.25 209.642 L1441.25 224.433 L1436.99 224.433 L1436.99 220.498 Q1435.53 222.859 1433.36 223.993 Q1431.18 225.104 1428.03 225.104 Q1424.05 225.104 1421.69 222.882 Q1419.35 220.637 1419.35 216.887 Q1419.35 212.512 1422.27 210.29 Q1425.21 208.068 1431.02 208.068 L1436.99 208.068 L1436.99 207.651 Q1436.99 204.711 1435.05 203.114 Q1433.12 201.493 1429.63 201.493 Q1427.41 201.493 1425.3 202.026 Q1423.19 202.558 1421.25 203.623 L1421.25 199.688 Q1423.59 198.785 1425.79 198.345 Q1427.99 197.882 1430.07 197.882 Q1435.69 197.882 1438.47 200.799 Q1441.25 203.716 1441.25 209.642 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1454.24 191.146 L1454.24 198.507 L1463.01 198.507 L1463.01 201.818 L1454.24 201.818 L1454.24 215.892 Q1454.24 219.063 1455.09 219.966 Q1455.97 220.868 1458.63 220.868 L1463.01 220.868 L1463.01 224.433 L1458.63 224.433 Q1453.7 224.433 1451.83 222.604 Q1449.95 220.753 1449.95 215.892 L1449.95 201.818 L1446.83 201.818 L1446.83 198.507 L1449.95 198.507 L1449.95 191.146 L1454.24 191.146 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1478.66 201.493 Q1475.23 201.493 1473.24 204.179 Q1471.25 206.841 1471.25 211.493 Q1471.25 216.146 1473.22 218.831 Q1475.21 221.493 1478.66 221.493 Q1482.06 221.493 1484.05 218.808 Q1486.04 216.123 1486.04 211.493 Q1486.04 206.887 1484.05 204.202 Q1482.06 201.493 1478.66 201.493 M1478.66 197.882 Q1484.21 197.882 1487.38 201.493 Q1490.56 205.105 1490.56 211.493 Q1490.56 217.859 1487.38 221.493 Q1484.21 225.104 1478.66 225.104 Q1473.08 225.104 1469.91 221.493 Q1466.76 217.859 1466.76 211.493 Q1466.76 205.105 1469.91 201.493 Q1473.08 197.882 1478.66 197.882 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1512.64 202.489 Q1511.92 202.072 1511.06 201.887 Q1510.23 201.679 1509.21 201.679 Q1505.6 201.679 1503.66 204.04 Q1501.74 206.378 1501.74 210.776 L1501.74 224.433 L1497.45 224.433 L1497.45 198.507 L1501.74 198.507 L1501.74 202.535 Q1503.08 200.174 1505.23 199.04 Q1507.38 197.882 1510.46 197.882 Q1510.9 197.882 1511.43 197.952 Q1511.97 197.998 1512.62 198.114 L1512.64 202.489 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1527.89 226.841 Q1526.09 231.47 1524.37 232.882 Q1522.66 234.294 1519.79 234.294 L1516.39 234.294 L1516.39 230.729 L1518.89 230.729 Q1520.65 230.729 1521.62 229.896 Q1522.59 229.063 1523.77 225.961 L1524.54 224.017 L1514.05 198.507 L1518.56 198.507 L1526.67 218.785 L1534.77 198.507 L1539.28 198.507 L1527.89 226.841 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1581.13 191.007 L1581.13 195.568 Q1578.47 194.294 1576.11 193.669 Q1573.75 193.044 1571.55 193.044 Q1567.73 193.044 1565.65 194.526 Q1563.59 196.007 1563.59 198.739 Q1563.59 201.031 1564.95 202.211 Q1566.34 203.368 1570.18 204.086 L1573.01 204.665 Q1578.24 205.66 1580.72 208.183 Q1583.22 210.683 1583.22 214.896 Q1583.22 219.919 1579.84 222.512 Q1576.48 225.104 1569.98 225.104 Q1567.52 225.104 1564.74 224.549 Q1561.99 223.993 1559.03 222.905 L1559.03 218.091 Q1561.87 219.688 1564.61 220.498 Q1567.34 221.308 1569.98 221.308 Q1573.98 221.308 1576.16 219.734 Q1578.33 218.16 1578.33 215.243 Q1578.33 212.697 1576.76 211.262 Q1575.21 209.827 1571.64 209.109 L1568.8 208.554 Q1563.56 207.512 1561.23 205.29 Q1558.89 203.068 1558.89 199.109 Q1558.89 194.526 1562.11 191.887 Q1565.35 189.248 1571.02 189.248 Q1573.45 189.248 1575.97 189.688 Q1578.49 190.128 1581.13 191.007 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1594.54 191.146 L1594.54 198.507 L1603.31 198.507 L1603.31 201.818 L1594.54 201.818 L1594.54 215.892 Q1594.54 219.063 1595.39 219.966 Q1596.27 220.868 1598.93 220.868 L1603.31 220.868 L1603.31 224.433 L1598.93 224.433 Q1594 224.433 1592.13 222.604 Q1590.25 220.753 1590.25 215.892 L1590.25 201.818 L1587.13 201.818 L1587.13 198.507 L1590.25 198.507 L1590.25 191.146 L1594.54 191.146 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1620.69 211.401 Q1615.53 211.401 1613.54 212.581 Q1611.55 213.762 1611.55 216.609 Q1611.55 218.878 1613.03 220.22 Q1614.54 221.54 1617.11 221.54 Q1620.65 221.54 1622.78 219.04 Q1624.93 216.517 1624.93 212.35 L1624.93 211.401 L1620.69 211.401 M1629.19 209.642 L1629.19 224.433 L1624.93 224.433 L1624.93 220.498 Q1623.47 222.859 1621.3 223.993 Q1619.12 225.104 1615.97 225.104 Q1611.99 225.104 1609.63 222.882 Q1607.29 220.637 1607.29 216.887 Q1607.29 212.512 1610.21 210.29 Q1613.15 208.068 1618.96 208.068 L1624.93 208.068 L1624.93 207.651 Q1624.93 204.711 1622.98 203.114 Q1621.06 201.493 1617.57 201.493 Q1615.35 201.493 1613.24 202.026 Q1611.13 202.558 1609.19 203.623 L1609.19 199.688 Q1611.53 198.785 1613.73 198.345 Q1615.92 197.882 1618.01 197.882 Q1623.63 197.882 1626.41 200.799 Q1629.19 203.716 1629.19 209.642 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1659.51 208.785 L1659.51 224.433 L1655.25 224.433 L1655.25 208.924 Q1655.25 205.243 1653.82 203.415 Q1652.38 201.586 1649.51 201.586 Q1646.06 201.586 1644.07 203.785 Q1642.08 205.984 1642.08 209.78 L1642.08 224.433 L1637.8 224.433 L1637.8 198.507 L1642.08 198.507 L1642.08 202.535 Q1643.61 200.197 1645.67 199.04 Q1647.75 197.882 1650.46 197.882 Q1654.93 197.882 1657.22 200.66 Q1659.51 203.415 1659.51 208.785 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1685.07 202.443 L1685.07 188.415 L1689.33 188.415 L1689.33 224.433 L1685.07 224.433 L1685.07 220.544 Q1683.73 222.859 1681.67 223.993 Q1679.63 225.104 1676.76 225.104 Q1672.06 225.104 1669.1 221.355 Q1666.16 217.605 1666.16 211.493 Q1666.16 205.382 1669.1 201.632 Q1672.06 197.882 1676.76 197.882 Q1679.63 197.882 1681.67 199.017 Q1683.73 200.128 1685.07 202.443 M1670.55 211.493 Q1670.55 216.192 1672.48 218.878 Q1674.42 221.54 1677.8 221.54 Q1681.18 221.54 1683.12 218.878 Q1685.07 216.192 1685.07 211.493 Q1685.07 206.794 1683.12 204.132 Q1681.18 201.447 1677.8 201.447 Q1674.42 201.447 1672.48 204.132 Q1670.55 206.794 1670.55 211.493 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1709.88 211.401 Q1704.72 211.401 1702.73 212.581 Q1700.74 213.762 1700.74 216.609 Q1700.74 218.878 1702.22 220.22 Q1703.73 221.54 1706.29 221.54 Q1709.84 221.54 1711.97 219.04 Q1714.12 216.517 1714.12 212.35 L1714.12 211.401 L1709.88 211.401 M1718.38 209.642 L1718.38 224.433 L1714.12 224.433 L1714.12 220.498 Q1712.66 222.859 1710.48 223.993 Q1708.31 225.104 1705.16 225.104 Q1701.18 225.104 1698.82 222.882 Q1696.48 220.637 1696.48 216.887 Q1696.48 212.512 1699.4 210.29 Q1702.34 208.068 1708.15 208.068 L1714.12 208.068 L1714.12 207.651 Q1714.12 204.711 1712.17 203.114 Q1710.25 201.493 1706.76 201.493 Q1704.54 201.493 1702.43 202.026 Q1700.32 202.558 1698.38 203.623 L1698.38 199.688 Q1700.72 198.785 1702.92 198.345 Q1705.11 197.882 1707.2 197.882 Q1712.82 197.882 1715.6 200.799 Q1718.38 203.716 1718.38 209.642 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1742.17 202.489 Q1741.46 202.072 1740.6 201.887 Q1739.77 201.679 1738.75 201.679 Q1735.14 201.679 1733.19 204.04 Q1731.27 206.378 1731.27 210.776 L1731.27 224.433 L1726.99 224.433 L1726.99 198.507 L1731.27 198.507 L1731.27 202.535 Q1732.61 200.174 1734.77 199.04 Q1736.92 197.882 1740 197.882 Q1740.44 197.882 1740.97 197.952 Q1741.5 197.998 1742.15 198.114 L1742.17 202.489 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip820)" d="M1762.87 202.443 L1762.87 188.415 L1767.13 188.415 L1767.13 224.433 L1762.87 224.433 L1762.87 220.544 Q1761.53 222.859 1759.47 223.993 Q1757.43 225.104 1754.56 225.104 Q1749.86 225.104 1746.9 221.355 Q1743.96 217.605 1743.96 211.493 Q1743.96 205.382 1746.9 201.632 Q1749.86 197.882 1754.56 197.882 Q1757.43 197.882 1759.47 199.017 Q1761.53 200.128 1762.87 202.443 M1748.35 211.493 Q1748.35 216.192 1750.28 218.878 Q1752.22 221.54 1755.6 221.54 Q1758.98 221.54 1760.92 218.878 Q1762.87 216.192 1762.87 211.493 Q1762.87 206.794 1760.92 204.132 Q1758.98 201.447 1755.6 201.447 Q1752.22 201.447 1750.28 204.132 Q1748.35 206.794 1748.35 211.493 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /></svg>

Figure 1: Simulated dissolved oxygen concentration for Problem 1 without
treatment.

1.  These lines of code separate the creation of the plot axis (with
    labels, legend positions, etc) using `p = plot(...)` from the
    plotting of the data with `plot!(p, ...)`. You can do this all in a
    single `plot()` call, but this may sometimes make things more
    readable when there are a lot of style arguments for the axes.

We can see that the DO concentration falls below the regulatory standard
of 4 mg/L before 20 km downstream. To find the minimum value, we can use
the `minimum()` function.

``` julia
@show minimum(C);
```

    minimum(C) = 3.694312052808094

So the minimum value is 3.7 mg/L.

To determine the minimum treatment of the waste streams needed to
maintain compliance, we will write a function which will apply treatment
levels `eff1` and `eff2` and evaluate our function above, returning the
minimum DO concentration.

``` julia
# waste_treat: function to simulate DO with treated discharges; treatments apply to CBOD and NBOD released
# inputs:
#   - eff1: efficiency of treatment for waste stream 1 (as a decimal)
#   - eff2: efficiency of treatment for waste stream 2 (as a decimal)
#   - inflow: tuple with inflow properties: (Volume, DO, CBOD, NBOD)
#   - waste1: tuple with waste stream 1 properties: (Volume, DO, CBOD, NBOD)
#   - waste2: tuple with waste stream 2 properties: (Volume, DO, CBOD, NBOD)
#   - U: velocity of river (km/d)
#   - Cₛ = saturation oxygen concentration (mg/L)
#   - ka, kc, kn: reaeration, CBOD decay, and NBOD decay rates, respectively (d^{-1})
function waste_treat(eff1, eff2, inflow, waste1, waste2, U, Cₛ, ka, kc, kn)
    waste1_treated = (waste1[1], waste1[2], (1 - eff1) * waste1[3], (1 - eff1) * waste1[4])    
    waste2_treated = (waste2[1], waste2[2], (1 - eff2) * waste2[3], (1 - eff2) * waste2[4])    
    C, B, N = do_simulate(inflow, waste1_treated, waste2_treated, U, Cₛ, ka, kc, kn)
    return minimum(C)
end
```

    waste_treat (generic function with 1 method)

Now we can evaluate this function over a range of treatment
efficiencies. There are a number of ways to do this, but we’ll use a
trick which Julia makes easy: broadcasting using an anonymous function.

``` julia
# evaluate treatment efficiencies between 0 and 1
effs = 0:0.01:1
treat1 = (e1 -> waste_treat(e1, 0, inflow, waste1, waste2, U, Cₛ, ka, kc, kn)).(effs)
treat2 = (e2 -> waste_treat(0, e2, inflow, waste1, waste2, U, Cₛ, ka, kc, kn)).(effs)
```

    101-element Vector{Float64}:
     3.694312052808094
     3.715106569604958
     3.7359010864018223
     3.7566956031986853
     3.7774901199955497
     3.7982846367924123
     3.819079153589276
     3.8398736703861394
     3.8606681871830038
     3.8814627039798673
     ⋮
     4.711700098308851
     4.711700098308851
     4.711700098308851
     4.711700098308851
     4.711700098308851
     4.711700098308851
     4.711700098308851
     4.711700098308851
     4.711700098308851

To find the minimum treatment level which ensures compliance, we can now
find the position of the first value where the minimum value is at least
4 mg/L, and look up the associated efficiency. Julia provides the
`findfirst()` function which lets you find the index of the first value
satisfying a Boolean condition.

``` julia
# find indices and treatment values
idx1 = findfirst(treat1 .>= 4.0)
idx2 = findfirst(treat2 .>= 4.0)
@show effs[idx1];
@show effs[idx2];
```

    effs[idx1] = 0.19
    effs[idx2] = 0.15

So the minimum treatment level for waste stream 1 is 19% and the minimum
treatment level for waste stream 2 is 15%.

There is no “right” answer to the question of which treatment option you
would pick, so long as your solution is thoughtful and justified.

-   For example, one could argue that as waste stream 1 on its own does
    not result in a lack of compliance (which we can see from
    <a href="#fig-do-untreated" class="quarto-xref">Figure 1</a>, as the
    initial “sag” has started to recover prior to waste stream 2), waste
    stream 2 ought to be treated.
-   On the other hand, waste stream 1 has a much more negative impact on
    the inflow dissolved oxygen levels, and waste stream 2 might not
    cause a lack of compliance without that effect, which might suggest
    that waste stream 1 should be treated.
-   Another consideration might be the relative cost of treating each
    waste stream, which we have no information on, or whether these
    waste streams are from municipal, residual, or industrial sources
    (in other words, non-profit vs. profit).

### Problem 2 (20 points)

Loading the data:

``` julia
# Dataset from https://zenodo.org/record/3973015
# The CSV is read into a DataFrame object, and we specify that it is comma delimited
forcings_all = CSV.read("data/ERF_ssp585_1750-2500.csv", DataFrame, delim=",")

# Separate out the individual components
# Get total aerosol forcings
forcing_aerosol_rad = forcings_all[!,"aerosol-radiation_interactions"]
forcing_aerosol_cloud = forcings_all[!,"aerosol-cloud_interactions"]
forcing_aerosol = forcing_aerosol_rad + forcing_aerosol_cloud
# Calculate non-aerosol forcings from the total.
forcing_total = forcings_all[!,"total"]
forcing_non_aerosol = forcing_total - forcing_aerosol
t = Int64.(forcings_all[!,"year"])
idx_2100 = findfirst(t .== 2100) # find the index which corresponds to the year 2100
```

    351

The energy balance model is given by
$$cd \frac{dT}{dt} = F - \lambda T.$$ Discretizing with Forward Euler,
we get $$
\begin{aligned}
cd\frac{T(t + \Delta t) - T(t)}{\Delta t} &= F(t) - \lambda T(t) \\
T(t + \Delta t) &= T(t) + \frac{\Delta t}{cd} \left(F(t) - \lambda T(t)\right).
\end{aligned}
$$

In Julia code (and with the assumed parameter values), this becomes the
following function (replacing
$F(t) = F_\text{nonaerosol} + \alpha F_\text{aerosol}$):

``` julia
# ebm: function to simulate the energy balance model
# inputs:
#   - t: vector of time values.
#   - F_nonaerosol: vector of non-aerosol radiative forcing values
#   - F_aerosol: vector of aerosol radiative forcing values
#   - c, d, λ, α, Δt: model parameters, see below.
function ebm(t, F_nonaerosol, F_aerosol, c, d, λ, α, Δt)
    T = zeros(length(t))
    F = F_nonaerosol + α * F_aerosol
    for s in 1:length(t)-1
        T[s+1] = T[s] + Δt * (F[s] - λ * T[s]) / (c * d)
    end
    return T
end

c = 4.184e6 # specific heat of water, J/K/m^2
d = 86 # ocean mixing depth, m
λ = 2.1 # climate feedback factor
α = 0.8 # aerosol scaling factor
Δt = 31_558_152 # annual time step, s

temps = ebm(t, forcing_non_aerosol, forcing_aerosol, c, d, λ, α, Δt)
```

    751-element Vector{Float64}:
     0.0
     0.022731483349329427
     0.040278289494741326
     0.05240826578450054
     0.05895069636157351
     0.06063584224928685
     0.058278631700672184
     0.036311110702349894
     0.03060499874837628
     0.03778068117354878
     ⋮
     6.0349064449781515
     6.033928949865114
     6.032952263810847
     6.031978595303466
     6.0310066214578395
     6.030034177683138
     6.029061472805756
     6.028090703835366
     6.027123866872981

1.  The `zeros` function initializes a vector with zero values of the
    given length. This is a good way to initialize storage when you
    aren’t worried about distinguishing non-set values from “true”
    zeroes.
2.  While normally `eachindex(t)` is preferred instead of `1:length(t)`,
    in this case we need to iterate only until the second-to-last index
    of `t` as we set the next value `T[s+1]` in the loop.

The simulated temperatures are plotted in
<a href="#fig-temps" class="quarto-xref">Figure 2</a>:

``` julia
p = plot(; xlabel="Year", ylabel="Global Mean Temperature Anomaly (°C)", legend=false)
plot!(p, t, temps)
xlims!(p, (1750, 2100))
```

<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="672" height="480" viewBox="0 0 2688 1920">
<defs>
  <clipPath id="clip880">
    <rect x="0" y="0" width="2688" height="1920"/>
  </clipPath>
</defs>
<path clip-path="url(#clip880)" d="M0 1920 L2688 1920 L2688 -4.26326e-14 L0 -4.26326e-14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip881">
    <rect x="537" y="0" width="1883" height="1883"/>
  </clipPath>
</defs>
<path clip-path="url(#clip880)" d="M181.719 1734.12 L2640.76 1734.12 L2640.76 47.2441 L181.719 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip882">
    <rect x="181" y="47" width="2460" height="1688"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip882)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,1734.12 181.719,47.2441 "/>
<polyline clip-path="url(#clip882)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="533.01,1734.12 533.01,47.2441 "/>
<polyline clip-path="url(#clip882)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="884.301,1734.12 884.301,47.2441 "/>
<polyline clip-path="url(#clip882)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1235.59,1734.12 1235.59,47.2441 "/>
<polyline clip-path="url(#clip882)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1586.88,1734.12 1586.88,47.2441 "/>
<polyline clip-path="url(#clip882)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1938.17,1734.12 1938.17,47.2441 "/>
<polyline clip-path="url(#clip882)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2289.46,1734.12 2289.46,47.2441 "/>
<polyline clip-path="url(#clip882)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2640.76,1734.12 2640.76,47.2441 "/>
<polyline clip-path="url(#clip880)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip880)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1734.12 181.719,1715.22 "/>
<polyline clip-path="url(#clip880)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="533.01,1734.12 533.01,1715.22 "/>
<polyline clip-path="url(#clip880)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="884.301,1734.12 884.301,1715.22 "/>
<polyline clip-path="url(#clip880)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1235.59,1734.12 1235.59,1715.22 "/>
<polyline clip-path="url(#clip880)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1586.88,1734.12 1586.88,1715.22 "/>
<polyline clip-path="url(#clip880)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1938.17,1734.12 1938.17,1715.22 "/>
<polyline clip-path="url(#clip880)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2289.46,1734.12 2289.46,1715.22 "/>
<polyline clip-path="url(#clip880)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2640.76,1734.12 2640.76,1715.22 "/>
<path clip-path="url(#clip880)" d="M126.245 1794.89 L133.884 1794.89 L133.884 1768.52 L125.574 1770.19 L125.574 1765.93 L133.838 1764.26 L138.514 1764.26 L138.514 1794.89 L146.153 1794.89 L146.153 1798.82 L126.245 1798.82 L126.245 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M154.416 1764.26 L176.639 1764.26 L176.639 1766.25 L164.092 1798.82 L159.208 1798.82 L171.014 1768.2 L154.416 1768.2 L154.416 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M185.805 1764.26 L204.162 1764.26 L204.162 1768.2 L190.088 1768.2 L190.088 1776.67 Q191.106 1776.32 192.125 1776.16 Q193.143 1775.97 194.162 1775.97 Q199.949 1775.97 203.328 1779.15 Q206.708 1782.32 206.708 1787.73 Q206.708 1793.31 203.236 1796.41 Q199.763 1799.49 193.444 1799.49 Q191.268 1799.49 189 1799.12 Q186.754 1798.75 184.347 1798.01 L184.347 1793.31 Q186.43 1794.45 188.652 1795 Q190.875 1795.56 193.351 1795.56 Q197.356 1795.56 199.694 1793.45 Q202.032 1791.34 202.032 1787.73 Q202.032 1784.12 199.694 1782.02 Q197.356 1779.91 193.351 1779.91 Q191.476 1779.91 189.601 1780.33 Q187.75 1780.74 185.805 1781.62 L185.805 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M225.921 1767.34 Q222.31 1767.34 220.481 1770.9 Q218.675 1774.45 218.675 1781.58 Q218.675 1788.68 220.481 1792.25 Q222.31 1795.79 225.921 1795.79 Q229.555 1795.79 231.36 1792.25 Q233.189 1788.68 233.189 1781.58 Q233.189 1774.45 231.36 1770.9 Q229.555 1767.34 225.921 1767.34 M225.921 1763.64 Q231.731 1763.64 234.786 1768.24 Q237.865 1772.83 237.865 1781.58 Q237.865 1790.3 234.786 1794.91 Q231.731 1799.49 225.921 1799.49 Q220.11 1799.49 217.032 1794.91 Q213.976 1790.3 213.976 1781.58 Q213.976 1772.83 217.032 1768.24 Q220.11 1763.64 225.921 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M477.536 1794.89 L485.175 1794.89 L485.175 1768.52 L476.865 1770.19 L476.865 1765.93 L485.129 1764.26 L489.805 1764.26 L489.805 1794.89 L497.443 1794.89 L497.443 1798.82 L477.536 1798.82 L477.536 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M516.888 1782.41 Q513.554 1782.41 511.633 1784.19 Q509.735 1785.97 509.735 1789.1 Q509.735 1792.22 511.633 1794.01 Q513.554 1795.79 516.888 1795.79 Q520.221 1795.79 522.142 1794.01 Q524.064 1792.2 524.064 1789.1 Q524.064 1785.97 522.142 1784.19 Q520.244 1782.41 516.888 1782.41 M512.212 1780.42 Q509.203 1779.68 507.513 1777.62 Q505.846 1775.56 505.846 1772.59 Q505.846 1768.45 508.786 1766.04 Q511.749 1763.64 516.888 1763.64 Q522.05 1763.64 524.99 1766.04 Q527.929 1768.45 527.929 1772.59 Q527.929 1775.56 526.24 1777.62 Q524.573 1779.68 521.587 1780.42 Q524.966 1781.21 526.841 1783.5 Q528.74 1785.79 528.74 1789.1 Q528.74 1794.12 525.661 1796.81 Q522.605 1799.49 516.888 1799.49 Q511.17 1799.49 508.092 1796.81 Q505.036 1794.12 505.036 1789.1 Q505.036 1785.79 506.934 1783.5 Q508.832 1781.21 512.212 1780.42 M510.499 1773.03 Q510.499 1775.72 512.166 1777.22 Q513.855 1778.73 516.888 1778.73 Q519.897 1778.73 521.587 1777.22 Q523.3 1775.72 523.3 1773.03 Q523.3 1770.35 521.587 1768.84 Q519.897 1767.34 516.888 1767.34 Q513.855 1767.34 512.166 1768.84 Q510.499 1770.35 510.499 1773.03 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M547.05 1767.34 Q543.439 1767.34 541.61 1770.9 Q539.804 1774.45 539.804 1781.58 Q539.804 1788.68 541.61 1792.25 Q543.439 1795.79 547.05 1795.79 Q550.684 1795.79 552.489 1792.25 Q554.318 1788.68 554.318 1781.58 Q554.318 1774.45 552.489 1770.9 Q550.684 1767.34 547.05 1767.34 M547.05 1763.64 Q552.86 1763.64 555.915 1768.24 Q558.994 1772.83 558.994 1781.58 Q558.994 1790.3 555.915 1794.91 Q552.86 1799.49 547.05 1799.49 Q541.24 1799.49 538.161 1794.91 Q535.105 1790.3 535.105 1781.58 Q535.105 1772.83 538.161 1768.24 Q541.24 1763.64 547.05 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M577.212 1767.34 Q573.6 1767.34 571.772 1770.9 Q569.966 1774.45 569.966 1781.58 Q569.966 1788.68 571.772 1792.25 Q573.6 1795.79 577.212 1795.79 Q580.846 1795.79 582.651 1792.25 Q584.48 1788.68 584.48 1781.58 Q584.48 1774.45 582.651 1770.9 Q580.846 1767.34 577.212 1767.34 M577.212 1763.64 Q583.022 1763.64 586.077 1768.24 Q589.156 1772.83 589.156 1781.58 Q589.156 1790.3 586.077 1794.91 Q583.022 1799.49 577.212 1799.49 Q571.401 1799.49 568.323 1794.91 Q565.267 1790.3 565.267 1781.58 Q565.267 1772.83 568.323 1768.24 Q571.401 1763.64 577.212 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M828.827 1794.89 L836.466 1794.89 L836.466 1768.52 L828.156 1770.19 L828.156 1765.93 L836.42 1764.26 L841.096 1764.26 L841.096 1794.89 L848.734 1794.89 L848.734 1798.82 L828.827 1798.82 L828.827 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M868.179 1782.41 Q864.845 1782.41 862.924 1784.19 Q861.026 1785.97 861.026 1789.1 Q861.026 1792.22 862.924 1794.01 Q864.845 1795.79 868.179 1795.79 Q871.512 1795.79 873.433 1794.01 Q875.355 1792.2 875.355 1789.1 Q875.355 1785.97 873.433 1784.19 Q871.535 1782.41 868.179 1782.41 M863.503 1780.42 Q860.494 1779.68 858.804 1777.62 Q857.137 1775.56 857.137 1772.59 Q857.137 1768.45 860.077 1766.04 Q863.04 1763.64 868.179 1763.64 Q873.341 1763.64 876.281 1766.04 Q879.22 1768.45 879.22 1772.59 Q879.22 1775.56 877.531 1777.62 Q875.864 1779.68 872.878 1780.42 Q876.257 1781.21 878.132 1783.5 Q880.031 1785.79 880.031 1789.1 Q880.031 1794.12 876.952 1796.81 Q873.896 1799.49 868.179 1799.49 Q862.461 1799.49 859.382 1796.81 Q856.327 1794.12 856.327 1789.1 Q856.327 1785.79 858.225 1783.5 Q860.123 1781.21 863.503 1780.42 M861.79 1773.03 Q861.79 1775.72 863.457 1777.22 Q865.146 1778.73 868.179 1778.73 Q871.188 1778.73 872.878 1777.22 Q874.591 1775.72 874.591 1773.03 Q874.591 1770.35 872.878 1768.84 Q871.188 1767.34 868.179 1767.34 Q865.146 1767.34 863.457 1768.84 Q861.79 1770.35 861.79 1773.03 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M888.387 1764.26 L906.743 1764.26 L906.743 1768.2 L892.669 1768.2 L892.669 1776.67 Q893.688 1776.32 894.706 1776.16 Q895.725 1775.97 896.743 1775.97 Q902.53 1775.97 905.91 1779.15 Q909.29 1782.32 909.29 1787.73 Q909.29 1793.31 905.817 1796.41 Q902.345 1799.49 896.026 1799.49 Q893.85 1799.49 891.581 1799.12 Q889.336 1798.75 886.929 1798.01 L886.929 1793.31 Q889.012 1794.45 891.234 1795 Q893.456 1795.56 895.933 1795.56 Q899.938 1795.56 902.276 1793.45 Q904.614 1791.34 904.614 1787.73 Q904.614 1784.12 902.276 1782.02 Q899.938 1779.91 895.933 1779.91 Q894.058 1779.91 892.183 1780.33 Q890.331 1780.74 888.387 1781.62 L888.387 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M928.502 1767.34 Q924.891 1767.34 923.063 1770.9 Q921.257 1774.45 921.257 1781.58 Q921.257 1788.68 923.063 1792.25 Q924.891 1795.79 928.502 1795.79 Q932.137 1795.79 933.942 1792.25 Q935.771 1788.68 935.771 1781.58 Q935.771 1774.45 933.942 1770.9 Q932.137 1767.34 928.502 1767.34 M928.502 1763.64 Q934.313 1763.64 937.368 1768.24 Q940.447 1772.83 940.447 1781.58 Q940.447 1790.3 937.368 1794.91 Q934.313 1799.49 928.502 1799.49 Q922.692 1799.49 919.614 1794.91 Q916.558 1790.3 916.558 1781.58 Q916.558 1772.83 919.614 1768.24 Q922.692 1763.64 928.502 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M1180.12 1794.89 L1187.76 1794.89 L1187.76 1768.52 L1179.45 1770.19 L1179.45 1765.93 L1187.71 1764.26 L1192.39 1764.26 L1192.39 1794.89 L1200.03 1794.89 L1200.03 1798.82 L1180.12 1798.82 L1180.12 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M1209.61 1798.1 L1209.61 1793.84 Q1211.37 1794.68 1213.17 1795.12 Q1214.98 1795.56 1216.72 1795.56 Q1221.34 1795.56 1223.78 1792.46 Q1226.23 1789.33 1226.58 1782.99 Q1225.23 1784.98 1223.17 1786.04 Q1221.11 1787.11 1218.61 1787.11 Q1213.43 1787.11 1210.4 1783.98 Q1207.39 1780.83 1207.39 1775.4 Q1207.39 1770.07 1210.53 1766.85 Q1213.68 1763.64 1218.91 1763.64 Q1224.91 1763.64 1228.06 1768.24 Q1231.23 1772.83 1231.23 1781.58 Q1231.23 1789.75 1227.34 1794.63 Q1223.47 1799.49 1216.92 1799.49 Q1215.16 1799.49 1213.36 1799.15 Q1211.55 1798.8 1209.61 1798.1 M1218.91 1783.45 Q1222.06 1783.45 1223.89 1781.3 Q1225.74 1779.15 1225.74 1775.4 Q1225.74 1771.67 1223.89 1769.52 Q1222.06 1767.34 1218.91 1767.34 Q1215.77 1767.34 1213.91 1769.52 Q1212.09 1771.67 1212.09 1775.4 Q1212.09 1779.15 1213.91 1781.3 Q1215.77 1783.45 1218.91 1783.45 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M1249.63 1767.34 Q1246.02 1767.34 1244.19 1770.9 Q1242.39 1774.45 1242.39 1781.58 Q1242.39 1788.68 1244.19 1792.25 Q1246.02 1795.79 1249.63 1795.79 Q1253.27 1795.79 1255.07 1792.25 Q1256.9 1788.68 1256.9 1781.58 Q1256.9 1774.45 1255.07 1770.9 Q1253.27 1767.34 1249.63 1767.34 M1249.63 1763.64 Q1255.44 1763.64 1258.5 1768.24 Q1261.58 1772.83 1261.58 1781.58 Q1261.58 1790.3 1258.5 1794.91 Q1255.44 1799.49 1249.63 1799.49 Q1243.82 1799.49 1240.74 1794.91 Q1237.69 1790.3 1237.69 1781.58 Q1237.69 1772.83 1240.74 1768.24 Q1243.82 1763.64 1249.63 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M1279.79 1767.34 Q1276.18 1767.34 1274.35 1770.9 Q1272.55 1774.45 1272.55 1781.58 Q1272.55 1788.68 1274.35 1792.25 Q1276.18 1795.79 1279.79 1795.79 Q1283.43 1795.79 1285.23 1792.25 Q1287.06 1788.68 1287.06 1781.58 Q1287.06 1774.45 1285.23 1770.9 Q1283.43 1767.34 1279.79 1767.34 M1279.79 1763.64 Q1285.6 1763.64 1288.66 1768.24 Q1291.74 1772.83 1291.74 1781.58 Q1291.74 1790.3 1288.66 1794.91 Q1285.6 1799.49 1279.79 1799.49 Q1273.98 1799.49 1270.9 1794.91 Q1267.85 1790.3 1267.85 1781.58 Q1267.85 1772.83 1270.9 1768.24 Q1273.98 1763.64 1279.79 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M1531.41 1794.89 L1539.05 1794.89 L1539.05 1768.52 L1530.74 1770.19 L1530.74 1765.93 L1539 1764.26 L1543.68 1764.26 L1543.68 1794.89 L1551.32 1794.89 L1551.32 1798.82 L1531.41 1798.82 L1531.41 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M1560.9 1798.1 L1560.9 1793.84 Q1562.66 1794.68 1564.46 1795.12 Q1566.27 1795.56 1568.01 1795.56 Q1572.64 1795.56 1575.07 1792.46 Q1577.52 1789.33 1577.87 1782.99 Q1576.52 1784.98 1574.46 1786.04 Q1572.4 1787.11 1569.9 1787.11 Q1564.72 1787.11 1561.69 1783.98 Q1558.68 1780.83 1558.68 1775.4 Q1558.68 1770.07 1561.83 1766.85 Q1564.97 1763.64 1570.21 1763.64 Q1576.2 1763.64 1579.35 1768.24 Q1582.52 1772.83 1582.52 1781.58 Q1582.52 1789.75 1578.63 1794.63 Q1574.77 1799.49 1568.21 1799.49 Q1566.46 1799.49 1564.65 1799.15 Q1562.84 1798.8 1560.9 1798.1 M1570.21 1783.45 Q1573.35 1783.45 1575.18 1781.3 Q1577.03 1779.15 1577.03 1775.4 Q1577.03 1771.67 1575.18 1769.52 Q1573.35 1767.34 1570.21 1767.34 Q1567.06 1767.34 1565.21 1769.52 Q1563.38 1771.67 1563.38 1775.4 Q1563.38 1779.15 1565.21 1781.3 Q1567.06 1783.45 1570.21 1783.45 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M1590.97 1764.26 L1609.33 1764.26 L1609.33 1768.2 L1595.25 1768.2 L1595.25 1776.67 Q1596.27 1776.32 1597.29 1776.16 Q1598.31 1775.97 1599.33 1775.97 Q1605.11 1775.97 1608.49 1779.15 Q1611.87 1782.32 1611.87 1787.73 Q1611.87 1793.31 1608.4 1796.41 Q1604.93 1799.49 1598.61 1799.49 Q1596.43 1799.49 1594.16 1799.12 Q1591.92 1798.75 1589.51 1798.01 L1589.51 1793.31 Q1591.59 1794.45 1593.82 1795 Q1596.04 1795.56 1598.52 1795.56 Q1602.52 1795.56 1604.86 1793.45 Q1607.2 1791.34 1607.2 1787.73 Q1607.2 1784.12 1604.86 1782.02 Q1602.52 1779.91 1598.52 1779.91 Q1596.64 1779.91 1594.77 1780.33 Q1592.91 1780.74 1590.97 1781.62 L1590.97 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M1631.08 1767.34 Q1627.47 1767.34 1625.64 1770.9 Q1623.84 1774.45 1623.84 1781.58 Q1623.84 1788.68 1625.64 1792.25 Q1627.47 1795.79 1631.08 1795.79 Q1634.72 1795.79 1636.52 1792.25 Q1638.35 1788.68 1638.35 1781.58 Q1638.35 1774.45 1636.52 1770.9 Q1634.72 1767.34 1631.08 1767.34 M1631.08 1763.64 Q1636.89 1763.64 1639.95 1768.24 Q1643.03 1772.83 1643.03 1781.58 Q1643.03 1790.3 1639.95 1794.91 Q1636.89 1799.49 1631.08 1799.49 Q1625.27 1799.49 1622.2 1794.91 Q1619.14 1790.3 1619.14 1781.58 Q1619.14 1772.83 1622.2 1768.24 Q1625.27 1763.64 1631.08 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M1886.79 1794.89 L1903.1 1794.89 L1903.1 1798.82 L1881.16 1798.82 L1881.16 1794.89 Q1883.82 1792.13 1888.41 1787.5 Q1893.01 1782.85 1894.19 1781.51 Q1896.44 1778.98 1897.32 1777.25 Q1898.22 1775.49 1898.22 1773.8 Q1898.22 1771.04 1896.28 1769.31 Q1894.35 1767.57 1891.25 1767.57 Q1889.05 1767.57 1886.6 1768.34 Q1884.17 1769.1 1881.39 1770.65 L1881.39 1765.93 Q1884.22 1764.79 1886.67 1764.21 Q1889.12 1763.64 1891.16 1763.64 Q1896.53 1763.64 1899.73 1766.32 Q1902.92 1769.01 1902.92 1773.5 Q1902.92 1775.63 1902.11 1777.55 Q1901.32 1779.45 1899.22 1782.04 Q1898.64 1782.71 1895.54 1785.93 Q1892.43 1789.12 1886.79 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M1922.92 1767.34 Q1919.31 1767.34 1917.48 1770.9 Q1915.67 1774.45 1915.67 1781.58 Q1915.67 1788.68 1917.48 1792.25 Q1919.31 1795.79 1922.92 1795.79 Q1926.55 1795.79 1928.36 1792.25 Q1930.19 1788.68 1930.19 1781.58 Q1930.19 1774.45 1928.36 1770.9 Q1926.55 1767.34 1922.92 1767.34 M1922.92 1763.64 Q1928.73 1763.64 1931.79 1768.24 Q1934.86 1772.83 1934.86 1781.58 Q1934.86 1790.3 1931.79 1794.91 Q1928.73 1799.49 1922.92 1799.49 Q1917.11 1799.49 1914.03 1794.91 Q1910.98 1790.3 1910.98 1781.58 Q1910.98 1772.83 1914.03 1768.24 Q1917.11 1763.64 1922.92 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M1953.08 1767.34 Q1949.47 1767.34 1947.64 1770.9 Q1945.84 1774.45 1945.84 1781.58 Q1945.84 1788.68 1947.64 1792.25 Q1949.47 1795.79 1953.08 1795.79 Q1956.72 1795.79 1958.52 1792.25 Q1960.35 1788.68 1960.35 1781.58 Q1960.35 1774.45 1958.52 1770.9 Q1956.72 1767.34 1953.08 1767.34 M1953.08 1763.64 Q1958.89 1763.64 1961.95 1768.24 Q1965.03 1772.83 1965.03 1781.58 Q1965.03 1790.3 1961.95 1794.91 Q1958.89 1799.49 1953.08 1799.49 Q1947.27 1799.49 1944.19 1794.91 Q1941.14 1790.3 1941.14 1781.58 Q1941.14 1772.83 1944.19 1768.24 Q1947.27 1763.64 1953.08 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M1983.24 1767.34 Q1979.63 1767.34 1977.8 1770.9 Q1976 1774.45 1976 1781.58 Q1976 1788.68 1977.8 1792.25 Q1979.63 1795.79 1983.24 1795.79 Q1986.88 1795.79 1988.68 1792.25 Q1990.51 1788.68 1990.51 1781.58 Q1990.51 1774.45 1988.68 1770.9 Q1986.88 1767.34 1983.24 1767.34 M1983.24 1763.64 Q1989.05 1763.64 1992.11 1768.24 Q1995.19 1772.83 1995.19 1781.58 Q1995.19 1790.3 1992.11 1794.91 Q1989.05 1799.49 1983.24 1799.49 Q1977.43 1799.49 1974.35 1794.91 Q1971.3 1790.3 1971.3 1781.58 Q1971.3 1772.83 1974.35 1768.24 Q1977.43 1763.64 1983.24 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M2238.08 1794.89 L2254.4 1794.89 L2254.4 1798.82 L2232.45 1798.82 L2232.45 1794.89 Q2235.11 1792.13 2239.7 1787.5 Q2244.3 1782.85 2245.48 1781.51 Q2247.73 1778.98 2248.61 1777.25 Q2249.51 1775.49 2249.51 1773.8 Q2249.51 1771.04 2247.57 1769.31 Q2245.65 1767.57 2242.54 1767.57 Q2240.34 1767.57 2237.89 1768.34 Q2235.46 1769.1 2232.68 1770.65 L2232.68 1765.93 Q2235.51 1764.79 2237.96 1764.21 Q2240.41 1763.64 2242.45 1763.64 Q2247.82 1763.64 2251.02 1766.32 Q2254.21 1769.01 2254.21 1773.5 Q2254.21 1775.63 2253.4 1777.55 Q2252.61 1779.45 2250.51 1782.04 Q2249.93 1782.71 2246.83 1785.93 Q2243.72 1789.12 2238.08 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M2274.21 1767.34 Q2270.6 1767.34 2268.77 1770.9 Q2266.97 1774.45 2266.97 1781.58 Q2266.97 1788.68 2268.77 1792.25 Q2270.6 1795.79 2274.21 1795.79 Q2277.84 1795.79 2279.65 1792.25 Q2281.48 1788.68 2281.48 1781.58 Q2281.48 1774.45 2279.65 1770.9 Q2277.84 1767.34 2274.21 1767.34 M2274.21 1763.64 Q2280.02 1763.64 2283.08 1768.24 Q2286.15 1772.83 2286.15 1781.58 Q2286.15 1790.3 2283.08 1794.91 Q2280.02 1799.49 2274.21 1799.49 Q2268.4 1799.49 2265.32 1794.91 Q2262.27 1790.3 2262.27 1781.58 Q2262.27 1772.83 2265.32 1768.24 Q2268.4 1763.64 2274.21 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M2294.42 1764.26 L2312.78 1764.26 L2312.78 1768.2 L2298.7 1768.2 L2298.7 1776.67 Q2299.72 1776.32 2300.74 1776.16 Q2301.76 1775.97 2302.78 1775.97 Q2308.56 1775.97 2311.94 1779.15 Q2315.32 1782.32 2315.32 1787.73 Q2315.32 1793.31 2311.85 1796.41 Q2308.38 1799.49 2302.06 1799.49 Q2299.88 1799.49 2297.61 1799.12 Q2295.37 1798.75 2292.96 1798.01 L2292.96 1793.31 Q2295.04 1794.45 2297.27 1795 Q2299.49 1795.56 2301.96 1795.56 Q2305.97 1795.56 2308.31 1793.45 Q2310.65 1791.34 2310.65 1787.73 Q2310.65 1784.12 2308.31 1782.02 Q2305.97 1779.91 2301.96 1779.91 Q2300.09 1779.91 2298.21 1780.33 Q2296.36 1780.74 2294.42 1781.62 L2294.42 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M2334.53 1767.34 Q2330.92 1767.34 2329.09 1770.9 Q2327.29 1774.45 2327.29 1781.58 Q2327.29 1788.68 2329.09 1792.25 Q2330.92 1795.79 2334.53 1795.79 Q2338.17 1795.79 2339.97 1792.25 Q2341.8 1788.68 2341.8 1781.58 Q2341.8 1774.45 2339.97 1770.9 Q2338.17 1767.34 2334.53 1767.34 M2334.53 1763.64 Q2340.34 1763.64 2343.4 1768.24 Q2346.48 1772.83 2346.48 1781.58 Q2346.48 1790.3 2343.4 1794.91 Q2340.34 1799.49 2334.53 1799.49 Q2328.72 1799.49 2325.65 1794.91 Q2322.59 1790.3 2322.59 1781.58 Q2322.59 1772.83 2325.65 1768.24 Q2328.72 1763.64 2334.53 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M2589.37 1794.89 L2605.69 1794.89 L2605.69 1798.82 L2583.74 1798.82 L2583.74 1794.89 Q2586.4 1792.13 2590.99 1787.5 Q2595.59 1782.85 2596.77 1781.51 Q2599.02 1778.98 2599.9 1777.25 Q2600.8 1775.49 2600.8 1773.8 Q2600.8 1771.04 2598.86 1769.31 Q2596.94 1767.57 2593.83 1767.57 Q2591.64 1767.57 2589.18 1768.34 Q2586.75 1769.1 2583.97 1770.65 L2583.97 1765.93 Q2586.8 1764.79 2589.25 1764.21 Q2591.71 1763.64 2593.74 1763.64 Q2599.11 1763.64 2602.31 1766.32 Q2605.5 1769.01 2605.5 1773.5 Q2605.5 1775.63 2604.69 1777.55 Q2603.9 1779.45 2601.8 1782.04 Q2601.22 1782.71 2598.12 1785.93 Q2595.02 1789.12 2589.37 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M2616.31 1794.89 L2623.95 1794.89 L2623.95 1768.52 L2615.64 1770.19 L2615.64 1765.93 L2623.9 1764.26 L2628.58 1764.26 L2628.58 1794.89 L2636.22 1794.89 L2636.22 1798.82 L2616.31 1798.82 L2616.31 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M2655.66 1767.34 Q2652.05 1767.34 2650.22 1770.9 Q2648.42 1774.45 2648.42 1781.58 Q2648.42 1788.68 2650.22 1792.25 Q2652.05 1795.79 2655.66 1795.79 Q2659.3 1795.79 2661.1 1792.25 Q2662.93 1788.68 2662.93 1781.58 Q2662.93 1774.45 2661.1 1770.9 Q2659.3 1767.34 2655.66 1767.34 M2655.66 1763.64 Q2661.47 1763.64 2664.53 1768.24 Q2667.61 1772.83 2667.61 1781.58 Q2667.61 1790.3 2664.53 1794.91 Q2661.47 1799.49 2655.66 1799.49 Q2649.85 1799.49 2646.77 1794.91 Q2643.72 1790.3 2643.72 1781.58 Q2643.72 1772.83 2646.77 1768.24 Q2649.85 1763.64 2655.66 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M2685.83 1767.34 Q2682.21 1767.34 2680.39 1770.9 Q2678.58 1774.45 2678.58 1781.58 Q2678.58 1788.68 2680.39 1792.25 Q2682.21 1795.79 2685.83 1795.79 Q2689.46 1795.79 2691.26 1792.25 Q2693.09 1788.68 2693.09 1781.58 Q2693.09 1774.45 2691.26 1770.9 Q2689.46 1767.34 2685.83 1767.34 M2685.83 1763.64 Q2691.64 1763.64 2694.69 1768.24 Q2697.77 1772.83 2697.77 1781.58 Q2697.77 1790.3 2694.69 1794.91 Q2691.64 1799.49 2685.83 1799.49 Q2680.01 1799.49 2676.94 1794.91 Q2673.88 1790.3 2673.88 1781.58 Q2673.88 1772.83 2676.94 1768.24 Q2680.01 1763.64 2685.83 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M1342.17 1835.78 L1349.08 1835.78 L1362.25 1855.32 L1375.34 1835.78 L1382.24 1835.78 L1365.44 1860.67 L1365.44 1883.3 L1358.98 1883.3 L1358.98 1860.67 L1342.17 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M1410.09 1864.01 L1410.09 1866.88 L1383.16 1866.88 Q1383.55 1872.92 1386.79 1876.11 Q1390.07 1879.26 1395.9 1879.26 Q1399.27 1879.26 1402.42 1878.43 Q1405.6 1877.6 1408.72 1875.95 L1408.72 1881.49 Q1405.57 1882.82 1402.26 1883.52 Q1398.95 1884.22 1395.55 1884.22 Q1387.02 1884.22 1382.02 1879.26 Q1377.05 1874.29 1377.05 1865.83 Q1377.05 1857.07 1381.76 1851.95 Q1386.51 1846.79 1394.53 1846.79 Q1401.72 1846.79 1405.89 1851.44 Q1410.09 1856.06 1410.09 1864.01 M1404.24 1862.29 Q1404.17 1857.49 1401.53 1854.62 Q1398.92 1851.76 1394.59 1851.76 Q1389.69 1851.76 1386.73 1854.53 Q1383.8 1857.3 1383.36 1862.33 L1404.24 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M1435.9 1865.38 Q1428.81 1865.38 1426.07 1867 Q1423.33 1868.63 1423.33 1872.54 Q1423.33 1875.66 1425.37 1877.51 Q1427.44 1879.32 1430.97 1879.32 Q1435.84 1879.32 1438.77 1875.88 Q1441.73 1872.42 1441.73 1866.69 L1441.73 1865.38 L1435.9 1865.38 M1447.59 1862.96 L1447.59 1883.3 L1441.73 1883.3 L1441.73 1877.89 Q1439.72 1881.14 1436.73 1882.7 Q1433.74 1884.22 1429.41 1884.22 Q1423.94 1884.22 1420.69 1881.17 Q1417.48 1878.08 1417.48 1872.92 Q1417.48 1866.91 1421.49 1863.85 Q1425.53 1860.8 1433.52 1860.8 L1441.73 1860.8 L1441.73 1860.23 Q1441.73 1856.18 1439.06 1853.99 Q1436.41 1851.76 1431.61 1851.76 Q1428.55 1851.76 1425.66 1852.49 Q1422.76 1853.22 1420.09 1854.69 L1420.09 1849.28 Q1423.3 1848.03 1426.32 1847.43 Q1429.35 1846.79 1432.21 1846.79 Q1439.95 1846.79 1443.77 1850.8 Q1447.59 1854.81 1447.59 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M1480.31 1853.13 Q1479.32 1852.55 1478.14 1852.3 Q1477 1852.01 1475.59 1852.01 Q1470.63 1852.01 1467.96 1855.26 Q1465.31 1858.47 1465.31 1864.52 L1465.31 1883.3 L1459.43 1883.3 L1459.43 1847.65 L1465.31 1847.65 L1465.31 1853.19 Q1467.16 1849.94 1470.12 1848.39 Q1473.08 1846.79 1477.31 1846.79 Q1477.92 1846.79 1478.65 1846.89 Q1479.38 1846.95 1480.27 1847.11 L1480.31 1853.13 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip882)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,1525.16 2640.76,1525.16 "/>
<polyline clip-path="url(#clip882)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,1302.48 2640.76,1302.48 "/>
<polyline clip-path="url(#clip882)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,1079.8 2640.76,1079.8 "/>
<polyline clip-path="url(#clip882)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,857.114 2640.76,857.114 "/>
<polyline clip-path="url(#clip882)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,634.431 2640.76,634.431 "/>
<polyline clip-path="url(#clip882)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,411.748 2640.76,411.748 "/>
<polyline clip-path="url(#clip882)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,189.065 2640.76,189.065 "/>
<polyline clip-path="url(#clip880)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1734.12 181.719,47.2441 "/>
<polyline clip-path="url(#clip880)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1525.16 200.617,1525.16 "/>
<polyline clip-path="url(#clip880)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1302.48 200.617,1302.48 "/>
<polyline clip-path="url(#clip880)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1079.8 200.617,1079.8 "/>
<polyline clip-path="url(#clip880)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,857.114 200.617,857.114 "/>
<polyline clip-path="url(#clip880)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,634.431 200.617,634.431 "/>
<polyline clip-path="url(#clip880)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,411.748 200.617,411.748 "/>
<polyline clip-path="url(#clip880)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,189.065 200.617,189.065 "/>
<path clip-path="url(#clip880)" d="M129.455 1510.96 Q125.844 1510.96 124.015 1514.53 Q122.21 1518.07 122.21 1525.2 Q122.21 1532.3 124.015 1535.87 Q125.844 1539.41 129.455 1539.41 Q133.089 1539.41 134.895 1535.87 Q136.724 1532.3 136.724 1525.2 Q136.724 1518.07 134.895 1514.53 Q133.089 1510.96 129.455 1510.96 M129.455 1507.26 Q135.265 1507.26 138.321 1511.86 Q141.399 1516.45 141.399 1525.2 Q141.399 1533.92 138.321 1538.53 Q135.265 1543.11 129.455 1543.11 Q123.645 1543.11 120.566 1538.53 Q117.511 1533.92 117.511 1525.2 Q117.511 1516.45 120.566 1511.86 Q123.645 1507.26 129.455 1507.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M121.492 1315.82 L129.131 1315.82 L129.131 1289.46 L120.821 1291.13 L120.821 1286.87 L129.085 1285.2 L133.761 1285.2 L133.761 1315.82 L141.399 1315.82 L141.399 1319.76 L121.492 1319.76 L121.492 1315.82 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M125.08 1093.14 L141.399 1093.14 L141.399 1097.08 L119.455 1097.08 L119.455 1093.14 Q122.117 1090.39 126.7 1085.76 Q131.307 1081.1 132.488 1079.76 Q134.733 1077.24 135.612 1075.5 Q136.515 1073.74 136.515 1072.05 Q136.515 1069.3 134.571 1067.56 Q132.65 1065.83 129.548 1065.83 Q127.349 1065.83 124.895 1066.59 Q122.464 1067.36 119.687 1068.91 L119.687 1064.18 Q122.511 1063.05 124.964 1062.47 Q127.418 1061.89 129.455 1061.89 Q134.825 1061.89 138.02 1064.58 Q141.214 1067.26 141.214 1071.75 Q141.214 1073.88 140.404 1075.8 Q139.617 1077.7 137.511 1080.29 Q136.932 1080.97 133.83 1084.18 Q130.728 1087.38 125.08 1093.14 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M134.27 855.76 Q137.626 856.478 139.501 858.746 Q141.399 861.015 141.399 864.348 Q141.399 869.464 137.881 872.265 Q134.362 875.065 127.881 875.065 Q125.705 875.065 123.39 874.626 Q121.099 874.209 118.645 873.352 L118.645 868.839 Q120.589 869.973 122.904 870.552 Q125.219 871.13 127.742 871.13 Q132.14 871.13 134.432 869.394 Q136.747 867.658 136.747 864.348 Q136.747 861.292 134.594 859.579 Q132.464 857.843 128.645 857.843 L124.617 857.843 L124.617 854.001 L128.83 854.001 Q132.279 854.001 134.108 852.635 Q135.937 851.246 135.937 848.654 Q135.937 845.992 134.038 844.579 Q132.163 843.144 128.645 843.144 Q126.724 843.144 124.525 843.561 Q122.326 843.978 119.687 844.857 L119.687 840.691 Q122.349 839.95 124.663 839.58 Q127.001 839.209 129.062 839.209 Q134.386 839.209 137.487 841.64 Q140.589 844.047 140.589 848.167 Q140.589 851.038 138.946 853.029 Q137.302 854.996 134.27 855.76 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M131.816 621.225 L120.011 639.674 L131.816 639.674 L131.816 621.225 M130.589 617.151 L136.469 617.151 L136.469 639.674 L141.399 639.674 L141.399 643.563 L136.469 643.563 L136.469 651.711 L131.816 651.711 L131.816 643.563 L116.214 643.563 L116.214 639.049 L130.589 617.151 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M120.497 394.468 L138.853 394.468 L138.853 398.403 L124.779 398.403 L124.779 406.876 Q125.798 406.528 126.816 406.366 Q127.835 406.181 128.853 406.181 Q134.64 406.181 138.02 409.353 Q141.399 412.524 141.399 417.94 Q141.399 423.519 137.927 426.621 Q134.455 429.7 128.136 429.7 Q125.96 429.7 123.691 429.329 Q121.446 428.959 119.039 428.218 L119.039 423.519 Q121.122 424.653 123.344 425.209 Q125.566 425.764 128.043 425.764 Q132.048 425.764 134.386 423.658 Q136.724 421.552 136.724 417.94 Q136.724 414.329 134.386 412.223 Q132.048 410.116 128.043 410.116 Q126.168 410.116 124.293 410.533 Q122.441 410.95 120.497 411.829 L120.497 394.468 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M129.872 187.202 Q126.724 187.202 124.872 189.355 Q123.043 191.508 123.043 195.258 Q123.043 198.984 124.872 201.16 Q126.724 203.313 129.872 203.313 Q133.02 203.313 134.849 201.16 Q136.7 198.984 136.7 195.258 Q136.7 191.508 134.849 189.355 Q133.02 187.202 129.872 187.202 M139.154 172.549 L139.154 176.809 Q137.395 175.975 135.589 175.535 Q133.807 175.096 132.048 175.096 Q127.418 175.096 124.964 178.221 Q122.534 181.346 122.187 187.665 Q123.552 185.651 125.613 184.586 Q127.673 183.498 130.15 183.498 Q135.358 183.498 138.367 186.67 Q141.399 189.818 141.399 195.258 Q141.399 200.582 138.251 203.799 Q135.103 207.017 129.872 207.017 Q123.876 207.017 120.705 202.433 Q117.534 197.827 117.534 189.1 Q117.534 180.906 121.423 176.045 Q125.312 171.16 131.863 171.16 Q133.622 171.16 135.404 171.508 Q137.21 171.855 139.154 172.549 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M54.859 1499.02 L42.0957 1499.02 L42.0957 1509.52 L36.8122 1509.52 L36.8122 1492.65 L57.2143 1492.65 Q59.856 1496.38 61.2247 1500.87 Q62.5615 1505.35 62.5615 1510.45 Q62.5615 1521.59 56.0684 1527.89 Q49.5436 1534.16 37.9262 1534.16 Q26.2769 1534.16 19.7839 1527.89 Q13.2591 1521.59 13.2591 1510.45 Q13.2591 1505.8 14.4049 1501.63 Q15.5507 1497.43 17.7787 1493.89 L24.6219 1493.89 Q21.5981 1497.46 20.0704 1501.47 Q18.5426 1505.48 18.5426 1509.9 Q18.5426 1518.63 23.4124 1523.02 Q28.2821 1527.38 37.9262 1527.38 Q47.5384 1527.38 52.4082 1523.02 Q57.2779 1518.63 57.2779 1509.9 Q57.2779 1506.5 56.705 1503.83 Q56.1003 1501.15 54.859 1499.02 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M12.1132 1481.16 L12.1132 1475.31 L61.6384 1475.31 L61.6384 1481.16 L12.1132 1481.16 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M30.0964 1449.24 Q30.0964 1453.95 33.7885 1456.69 Q37.4488 1459.42 43.8463 1459.42 Q50.2438 1459.42 53.9359 1456.72 Q57.5962 1453.98 57.5962 1449.24 Q57.5962 1444.56 53.9041 1441.82 Q50.212 1439.09 43.8463 1439.09 Q37.5124 1439.09 33.8203 1441.82 Q30.0964 1444.56 30.0964 1449.24 M25.1311 1449.24 Q25.1311 1441.6 30.0964 1437.24 Q35.0616 1432.88 43.8463 1432.88 Q52.5991 1432.88 57.5962 1437.24 Q62.5615 1441.6 62.5615 1449.24 Q62.5615 1456.91 57.5962 1461.27 Q52.5991 1465.6 43.8463 1465.6 Q35.0616 1465.6 30.0964 1461.27 Q25.1311 1456.91 25.1311 1449.24 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M43.8463 1397.58 Q37.3851 1397.58 33.7248 1400.26 Q30.0327 1402.9 30.0327 1407.54 Q30.0327 1412.19 33.7248 1414.86 Q37.3851 1417.51 43.8463 1417.51 Q50.3075 1417.51 53.9996 1414.86 Q57.6599 1412.19 57.6599 1407.54 Q57.6599 1402.9 53.9996 1400.26 Q50.3075 1397.58 43.8463 1397.58 M31.4013 1417.51 Q28.2185 1415.66 26.6907 1412.86 Q25.1311 1410.03 25.1311 1406.11 Q25.1311 1399.62 30.2873 1395.58 Q35.4436 1391.5 43.8463 1391.5 Q52.249 1391.5 57.4052 1395.58 Q62.5615 1399.62 62.5615 1406.11 Q62.5615 1410.03 61.0337 1412.86 Q59.4741 1415.66 56.2912 1417.51 L61.6384 1417.51 L61.6384 1423.39 L12.1132 1423.39 L12.1132 1417.51 L31.4013 1417.51 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M43.719 1365.59 Q43.719 1372.69 45.3422 1375.43 Q46.9655 1378.17 50.8804 1378.17 Q53.9996 1378.17 55.8457 1376.13 Q57.6599 1374.06 57.6599 1370.53 Q57.6599 1365.66 54.2224 1362.73 Q50.7531 1359.77 45.0239 1359.77 L43.719 1359.77 L43.719 1365.59 M41.3 1353.91 L61.6384 1353.91 L61.6384 1359.77 L56.2276 1359.77 Q59.4741 1361.77 61.0337 1364.77 Q62.5615 1367.76 62.5615 1372.09 Q62.5615 1377.56 59.5059 1380.81 Q56.4186 1384.02 51.2623 1384.02 Q45.2467 1384.02 42.1912 1380.01 Q39.1357 1375.97 39.1357 1367.98 L39.1357 1359.77 L38.5628 1359.77 Q34.5205 1359.77 32.3244 1362.44 Q30.0964 1365.08 30.0964 1369.89 Q30.0964 1372.95 30.8284 1375.84 Q31.5605 1378.74 33.0246 1381.41 L27.6137 1381.41 Q26.3724 1378.2 25.7677 1375.17 Q25.1311 1372.15 25.1311 1369.29 Q25.1311 1361.55 29.1415 1357.73 Q33.1519 1353.91 41.3 1353.91 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M12.1132 1341.85 L12.1132 1335.99 L61.6384 1335.99 L61.6384 1341.85 L12.1132 1341.85 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M14.1184 1302.76 L14.1184 1293.18 L46.4562 1281.06 L14.1184 1268.87 L14.1184 1259.29 L61.6384 1259.29 L61.6384 1265.56 L19.9112 1265.56 L52.5037 1277.81 L52.5037 1284.27 L19.9112 1296.53 L61.6384 1296.53 L61.6384 1302.76 L14.1184 1302.76 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M42.3504 1216.29 L45.2149 1216.29 L45.2149 1243.21 Q51.2623 1242.83 54.4452 1239.58 Q57.5962 1236.31 57.5962 1230.48 Q57.5962 1227.11 56.7687 1223.96 Q55.9411 1220.77 54.2861 1217.66 L59.8242 1217.66 Q61.161 1220.81 61.8612 1224.12 Q62.5615 1227.43 62.5615 1230.83 Q62.5615 1239.36 57.5962 1244.36 Q52.631 1249.32 44.1646 1249.32 Q35.4117 1249.32 30.2873 1244.61 Q25.1311 1239.87 25.1311 1231.85 Q25.1311 1224.66 29.7781 1220.49 Q34.3932 1216.29 42.3504 1216.29 M40.6316 1222.14 Q35.8255 1222.21 32.9609 1224.85 Q30.0964 1227.46 30.0964 1231.79 Q30.0964 1236.69 32.8654 1239.65 Q35.6345 1242.58 40.6634 1243.02 L40.6316 1222.14 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M43.719 1190.47 Q43.719 1197.57 45.3422 1200.31 Q46.9655 1203.05 50.8804 1203.05 Q53.9996 1203.05 55.8457 1201.01 Q57.6599 1198.94 57.6599 1195.41 Q57.6599 1190.54 54.2224 1187.61 Q50.7531 1184.65 45.0239 1184.65 L43.719 1184.65 L43.719 1190.47 M41.3 1178.79 L61.6384 1178.79 L61.6384 1184.65 L56.2276 1184.65 Q59.4741 1186.65 61.0337 1189.65 Q62.5615 1192.64 62.5615 1196.97 Q62.5615 1202.44 59.5059 1205.69 Q56.4186 1208.9 51.2623 1208.9 Q45.2467 1208.9 42.1912 1204.89 Q39.1357 1200.85 39.1357 1192.86 L39.1357 1184.65 L38.5628 1184.65 Q34.5205 1184.65 32.3244 1187.32 Q30.0964 1189.96 30.0964 1194.77 Q30.0964 1197.83 30.8284 1200.72 Q31.5605 1203.62 33.0246 1206.29 L27.6137 1206.29 Q26.3724 1203.08 25.7677 1200.05 Q25.1311 1197.03 25.1311 1194.17 Q25.1311 1186.43 29.1415 1182.61 Q33.1519 1178.79 41.3 1178.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M40.1224 1137.1 L61.6384 1137.1 L61.6384 1142.95 L40.3133 1142.95 Q35.2526 1142.95 32.7381 1144.93 Q30.2237 1146.9 30.2237 1150.85 Q30.2237 1155.59 33.2474 1158.33 Q36.2711 1161.06 41.491 1161.06 L61.6384 1161.06 L61.6384 1166.95 L25.9905 1166.95 L25.9905 1161.06 L31.5287 1161.06 Q28.314 1158.96 26.7225 1156.13 Q25.1311 1153.27 25.1311 1149.54 Q25.1311 1143.4 28.9505 1140.25 Q32.7381 1137.1 40.1224 1137.1 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M14.1184 1111.03 L14.1184 1070.83 L19.5293 1070.83 L19.5293 1087.7 L61.6384 1087.7 L61.6384 1094.16 L19.5293 1094.16 L19.5293 1111.03 L14.1184 1111.03 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M42.3504 1045.46 L45.2149 1045.46 L45.2149 1072.39 Q51.2623 1072.01 54.4452 1068.76 Q57.5962 1065.48 57.5962 1059.66 Q57.5962 1056.28 56.7687 1053.13 Q55.9411 1049.95 54.2861 1046.83 L59.8242 1046.83 Q61.161 1049.98 61.8612 1053.29 Q62.5615 1056.6 62.5615 1060.01 Q62.5615 1068.54 57.5962 1073.54 Q52.631 1078.5 44.1646 1078.5 Q35.4117 1078.5 30.2873 1073.79 Q25.1311 1069.05 25.1311 1061.03 Q25.1311 1053.83 29.7781 1049.66 Q34.3932 1045.46 42.3504 1045.46 M40.6316 1051.32 Q35.8255 1051.38 32.9609 1054.02 Q30.0964 1056.63 30.0964 1060.96 Q30.0964 1065.86 32.8654 1068.82 Q35.6345 1071.75 40.6634 1072.2 L40.6316 1051.32 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M32.8336 1008.1 Q28.8869 1005.9 27.009 1002.84 Q25.1311 999.789 25.1311 995.651 Q25.1311 990.081 29.046 987.057 Q32.9291 984.034 40.1224 984.034 L61.6384 984.034 L61.6384 989.922 L40.3133 989.922 Q35.1889 989.922 32.7063 991.736 Q30.2237 993.55 30.2237 997.274 Q30.2237 1001.83 33.2474 1004.47 Q36.2711 1007.11 41.491 1007.11 L61.6384 1007.11 L61.6384 1013 L40.3133 1013 Q35.1571 1013 32.7063 1014.81 Q30.2237 1016.63 30.2237 1020.41 Q30.2237 1024.9 33.2792 1027.54 Q36.3029 1030.19 41.491 1030.19 L61.6384 1030.19 L61.6384 1036.07 L25.9905 1036.07 L25.9905 1030.19 L31.5287 1030.19 Q28.2503 1028.18 26.6907 1025.38 Q25.1311 1022.58 25.1311 1018.73 Q25.1311 1014.84 27.1045 1012.14 Q29.0779 1009.4 32.8336 1008.1 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M56.2912 966.687 L75.1974 966.687 L75.1974 972.575 L25.9905 972.575 L25.9905 966.687 L31.4013 966.687 Q28.2185 964.841 26.6907 962.04 Q25.1311 959.207 25.1311 955.292 Q25.1311 948.799 30.2873 944.757 Q35.4436 940.683 43.8463 940.683 Q52.249 940.683 57.4052 944.757 Q62.5615 948.799 62.5615 955.292 Q62.5615 959.207 61.0337 962.04 Q59.4741 964.841 56.2912 966.687 M43.8463 946.762 Q37.3851 946.762 33.7248 949.436 Q30.0327 952.078 30.0327 956.725 Q30.0327 961.372 33.7248 964.045 Q37.3851 966.687 43.8463 966.687 Q50.3075 966.687 53.9996 964.045 Q57.6599 961.372 57.6599 956.725 Q57.6599 952.078 53.9996 949.436 Q50.3075 946.762 43.8463 946.762 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M42.3504 900.484 L45.2149 900.484 L45.2149 927.411 Q51.2623 927.029 54.4452 923.782 Q57.5962 920.504 57.5962 914.679 Q57.5962 911.305 56.7687 908.154 Q55.9411 904.972 54.2861 901.852 L59.8242 901.852 Q61.161 905.003 61.8612 908.314 Q62.5615 911.624 62.5615 915.029 Q62.5615 923.559 57.5962 928.556 Q52.631 933.522 44.1646 933.522 Q35.4117 933.522 30.2873 928.811 Q25.1311 924.069 25.1311 916.048 Q25.1311 908.855 29.7781 904.685 Q34.3932 900.484 42.3504 900.484 M40.6316 906.34 Q35.8255 906.404 32.9609 909.046 Q30.0964 911.656 30.0964 915.984 Q30.0964 920.886 32.8654 923.846 Q35.6345 926.774 40.6634 927.22 L40.6316 906.34 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M31.465 870.215 Q30.8921 871.201 30.6375 872.379 Q30.351 873.525 30.351 874.925 Q30.351 879.891 33.5975 882.564 Q36.8122 885.206 42.8596 885.206 L61.6384 885.206 L61.6384 891.094 L25.9905 891.094 L25.9905 885.206 L31.5287 885.206 Q28.2821 883.36 26.7225 880.4 Q25.1311 877.44 25.1311 873.207 Q25.1311 872.602 25.2266 871.87 Q25.2903 871.138 25.4494 870.247 L31.465 870.215 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M43.719 847.871 Q43.719 854.969 45.3422 857.706 Q46.9655 860.443 50.8804 860.443 Q53.9996 860.443 55.8457 858.406 Q57.6599 856.338 57.6599 852.805 Q57.6599 847.935 54.2224 845.007 Q50.7531 842.047 45.0239 842.047 L43.719 842.047 L43.719 847.871 M41.3 836.19 L61.6384 836.19 L61.6384 842.047 L56.2276 842.047 Q59.4741 844.052 61.0337 847.044 Q62.5615 850.036 62.5615 854.364 Q62.5615 859.839 59.5059 863.085 Q56.4186 866.3 51.2623 866.3 Q45.2467 866.3 42.1912 862.289 Q39.1357 858.247 39.1357 850.258 L39.1357 842.047 L38.5628 842.047 Q34.5205 842.047 32.3244 844.72 Q30.0964 847.362 30.0964 852.168 Q30.0964 855.224 30.8284 858.12 Q31.5605 861.016 33.0246 863.69 L27.6137 863.69 Q26.3724 860.475 25.7677 857.452 Q25.1311 854.428 25.1311 851.563 Q25.1311 843.829 29.1415 840.01 Q33.1519 836.19 41.3 836.19 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M15.869 818.334 L25.9905 818.334 L25.9905 806.271 L30.542 806.271 L30.542 818.334 L49.8937 818.334 Q54.2542 818.334 55.4955 817.157 Q56.7368 815.947 56.7368 812.287 L56.7368 806.271 L61.6384 806.271 L61.6384 812.287 Q61.6384 819.066 59.124 821.644 Q56.5777 824.223 49.8937 824.223 L30.542 824.223 L30.542 828.519 L25.9905 828.519 L25.9905 824.223 L15.869 824.223 L15.869 818.334 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M47.5702 799.174 L25.9905 799.174 L25.9905 793.317 L47.3474 793.317 Q52.4082 793.317 54.9545 791.344 Q57.4689 789.37 57.4689 785.424 Q57.4689 780.681 54.4452 777.944 Q51.4215 775.175 46.2016 775.175 L25.9905 775.175 L25.9905 769.318 L61.6384 769.318 L61.6384 775.175 L56.1639 775.175 Q59.4104 777.307 61.0019 780.14 Q62.5615 782.941 62.5615 786.665 Q62.5615 792.808 58.742 795.991 Q54.9226 799.174 47.5702 799.174 M25.1311 784.437 L25.1311 784.437 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M31.465 736.599 Q30.8921 737.585 30.6375 738.763 Q30.351 739.909 30.351 741.309 Q30.351 746.274 33.5975 748.948 Q36.8122 751.59 42.8596 751.59 L61.6384 751.59 L61.6384 757.478 L25.9905 757.478 L25.9905 751.59 L31.5287 751.59 Q28.2821 749.744 26.7225 746.784 Q25.1311 743.824 25.1311 739.59 Q25.1311 738.986 25.2266 738.254 Q25.2903 737.522 25.4494 736.63 L31.465 736.599 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M42.3504 701.396 L45.2149 701.396 L45.2149 728.323 Q51.2623 727.941 54.4452 724.695 Q57.5962 721.416 57.5962 715.592 Q57.5962 712.218 56.7687 709.067 Q55.9411 705.884 54.2861 702.765 L59.8242 702.765 Q61.161 705.916 61.8612 709.226 Q62.5615 712.536 62.5615 715.942 Q62.5615 724.472 57.5962 729.469 Q52.631 734.434 44.1646 734.434 Q35.4117 734.434 30.2873 729.724 Q25.1311 724.981 25.1311 716.96 Q25.1311 709.767 29.7781 705.598 Q34.3932 701.396 42.3504 701.396 M40.6316 707.253 Q35.8255 707.316 32.9609 709.958 Q30.0964 712.568 30.0964 716.897 Q30.0964 721.798 32.8654 724.758 Q35.6345 727.687 40.6634 728.132 L40.6316 707.253 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M20.4523 654.927 L44.1009 663.648 L44.1009 646.174 L20.4523 654.927 M14.1184 658.555 L14.1184 651.266 L61.6384 633.156 L61.6384 639.84 L49.4481 644.169 L49.4481 665.589 L61.6384 669.918 L61.6384 676.697 L14.1184 658.555 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M40.1224 596.84 L61.6384 596.84 L61.6384 602.696 L40.3133 602.696 Q35.2526 602.696 32.7381 604.669 Q30.2237 606.643 30.2237 610.589 Q30.2237 615.332 33.2474 618.069 Q36.2711 620.806 41.491 620.806 L61.6384 620.806 L61.6384 626.695 L25.9905 626.695 L25.9905 620.806 L31.5287 620.806 Q28.314 618.706 26.7225 615.873 Q25.1311 613.008 25.1311 609.284 Q25.1311 603.142 28.9505 599.991 Q32.7381 596.84 40.1224 596.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M30.0964 571.345 Q30.0964 576.055 33.7885 578.793 Q37.4488 581.53 43.8463 581.53 Q50.2438 581.53 53.9359 578.825 Q57.5962 576.087 57.5962 571.345 Q57.5962 566.666 53.9041 563.929 Q50.212 561.192 43.8463 561.192 Q37.5124 561.192 33.8203 563.929 Q30.0964 566.666 30.0964 571.345 M25.1311 571.345 Q25.1311 563.706 30.0964 559.346 Q35.0616 554.985 43.8463 554.985 Q52.5991 554.985 57.5962 559.346 Q62.5615 563.706 62.5615 571.345 Q62.5615 579.016 57.5962 583.376 Q52.5991 587.705 43.8463 587.705 Q35.0616 587.705 30.0964 583.376 Q25.1311 579.016 25.1311 571.345 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M32.8336 517.523 Q28.8869 515.327 27.009 512.271 Q25.1311 509.216 25.1311 505.078 Q25.1311 499.508 29.046 496.484 Q32.9291 493.46 40.1224 493.46 L61.6384 493.46 L61.6384 499.349 L40.3133 499.349 Q35.1889 499.349 32.7063 501.163 Q30.2237 502.977 30.2237 506.701 Q30.2237 511.253 33.2474 513.894 Q36.2711 516.536 41.491 516.536 L61.6384 516.536 L61.6384 522.424 L40.3133 522.424 Q35.1571 522.424 32.7063 524.239 Q30.2237 526.053 30.2237 529.84 Q30.2237 534.328 33.2792 536.97 Q36.3029 539.612 41.491 539.612 L61.6384 539.612 L61.6384 545.5 L25.9905 545.5 L25.9905 539.612 L31.5287 539.612 Q28.2503 537.607 26.6907 534.806 Q25.1311 532.005 25.1311 528.154 Q25.1311 524.27 27.1045 521.565 Q29.0779 518.828 32.8336 517.523 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M43.719 465.579 Q43.719 472.676 45.3422 475.414 Q46.9655 478.151 50.8804 478.151 Q53.9996 478.151 55.8457 476.114 Q57.6599 474.045 57.6599 470.512 Q57.6599 465.642 54.2224 462.714 Q50.7531 459.754 45.0239 459.754 L43.719 459.754 L43.719 465.579 M41.3 453.898 L61.6384 453.898 L61.6384 459.754 L56.2276 459.754 Q59.4741 461.759 61.0337 464.751 Q62.5615 467.743 62.5615 472.072 Q62.5615 477.546 59.5059 480.793 Q56.4186 484.007 51.2623 484.007 Q45.2467 484.007 42.1912 479.997 Q39.1357 475.955 39.1357 467.966 L39.1357 459.754 L38.5628 459.754 Q34.5205 459.754 32.3244 462.428 Q30.0964 465.069 30.0964 469.876 Q30.0964 472.931 30.8284 475.827 Q31.5605 478.724 33.0246 481.397 L27.6137 481.397 Q26.3724 478.183 25.7677 475.159 Q25.1311 472.135 25.1311 469.271 Q25.1311 461.536 29.1415 457.717 Q33.1519 453.898 41.3 453.898 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M12.1132 441.835 L12.1132 435.978 L61.6384 435.978 L61.6384 441.835 L12.1132 441.835 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M64.9486 408.892 Q71.3143 411.375 73.2559 413.73 Q75.1974 416.085 75.1974 420.032 L75.1974 424.711 L70.2958 424.711 L70.2958 421.273 Q70.2958 418.854 69.15 417.518 Q68.0041 416.181 63.7391 414.558 L61.0655 413.507 L25.9905 427.925 L25.9905 421.719 L53.8723 410.579 L25.9905 399.439 L25.9905 393.232 L64.9486 408.892 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M12.1769 350.359 Q19.4975 354.624 26.6589 356.693 Q33.8203 358.762 41.1727 358.762 Q48.5251 358.762 55.7502 356.693 Q62.9434 354.593 70.2321 350.359 L70.2321 355.452 Q62.7524 360.226 55.5274 362.613 Q48.3023 364.969 41.1727 364.969 Q34.0749 364.969 26.8817 362.613 Q19.6884 360.258 12.1769 355.452 L12.1769 350.359 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M17.365 328.843 Q17.365 331.39 19.1474 333.14 Q20.8979 334.891 23.4442 334.891 Q25.9587 334.891 27.7092 333.14 Q29.428 331.39 29.428 328.843 Q29.428 326.297 27.7092 324.546 Q25.9587 322.796 23.4442 322.796 Q20.9297 322.796 19.1474 324.578 Q17.365 326.329 17.365 328.843 M13.2591 328.843 Q13.2591 326.806 14.0548 324.928 Q14.8187 323.05 16.2828 321.682 Q17.7151 320.218 19.5293 319.486 Q21.3435 318.754 23.4442 318.754 Q27.6456 318.754 30.5738 321.714 Q33.4702 324.642 33.4702 328.907 Q33.4702 333.204 30.6056 336.068 Q27.7411 338.933 23.4442 338.933 Q19.1792 338.933 16.2191 336.005 Q13.2591 333.076 13.2591 328.843 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M17.7787 270.565 L24.5582 270.565 Q21.5345 273.812 20.0386 277.504 Q18.5426 281.164 18.5426 285.302 Q18.5426 293.45 23.5397 297.779 Q28.5049 302.107 37.9262 302.107 Q47.3156 302.107 52.3127 297.779 Q57.2779 293.45 57.2779 285.302 Q57.2779 281.164 55.782 277.504 Q54.2861 273.812 51.2623 270.565 L57.9782 270.565 Q60.2698 273.939 61.4156 277.727 Q62.5615 281.482 62.5615 285.684 Q62.5615 296.474 55.973 302.68 Q49.3526 308.887 37.9262 308.887 Q26.4679 308.887 19.8794 302.68 Q13.2591 296.474 13.2591 285.684 Q13.2591 281.419 14.4049 277.663 Q15.5189 273.875 17.7787 270.565 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip880)" d="M12.1769 261.812 L12.1769 256.72 Q19.6884 251.946 26.8817 249.59 Q34.0749 247.203 41.1727 247.203 Q48.3023 247.203 55.5274 249.59 Q62.7524 251.946 70.2321 256.72 L70.2321 261.812 Q62.9434 257.579 55.7502 255.51 Q48.5251 253.41 41.1727 253.41 Q33.8203 253.41 26.6589 255.51 Q19.4975 257.579 12.1769 261.812 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip882)" style="stroke:#009af9; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1525.16 188.745,1520.1 195.771,1516.19 202.797,1513.49 209.823,1512.04 216.849,1511.66 223.874,1512.19 230.9,1517.08 237.926,1518.35 244.952,1516.75 251.978,1513.92 259.003,1511.04 266.029,1508.65 273.055,1524.7 280.081,1532.93 287.107,1532.24 294.133,1529.64 301.158,1533.25 308.184,1536.92 315.21,1534.01 322.236,1528.16 329.262,1524.62 336.287,1520.63 343.313,1516.47 350.339,1513.33 357.365,1511.68 364.391,1510.79 371.417,1510.33 378.442,1509.95 385.468,1508.83 392.494,1506.37 399.52,1503.65 406.546,1501.3 413.571,1499.77 420.597,1534.18 427.623,1599.61 434.649,1611.61 441.675,1605.3 448.701,1592.31 455.726,1576.99 462.752,1561.48 469.778,1547.85 476.804,1536.62 483.83,1527.65 490.855,1520.9 497.881,1516.34 504.907,1513.2 511.933,1513.7 518.959,1516.62 525.985,1517.33 533.01,1516.17 540.036,1514.33 547.062,1511.93 554.088,1509.33 561.114,1506.7 568.14,1504.39 575.165,1502.8 582.191,1501.68 589.217,1501.19 596.243,1501.41 603.269,1571.17 610.294,1607.79 617.32,1606.29 624.346,1593.21 631.372,1578.31 638.398,1564.66 645.424,1621.24 652.449,1686.38 659.475,1684.62 666.501,1661.48 673.527,1635.22 680.553,1611.7 687.578,1593.91 694.604,1579.46 701.63,1574.13 708.656,1564.79 715.682,1553.97 722.708,1543.41 729.733,1534.07 736.759,1526.47 743.785,1520.26 750.811,1515.61 757.837,1559.59 764.862,1581.31 771.888,1578.3 778.914,1568.22 785.94,1591.3 792.966,1597.39 799.992,1585.84 807.017,1570.48 814.043,1556.18 821.069,1544.31 828.095,1535.08 835.121,1528.09 842.146,1522.78 849.172,1518.74 856.198,1515.61 863.224,1516.21 870.25,1515.48 877.276,1513.25 884.301,1510.47 891.327,1507.81 898.353,1505.43 905.379,1503.44 912.405,1505.23 919.43,1507.81 926.456,1507.41 933.482,1506.9 940.508,1509.68 947.534,1509.07 954.56,1506.89 961.585,1504.43 968.611,1503.52 975.637,1517.37 982.663,1521.46 989.689,1519.07 996.714,1515.32 1003.74,1511.5 1010.77,1508.33 1017.79,1505.6 1024.82,1502.79 1031.84,1500.54 1038.87,1498.26 1045.9,1496.36 1052.92,1499.19 1059.95,1499.39 1066.97,1499.24 1074,1499.07 1081.02,1498.21 1088.05,1497.3 1095.08,1496.51 1102.1,1496.25 1109.13,1495.67 1116.15,1494.38 1123.18,1503.18 1130.2,1539.03 1137.23,1545.62 1144.26,1540.23 1151.28,1534.85 1158.31,1527.55 1165.33,1520.36 1172.36,1516.72 1179.39,1516.31 1186.41,1511.8 1193.44,1505.58 1200.46,1500.13 1207.49,1495.88 1214.51,1492.75 1221.54,1490.63 1228.57,1489 1235.59,1488.03 1242.62,1487.26 1249.64,1486.66 1256.67,1488.01 1263.7,1504.32 1270.72,1508.41 1277.75,1505.4 1284.77,1500.44 1291.8,1499.13 1298.82,1496.31 1305.85,1492.89 1312.88,1490.43 1319.9,1488.33 1326.93,1495.11 1333.95,1501.89 1340.98,1500.34 1348.01,1495.03 1355.03,1489.57 1362.06,1484.68 1369.08,1480.41 1376.11,1476.71 1383.13,1474.47 1390.16,1471.97 1397.19,1472.12 1404.21,1472.41 1411.24,1470.73 1418.26,1468.89 1425.29,1466.73 1432.32,1464.76 1439.34,1463.82 1446.37,1466.51 1453.39,1465.09 1460.42,1463.11 1467.44,1464.08 1474.47,1462.88 1481.5,1461.44 1488.52,1459.79 1495.55,1457.51 1502.57,1455.51 1509.6,1453.26 1516.62,1451.78 1523.65,1451.04 1530.68,1450.68 1537.7,1450.86 1544.73,1451.19 1551.75,1451.21 1558.78,1449.7 1565.81,1447.79 1572.83,1445.78 1579.86,1444.07 1586.88,1442.75 1593.91,1443.08 1600.93,1443.75 1607.96,1444.37 1614.99,1445.15 1622.01,1445.52 1629.04,1445.56 1636.06,1444.96 1643.09,1443.71 1650.12,1441.76 1657.14,1440.78 1664.17,1440.23 1671.19,1441.98 1678.22,1446.49 1685.24,1465.18 1692.27,1477.78 1699.3,1481.47 1706.32,1475.47 1713.35,1473.44 1720.37,1469.41 1727.4,1463.98 1734.43,1459.25 1741.45,1457.57 1748.48,1454.59 1755.5,1449.26 1762.53,1445.34 1769.55,1447.24 1776.58,1444.32 1783.61,1438.69 1790.63,1431.94 1797.66,1425.03 1804.68,1419.18 1811.71,1413.64 1818.74,1415.07 1825.76,1423.25 1832.79,1422.39 1839.81,1416.82 1846.84,1411.36 1853.86,1405.01 1860.89,1396.82 1867.92,1387.11 1874.94,1378.99 1881.97,1386.26 1888.99,1410.58 1896.02,1413.75 1903.04,1405.15 1910.07,1393.01 1917.1,1380.99 1924.12,1372.96 1931.15,1363.93 1938.17,1352.23 1945.2,1340.92 1952.23,1331.36 1959.25,1324.3 1966.28,1319.58 1973.3,1315.14 1980.33,1312.25 1987.35,1310.11 1994.38,1307.56 2001.41,1303.65 2008.43,1299.88 2015.46,1295.49 2022.48,1290.57 2029.51,1285.74 2036.54,1279.34 2043.56,1274.35 2050.59,1268.41 2057.61,1262.78 2064.64,1256.89 2071.66,1250.71 2078.69,1244.23 2085.72,1237.39 2092.74,1230.07 2099.77,1222.26 2106.79,1214.37 2113.82,1206.65 2120.85,1199.37 2127.87,1192.49 2134.9,1185.92 2141.92,1179.26 2148.95,1172.66 2155.97,1165.91 2163,1159.2 2170.03,1152.27 2177.05,1144.75 2184.08,1136.38 2191.1,1127.96 2198.13,1119.2 2205.16,1110.82 2212.18,1102.39 2219.21,1094.35 2226.23,1086.57 2233.26,1078.93 2240.28,1071.08 2247.31,1062.89 2254.34,1054.2 2261.36,1044.61 2268.39,1034.5 2275.41,1023.88 2282.44,1013.31 2289.46,1003.25 2296.49,993.821 2303.52,984.661 2310.54,975.679 2317.57,966.749 2324.59,957.55 2331.62,947.798 2338.65,937.49 2345.67,927.21 2352.7,916.874 2359.72,906.505 2366.75,896.581 2373.77,886.92 2380.8,877.4 2387.83,867.797 2394.85,858.05 2401.88,848.013 2408.9,837.298 2415.93,826.078 2422.96,814.661 2429.98,803.461 2437.01,792.626 2444.03,782.155 2451.06,771.755 2458.08,761.367 2465.11,751.069 2472.14,740.715 2479.16,730.321 2486.19,719.681 2493.21,708.62 2500.24,697.193 2507.27,685.582 2514.29,674.06 2521.32,662.685 2528.34,651.726 2535.37,640.986 2542.39,630.748 2549.42,620.76 2556.45,610.789 2563.47,600.806 2570.5,590.546 2577.52,579.869 2584.55,569.024 2591.58,557.945 2598.6,547.43 2605.63,537.618 2612.65,528.31 2619.68,519.492 2626.7,510.828 2633.73,502.299 2640.76,493.585 2647.78,484.497 2654.81,475.119 2661.83,466.06 2668.86,457.282 2675.88,448.956 2682.91,441.221 2689.94,433.816 2696.96,426.662 2703.99,419.564 2711.01,412.387 2718.04,404.892 2725.07,396.718 2732.09,388.818 2739.12,381.154 2746.14,374.237 2753.17,367.606 2760.19,361.501 2767.22,355.647 2774.25,349.941 2781.27,344.231 2788.3,338.156 2795.32,331.585 2802.35,324.592 2809.38,318.278 2816.4,312.084 2823.43,306.612 2830.45,301.685 2837.48,296.991 2844.5,292.462 2851.53,287.846 2858.56,282.835 2865.58,276.902 2872.61,271.079 2879.63,265.332 2886.66,260.181 2893.69,255.701 2900.71,251.88 2907.74,248.2 2914.76,244.645 2921.79,241.114 2928.81,237.466 2935.84,233.399 2942.87,228.99 2949.89,224.676 2956.92,220.417 2963.94,216.378 2970.97,212.824 2978,209.36 2985.02,206.182 2992.05,203.141 2999.07,200.152 3006.1,197.149 3013.12,193.801 3020.15,189.855 3027.18,185.595 3034.2,181.634 3041.23,178.003 3048.25,174.881 3055.28,172.327 3062.31,170.166 3069.33,168.161 3076.36,166.065 3083.38,163.576 3090.41,160.423 3097.43,156.761 3104.46,153.593 3111.49,150.519 3118.51,148.296 3125.54,146.588 3132.56,145.176 3139.59,143.87 3146.61,142.518 3153.64,140.888 3160.67,138.634 3167.69,136.133 3174.72,133.478 3181.74,131.009 3188.77,129.032 3195.8,127.603 3202.82,126.45 3209.85,125.558 3216.87,124.767 3223.9,124.045 3230.92,123.291 3237.95,122.38 3244.98,121.15 3252,119.296 3259.03,117.481 3266.05,115.624 3273.08,113.793 3280.11,112.569 3287.13,111.791 3294.16,111.248 3301.18,110.737 3308.21,110.214 3315.23,109.484 3322.26,108.178 3329.29,106.381 3336.31,104.453 3343.34,102.917 3350.36,101.843 3357.39,101.368 3364.42,101.092 3371.44,100.946 3378.47,101.019 3385.49,101.106 3392.52,101.231 3399.54,101.153 3406.57,100.68 3413.6,99.8475 3420.62,98.827 3427.65,97.9569 3434.67,97.2561 3441.7,96.9634 3448.73,96.8752 3455.75,97.2408 3462.78,97.7884 3469.8,98.256 3476.83,98.5953 3483.85,98.5342 3490.88,97.9604 3497.91,97.109 3504.93,95.8558 3511.96,95.1156 3518.98,94.9858 3526.01,95.2983 3533.03,96.0624 3540.06,96.8573 3547.09,97.7547 3554.11,98.3438 3561.14,98.4007 3568.16,98.1934 3575.19,98.0245 3582.22,97.782 3589.24,97.9053 3596.27,98.3045 3603.29,99.1861 3610.32,100.19 3617.34,101.233 3624.37,102.169 3631.4,102.803 3638.42,102.842 3645.45,102.462 3652.47,102.214 3659.5,102.221 3666.53,102.769 3673.55,103.775 3680.58,104.921 3687.6,106.29 3694.63,107.738 3701.65,109.213 3708.68,110.631 3715.71,111.787 3722.73,112.509 3729.76,112.807 3736.78,113.139 3743.81,113.447 3750.84,113.929 3757.86,114.764 3764.89,115.793 3771.91,117.175 3778.94,118.627 3785.96,119.915 3792.99,120.999 3800.02,121.577 3807.04,121.571 3814.07,121.225 3821.09,120.625 3828.12,120.682 3835.15,121.168 3842.17,122.117 3849.2,123.257 3856.22,124.539 3863.25,125.672 3870.27,126.362 3877.3,126.545 3884.33,126.439 3891.35,126.437 3898.38,126.583 3905.4,127.025 3912.43,127.909 3919.45,128.936 3926.48,129.936 3933.51,130.868 3940.53,131.472 3947.56,131.479 3954.58,130.967 3961.61,130.423 3968.64,130.062 3975.66,130.494 3982.69,131.063 3989.71,131.947 3996.74,133.03 4003.76,134.144 4010.79,134.985 4017.82,135.222 4024.84,135.02 4031.87,134.427 4038.89,133.989 4045.92,134.066 4052.95,134.22 4059.97,134.405 4067,134.613 4074.02,134.841 4081.05,135.084 4088.07,135.34 4095.1,135.605 4102.13,135.878 4109.15,136.157 4116.18,136.442 4123.2,136.729 4130.23,137.018 4137.26,137.308 4144.28,137.599 4151.31,137.891 4158.33,138.184 4165.36,138.477 4172.38,138.77 4179.41,139.063 4186.44,139.356 4193.46,139.647 4200.49,139.938 4207.51,140.228 4214.54,140.517 4221.57,140.805 4228.59,141.094 4235.62,141.382 4242.64,141.668 4249.67,141.953 4256.69,142.237 4263.72,142.52 4270.75,142.801 4277.77,143.082 4284.8,143.364 4291.82,143.643 4298.85,143.921 4305.87,144.2 4312.9,144.479 4319.93,144.756 4326.95,145.032 4333.98,145.306 4341,145.58 4348.03,145.852 4355.06,146.123 4362.08,146.392 4369.11,146.66 4376.13,146.928 4383.16,147.195 4390.18,147.461 4397.21,147.726 4404.24,147.99 4411.26,148.254 4418.29,148.516 4425.31,148.778 4432.34,149.039 4439.37,149.299 4446.39,149.559 4453.42,149.818 4460.44,150.077 4467.47,150.335 4474.49,150.593 4481.52,150.849 4488.55,151.105 4495.57,151.361 4502.6,151.616 4509.62,151.87 4516.65,152.124 4523.68,152.378 4530.7,152.632 4537.73,152.884 4544.75,153.137 4551.78,153.388 4558.8,153.639 4565.83,153.89 4572.86,154.139 4579.88,154.389 4586.91,154.639 4593.93,154.888 4600.96,155.136 4607.99,155.384 4615.01,155.632 4622.04,155.88 4629.06,156.128 4636.09,156.375 4643.11,156.62 4650.14,156.866 4657.17,157.112 4664.19,157.358 4671.22,157.603 4678.24,157.847 4685.27,158.091 4692.29,158.336 4699.32,158.579 4706.35,158.822 4713.37,159.064 4720.4,159.307 4727.42,159.55 4734.45,159.792 4741.48,160.033 4748.5,160.274 4755.53,160.513 4762.55,160.754 4769.58,160.994 4776.6,161.232 4783.63,161.471 4790.66,161.71 4797.68,161.949 4804.71,162.188 4811.73,162.426 4818.76,162.664 4825.79,162.902 4832.81,163.139 4839.84,163.377 4846.86,163.613 4853.89,163.849 4860.91,164.086 4867.94,164.322 4874.97,164.557 4881.99,164.793 4889.02,165.028 4896.04,165.262 4903.07,165.497 4910.1,165.731 4917.12,165.965 4924.15,166.199 4931.17,166.432 4938.2,166.665 4945.22,166.897 4952.25,167.13 4959.28,167.363 4966.3,167.595 4973.33,167.826 4980.35,168.057 4987.38,168.288 4994.41,168.519 5001.43,168.749 5008.46,168.98 5015.48,169.211 5022.51,169.442 5029.53,169.672 5036.56,169.902 5043.59,170.132 5050.61,170.361 5057.64,170.591 5064.66,170.819 5071.69,171.048 5078.71,171.276 5085.74,171.504 5092.77,171.731 5099.79,171.958 5106.82,172.184 5113.84,172.411 5120.87,172.637 5127.9,172.863 5134.92,173.089 5141.95,173.314 5148.97,173.54 5156,173.766 5163.02,173.991 5170.05,174.216 5177.08,174.44 5184.1,174.664 5191.13,174.889 5198.15,175.113 5205.18,175.337 5212.21,175.561 5219.23,175.783 5226.26,176.006 5233.28,176.229 5240.31,176.452 5247.33,176.674 5254.36,176.896 5261.39,177.118 5268.41,177.34 5275.44,177.562 5282.46,177.784 5289.49,178.006 5296.52,178.227 5303.54,178.448 5310.57,178.669 5317.59,178.889 5324.62,179.108 5331.64,179.327 5338.67,179.547 5345.7,179.764 5352.72,179.983 5359.75,180.202 5366.77,180.42 5373.8,180.639 5380.83,180.858 5387.85,181.075 5394.88,181.292 5401.9,181.51 5408.93,181.728 5415.95,181.944 5422.98,182.161 5430.01,182.377 5437.03,182.594 5444.06,182.81 5451.08,183.025 "/>
</svg>

Figure 2: Global mean temperature anomalies (in °C, relative to 1750)
from 1750 to 2100 simulated using the energy balance model and RCP 8.5
forcings and parameters given in the problem statement.

1.  This syntax creates an empty plot object with the right axis labels,
    legend settings, etc; this is not necessary but can make the code
    more readable.
2.  `plot!` is different than `plot` because it adds elements to an
    existing plot object (in this case, `p`) rather than creating a new
    one. Since `p` is the last plot object created, we could have just
    written `plot!(t, temps)`, but adding `p` explicitly minimizes the
    chances of accidentally plotting onto the wrong axis.
3.  Similarly, `xlims!` changes the x limits for an existing plot object
    (analogously, there is also `ylims!`).

For the Monte Carlo analysis, we’ll start with 10,000 samples of
$\lambda$ and see how the simulation looks.

``` julia
n_samples = 10_000
λ_dist = LogNormal(log(2.1), log(2) / 4)
λ_samples = rand(λ_dist, n_samples)
h = histogram(λ_samples; xlabel="Climate Feedback Factor (°C/W/m²)", ylabel="Count", legend=false)
vline!(h, [2.1], color=:red, linestyle=:dash) # the original value for context
```

<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="672" height="480" viewBox="0 0 2688 1920">
<defs>
  <clipPath id="clip940">
    <rect x="0" y="0" width="2688" height="1920"/>
  </clipPath>
</defs>
<path clip-path="url(#clip940)" d="M0 1920 L2688 1920 L2688 -4.26326e-14 L0 -4.26326e-14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip941">
    <rect x="537" y="0" width="1883" height="1883"/>
  </clipPath>
</defs>
<path clip-path="url(#clip940)" d="M241.557 1734.12 L2640.76 1734.12 L2640.76 47.2441 L241.557 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip942">
    <rect x="241" y="47" width="2400" height="1688"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="309.778,1734.12 309.778,47.2441 "/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="947.174,1734.12 947.174,47.2441 "/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1584.57,1734.12 1584.57,47.2441 "/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2221.97,1734.12 2221.97,47.2441 "/>
<polyline clip-path="url(#clip940)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip940)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="309.778,1734.12 309.778,1715.22 "/>
<polyline clip-path="url(#clip940)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="947.174,1734.12 947.174,1715.22 "/>
<polyline clip-path="url(#clip940)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1584.57,1734.12 1584.57,1715.22 "/>
<polyline clip-path="url(#clip940)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2221.97,1734.12 2221.97,1715.22 "/>
<path clip-path="url(#clip940)" d="M300.16 1794.89 L307.799 1794.89 L307.799 1768.52 L299.488 1770.19 L299.488 1765.93 L307.752 1764.26 L312.428 1764.26 L312.428 1794.89 L320.067 1794.89 L320.067 1798.82 L300.16 1798.82 L300.16 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M941.827 1794.89 L958.146 1794.89 L958.146 1798.82 L936.202 1798.82 L936.202 1794.89 Q938.864 1792.13 943.447 1787.5 Q948.054 1782.85 949.234 1781.51 Q951.48 1778.98 952.359 1777.25 Q953.262 1775.49 953.262 1773.8 Q953.262 1771.04 951.318 1769.31 Q949.396 1767.57 946.295 1767.57 Q944.096 1767.57 941.642 1768.34 Q939.211 1769.1 936.434 1770.65 L936.434 1765.93 Q939.258 1764.79 941.711 1764.21 Q944.165 1763.64 946.202 1763.64 Q951.572 1763.64 954.767 1766.32 Q957.961 1769.01 957.961 1773.5 Q957.961 1775.63 957.151 1777.55 Q956.364 1779.45 954.258 1782.04 Q953.679 1782.71 950.577 1785.93 Q947.475 1789.12 941.827 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1588.82 1780.19 Q1592.17 1780.9 1594.05 1783.17 Q1595.95 1785.44 1595.95 1788.77 Q1595.95 1793.89 1592.43 1796.69 Q1588.91 1799.49 1582.43 1799.49 Q1580.25 1799.49 1577.94 1799.05 Q1575.65 1798.64 1573.19 1797.78 L1573.19 1793.27 Q1575.14 1794.4 1577.45 1794.98 Q1579.77 1795.56 1582.29 1795.56 Q1586.69 1795.56 1588.98 1793.82 Q1591.3 1792.08 1591.3 1788.77 Q1591.3 1785.72 1589.14 1784.01 Q1587.01 1782.27 1583.19 1782.27 L1579.17 1782.27 L1579.17 1778.43 L1583.38 1778.43 Q1586.83 1778.43 1588.66 1777.06 Q1590.49 1775.67 1590.49 1773.08 Q1590.49 1770.42 1588.59 1769.01 Q1586.71 1767.57 1583.19 1767.57 Q1581.27 1767.57 1579.07 1767.99 Q1576.87 1768.4 1574.24 1769.28 L1574.24 1765.12 Q1576.9 1764.38 1579.21 1764.01 Q1581.55 1763.64 1583.61 1763.64 Q1588.93 1763.64 1592.04 1766.07 Q1595.14 1768.47 1595.14 1772.59 Q1595.14 1775.46 1593.49 1777.46 Q1591.85 1779.42 1588.82 1780.19 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M2224.98 1768.34 L2213.17 1786.78 L2224.98 1786.78 L2224.98 1768.34 M2223.75 1764.26 L2229.63 1764.26 L2229.63 1786.78 L2234.56 1786.78 L2234.56 1790.67 L2229.63 1790.67 L2229.63 1798.82 L2224.98 1798.82 L2224.98 1790.67 L2209.37 1790.67 L2209.37 1786.16 L2223.75 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M909.143 1839.44 L909.143 1846.22 Q905.896 1843.2 902.204 1841.7 Q898.544 1840.21 894.406 1840.21 Q886.258 1840.21 881.929 1845.2 Q877.601 1850.17 877.601 1859.59 Q877.601 1868.98 881.929 1873.98 Q886.258 1878.94 894.406 1878.94 Q898.544 1878.94 902.204 1877.44 Q905.896 1875.95 909.143 1872.92 L909.143 1879.64 Q905.769 1881.93 901.981 1883.08 Q898.225 1884.22 894.024 1884.22 Q883.234 1884.22 877.028 1877.64 Q870.821 1871.02 870.821 1859.59 Q870.821 1848.13 877.028 1841.54 Q883.234 1834.92 894.024 1834.92 Q898.289 1834.92 902.045 1836.07 Q905.832 1837.18 909.143 1839.44 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M918.818 1833.78 L924.675 1833.78 L924.675 1883.3 L918.818 1883.3 L918.818 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M936.929 1847.65 L942.785 1847.65 L942.785 1883.3 L936.929 1883.3 L936.929 1847.65 M936.929 1833.78 L942.785 1833.78 L942.785 1841.19 L936.929 1841.19 L936.929 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M982.794 1854.5 Q984.99 1850.55 988.046 1848.67 Q991.101 1846.79 995.239 1846.79 Q1000.81 1846.79 1003.83 1850.71 Q1006.86 1854.59 1006.86 1861.78 L1006.86 1883.3 L1000.97 1883.3 L1000.97 1861.98 Q1000.97 1856.85 999.154 1854.37 Q997.339 1851.89 993.616 1851.89 Q989.064 1851.89 986.422 1854.91 Q983.78 1857.93 983.78 1863.15 L983.78 1883.3 L977.892 1883.3 L977.892 1861.98 Q977.892 1856.82 976.078 1854.37 Q974.264 1851.89 970.476 1851.89 Q965.988 1851.89 963.347 1854.94 Q960.705 1857.97 960.705 1863.15 L960.705 1883.3 L954.817 1883.3 L954.817 1847.65 L960.705 1847.65 L960.705 1853.19 Q962.71 1849.91 965.511 1848.35 Q968.312 1846.79 972.163 1846.79 Q976.046 1846.79 978.752 1848.77 Q981.489 1850.74 982.794 1854.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1034.74 1865.38 Q1027.64 1865.38 1024.9 1867 Q1022.17 1868.63 1022.17 1872.54 Q1022.17 1875.66 1024.2 1877.51 Q1026.27 1879.32 1029.8 1879.32 Q1034.67 1879.32 1037.6 1875.88 Q1040.56 1872.42 1040.56 1866.69 L1040.56 1865.38 L1034.74 1865.38 M1046.42 1862.96 L1046.42 1883.3 L1040.56 1883.3 L1040.56 1877.89 Q1038.56 1881.14 1035.57 1882.7 Q1032.57 1884.22 1028.24 1884.22 Q1022.77 1884.22 1019.52 1881.17 Q1016.31 1878.08 1016.31 1872.92 Q1016.31 1866.91 1020.32 1863.85 Q1024.36 1860.8 1032.35 1860.8 L1040.56 1860.8 L1040.56 1860.23 Q1040.56 1856.18 1037.89 1853.99 Q1035.25 1851.76 1030.44 1851.76 Q1027.39 1851.76 1024.49 1852.49 Q1021.59 1853.22 1018.92 1854.69 L1018.92 1849.28 Q1022.13 1848.03 1025.16 1847.43 Q1028.18 1846.79 1031.05 1846.79 Q1038.78 1846.79 1042.6 1850.8 Q1046.42 1854.81 1046.42 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1064.27 1837.53 L1064.27 1847.65 L1076.34 1847.65 L1076.34 1852.2 L1064.27 1852.2 L1064.27 1871.56 Q1064.27 1875.92 1065.45 1877.16 Q1066.66 1878.4 1070.32 1878.4 L1076.34 1878.4 L1076.34 1883.3 L1070.32 1883.3 Q1063.54 1883.3 1060.96 1880.79 Q1058.39 1878.24 1058.39 1871.56 L1058.39 1852.2 L1054.09 1852.2 L1054.09 1847.65 L1058.39 1847.65 L1058.39 1837.53 L1064.27 1837.53 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1114.53 1864.01 L1114.53 1866.88 L1087.61 1866.88 Q1087.99 1872.92 1091.23 1876.11 Q1094.51 1879.26 1100.34 1879.26 Q1103.71 1879.26 1106.86 1878.43 Q1110.04 1877.6 1113.16 1875.95 L1113.16 1881.49 Q1110.01 1882.82 1106.7 1883.52 Q1103.39 1884.22 1099.99 1884.22 Q1091.46 1884.22 1086.46 1879.26 Q1081.49 1874.29 1081.49 1865.83 Q1081.49 1857.07 1086.2 1851.95 Q1090.95 1846.79 1098.97 1846.79 Q1106.16 1846.79 1110.33 1851.44 Q1114.53 1856.06 1114.53 1864.01 M1108.68 1862.29 Q1108.61 1857.49 1105.97 1854.62 Q1103.36 1851.76 1099.03 1851.76 Q1094.13 1851.76 1091.17 1854.53 Q1088.24 1857.3 1087.8 1862.33 L1108.68 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1145.12 1835.78 L1172.43 1835.78 L1172.43 1841.19 L1151.55 1841.19 L1151.55 1855.2 L1170.39 1855.2 L1170.39 1860.61 L1151.55 1860.61 L1151.55 1883.3 L1145.12 1883.3 L1145.12 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1209.29 1864.01 L1209.29 1866.88 L1182.36 1866.88 Q1182.74 1872.92 1185.99 1876.11 Q1189.27 1879.26 1195.09 1879.26 Q1198.46 1879.26 1201.61 1878.43 Q1204.8 1877.6 1207.92 1875.95 L1207.92 1881.49 Q1204.77 1882.82 1201.46 1883.52 Q1198.15 1884.22 1194.74 1884.22 Q1186.21 1884.22 1181.21 1879.26 Q1176.25 1874.29 1176.25 1865.83 Q1176.25 1857.07 1180.96 1851.95 Q1185.7 1846.79 1193.72 1846.79 Q1200.91 1846.79 1205.08 1851.44 Q1209.29 1856.06 1209.29 1864.01 M1203.43 1862.29 Q1203.37 1857.49 1200.72 1854.62 Q1198.11 1851.76 1193.79 1851.76 Q1188.88 1851.76 1185.92 1854.53 Q1183 1857.3 1182.55 1862.33 L1203.43 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1249.39 1864.01 L1249.39 1866.88 L1222.46 1866.88 Q1222.84 1872.92 1226.09 1876.11 Q1229.37 1879.26 1235.19 1879.26 Q1238.57 1879.26 1241.72 1878.43 Q1244.9 1877.6 1248.02 1875.95 L1248.02 1881.49 Q1244.87 1882.82 1241.56 1883.52 Q1238.25 1884.22 1234.84 1884.22 Q1226.31 1884.22 1221.32 1879.26 Q1216.35 1874.29 1216.35 1865.83 Q1216.35 1857.07 1221.06 1851.95 Q1225.8 1846.79 1233.83 1846.79 Q1241.02 1846.79 1245.19 1851.44 Q1249.39 1856.06 1249.39 1864.01 M1243.53 1862.29 Q1243.47 1857.49 1240.83 1854.62 Q1238.22 1851.76 1233.89 1851.76 Q1228.99 1851.76 1226.03 1854.53 Q1223.1 1857.3 1222.65 1862.33 L1243.53 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1282.46 1853.06 L1282.46 1833.78 L1288.32 1833.78 L1288.32 1883.3 L1282.46 1883.3 L1282.46 1877.95 Q1280.61 1881.14 1277.78 1882.7 Q1274.98 1884.22 1271.03 1884.22 Q1264.57 1884.22 1260.5 1879.07 Q1256.46 1873.91 1256.46 1865.51 Q1256.46 1857.11 1260.5 1851.95 Q1264.57 1846.79 1271.03 1846.79 Q1274.98 1846.79 1277.78 1848.35 Q1280.61 1849.88 1282.46 1853.06 M1262.5 1865.51 Q1262.5 1871.97 1265.14 1875.66 Q1267.82 1879.32 1272.47 1879.32 Q1277.11 1879.32 1279.79 1875.66 Q1282.46 1871.97 1282.46 1865.51 Q1282.46 1859.05 1279.79 1855.39 Q1277.11 1851.7 1272.47 1851.7 Q1267.82 1851.7 1265.14 1855.39 Q1262.5 1859.05 1262.5 1865.51 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1325.97 1865.51 Q1325.97 1859.05 1323.3 1855.39 Q1320.65 1851.7 1316.01 1851.7 Q1311.36 1851.7 1308.69 1855.39 Q1306.04 1859.05 1306.04 1865.51 Q1306.04 1871.97 1308.69 1875.66 Q1311.36 1879.32 1316.01 1879.32 Q1320.65 1879.32 1323.3 1875.66 Q1325.97 1871.97 1325.97 1865.51 M1306.04 1853.06 Q1307.89 1849.88 1310.69 1848.35 Q1313.52 1846.79 1317.44 1846.79 Q1323.93 1846.79 1327.97 1851.95 Q1332.05 1857.11 1332.05 1865.51 Q1332.05 1873.91 1327.97 1879.07 Q1323.93 1884.22 1317.44 1884.22 Q1313.52 1884.22 1310.69 1882.7 Q1307.89 1881.14 1306.04 1877.95 L1306.04 1883.3 L1300.16 1883.3 L1300.16 1833.78 L1306.04 1833.78 L1306.04 1853.06 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1357.96 1865.38 Q1350.86 1865.38 1348.12 1867 Q1345.38 1868.63 1345.38 1872.54 Q1345.38 1875.66 1347.42 1877.51 Q1349.49 1879.32 1353.02 1879.32 Q1357.89 1879.32 1360.82 1875.88 Q1363.78 1872.42 1363.78 1866.69 L1363.78 1865.38 L1357.96 1865.38 M1369.64 1862.96 L1369.64 1883.3 L1363.78 1883.3 L1363.78 1877.89 Q1361.78 1881.14 1358.78 1882.7 Q1355.79 1884.22 1351.46 1884.22 Q1345.99 1884.22 1342.74 1881.17 Q1339.53 1878.08 1339.53 1872.92 Q1339.53 1866.91 1343.54 1863.85 Q1347.58 1860.8 1355.57 1860.8 L1363.78 1860.8 L1363.78 1860.23 Q1363.78 1856.18 1361.11 1853.99 Q1358.47 1851.76 1353.66 1851.76 Q1350.6 1851.76 1347.71 1852.49 Q1344.81 1853.22 1342.14 1854.69 L1342.14 1849.28 Q1345.35 1848.03 1348.38 1847.43 Q1351.4 1846.79 1354.26 1846.79 Q1362 1846.79 1365.82 1850.8 Q1369.64 1854.81 1369.64 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1407.35 1849.02 L1407.35 1854.5 Q1404.87 1853.13 1402.36 1852.46 Q1399.87 1851.76 1397.33 1851.76 Q1391.63 1851.76 1388.48 1855.39 Q1385.33 1858.98 1385.33 1865.51 Q1385.33 1872.03 1388.48 1875.66 Q1391.63 1879.26 1397.33 1879.26 Q1399.87 1879.26 1402.36 1878.59 Q1404.87 1877.89 1407.35 1876.52 L1407.35 1881.93 Q1404.9 1883.08 1402.26 1883.65 Q1399.65 1884.22 1396.69 1884.22 Q1388.64 1884.22 1383.9 1879.16 Q1379.15 1874.1 1379.15 1865.51 Q1379.15 1856.79 1383.93 1851.79 Q1388.73 1846.79 1397.07 1846.79 Q1399.78 1846.79 1402.36 1847.37 Q1404.94 1847.91 1407.35 1849.02 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1417.32 1833.78 L1423.21 1833.78 L1423.21 1863.03 L1440.68 1847.65 L1448.16 1847.65 L1429.25 1864.33 L1448.95 1883.3 L1441.32 1883.3 L1423.21 1865.89 L1423.21 1883.3 L1417.32 1883.3 L1417.32 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1476.26 1835.78 L1503.57 1835.78 L1503.57 1841.19 L1482.69 1841.19 L1482.69 1855.2 L1501.54 1855.2 L1501.54 1860.61 L1482.69 1860.61 L1482.69 1883.3 L1476.26 1883.3 L1476.26 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1523.72 1865.38 Q1516.62 1865.38 1513.88 1867 Q1511.15 1868.63 1511.15 1872.54 Q1511.15 1875.66 1513.18 1877.51 Q1515.25 1879.32 1518.79 1879.32 Q1523.66 1879.32 1526.58 1875.88 Q1529.54 1872.42 1529.54 1866.69 L1529.54 1865.38 L1523.72 1865.38 M1535.4 1862.96 L1535.4 1883.3 L1529.54 1883.3 L1529.54 1877.89 Q1527.54 1881.14 1524.55 1882.7 Q1521.56 1884.22 1517.23 1884.22 Q1511.75 1884.22 1508.51 1881.17 Q1505.29 1878.08 1505.29 1872.92 Q1505.29 1866.91 1509.3 1863.85 Q1513.34 1860.8 1521.33 1860.8 L1529.54 1860.8 L1529.54 1860.23 Q1529.54 1856.18 1526.87 1853.99 Q1524.23 1851.76 1519.42 1851.76 Q1516.37 1851.76 1513.47 1852.49 Q1510.57 1853.22 1507.9 1854.69 L1507.9 1849.28 Q1511.12 1848.03 1514.14 1847.43 Q1517.16 1846.79 1520.03 1846.79 Q1527.76 1846.79 1531.58 1850.8 Q1535.4 1854.81 1535.4 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1573.12 1849.02 L1573.12 1854.5 Q1570.63 1853.13 1568.12 1852.46 Q1565.64 1851.76 1563.09 1851.76 Q1557.39 1851.76 1554.24 1855.39 Q1551.09 1858.98 1551.09 1865.51 Q1551.09 1872.03 1554.24 1875.66 Q1557.39 1879.26 1563.09 1879.26 Q1565.64 1879.26 1568.12 1878.59 Q1570.63 1877.89 1573.12 1876.52 L1573.12 1881.93 Q1570.67 1883.08 1568.03 1883.65 Q1565.42 1884.22 1562.46 1884.22 Q1554.4 1884.22 1549.66 1879.16 Q1544.92 1874.1 1544.92 1865.51 Q1544.92 1856.79 1549.69 1851.79 Q1554.5 1846.79 1562.84 1846.79 Q1565.54 1846.79 1568.12 1847.37 Q1570.7 1847.91 1573.12 1849.02 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1589.1 1837.53 L1589.1 1847.65 L1601.16 1847.65 L1601.16 1852.2 L1589.1 1852.2 L1589.1 1871.56 Q1589.1 1875.92 1590.27 1877.16 Q1591.48 1878.4 1595.14 1878.4 L1601.16 1878.4 L1601.16 1883.3 L1595.14 1883.3 Q1588.36 1883.3 1585.79 1880.79 Q1583.21 1878.24 1583.21 1871.56 L1583.21 1852.2 L1578.91 1852.2 L1578.91 1847.65 L1583.21 1847.65 L1583.21 1837.53 L1589.1 1837.53 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1622.67 1851.76 Q1617.96 1851.76 1615.23 1855.45 Q1612.49 1859.11 1612.49 1865.51 Q1612.49 1871.91 1615.19 1875.6 Q1617.93 1879.26 1622.67 1879.26 Q1627.35 1879.26 1630.09 1875.57 Q1632.83 1871.87 1632.83 1865.51 Q1632.83 1859.17 1630.09 1855.48 Q1627.35 1851.76 1622.67 1851.76 M1622.67 1846.79 Q1630.31 1846.79 1634.67 1851.76 Q1639.03 1856.72 1639.03 1865.51 Q1639.03 1874.26 1634.67 1879.26 Q1630.31 1884.22 1622.67 1884.22 Q1615 1884.22 1610.64 1879.26 Q1606.31 1874.26 1606.31 1865.51 Q1606.31 1856.72 1610.64 1851.76 Q1615 1846.79 1622.67 1846.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1669.4 1853.13 Q1668.41 1852.55 1667.23 1852.3 Q1666.09 1852.01 1664.69 1852.01 Q1659.72 1852.01 1657.05 1855.26 Q1654.41 1858.47 1654.41 1864.52 L1654.41 1883.3 L1648.52 1883.3 L1648.52 1847.65 L1654.41 1847.65 L1654.41 1853.19 Q1656.25 1849.94 1659.21 1848.39 Q1662.17 1846.79 1666.41 1846.79 Q1667.01 1846.79 1667.74 1846.89 Q1668.48 1846.95 1669.37 1847.11 L1669.4 1853.13 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1710.33 1833.84 Q1706.07 1841.16 1704 1848.32 Q1701.93 1855.48 1701.93 1862.84 Q1701.93 1870.19 1704 1877.41 Q1706.1 1884.61 1710.33 1891.89 L1705.24 1891.89 Q1700.46 1884.41 1698.08 1877.19 Q1695.72 1869.96 1695.72 1862.84 Q1695.72 1855.74 1698.08 1848.54 Q1700.43 1841.35 1705.24 1833.84 L1710.33 1833.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1731.85 1839.03 Q1729.3 1839.03 1727.55 1840.81 Q1725.8 1842.56 1725.8 1845.11 Q1725.8 1847.62 1727.55 1849.37 Q1729.3 1851.09 1731.85 1851.09 Q1734.39 1851.09 1736.14 1849.37 Q1737.89 1847.62 1737.89 1845.11 Q1737.89 1842.59 1736.11 1840.81 Q1734.36 1839.03 1731.85 1839.03 M1731.85 1834.92 Q1733.88 1834.92 1735.76 1835.72 Q1737.64 1836.48 1739.01 1837.95 Q1740.47 1839.38 1741.2 1841.19 Q1741.94 1843.01 1741.94 1845.11 Q1741.94 1849.31 1738.98 1852.24 Q1736.05 1855.13 1731.78 1855.13 Q1727.49 1855.13 1724.62 1852.27 Q1721.76 1849.4 1721.76 1845.11 Q1721.76 1840.84 1724.69 1837.88 Q1727.61 1834.92 1731.85 1834.92 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1790.12 1839.44 L1790.12 1846.22 Q1786.88 1843.2 1783.19 1841.7 Q1779.53 1840.21 1775.39 1840.21 Q1767.24 1840.21 1762.91 1845.2 Q1758.58 1850.17 1758.58 1859.59 Q1758.58 1868.98 1762.91 1873.98 Q1767.24 1878.94 1775.39 1878.94 Q1779.53 1878.94 1783.19 1877.44 Q1786.88 1875.95 1790.12 1872.92 L1790.12 1879.64 Q1786.75 1881.93 1782.96 1883.08 Q1779.21 1884.22 1775.01 1884.22 Q1764.22 1884.22 1758.01 1877.64 Q1751.8 1871.02 1751.8 1859.59 Q1751.8 1848.13 1758.01 1841.54 Q1764.22 1834.92 1775.01 1834.92 Q1779.27 1834.92 1783.03 1836.07 Q1786.81 1837.18 1790.12 1839.44 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1810.21 1835.78 L1815.62 1835.78 L1799.07 1889.35 L1793.66 1889.35 L1810.21 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1817.78 1835.78 L1824.28 1835.78 L1834.27 1875.95 L1844.23 1835.78 L1851.46 1835.78 L1861.45 1875.95 L1871.41 1835.78 L1877.94 1835.78 L1866 1883.3 L1857.92 1883.3 L1847.89 1842.05 L1837.77 1883.3 L1829.69 1883.3 L1817.78 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1896.62 1835.78 L1902.03 1835.78 L1885.48 1889.35 L1880.07 1889.35 L1896.62 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1935.93 1854.5 Q1938.13 1850.55 1941.18 1848.67 Q1944.24 1846.79 1948.38 1846.79 Q1953.95 1846.79 1956.97 1850.71 Q1959.99 1854.59 1959.99 1861.78 L1959.99 1883.3 L1954.11 1883.3 L1954.11 1861.98 Q1954.11 1856.85 1952.29 1854.37 Q1950.48 1851.89 1946.75 1851.89 Q1942.2 1851.89 1939.56 1854.91 Q1936.92 1857.93 1936.92 1863.15 L1936.92 1883.3 L1931.03 1883.3 L1931.03 1861.98 Q1931.03 1856.82 1929.22 1854.37 Q1927.4 1851.89 1923.61 1851.89 Q1919.13 1851.89 1916.48 1854.94 Q1913.84 1857.97 1913.84 1863.15 L1913.84 1883.3 L1907.95 1883.3 L1907.95 1847.65 L1913.84 1847.65 L1913.84 1853.19 Q1915.85 1849.91 1918.65 1848.35 Q1921.45 1846.79 1925.3 1846.79 Q1929.18 1846.79 1931.89 1848.77 Q1934.63 1850.74 1935.93 1854.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1974.06 1858.41 L1987.56 1858.41 L1987.56 1862.04 L1968.52 1862.04 L1968.52 1858.54 Q1969.61 1857.55 1971.61 1855.77 Q1982.56 1846.06 1982.56 1843.07 Q1982.56 1840.97 1980.9 1839.7 Q1979.25 1838.39 1976.54 1838.39 Q1974.89 1838.39 1972.95 1838.96 Q1971.01 1839.5 1968.71 1840.62 L1968.71 1836.7 Q1971.17 1835.81 1973.27 1835.37 Q1975.4 1834.92 1977.21 1834.92 Q1981.83 1834.92 1984.6 1837.02 Q1987.37 1839.12 1987.37 1842.56 Q1987.37 1846.98 1976.83 1856.02 Q1975.05 1857.55 1974.06 1858.41 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M1996.88 1833.84 L2001.98 1833.84 Q2006.75 1841.35 2009.1 1848.54 Q2011.49 1855.74 2011.49 1862.84 Q2011.49 1869.96 2009.1 1877.19 Q2006.75 1884.41 2001.98 1891.89 L1996.88 1891.89 Q2001.12 1884.61 2003.18 1877.41 Q2005.29 1870.19 2005.29 1862.84 Q2005.29 1855.48 2003.18 1848.32 Q2001.12 1841.16 1996.88 1833.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="241.557,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="241.557,1437.13 2640.76,1437.13 "/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="241.557,1140.15 2640.76,1140.15 "/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="241.557,843.163 2640.76,843.163 "/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="241.557,546.178 2640.76,546.178 "/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="241.557,249.194 2640.76,249.194 "/>
<polyline clip-path="url(#clip940)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,1734.12 241.557,47.2441 "/>
<polyline clip-path="url(#clip940)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,1734.12 260.455,1734.12 "/>
<polyline clip-path="url(#clip940)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,1437.13 260.455,1437.13 "/>
<polyline clip-path="url(#clip940)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,1140.15 260.455,1140.15 "/>
<polyline clip-path="url(#clip940)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,843.163 260.455,843.163 "/>
<polyline clip-path="url(#clip940)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,546.178 260.455,546.178 "/>
<polyline clip-path="url(#clip940)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,249.194 260.455,249.194 "/>
<path clip-path="url(#clip940)" d="M189.293 1719.92 Q185.682 1719.92 183.853 1723.48 Q182.047 1727.02 182.047 1734.15 Q182.047 1741.26 183.853 1744.82 Q185.682 1748.36 189.293 1748.36 Q192.927 1748.36 194.733 1744.82 Q196.561 1741.26 196.561 1734.15 Q196.561 1727.02 194.733 1723.48 Q192.927 1719.92 189.293 1719.92 M189.293 1716.21 Q195.103 1716.21 198.158 1720.82 Q201.237 1725.4 201.237 1734.15 Q201.237 1742.88 198.158 1747.48 Q195.103 1752.07 189.293 1752.07 Q183.483 1752.07 180.404 1747.48 Q177.348 1742.88 177.348 1734.15 Q177.348 1725.4 180.404 1720.82 Q183.483 1716.21 189.293 1716.21 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M119.779 1450.48 L127.418 1450.48 L127.418 1424.11 L119.108 1425.78 L119.108 1421.52 L127.372 1419.85 L132.048 1419.85 L132.048 1450.48 L139.687 1450.48 L139.687 1454.41 L119.779 1454.41 L119.779 1450.48 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M159.131 1422.93 Q155.52 1422.93 153.691 1426.5 Q151.886 1430.04 151.886 1437.17 Q151.886 1444.27 153.691 1447.84 Q155.52 1451.38 159.131 1451.38 Q162.765 1451.38 164.571 1447.84 Q166.399 1444.27 166.399 1437.17 Q166.399 1430.04 164.571 1426.5 Q162.765 1422.93 159.131 1422.93 M159.131 1419.23 Q164.941 1419.23 167.997 1423.83 Q171.075 1428.42 171.075 1437.17 Q171.075 1445.89 167.997 1450.5 Q164.941 1455.08 159.131 1455.08 Q153.321 1455.08 150.242 1450.5 Q147.186 1445.89 147.186 1437.17 Q147.186 1428.42 150.242 1423.83 Q153.321 1419.23 159.131 1419.23 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M189.293 1422.93 Q185.682 1422.93 183.853 1426.5 Q182.047 1430.04 182.047 1437.17 Q182.047 1444.27 183.853 1447.84 Q185.682 1451.38 189.293 1451.38 Q192.927 1451.38 194.733 1447.84 Q196.561 1444.27 196.561 1437.17 Q196.561 1430.04 194.733 1426.5 Q192.927 1422.93 189.293 1422.93 M189.293 1419.23 Q195.103 1419.23 198.158 1423.83 Q201.237 1428.42 201.237 1437.17 Q201.237 1445.89 198.158 1450.5 Q195.103 1455.08 189.293 1455.08 Q183.483 1455.08 180.404 1450.5 Q177.348 1445.89 177.348 1437.17 Q177.348 1428.42 180.404 1423.83 Q183.483 1419.23 189.293 1419.23 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M122.997 1153.49 L139.316 1153.49 L139.316 1157.43 L117.372 1157.43 L117.372 1153.49 Q120.034 1150.74 124.617 1146.11 Q129.224 1141.46 130.404 1140.11 Q132.65 1137.59 133.529 1135.85 Q134.432 1134.09 134.432 1132.4 Q134.432 1129.65 132.488 1127.91 Q130.566 1126.18 127.464 1126.18 Q125.265 1126.18 122.812 1126.94 Q120.381 1127.71 117.603 1129.26 L117.603 1124.53 Q120.427 1123.4 122.881 1122.82 Q125.335 1122.24 127.372 1122.24 Q132.742 1122.24 135.937 1124.93 Q139.131 1127.61 139.131 1132.1 Q139.131 1134.23 138.321 1136.15 Q137.534 1138.05 135.427 1140.65 Q134.849 1141.32 131.747 1144.53 Q128.645 1147.73 122.997 1153.49 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M159.131 1125.95 Q155.52 1125.95 153.691 1129.51 Q151.886 1133.05 151.886 1140.18 Q151.886 1147.29 153.691 1150.85 Q155.52 1154.4 159.131 1154.4 Q162.765 1154.4 164.571 1150.85 Q166.399 1147.29 166.399 1140.18 Q166.399 1133.05 164.571 1129.51 Q162.765 1125.95 159.131 1125.95 M159.131 1122.24 Q164.941 1122.24 167.997 1126.85 Q171.075 1131.43 171.075 1140.18 Q171.075 1148.91 167.997 1153.52 Q164.941 1158.1 159.131 1158.1 Q153.321 1158.1 150.242 1153.52 Q147.186 1148.91 147.186 1140.18 Q147.186 1131.43 150.242 1126.85 Q153.321 1122.24 159.131 1122.24 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M189.293 1125.95 Q185.682 1125.95 183.853 1129.51 Q182.047 1133.05 182.047 1140.18 Q182.047 1147.29 183.853 1150.85 Q185.682 1154.4 189.293 1154.4 Q192.927 1154.4 194.733 1150.85 Q196.561 1147.29 196.561 1140.18 Q196.561 1133.05 194.733 1129.51 Q192.927 1125.95 189.293 1125.95 M189.293 1122.24 Q195.103 1122.24 198.158 1126.85 Q201.237 1131.43 201.237 1140.18 Q201.237 1148.91 198.158 1153.52 Q195.103 1158.1 189.293 1158.1 Q183.483 1158.1 180.404 1153.52 Q177.348 1148.91 177.348 1140.18 Q177.348 1131.43 180.404 1126.85 Q183.483 1122.24 189.293 1122.24 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M133.136 841.809 Q136.492 842.526 138.367 844.795 Q140.265 847.063 140.265 850.397 Q140.265 855.512 136.747 858.313 Q133.228 861.114 126.747 861.114 Q124.571 861.114 122.256 860.674 Q119.964 860.258 117.511 859.401 L117.511 854.887 Q119.455 856.022 121.77 856.6 Q124.085 857.179 126.608 857.179 Q131.006 857.179 133.298 855.443 Q135.612 853.707 135.612 850.397 Q135.612 847.341 133.46 845.628 Q131.33 843.892 127.511 843.892 L123.483 843.892 L123.483 840.05 L127.696 840.05 Q131.145 840.05 132.974 838.684 Q134.802 837.295 134.802 834.702 Q134.802 832.04 132.904 830.628 Q131.029 829.193 127.511 829.193 Q125.589 829.193 123.39 829.61 Q121.191 830.026 118.552 830.906 L118.552 826.739 Q121.214 825.999 123.529 825.628 Q125.867 825.258 127.927 825.258 Q133.251 825.258 136.353 827.689 Q139.455 830.096 139.455 834.216 Q139.455 837.087 137.812 839.077 Q136.168 841.045 133.136 841.809 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M159.131 828.962 Q155.52 828.962 153.691 832.526 Q151.886 836.068 151.886 843.198 Q151.886 850.304 153.691 853.869 Q155.52 857.411 159.131 857.411 Q162.765 857.411 164.571 853.869 Q166.399 850.304 166.399 843.198 Q166.399 836.068 164.571 832.526 Q162.765 828.962 159.131 828.962 M159.131 825.258 Q164.941 825.258 167.997 829.864 Q171.075 834.448 171.075 843.198 Q171.075 851.925 167.997 856.531 Q164.941 861.114 159.131 861.114 Q153.321 861.114 150.242 856.531 Q147.186 851.925 147.186 843.198 Q147.186 834.448 150.242 829.864 Q153.321 825.258 159.131 825.258 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M189.293 828.962 Q185.682 828.962 183.853 832.526 Q182.047 836.068 182.047 843.198 Q182.047 850.304 183.853 853.869 Q185.682 857.411 189.293 857.411 Q192.927 857.411 194.733 853.869 Q196.561 850.304 196.561 843.198 Q196.561 836.068 194.733 832.526 Q192.927 828.962 189.293 828.962 M189.293 825.258 Q195.103 825.258 198.158 829.864 Q201.237 834.448 201.237 843.198 Q201.237 851.925 198.158 856.531 Q195.103 861.114 189.293 861.114 Q183.483 861.114 180.404 856.531 Q177.348 851.925 177.348 843.198 Q177.348 834.448 180.404 829.864 Q183.483 825.258 189.293 825.258 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M131.816 532.972 L120.011 551.421 L131.816 551.421 L131.816 532.972 M130.589 528.898 L136.469 528.898 L136.469 551.421 L141.399 551.421 L141.399 555.31 L136.469 555.31 L136.469 563.458 L131.816 563.458 L131.816 555.31 L116.214 555.31 L116.214 550.796 L130.589 528.898 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M159.131 531.977 Q155.52 531.977 153.691 535.542 Q151.886 539.083 151.886 546.213 Q151.886 553.319 153.691 556.884 Q155.52 560.426 159.131 560.426 Q162.765 560.426 164.571 556.884 Q166.399 553.319 166.399 546.213 Q166.399 539.083 164.571 535.542 Q162.765 531.977 159.131 531.977 M159.131 528.273 Q164.941 528.273 167.997 532.88 Q171.075 537.463 171.075 546.213 Q171.075 554.94 167.997 559.546 Q164.941 564.13 159.131 564.13 Q153.321 564.13 150.242 559.546 Q147.186 554.94 147.186 546.213 Q147.186 537.463 150.242 532.88 Q153.321 528.273 159.131 528.273 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M189.293 531.977 Q185.682 531.977 183.853 535.542 Q182.047 539.083 182.047 546.213 Q182.047 553.319 183.853 556.884 Q185.682 560.426 189.293 560.426 Q192.927 560.426 194.733 556.884 Q196.561 553.319 196.561 546.213 Q196.561 539.083 194.733 535.542 Q192.927 531.977 189.293 531.977 M189.293 528.273 Q195.103 528.273 198.158 532.88 Q201.237 537.463 201.237 546.213 Q201.237 554.94 198.158 559.546 Q195.103 564.13 189.293 564.13 Q183.483 564.13 180.404 559.546 Q177.348 554.94 177.348 546.213 Q177.348 537.463 180.404 532.88 Q183.483 528.273 189.293 528.273 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M119.015 231.914 L137.372 231.914 L137.372 235.849 L123.298 235.849 L123.298 244.321 Q124.316 243.974 125.335 243.812 Q126.353 243.627 127.372 243.627 Q133.159 243.627 136.538 246.798 Q139.918 249.969 139.918 255.386 Q139.918 260.964 136.446 264.066 Q132.974 267.145 126.654 267.145 Q124.478 267.145 122.21 266.775 Q119.964 266.404 117.557 265.663 L117.557 260.964 Q119.64 262.099 121.863 262.654 Q124.085 263.21 126.562 263.21 Q130.566 263.21 132.904 261.103 Q135.242 258.997 135.242 255.386 Q135.242 251.775 132.904 249.668 Q130.566 247.562 126.562 247.562 Q124.687 247.562 122.812 247.978 Q120.96 248.395 119.015 249.275 L119.015 231.914 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M159.131 234.992 Q155.52 234.992 153.691 238.557 Q151.886 242.099 151.886 249.228 Q151.886 256.335 153.691 259.9 Q155.52 263.441 159.131 263.441 Q162.765 263.441 164.571 259.9 Q166.399 256.335 166.399 249.228 Q166.399 242.099 164.571 238.557 Q162.765 234.992 159.131 234.992 M159.131 231.289 Q164.941 231.289 167.997 235.895 Q171.075 240.478 171.075 249.228 Q171.075 257.955 167.997 262.562 Q164.941 267.145 159.131 267.145 Q153.321 267.145 150.242 262.562 Q147.186 257.955 147.186 249.228 Q147.186 240.478 150.242 235.895 Q153.321 231.289 159.131 231.289 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M189.293 234.992 Q185.682 234.992 183.853 238.557 Q182.047 242.099 182.047 249.228 Q182.047 256.335 183.853 259.9 Q185.682 263.441 189.293 263.441 Q192.927 263.441 194.733 259.9 Q196.561 256.335 196.561 249.228 Q196.561 242.099 194.733 238.557 Q192.927 234.992 189.293 234.992 M189.293 231.289 Q195.103 231.289 198.158 235.895 Q201.237 240.478 201.237 249.228 Q201.237 257.955 198.158 262.562 Q195.103 267.145 189.293 267.145 Q183.483 267.145 180.404 262.562 Q177.348 257.955 177.348 249.228 Q177.348 240.478 180.404 235.895 Q183.483 231.289 189.293 231.289 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M17.7787 946.54 L24.5582 946.54 Q21.5345 949.786 20.0386 953.478 Q18.5426 957.139 18.5426 961.276 Q18.5426 969.424 23.5397 973.753 Q28.5049 978.082 37.9262 978.082 Q47.3156 978.082 52.3127 973.753 Q57.2779 969.424 57.2779 961.276 Q57.2779 957.139 55.782 953.478 Q54.2861 949.786 51.2623 946.54 L57.9782 946.54 Q60.2698 949.913 61.4156 953.701 Q62.5615 957.457 62.5615 961.658 Q62.5615 972.448 55.973 978.655 Q49.3526 984.861 37.9262 984.861 Q26.4679 984.861 19.8794 978.655 Q13.2591 972.448 13.2591 961.658 Q13.2591 957.393 14.4049 953.637 Q15.5189 949.85 17.7787 946.54 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M30.0964 923.05 Q30.0964 927.761 33.7885 930.498 Q37.4488 933.235 43.8463 933.235 Q50.2438 933.235 53.9359 930.53 Q57.5962 927.793 57.5962 923.05 Q57.5962 918.371 53.9041 915.634 Q50.212 912.897 43.8463 912.897 Q37.5124 912.897 33.8203 915.634 Q30.0964 918.371 30.0964 923.05 M25.1311 923.05 Q25.1311 915.411 30.0964 911.051 Q35.0616 906.69 43.8463 906.69 Q52.5991 906.69 57.5962 911.051 Q62.5615 915.411 62.5615 923.05 Q62.5615 930.721 57.5962 935.081 Q52.5991 939.41 43.8463 939.41 Q35.0616 939.41 30.0964 935.081 Q25.1311 930.721 25.1311 923.05 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M47.5702 897.587 L25.9905 897.587 L25.9905 891.731 L47.3474 891.731 Q52.4082 891.731 54.9545 889.758 Q57.4689 887.784 57.4689 883.837 Q57.4689 879.095 54.4452 876.358 Q51.4215 873.589 46.2016 873.589 L25.9905 873.589 L25.9905 867.732 L61.6384 867.732 L61.6384 873.589 L56.1639 873.589 Q59.4104 875.721 61.0019 878.554 Q62.5615 881.355 62.5615 885.079 Q62.5615 891.222 58.742 894.404 Q54.9226 897.587 47.5702 897.587 M25.1311 882.851 L25.1311 882.851 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M40.1224 826.037 L61.6384 826.037 L61.6384 831.893 L40.3133 831.893 Q35.2526 831.893 32.7381 833.867 Q30.2237 835.84 30.2237 839.787 Q30.2237 844.529 33.2474 847.266 Q36.2711 850.004 41.491 850.004 L61.6384 850.004 L61.6384 855.892 L25.9905 855.892 L25.9905 850.004 L31.5287 850.004 Q28.314 847.903 26.7225 845.07 Q25.1311 842.206 25.1311 838.482 Q25.1311 832.339 28.9505 829.188 Q32.7381 826.037 40.1224 826.037 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip940)" d="M15.869 808.563 L25.9905 808.563 L25.9905 796.5 L30.542 796.5 L30.542 808.563 L49.8937 808.563 Q54.2542 808.563 55.4955 807.385 Q56.7368 806.176 56.7368 802.516 L56.7368 796.5 L61.6384 796.5 L61.6384 802.516 Q61.6384 809.295 59.124 811.873 Q56.5777 814.451 49.8937 814.451 L30.542 814.451 L30.542 818.748 L25.9905 818.748 L25.9905 814.451 L15.869 814.451 L15.869 808.563 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip942)" d="M373.517 1728.18 L373.517 1734.12 L405.387 1734.12 L405.387 1728.18 L373.517 1728.18 L373.517 1728.18  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="373.517,1728.18 373.517,1734.12 405.387,1734.12 405.387,1728.18 373.517,1728.18 "/>
<path clip-path="url(#clip942)" d="M405.387 1731.15 L405.387 1734.12 L437.257 1734.12 L437.257 1731.15 L405.387 1731.15 L405.387 1731.15  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="405.387,1731.15 405.387,1734.12 437.257,1734.12 437.257,1731.15 405.387,1731.15 "/>
<path clip-path="url(#clip942)" d="M437.257 1716.3 L437.257 1734.12 L469.127 1734.12 L469.127 1716.3 L437.257 1716.3 L437.257 1716.3  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="437.257,1716.3 437.257,1734.12 469.127,1734.12 469.127,1716.3 437.257,1716.3 "/>
<path clip-path="url(#clip942)" d="M469.127 1689.57 L469.127 1734.12 L500.997 1734.12 L500.997 1689.57 L469.127 1689.57 L469.127 1689.57  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="469.127,1689.57 469.127,1734.12 500.997,1734.12 500.997,1689.57 469.127,1689.57 "/>
<path clip-path="url(#clip942)" d="M500.997 1665.81 L500.997 1734.12 L532.866 1734.12 L532.866 1665.81 L500.997 1665.81 L500.997 1665.81  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="500.997,1665.81 500.997,1734.12 532.866,1734.12 532.866,1665.81 500.997,1665.81 "/>
<path clip-path="url(#clip942)" d="M532.866 1591.56 L532.866 1734.12 L564.736 1734.12 L564.736 1591.56 L532.866 1591.56 L532.866 1591.56  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="532.866,1591.56 532.866,1734.12 564.736,1734.12 564.736,1591.56 532.866,1591.56 "/>
<path clip-path="url(#clip942)" d="M564.736 1517.32 L564.736 1734.12 L596.606 1734.12 L596.606 1517.32 L564.736 1517.32 L564.736 1517.32  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="564.736,1517.32 564.736,1734.12 596.606,1734.12 596.606,1517.32 564.736,1517.32 "/>
<path clip-path="url(#clip942)" d="M596.606 1434.16 L596.606 1734.12 L628.476 1734.12 L628.476 1434.16 L596.606 1434.16 L596.606 1434.16  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="596.606,1434.16 596.606,1734.12 628.476,1734.12 628.476,1434.16 596.606,1434.16 "/>
<path clip-path="url(#clip942)" d="M628.476 1312.4 L628.476 1734.12 L660.346 1734.12 L660.346 1312.4 L628.476 1312.4 L628.476 1312.4  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="628.476,1312.4 628.476,1734.12 660.346,1734.12 660.346,1312.4 628.476,1312.4 "/>
<path clip-path="url(#clip942)" d="M660.346 1226.27 L660.346 1734.12 L692.216 1734.12 L692.216 1226.27 L660.346 1226.27 L660.346 1226.27  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="660.346,1226.27 660.346,1734.12 692.216,1734.12 692.216,1226.27 660.346,1226.27 "/>
<path clip-path="url(#clip942)" d="M692.216 947.108 L692.216 1734.12 L724.085 1734.12 L724.085 947.108 L692.216 947.108 L692.216 947.108  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="692.216,947.108 692.216,1734.12 724.085,1734.12 724.085,947.108 692.216,947.108 "/>
<path clip-path="url(#clip942)" d="M724.085 751.098 L724.085 1734.12 L755.955 1734.12 L755.955 751.098 L724.085 751.098 L724.085 751.098  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="724.085,751.098 724.085,1734.12 755.955,1734.12 755.955,751.098 724.085,751.098 "/>
<path clip-path="url(#clip942)" d="M755.955 700.61 L755.955 1734.12 L787.825 1734.12 L787.825 700.61 L755.955 700.61 L755.955 700.61  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="755.955,700.61 755.955,1734.12 787.825,1734.12 787.825,700.61 755.955,700.61 "/>
<path clip-path="url(#clip942)" d="M787.825 563.997 L787.825 1734.12 L819.695 1734.12 L819.695 563.997 L787.825 563.997 L787.825 563.997  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="787.825,563.997 787.825,1734.12 819.695,1734.12 819.695,563.997 787.825,563.997 "/>
<path clip-path="url(#clip942)" d="M819.695 385.807 L819.695 1734.12 L851.565 1734.12 L851.565 385.807 L819.695 385.807 L819.695 385.807  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="819.695,385.807 819.695,1734.12 851.565,1734.12 851.565,385.807 819.695,385.807 "/>
<path clip-path="url(#clip942)" d="M851.565 243.254 L851.565 1734.12 L883.435 1734.12 L883.435 243.254 L851.565 243.254 L851.565 243.254  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="851.565,243.254 851.565,1734.12 883.435,1734.12 883.435,243.254 851.565,243.254 "/>
<path clip-path="url(#clip942)" d="M883.435 207.616 L883.435 1734.12 L915.304 1734.12 L915.304 207.616 L883.435 207.616 L883.435 207.616  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="883.435,207.616 883.435,1734.12 915.304,1734.12 915.304,207.616 883.435,207.616 "/>
<path clip-path="url(#clip942)" d="M915.304 145.249 L915.304 1734.12 L947.174 1734.12 L947.174 145.249 L915.304 145.249 L915.304 145.249  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="915.304,145.249 915.304,1734.12 947.174,1734.12 947.174,145.249 915.304,145.249 "/>
<path clip-path="url(#clip942)" d="M947.174 76.9426 L947.174 1734.12 L979.044 1734.12 L979.044 76.9426 L947.174 76.9426 L947.174 76.9426  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="947.174,76.9426 947.174,1734.12 979.044,1734.12 979.044,76.9426 947.174,76.9426 "/>
<path clip-path="url(#clip942)" d="M979.044 71.0029 L979.044 1734.12 L1010.91 1734.12 L1010.91 71.0029 L979.044 71.0029 L979.044 71.0029  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="979.044,71.0029 979.044,1734.12 1010.91,1734.12 1010.91,71.0029 979.044,71.0029 "/>
<path clip-path="url(#clip942)" d="M1010.91 47.2441 L1010.91 1734.12 L1042.78 1734.12 L1042.78 47.2441 L1010.91 47.2441 L1010.91 47.2441  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1010.91,47.2441 1010.91,1734.12 1042.78,1734.12 1042.78,47.2441 1010.91,47.2441 "/>
<path clip-path="url(#clip942)" d="M1042.78 171.978 L1042.78 1734.12 L1074.65 1734.12 L1074.65 171.978 L1042.78 171.978 L1042.78 171.978  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1042.78,171.978 1042.78,1734.12 1074.65,1734.12 1074.65,171.978 1042.78,171.978 "/>
<path clip-path="url(#clip942)" d="M1074.65 275.922 L1074.65 1734.12 L1106.52 1734.12 L1106.52 275.922 L1074.65 275.922 L1074.65 275.922  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1074.65,275.922 1074.65,1734.12 1106.52,1734.12 1106.52,275.922 1074.65,275.922 "/>
<path clip-path="url(#clip942)" d="M1106.52 317.5 L1106.52 1734.12 L1138.39 1734.12 L1138.39 317.5 L1106.52 317.5 L1106.52 317.5  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1106.52,317.5 1106.52,1734.12 1138.39,1734.12 1138.39,317.5 1106.52,317.5 "/>
<path clip-path="url(#clip942)" d="M1138.39 537.269 L1138.39 1734.12 L1170.26 1734.12 L1170.26 537.269 L1138.39 537.269 L1138.39 537.269  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1138.39,537.269 1138.39,1734.12 1170.26,1734.12 1170.26,537.269 1138.39,537.269 "/>
<path clip-path="url(#clip942)" d="M1170.26 739.218 L1170.26 1734.12 L1202.13 1734.12 L1202.13 739.218 L1170.26 739.218 L1170.26 739.218  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1170.26,739.218 1170.26,1734.12 1202.13,1734.12 1202.13,739.218 1170.26,739.218 "/>
<path clip-path="url(#clip942)" d="M1202.13 754.068 L1202.13 1734.12 L1234 1734.12 L1234 754.068 L1202.13 754.068 L1202.13 754.068  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1202.13,754.068 1202.13,1734.12 1234,1734.12 1234,754.068 1202.13,754.068 "/>
<path clip-path="url(#clip942)" d="M1234 932.258 L1234 1734.12 L1265.87 1734.12 L1265.87 932.258 L1234 932.258 L1234 932.258  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1234,932.258 1234,1734.12 1265.87,1734.12 1265.87,932.258 1234,932.258 "/>
<path clip-path="url(#clip942)" d="M1265.87 947.108 L1265.87 1734.12 L1297.74 1734.12 L1297.74 947.108 L1265.87 947.108 L1265.87 947.108  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1265.87,947.108 1265.87,1734.12 1297.74,1734.12 1297.74,947.108 1265.87,947.108 "/>
<path clip-path="url(#clip942)" d="M1297.74 1095.6 L1297.74 1734.12 L1329.61 1734.12 L1329.61 1095.6 L1297.74 1095.6 L1297.74 1095.6  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1297.74,1095.6 1297.74,1734.12 1329.61,1734.12 1329.61,1095.6 1297.74,1095.6 "/>
<path clip-path="url(#clip942)" d="M1329.61 1214.39 L1329.61 1734.12 L1361.48 1734.12 L1361.48 1214.39 L1329.61 1214.39 L1329.61 1214.39  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1329.61,1214.39 1329.61,1734.12 1361.48,1734.12 1361.48,1214.39 1329.61,1214.39 "/>
<path clip-path="url(#clip942)" d="M1361.48 1273.79 L1361.48 1734.12 L1393.35 1734.12 L1393.35 1273.79 L1361.48 1273.79 L1361.48 1273.79  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1361.48,1273.79 1361.48,1734.12 1393.35,1734.12 1393.35,1273.79 1361.48,1273.79 "/>
<path clip-path="url(#clip942)" d="M1393.35 1315.37 L1393.35 1734.12 L1425.22 1734.12 L1425.22 1315.37 L1393.35 1315.37 L1393.35 1315.37  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1393.35,1315.37 1393.35,1734.12 1425.22,1734.12 1425.22,1315.37 1393.35,1315.37 "/>
<path clip-path="url(#clip942)" d="M1425.22 1443.07 L1425.22 1734.12 L1457.09 1734.12 L1457.09 1443.07 L1425.22 1443.07 L1425.22 1443.07  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1425.22,1443.07 1425.22,1734.12 1457.09,1734.12 1457.09,1443.07 1425.22,1443.07 "/>
<path clip-path="url(#clip942)" d="M1457.09 1440.1 L1457.09 1734.12 L1488.96 1734.12 L1488.96 1440.1 L1457.09 1440.1 L1457.09 1440.1  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1457.09,1440.1 1457.09,1734.12 1488.96,1734.12 1488.96,1440.1 1457.09,1440.1 "/>
<path clip-path="url(#clip942)" d="M1488.96 1499.5 L1488.96 1734.12 L1520.83 1734.12 L1520.83 1499.5 L1488.96 1499.5 L1488.96 1499.5  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1488.96,1499.5 1488.96,1734.12 1520.83,1734.12 1520.83,1499.5 1488.96,1499.5 "/>
<path clip-path="url(#clip942)" d="M1520.83 1520.29 L1520.83 1734.12 L1552.7 1734.12 L1552.7 1520.29 L1520.83 1520.29 L1520.83 1520.29  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1520.83,1520.29 1520.83,1734.12 1552.7,1734.12 1552.7,1520.29 1520.83,1520.29 "/>
<path clip-path="url(#clip942)" d="M1552.7 1552.96 L1552.7 1734.12 L1584.57 1734.12 L1584.57 1552.96 L1552.7 1552.96 L1552.7 1552.96  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1552.7,1552.96 1552.7,1734.12 1584.57,1734.12 1584.57,1552.96 1552.7,1552.96 "/>
<path clip-path="url(#clip942)" d="M1584.57 1618.29 L1584.57 1734.12 L1616.44 1734.12 L1616.44 1618.29 L1584.57 1618.29 L1584.57 1618.29  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1584.57,1618.29 1584.57,1734.12 1616.44,1734.12 1616.44,1618.29 1584.57,1618.29 "/>
<path clip-path="url(#clip942)" d="M1616.44 1624.23 L1616.44 1734.12 L1648.31 1734.12 L1648.31 1624.23 L1616.44 1624.23 L1616.44 1624.23  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1616.44,1624.23 1616.44,1734.12 1648.31,1734.12 1648.31,1624.23 1616.44,1624.23 "/>
<path clip-path="url(#clip942)" d="M1648.31 1662.84 L1648.31 1734.12 L1680.18 1734.12 L1680.18 1662.84 L1648.31 1662.84 L1648.31 1662.84  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1648.31,1662.84 1648.31,1734.12 1680.18,1734.12 1680.18,1662.84 1648.31,1662.84 "/>
<path clip-path="url(#clip942)" d="M1680.18 1680.66 L1680.18 1734.12 L1712.05 1734.12 L1712.05 1680.66 L1680.18 1680.66 L1680.18 1680.66  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1680.18,1680.66 1680.18,1734.12 1712.05,1734.12 1712.05,1680.66 1680.18,1680.66 "/>
<path clip-path="url(#clip942)" d="M1712.05 1653.93 L1712.05 1734.12 L1743.92 1734.12 L1743.92 1653.93 L1712.05 1653.93 L1712.05 1653.93  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1712.05,1653.93 1712.05,1734.12 1743.92,1734.12 1743.92,1653.93 1712.05,1653.93 "/>
<path clip-path="url(#clip942)" d="M1743.92 1704.42 L1743.92 1734.12 L1775.79 1734.12 L1775.79 1704.42 L1743.92 1704.42 L1743.92 1704.42  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1743.92,1704.42 1743.92,1734.12 1775.79,1734.12 1775.79,1704.42 1743.92,1704.42 "/>
<path clip-path="url(#clip942)" d="M1775.79 1707.39 L1775.79 1734.12 L1807.66 1734.12 L1807.66 1707.39 L1775.79 1707.39 L1775.79 1707.39  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1775.79,1707.39 1775.79,1734.12 1807.66,1734.12 1807.66,1707.39 1775.79,1707.39 "/>
<path clip-path="url(#clip942)" d="M1807.66 1710.36 L1807.66 1734.12 L1839.53 1734.12 L1839.53 1710.36 L1807.66 1710.36 L1807.66 1710.36  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1807.66,1710.36 1807.66,1734.12 1839.53,1734.12 1839.53,1710.36 1807.66,1710.36 "/>
<path clip-path="url(#clip942)" d="M1839.53 1713.33 L1839.53 1734.12 L1871.4 1734.12 L1871.4 1713.33 L1839.53 1713.33 L1839.53 1713.33  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1839.53,1713.33 1839.53,1734.12 1871.4,1734.12 1871.4,1713.33 1839.53,1713.33 "/>
<path clip-path="url(#clip942)" d="M1871.4 1728.18 L1871.4 1734.12 L1903.27 1734.12 L1903.27 1728.18 L1871.4 1728.18 L1871.4 1728.18  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1871.4,1728.18 1871.4,1734.12 1903.27,1734.12 1903.27,1728.18 1871.4,1728.18 "/>
<path clip-path="url(#clip942)" d="M1903.27 1728.18 L1903.27 1734.12 L1935.14 1734.12 L1935.14 1728.18 L1903.27 1728.18 L1903.27 1728.18  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1903.27,1728.18 1903.27,1734.12 1935.14,1734.12 1935.14,1728.18 1903.27,1728.18 "/>
<path clip-path="url(#clip942)" d="M1935.14 1716.3 L1935.14 1734.12 L1967.01 1734.12 L1967.01 1716.3 L1935.14 1716.3 L1935.14 1716.3  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1935.14,1716.3 1935.14,1734.12 1967.01,1734.12 1967.01,1716.3 1935.14,1716.3 "/>
<path clip-path="url(#clip942)" d="M1967.01 1731.15 L1967.01 1734.12 L1998.88 1734.12 L1998.88 1731.15 L1967.01 1731.15 L1967.01 1731.15  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1967.01,1731.15 1967.01,1734.12 1998.88,1734.12 1998.88,1731.15 1967.01,1731.15 "/>
<path clip-path="url(#clip942)" d="M1998.88 1728.18 L1998.88 1734.12 L2030.75 1734.12 L2030.75 1728.18 L1998.88 1728.18 L1998.88 1728.18  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1998.88,1728.18 1998.88,1734.12 2030.75,1734.12 2030.75,1728.18 1998.88,1728.18 "/>
<path clip-path="url(#clip942)" d="M2030.75 1728.18 L2030.75 1734.12 L2062.62 1734.12 L2062.62 1728.18 L2030.75 1728.18 L2030.75 1728.18  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2030.75,1728.18 2030.75,1734.12 2062.62,1734.12 2062.62,1728.18 2030.75,1728.18 "/>
<path clip-path="url(#clip942)" d="M2062.62 1734.12 L2062.62 1734.12 L2094.49 1734.12 L2094.49 1734.12 L2062.62 1734.12 L2062.62 1734.12  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2062.62,1734.12 2062.62,1734.12 2094.49,1734.12 2062.62,1734.12 "/>
<path clip-path="url(#clip942)" d="M2094.49 1731.15 L2094.49 1734.12 L2126.36 1734.12 L2126.36 1731.15 L2094.49 1731.15 L2094.49 1731.15  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2094.49,1731.15 2094.49,1734.12 2126.36,1734.12 2126.36,1731.15 2094.49,1731.15 "/>
<path clip-path="url(#clip942)" d="M2126.36 1734.12 L2126.36 1734.12 L2158.23 1734.12 L2158.23 1734.12 L2126.36 1734.12 L2126.36 1734.12  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2126.36,1734.12 2126.36,1734.12 2158.23,1734.12 2126.36,1734.12 "/>
<path clip-path="url(#clip942)" d="M2158.23 1734.12 L2158.23 1734.12 L2190.1 1734.12 L2190.1 1734.12 L2158.23 1734.12 L2158.23 1734.12  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2158.23,1734.12 2158.23,1734.12 2190.1,1734.12 2158.23,1734.12 "/>
<path clip-path="url(#clip942)" d="M2190.1 1734.12 L2190.1 1734.12 L2221.97 1734.12 L2221.97 1734.12 L2190.1 1734.12 L2190.1 1734.12  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2190.1,1734.12 2190.1,1734.12 2221.97,1734.12 2190.1,1734.12 "/>
<path clip-path="url(#clip942)" d="M2221.97 1734.12 L2221.97 1734.12 L2253.84 1734.12 L2253.84 1734.12 L2221.97 1734.12 L2221.97 1734.12  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2221.97,1734.12 2221.97,1734.12 2253.84,1734.12 2221.97,1734.12 "/>
<path clip-path="url(#clip942)" d="M2253.84 1731.15 L2253.84 1734.12 L2285.71 1734.12 L2285.71 1731.15 L2253.84 1731.15 L2253.84 1731.15  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2253.84,1731.15 2253.84,1734.12 2285.71,1734.12 2285.71,1731.15 2253.84,1731.15 "/>
<path clip-path="url(#clip942)" d="M2285.71 1734.12 L2285.71 1734.12 L2317.58 1734.12 L2317.58 1734.12 L2285.71 1734.12 L2285.71 1734.12  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2285.71,1734.12 2285.71,1734.12 2317.58,1734.12 2285.71,1734.12 "/>
<path clip-path="url(#clip942)" d="M2317.58 1734.12 L2317.58 1734.12 L2349.45 1734.12 L2349.45 1734.12 L2317.58 1734.12 L2317.58 1734.12  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2317.58,1734.12 2317.58,1734.12 2349.45,1734.12 2317.58,1734.12 "/>
<path clip-path="url(#clip942)" d="M2349.45 1734.12 L2349.45 1734.12 L2381.32 1734.12 L2381.32 1734.12 L2349.45 1734.12 L2349.45 1734.12  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2349.45,1734.12 2349.45,1734.12 2381.32,1734.12 2349.45,1734.12 "/>
<path clip-path="url(#clip942)" d="M2381.32 1734.12 L2381.32 1734.12 L2413.19 1734.12 L2413.19 1734.12 L2381.32 1734.12 L2381.32 1734.12  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2381.32,1734.12 2381.32,1734.12 2413.19,1734.12 2381.32,1734.12 "/>
<path clip-path="url(#clip942)" d="M2413.19 1734.12 L2413.19 1734.12 L2445.06 1734.12 L2445.06 1734.12 L2413.19 1734.12 L2413.19 1734.12  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2413.19,1734.12 2413.19,1734.12 2445.06,1734.12 2413.19,1734.12 "/>
<path clip-path="url(#clip942)" d="M2445.06 1734.12 L2445.06 1734.12 L2476.93 1734.12 L2476.93 1734.12 L2445.06 1734.12 L2445.06 1734.12  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2445.06,1734.12 2445.06,1734.12 2476.93,1734.12 2445.06,1734.12 "/>
<path clip-path="url(#clip942)" d="M2476.93 1731.15 L2476.93 1734.12 L2508.8 1734.12 L2508.8 1731.15 L2476.93 1731.15 L2476.93 1731.15  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip942)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2476.93,1731.15 2476.93,1734.12 2508.8,1734.12 2508.8,1731.15 2476.93,1731.15 "/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="389.452" cy="1728.18" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="421.322" cy="1731.15" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="453.192" cy="1716.3" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="485.062" cy="1689.57" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="516.932" cy="1665.81" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="548.801" cy="1591.56" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="580.671" cy="1517.32" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="612.541" cy="1434.16" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="644.411" cy="1312.4" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="676.281" cy="1226.27" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="708.151" cy="947.108" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="740.02" cy="751.098" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="771.89" cy="700.61" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="803.76" cy="563.997" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="835.63" cy="385.807" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="867.5" cy="243.254" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="899.369" cy="207.616" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="931.239" cy="145.249" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="963.109" cy="76.9426" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="994.979" cy="71.0029" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1058.72" cy="171.978" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1090.59" cy="275.922" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1122.46" cy="317.5" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1154.33" cy="537.269" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1186.2" cy="739.218" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1218.07" cy="754.068" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1249.94" cy="932.258" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1281.81" cy="947.108" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1313.68" cy="1095.6" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1345.55" cy="1214.39" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1377.42" cy="1273.79" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1409.29" cy="1315.37" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1441.16" cy="1443.07" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1473.03" cy="1440.1" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1504.9" cy="1499.5" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1536.77" cy="1520.29" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1568.64" cy="1552.96" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1600.51" cy="1618.29" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1632.38" cy="1624.23" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1664.25" cy="1662.84" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1696.12" cy="1680.66" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1727.98" cy="1653.93" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1759.85" cy="1704.42" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1791.72" cy="1707.39" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1823.59" cy="1710.36" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1855.46" cy="1713.33" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1887.33" cy="1728.18" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1919.2" cy="1728.18" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1951.07" cy="1716.3" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1982.94" cy="1731.15" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2014.81" cy="1728.18" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2046.68" cy="1728.18" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2078.55" cy="1734.12" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2110.42" cy="1731.15" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2142.29" cy="1734.12" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2174.16" cy="1734.12" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2206.03" cy="1734.12" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2237.9" cy="1734.12" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2269.77" cy="1731.15" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2301.64" cy="1734.12" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2333.51" cy="1734.12" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2365.38" cy="1734.12" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2397.25" cy="1734.12" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2429.12" cy="1734.12" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2460.99" cy="1734.12" r="2"/>
<circle clip-path="url(#clip942)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2492.86" cy="1731.15" r="2"/>
<polyline clip-path="url(#clip942)" style="stroke:#ff0000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" stroke-dasharray="16, 10" points="1010.91,3420.99 1010.91,-1639.63 "/>
</svg>

Figure 3: Samples of the climate feedback factor $\lambda$. The red line
is the deterministic value we used in the original simulation above.

1.  Creating a variable for the number of samples which can then be used
    later makes it easier to change the number of samples down the road
    without possibly creating bugs (if *e.g.* we forget to change the
    value everywhere).

Now, simulate:

``` julia
temp_samples = [ebm(t, forcing_non_aerosol, forcing_aerosol, c, d, λ, α, Δt)[idx_2100] for λ in λ_samples]
```

    10000-element Vector{Float64}:
     3.770102072287724
     4.431909209107202
     4.895585018469363
     3.99172560030267
     4.175893731928025
     3.4593489935324855
     4.972144674738669
     3.534907633752246
     5.8917541311702895
     4.672123702256078
     ⋮
     4.760388722290486
     4.348540697641128
     5.983746721565329
     6.971211824249179
     4.215380495318078
     4.764880951209334
     4.719871562009849
     5.580705422663016
     3.512841374031473

1.  We directly get the values of `λ` from `λ_samples` instead of
    looping over the indices (which is slightly slower and makes the
    line longer).

The expected value after this many simulations is the mean, or 4.7, and
the 95% confidence interval is (4.69, 4.72) (using the formula for the
standard error). The confidence interval around the expected value is
pretty tight, but let’s look at how the estimates of the mean and the
confidence interval evolve over the course of the simulations to see
what an “efficient” sample size might have been:

``` julia
# compute running estimate of the expected value and standard deviation
# first, pre-allocate memory for better efficiency
T_est = zeros(n_samples)
T_se = zeros(n_samples)
for i = 1:n_samples
    T_est[i] = mean(temp_samples[1:i])
    if i > 1
        T_se[i] = std(temp_samples[1:i]) / sqrt(i)
    end    
end

p = plot(; xlabel="Number of Samples", ylabel="Global Temperature Anomaly (°C)", legend=false)
plot!(p, 1:n_samples, T_est, ribbon=1.96 * T_se)
ylims!(p, (4.5, 5.0))
```

<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="672" height="480" viewBox="0 0 2688 1920">
<defs>
  <clipPath id="clip000">
    <rect x="0" y="0" width="2688" height="1920"/>
  </clipPath>
</defs>
<path clip-path="url(#clip000)" d="M0 1920 L2688 1920 L2688 -4.26326e-14 L0 -4.26326e-14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip001">
    <rect x="537" y="0" width="1883" height="1883"/>
  </clipPath>
</defs>
<path clip-path="url(#clip000)" d="M226.627 1734.12 L2640.76 1734.12 L2640.76 47.2441 L226.627 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip002">
    <rect x="226" y="47" width="2415" height="1688"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip002)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="294.723,1734.12 294.723,47.2441 "/>
<polyline clip-path="url(#clip002)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="864.15,1734.12 864.15,47.2441 "/>
<polyline clip-path="url(#clip002)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1433.58,1734.12 1433.58,47.2441 "/>
<polyline clip-path="url(#clip002)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2003,1734.12 2003,47.2441 "/>
<polyline clip-path="url(#clip002)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2572.43,1734.12 2572.43,47.2441 "/>
<polyline clip-path="url(#clip000)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip000)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="294.723,1734.12 294.723,1715.22 "/>
<polyline clip-path="url(#clip000)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="864.15,1734.12 864.15,1715.22 "/>
<polyline clip-path="url(#clip000)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1433.58,1734.12 1433.58,1715.22 "/>
<polyline clip-path="url(#clip000)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2003,1734.12 2003,1715.22 "/>
<polyline clip-path="url(#clip000)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2572.43,1734.12 2572.43,1715.22 "/>
<path clip-path="url(#clip000)" d="M294.723 1767.34 Q291.112 1767.34 289.284 1770.9 Q287.478 1774.45 287.478 1781.58 Q287.478 1788.68 289.284 1792.25 Q291.112 1795.79 294.723 1795.79 Q298.358 1795.79 300.163 1792.25 Q301.992 1788.68 301.992 1781.58 Q301.992 1774.45 300.163 1770.9 Q298.358 1767.34 294.723 1767.34 M294.723 1763.64 Q300.533 1763.64 303.589 1768.24 Q306.668 1772.83 306.668 1781.58 Q306.668 1790.3 303.589 1794.91 Q300.533 1799.49 294.723 1799.49 Q288.913 1799.49 285.834 1794.91 Q282.779 1790.3 282.779 1781.58 Q282.779 1772.83 285.834 1768.24 Q288.913 1763.64 294.723 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M812.762 1794.89 L829.081 1794.89 L829.081 1798.82 L807.137 1798.82 L807.137 1794.89 Q809.799 1792.13 814.382 1787.5 Q818.989 1782.85 820.169 1781.51 Q822.414 1778.98 823.294 1777.25 Q824.197 1775.49 824.197 1773.8 Q824.197 1771.04 822.252 1769.31 Q820.331 1767.57 817.229 1767.57 Q815.03 1767.57 812.577 1768.34 Q810.146 1769.1 807.368 1770.65 L807.368 1765.93 Q810.192 1764.79 812.646 1764.21 Q815.1 1763.64 817.137 1763.64 Q822.507 1763.64 825.701 1766.32 Q828.896 1769.01 828.896 1773.5 Q828.896 1775.63 828.086 1777.55 Q827.299 1779.45 825.192 1782.04 Q824.614 1782.71 821.512 1785.93 Q818.41 1789.12 812.762 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M838.942 1764.26 L857.299 1764.26 L857.299 1768.2 L843.225 1768.2 L843.225 1776.67 Q844.243 1776.32 845.262 1776.16 Q846.28 1775.97 847.299 1775.97 Q853.086 1775.97 856.465 1779.15 Q859.845 1782.32 859.845 1787.73 Q859.845 1793.31 856.373 1796.41 Q852.9 1799.49 846.581 1799.49 Q844.405 1799.49 842.137 1799.12 Q839.891 1798.75 837.484 1798.01 L837.484 1793.31 Q839.567 1794.45 841.789 1795 Q844.012 1795.56 846.488 1795.56 Q850.493 1795.56 852.831 1793.45 Q855.169 1791.34 855.169 1787.73 Q855.169 1784.12 852.831 1782.02 Q850.493 1779.91 846.488 1779.91 Q844.613 1779.91 842.738 1780.33 Q840.887 1780.74 838.942 1781.62 L838.942 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M879.058 1767.34 Q875.447 1767.34 873.618 1770.9 Q871.812 1774.45 871.812 1781.58 Q871.812 1788.68 873.618 1792.25 Q875.447 1795.79 879.058 1795.79 Q882.692 1795.79 884.497 1792.25 Q886.326 1788.68 886.326 1781.58 Q886.326 1774.45 884.497 1770.9 Q882.692 1767.34 879.058 1767.34 M879.058 1763.64 Q884.868 1763.64 887.923 1768.24 Q891.002 1772.83 891.002 1781.58 Q891.002 1790.3 887.923 1794.91 Q884.868 1799.49 879.058 1799.49 Q873.248 1799.49 870.169 1794.91 Q867.113 1790.3 867.113 1781.58 Q867.113 1772.83 870.169 1768.24 Q873.248 1763.64 879.058 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M909.22 1767.34 Q905.608 1767.34 903.78 1770.9 Q901.974 1774.45 901.974 1781.58 Q901.974 1788.68 903.78 1792.25 Q905.608 1795.79 909.22 1795.79 Q912.854 1795.79 914.659 1792.25 Q916.488 1788.68 916.488 1781.58 Q916.488 1774.45 914.659 1770.9 Q912.854 1767.34 909.22 1767.34 M909.22 1763.64 Q915.03 1763.64 918.085 1768.24 Q921.164 1772.83 921.164 1781.58 Q921.164 1790.3 918.085 1794.91 Q915.03 1799.49 909.22 1799.49 Q903.409 1799.49 900.331 1794.91 Q897.275 1790.3 897.275 1781.58 Q897.275 1772.83 900.331 1768.24 Q903.409 1763.64 909.22 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1378.11 1764.26 L1396.47 1764.26 L1396.47 1768.2 L1382.4 1768.2 L1382.4 1776.67 Q1383.42 1776.32 1384.43 1776.16 Q1385.45 1775.97 1386.47 1775.97 Q1392.26 1775.97 1395.64 1779.15 Q1399.02 1782.32 1399.02 1787.73 Q1399.02 1793.31 1395.55 1796.41 Q1392.07 1799.49 1385.75 1799.49 Q1383.58 1799.49 1381.31 1799.12 Q1379.06 1798.75 1376.66 1798.01 L1376.66 1793.31 Q1378.74 1794.45 1380.96 1795 Q1383.18 1795.56 1385.66 1795.56 Q1389.67 1795.56 1392 1793.45 Q1394.34 1791.34 1394.34 1787.73 Q1394.34 1784.12 1392 1782.02 Q1389.67 1779.91 1385.66 1779.91 Q1383.79 1779.91 1381.91 1780.33 Q1380.06 1780.74 1378.11 1781.62 L1378.11 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1418.23 1767.34 Q1414.62 1767.34 1412.79 1770.9 Q1410.98 1774.45 1410.98 1781.58 Q1410.98 1788.68 1412.79 1792.25 Q1414.62 1795.79 1418.23 1795.79 Q1421.86 1795.79 1423.67 1792.25 Q1425.5 1788.68 1425.5 1781.58 Q1425.5 1774.45 1423.67 1770.9 Q1421.86 1767.34 1418.23 1767.34 M1418.23 1763.64 Q1424.04 1763.64 1427.1 1768.24 Q1430.17 1772.83 1430.17 1781.58 Q1430.17 1790.3 1427.1 1794.91 Q1424.04 1799.49 1418.23 1799.49 Q1412.42 1799.49 1409.34 1794.91 Q1406.29 1790.3 1406.29 1781.58 Q1406.29 1772.83 1409.34 1768.24 Q1412.42 1763.64 1418.23 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1448.39 1767.34 Q1444.78 1767.34 1442.95 1770.9 Q1441.15 1774.45 1441.15 1781.58 Q1441.15 1788.68 1442.95 1792.25 Q1444.78 1795.79 1448.39 1795.79 Q1452.03 1795.79 1453.83 1792.25 Q1455.66 1788.68 1455.66 1781.58 Q1455.66 1774.45 1453.83 1770.9 Q1452.03 1767.34 1448.39 1767.34 M1448.39 1763.64 Q1454.2 1763.64 1457.26 1768.24 Q1460.34 1772.83 1460.34 1781.58 Q1460.34 1790.3 1457.26 1794.91 Q1454.2 1799.49 1448.39 1799.49 Q1442.58 1799.49 1439.5 1794.91 Q1436.45 1790.3 1436.45 1781.58 Q1436.45 1772.83 1439.5 1768.24 Q1442.58 1763.64 1448.39 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1478.55 1767.34 Q1474.94 1767.34 1473.11 1770.9 Q1471.31 1774.45 1471.31 1781.58 Q1471.31 1788.68 1473.11 1792.25 Q1474.94 1795.79 1478.55 1795.79 Q1482.19 1795.79 1483.99 1792.25 Q1485.82 1788.68 1485.82 1781.58 Q1485.82 1774.45 1483.99 1770.9 Q1482.19 1767.34 1478.55 1767.34 M1478.55 1763.64 Q1484.36 1763.64 1487.42 1768.24 Q1490.5 1772.83 1490.5 1781.58 Q1490.5 1790.3 1487.42 1794.91 Q1484.36 1799.49 1478.55 1799.49 Q1472.74 1799.49 1469.67 1794.91 Q1466.61 1790.3 1466.61 1781.58 Q1466.61 1772.83 1469.67 1768.24 Q1472.74 1763.64 1478.55 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1946.2 1764.26 L1968.42 1764.26 L1968.42 1766.25 L1955.88 1798.82 L1950.99 1798.82 L1962.8 1768.2 L1946.2 1768.2 L1946.2 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1977.59 1764.26 L1995.94 1764.26 L1995.94 1768.2 L1981.87 1768.2 L1981.87 1776.67 Q1982.89 1776.32 1983.91 1776.16 Q1984.93 1775.97 1985.94 1775.97 Q1991.73 1775.97 1995.11 1779.15 Q1998.49 1782.32 1998.49 1787.73 Q1998.49 1793.31 1995.02 1796.41 Q1991.55 1799.49 1985.23 1799.49 Q1983.05 1799.49 1980.78 1799.12 Q1978.54 1798.75 1976.13 1798.01 L1976.13 1793.31 Q1978.21 1794.45 1980.44 1795 Q1982.66 1795.56 1985.13 1795.56 Q1989.14 1795.56 1991.48 1793.45 Q1993.81 1791.34 1993.81 1787.73 Q1993.81 1784.12 1991.48 1782.02 Q1989.14 1779.91 1985.13 1779.91 Q1983.26 1779.91 1981.38 1780.33 Q1979.53 1780.74 1977.59 1781.62 L1977.59 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M2017.7 1767.34 Q2014.09 1767.34 2012.26 1770.9 Q2010.46 1774.45 2010.46 1781.58 Q2010.46 1788.68 2012.26 1792.25 Q2014.09 1795.79 2017.7 1795.79 Q2021.34 1795.79 2023.14 1792.25 Q2024.97 1788.68 2024.97 1781.58 Q2024.97 1774.45 2023.14 1770.9 Q2021.34 1767.34 2017.7 1767.34 M2017.7 1763.64 Q2023.51 1763.64 2026.57 1768.24 Q2029.65 1772.83 2029.65 1781.58 Q2029.65 1790.3 2026.57 1794.91 Q2023.51 1799.49 2017.7 1799.49 Q2011.89 1799.49 2008.81 1794.91 Q2005.76 1790.3 2005.76 1781.58 Q2005.76 1772.83 2008.81 1768.24 Q2011.89 1763.64 2017.7 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M2047.87 1767.34 Q2044.25 1767.34 2042.43 1770.9 Q2040.62 1774.45 2040.62 1781.58 Q2040.62 1788.68 2042.43 1792.25 Q2044.25 1795.79 2047.87 1795.79 Q2051.5 1795.79 2053.31 1792.25 Q2055.13 1788.68 2055.13 1781.58 Q2055.13 1774.45 2053.31 1770.9 Q2051.5 1767.34 2047.87 1767.34 M2047.87 1763.64 Q2053.68 1763.64 2056.73 1768.24 Q2059.81 1772.83 2059.81 1781.58 Q2059.81 1790.3 2056.73 1794.91 Q2053.68 1799.49 2047.87 1799.49 Q2042.06 1799.49 2038.98 1794.91 Q2035.92 1790.3 2035.92 1781.58 Q2035.92 1772.83 2038.98 1768.24 Q2042.06 1763.64 2047.87 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M2501.88 1794.89 L2509.52 1794.89 L2509.52 1768.52 L2501.21 1770.19 L2501.21 1765.93 L2509.47 1764.26 L2514.14 1764.26 L2514.14 1794.89 L2521.78 1794.89 L2521.78 1798.82 L2501.88 1798.82 L2501.88 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M2541.23 1767.34 Q2537.62 1767.34 2535.79 1770.9 Q2533.98 1774.45 2533.98 1781.58 Q2533.98 1788.68 2535.79 1792.25 Q2537.62 1795.79 2541.23 1795.79 Q2544.86 1795.79 2546.67 1792.25 Q2548.5 1788.68 2548.5 1781.58 Q2548.5 1774.45 2546.67 1770.9 Q2544.86 1767.34 2541.23 1767.34 M2541.23 1763.64 Q2547.04 1763.64 2550.09 1768.24 Q2553.17 1772.83 2553.17 1781.58 Q2553.17 1790.3 2550.09 1794.91 Q2547.04 1799.49 2541.23 1799.49 Q2535.42 1799.49 2532.34 1794.91 Q2529.28 1790.3 2529.28 1781.58 Q2529.28 1772.83 2532.34 1768.24 Q2535.42 1763.64 2541.23 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M2571.39 1767.34 Q2567.78 1767.34 2565.95 1770.9 Q2564.14 1774.45 2564.14 1781.58 Q2564.14 1788.68 2565.95 1792.25 Q2567.78 1795.79 2571.39 1795.79 Q2575.02 1795.79 2576.83 1792.25 Q2578.66 1788.68 2578.66 1781.58 Q2578.66 1774.45 2576.83 1770.9 Q2575.02 1767.34 2571.39 1767.34 M2571.39 1763.64 Q2577.2 1763.64 2580.26 1768.24 Q2583.33 1772.83 2583.33 1781.58 Q2583.33 1790.3 2580.26 1794.91 Q2577.2 1799.49 2571.39 1799.49 Q2565.58 1799.49 2562.5 1794.91 Q2559.45 1790.3 2559.45 1781.58 Q2559.45 1772.83 2562.5 1768.24 Q2565.58 1763.64 2571.39 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M2601.55 1767.34 Q2597.94 1767.34 2596.11 1770.9 Q2594.31 1774.45 2594.31 1781.58 Q2594.31 1788.68 2596.11 1792.25 Q2597.94 1795.79 2601.55 1795.79 Q2605.19 1795.79 2606.99 1792.25 Q2608.82 1788.68 2608.82 1781.58 Q2608.82 1774.45 2606.99 1770.9 Q2605.19 1767.34 2601.55 1767.34 M2601.55 1763.64 Q2607.36 1763.64 2610.42 1768.24 Q2613.5 1772.83 2613.5 1781.58 Q2613.5 1790.3 2610.42 1794.91 Q2607.36 1799.49 2601.55 1799.49 Q2595.74 1799.49 2592.66 1794.91 Q2589.61 1790.3 2589.61 1781.58 Q2589.61 1772.83 2592.66 1768.24 Q2595.74 1763.64 2601.55 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M2631.71 1767.34 Q2628.1 1767.34 2626.27 1770.9 Q2624.47 1774.45 2624.47 1781.58 Q2624.47 1788.68 2626.27 1792.25 Q2628.1 1795.79 2631.71 1795.79 Q2635.35 1795.79 2637.15 1792.25 Q2638.98 1788.68 2638.98 1781.58 Q2638.98 1774.45 2637.15 1770.9 Q2635.35 1767.34 2631.71 1767.34 M2631.71 1763.64 Q2637.52 1763.64 2640.58 1768.24 Q2643.66 1772.83 2643.66 1781.58 Q2643.66 1790.3 2640.58 1794.91 Q2637.52 1799.49 2631.71 1799.49 Q2625.9 1799.49 2622.82 1794.91 Q2619.77 1790.3 2619.77 1781.58 Q2619.77 1772.83 2622.82 1768.24 Q2625.9 1763.64 2631.71 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1116.23 1835.78 L1124.89 1835.78 L1145.96 1875.53 L1145.96 1835.78 L1152.2 1835.78 L1152.2 1883.3 L1143.54 1883.3 L1122.47 1843.55 L1122.47 1883.3 L1116.23 1883.3 L1116.23 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1164.14 1869.23 L1164.14 1847.65 L1169.99 1847.65 L1169.99 1869.01 Q1169.99 1874.07 1171.97 1876.62 Q1173.94 1879.13 1177.89 1879.13 Q1182.63 1879.13 1185.37 1876.11 Q1188.13 1873.08 1188.13 1867.86 L1188.13 1847.65 L1193.99 1847.65 L1193.99 1883.3 L1188.13 1883.3 L1188.13 1877.83 Q1186 1881.07 1183.17 1882.66 Q1180.37 1884.22 1176.64 1884.22 Q1170.5 1884.22 1167.32 1880.4 Q1164.14 1876.59 1164.14 1869.23 M1178.87 1846.79 L1178.87 1846.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1233.81 1854.5 Q1236 1850.55 1239.06 1848.67 Q1242.12 1846.79 1246.25 1846.79 Q1251.82 1846.79 1254.85 1850.71 Q1257.87 1854.59 1257.87 1861.78 L1257.87 1883.3 L1251.98 1883.3 L1251.98 1861.98 Q1251.98 1856.85 1250.17 1854.37 Q1248.35 1851.89 1244.63 1851.89 Q1240.08 1851.89 1237.44 1854.91 Q1234.79 1857.93 1234.79 1863.15 L1234.79 1883.3 L1228.91 1883.3 L1228.91 1861.98 Q1228.91 1856.82 1227.09 1854.37 Q1225.28 1851.89 1221.49 1851.89 Q1217 1851.89 1214.36 1854.94 Q1211.72 1857.97 1211.72 1863.15 L1211.72 1883.3 L1205.83 1883.3 L1205.83 1847.65 L1211.72 1847.65 L1211.72 1853.19 Q1213.72 1849.91 1216.53 1848.35 Q1219.33 1846.79 1223.18 1846.79 Q1227.06 1846.79 1229.77 1848.77 Q1232.5 1850.74 1233.81 1854.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1295.14 1865.51 Q1295.14 1859.05 1292.47 1855.39 Q1289.83 1851.7 1285.18 1851.7 Q1280.53 1851.7 1277.86 1855.39 Q1275.22 1859.05 1275.22 1865.51 Q1275.22 1871.97 1277.86 1875.66 Q1280.53 1879.32 1285.18 1879.32 Q1289.83 1879.32 1292.47 1875.66 Q1295.14 1871.97 1295.14 1865.51 M1275.22 1853.06 Q1277.06 1849.88 1279.86 1848.35 Q1282.7 1846.79 1286.61 1846.79 Q1293.1 1846.79 1297.15 1851.95 Q1301.22 1857.11 1301.22 1865.51 Q1301.22 1873.91 1297.15 1879.07 Q1293.1 1884.22 1286.61 1884.22 Q1282.7 1884.22 1279.86 1882.7 Q1277.06 1881.14 1275.22 1877.95 L1275.22 1883.3 L1269.33 1883.3 L1269.33 1833.78 L1275.22 1833.78 L1275.22 1853.06 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1341.42 1864.01 L1341.42 1866.88 L1314.49 1866.88 Q1314.88 1872.92 1318.12 1876.11 Q1321.4 1879.26 1327.22 1879.26 Q1330.6 1879.26 1333.75 1878.43 Q1336.93 1877.6 1340.05 1875.95 L1340.05 1881.49 Q1336.9 1882.82 1333.59 1883.52 Q1330.28 1884.22 1326.87 1884.22 Q1318.34 1884.22 1313.35 1879.26 Q1308.38 1874.29 1308.38 1865.83 Q1308.38 1857.07 1313.09 1851.95 Q1317.84 1846.79 1325.86 1846.79 Q1333.05 1846.79 1337.22 1851.44 Q1341.42 1856.06 1341.42 1864.01 M1335.56 1862.29 Q1335.5 1857.49 1332.86 1854.62 Q1330.25 1851.76 1325.92 1851.76 Q1321.02 1851.76 1318.06 1854.53 Q1315.13 1857.3 1314.68 1862.33 L1335.56 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1371.69 1853.13 Q1370.7 1852.55 1369.52 1852.3 Q1368.38 1852.01 1366.98 1852.01 Q1362.01 1852.01 1359.34 1855.26 Q1356.7 1858.47 1356.7 1864.52 L1356.7 1883.3 L1350.81 1883.3 L1350.81 1847.65 L1356.7 1847.65 L1356.7 1853.19 Q1358.54 1849.94 1361.5 1848.39 Q1364.46 1846.79 1368.7 1846.79 Q1369.3 1846.79 1370.03 1846.89 Q1370.77 1846.95 1371.66 1847.11 L1371.69 1853.13 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1412.37 1851.76 Q1407.66 1851.76 1404.92 1855.45 Q1402.18 1859.11 1402.18 1865.51 Q1402.18 1871.91 1404.89 1875.6 Q1407.62 1879.26 1412.37 1879.26 Q1417.04 1879.26 1419.78 1875.57 Q1422.52 1871.87 1422.52 1865.51 Q1422.52 1859.17 1419.78 1855.48 Q1417.04 1851.76 1412.37 1851.76 M1412.37 1846.79 Q1420.01 1846.79 1424.37 1851.76 Q1428.73 1856.72 1428.73 1865.51 Q1428.73 1874.26 1424.37 1879.26 Q1420.01 1884.22 1412.37 1884.22 Q1404.7 1884.22 1400.33 1879.26 Q1396.01 1874.26 1396.01 1865.51 Q1396.01 1856.72 1400.33 1851.76 Q1404.7 1846.79 1412.37 1846.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1456.48 1833.78 L1456.48 1838.65 L1450.88 1838.65 Q1447.73 1838.65 1446.49 1839.92 Q1445.28 1841.19 1445.28 1844.5 L1445.28 1847.65 L1454.92 1847.65 L1454.92 1852.2 L1445.28 1852.2 L1445.28 1883.3 L1439.39 1883.3 L1439.39 1852.2 L1433.79 1852.2 L1433.79 1847.65 L1439.39 1847.65 L1439.39 1845.17 Q1439.39 1839.22 1442.16 1836.51 Q1444.93 1833.78 1450.94 1833.78 L1456.48 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1510.84 1837.34 L1510.84 1843.61 Q1507.18 1841.86 1503.94 1841 Q1500.69 1840.14 1497.67 1840.14 Q1492.41 1840.14 1489.55 1842.18 Q1486.72 1844.22 1486.72 1847.97 Q1486.72 1851.12 1488.6 1852.75 Q1490.51 1854.34 1495.79 1855.32 L1499.67 1856.12 Q1506.87 1857.49 1510.27 1860.96 Q1513.71 1864.39 1513.71 1870.19 Q1513.71 1877.09 1509.06 1880.66 Q1504.45 1884.22 1495.5 1884.22 Q1492.13 1884.22 1488.31 1883.46 Q1484.52 1882.7 1480.45 1881.2 L1480.45 1874.58 Q1484.36 1876.78 1488.12 1877.89 Q1491.87 1879 1495.5 1879 Q1501.01 1879 1504 1876.84 Q1506.99 1874.68 1506.99 1870.67 Q1506.99 1867.16 1504.83 1865.19 Q1502.7 1863.22 1497.79 1862.23 L1493.88 1861.47 Q1486.69 1860.03 1483.47 1856.98 Q1480.26 1853.92 1480.26 1848.48 Q1480.26 1842.18 1484.68 1838.55 Q1489.14 1834.92 1496.93 1834.92 Q1500.28 1834.92 1503.75 1835.53 Q1507.22 1836.13 1510.84 1837.34 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1539.68 1865.38 Q1532.58 1865.38 1529.85 1867 Q1527.11 1868.63 1527.11 1872.54 Q1527.11 1875.66 1529.15 1877.51 Q1531.21 1879.32 1534.75 1879.32 Q1539.62 1879.32 1542.54 1875.88 Q1545.5 1872.42 1545.5 1866.69 L1545.5 1865.38 L1539.68 1865.38 M1551.36 1862.96 L1551.36 1883.3 L1545.5 1883.3 L1545.5 1877.89 Q1543.5 1881.14 1540.51 1882.7 Q1537.52 1884.22 1533.19 1884.22 Q1527.71 1884.22 1524.47 1881.17 Q1521.25 1878.08 1521.25 1872.92 Q1521.25 1866.91 1525.26 1863.85 Q1529.3 1860.8 1537.29 1860.8 L1545.5 1860.8 L1545.5 1860.23 Q1545.5 1856.18 1542.83 1853.99 Q1540.19 1851.76 1535.38 1851.76 Q1532.33 1851.76 1529.43 1852.49 Q1526.54 1853.22 1523.86 1854.69 L1523.86 1849.28 Q1527.08 1848.03 1530.1 1847.43 Q1533.12 1846.79 1535.99 1846.79 Q1543.72 1846.79 1547.54 1850.8 Q1551.36 1854.81 1551.36 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1591.18 1854.5 Q1593.38 1850.55 1596.43 1848.67 Q1599.49 1846.79 1603.62 1846.79 Q1609.19 1846.79 1612.22 1850.71 Q1615.24 1854.59 1615.24 1861.78 L1615.24 1883.3 L1609.35 1883.3 L1609.35 1861.98 Q1609.35 1856.85 1607.54 1854.37 Q1605.72 1851.89 1602 1851.89 Q1597.45 1851.89 1594.81 1854.91 Q1592.17 1857.93 1592.17 1863.15 L1592.17 1883.3 L1586.28 1883.3 L1586.28 1861.98 Q1586.28 1856.82 1584.46 1854.37 Q1582.65 1851.89 1578.86 1851.89 Q1574.37 1851.89 1571.73 1854.94 Q1569.09 1857.97 1569.09 1863.15 L1569.09 1883.3 L1563.2 1883.3 L1563.2 1847.65 L1569.09 1847.65 L1569.09 1853.19 Q1571.1 1849.91 1573.9 1848.35 Q1576.7 1846.79 1580.55 1846.79 Q1584.43 1846.79 1587.14 1848.77 Q1589.87 1850.74 1591.18 1854.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1632.59 1877.95 L1632.59 1896.86 L1626.7 1896.86 L1626.7 1847.65 L1632.59 1847.65 L1632.59 1853.06 Q1634.43 1849.88 1637.23 1848.35 Q1640.07 1846.79 1643.98 1846.79 Q1650.48 1846.79 1654.52 1851.95 Q1658.59 1857.11 1658.59 1865.51 Q1658.59 1873.91 1654.52 1879.07 Q1650.48 1884.22 1643.98 1884.22 Q1640.07 1884.22 1637.23 1882.7 Q1634.43 1881.14 1632.59 1877.95 M1652.51 1865.51 Q1652.51 1859.05 1649.84 1855.39 Q1647.2 1851.7 1642.55 1851.7 Q1637.9 1851.7 1635.23 1855.39 Q1632.59 1859.05 1632.59 1865.51 Q1632.59 1871.97 1635.23 1875.66 Q1637.9 1879.32 1642.55 1879.32 Q1647.2 1879.32 1649.84 1875.66 Q1652.51 1871.97 1652.51 1865.51 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1668.3 1833.78 L1674.16 1833.78 L1674.16 1883.3 L1668.3 1883.3 L1668.3 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1716.9 1864.01 L1716.9 1866.88 L1689.97 1866.88 Q1690.36 1872.92 1693.6 1876.11 Q1696.88 1879.26 1702.71 1879.26 Q1706.08 1879.26 1709.23 1878.43 Q1712.41 1877.6 1715.53 1875.95 L1715.53 1881.49 Q1712.38 1882.82 1709.07 1883.52 Q1705.76 1884.22 1702.36 1884.22 Q1693.83 1884.22 1688.83 1879.26 Q1683.86 1874.29 1683.86 1865.83 Q1683.86 1857.07 1688.57 1851.95 Q1693.32 1846.79 1701.34 1846.79 Q1708.53 1846.79 1712.7 1851.44 Q1716.9 1856.06 1716.9 1864.01 M1711.05 1862.29 Q1710.98 1857.49 1708.34 1854.62 Q1705.73 1851.76 1701.4 1851.76 Q1696.5 1851.76 1693.54 1854.53 Q1690.61 1857.3 1690.17 1862.33 L1711.05 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M1749.24 1848.7 L1749.24 1854.24 Q1746.76 1852.97 1744.08 1852.33 Q1741.41 1851.7 1738.54 1851.7 Q1734.18 1851.7 1731.99 1853.03 Q1729.82 1854.37 1729.82 1857.04 Q1729.82 1859.08 1731.38 1860.26 Q1732.94 1861.4 1737.65 1862.45 L1739.66 1862.9 Q1745.9 1864.24 1748.51 1866.69 Q1751.15 1869.11 1751.15 1873.47 Q1751.15 1878.43 1747.2 1881.33 Q1743.29 1884.22 1736.41 1884.22 Q1733.55 1884.22 1730.43 1883.65 Q1727.34 1883.11 1723.9 1882 L1723.9 1875.95 Q1727.15 1877.64 1730.3 1878.49 Q1733.45 1879.32 1736.54 1879.32 Q1740.68 1879.32 1742.91 1877.92 Q1745.13 1876.49 1745.13 1873.91 Q1745.13 1871.52 1743.51 1870.25 Q1741.92 1868.98 1736.48 1867.8 L1734.44 1867.32 Q1729 1866.18 1726.58 1863.82 Q1724.16 1861.43 1724.16 1857.3 Q1724.16 1852.27 1727.72 1849.53 Q1731.29 1846.79 1737.84 1846.79 Q1741.09 1846.79 1743.96 1847.27 Q1746.82 1847.75 1749.24 1848.7 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip002)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.627,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip002)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.627,1396.74 2640.76,1396.74 "/>
<polyline clip-path="url(#clip002)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.627,1059.37 2640.76,1059.37 "/>
<polyline clip-path="url(#clip002)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.627,721.993 2640.76,721.993 "/>
<polyline clip-path="url(#clip002)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.627,384.619 2640.76,384.619 "/>
<polyline clip-path="url(#clip002)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.627,47.2441 2640.76,47.2441 "/>
<polyline clip-path="url(#clip000)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,1734.12 226.627,47.2441 "/>
<polyline clip-path="url(#clip000)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,1734.12 245.524,1734.12 "/>
<polyline clip-path="url(#clip000)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,1396.74 245.524,1396.74 "/>
<polyline clip-path="url(#clip000)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,1059.37 245.524,1059.37 "/>
<polyline clip-path="url(#clip000)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,721.993 245.524,721.993 "/>
<polyline clip-path="url(#clip000)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,384.619 245.524,384.619 "/>
<polyline clip-path="url(#clip000)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,47.2441 245.524,47.2441 "/>
<path clip-path="url(#clip000)" d="M132.974 1720.91 L121.168 1739.36 L132.974 1739.36 L132.974 1720.91 M131.747 1716.84 L137.626 1716.84 L137.626 1739.36 L142.557 1739.36 L142.557 1743.25 L137.626 1743.25 L137.626 1751.4 L132.974 1751.4 L132.974 1743.25 L117.372 1743.25 L117.372 1738.74 L131.747 1716.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M150.288 1745.52 L155.173 1745.52 L155.173 1751.4 L150.288 1751.4 L150.288 1745.52 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M165.404 1716.84 L183.76 1716.84 L183.76 1720.77 L169.686 1720.77 L169.686 1729.24 Q170.705 1728.9 171.723 1728.74 Q172.742 1728.55 173.76 1728.55 Q179.547 1728.55 182.927 1731.72 Q186.307 1734.89 186.307 1740.31 Q186.307 1745.89 182.834 1748.99 Q179.362 1752.07 173.043 1752.07 Q170.867 1752.07 168.598 1751.7 Q166.353 1751.33 163.946 1750.59 L163.946 1745.89 Q166.029 1747.02 168.251 1747.58 Q170.473 1748.13 172.95 1748.13 Q176.955 1748.13 179.293 1746.03 Q181.631 1743.92 181.631 1740.31 Q181.631 1736.7 179.293 1734.59 Q176.955 1732.49 172.95 1732.49 Q171.075 1732.49 169.2 1732.9 Q167.348 1733.32 165.404 1734.2 L165.404 1716.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M131.816 1383.54 L120.011 1401.99 L131.816 1401.99 L131.816 1383.54 M130.589 1379.46 L136.469 1379.46 L136.469 1401.99 L141.399 1401.99 L141.399 1405.87 L136.469 1405.87 L136.469 1414.02 L131.816 1414.02 L131.816 1405.87 L116.214 1405.87 L116.214 1401.36 L130.589 1379.46 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M149.131 1408.14 L154.015 1408.14 L154.015 1414.02 L149.131 1414.02 L149.131 1408.14 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M174.779 1394.88 Q171.631 1394.88 169.779 1397.03 Q167.95 1399.18 167.95 1402.93 Q167.95 1406.66 169.779 1408.84 Q171.631 1410.99 174.779 1410.99 Q177.927 1410.99 179.756 1408.84 Q181.608 1406.66 181.608 1402.93 Q181.608 1399.18 179.756 1397.03 Q177.927 1394.88 174.779 1394.88 M184.061 1380.23 L184.061 1384.49 Q182.302 1383.65 180.497 1383.21 Q178.714 1382.77 176.955 1382.77 Q172.325 1382.77 169.872 1385.9 Q167.441 1389.02 167.094 1395.34 Q168.46 1393.33 170.52 1392.26 Q172.58 1391.18 175.057 1391.18 Q180.265 1391.18 183.274 1394.35 Q186.307 1397.49 186.307 1402.93 Q186.307 1408.26 183.159 1411.48 Q180.01 1414.69 174.779 1414.69 Q168.784 1414.69 165.612 1410.11 Q162.441 1405.5 162.441 1396.78 Q162.441 1388.58 166.33 1383.72 Q170.219 1378.84 176.77 1378.84 Q178.529 1378.84 180.311 1379.18 Q182.117 1379.53 184.061 1380.23 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M132.881 1046.16 L121.076 1064.61 L132.881 1064.61 L132.881 1046.16 M131.654 1042.09 L137.534 1042.09 L137.534 1064.61 L142.464 1064.61 L142.464 1068.5 L137.534 1068.5 L137.534 1076.65 L132.881 1076.65 L132.881 1068.5 L117.279 1068.5 L117.279 1063.99 L131.654 1042.09 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M150.196 1070.77 L155.08 1070.77 L155.08 1076.65 L150.196 1076.65 L150.196 1070.77 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M164.085 1042.09 L186.307 1042.09 L186.307 1044.08 L173.76 1076.65 L168.876 1076.65 L180.682 1046.02 L164.085 1046.02 L164.085 1042.09 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M132.071 708.787 L120.265 727.236 L132.071 727.236 L132.071 708.787 M130.844 704.713 L136.724 704.713 L136.724 727.236 L141.654 727.236 L141.654 731.125 L136.724 731.125 L136.724 739.273 L132.071 739.273 L132.071 731.125 L116.469 731.125 L116.469 726.611 L130.844 704.713 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M149.386 733.394 L154.27 733.394 L154.27 739.273 L149.386 739.273 L149.386 733.394 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M174.455 722.861 Q171.122 722.861 169.2 724.644 Q167.302 726.426 167.302 729.551 Q167.302 732.676 169.2 734.458 Q171.122 736.241 174.455 736.241 Q177.788 736.241 179.709 734.458 Q181.631 732.653 181.631 729.551 Q181.631 726.426 179.709 724.644 Q177.811 722.861 174.455 722.861 M169.779 720.871 Q166.77 720.13 165.08 718.07 Q163.413 716.009 163.413 713.047 Q163.413 708.903 166.353 706.496 Q169.316 704.088 174.455 704.088 Q179.617 704.088 182.557 706.496 Q185.496 708.903 185.496 713.047 Q185.496 716.009 183.807 718.07 Q182.14 720.13 179.154 720.871 Q182.534 721.658 184.409 723.949 Q186.307 726.241 186.307 729.551 Q186.307 734.574 183.228 737.259 Q180.172 739.945 174.455 739.945 Q168.737 739.945 165.659 737.259 Q162.603 734.574 162.603 729.551 Q162.603 726.241 164.501 723.949 Q166.399 721.658 169.779 720.871 M168.066 713.486 Q168.066 716.172 169.733 717.676 Q171.422 719.181 174.455 719.181 Q177.464 719.181 179.154 717.676 Q180.867 716.172 180.867 713.486 Q180.867 710.801 179.154 709.297 Q177.464 707.792 174.455 707.792 Q171.422 707.792 169.733 709.297 Q168.066 710.801 168.066 713.486 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M132.163 371.413 L120.358 389.862 L132.163 389.862 L132.163 371.413 M130.937 367.339 L136.816 367.339 L136.816 389.862 L141.747 389.862 L141.747 393.751 L136.816 393.751 L136.816 401.899 L132.163 401.899 L132.163 393.751 L116.562 393.751 L116.562 389.237 L130.937 367.339 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M149.478 396.019 L154.362 396.019 L154.362 401.899 L149.478 401.899 L149.478 396.019 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M164.686 401.181 L164.686 396.922 Q166.446 397.755 168.251 398.195 Q170.057 398.635 171.793 398.635 Q176.422 398.635 178.853 395.533 Q181.307 392.408 181.654 386.065 Q180.311 388.056 178.251 389.121 Q176.191 390.186 173.691 390.186 Q168.506 390.186 165.473 387.061 Q162.464 383.913 162.464 378.473 Q162.464 373.149 165.612 369.931 Q168.76 366.714 173.992 366.714 Q179.987 366.714 183.135 371.32 Q186.307 375.903 186.307 384.653 Q186.307 392.825 182.418 397.709 Q178.552 402.57 172.001 402.57 Q170.242 402.57 168.436 402.223 Q166.631 401.876 164.686 401.181 M173.992 386.528 Q177.14 386.528 178.969 384.376 Q180.821 382.223 180.821 378.473 Q180.821 374.746 178.969 372.593 Q177.14 370.417 173.992 370.417 Q170.844 370.417 168.992 372.593 Q167.163 374.746 167.163 378.473 Q167.163 382.223 168.992 384.376 Q170.844 386.528 173.992 386.528 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M119.177 29.9641 L137.534 29.9641 L137.534 33.8993 L123.46 33.8993 L123.46 42.3714 Q124.478 42.0242 125.497 41.8622 Q126.515 41.677 127.534 41.677 Q133.321 41.677 136.7 44.8483 Q140.08 48.0196 140.08 53.4362 Q140.08 59.0149 136.608 62.1167 Q133.136 65.1954 126.816 65.1954 Q124.64 65.1954 122.372 64.825 Q120.126 64.4547 117.719 63.7139 L117.719 59.0149 Q119.802 60.1491 122.025 60.7047 Q124.247 61.2602 126.724 61.2602 Q130.728 61.2602 133.066 59.1538 Q135.404 57.0473 135.404 53.4362 Q135.404 49.8251 133.066 47.7186 Q130.728 45.6122 126.724 45.6122 Q124.849 45.6122 122.974 46.0288 Q121.122 46.4455 119.177 47.3251 L119.177 29.9641 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M149.293 58.6445 L154.177 58.6445 L154.177 64.5241 L149.293 64.5241 L149.293 58.6445 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M174.362 33.0428 Q170.751 33.0428 168.922 36.6076 Q167.117 40.1492 167.117 47.2788 Q167.117 54.3853 168.922 57.9501 Q170.751 61.4917 174.362 61.4917 Q177.997 61.4917 179.802 57.9501 Q181.631 54.3853 181.631 47.2788 Q181.631 40.1492 179.802 36.6076 Q177.997 33.0428 174.362 33.0428 M174.362 29.3391 Q180.172 29.3391 183.228 33.9456 Q186.307 38.5289 186.307 47.2788 Q186.307 56.0056 183.228 60.6121 Q180.172 65.1954 174.362 65.1954 Q168.552 65.1954 165.473 60.6121 Q162.418 56.0056 162.418 47.2788 Q162.418 38.5289 165.473 33.9456 Q168.552 29.3391 174.362 29.3391 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M54.859 1399.86 L42.0957 1399.86 L42.0957 1410.36 L36.8122 1410.36 L36.8122 1393.49 L57.2143 1393.49 Q59.856 1397.22 61.2247 1401.7 Q62.5615 1406.19 62.5615 1411.28 Q62.5615 1422.42 56.0684 1428.73 Q49.5436 1435 37.9262 1435 Q26.2769 1435 19.7839 1428.73 Q13.2591 1422.42 13.2591 1411.28 Q13.2591 1406.64 14.4049 1402.47 Q15.5507 1398.27 17.7787 1394.73 L24.6219 1394.73 Q21.5981 1398.3 20.0704 1402.31 Q18.5426 1406.32 18.5426 1410.74 Q18.5426 1419.46 23.4124 1423.86 Q28.2821 1428.22 37.9262 1428.22 Q47.5384 1428.22 52.4082 1423.86 Q57.2779 1419.46 57.2779 1410.74 Q57.2779 1407.34 56.705 1404.66 Q56.1003 1401.99 54.859 1399.86 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M12.1132 1382 L12.1132 1376.15 L61.6384 1376.15 L61.6384 1382 L12.1132 1382 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M30.0964 1350.08 Q30.0964 1354.79 33.7885 1357.53 Q37.4488 1360.26 43.8463 1360.26 Q50.2438 1360.26 53.9359 1357.56 Q57.5962 1354.82 57.5962 1350.08 Q57.5962 1345.4 53.9041 1342.66 Q50.212 1339.92 43.8463 1339.92 Q37.5124 1339.92 33.8203 1342.66 Q30.0964 1345.4 30.0964 1350.08 M25.1311 1350.08 Q25.1311 1342.44 30.0964 1338.08 Q35.0616 1333.72 43.8463 1333.72 Q52.5991 1333.72 57.5962 1338.08 Q62.5615 1342.44 62.5615 1350.08 Q62.5615 1357.75 57.5962 1362.11 Q52.5991 1366.44 43.8463 1366.44 Q35.0616 1366.44 30.0964 1362.11 Q25.1311 1357.75 25.1311 1350.08 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M43.8463 1298.42 Q37.3851 1298.42 33.7248 1301.09 Q30.0327 1303.74 30.0327 1308.38 Q30.0327 1313.03 33.7248 1315.7 Q37.3851 1318.34 43.8463 1318.34 Q50.3075 1318.34 53.9996 1315.7 Q57.6599 1313.03 57.6599 1308.38 Q57.6599 1303.74 53.9996 1301.09 Q50.3075 1298.42 43.8463 1298.42 M31.4013 1318.34 Q28.2185 1316.5 26.6907 1313.7 Q25.1311 1310.86 25.1311 1306.95 Q25.1311 1300.46 30.2873 1296.41 Q35.4436 1292.34 43.8463 1292.34 Q52.249 1292.34 57.4052 1296.41 Q62.5615 1300.46 62.5615 1306.95 Q62.5615 1310.86 61.0337 1313.7 Q59.4741 1316.5 56.2912 1318.34 L61.6384 1318.34 L61.6384 1324.23 L12.1132 1324.23 L12.1132 1318.34 L31.4013 1318.34 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M43.719 1266.43 Q43.719 1273.53 45.3422 1276.27 Q46.9655 1279 50.8804 1279 Q53.9996 1279 55.8457 1276.97 Q57.6599 1274.9 57.6599 1271.37 Q57.6599 1266.5 54.2224 1263.57 Q50.7531 1260.61 45.0239 1260.61 L43.719 1260.61 L43.719 1266.43 M41.3 1254.75 L61.6384 1254.75 L61.6384 1260.61 L56.2276 1260.61 Q59.4741 1262.61 61.0337 1265.6 Q62.5615 1268.6 62.5615 1272.93 Q62.5615 1278.4 59.5059 1281.65 Q56.4186 1284.86 51.2623 1284.86 Q45.2467 1284.86 42.1912 1280.85 Q39.1357 1276.81 39.1357 1268.82 L39.1357 1260.61 L38.5628 1260.61 Q34.5205 1260.61 32.3244 1263.28 Q30.0964 1265.92 30.0964 1270.73 Q30.0964 1273.78 30.8284 1276.68 Q31.5605 1279.58 33.0246 1282.25 L27.6137 1282.25 Q26.3724 1279.04 25.7677 1276.01 Q25.1311 1272.99 25.1311 1270.12 Q25.1311 1262.39 29.1415 1258.57 Q33.1519 1254.75 41.3 1254.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M12.1132 1242.69 L12.1132 1236.83 L61.6384 1236.83 L61.6384 1242.69 L12.1132 1242.69 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M14.1184 1210.19 L14.1184 1169.99 L19.5293 1169.99 L19.5293 1186.86 L61.6384 1186.86 L61.6384 1193.32 L19.5293 1193.32 L19.5293 1210.19 L14.1184 1210.19 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M42.3504 1144.62 L45.2149 1144.62 L45.2149 1171.55 Q51.2623 1171.17 54.4452 1167.92 Q57.5962 1164.64 57.5962 1158.82 Q57.5962 1155.45 56.7687 1152.3 Q55.9411 1149.11 54.2861 1145.99 L59.8242 1145.99 Q61.161 1149.14 61.8612 1152.45 Q62.5615 1155.76 62.5615 1159.17 Q62.5615 1167.7 57.5962 1172.7 Q52.631 1177.66 44.1646 1177.66 Q35.4117 1177.66 30.2873 1172.95 Q25.1311 1168.21 25.1311 1160.19 Q25.1311 1153 29.7781 1148.83 Q34.3932 1144.62 42.3504 1144.62 M40.6316 1150.48 Q35.8255 1150.54 32.9609 1153.19 Q30.0964 1155.8 30.0964 1160.12 Q30.0964 1165.03 32.8654 1167.99 Q35.6345 1170.91 40.6634 1171.36 L40.6316 1150.48 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M32.8336 1107.26 Q28.8869 1105.06 27.009 1102.01 Q25.1311 1098.95 25.1311 1094.81 Q25.1311 1089.24 29.046 1086.22 Q32.9291 1083.2 40.1224 1083.2 L61.6384 1083.2 L61.6384 1089.08 L40.3133 1089.08 Q35.1889 1089.08 32.7063 1090.9 Q30.2237 1092.71 30.2237 1096.44 Q30.2237 1100.99 33.2474 1103.63 Q36.2711 1106.27 41.491 1106.27 L61.6384 1106.27 L61.6384 1112.16 L40.3133 1112.16 Q35.1571 1112.16 32.7063 1113.97 Q30.2237 1115.79 30.2237 1119.58 Q30.2237 1124.06 33.2792 1126.71 Q36.3029 1129.35 41.491 1129.35 L61.6384 1129.35 L61.6384 1135.24 L25.9905 1135.24 L25.9905 1129.35 L31.5287 1129.35 Q28.2503 1127.34 26.6907 1124.54 Q25.1311 1121.74 25.1311 1117.89 Q25.1311 1114.01 27.1045 1111.3 Q29.0779 1108.56 32.8336 1107.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M56.2912 1065.85 L75.1974 1065.85 L75.1974 1071.74 L25.9905 1071.74 L25.9905 1065.85 L31.4013 1065.85 Q28.2185 1064 26.6907 1061.2 Q25.1311 1058.37 25.1311 1054.45 Q25.1311 1047.96 30.2873 1043.92 Q35.4436 1039.84 43.8463 1039.84 Q52.249 1039.84 57.4052 1043.92 Q62.5615 1047.96 62.5615 1054.45 Q62.5615 1058.37 61.0337 1061.2 Q59.4741 1064 56.2912 1065.85 M43.8463 1045.92 Q37.3851 1045.92 33.7248 1048.6 Q30.0327 1051.24 30.0327 1055.89 Q30.0327 1060.53 33.7248 1063.21 Q37.3851 1065.85 43.8463 1065.85 Q50.3075 1065.85 53.9996 1063.21 Q57.6599 1060.53 57.6599 1055.89 Q57.6599 1051.24 53.9996 1048.6 Q50.3075 1045.92 43.8463 1045.92 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M42.3504 999.646 L45.2149 999.646 L45.2149 1026.57 Q51.2623 1026.19 54.4452 1022.94 Q57.5962 1019.67 57.5962 1013.84 Q57.5962 1010.47 56.7687 1007.32 Q55.9411 1004.13 54.2861 1001.01 L59.8242 1001.01 Q61.161 1004.17 61.8612 1007.48 Q62.5615 1010.79 62.5615 1014.19 Q62.5615 1022.72 57.5962 1027.72 Q52.631 1032.68 44.1646 1032.68 Q35.4117 1032.68 30.2873 1027.97 Q25.1311 1023.23 25.1311 1015.21 Q25.1311 1008.02 29.7781 1003.85 Q34.3932 999.646 42.3504 999.646 M40.6316 1005.5 Q35.8255 1005.57 32.9609 1008.21 Q30.0964 1010.82 30.0964 1015.15 Q30.0964 1020.05 32.8654 1023.01 Q35.6345 1025.94 40.6634 1026.38 L40.6316 1005.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M31.465 969.377 Q30.8921 970.363 30.6375 971.541 Q30.351 972.687 30.351 974.087 Q30.351 979.052 33.5975 981.726 Q36.8122 984.368 42.8596 984.368 L61.6384 984.368 L61.6384 990.256 L25.9905 990.256 L25.9905 984.368 L31.5287 984.368 Q28.2821 982.522 26.7225 979.562 Q25.1311 976.602 25.1311 972.368 Q25.1311 971.764 25.2266 971.032 Q25.2903 970.3 25.4494 969.408 L31.465 969.377 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M43.719 947.033 Q43.719 954.131 45.3422 956.868 Q46.9655 959.605 50.8804 959.605 Q53.9996 959.605 55.8457 957.568 Q57.6599 955.499 57.6599 951.966 Q57.6599 947.097 54.2224 944.168 Q50.7531 941.208 45.0239 941.208 L43.719 941.208 L43.719 947.033 M41.3 935.352 L61.6384 935.352 L61.6384 941.208 L56.2276 941.208 Q59.4741 943.214 61.0337 946.205 Q62.5615 949.197 62.5615 953.526 Q62.5615 959 59.5059 962.247 Q56.4186 965.462 51.2623 965.462 Q45.2467 965.462 42.1912 961.451 Q39.1357 957.409 39.1357 949.42 L39.1357 941.208 L38.5628 941.208 Q34.5205 941.208 32.3244 943.882 Q30.0964 946.524 30.0964 951.33 Q30.0964 954.385 30.8284 957.282 Q31.5605 960.178 33.0246 962.852 L27.6137 962.852 Q26.3724 959.637 25.7677 956.613 Q25.1311 953.59 25.1311 950.725 Q25.1311 942.991 29.1415 939.171 Q33.1519 935.352 41.3 935.352 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M15.869 917.496 L25.9905 917.496 L25.9905 905.433 L30.542 905.433 L30.542 917.496 L49.8937 917.496 Q54.2542 917.496 55.4955 916.318 Q56.7368 915.109 56.7368 911.449 L56.7368 905.433 L61.6384 905.433 L61.6384 911.449 Q61.6384 918.228 59.124 920.806 Q56.5777 923.384 49.8937 923.384 L30.542 923.384 L30.542 927.681 L25.9905 927.681 L25.9905 923.384 L15.869 923.384 L15.869 917.496 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M47.5702 898.335 L25.9905 898.335 L25.9905 892.479 L47.3474 892.479 Q52.4082 892.479 54.9545 890.505 Q57.4689 888.532 57.4689 884.585 Q57.4689 879.843 54.4452 877.106 Q51.4215 874.337 46.2016 874.337 L25.9905 874.337 L25.9905 868.48 L61.6384 868.48 L61.6384 874.337 L56.1639 874.337 Q59.4104 876.469 61.0019 879.302 Q62.5615 882.103 62.5615 885.827 Q62.5615 891.97 58.742 895.152 Q54.9226 898.335 47.5702 898.335 M25.1311 883.599 L25.1311 883.599 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M31.465 835.76 Q30.8921 836.747 30.6375 837.925 Q30.351 839.071 30.351 840.471 Q30.351 845.436 33.5975 848.11 Q36.8122 850.752 42.8596 850.752 L61.6384 850.752 L61.6384 856.64 L25.9905 856.64 L25.9905 850.752 L31.5287 850.752 Q28.2821 848.906 26.7225 845.946 Q25.1311 842.985 25.1311 838.752 Q25.1311 838.148 25.2266 837.415 Q25.2903 836.683 25.4494 835.792 L31.465 835.76 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M42.3504 800.558 L45.2149 800.558 L45.2149 827.485 Q51.2623 827.103 54.4452 823.857 Q57.5962 820.578 57.5962 814.754 Q57.5962 811.38 56.7687 808.229 Q55.9411 805.046 54.2861 801.927 L59.8242 801.927 Q61.161 805.078 61.8612 808.388 Q62.5615 811.698 62.5615 815.104 Q62.5615 823.634 57.5962 828.631 Q52.631 833.596 44.1646 833.596 Q35.4117 833.596 30.2873 828.885 Q25.1311 824.143 25.1311 816.122 Q25.1311 808.929 29.7781 804.759 Q34.3932 800.558 42.3504 800.558 M40.6316 806.414 Q35.8255 806.478 32.9609 809.12 Q30.0964 811.73 30.0964 816.059 Q30.0964 820.96 32.8654 823.92 Q35.6345 826.848 40.6634 827.294 L40.6316 806.414 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M20.4523 754.088 L44.1009 762.809 L44.1009 745.336 L20.4523 754.088 M14.1184 757.717 L14.1184 750.428 L61.6384 732.318 L61.6384 739.002 L49.4481 743.33 L49.4481 764.751 L61.6384 769.08 L61.6384 775.859 L14.1184 757.717 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M40.1224 696.001 L61.6384 696.001 L61.6384 701.858 L40.3133 701.858 Q35.2526 701.858 32.7381 703.831 Q30.2237 705.805 30.2237 709.751 Q30.2237 714.494 33.2474 717.231 Q36.2711 719.968 41.491 719.968 L61.6384 719.968 L61.6384 725.856 L25.9905 725.856 L25.9905 719.968 L31.5287 719.968 Q28.314 717.868 26.7225 715.035 Q25.1311 712.17 25.1311 708.446 Q25.1311 702.303 28.9505 699.152 Q32.7381 696.001 40.1224 696.001 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M30.0964 670.507 Q30.0964 675.217 33.7885 677.955 Q37.4488 680.692 43.8463 680.692 Q50.2438 680.692 53.9359 677.986 Q57.5962 675.249 57.5962 670.507 Q57.5962 665.828 53.9041 663.091 Q50.212 660.353 43.8463 660.353 Q37.5124 660.353 33.8203 663.091 Q30.0964 665.828 30.0964 670.507 M25.1311 670.507 Q25.1311 662.868 30.0964 658.507 Q35.0616 654.147 43.8463 654.147 Q52.5991 654.147 57.5962 658.507 Q62.5615 662.868 62.5615 670.507 Q62.5615 678.177 57.5962 682.538 Q52.5991 686.867 43.8463 686.867 Q35.0616 686.867 30.0964 682.538 Q25.1311 678.177 25.1311 670.507 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M32.8336 616.685 Q28.8869 614.488 27.009 611.433 Q25.1311 608.377 25.1311 604.24 Q25.1311 598.67 29.046 595.646 Q32.9291 592.622 40.1224 592.622 L61.6384 592.622 L61.6384 598.511 L40.3133 598.511 Q35.1889 598.511 32.7063 600.325 Q30.2237 602.139 30.2237 605.863 Q30.2237 610.414 33.2474 613.056 Q36.2711 615.698 41.491 615.698 L61.6384 615.698 L61.6384 621.586 L40.3133 621.586 Q35.1571 621.586 32.7063 623.4 Q30.2237 625.215 30.2237 629.002 Q30.2237 633.49 33.2792 636.132 Q36.3029 638.774 41.491 638.774 L61.6384 638.774 L61.6384 644.662 L25.9905 644.662 L25.9905 638.774 L31.5287 638.774 Q28.2503 636.768 26.6907 633.968 Q25.1311 631.167 25.1311 627.315 Q25.1311 623.432 27.1045 620.727 Q29.0779 617.99 32.8336 616.685 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M43.719 564.74 Q43.719 571.838 45.3422 574.575 Q46.9655 577.313 50.8804 577.313 Q53.9996 577.313 55.8457 575.276 Q57.6599 573.207 57.6599 569.674 Q57.6599 564.804 54.2224 561.876 Q50.7531 558.916 45.0239 558.916 L43.719 558.916 L43.719 564.74 M41.3 553.059 L61.6384 553.059 L61.6384 558.916 L56.2276 558.916 Q59.4741 560.921 61.0337 563.913 Q62.5615 566.905 62.5615 571.233 Q62.5615 576.708 59.5059 579.954 Q56.4186 583.169 51.2623 583.169 Q45.2467 583.169 42.1912 579.159 Q39.1357 575.117 39.1357 567.128 L39.1357 558.916 L38.5628 558.916 Q34.5205 558.916 32.3244 561.589 Q30.0964 564.231 30.0964 569.037 Q30.0964 572.093 30.8284 574.989 Q31.5605 577.886 33.0246 580.559 L27.6137 580.559 Q26.3724 577.345 25.7677 574.321 Q25.1311 571.297 25.1311 568.433 Q25.1311 560.698 29.1415 556.879 Q33.1519 553.059 41.3 553.059 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M12.1132 540.996 L12.1132 535.14 L61.6384 535.14 L61.6384 540.996 L12.1132 540.996 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M64.9486 508.054 Q71.3143 510.536 73.2559 512.892 Q75.1974 515.247 75.1974 519.194 L75.1974 523.873 L70.2958 523.873 L70.2958 520.435 Q70.2958 518.016 69.15 516.679 Q68.0041 515.343 63.7391 513.719 L61.0655 512.669 L25.9905 527.087 L25.9905 520.881 L53.8723 509.741 L25.9905 498.601 L25.9905 492.394 L64.9486 508.054 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M12.1769 449.521 Q19.4975 453.786 26.6589 455.855 Q33.8203 457.924 41.1727 457.924 Q48.5251 457.924 55.7502 455.855 Q62.9434 453.754 70.2321 449.521 L70.2321 454.614 Q62.7524 459.388 55.5274 461.775 Q48.3023 464.13 41.1727 464.13 Q34.0749 464.13 26.8817 461.775 Q19.6884 459.42 12.1769 454.614 L12.1769 449.521 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M17.365 428.005 Q17.365 430.551 19.1474 432.302 Q20.8979 434.052 23.4442 434.052 Q25.9587 434.052 27.7092 432.302 Q29.428 430.551 29.428 428.005 Q29.428 425.459 27.7092 423.708 Q25.9587 421.958 23.4442 421.958 Q20.9297 421.958 19.1474 423.74 Q17.365 425.491 17.365 428.005 M13.2591 428.005 Q13.2591 425.968 14.0548 424.09 Q14.8187 422.212 16.2828 420.844 Q17.7151 419.38 19.5293 418.647 Q21.3435 417.915 23.4442 417.915 Q27.6456 417.915 30.5738 420.875 Q33.4702 423.804 33.4702 428.069 Q33.4702 432.366 30.6056 435.23 Q27.7411 438.095 23.4442 438.095 Q19.1792 438.095 16.2191 435.166 Q13.2591 432.238 13.2591 428.005 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M17.7787 369.727 L24.5582 369.727 Q21.5345 372.974 20.0386 376.666 Q18.5426 380.326 18.5426 384.464 Q18.5426 392.612 23.5397 396.94 Q28.5049 401.269 37.9262 401.269 Q47.3156 401.269 52.3127 396.94 Q57.2779 392.612 57.2779 384.464 Q57.2779 380.326 55.782 376.666 Q54.2861 372.974 51.2623 369.727 L57.9782 369.727 Q60.2698 373.101 61.4156 376.888 Q62.5615 380.644 62.5615 384.846 Q62.5615 395.635 55.973 401.842 Q49.3526 408.049 37.9262 408.049 Q26.4679 408.049 19.8794 401.842 Q13.2591 395.635 13.2591 384.846 Q13.2591 380.581 14.4049 376.825 Q15.5189 373.037 17.7787 369.727 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip000)" d="M12.1769 360.974 L12.1769 355.882 Q19.6884 351.107 26.8817 348.752 Q34.0749 346.365 41.1727 346.365 Q48.3023 346.365 55.5274 348.752 Q62.7524 351.107 70.2321 355.882 L70.2321 360.974 Q62.9434 356.741 55.7502 354.672 Q48.5251 352.571 41.1727 352.571 Q33.8203 352.571 26.6589 354.672 Q19.4975 356.741 12.1769 360.974 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip002)" d="M294.951 4196.61 L295.179 5268.34 L295.407 4346.13 L295.634 4149.7 L295.862 3849.78 L296.09 4378 L296.318 4009.15 L296.545 4252.3 L296.773 3996.37 L297.001 3717.06 L297.229 3448.97 L297.457 3141.53 L297.684 2813.53 L297.912 2708 L298.14 2452.42 L298.368 2367.03 L298.595 2203.43 L298.823 2003.5 L299.051 2052.82 L299.279 1966.38 L299.506 1867.11 L299.734 1835.28 L299.962 1719.01 L300.19 1620.04 L300.418 1873.03 L300.645 1801 L300.873 1669.38 L301.101 1747.62 L301.329 1720.06 L301.556 1600.96 L301.784 1549.83 L302.012 1661.11 L302.24 1662.89 L302.468 1661.9 L302.695 1557.57 L302.923 1519.02 L303.151 1463.23 L303.379 1400.14 L303.606 1342.33 L303.834 1265.11 L304.062 1200.31 L304.29 1216.4 L304.517 1151.01 L304.745 1129.58 L304.973 1150.2 L305.201 1113.91 L305.429 1164.95 L305.656 1186.74 L305.884 1197.3 L306.112 1217.71 L306.34 1154.42 L306.567 1102.43 L306.795 1071.9 L307.023 1065.81 L307.251 1067.2 L307.478 1135.16 L307.706 1134.7 L307.934 1163.08 L308.162 1149.46 L308.39 1128.45 L308.617 1108.22 L308.845 1077.42 L309.073 1100.52 L309.301 1033.87 L309.528 1039.53 L309.756 1031.69 L309.984 1064.36 L310.212 1110.18 L310.439 1082.6 L310.667 1154.83 L310.895 1123.8 L311.123 1111.47 L311.351 1129.31 L311.578 1099.67 L311.806 1077.26 L312.034 1077.17 L312.262 1075.45 L312.489 1031.04 L312.717 1017.02 L312.945 958.072 L313.173 895.978 L313.401 991.949 L313.628 998.171 L313.856 1016.47 L314.084 997.82 L314.312 1050.51 L314.539 1061.36 L314.767 1091.34 L314.995 1063.13 L315.223 1094.24 L315.45 1141.9 L315.678 1169.74 L315.906 1193.43 L316.134 1196.06 L316.362 1187.33 L316.589 1160.45 L316.817 1128.29 L317.045 1117.64 L317.273 1118.49 L317.5 1133.64 L317.728 1071.3 L317.956 1020.15 L318.184 1004.01 L318.411 1020.59 L318.639 1031.58 L318.867 1024.81 L319.095 1011.71 L319.323 1010.86 L319.55 1041 L319.778 1066.44 L320.006 1059.05 L320.234 1020.53 L320.461 1018.93 L320.689 1044.62 L320.917 1053.38 L321.145 1034.6 L321.372 1002.76 L321.6 1025.01 L321.828 996.546 L322.056 1013.24 L322.284 1005.17 L322.511 974.125 L322.739 964.133 L322.967 975.855 L323.195 967.468 L323.422 991.931 L323.65 1023.1 L323.878 1025.82 L324.106 1016.76 L324.334 1005.48 L324.561 1027.88 L324.789 1019.43 L325.017 1030.49 L325.245 1032.97 L325.472 1019.43 L325.7 1038.36 L325.928 1064.69 L326.156 1074.28 L326.383 1086.07 L326.611 1077.46 L326.839 1087.48 L327.067 1092.74 L327.295 1109.13 L327.522 1119.46 L327.75 1137.28 L327.978 1159.64 L328.206 1174.94 L328.433 1175.22 L328.661 1176.02 L328.889 1166.96 L329.117 1159.19 L329.344 1168.36 L329.572 1199.43 L329.8 1195.57 L330.028 1161.26 L330.256 1156.11 L330.483 1163.98 L330.711 1154.38 L330.939 1168.6 L331.167 1160.63 L331.394 1171.51 L331.622 1176.86 L331.85 1144.83 L332.078 1145.54 L332.305 1138.09 L332.533 1140.87 L332.761 1139.23 L332.989 1112.12 L333.217 1078.49 L333.444 1054.38 L333.672 1042.48 L333.9 1046.58 L334.128 1061.73 L334.355 1078.17 L334.583 1046.32 L334.811 1054.7 L335.039 1008.72 L335.267 1009.49 L335.494 986.244 L335.722 971.548 L335.95 983.734 L336.178 993.162 L336.405 985.492 L336.633 991.747 L336.861 982.399 L337.089 978.032 L337.316 978.811 L337.544 985.809 L337.772 965.807 L338 955.827 L338.228 930.84 L338.455 915.048 L338.683 931.514 L338.911 942.207 L339.139 923.745 L339.366 920.059 L339.594 924.123 L339.822 918.501 L340.05 915.299 L340.277 904.985 L340.505 885.11 L340.733 885.147 L340.961 884.815 L341.189 887.03 L341.416 884.341 L341.644 873.094 L341.872 875.618 L342.1 864.354 L342.327 852.247 L342.555 863.347 L342.783 881.854 L343.011 872.905 L343.238 851.852 L343.466 860.21 L343.694 837.591 L343.922 856.087 L344.15 879.116 L344.377 852.294 L344.605 842.916 L344.833 850.839 L345.061 869.229 L345.288 876.014 L345.516 890.434 L345.744 902.296 L345.972 894.271 L346.2 903.314 L346.427 918.639 L346.655 918.634 L346.883 913.575 L347.111 894.276 L347.338 873.622 L347.566 868.457 L347.794 845.542 L348.022 851.156 L348.249 846.345 L348.477 834.774 L348.705 832.704 L348.933 846.913 L349.161 841.247 L349.388 853.859 L349.616 847.703 L349.844 823.473 L350.072 833.745 L350.299 824.056 L350.527 836.411 L350.755 854.039 L350.983 855.291 L351.21 859.029 L351.438 871.36 L351.666 848.693 L351.894 844.897 L352.122 827.654 L352.349 814.211 L352.577 813.395 L352.805 826.965 L353.033 832.557 L353.26 829.699 L353.488 834.778 L353.716 844.977 L353.944 842.567 L354.171 841.683 L354.399 847.26 L354.627 839.69 L354.855 824.002 L355.083 836.025 L355.31 836.427 L355.538 837.673 L355.766 836.127 L355.994 836.83 L356.221 850.697 L356.449 845.128 L356.677 851.113 L356.905 847.237 L357.133 856.318 L357.36 854.306 L357.588 854.822 L357.816 855.598 L358.044 859.364 L358.271 857.797 L358.499 864.209 L358.727 855.119 L358.955 852.692 L359.182 863.859 L359.41 855.03 L359.638 842.818 L359.866 844.9 L360.094 831.699 L360.321 845.717 L360.549 841.14 L360.777 841.219 L361.005 828.56 L361.232 841.673 L361.46 844.421 L361.688 852.747 L361.916 859.173 L362.143 874.494 L362.371 879.256 L362.599 878.052 L362.827 892.287 L363.055 894.109 L363.282 900.957 L363.51 892.694 L363.738 888.21 L363.966 888.211 L364.193 882.139 L364.421 882.4 L364.649 876.201 L364.877 876.204 L365.104 871.75 L365.332 869.744 L365.56 874.856 L365.788 870.997 L366.016 854.652 L366.243 856.536 L366.471 863.246 L366.699 855.874 L366.927 842.084 L367.154 843.738 L367.382 859.528 L367.61 869.025 L367.838 876.992 L368.066 874.927 L368.293 858.551 L368.521 856.695 L368.749 872.314 L368.977 878.545 L369.204 891.002 L369.432 884.78 L369.66 880.442 L369.888 889.214 L370.115 890.493 L370.343 889.952 L370.571 885.388 L370.799 878.79 L371.027 889.187 L371.254 894.955 L371.482 905.81 L371.71 899.96 L371.938 909.094 L372.165 912.333 L372.393 904.155 L372.621 915.107 L372.849 914.508 L373.076 902.455 L373.304 894.38 L373.532 892.262 L373.76 885.631 L373.988 882.827 L374.215 885.994 L374.443 883.719 L374.671 883.739 L374.899 881.634 L375.126 875.218 L375.354 882.366 L375.582 880.218 L375.81 882.854 L376.037 873.919 L376.265 889.26 L376.493 886.014 L376.721 898.693 L376.949 892.439 L377.176 894.501 L377.404 899.56 L377.632 899.776 L377.86 891.92 L378.087 902.779 L378.315 895.76 L378.543 896.163 L378.771 906.048 L378.999 896.231 L379.226 899.272 L379.454 910.317 L379.682 909.658 L379.91 917.115 L380.137 919.552 L380.365 906.658 L380.593 917.875 L380.821 921.774 L381.048 934.626 L381.276 926.26 L381.504 926.57 L381.732 934.34 L381.96 929.966 L382.187 921.718 L382.415 928.57 L382.643 931.895 L382.871 935.946 L383.098 935.27 L383.326 926.211 L383.554 925.252 L383.782 920.737 L384.009 923.733 L384.237 931.225 L384.465 920.27 L384.693 928.396 L384.921 939.315 L385.148 937.914 L385.376 943.003 L385.604 922.165 L385.832 918.292 L386.059 922.429 L386.287 911.333 L386.515 916.828 L386.743 910.214 L386.97 909.985 L387.198 906.489 L387.426 901.62 L387.654 907.868 L387.882 904.66 L388.109 911.442 L388.337 906.764 L388.565 907.069 L388.793 902.89 L389.02 912.802 L389.248 915.645 L389.476 919.269 L389.704 918.268 L389.932 928.021 L390.159 931.27 L390.387 935.07 L390.615 941.145 L390.843 934.499 L391.07 942.17 L391.298 941.823 L391.526 933.746 L391.754 932.04 L391.981 936.629 L392.209 939.642 L392.437 941.006 L392.665 941.778 L392.893 937.234 L393.12 939.557 L393.348 941.976 L393.576 944.022 L393.804 945.986 L394.031 943.364 L394.259 946.851 L394.487 954.024 L394.715 963.501 L394.942 961.913 L395.17 971.234 L395.398 976.29 L395.626 973.626 L395.854 978.128 L396.081 980.196 L396.309 986.693 L396.537 988.976 L396.765 982.927 L396.992 981.875 L397.22 975.544 L397.448 977.554 L397.676 986.13 L397.903 973.992 L398.131 980.375 L398.359 975.955 L398.587 977.546 L398.815 981.48 L399.042 980.594 L399.27 974.197 L399.498 975.367 L399.726 983.705 L399.953 972.557 L400.181 970.924 L400.409 971.761 L400.637 961.911 L400.865 970.704 L401.092 973.128 L401.32 969.666 L401.548 978.373 L401.776 974.731 L402.003 966.469 L402.231 969.34 L402.459 967.328 L402.687 961.464 L402.914 961.937 L403.142 966.573 L403.37 977.527 L403.598 975.157 L403.826 967.684 L404.053 968.564 L404.281 965.571 L404.509 961.125 L404.737 960.422 L404.964 955.954 L405.192 955.969 L405.42 959.628 L405.648 969.044 L405.875 956.817 L406.103 958.408 L406.331 952.254 L406.559 953.362 L406.787 952.317 L407.014 952.892 L407.242 961.059 L407.47 958.804 L407.698 957.534 L407.925 957.586 L408.153 958.805 L408.381 962.746 L408.609 967.594 L408.836 972.927 L409.064 972.654 L409.292 976.516 L409.52 974.833 L409.748 967.905 L409.975 968.484 L410.203 959.797 L410.431 960.573 L410.659 966.624 L410.886 972.828 L411.114 969.824 L411.342 969.871 L411.57 966.898 L411.798 970.307 L412.025 975.637 L412.253 979.77 L412.481 986.218 L412.709 983.186 L412.936 984 L413.164 985.393 L413.392 988.968 L413.62 993.355 L413.847 988.593 L414.075 987.475 L414.303 987.375 L414.531 987.206 L414.759 991.882 L414.986 984.425 L415.214 983.903 L415.442 987.784 L415.67 983.269 L415.897 985.594 L416.125 981.108 L416.353 977.582 L416.581 973.537 L416.808 975.388 L417.036 984.206 L417.264 984.783 L417.492 988.337 L417.72 988.913 L417.947 996.949 L418.175 1006.12 L418.403 1010.34 L418.631 1011.21 L418.858 1005.88 L419.086 1003.99 L419.314 1001.33 L419.542 1011.41 L419.769 1006.81 L419.997 1009.33 L420.225 1012.92 L420.453 1009.63 L420.681 1006.75 L420.908 1004.63 L421.136 1004.08 L421.364 997.713 L421.592 993.093 L421.819 987.466 L422.047 984.791 L422.275 984.394 L422.503 985.959 L422.731 977.165 L422.958 980.386 L423.186 986.361 L423.414 984.969 L423.642 984.817 L423.869 984.253 L424.097 986.671 L424.325 991.474 L424.553 988.901 L424.78 991.331 L425.008 987.066 L425.236 993.224 L425.464 993.845 L425.692 998.036 L425.919 996.157 L426.147 996.245 L426.375 992.373 L426.603 998.095 L426.83 989.766 L427.058 992.199 L427.286 1000.54 L427.514 1003.21 L427.741 1002.25 L427.969 1000 L428.197 1001.6 L428.425 1002.64 L428.653 1003.44 L428.88 1001.11 L429.108 1006.76 L429.336 1008.35 L429.564 1013.37 L429.791 1015.82 L430.019 1016.18 L430.247 1020.27 L430.475 1021.48 L430.702 1024.29 L430.93 1024.04 L431.158 1023.7 L431.386 1019.11 L431.614 1019.6 L431.841 1013.55 L432.069 1018.87 L432.297 1023.64 L432.525 1015.18 L432.752 1015.6 L432.98 1018.22 L433.208 1011.43 L433.436 1011.16 L433.664 1012.4 L433.891 1021.69 L434.119 1022.55 L434.347 1021.61 L434.575 1017.89 L434.802 1017.91 L435.03 1019.24 L435.258 1016.17 L435.486 1009.62 L435.713 1007.82 L435.941 1008.94 L436.169 1009.69 L436.397 1009.86 L436.625 1009.42 L436.852 1006.43 L437.08 1007.83 L437.308 1005.66 L437.536 1007.28 L437.763 1010.65 L437.991 1019.09 L438.219 1022.09 L438.447 1022.99 L438.674 1028.98 L438.902 1030.19 L439.13 1031.12 L439.358 1032.17 L439.586 1027.94 L439.813 1022.26 L440.041 1026.5 L440.269 1024.35 L440.497 1026.72 L440.724 1023.66 L440.952 1026.64 L441.18 1025.86 L441.408 1028.14 L441.635 1020.17 L441.863 1018.18 L442.091 1014.92 L442.319 1016.23 L442.547 1012.47 L442.774 1013.59 L443.002 1014.75 L443.23 1011.01 L443.458 1010.12 L443.685 1013.7 L443.913 1014.83 L444.141 1017.44 L444.369 1015.65 L444.597 1021.16 L444.824 1018.82 L445.052 1018.49 L445.28 1019.62 L445.508 1017.36 L445.735 1021.93 L445.963 1018.45 L446.191 1018.91 L446.419 1021.67 L446.646 1027.67 L446.874 1030.18 L447.102 1024.73 L447.33 1026.74 L447.558 1027.47 L447.785 1027.1 L448.013 1029.53 L448.241 1027.82 L448.469 1023.16 L448.696 1025.19 L448.924 1024.68 L449.152 1029.71 L449.38 1028.8 L449.607 1018.62 L449.835 1021.11 L450.063 1020.46 L450.291 1020.66 L450.519 1024.95 L450.746 1019.05 L450.974 1020.57 L451.202 1024.35 L451.43 1029.09 L451.657 1026.57 L451.885 1025.14 L452.113 1022.43 L452.341 1027.73 L452.568 1024.32 L452.796 1024.75 L453.024 1025.41 L453.252 1025.03 L453.48 1022.1 L453.707 1023.53 L453.935 1022.62 L454.163 1021.93 L454.391 1019.51 L454.618 1019.78 L454.846 1027.15 L455.074 1027.28 L455.302 1023.23 L455.53 1019.46 L455.757 1019.91 L455.985 1018.03 L456.213 1014.88 L456.441 1011.74 L456.668 1020.38 L456.896 1022.24 L457.124 1016.16 L457.352 1022.24 L457.579 1019.43 L457.807 1008.6 L458.035 1009.8 L458.263 1006.44 L458.491 998.723 L458.718 1005.39 L458.946 1006.12 L459.174 1008.85 L459.402 1011.93 L459.629 1006.31 L459.857 1001.66 L460.085 1006.67 L460.313 1005.22 L460.54 1006.61 L460.768 1010.58 L460.996 1012.02 L461.224 1014.15 L461.452 1015.86 L461.679 1022.42 L461.907 1016.79 L462.135 1019.6 L462.363 1019 L462.59 1017.24 L462.818 1018.01 L463.046 1014.21 L463.274 1013.68 L463.501 1015.19 L463.729 1011.71 L463.957 1017.29 L464.185 1018.39 L464.413 1020.51 L464.64 1017.46 L464.868 1017.44 L465.096 1018.3 L465.324 1020.92 L465.551 1024.55 L465.779 1026.71 L466.007 1023.79 L466.235 1026.18 L466.462 1030.06 L466.69 1033.1 L466.918 1036.27 L467.146 1034.1 L467.374 1034.77 L467.601 1036.59 L467.829 1034.72 L468.057 1036.77 L468.285 1036.38 L468.512 1041.32 L468.74 1037.91 L468.968 1034.47 L469.196 1028.8 L469.424 1029.59 L469.651 1030.25 L469.879 1020.7 L470.107 1023.58 L470.335 1019.65 L470.562 1019.51 L470.79 1023.89 L471.018 1022.86 L471.246 1031.3 L471.473 1032.61 L471.701 1033.24 L471.929 1032.28 L472.157 1025.46 L472.385 1020.41 L472.612 1022.32 L472.84 1025.96 L473.068 1027.61 L473.296 1029.94 L473.523 1028.38 L473.751 1033.75 L473.979 1031.98 L474.207 1027.77 L474.434 1027.17 L474.662 1028.67 L474.89 1030.52 L475.118 1030.21 L475.346 1037.51 L475.573 1041.04 L475.801 1045.83 L476.029 1052.24 L476.257 1052.84 L476.484 1047.84 L476.712 1050.78 L476.94 1062.23 L477.168 1063.17 L477.395 1067.15 L477.623 1064.45 L477.851 1065.31 L478.079 1062.45 L478.307 1063.23 L478.534 1063.42 L478.762 1056.68 L478.99 1053.88 L479.218 1057.27 L479.445 1054.83 L479.673 1054.68 L479.901 1052.09 L480.129 1050.82 L480.357 1050.42 L480.584 1057.28 L480.812 1059.44 L481.04 1054.63 L481.268 1055.74 L481.495 1059.68 L481.723 1063.48 L481.951 1054.98 L482.179 1057.91 L482.406 1059.94 L482.634 1068.43 L482.862 1066.67 L483.09 1067.99 L483.318 1069.43 L483.545 1068.52 L483.773 1070.08 L484.001 1064.42 L484.229 1070.52 L484.456 1072.83 L484.684 1077.28 L484.912 1073.34 L485.14 1071.51 L485.367 1065.8 L485.595 1066.37 L485.823 1069.94 L486.051 1077.66 L486.279 1078.29 L486.506 1074.98 L486.734 1078.53 L486.962 1081.57 L487.19 1080.53 L487.417 1073.65 L487.645 1075.32 L487.873 1073.22 L488.101 1071.72 L488.328 1070.15 L488.556 1066.08 L488.784 1071.08 L489.012 1064.96 L489.24 1065.31 L489.467 1068.8 L489.695 1069.03 L489.923 1068.71 L490.151 1070.85 L490.378 1072.96 L490.606 1073.99 L490.834 1076.16 L491.062 1075.68 L491.29 1067.27 L491.517 1070.53 L491.745 1067.27 L491.973 1069.04 L492.201 1070.24 L492.428 1070.34 L492.656 1069.47 L492.884 1070.78 L493.112 1072.95 L493.339 1067.31 L493.567 1068 L493.795 1068.88 L494.023 1070.57 L494.251 1067.29 L494.478 1066.83 L494.706 1062.49 L494.934 1063.85 L495.162 1064.79 L495.389 1067.25 L495.617 1067.77 L495.845 1068.58 L496.073 1068.27 L496.3 1073.25 L496.528 1073.48 L496.756 1071.8 L496.984 1074.5 L497.212 1076.25 L497.439 1077.97 L497.667 1077.84 L497.895 1078.22 L498.123 1075.54 L498.35 1074.51 L498.578 1081.2 L498.806 1083.18 L499.034 1089.02 L499.261 1089.9 L499.489 1089.65 L499.717 1089.57 L499.945 1090.18 L500.173 1092.24 L500.4 1095.98 L500.628 1092.22 L500.856 1092.03 L501.084 1095.42 L501.311 1095.21 L501.539 1098.52 L501.767 1100.37 L501.995 1098.32 L502.223 1102.54 L502.45 1102.46 L502.678 1101.96 L502.906 1105.29 L503.134 1106.89 L503.361 1101.69 L503.589 1098.93 L503.817 1097.76 L504.045 1104.11 L504.272 1102.09 L504.5 1102.04 L504.728 1097.82 L504.956 1091.96 L505.184 1091.65 L505.411 1096.17 L505.639 1094.52 L505.867 1097.5 L506.095 1101.33 L506.322 1100.82 L506.55 1097.73 L506.778 1096.18 L507.006 1097.63 L507.233 1098.88 L507.461 1099 L507.689 1098.28 L507.917 1100.17 L508.145 1100.57 L508.372 1093.11 L508.6 1088.25 L508.828 1090.28 L509.056 1086.72 L509.283 1090.11 L509.511 1088.24 L509.739 1088.21 L509.967 1089.46 L510.194 1091.44 L510.422 1094.75 L510.65 1092.41 L510.878 1091.59 L511.106 1088.66 L511.333 1089.81 L511.561 1091.35 L511.789 1096 L512.017 1096.99 L512.244 1098.31 L512.472 1100.43 L512.7 1101.07 L512.928 1103.55 L513.156 1100.64 L513.383 1103.4 L513.611 1098.13 L513.839 1099.11 L514.067 1097.21 L514.294 1101.21 L514.522 1101.1 L514.75 1106.66 L514.978 1111.52 L515.205 1112.36 L515.433 1112.07 L515.661 1115.21 L515.889 1118.14 L516.117 1119.71 L516.344 1117.2 L516.572 1116.45 L516.8 1119.5 L517.028 1122.61 L517.255 1123.17 L517.483 1119.55 L517.711 1116.97 L517.939 1114.53 L518.166 1117.38 L518.394 1117.34 L518.622 1115.64 L518.85 1114 L519.078 1110.16 L519.305 1107.32 L519.533 1105.98 L519.761 1103.25 L519.989 1105.87 L520.216 1107.94 L520.444 1107.31 L520.672 1108.61 L520.9 1106.54 L521.127 1107.38 L521.355 1107.59 L521.583 1110.87 L521.811 1108.82 L522.039 1109.82 L522.266 1110.15 L522.494 1111.48 L522.722 1113.49 L522.95 1118.1 L523.177 1116.46 L523.405 1111.11 L523.633 1108.75 L523.861 1109.32 L524.089 1106.29 L524.316 1106.86 L524.544 1106.65 L524.772 1110.83 L525 1113.74 L525.227 1112.28 L525.455 1108.34 L525.683 1113.33 L525.911 1110.94 L526.138 1106.11 L526.366 1106.97 L526.594 1113.38 L526.822 1111.94 L527.05 1112.41 L527.277 1112.99 L527.505 1110.43 L527.733 1111.51 L527.961 1110.2 L528.188 1109.82 L528.416 1104.16 L528.644 1101.11 L528.872 1100 L529.099 1096.24 L529.327 1090.95 L529.555 1088.93 L529.783 1090.67 L530.011 1091.75 L530.238 1091.93 L530.466 1087.91 L530.694 1084.65 L530.922 1085.99 L531.149 1082.57 L531.377 1085.38 L531.605 1084.95 L531.833 1081.02 L532.06 1078.05 L532.288 1080.12 L532.516 1082.3 L532.744 1079.83 L532.972 1077.33 L533.199 1079.22 L533.427 1077.23 L533.655 1074.27 L533.883 1076.08 L534.11 1076.78 L534.338 1075.75 L534.566 1074.29 L534.794 1072.34 L535.022 1077.04 L535.249 1078.23 L535.477 1079.79 L535.705 1081.7 L535.933 1082.67 L536.16 1082.15 L536.388 1083.85 L536.616 1082.93 L536.844 1084.31 L537.071 1086.04 L537.299 1089.32 L537.527 1090.77 L537.755 1093.96 L537.983 1090.09 L538.21 1094.59 L538.438 1097.56 L538.666 1099.12 L538.894 1095.79 L539.121 1097.38 L539.349 1098.29 L539.577 1098.59 L539.805 1096.9 L540.032 1099.77 L540.26 1098.7 L540.488 1098.66 L540.716 1098.11 L540.944 1100.07 L541.171 1098.97 L541.399 1099.09 L541.627 1094.97 L541.855 1094.48 L542.082 1096.5 L542.31 1091.13 L542.538 1091.32 L542.766 1088.53 L542.993 1089.69 L543.221 1091.59 L543.449 1094.22 L543.677 1097.61 L543.905 1099.93 L544.132 1098.47 L544.36 1092.46 L544.588 1094.98 L544.816 1093.62 L545.043 1092 L545.271 1087.71 L545.499 1083.94 L545.727 1084.2 L545.955 1080.74 L546.182 1078.12 L546.41 1078.61 L546.638 1077.44 L546.866 1075.4 L547.093 1078.93 L547.321 1082.13 L547.549 1082.29 L547.777 1080.77 L548.004 1080.8 L548.232 1080.92 L548.46 1080.94 L548.688 1080.81 L548.916 1082.08 L549.143 1085.52 L549.371 1085.46 L549.599 1086.31 L549.827 1087.91 L550.054 1086.37 L550.282 1084.68 L550.51 1080.91 L550.738 1086 L550.965 1088.11 L551.193 1089.25 L551.421 1090.02 L551.649 1089.92 L551.877 1083.1 L552.104 1086.69 L552.332 1088.19 L552.56 1089.16 L552.788 1091.31 L553.015 1091.28 L553.243 1087.87 L553.471 1087.34 L553.699 1089.22 L553.926 1089.05 L554.154 1089.42 L554.382 1090.06 L554.61 1090.69 L554.838 1090.73 L555.065 1093.48 L555.293 1096.63 L555.521 1098.42 L555.749 1098.59 L555.976 1099.79 L556.204 1101.21 L556.432 1098.08 L556.66 1100.58 L556.888 1104.21 L557.115 1107.65 L557.343 1106.88 L557.571 1104.86 L557.799 1106.65 L558.026 1104.28 L558.254 1108.36 L558.482 1104.44 L558.71 1103.44 L558.937 1103.21 L559.165 1101.45 L559.393 1098.24 L559.621 1097.29 L559.849 1096.24 L560.076 1096.36 L560.304 1092.31 L560.532 1095.52 L560.76 1095.87 L560.987 1095.84 L561.215 1098.15 L561.443 1098.3 L561.671 1097.78 L561.898 1098.31 L562.126 1101.52 L562.354 1099.4 L562.582 1100.15 L562.81 1097.13 L563.037 1098.59 L563.265 1099.78 L563.493 1096.76 L563.721 1097.23 L563.948 1096.27 L564.176 1096.3 L564.404 1096.73 L564.632 1095.03 L564.859 1096.62 L565.087 1099.02 L565.315 1099.2 L565.543 1099.04 L565.771 1098.61 L565.998 1102.08 L566.226 1100.51 L566.454 1097.16 L566.682 1095.2 L566.909 1097.92 L567.137 1100.35 L567.365 1098.84 L567.593 1093.22 L567.821 1092.92 L568.048 1095.91 L568.276 1097.72 L568.504 1096.24 L568.732 1098 L568.959 1092.92 L569.187 1093.09 L569.415 1094.31 L569.643 1099.26 L569.87 1098.85 L570.098 1099.82 L570.326 1102.36 L570.554 1101.92 L570.782 1102.02 L571.009 1103.16 L571.237 1107.55 L571.465 1107.3 L571.693 1107.96 L571.92 1107.24 L572.148 1107.75 L572.376 1104.47 L572.604 1104.52 L572.831 1101.33 L573.059 1098.69 L573.287 1100.39 L573.515 1096.27 L573.743 1095.56 L573.97 1096.05 L574.198 1094.72 L574.426 1092.57 L574.654 1092.11 L574.881 1093.49 L575.109 1089.59 L575.337 1090.26 L575.565 1089.34 L575.792 1089.87 L576.02 1089.23 L576.248 1085.94 L576.476 1085.42 L576.704 1083.65 L576.931 1081.4 L577.159 1078.8 L577.387 1078.26 L577.615 1077.22 L577.842 1075.09 L578.07 1075.62 L578.298 1078.97 L578.526 1078.94 L578.754 1080.66 L578.981 1082.59 L579.209 1080.65 L579.437 1082.15 L579.665 1081.53 L579.892 1082.51 L580.12 1082.85 L580.348 1081.21 L580.576 1079.05 L580.803 1080.05 L581.031 1080.49 L581.259 1079.96 L581.487 1075.27 L581.715 1074.9 L581.942 1074.44 L582.17 1076.17 L582.398 1074.41 L582.626 1072.64 L582.853 1070.71 L583.081 1071.76 L583.309 1073.98 L583.537 1076.94 L583.764 1078.3 L583.992 1078.43 L584.22 1081.55 L584.448 1083.69 L584.676 1084.48 L584.903 1085.91 L585.131 1086.52 L585.359 1087.8 L585.587 1088.71 L585.814 1091.39 L586.042 1088.24 L586.27 1089.46 L586.498 1088.75 L586.725 1086.55 L586.953 1081.51 L587.181 1081.25 L587.409 1079.04 L587.637 1077.63 L587.864 1077 L588.092 1079.83 L588.32 1078.36 L588.548 1081.68 L588.775 1082.28 L589.003 1082.16 L589.231 1082.09 L589.459 1080.68 L589.687 1081.21 L589.914 1080.22 L590.142 1080.64 L590.37 1079.42 L590.598 1080.17 L590.825 1080.94 L591.053 1081.1 L591.281 1081.86 L591.509 1081.41 L591.736 1078.41 L591.964 1082.44 L592.192 1082.84 L592.42 1083.33 L592.648 1085.34 L592.875 1084.73 L593.103 1085.59 L593.331 1086.88 L593.559 1089.93 L593.786 1090.23 L594.014 1091.08 L594.242 1091.93 L594.47 1089.99 L594.697 1092.2 L594.925 1091.4 L595.153 1092.98 L595.381 1094.74 L595.609 1095.12 L595.836 1096.31 L596.064 1095.76 L596.292 1092.38 L596.52 1090.26 L596.747 1090 L596.975 1086.94 L597.203 1087.96 L597.431 1090.38 L597.658 1089.22 L597.886 1089.51 L598.114 1088.07 L598.342 1085.84 L598.57 1085.96 L598.797 1088.18 L599.025 1086.83 L599.253 1087.75 L599.481 1083.05 L599.708 1081.76 L599.936 1081.8 L600.164 1079.02 L600.392 1081.14 L600.62 1082.7 L600.847 1080.22 L601.075 1079.05 L601.303 1078.89 L601.531 1082.42 L601.758 1080.84 L601.986 1080.61 L602.214 1080.48 L602.442 1081.82 L602.669 1082.33 L602.897 1083.69 L603.125 1079.87 L603.353 1083.3 L603.581 1084.4 L603.808 1086.09 L604.036 1090.3 L604.264 1090.65 L604.492 1094.17 L604.719 1094.24 L604.947 1092.86 L605.175 1091.14 L605.403 1090.85 L605.63 1095.91 L605.858 1094.77 L606.086 1090.46 L606.314 1091.29 L606.542 1089.16 L606.769 1089.64 L606.997 1090.22 L607.225 1090.72 L607.453 1091.05 L607.68 1090.78 L607.908 1092.12 L608.136 1093.2 L608.364 1093.25 L608.591 1094.51 L608.819 1092.83 L609.047 1092.57 L609.275 1094.25 L609.503 1093.36 L609.73 1096.59 L609.958 1091.62 L610.186 1093.47 L610.414 1093.93 L610.641 1093.02 L610.869 1091.01 L611.097 1086.83 L611.325 1087.18 L611.553 1086.99 L611.78 1085.48 L612.008 1083.4 L612.236 1083.44 L612.464 1085.45 L612.691 1085.18 L612.919 1085.51 L613.147 1084.39 L613.375 1082.25 L613.602 1081.71 L613.83 1078.75 L614.058 1075.46 L614.286 1076.47 L614.514 1077.85 L614.741 1077.39 L614.969 1073.61 L615.197 1073.75 L615.425 1072.52 L615.652 1071.29 L615.88 1072.71 L616.108 1072.86 L616.336 1072.69 L616.563 1074.31 L616.791 1074.41 L617.019 1074.25 L617.247 1073.77 L617.475 1071.76 L617.702 1071.02 L617.93 1069.48 L618.158 1070.01 L618.386 1068.43 L618.613 1066.28 L618.841 1069.99 L619.069 1071.97 L619.297 1074.33 L619.524 1074.42 L619.752 1074.67 L619.98 1077.53 L620.208 1078.23 L620.436 1076.97 L620.663 1073.53 L620.891 1071.89 L621.119 1069.29 L621.347 1067.39 L621.574 1066.39 L621.802 1068.04 L622.03 1069.87 L622.258 1072.31 L622.486 1073.48 L622.713 1074.45 L622.941 1075.32 L623.169 1073.66 L623.397 1073.16 L623.624 1071.7 L623.852 1071.69 L624.08 1072.51 L624.308 1071.67 L624.535 1071.28 L624.763 1074.23 L624.991 1076.97 L625.219 1073.87 L625.447 1072.51 L625.674 1071.37 L625.902 1074.28 L626.13 1074.13 L626.358 1071.84 L626.585 1069.92 L626.813 1071.57 L627.041 1071.6 L627.269 1071.38 L627.496 1070.89 L627.724 1072.85 L627.952 1074.99 L628.18 1075.8 L628.408 1075.09 L628.635 1075.72 L628.863 1073.9 L629.091 1075.76 L629.319 1074.31 L629.546 1074.39 L629.774 1073.6 L630.002 1075.44 L630.23 1076.2 L630.457 1078.54 L630.685 1076.55 L630.913 1071.93 L631.141 1071.4 L631.369 1071.62 L631.596 1069.15 L631.824 1070.17 L632.052 1070.14 L632.28 1070.13 L632.507 1073.87 L632.735 1072.78 L632.963 1070.42 L633.191 1070.34 L633.419 1071.31 L633.646 1070.14 L633.874 1069.8 L634.102 1072.24 L634.33 1071.32 L634.557 1072 L634.785 1069.31 L635.013 1070.5 L635.241 1072.19 L635.468 1074.03 L635.696 1074.99 L635.924 1076.98 L636.152 1078.88 L636.38 1076.94 L636.607 1077.06 L636.835 1076.97 L637.063 1076.13 L637.291 1073.91 L637.518 1075.23 L637.746 1074.17 L637.974 1072.68 L638.202 1071.91 L638.429 1072.27 L638.657 1074.73 L638.885 1071.59 L639.113 1071.45 L639.341 1072.67 L639.568 1070.36 L639.796 1071.96 L640.024 1072.67 L640.252 1075.51 L640.479 1075.8 L640.707 1072.33 L640.935 1072.72 L641.163 1069.32 L641.39 1070.15 L641.618 1067.61 L641.846 1070.11 L642.074 1068.38 L642.302 1068.49 L642.529 1069.14 L642.757 1068.54 L642.985 1067 L643.213 1067.7 L643.44 1068.43 L643.668 1066.97 L643.896 1069.48 L644.124 1067.22 L644.352 1065.62 L644.579 1065.96 L644.807 1068.83 L645.035 1069.34 L645.263 1070.09 L645.49 1066.56 L645.718 1068.35 L645.946 1071.39 L646.174 1072.7 L646.401 1073.72 L646.629 1073.14 L646.857 1074.03 L647.085 1074.86 L647.313 1076.59 L647.54 1078.16 L647.768 1078.01 L647.996 1078.28 L648.224 1078.11 L648.451 1076.58 L648.679 1077.75 L648.907 1079.38 L649.135 1081.86 L649.362 1078.66 L649.59 1075.71 L649.818 1078.05 L650.046 1077.54 L650.274 1078.18 L650.501 1077.43 L650.729 1078.89 L650.957 1077.96 L651.185 1077.86 L651.412 1076.13 L651.64 1076.97 L651.868 1078.02 L652.096 1078.36 L652.323 1076.57 L652.551 1076.9 L652.779 1075.42 L653.007 1075.08 L653.235 1075.11 L653.462 1073.7 L653.69 1071.71 L653.918 1072.47 L654.146 1073.43 L654.373 1071.28 L654.601 1069.15 L654.829 1065.76 L655.057 1065.19 L655.285 1065.82 L655.512 1064.84 L655.74 1064.89 L655.968 1065.23 L656.196 1063.79 L656.423 1061.91 L656.651 1061.35 L656.879 1062.38 L657.107 1061.41 L657.334 1061.68 L657.562 1059.06 L657.79 1060.43 L658.018 1062.1 L658.246 1063.58 L658.473 1063.76 L658.701 1060.45 L658.929 1062.16 L659.157 1062.06 L659.384 1063.7 L659.612 1066.09 L659.84 1065.5 L660.068 1065.74 L660.295 1065.93 L660.523 1068.92 L660.751 1068.31 L660.979 1068.37 L661.207 1069 L661.434 1068.72 L661.662 1067.49 L661.89 1067.79 L662.118 1069.9 L662.345 1067.85 L662.573 1067.73 L662.801 1068.39 L663.029 1068.58 L663.256 1068.68 L663.484 1069.63 L663.712 1071.33 L663.94 1067.68 L664.168 1065.52 L664.395 1067.42 L664.623 1067.12 L664.851 1066.7 L665.079 1066.65 L665.306 1068.5 L665.534 1064.26 L665.762 1065.24 L665.99 1065.08 L666.218 1066.92 L666.445 1066.43 L666.673 1066.72 L666.901 1067.39 L667.129 1066.69 L667.356 1065.88 L667.584 1062.22 L667.812 1062.38 L668.04 1060.94 L668.267 1058.58 L668.495 1059.81 L668.723 1058.2 L668.951 1056.67 L669.179 1057.39 L669.406 1056.31 L669.634 1058.38 L669.862 1057.11 L670.09 1057.56 L670.317 1058.44 L670.545 1060.12 L670.773 1060.92 L671.001 1060.27 L671.228 1063.14 L671.456 1063.04 L671.684 1064.65 L671.912 1064.43 L672.14 1063.94 L672.367 1062.79 L672.595 1064.6 L672.823 1063.82 L673.051 1064.06 L673.278 1063.35 L673.506 1061.83 L673.734 1063.18 L673.962 1064.46 L674.189 1061.59 L674.417 1062.49 L674.645 1060.88 L674.873 1062.59 L675.101 1060.05 L675.328 1060.6 L675.556 1060.8 L675.784 1061.49 L676.012 1059.76 L676.239 1058.99 L676.467 1060.44 L676.695 1062.93 L676.923 1060.75 L677.151 1061.32 L677.378 1063.97 L677.606 1062.16 L677.834 1061.71 L678.062 1063.88 L678.289 1061.59 L678.517 1061.18 L678.745 1061.12 L678.973 1059.68 L679.2 1059.36 L679.428 1061.85 L679.656 1062.47 L679.884 1064.13 L680.112 1064.34 L680.339 1064.29 L680.567 1065.18 L680.795 1067.08 L681.023 1064.96 L681.25 1065.24 L681.478 1064.23 L681.706 1064.24 L681.934 1064.28 L682.161 1062.62 L682.389 1061.14 L682.617 1058.58 L682.845 1059.74 L683.073 1057.73 L683.3 1056.23 L683.528 1053.54 L683.756 1050.85 L683.984 1047.9 L684.211 1048.37 L684.439 1048.92 L684.667 1050.91 L684.895 1051.43 L685.122 1049.28 L685.35 1051.68 L685.578 1050.34 L685.806 1050.15 L686.034 1049.87 L686.261 1050.36 L686.489 1050.67 L686.717 1053.07 L686.945 1054 L687.172 1053.88 L687.4 1056.3 L687.628 1056.76 L687.856 1058.15 L688.084 1059.62 L688.311 1060.92 L688.539 1061.54 L688.767 1061.46 L688.995 1059.83 L689.222 1061.38 L689.45 1061.9 L689.678 1062.9 L689.906 1062.32 L690.133 1061.47 L690.361 1062.82 L690.589 1063.72 L690.817 1063.84 L691.045 1063.44 L691.272 1060.27 L691.5 1061.01 L691.728 1061.93 L691.956 1063.49 L692.183 1064.88 L692.411 1062.1 L692.639 1061.32 L692.867 1061.45 L693.094 1062.83 L693.322 1064.52 L693.55 1067.66 L693.778 1066.94 L694.006 1064.68 L694.233 1064.87 L694.461 1067.95 L694.689 1068.1 L694.917 1069.13 L695.144 1071.33 L695.372 1072.83 L695.6 1073.81 L695.828 1075.58 L696.055 1074.62 L696.283 1072.87 L696.511 1073.47 L696.739 1073.96 L696.967 1073.99 L697.194 1075.86 L697.422 1076.9 L697.65 1074.99 L697.878 1074.25 L698.105 1075.1 L698.333 1074.8 L698.561 1074.68 L698.789 1075.22 L699.017 1077.06 L699.244 1077.7 L699.472 1079.14 L699.7 1078.62 L699.928 1079.09 L700.155 1079.06 L700.383 1081.12 L700.611 1082.04 L700.839 1081.99 L701.066 1081.13 L701.294 1079.79 L701.522 1080.67 L701.75 1082.39 L701.978 1082.1 L702.205 1082.46 L702.433 1081.11 L702.661 1081.05 L702.889 1079.67 L703.116 1082.08 L703.344 1080.37 L703.572 1080.57 L703.8 1081.79 L704.027 1082.11 L704.255 1077.37 L704.483 1078.65 L704.711 1075.25 L704.939 1077.47 L705.166 1080.03 L705.394 1080.53 L705.622 1079.74 L705.85 1080.97 L706.077 1081.01 L706.305 1082.13 L706.533 1083.22 L706.761 1082.47 L706.988 1081.27 L707.216 1081.15 L707.444 1081.07 L707.672 1079.96 L707.9 1080.87 L708.127 1079.84 L708.355 1080.35 L708.583 1081.51 L708.811 1081.66 L709.038 1080.21 L709.266 1079.31 L709.494 1076.96 L709.722 1076.63 L709.95 1080.48 L710.177 1079.01 L710.405 1078.42 L710.633 1077.35 L710.861 1076.96 L711.088 1078.73 L711.316 1078.9 L711.544 1079.49 L711.772 1078.83 L711.999 1080.26 L712.227 1080.46 L712.455 1079.98 L712.683 1081.69 L712.911 1081.62 L713.138 1080.1 L713.366 1079.42 L713.594 1081.07 L713.822 1081.06 L714.049 1081.41 L714.277 1081.81 L714.505 1083.24 L714.733 1083.15 L714.96 1085.42 L715.188 1087.47 L715.416 1088.05 L715.644 1090.42 L715.872 1093.51 L716.099 1095.06 L716.327 1092.8 L716.555 1092.28 L716.783 1092.71 L717.01 1094.78 L717.238 1096.15 L717.466 1097.75 L717.694 1095.75 L717.921 1098.53 L718.149 1100.1 L718.377 1097.71 L718.605 1098.04 L718.833 1099 L719.06 1096.56 L719.288 1095.44 L719.516 1094.36 L719.744 1094.18 L719.971 1093.58 L720.199 1094.03 L720.427 1094.44 L720.655 1094.43 L720.883 1096.12 L721.11 1098.59 L721.338 1098.43 L721.566 1099.65 L721.794 1100.66 L722.021 1098.47 L722.249 1096.89 L722.477 1098.71 L722.705 1099.34 L722.932 1100.84 L723.16 1099.64 L723.388 1100.51 L723.616 1100.92 L723.844 1101.94 L724.071 1099.25 L724.299 1098.53 L724.527 1099.28 L724.755 1099.4 L724.982 1102.36 L725.21 1103.89 L725.438 1104.46 L725.666 1104.05 L725.893 1100.64 L726.121 1098.98 L726.349 1100.84 L726.577 1101.44 L726.805 1102.85 L727.032 1101.96 L727.26 1104.52 L727.488 1105.77 L727.716 1107.34 L727.943 1106.69 L728.171 1104.89 L728.399 1104.24 L728.627 1103.14 L728.854 1102.6 L729.082 1100.99 L729.31 1103.01 L729.538 1103 L729.766 1101.9 L729.993 1102.22 L730.221 1101.67 L730.449 1102.43 L730.677 1100.85 L730.904 1099.42 L731.132 1101.33 L731.36 1102.68 L731.588 1102.57 L731.816 1102 L732.043 1103.05 L732.271 1101.43 L732.499 1102.78 L732.727 1101.77 L732.954 1100.66 L733.182 1101.57 L733.41 1102.37 L733.638 1101.29 L733.865 1103.52 L734.093 1105.62 L734.321 1105.94 L734.549 1104.84 L734.777 1105.71 L735.004 1108.34 L735.232 1107.97 L735.46 1107.91 L735.688 1107.34 L735.915 1104.91 L736.143 1103.05 L736.371 1104.01 L736.599 1104.35 L736.826 1105.03 L737.054 1103.37 L737.282 1104.47 L737.51 1104.18 L737.738 1102.56 L737.965 1102.92 L738.193 1102.96 L738.421 1103.64 L738.649 1105.24 L738.876 1103.37 L739.104 1104.09 L739.332 1104.33 L739.56 1105.44 L739.787 1107.52 L740.015 1107.74 L740.243 1108.26 L740.471 1109.8 L740.699 1109.22 L740.926 1109.63 L741.154 1109.76 L741.382 1109.19 L741.61 1111.25 L741.837 1111.93 L742.065 1113.96 L742.293 1115.4 L742.521 1115.22 L742.749 1112.01 L742.976 1112.37 L743.204 1112.52 L743.432 1112.79 L743.66 1109.96 L743.887 1107.21 L744.115 1107.76 L744.343 1106.73 L744.571 1108 L744.798 1108.36 L745.026 1106.33 L745.254 1106.22 L745.482 1105.83 L745.71 1105.44 L745.937 1106.5 L746.165 1106.47 L746.393 1105.48 L746.621 1105.84 L746.848 1106.06 L747.076 1105.24 L747.304 1106.44 L747.532 1106.46 L747.759 1107.63 L747.987 1106.53 L748.215 1107.98 L748.443 1109.29 L748.671 1110.32 L748.898 1109.85 L749.126 1110.3 L749.354 1109.37 L749.582 1109.06 L749.809 1108.4 L750.037 1109.34 L750.265 1106.77 L750.493 1105.77 L750.72 1103.05 L750.948 1102.08 L751.176 1098.84 L751.404 1098.83 L751.632 1101.1 L751.859 1101.35 L752.087 1101.37 L752.315 1101.54 L752.543 1100.83 L752.77 1103.41 L752.998 1104.08 L753.226 1104.67 L753.454 1103.65 L753.682 1100.75 L753.909 1103.68 L754.137 1099.79 L754.365 1100.19 L754.593 1099.81 L754.82 1098.74 L755.048 1099.33 L755.276 1099.71 L755.504 1100.76 L755.731 1098.53 L755.959 1098.87 L756.187 1096.98 L756.415 1097.86 L756.643 1096.86 L756.87 1093.52 L757.098 1094.48 L757.326 1092.6 L757.554 1091.4 L757.781 1090.93 L758.009 1089.36 L758.237 1090.25 L758.465 1090.6 L758.692 1089.99 L758.92 1090.53 L759.148 1090.33 L759.376 1091.27 L759.604 1092.8 L759.831 1092.92 L760.059 1093.55 L760.287 1093.85 L760.515 1094.43 L760.742 1095.17 L760.97 1094.85 L761.198 1095.96 L761.426 1097.86 L761.653 1096.47 L761.881 1095.48 L762.109 1093.74 L762.337 1091.74 L762.565 1091.7 L762.792 1092.7 L763.02 1095.01 L763.248 1092.41 L763.476 1090.79 L763.703 1092.53 L763.931 1089.29 L764.159 1087.52 L764.387 1089.2 L764.615 1088.6 L764.842 1090.61 L765.07 1090.68 L765.298 1093.06 L765.526 1093 L765.753 1094.48 L765.981 1093.72 L766.209 1091.85 L766.437 1094.32 L766.664 1094.65 L766.892 1094.52 L767.12 1096.27 L767.348 1097.51 L767.576 1097.39 L767.803 1098.5 L768.031 1098.42 L768.259 1099.95 L768.487 1101.52 L768.714 1101.66 L768.942 1101.43 L769.17 1102.1 L769.398 1102.19 L769.625 1102.76 L769.853 1101.95 L770.081 1097.64 L770.309 1096.93 L770.537 1095.44 L770.764 1096.22 L770.992 1095.42 L771.22 1092.88 L771.448 1092.74 L771.675 1092.06 L771.903 1094.26 L772.131 1095.15 L772.359 1095.65 L772.586 1094.87 L772.814 1094.23 L773.042 1091.12 L773.27 1090.04 L773.498 1088.42 L773.725 1087.82 L773.953 1084.83 L774.181 1084.85 L774.409 1082.34 L774.636 1082.3 L774.864 1082.27 L775.092 1082.44 L775.32 1081.99 L775.548 1079.64 L775.775 1080.66 L776.003 1081.69 L776.231 1082.68 L776.459 1082.48 L776.686 1081.82 L776.914 1081.45 L777.142 1081.75 L777.37 1080.32 L777.597 1082.29 L777.825 1083.42 L778.053 1085.47 L778.281 1084.66 L778.509 1083.93 L778.736 1084.26 L778.964 1085.36 L779.192 1084.76 L779.42 1085.74 L779.647 1087.87 L779.875 1088.79 L780.103 1087.03 L780.331 1086.11 L780.558 1088.49 L780.786 1088.33 L781.014 1089.08 L781.242 1089.15 L781.47 1089.07 L781.697 1087.93 L781.925 1088.72 L782.153 1089.32 L782.381 1090.83 L782.608 1091.99 L782.836 1090.63 L783.064 1090.49 L783.292 1092.01 L783.519 1092.48 L783.747 1092.32 L783.975 1092.84 L784.203 1093.02 L784.431 1091.22 L784.658 1092.54 L784.886 1093.57 L785.114 1092.92 L785.342 1091.68 L785.569 1093.26 L785.797 1092.26 L786.025 1090.62 L786.253 1091.52 L786.481 1093.41 L786.708 1095.1 L786.936 1092.71 L787.164 1092.03 L787.392 1091.3 L787.619 1091.87 L787.847 1093.08 L788.075 1094.59 L788.303 1094.43 L788.53 1095.03 L788.758 1095.45 L788.986 1095.77 L789.214 1097.03 L789.442 1094.49 L789.669 1096.58 L789.897 1096.07 L790.125 1096.78 L790.353 1096.28 L790.58 1095.56 L790.808 1096.72 L791.036 1097.72 L791.264 1093.89 L791.491 1095.68 L791.719 1095.29 L791.947 1094.11 L792.175 1093.48 L792.403 1093.87 L792.63 1094 L792.858 1094.54 L793.086 1093.28 L793.314 1094.85 L793.541 1094.98 L793.769 1096.53 L793.997 1094.41 L794.225 1094.61 L794.452 1094.21 L794.68 1093.21 L794.908 1093.42 L795.136 1090.39 L795.364 1089.24 L795.591 1089.64 L795.819 1089.26 L796.047 1090.45 L796.275 1090.68 L796.502 1090.76 L796.73 1088.91 L796.958 1092.19 L797.186 1093.01 L797.413 1093.68 L797.641 1093.37 L797.869 1093.58 L798.097 1093.84 L798.325 1093.94 L798.552 1093.33 L798.78 1093.59 L799.008 1092.8 L799.236 1090.1 L799.463 1090.59 L799.691 1091.91 L799.919 1091.76 L800.147 1091.3 L800.375 1092.39 L800.602 1092.12 L800.83 1090.56 L801.058 1089.86 L801.286 1092.03 L801.513 1093.98 L801.741 1093.15 L801.969 1094.53 L802.197 1094.34 L802.424 1094.39 L802.652 1092.83 L802.88 1092.12 L803.108 1093.04 L803.336 1094.86 L803.563 1095.76 L803.791 1095.72 L804.019 1096.97 L804.247 1093.75 L804.474 1093.56 L804.702 1094.09 L804.93 1093.68 L805.158 1094.13 L805.385 1092.94 L805.613 1092.85 L805.841 1093.28 L806.069 1092.4 L806.297 1092.37 L806.524 1091.93 L806.752 1092.15 L806.98 1092.01 L807.208 1090.98 L807.435 1092.72 L807.663 1095 L807.891 1095.45 L808.119 1095.77 L808.346 1096.05 L808.574 1094.64 L808.802 1095.21 L809.03 1095.3 L809.258 1095.03 L809.485 1095.31 L809.713 1096.51 L809.941 1095.87 L810.169 1096.3 L810.396 1095.34 L810.624 1095.65 L810.852 1095.31 L811.08 1094.96 L811.308 1095.97 L811.535 1096.27 L811.763 1097.96 L811.991 1096.83 L812.219 1097.17 L812.446 1097.82 L812.674 1097.63 L812.902 1097.2 L813.13 1096.87 L813.357 1097.28 L813.585 1097.6 L813.813 1099.02 L814.041 1099.04 L814.269 1100.86 L814.496 1098.38 L814.724 1098.07 L814.952 1097.84 L815.18 1096.74 L815.407 1095.08 L815.635 1094.57 L815.863 1096.23 L816.091 1096.12 L816.318 1095.7 L816.546 1096.84 L816.774 1097.22 L817.002 1096.1 L817.23 1094.38 L817.457 1094.93 L817.685 1093.75 L817.913 1094.52 L818.141 1094.53 L818.368 1092.3 L818.596 1090.73 L818.824 1089.92 L819.052 1090.76 L819.279 1091.46 L819.507 1090.05 L819.735 1090.04 L819.963 1091.18 L820.191 1092.08 L820.418 1091.87 L820.646 1091.61 L820.874 1092.89 L821.102 1093.82 L821.329 1094.02 L821.557 1092.63 L821.785 1093.75 L822.013 1092.45 L822.241 1093.82 L822.468 1095.32 L822.696 1097.1 L822.924 1098.3 L823.152 1098.4 L823.379 1098.73 L823.607 1099.72 L823.835 1098.03 L824.063 1099.43 L824.29 1100.2 L824.518 1101.81 L824.746 1101.46 L824.974 1103.29 L825.202 1103.68 L825.429 1103.62 L825.657 1102.87 L825.885 1105.32 L826.113 1106.21 L826.34 1106.37 L826.568 1106.35 L826.796 1106.23 L827.024 1107.23 L827.251 1107.61 L827.479 1106.33 L827.707 1107.4 L827.935 1107.24 L828.163 1108.19 L828.39 1109.41 L828.618 1110.36 L828.846 1110.56 L829.074 1106.92 L829.301 1107.7 L829.529 1108.69 L829.757 1109.06 L829.985 1109.98 L830.212 1108.81 L830.44 1109.64 L830.668 1106.73 L830.896 1107.49 L831.124 1108.47 L831.351 1109.68 L831.579 1110.95 L831.807 1111.94 L832.035 1112.47 L832.262 1113.25 L832.49 1112.72 L832.718 1113.42 L832.946 1113.11 L833.174 1113.68 L833.401 1114.41 L833.629 1114.08 L833.857 1114.75 L834.085 1112.31 L834.312 1112.62 L834.54 1113.42 L834.768 1112.07 L834.996 1112.09 L835.223 1110.83 L835.451 1109.06 L835.679 1108.25 L835.907 1107.86 L836.135 1107.58 L836.362 1107.98 L836.59 1108.06 L836.818 1108.98 L837.046 1109.73 L837.273 1109.8 L837.501 1110.93 L837.729 1109.91 L837.957 1111.25 L838.184 1113.19 L838.412 1113.63 L838.64 1113.85 L838.868 1114.51 L839.096 1112.16 L839.323 1112.19 L839.551 1112.92 L839.779 1113.49 L840.007 1112.72 L840.234 1113.19 L840.462 1114.51 L840.69 1116.27 L840.918 1116.26 L841.145 1118.41 L841.373 1117.2 L841.601 1116.27 L841.829 1116.36 L842.057 1116.31 L842.284 1116.02 L842.512 1116.67 L842.74 1116.49 L842.968 1116.32 L843.195 1117.12 L843.423 1117.63 L843.651 1117.9 L843.879 1117.5 L844.107 1116.09 L844.334 1115.95 L844.562 1116.94 L844.79 1114.72 L845.018 1114.73 L845.245 1115.27 L845.473 1116.62 L845.701 1116.47 L845.929 1114.79 L846.156 1114.64 L846.384 1112.87 L846.612 1112.62 L846.84 1111.8 L847.068 1114.41 L847.295 1115.83 L847.523 1117.02 L847.751 1117.62 L847.979 1117.75 L848.206 1119.04 L848.434 1119.06 L848.662 1120.03 L848.89 1119.58 L849.117 1121.01 L849.345 1122.93 L849.573 1122.46 L849.801 1122.96 L850.029 1123.15 L850.256 1123.45 L850.484 1124.05 L850.712 1124.03 L850.94 1123.66 L851.167 1123.42 L851.395 1125.48 L851.623 1125.58 L851.851 1126.29 L852.078 1125.61 L852.306 1125.21 L852.534 1124.57 L852.762 1125.29 L852.99 1125.67 L853.217 1125.95 L853.445 1128.09 L853.673 1126.63 L853.901 1126.66 L854.128 1128.94 L854.356 1129.55 L854.584 1128.14 L854.812 1126.62 L855.04 1127.46 L855.267 1128.27 L855.495 1128.44 L855.723 1129.11 L855.951 1129.81 L856.178 1128.42 L856.406 1130.14 L856.634 1130.37 L856.862 1132.26 L857.089 1131.63 L857.317 1131.89 L857.545 1132.31 L857.773 1132.31 L858.001 1133.35 L858.228 1133.89 L858.456 1131.83 L858.684 1131.06 L858.912 1128.7 L859.139 1128.98 L859.367 1129.13 L859.595 1130.1 L859.823 1129.71 L860.05 1130.43 L860.278 1131.82 L860.506 1133 L860.734 1132.86 L860.962 1132.4 L861.189 1132.71 L861.417 1133.12 L861.645 1132.84 L861.873 1131.14 L862.1 1130.63 L862.328 1130.47 L862.556 1130.68 L862.784 1130.13 L863.011 1131.2 L863.239 1131.7 L863.467 1132.38 L863.695 1134.13 L863.923 1133.24 L864.15 1132.88 L864.378 1133.97 L864.606 1134.52 L864.834 1136.07 L865.061 1137.05 L865.289 1138.06 L865.517 1138.36 L865.745 1138.06 L865.973 1138.01 L866.2 1138.38 L866.428 1137.87 L866.656 1138.88 L866.884 1138.94 L867.111 1137.54 L867.339 1138.5 L867.567 1137.13 L867.795 1137.89 L868.022 1138.4 L868.25 1138.26 L868.478 1137.18 L868.706 1136.11 L868.934 1135.42 L869.161 1136.61 L869.389 1135.73 L869.617 1135.69 L869.845 1134.28 L870.072 1134.56 L870.3 1133.54 L870.528 1132.92 L870.756 1133.72 L870.983 1133.32 L871.211 1133.45 L871.439 1132.4 L871.667 1131.27 L871.895 1132.07 L872.122 1131.39 L872.35 1130.56 L872.578 1130.72 L872.806 1131.29 L873.033 1131.57 L873.261 1131.69 L873.489 1131.54 L873.717 1130.73 L873.944 1128.91 L874.172 1129.65 L874.4 1128.98 L874.628 1129.54 L874.856 1129.29 L875.083 1129.78 L875.311 1128.42 L875.539 1128.94 L875.767 1128.26 L875.994 1129.13 L876.222 1129.24 L876.45 1129.18 L876.678 1129.2 L876.906 1129.49 L877.133 1126.93 L877.361 1126.54 L877.589 1125.76 L877.817 1126.75 L878.044 1127.46 L878.272 1128.87 L878.5 1128.27 L878.728 1127.01 L878.955 1126.94 L879.183 1127.33 L879.411 1127.6 L879.639 1130.15 L879.867 1130.59 L880.094 1129.88 L880.322 1131.25 L880.55 1130.24 L880.778 1130.84 L881.005 1130.5 L881.233 1128.86 L881.461 1128.95 L881.689 1128.1 L881.916 1127.79 L882.144 1128.81 L882.372 1129.03 L882.6 1129.54 L882.828 1128.36 L883.055 1128.44 L883.283 1127.85 L883.511 1128.2 L883.739 1126.47 L883.966 1126.15 L884.194 1127.74 L884.422 1128.65 L884.65 1127.43 L884.877 1128.2 L885.105 1127.34 L885.333 1128.16 L885.561 1127.15 L885.789 1128.58 L886.016 1129.58 L886.244 1129.69 L886.472 1130.67 L886.7 1130.71 L886.927 1131.72 L887.155 1133.02 L887.383 1132.36 L887.611 1132.28 L887.839 1132.15 L888.066 1133.92 L888.294 1134.16 L888.522 1135.27 L888.75 1135.7 L888.977 1134.56 L889.205 1134.89 L889.433 1134.06 L889.661 1132.67 L889.888 1132.72 L890.116 1133.2 L890.344 1134.2 L890.572 1134.47 L890.8 1133.08 L891.027 1134.78 L891.255 1135.57 L891.483 1136.62 L891.711 1136.66 L891.938 1135.8 L892.166 1135.9 L892.394 1136.09 L892.622 1135.44 L892.849 1136.04 L893.077 1135.16 L893.305 1136.32 L893.533 1137.39 L893.761 1137.65 L893.988 1136.78 L894.216 1137.04 L894.444 1137.8 L894.672 1136.98 L894.899 1137.53 L895.127 1138.79 L895.355 1139.46 L895.583 1138.69 L895.81 1138.47 L896.038 1135.35 L896.266 1136 L896.494 1135.86 L896.722 1136.72 L896.949 1137.73 L897.177 1138.28 L897.405 1137.3 L897.633 1138.83 L897.86 1139.54 L898.088 1139 L898.316 1140.15 L898.544 1139.27 L898.772 1139.48 L898.999 1140.17 L899.227 1141.93 L899.455 1141.26 L899.683 1138.86 L899.91 1138.45 L900.138 1138.91 L900.366 1138.95 L900.594 1140.31 L900.821 1142.27 L901.049 1144.18 L901.277 1144.26 L901.505 1143.27 L901.733 1143.55 L901.96 1141.76 L902.188 1143.21 L902.416 1144.42 L902.644 1145.58 L902.871 1145.5 L903.099 1146.57 L903.327 1146.23 L903.555 1144.09 L903.782 1144.02 L904.01 1143.7 L904.238 1142.98 L904.466 1141.48 L904.694 1140.85 L904.921 1141.77 L905.149 1140.28 L905.377 1138.98 L905.605 1137.53 L905.832 1137.53 L906.06 1138.25 L906.288 1138.46 L906.516 1138.51 L906.743 1138.62 L906.971 1138.4 L907.199 1139.59 L907.427 1138.72 L907.655 1138.02 L907.882 1137.75 L908.11 1138.39 L908.338 1138.78 L908.566 1138.96 L908.793 1138.85 L909.021 1139.06 L909.249 1138.04 L909.477 1137.78 L909.705 1138.13 L909.932 1137.24 L910.16 1139.03 L910.388 1138.6 L910.616 1138.49 L910.843 1139.56 L911.071 1140.01 L911.299 1140.48 L911.527 1139.02 L911.754 1137.97 L911.982 1137.88 L912.21 1138.01 L912.438 1138.57 L912.666 1139.07 L912.893 1139.71 L913.121 1140.85 L913.349 1143.41 L913.577 1142.92 L913.804 1142.52 L914.032 1143.42 L914.26 1143.38 L914.488 1144.21 L914.715 1145.62 L914.943 1145.02 L915.171 1145 L915.399 1144.97 L915.627 1144.64 L915.854 1143.91 L916.082 1144.42 L916.31 1142.95 L916.538 1142.02 L916.765 1140.82 L916.993 1140.13 L917.221 1139.8 L917.449 1139.22 L917.676 1140.14 L917.904 1140.16 L918.132 1138.4 L918.36 1138.59 L918.588 1139.5 L918.815 1138.98 L919.043 1140.29 L919.271 1139.06 L919.499 1138.94 L919.726 1138.39 L919.954 1139.53 L920.182 1139.46 L920.41 1138.74 L920.638 1139.28 L920.865 1139.67 L921.093 1137.19 L921.321 1137.71 L921.549 1134.79 L921.776 1136.09 L922.004 1135.6 L922.232 1136.25 L922.46 1136.56 L922.687 1136.54 L922.915 1136.77 L923.143 1136.71 L923.371 1137.56 L923.599 1135.86 L923.826 1136.13 L924.054 1136.87 L924.282 1137.69 L924.51 1136.2 L924.737 1135.15 L924.965 1135.72 L925.193 1136.77 L925.421 1137.27 L925.648 1138.73 L925.876 1137.38 L926.104 1138.21 L926.332 1137.45 L926.56 1137.66 L926.787 1137.82 L927.015 1137.55 L927.243 1137.96 L927.471 1138.89 L927.698 1137.93 L927.926 1138.39 L928.154 1137.82 L928.382 1138.32 L928.609 1135.62 L928.837 1133.81 L929.065 1134.74 L929.293 1134.36 L929.521 1134.13 L929.748 1133.44 L929.976 1131.8 L930.204 1133.61 L930.432 1134.94 L930.659 1134.31 L930.887 1136.35 L931.115 1137.47 L931.343 1136.58 L931.571 1136.53 L931.798 1136.35 L932.026 1137.4 L932.254 1137.44 L932.482 1139.4 L932.709 1138.75 L932.937 1137.58 L933.165 1137.41 L933.393 1136.15 L933.62 1136.28 L933.848 1137.02 L934.076 1137.77 L934.304 1137.24 L934.532 1138.7 L934.759 1139.9 L934.987 1141.51 L935.215 1141.04 L935.443 1141 L935.67 1141.46 L935.898 1141.38 L936.126 1141.81 L936.354 1142.65 L936.581 1143.88 L936.809 1143.09 L937.037 1143.12 L937.265 1144.07 L937.493 1144.06 L937.72 1145.11 L937.948 1144.97 L938.176 1145.37 L938.404 1146.67 L938.631 1146.17 L938.859 1146.66 L939.087 1146.28 L939.315 1146.66 L939.542 1145.49 L939.77 1144.93 L939.998 1145.23 L940.226 1145.37 L940.454 1147.17 L940.681 1148 L940.909 1148.47 L941.137 1149.86 L941.365 1150.17 L941.592 1151.08 L941.82 1151.35 L942.048 1151.85 L942.276 1152.06 L942.504 1152.29 L942.731 1153.44 L942.959 1153.63 L943.187 1154.06 L943.415 1153.45 L943.642 1153.99 L943.87 1154.22 L944.098 1154.6 L944.326 1153.17 L944.553 1153.8 L944.781 1153.31 L945.009 1153.66 L945.237 1154.15 L945.465 1153.68 L945.692 1153.69 L945.92 1154.2 L946.148 1153.17 L946.376 1151.42 L946.603 1152.84 L946.831 1153.86 L947.059 1154.11 L947.287 1154.21 L947.514 1153.6 L947.742 1153.58 L947.97 1154.36 L948.198 1155.67 L948.426 1154.27 L948.653 1152.94 L948.881 1154.32 L949.109 1154.21 L949.337 1154.37 L949.564 1154.18 L949.792 1154.72 L950.02 1154.95 L950.248 1156.44 L950.475 1154.45 L950.703 1154.46 L950.931 1153.5 L951.159 1153.14 L951.387 1152.35 L951.614 1153.23 L951.842 1153.31 L952.07 1152.69 L952.298 1152.47 L952.525 1150.89 L952.753 1149.43 L952.981 1148.08 L953.209 1148.12 L953.437 1147.92 L953.664 1147.23 L953.892 1146.86 L954.12 1147.62 L954.348 1148.21 L954.575 1147.75 L954.803 1146.51 L955.031 1144.87 L955.259 1144.28 L955.486 1143.71 L955.714 1143.51 L955.942 1144.05 L956.17 1143.45 L956.398 1143.98 L956.625 1142.66 L956.853 1140.37 L957.081 1140.47 L957.309 1140.39 L957.536 1140.79 L957.764 1141.02 L957.992 1141.84 L958.22 1139.15 L958.447 1138.43 L958.675 1138.83 L958.903 1139.03 L959.131 1139.9 L959.359 1140.32 L959.586 1141.59 L959.814 1140.03 L960.042 1140.6 L960.27 1140.69 L960.497 1140.68 L960.725 1140.82 L960.953 1141.29 L961.181 1141.17 L961.408 1140.57 L961.636 1140.43 L961.864 1139.06 L962.092 1139.05 L962.32 1139.86 L962.547 1141.48 L962.775 1142.15 L963.003 1142.51 L963.231 1143.3 L963.458 1142.04 L963.686 1143.17 L963.914 1142.14 L964.142 1141.65 L964.37 1142.03 L964.597 1142.75 L964.825 1141.79 L965.053 1142.05 L965.281 1142.39 L965.508 1142.24 L965.736 1142.8 L965.964 1142.79 L966.192 1142.59 L966.419 1141.13 L966.647 1142.15 L966.875 1140.71 L967.103 1139.56 L967.331 1140.91 L967.558 1141.86 L967.786 1142.97 L968.014 1141.28 L968.242 1141.62 L968.469 1140.53 L968.697 1139.96 L968.925 1139.31 L969.153 1138.53 L969.38 1138.54 L969.608 1139.93 L969.836 1139.77 L970.064 1138.45 L970.292 1138.49 L970.519 1136.95 L970.747 1136.22 L970.975 1135.86 L971.203 1136.62 L971.43 1136.18 L971.658 1136.39 L971.886 1136.31 L972.114 1135.42 L972.341 1134.66 L972.569 1135.03 L972.797 1134.69 L973.025 1135.68 L973.253 1134.18 L973.48 1134.77 L973.708 1133.44 L973.936 1132.86 L974.164 1133.12 L974.391 1133.15 L974.619 1131.69 L974.847 1132.5 L975.075 1132.37 L975.303 1130.9 L975.53 1129.88 L975.758 1129.91 L975.986 1130.06 L976.214 1130.52 L976.441 1131.14 L976.669 1131.41 L976.897 1131.36 L977.125 1131.95 L977.352 1132.48 L977.58 1132.5 L977.808 1132.28 L978.036 1133.6 L978.264 1133.77 L978.491 1133.56 L978.719 1134.06 L978.947 1134.01 L979.175 1134.78 L979.402 1134.85 L979.63 1135.04 L979.858 1134.54 L980.086 1134.26 L980.313 1134.23 L980.541 1133.26 L980.769 1133.39 L980.997 1131.92 L981.225 1131.37 L981.452 1132.52 L981.68 1130.79 L981.908 1130.69 L982.136 1131.09 L982.363 1132.96 L982.591 1131.87 L982.819 1130.68 L983.047 1131.05 L983.274 1132.52 L983.502 1132.55 L983.73 1133.57 L983.958 1133.04 L984.186 1133.16 L984.413 1133.81 L984.641 1135.9 L984.869 1135.53 L985.097 1134.57 L985.324 1134.33 L985.552 1134.05 L985.78 1134.9 L986.008 1135.76 L986.236 1137.52 L986.463 1139.08 L986.691 1138.74 L986.919 1140.28 L987.147 1140.6 L987.374 1142.11 L987.602 1143.82 L987.83 1143.91 L988.058 1144.14 L988.285 1143.9 L988.513 1144.92 L988.741 1144.21 L988.969 1143.09 L989.197 1143.05 L989.424 1141.14 L989.652 1140.08 L989.88 1139.79 L990.108 1140.06 L990.335 1140.33 L990.563 1141.74 L990.791 1141.31 L991.019 1140.77 L991.246 1140.72 L991.474 1141.02 L991.702 1141.26 L991.93 1141.21 L992.158 1140.86 L992.385 1138.25 L992.613 1137.49 L992.841 1138.02 L993.069 1138.73 L993.296 1139.73 L993.524 1138.31 L993.752 1138.78 L993.98 1139.35 L994.207 1138.02 L994.435 1137.6 L994.663 1137.19 L994.891 1137.04 L995.119 1136.79 L995.346 1137.69 L995.574 1137.42 L995.802 1137.48 L996.03 1137.04 L996.257 1137.92 L996.485 1138.13 L996.713 1138.81 L996.941 1138.83 L997.169 1139.43 L997.396 1139.75 L997.624 1139.7 L997.852 1139.06 L998.08 1139.81 L998.307 1138.96 L998.535 1139.15 L998.763 1139.4 L998.991 1139.63 L999.218 1140.02 L999.446 1139.83 L999.674 1139.33 L999.902 1140.33 L1000.13 1140.24 L1000.36 1140.86 L1000.59 1140.81 L1000.81 1140.36 L1001.04 1141.96 L1001.27 1139.96 L1001.5 1141.82 L1001.72 1141.16 L1001.95 1141.63 L1002.18 1140.69 L1002.41 1141.68 L1002.64 1141.91 L1002.86 1140.66 L1003.09 1139.78 L1003.32 1139.34 L1003.55 1139.48 L1003.77 1138.3 L1004 1139.11 L1004.23 1138.72 L1004.46 1139.14 L1004.68 1139.54 L1004.91 1139.34 L1005.14 1140.7 L1005.37 1140.87 L1005.6 1140.6 L1005.82 1140.63 L1006.05 1140.14 L1006.28 1140.72 L1006.51 1142.36 L1006.73 1142.97 L1006.96 1144.07 L1007.19 1142.48 L1007.42 1142.63 L1007.65 1142.82 L1007.87 1142.66 L1008.1 1143.73 L1008.33 1143.87 L1008.56 1143.36 L1008.78 1143.17 L1009.01 1143.95 L1009.24 1143.53 L1009.47 1144.28 L1009.7 1144.61 L1009.92 1145.43 L1010.15 1144.68 L1010.38 1144.43 L1010.61 1144.78 L1010.83 1144.57 L1011.06 1143.27 L1011.29 1142.35 L1011.52 1143.31 L1011.75 1141.77 L1011.97 1142.96 L1012.2 1144.26 L1012.43 1144.65 L1012.66 1144.77 L1012.88 1144.91 L1013.11 1143.88 L1013.34 1144.04 L1013.57 1144.36 L1013.8 1143.97 L1014.02 1144.17 L1014.25 1144.27 L1014.48 1142.97 L1014.71 1142.96 L1014.93 1143.41 L1015.16 1142.78 L1015.39 1142.72 L1015.62 1143 L1015.85 1143.67 L1016.07 1142.53 L1016.3 1141.96 L1016.53 1140.63 L1016.76 1140.69 L1016.98 1141.34 L1017.21 1140.11 L1017.44 1140.02 L1017.67 1140.19 L1017.9 1140.1 L1018.12 1139.92 L1018.35 1138.7 L1018.58 1138.81 L1018.81 1139.11 L1019.03 1139.26 L1019.26 1139.69 L1019.49 1138.64 L1019.72 1139 L1019.95 1138.2 L1020.17 1136.2 L1020.4 1135.63 L1020.63 1136.03 L1020.86 1135.53 L1021.08 1136.2 L1021.31 1136 L1021.54 1135.38 L1021.77 1134.09 L1022 1133.67 L1022.22 1133.29 L1022.45 1133.88 L1022.68 1134.92 L1022.91 1134.2 L1023.13 1132.61 L1023.36 1132.31 L1023.59 1134.47 L1023.82 1135.44 L1024.05 1134.14 L1024.27 1134.11 L1024.5 1134.52 L1024.73 1134.6 L1024.96 1135.11 L1025.18 1136.03 L1025.41 1137.47 L1025.64 1137.71 L1025.87 1137.28 L1026.1 1137.45 L1026.32 1137.74 L1026.55 1137.08 L1026.78 1137.16 L1027.01 1135.69 L1027.23 1135.45 L1027.46 1135.91 L1027.69 1135.02 L1027.92 1134.55 L1028.15 1133.72 L1028.37 1132.64 L1028.6 1133.35 L1028.83 1132.43 L1029.06 1133.21 L1029.28 1133.34 L1029.51 1132.61 L1029.74 1133.66 L1029.97 1133.29 L1030.2 1133.46 L1030.42 1133.16 L1030.65 1133.78 L1030.88 1132.31 L1031.11 1132.57 L1031.33 1132.96 L1031.56 1132.46 L1031.79 1131.29 L1032.02 1131.48 L1032.25 1131.41 L1032.47 1130.96 L1032.7 1129.77 L1032.93 1130.12 L1033.16 1129.59 L1033.38 1131.31 L1033.61 1131.13 L1033.84 1131.25 L1034.07 1130.29 L1034.3 1129.73 L1034.52 1130.41 L1034.75 1130.28 L1034.98 1130.2 L1035.21 1129.93 L1035.43 1128.97 L1035.66 1129.39 L1035.89 1128.5 L1036.12 1127.11 L1036.35 1127.15 L1036.57 1127.66 L1036.8 1127.66 L1037.03 1128.7 L1037.26 1129.37 L1037.48 1127.91 L1037.71 1127.32 L1037.94 1126.69 L1038.17 1127.24 L1038.4 1127.51 L1038.62 1127.71 L1038.85 1128.21 L1039.08 1127.91 L1039.31 1126.89 L1039.53 1127.58 L1039.76 1127.14 L1039.99 1127.26 L1040.22 1125.9 L1040.44 1126.18 L1040.67 1125.91 L1040.9 1125.73 L1041.13 1126.03 L1041.36 1126.75 L1041.58 1127.49 L1041.81 1127.66 L1042.04 1128.26 L1042.27 1128.36 L1042.49 1128.58 L1042.72 1128.24 L1042.95 1128.57 L1043.18 1128.07 L1043.41 1128.69 L1043.63 1129.74 L1043.86 1128.41 L1044.09 1128.62 L1044.32 1129.34 L1044.54 1128.9 L1044.77 1129.8 L1045 1128.66 L1045.23 1127.52 L1045.46 1127.74 L1045.68 1128.73 L1045.91 1128.78 L1046.14 1128.17 L1046.37 1128.15 L1046.59 1127.88 L1046.82 1129.22 L1047.05 1129.51 L1047.28 1128.18 L1047.51 1126.69 L1047.73 1126.13 L1047.96 1126.56 L1048.19 1126.7 L1048.42 1127.09 L1048.64 1126.36 L1048.87 1126.17 L1049.1 1126.92 L1049.33 1127.36 L1049.56 1127.03 L1049.78 1127.48 L1050.01 1128.43 L1050.24 1128.42 L1050.47 1129.41 L1050.69 1130.18 L1050.92 1131 L1051.15 1131.1 L1051.38 1131.64 L1051.61 1131.28 L1051.83 1130.6 L1052.06 1130.69 L1052.29 1131 L1052.52 1130.18 L1052.74 1129.9 L1052.97 1131.08 L1053.2 1130.41 L1053.43 1131.34 L1053.66 1130.61 L1053.88 1130.87 L1054.11 1131.09 L1054.34 1130.99 L1054.57 1129.68 L1054.79 1129.12 L1055.02 1129.83 L1055.25 1130.43 L1055.48 1129.71 L1055.71 1130.12 L1055.93 1131.21 L1056.16 1130.69 L1056.39 1131.77 L1056.62 1132 L1056.84 1131.8 L1057.07 1131.47 L1057.3 1131.64 L1057.53 1132.67 L1057.76 1133.04 L1057.98 1133.88 L1058.21 1134.93 L1058.44 1134.67 L1058.67 1133.75 L1058.89 1133.23 L1059.12 1132.63 L1059.35 1133.16 L1059.58 1132.34 L1059.81 1132.81 L1060.03 1132.72 L1060.26 1131.85 L1060.49 1131.54 L1060.72 1130.75 L1060.94 1131.05 L1061.17 1130.4 L1061.4 1131.39 L1061.63 1131.88 L1061.86 1132.42 L1062.08 1132.99 L1062.31 1134.1 L1062.54 1134.3 L1062.77 1135.01 L1062.99 1135.88 L1063.22 1135.19 L1063.45 1136.87 L1063.68 1136.68 L1063.91 1137.96 L1064.13 1138.42 L1064.36 1138.48 L1064.59 1138.6 L1064.82 1138.65 L1065.04 1138.21 L1065.27 1137.27 L1065.5 1137.61 L1065.73 1139.28 L1065.96 1138.3 L1066.18 1136.42 L1066.41 1136.14 L1066.64 1136.03 L1066.87 1136.41 L1067.09 1136.04 L1067.32 1135.83 L1067.55 1136.61 L1067.78 1136.54 L1068.01 1136.95 L1068.23 1137.96 L1068.46 1138.22 L1068.69 1137.72 L1068.92 1138.27 L1069.14 1137 L1069.37 1137.16 L1069.6 1137.22 L1069.83 1136.18 L1070.06 1135.37 L1070.28 1135.89 L1070.51 1133.88 L1070.74 1133 L1070.97 1133.41 L1071.19 1133.98 L1071.42 1133.11 L1071.65 1133.46 L1071.88 1133.98 L1072.11 1135.36 L1072.33 1133.91 L1072.56 1134.82 L1072.79 1134.93 L1073.02 1135.85 L1073.24 1134.15 L1073.47 1134.19 L1073.7 1134.48 L1073.93 1135.74 L1074.16 1134.52 L1074.38 1134.82 L1074.61 1134.89 L1074.84 1133.18 L1075.07 1133.35 L1075.29 1133.84 L1075.52 1133.01 L1075.75 1132.86 L1075.98 1132.19 L1076.2 1131.57 L1076.43 1131.4 L1076.66 1131.25 L1076.89 1131.95 L1077.12 1132.1 L1077.34 1131.88 L1077.57 1131.22 L1077.8 1131.15 L1078.03 1130.39 L1078.25 1129.4 L1078.48 1130.45 L1078.71 1131.32 L1078.94 1128.65 L1079.17 1127.23 L1079.39 1126.59 L1079.62 1127.73 L1079.85 1128.1 L1080.08 1127.36 L1080.3 1126.72 L1080.53 1125.73 L1080.76 1125.58 L1080.99 1124.8 L1081.22 1125.11 L1081.44 1126.04 L1081.67 1126.7 L1081.9 1125.97 L1082.13 1126.51 L1082.35 1126.92 L1082.58 1127.89 L1082.81 1128.01 L1083.04 1129.25 L1083.27 1128.89 L1083.49 1126.19 L1083.72 1126.67 L1083.95 1127.42 L1084.18 1127.3 L1084.4 1126.61 L1084.63 1125.66 L1084.86 1125.77 L1085.09 1126.27 L1085.32 1126.67 L1085.54 1127.16 L1085.77 1127 L1086 1128.29 L1086.23 1128.75 L1086.45 1127.94 L1086.68 1127.51 L1086.91 1127.77 L1087.14 1127.27 L1087.37 1127.63 L1087.59 1127.62 L1087.82 1126.9 L1088.05 1127.73 L1088.28 1127.53 L1088.5 1128.49 L1088.73 1129.09 L1088.96 1127.83 L1089.19 1127.51 L1089.42 1127.56 L1089.64 1128.04 L1089.87 1128.79 L1090.1 1128.3 L1090.33 1127.73 L1090.55 1127.17 L1090.78 1126.47 L1091.01 1127.05 L1091.24 1127.62 L1091.47 1127.84 L1091.69 1128.21 L1091.92 1127.04 L1092.15 1126.46 L1092.38 1126.1 L1092.6 1126.37 L1092.83 1124.43 L1093.06 1124.02 L1093.29 1125.63 L1093.52 1125.23 L1093.74 1124.65 L1093.97 1124.09 L1094.2 1122.56 L1094.43 1122.55 L1094.65 1121.23 L1094.88 1122.09 L1095.11 1122.56 L1095.34 1122.14 L1095.57 1121.87 L1095.79 1122.28 L1096.02 1122.67 L1096.25 1122.24 L1096.48 1121.55 L1096.7 1121.47 L1096.93 1122.42 L1097.16 1121.19 L1097.39 1120.4 L1097.62 1119.91 L1097.84 1120.76 L1098.07 1121.14 L1098.3 1121.02 L1098.53 1121.6 L1098.75 1120.71 L1098.98 1121.11 L1099.21 1121.08 L1099.44 1120.92 L1099.67 1120.62 L1099.89 1119.88 L1100.12 1120.72 L1100.35 1120.95 L1100.58 1121.38 L1100.8 1121.64 L1101.03 1121.68 L1101.26 1119.66 L1101.49 1120.51 L1101.72 1120.68 L1101.94 1121.64 L1102.17 1120.83 L1102.4 1120.48 L1102.63 1121.05 L1102.85 1120.63 L1103.08 1120.06 L1103.31 1119.17 L1103.54 1119.68 L1103.77 1120.72 L1103.99 1121.1 L1104.22 1120.49 L1104.45 1120.43 L1104.68 1120.2 L1104.9 1119.63 L1105.13 1120.04 L1105.36 1120.74 L1105.59 1121.37 L1105.82 1120.82 L1106.04 1119.57 L1106.27 1120.77 L1106.5 1121.63 L1106.73 1121.2 L1106.95 1121.48 L1107.18 1121.72 L1107.41 1122.61 L1107.64 1122.73 L1107.87 1122.71 L1108.09 1122.56 L1108.32 1123.63 L1108.55 1124.41 L1108.78 1125.51 L1109 1125.57 L1109.23 1125.43 L1109.46 1124.13 L1109.69 1123.65 L1109.92 1124.6 L1110.14 1125.88 L1110.37 1126.38 L1110.6 1126.6 L1110.83 1125.92 L1111.05 1127.63 L1111.28 1127.75 L1111.51 1127.78 L1111.74 1127.29 L1111.97 1128.59 L1112.19 1129.38 L1112.42 1129.22 L1112.65 1130.19 L1112.88 1130.49 L1113.1 1129.68 L1113.33 1129.24 L1113.56 1129.62 L1113.79 1130.57 L1114.01 1130.98 L1114.24 1131.06 L1114.47 1130.13 L1114.7 1130.12 L1114.93 1129.16 L1115.15 1128.85 L1115.38 1128.34 L1115.61 1128.84 L1115.84 1129.93 L1116.06 1129.24 L1116.29 1128.99 L1116.52 1129.62 L1116.75 1130.39 L1116.98 1130.02 L1117.2 1130.24 L1117.43 1130.58 L1117.66 1130.76 L1117.89 1130.21 L1118.11 1130 L1118.34 1129.16 L1118.57 1128.81 L1118.8 1128.66 L1119.03 1128.6 L1119.25 1127.62 L1119.48 1128.18 L1119.71 1127.4 L1119.94 1126.94 L1120.16 1127.76 L1120.39 1127.52 L1120.62 1127.08 L1120.85 1126.8 L1121.08 1125.77 L1121.3 1125.63 L1121.53 1126.01 L1121.76 1126.23 L1121.99 1125.56 L1122.21 1125.99 L1122.44 1126.17 L1122.67 1125.7 L1122.9 1126.02 L1123.13 1126.53 L1123.35 1126.48 L1123.58 1125.74 L1123.81 1124.93 L1124.04 1125.3 L1124.26 1124.81 L1124.49 1124.33 L1124.72 1124.09 L1124.95 1125.21 L1125.18 1124.77 L1125.4 1123.97 L1125.63 1123.76 L1125.86 1123.57 L1126.09 1124.15 L1126.31 1123.77 L1126.54 1122.87 L1126.77 1122.96 L1127 1123.57 L1127.23 1123.91 L1127.45 1123.02 L1127.68 1122.99 L1127.91 1122.51 L1128.14 1122.17 L1128.36 1122.78 L1128.59 1123.3 L1128.82 1122.97 L1129.05 1123.06 L1129.28 1122.3 L1129.5 1123.36 L1129.73 1124.02 L1129.96 1123.9 L1130.19 1125.54 L1130.41 1125.72 L1130.64 1125.84 L1130.87 1127.01 L1131.1 1126.52 L1131.33 1126.3 L1131.55 1126.14 L1131.78 1126.19 L1132.01 1127.13 L1132.24 1128.03 L1132.46 1129.27 L1132.69 1129.2 L1132.92 1130.01 L1133.15 1128.87 L1133.38 1128.33 L1133.6 1129.26 L1133.83 1128.29 L1134.06 1128.58 L1134.29 1128.33 L1134.51 1128 L1134.74 1127.38 L1134.97 1127.41 L1135.2 1127.26 L1135.43 1127.73 L1135.65 1128.06 L1135.88 1127.95 L1136.11 1126.52 L1136.34 1126.5 L1136.56 1126.57 L1136.79 1128.04 L1137.02 1126.67 L1137.25 1126.25 L1137.48 1126.37 L1137.7 1126.16 L1137.93 1124.66 L1138.16 1124.97 L1138.39 1125.46 L1138.61 1125.67 L1138.84 1126.44 L1139.07 1126.1 L1139.3 1127.17 L1139.53 1127.33 L1139.75 1126.31 L1139.98 1126.02 L1140.21 1125.92 L1140.44 1126.1 L1140.66 1125.71 L1140.89 1124.08 L1141.12 1124.21 L1141.35 1123.92 L1141.58 1125.03 L1141.8 1125.65 L1142.03 1124.3 L1142.26 1123.96 L1142.49 1123.9 L1142.71 1124.04 L1142.94 1122.1 L1143.17 1121.06 L1143.4 1120.82 L1143.63 1120.06 L1143.85 1119.2 L1144.08 1119.09 L1144.31 1118.77 L1144.54 1116.94 L1144.76 1117.73 L1144.99 1117.58 L1145.22 1117.53 L1145.45 1117.86 L1145.68 1118.24 L1145.9 1118.62 L1146.13 1118.56 L1146.36 1118.28 L1146.59 1117.79 L1146.81 1116.26 L1147.04 1116.18 L1147.27 1115.5 L1147.5 1116.92 L1147.73 1116.38 L1147.95 1116.2 L1148.18 1115.41 L1148.41 1115.96 L1148.64 1116.53 L1148.86 1116.29 L1149.09 1115.39 L1149.32 1116.77 L1149.55 1117.06 L1149.77 1116.57 L1150 1116.33 L1150.23 1116.92 L1150.46 1116.53 L1150.69 1116.51 L1150.91 1116.12 L1151.14 1116.53 L1151.37 1115.7 L1151.6 1115.04 L1151.82 1114.48 L1152.05 1112.6 L1152.28 1112.26 L1152.51 1112.21 L1152.74 1113.02 L1152.96 1112.68 L1153.19 1113.27 L1153.42 1112.99 L1153.65 1113.32 L1153.87 1112.57 L1154.1 1113.61 L1154.33 1114.69 L1154.56 1115.65 L1154.79 1116.24 L1155.01 1115.82 L1155.24 1116 L1155.47 1116.22 L1155.7 1117.21 L1155.92 1117.11 L1156.15 1117.15 L1156.38 1117.42 L1156.61 1117.63 L1156.84 1116.59 L1157.06 1116.74 L1157.29 1116.27 L1157.52 1116.05 L1157.75 1115.17 L1157.97 1115.24 L1158.2 1115.44 L1158.43 1115.39 L1158.66 1113.74 L1158.89 1114.03 L1159.11 1114.17 L1159.34 1114.24 L1159.57 1115.12 L1159.8 1115.6 L1160.02 1115.64 L1160.25 1114.56 L1160.48 1113.78 L1160.71 1113.97 L1160.94 1114.26 L1161.16 1114.56 L1161.39 1114.36 L1161.62 1113.46 L1161.85 1113.36 L1162.07 1111.78 L1162.3 1111.79 L1162.53 1112.23 L1162.76 1111.99 L1162.99 1112.5 L1163.21 1112.61 L1163.44 1112.03 L1163.67 1112.24 L1163.9 1111.76 L1164.12 1112.59 L1164.35 1112.18 L1164.58 1112.04 L1164.81 1112.45 L1165.04 1110.62 L1165.26 1110.73 L1165.49 1111.61 L1165.72 1111.46 L1165.95 1111.18 L1166.17 1112.01 L1166.4 1112.84 L1166.63 1112.93 L1166.86 1112.75 L1167.09 1112.2 L1167.31 1112.16 L1167.54 1112.92 L1167.77 1113.04 L1168 1113.72 L1168.22 1114.44 L1168.45 1115.02 L1168.68 1114.91 L1168.91 1115.18 L1169.14 1115.59 L1169.36 1115.25 L1169.59 1115.26 L1169.82 1115.59 L1170.05 1114.9 L1170.27 1114.45 L1170.5 1114.79 L1170.73 1114.27 L1170.96 1114.75 L1171.19 1114.18 L1171.41 1113.8 L1171.64 1113.74 L1171.87 1113.54 L1172.1 1112.57 L1172.32 1111.85 L1172.55 1111.04 L1172.78 1111.28 L1173.01 1111.07 L1173.24 1109.9 L1173.46 1111.07 L1173.69 1111.09 L1173.92 1110.09 L1174.15 1109.51 L1174.37 1110.53 L1174.6 1109.4 L1174.83 1109.02 L1175.06 1108.84 L1175.29 1109.32 L1175.51 1107.65 L1175.74 1108.46 L1175.97 1108.48 L1176.2 1108.36 L1176.42 1107.67 L1176.65 1108.11 L1176.88 1108.39 L1177.11 1108.65 L1177.34 1109.11 L1177.56 1109.22 L1177.79 1109.75 L1178.02 1110.97 L1178.25 1110.92 L1178.47 1111.24 L1178.7 1111.28 L1178.93 1110.58 L1179.16 1110.59 L1179.39 1109.74 L1179.61 1109.46 L1179.84 1110.22 L1180.07 1110.19 L1180.3 1110.2 L1180.52 1109.85 L1180.75 1109.92 L1180.98 1109.55 L1181.21 1109.03 L1181.44 1108.54 L1181.66 1108.17 L1181.89 1107.93 L1182.12 1107.89 L1182.35 1107.73 L1182.57 1109.15 L1182.8 1109.24 L1183.03 1108.42 L1183.26 1108.18 L1183.49 1108.49 L1183.71 1109.46 L1183.94 1107.74 L1184.17 1107.35 L1184.4 1107.82 L1184.62 1107.09 L1184.85 1108.03 L1185.08 1108.45 L1185.31 1108.56 L1185.53 1108.48 L1185.76 1108.09 L1185.99 1108.8 L1186.22 1110.07 L1186.45 1109.52 L1186.67 1108.99 L1186.9 1109.17 L1187.13 1110.42 L1187.36 1110.61 L1187.58 1110.71 L1187.81 1111.98 L1188.04 1111.58 L1188.27 1111.59 L1188.5 1111.42 L1188.72 1112.11 L1188.95 1111.71 L1189.18 1112 L1189.41 1110.97 L1189.63 1110.59 L1189.86 1110.38 L1190.09 1109.99 L1190.32 1109.2 L1190.55 1110.63 L1190.77 1111.31 L1191 1111.99 L1191.23 1110.97 L1191.46 1110.16 L1191.68 1110.26 L1191.91 1110.7 L1192.14 1110.61 L1192.37 1112.31 L1192.6 1112.93 L1192.82 1113.45 L1193.05 1112.37 L1193.28 1111.07 L1193.51 1111.15 L1193.73 1111.75 L1193.96 1111.04 L1194.19 1111.12 L1194.42 1111.65 L1194.65 1110.1 L1194.87 1111.29 L1195.1 1111.28 L1195.33 1110.88 L1195.56 1111.47 L1195.78 1112.56 L1196.01 1113.06 L1196.24 1113.6 L1196.47 1113.22 L1196.7 1113.6 L1196.92 1113.08 L1197.15 1112.15 L1197.38 1111.94 L1197.61 1112.45 L1197.83 1111.81 L1198.06 1111.51 L1198.29 1112.12 L1198.52 1112.21 L1198.75 1111.32 L1198.97 1111.86 L1199.2 1112.68 L1199.43 1113.73 L1199.66 1113.84 L1199.88 1113.55 L1200.11 1113.83 L1200.34 1114.18 L1200.57 1114.5 L1200.8 1114.14 L1201.02 1113.88 L1201.25 1114.28 L1201.48 1113.81 L1201.71 1114.5 L1201.93 1114.63 L1202.16 1114.29 L1202.39 1113.91 L1202.62 1114.01 L1202.85 1114.07 L1203.07 1114.8 L1203.3 1114.73 L1203.53 1115.33 L1203.76 1115.43 L1203.98 1116.19 L1204.21 1117.41 L1204.44 1118.25 L1204.67 1117.74 L1204.9 1117.07 L1205.12 1116.81 L1205.35 1116.32 L1205.58 1116.82 L1205.81 1117.21 L1206.03 1117.7 L1206.26 1117.63 L1206.49 1117.05 L1206.72 1116.76 L1206.95 1116.77 L1207.17 1117.24 L1207.4 1117.86 L1207.63 1117.52 L1207.86 1117.67 L1208.08 1115.97 L1208.31 1116.49 L1208.54 1115.58 L1208.77 1115.36 L1209 1114.46 L1209.22 1113.99 L1209.45 1113.41 L1209.68 1114.42 L1209.91 1114.49 L1210.13 1114.91 L1210.36 1113.71 L1210.59 1114.03 L1210.82 1113.93 L1211.05 1114.23 L1211.27 1113.53 L1211.5 1114.41 L1211.73 1115.05 L1211.96 1115.55 L1212.18 1116.11 L1212.41 1116.42 L1212.64 1117.67 L1212.87 1117.5 L1213.1 1117.57 L1213.32 1117.3 L1213.55 1117.46 L1213.78 1117.56 L1214.01 1118.06 L1214.23 1117.84 L1214.46 1118.01 L1214.69 1119.01 L1214.92 1119.43 L1215.15 1119.63 L1215.37 1119.39 L1215.6 1117.95 L1215.83 1118.54 L1216.06 1119.43 L1216.28 1119.51 L1216.51 1119.61 L1216.74 1119.9 L1216.97 1120.66 L1217.2 1120.65 L1217.42 1122.03 L1217.65 1121.9 L1217.88 1121.77 L1218.11 1120.85 L1218.33 1120.17 L1218.56 1121.54 L1218.79 1121.08 L1219.02 1121.08 L1219.25 1120.54 L1219.47 1121.01 L1219.7 1121.48 L1219.93 1121.74 L1220.16 1122.22 L1220.38 1121.48 L1220.61 1121.2 L1220.84 1120.38 L1221.07 1120.69 L1221.29 1121.21 L1221.52 1122.39 L1221.75 1123.08 L1221.98 1121.77 L1222.21 1120.36 L1222.43 1120.55 L1222.66 1120.78 L1222.89 1121.18 L1223.12 1120.95 L1223.34 1121.12 L1223.57 1120.58 L1223.8 1119.85 L1224.03 1118.98 L1224.26 1118.81 L1224.48 1118.94 L1224.71 1117.41 L1224.94 1117.92 L1225.17 1118.09 L1225.39 1118.78 L1225.62 1118.98 L1225.85 1117.65 L1226.08 1116.31 L1226.31 1115.01 L1226.53 1115.38 L1226.76 1114.71 L1226.99 1114.6 L1227.22 1114.8 L1227.44 1114.9 L1227.67 1114.96 L1227.9 1115 L1228.13 1115.75 L1228.36 1114.85 L1228.58 1115.13 L1228.81 1114.72 L1229.04 1115 L1229.27 1114.67 L1229.49 1115.13 L1229.72 1115.29 L1229.95 1114.41 L1230.18 1115.13 L1230.41 1115.23 L1230.63 1115.92 L1230.86 1116.38 L1231.09 1116.22 L1231.32 1116.1 L1231.54 1116.31 L1231.77 1115.41 L1232 1114.15 L1232.23 1115.18 L1232.46 1114.49 L1232.68 1114.58 L1232.91 1114.4 L1233.14 1115 L1233.37 1115.52 L1233.59 1115.42 L1233.82 1115.01 L1234.05 1115.11 L1234.28 1115.66 L1234.51 1115.49 L1234.73 1114.25 L1234.96 1113.91 L1235.19 1113.7 L1235.42 1113.91 L1235.64 1114.16 L1235.87 1113.61 L1236.1 1114.01 L1236.33 1114.53 L1236.56 1113.73 L1236.78 1114.93 L1237.01 1114.67 L1237.24 1115.12 L1237.47 1114.61 L1237.69 1114.69 L1237.92 1113.6 L1238.15 1113.85 L1238.38 1114.07 L1238.61 1113.44 L1238.83 1113.82 L1239.06 1112.91 L1239.29 1112.88 L1239.52 1112.79 L1239.74 1113.53 L1239.97 1113.26 L1240.2 1113.96 L1240.43 1113.32 L1240.66 1113.31 L1240.88 1113.3 L1241.11 1112.74 L1241.34 1113.3 L1241.57 1112.24 L1241.79 1112.34 L1242.02 1112.45 L1242.25 1112.02 L1242.48 1112.52 L1242.71 1113.03 L1242.93 1110.8 L1243.16 1111.19 L1243.39 1111.02 L1243.62 1110.95 L1243.84 1110.78 L1244.07 1110.67 L1244.3 1110.2 L1244.53 1109.92 L1244.76 1110.08 L1244.98 1111.54 L1245.21 1111.78 L1245.44 1110.64 L1245.67 1111.69 L1245.89 1111.51 L1246.12 1111.7 L1246.35 1111.68 L1246.58 1111.18 L1246.81 1111.99 L1247.03 1112.11 L1247.26 1112.49 L1247.49 1112.49 L1247.72 1113.24 L1247.94 1114.1 L1248.17 1114.82 L1248.4 1114.75 L1248.63 1115.34 L1248.86 1115.08 L1249.08 1114.84 L1249.31 1115.51 L1249.54 1115.45 L1249.77 1115.65 L1249.99 1115.45 L1250.22 1115.02 L1250.45 1114.77 L1250.68 1114.66 L1250.91 1114.43 L1251.13 1114.55 L1251.36 1114.23 L1251.59 1115 L1251.82 1114.11 L1252.04 1112.57 L1252.27 1112.57 L1252.5 1112.71 L1252.73 1112.79 L1252.96 1113.44 L1253.18 1113.39 L1253.41 1113.18 L1253.64 1112.87 L1253.87 1112.48 L1254.09 1112.68 L1254.32 1113.95 L1254.55 1113.7 L1254.78 1114.6 L1255.01 1113.4 L1255.23 1112.29 L1255.46 1112.29 L1255.69 1111.97 L1255.92 1112.38 L1256.14 1113.2 L1256.37 1113.65 L1256.6 1112.68 L1256.83 1112.03 L1257.06 1112.38 L1257.28 1112.51 L1257.51 1111.91 L1257.74 1111.55 L1257.97 1110.6 L1258.19 1110.89 L1258.42 1110.49 L1258.65 1109.88 L1258.88 1109.95 L1259.1 1110.22 L1259.33 1110.66 L1259.56 1109.83 L1259.79 1109.19 L1260.02 1110.04 L1260.24 1110.12 L1260.47 1109.63 L1260.7 1110.34 L1260.93 1110.41 L1261.15 1110.74 L1261.38 1110.34 L1261.61 1110.44 L1261.84 1111.18 L1262.07 1110.7 L1262.29 1109.64 L1262.52 1108.54 L1262.75 1108.63 L1262.98 1107.76 L1263.2 1106.82 L1263.43 1108.04 L1263.66 1108.06 L1263.89 1108.01 L1264.12 1107.92 L1264.34 1107.11 L1264.57 1107.47 L1264.8 1108.54 L1265.03 1108.19 L1265.25 1108.68 L1265.48 1107.97 L1265.71 1107.27 L1265.94 1108.07 L1266.17 1107.73 L1266.39 1107.6 L1266.62 1107.81 L1266.85 1107.8 L1267.08 1106.91 L1267.3 1107.25 L1267.53 1107.03 L1267.76 1107.25 L1267.99 1107.09 L1268.22 1106.55 L1268.44 1105.61 L1268.67 1105.49 L1268.9 1104.86 L1269.13 1105.18 L1269.35 1104.95 L1269.58 1105.63 L1269.81 1104.26 L1270.04 1103.3 L1270.27 1103 L1270.49 1103.12 L1270.72 1103.12 L1270.95 1103.08 L1271.18 1101.95 L1271.4 1102.53 L1271.63 1102.75 L1271.86 1102.86 L1272.09 1102.46 L1272.32 1102.78 L1272.54 1102.88 L1272.77 1102.47 L1273 1102.81 L1273.23 1103.21 L1273.45 1102.42 L1273.68 1101.07 L1273.91 1101.64 L1274.14 1102.28 L1274.37 1101.55 L1274.59 1102.64 L1274.82 1102.6 L1275.05 1102.86 L1275.28 1102.75 L1275.5 1103.09 L1275.73 1102.92 L1275.96 1103.16 L1276.19 1102.81 L1276.42 1102.38 L1276.64 1102.08 L1276.87 1101.91 L1277.1 1101.18 L1277.33 1101.13 L1277.55 1101 L1277.78 1102.09 L1278.01 1102.64 L1278.24 1102.16 L1278.47 1102.58 L1278.69 1102.76 L1278.92 1102.41 L1279.15 1101.9 L1279.38 1101.75 L1279.6 1103.2 L1279.83 1103.01 L1280.06 1103 L1280.29 1102.91 L1280.52 1102.99 L1280.74 1102.43 L1280.97 1102.22 L1281.2 1102.08 L1281.43 1102.75 L1281.65 1103.1 L1281.88 1103.21 L1282.11 1102.67 L1282.34 1102.43 L1282.57 1103.09 L1282.79 1102.19 L1283.02 1101.23 L1283.25 1101.1 L1283.48 1101.4 L1283.7 1101.9 L1283.93 1100.84 L1284.16 1101.67 L1284.39 1101.39 L1284.62 1101.66 L1284.84 1101.84 L1285.07 1102.31 L1285.3 1101.99 L1285.53 1101.39 L1285.75 1101.41 L1285.98 1101.09 L1286.21 1100.7 L1286.44 1101.25 L1286.67 1102.01 L1286.89 1102.34 L1287.12 1102.56 L1287.35 1102.7 L1287.58 1102.87 L1287.8 1102.78 L1288.03 1102.34 L1288.26 1102.56 L1288.49 1102.31 L1288.72 1101.98 L1288.94 1101.82 L1289.17 1101.5 L1289.4 1101.32 L1289.63 1100.51 L1289.85 1100.65 L1290.08 1101.08 L1290.31 1101.82 L1290.54 1103.62 L1290.77 1103.03 L1290.99 1103.79 L1291.22 1103.38 L1291.45 1103.12 L1291.68 1103.55 L1291.9 1104.24 L1292.13 1104.46 L1292.36 1104.78 L1292.59 1104.46 L1292.82 1104.6 L1293.04 1103.33 L1293.27 1103.02 L1293.5 1101.73 L1293.73 1101.89 L1293.95 1100.99 L1294.18 1101.8 L1294.41 1102.66 L1294.64 1102.6 L1294.86 1102.27 L1295.09 1102.6 L1295.32 1102.06 L1295.55 1102.9 L1295.78 1101.86 L1296 1102.02 L1296.23 1102.38 L1296.46 1102.45 L1296.69 1102.42 L1296.91 1102.31 L1297.14 1102.38 L1297.37 1102.09 L1297.6 1101.53 L1297.83 1099.81 L1298.05 1100.1 L1298.28 1101.15 L1298.51 1101.73 L1298.74 1102.02 L1298.96 1102.29 L1299.19 1103.41 L1299.42 1103.07 L1299.65 1101.79 L1299.88 1101.67 L1300.1 1100.89 L1300.33 1100.53 L1300.56 1100.39 L1300.79 1100.23 L1301.01 1100.26 L1301.24 1100.82 L1301.47 1100.33 L1301.7 1099.97 L1301.93 1099.37 L1302.15 1099.46 L1302.38 1099.87 L1302.61 1099.41 L1302.84 1099.17 L1303.06 1099.08 L1303.29 1098.91 L1303.52 1098.9 L1303.75 1099.6 L1303.98 1099.51 L1304.2 1099.54 L1304.43 1099.66 L1304.66 1099.88 L1304.89 1099.39 L1305.11 1099.03 L1305.34 1099.21 L1305.57 1100.08 L1305.8 1100.47 L1306.03 1100.78 L1306.25 1101.53 L1306.48 1101.71 L1306.71 1101.4 L1306.94 1101.82 L1307.16 1101.74 L1307.39 1102.3 L1307.62 1102.13 L1307.85 1102.52 L1308.08 1103.62 L1308.3 1103.53 L1308.53 1103.01 L1308.76 1103.45 L1308.99 1103.51 L1309.21 1103.3 L1309.44 1103.47 L1309.67 1104.43 L1309.9 1105.31 L1310.13 1105.94 L1310.35 1106.05 L1310.58 1105.78 L1310.81 1104.89 L1311.04 1104.45 L1311.26 1103.69 L1311.49 1104.64 L1311.72 1104.69 L1311.95 1104.11 L1312.18 1102.9 L1312.4 1103.04 L1312.63 1102.94 L1312.86 1103.76 L1313.09 1103.24 L1313.31 1104.12 L1313.54 1103.66 L1313.77 1103.44 L1314 1103.62 L1314.23 1104.05 L1314.45 1103.81 L1314.68 1104.23 L1314.91 1105.16 L1315.14 1104.9 L1315.36 1105.09 L1315.59 1104.83 L1315.82 1105.32 L1316.05 1105.5 L1316.28 1106 L1316.5 1104.85 L1316.73 1104.5 L1316.96 1104.8 L1317.19 1104.46 L1317.41 1105.26 L1317.64 1105.26 L1317.87 1105.5 L1318.1 1105.4 L1318.33 1104.47 L1318.55 1104.5 L1318.78 1105.25 L1319.01 1106.01 L1319.24 1106.61 L1319.46 1106.99 L1319.69 1105.94 L1319.92 1105.72 L1320.15 1106.52 L1320.38 1106.43 L1320.6 1106.43 L1320.83 1106.71 L1321.06 1106.26 L1321.29 1106.93 L1321.51 1107.27 L1321.74 1107.55 L1321.97 1107.03 L1322.2 1107.05 L1322.43 1107.4 L1322.65 1107.39 L1322.88 1107.1 L1323.11 1106.91 L1323.34 1107.97 L1323.56 1108.66 L1323.79 1109.18 L1324.02 1109.79 L1324.25 1110.03 L1324.48 1110.72 L1324.7 1111.16 L1324.93 1111.7 L1325.16 1111.91 L1325.39 1112.19 L1325.61 1111.33 L1325.84 1111.94 L1326.07 1111.84 L1326.3 1111.7 L1326.53 1111.88 L1326.75 1112.39 L1326.98 1111.56 L1327.21 1111.52 L1327.44 1111.71 L1327.66 1111.68 L1327.89 1112.19 L1328.12 1112.9 L1328.35 1113.43 L1328.58 1114.06 L1328.8 1114.23 L1329.03 1114.19 L1329.26 1113.77 L1329.49 1112.75 L1329.71 1112.36 L1329.94 1111.65 L1330.17 1111.67 L1330.4 1111.56 L1330.62 1111.37 L1330.85 1111.45 L1331.08 1111.06 L1331.31 1111.53 L1331.54 1111.1 L1331.76 1109.98 L1331.99 1108.96 L1332.22 1109.83 L1332.45 1109.73 L1332.67 1109.6 L1332.9 1108.5 L1333.13 1108.27 L1333.36 1108.13 L1333.59 1106.48 L1333.81 1107.32 L1334.04 1107.58 L1334.27 1108.25 L1334.5 1108.93 L1334.72 1109.97 L1334.95 1110.58 L1335.18 1110.84 L1335.41 1111.52 L1335.64 1111.12 L1335.86 1110.89 L1336.09 1111.08 L1336.32 1110.25 L1336.55 1109.7 L1336.77 1109.34 L1337 1109.18 L1337.23 1109.28 L1337.46 1110.22 L1337.69 1110.67 L1337.91 1110.52 L1338.14 1110.62 L1338.37 1111.48 L1338.6 1111.05 L1338.82 1111.09 L1339.05 1111.26 L1339.28 1111.85 L1339.51 1111.96 L1339.74 1112 L1339.96 1111.99 L1340.19 1112.18 L1340.42 1110.84 L1340.65 1111.33 L1340.87 1112.35 L1341.1 1113.3 L1341.33 1113.66 L1341.56 1114.35 L1341.79 1114.24 L1342.01 1114.48 L1342.24 1113.83 L1342.47 1114.25 L1342.7 1114.8 L1342.92 1115.33 L1343.15 1115.71 L1343.38 1114.4 L1343.61 1113.97 L1343.84 1113.69 L1344.06 1113.93 L1344.29 1113.44 L1344.52 1112.81 L1344.75 1113.42 L1344.97 1112.31 L1345.2 1112.47 L1345.43 1112.32 L1345.66 1111.17 L1345.89 1109.59 L1346.11 1109.64 L1346.34 1107.78 L1346.57 1107.3 L1346.8 1107.51 L1347.02 1107.21 L1347.25 1107.34 L1347.48 1106.67 L1347.71 1106.85 L1347.94 1106.58 L1348.16 1106.72 L1348.39 1106.7 L1348.62 1106.19 L1348.85 1106.15 L1349.07 1105.91 L1349.3 1105.95 L1349.53 1104.16 L1349.76 1104.89 L1349.99 1105.67 L1350.21 1105.53 L1350.44 1105.05 L1350.67 1104.97 L1350.9 1105.77 L1351.12 1105.93 L1351.35 1105.95 L1351.58 1105.81 L1351.81 1106.8 L1352.04 1106.46 L1352.26 1107.11 L1352.49 1107.8 L1352.72 1108.12 L1352.95 1107.4 L1353.17 1108.39 L1353.4 1108.27 L1353.63 1109.26 L1353.86 1108.26 L1354.09 1108.83 L1354.31 1108.47 L1354.54 1109.33 L1354.77 1107.93 L1355 1108.04 L1355.22 1108.24 L1355.45 1107.93 L1355.68 1108.18 L1355.91 1107.57 L1356.14 1106.07 L1356.36 1106.67 L1356.59 1106.5 L1356.82 1107.41 L1357.05 1106.91 L1357.27 1107.58 L1357.5 1108.47 L1357.73 1109.02 L1357.96 1108.57 L1358.19 1108.57 L1358.41 1108.48 L1358.64 1108.6 L1358.87 1108.14 L1359.1 1107.67 L1359.32 1108.03 L1359.55 1108.73 L1359.78 1108.46 L1360.01 1107.87 L1360.24 1108.3 L1360.46 1107.69 L1360.69 1106.97 L1360.92 1106.91 L1361.15 1107.12 L1361.37 1107.51 L1361.6 1107.74 L1361.83 1106.84 L1362.06 1107.19 L1362.29 1106.98 L1362.51 1105.9 L1362.74 1106.67 L1362.97 1106.88 L1363.2 1107.24 L1363.42 1107.45 L1363.65 1108.01 L1363.88 1107.79 L1364.11 1107.82 L1364.34 1107.49 L1364.56 1107.79 L1364.79 1107.2 L1365.02 1106.93 L1365.25 1106.3 L1365.47 1106.9 L1365.7 1107.78 L1365.93 1107.93 L1366.16 1107.75 L1366.39 1107.85 L1366.61 1109.34 L1366.84 1108.59 L1367.07 1109.63 L1367.3 1109.82 L1367.52 1109.57 L1367.75 1109.15 L1367.98 1109.43 L1368.21 1110.19 L1368.43 1110.25 L1368.66 1110.71 L1368.89 1110.95 L1369.12 1111.36 L1369.35 1111.72 L1369.57 1110.99 L1369.8 1111.75 L1370.03 1112.15 L1370.26 1112.72 L1370.48 1112.68 L1370.71 1112.82 L1370.94 1112.72 L1371.17 1112.84 L1371.4 1112.79 L1371.62 1112.67 L1371.85 1113.26 L1372.08 1113.59 L1372.31 1112.79 L1372.53 1112.87 L1372.76 1114.25 L1372.99 1113.77 L1373.22 1113.68 L1373.45 1113.74 L1373.67 1114.25 L1373.9 1114.39 L1374.13 1114.45 L1374.36 1114.1 L1374.58 1113.85 L1374.81 1113.28 L1375.04 1113.51 L1375.27 1113.87 L1375.5 1113.16 L1375.72 1112.56 L1375.95 1111.3 L1376.18 1111.66 L1376.41 1111.52 L1376.63 1112.35 L1376.86 1111.89 L1377.09 1111.62 L1377.32 1112.16 L1377.55 1111.98 L1377.77 1112.09 L1378 1113 L1378.23 1113.66 L1378.46 1114.54 L1378.68 1114.61 L1378.91 1114.51 L1379.14 1115.63 L1379.37 1115.08 L1379.6 1114.95 L1379.82 1114.87 L1380.05 1114.44 L1380.28 1114.9 L1380.51 1114.45 L1380.73 1114.59 L1380.96 1114.65 L1381.19 1114 L1381.42 1113.69 L1381.65 1113.2 L1381.87 1113.61 L1382.1 1113.64 L1382.33 1114.08 L1382.56 1114.32 L1382.78 1114.2 L1383.01 1113.99 L1383.24 1114.3 L1383.47 1114.72 L1383.7 1114.25 L1383.92 1114.56 L1384.15 1113.76 L1384.38 1114.95 L1384.61 1115.43 L1384.83 1115.21 L1385.06 1114.53 L1385.29 1113.87 L1385.52 1113.47 L1385.75 1112.49 L1385.97 1112.26 L1386.2 1112.29 L1386.43 1112.21 L1386.66 1112.1 L1386.88 1112.11 L1387.11 1112.46 L1387.34 1112.74 L1387.57 1112.17 L1387.8 1112.76 L1388.02 1112.15 L1388.25 1112.09 L1388.48 1111.88 L1388.71 1112.45 L1388.93 1112.14 L1389.16 1111.99 L1389.39 1110.36 L1389.62 1110.36 L1389.85 1110.17 L1390.07 1110.01 L1390.3 1110.02 L1390.53 1109.76 L1390.76 1110.07 L1390.98 1110.71 L1391.21 1111.1 L1391.44 1111.83 L1391.67 1111.89 L1391.9 1112.37 L1392.12 1112.41 L1392.35 1112.99 L1392.58 1113.08 L1392.81 1112.91 L1393.03 1113.43 L1393.26 1112.92 L1393.49 1111.58 L1393.72 1111.59 L1393.95 1110.77 L1394.17 1111.37 L1394.4 1111.35 L1394.63 1111.62 L1394.86 1112.67 L1395.08 1112.86 L1395.31 1111.54 L1395.54 1109.96 L1395.77 1109.5 L1396 1110.28 L1396.22 1110.2 L1396.45 1111.28 L1396.68 1111.02 L1396.91 1111.37 L1397.13 1112.08 L1397.36 1112.27 L1397.59 1112.94 L1397.82 1112.7 L1398.05 1112.33 L1398.27 1112.42 L1398.5 1112.02 L1398.73 1111.99 L1398.96 1111.49 L1399.18 1112.4 L1399.41 1112.5 L1399.64 1113.23 L1399.87 1113.92 L1400.1 1113.91 L1400.32 1114.1 L1400.55 1114.09 L1400.78 1114.45 L1401.01 1114.46 L1401.23 1114.92 L1401.46 1115.73 L1401.69 1115.95 L1401.92 1115.88 L1402.15 1116.22 L1402.37 1116.74 L1402.6 1117.27 L1402.83 1117.65 L1403.06 1115.86 L1403.28 1115.32 L1403.51 1115.79 L1403.74 1115.32 L1403.97 1114.4 L1404.19 1114.47 L1404.42 1114.68 L1404.65 1115.26 L1404.88 1115.63 L1405.11 1115.94 L1405.33 1116.13 L1405.56 1115.98 L1405.79 1115.48 L1406.02 1115.59 L1406.24 1115.05 L1406.47 1114.79 L1406.7 1113.58 L1406.93 1113.91 L1407.16 1113.74 L1407.38 1113.1 L1407.61 1113.31 L1407.84 1112.74 L1408.07 1112.62 L1408.29 1112.19 L1408.52 1111.52 L1408.75 1111.55 L1408.98 1111.53 L1409.21 1111.2 L1409.43 1111.2 L1409.66 1111.37 L1409.89 1111.5 L1410.12 1111.68 L1410.34 1111.7 L1410.57 1111.57 L1410.8 1111.5 L1411.03 1111.12 L1411.26 1110.54 L1411.48 1111.12 L1411.71 1111.44 L1411.94 1111.24 L1412.17 1111.45 L1412.39 1111.53 L1412.62 1111.61 L1412.85 1110.7 L1413.08 1110.7 L1413.31 1111.29 L1413.53 1111.52 L1413.76 1110.7 L1413.99 1110.21 L1414.22 1109.88 L1414.44 1109.36 L1414.67 1109.92 L1414.9 1109.35 L1415.13 1109.55 L1415.36 1108.76 L1415.58 1109.28 L1415.81 1108.23 L1416.04 1109.01 L1416.27 1109.13 L1416.49 1109.45 L1416.72 1109.98 L1416.95 1109.33 L1417.18 1109.53 L1417.41 1109.88 L1417.63 1109.21 L1417.86 1109.05 L1418.09 1108.97 L1418.32 1109.27 L1418.54 1108.72 L1418.77 1108.93 L1419 1107.64 L1419.23 1106.92 L1419.46 1107.09 L1419.68 1107.15 L1419.91 1106.41 L1420.14 1106.57 L1420.37 1106.26 L1420.59 1105.94 L1420.82 1105.09 L1421.05 1104.93 L1421.28 1105.22 L1421.51 1106.13 L1421.73 1106.16 L1421.96 1106.61 L1422.19 1106.13 L1422.42 1106.33 L1422.64 1106.86 L1422.87 1106.76 L1423.1 1106.45 L1423.33 1106.85 L1423.56 1107.08 L1423.78 1108.03 L1424.01 1108.35 L1424.24 1108.5 L1424.47 1108.08 L1424.69 1108.51 L1424.92 1108.96 L1425.15 1108.72 L1425.38 1108.95 L1425.61 1108.96 L1425.83 1108.85 L1426.06 1109.74 L1426.29 1109.6 L1426.52 1110.29 L1426.74 1110.74 L1426.97 1110.37 L1427.2 1110.51 L1427.43 1110.37 L1427.66 1110.09 L1427.88 1110.05 L1428.11 1108.98 L1428.34 1109.02 L1428.57 1108.97 L1428.79 1108.37 L1429.02 1107.98 L1429.25 1108.39 L1429.48 1107.26 L1429.71 1106.96 L1429.93 1106.38 L1430.16 1106.73 L1430.39 1107.14 L1430.62 1107.88 L1430.84 1108.85 L1431.07 1108.23 L1431.3 1108.07 L1431.53 1107.63 L1431.76 1107.27 L1431.98 1107.75 L1432.21 1107.25 L1432.44 1107.33 L1432.67 1107.01 L1432.89 1106.93 L1433.12 1106.35 L1433.35 1105.55 L1433.58 1105.01 L1433.81 1105.41 L1434.03 1104.97 L1434.26 1105.18 L1434.49 1105.16 L1434.72 1105.37 L1434.94 1105.22 L1435.17 1104.96 L1435.4 1105.31 L1435.63 1105.47 L1435.86 1105.94 L1436.08 1105.92 L1436.31 1106.16 L1436.54 1105.91 L1436.77 1105.96 L1436.99 1106.35 L1437.22 1106.6 L1437.45 1107.52 L1437.68 1107.51 L1437.91 1107.34 L1438.13 1106.95 L1438.36 1106.19 L1438.59 1106.11 L1438.82 1106.03 L1439.04 1106.15 L1439.27 1105.91 L1439.5 1106.44 L1439.73 1106.91 L1439.95 1106.5 L1440.18 1107.52 L1440.41 1107.62 L1440.64 1107.9 L1440.87 1108.55 L1441.09 1109.01 L1441.32 1109.69 L1441.55 1109.97 L1441.78 1110.03 L1442 1109.51 L1442.23 1109.09 L1442.46 1109.49 L1442.69 1109.99 L1442.92 1110.16 L1443.14 1110.57 L1443.37 1110.57 L1443.6 1111.14 L1443.83 1111.42 L1444.05 1112.03 L1444.28 1111.42 L1444.51 1110.32 L1444.74 1110.19 L1444.97 1110.58 L1445.19 1111.58 L1445.42 1112.78 L1445.65 1112.52 L1445.88 1112.52 L1446.1 1112.33 L1446.33 1112.28 L1446.56 1112.06 L1446.79 1111.69 L1447.02 1111.89 L1447.24 1112.63 L1447.47 1113.22 L1447.7 1113.47 L1447.93 1113.82 L1448.15 1113.16 L1448.38 1112.97 L1448.61 1112.4 L1448.84 1112.46 L1449.07 1112.55 L1449.29 1111.54 L1449.52 1112.08 L1449.75 1112.15 L1449.98 1111.22 L1450.2 1111.64 L1450.43 1112.04 L1450.66 1112.26 L1450.89 1113.14 L1451.12 1112.67 L1451.34 1112.53 L1451.57 1112.59 L1451.8 1112.58 L1452.03 1112.58 L1452.25 1112.5 L1452.48 1112.72 L1452.71 1113.54 L1452.94 1113.99 L1453.17 1113.84 L1453.39 1114.52 L1453.62 1115.1 L1453.85 1115.54 L1454.08 1115.15 L1454.3 1115.22 L1454.53 1114.93 L1454.76 1114.52 L1454.99 1114.47 L1455.22 1114.43 L1455.44 1114.46 L1455.67 1114.22 L1455.9 1113.61 L1456.13 1113.58 L1456.35 1112.7 L1456.58 1112.4 L1456.81 1112.37 L1457.04 1111.52 L1457.27 1110.74 L1457.49 1111.33 L1457.72 1111.93 L1457.95 1111.41 L1458.18 1111.76 L1458.4 1111.61 L1458.63 1112.05 L1458.86 1112.1 L1459.09 1112.59 L1459.32 1112.01 L1459.54 1111.96 L1459.77 1111.98 L1460 1111.39 L1460.23 1111.52 L1460.45 1111.18 L1460.68 1110.66 L1460.91 1110.69 L1461.14 1109.94 L1461.37 1110.21 L1461.59 1110.23 L1461.82 1110.79 L1462.05 1110.17 L1462.28 1111.02 L1462.5 1111.07 L1462.73 1111.19 L1462.96 1111.03 L1463.19 1111.29 L1463.42 1110.3 L1463.64 1111.53 L1463.87 1111.62 L1464.1 1111.99 L1464.33 1112.02 L1464.55 1112.2 L1464.78 1111.84 L1465.01 1112.38 L1465.24 1112.4 L1465.47 1112.63 L1465.69 1112.96 L1465.92 1112.84 L1466.15 1113.01 L1466.38 1113.4 L1466.6 1111.91 L1466.83 1111.6 L1467.06 1111.13 L1467.29 1110.43 L1467.52 1110.36 L1467.74 1110.9 L1467.97 1109.58 L1468.2 1109.86 L1468.43 1110.27 L1468.65 1109 L1468.88 1109.59 L1469.11 1110 L1469.34 1109.4 L1469.57 1109.9 L1469.79 1109.49 L1470.02 1110.44 L1470.25 1110.65 L1470.48 1111.18 L1470.7 1110.53 L1470.93 1110.49 L1471.16 1111.1 L1471.39 1110.55 L1471.62 1110.62 L1471.84 1110.67 L1472.07 1110.74 L1472.3 1110.12 L1472.53 1110.34 L1472.75 1110.59 L1472.98 1110.87 L1473.21 1111.63 L1473.44 1111.55 L1473.67 1112.55 L1473.89 1112.37 L1474.12 1111.65 L1474.35 1111.47 L1474.58 1110.85 L1474.8 1110.65 L1475.03 1110.3 L1475.26 1111.03 L1475.49 1111.29 L1475.71 1110.99 L1475.94 1110.77 L1476.17 1109.51 L1476.4 1109.57 L1476.63 1110.47 L1476.85 1110.24 L1477.08 1111.07 L1477.31 1111.47 L1477.54 1110.86 L1477.76 1110.98 L1477.99 1110.69 L1478.22 1109.77 L1478.45 1110.64 L1478.68 1109.88 L1478.9 1109.71 L1479.13 1109.09 L1479.36 1108.92 L1479.59 1108.57 L1479.81 1108.25 L1480.04 1108.48 L1480.27 1108.32 L1480.5 1107.48 L1480.73 1107.6 L1480.95 1107.85 L1481.18 1107.04 L1481.41 1107.46 L1481.64 1107.52 L1481.86 1107.44 L1482.09 1107.76 L1482.32 1108.29 L1482.55 1108.36 L1482.78 1107.66 L1483 1108.13 L1483.23 1108.13 L1483.46 1107.24 L1483.69 1106.37 L1483.91 1106.22 L1484.14 1106.15 L1484.37 1106.25 L1484.6 1105.76 L1484.83 1105.43 L1485.05 1105.42 L1485.28 1105.31 L1485.51 1105.49 L1485.74 1105.79 L1485.96 1105.83 L1486.19 1105.98 L1486.42 1106.36 L1486.65 1107 L1486.88 1106.16 L1487.1 1105.4 L1487.33 1105.22 L1487.56 1105.42 L1487.79 1105.43 L1488.01 1104.64 L1488.24 1104.94 L1488.47 1104.83 L1488.7 1104.69 L1488.93 1104.63 L1489.15 1105.9 L1489.38 1105.93 L1489.61 1106.29 L1489.84 1105.87 L1490.06 1104.35 L1490.29 1104 L1490.52 1104.52 L1490.75 1104.85 L1490.98 1104.51 L1491.2 1104.97 L1491.43 1104.87 L1491.66 1104.15 L1491.89 1104.56 L1492.11 1104.72 L1492.34 1105.34 L1492.57 1104.94 L1492.8 1104.71 L1493.03 1105.04 L1493.25 1105.12 L1493.48 1105.36 L1493.71 1106.2 L1493.94 1106.68 L1494.16 1106.7 L1494.39 1107.01 L1494.62 1107.71 L1494.85 1107.22 L1495.08 1107.64 L1495.3 1107.91 L1495.53 1107.8 L1495.76 1108.07 L1495.99 1108.55 L1496.21 1108.78 L1496.44 1108.96 L1496.67 1109.71 L1496.9 1109 L1497.13 1109.08 L1497.35 1109.38 L1497.58 1108.87 L1497.81 1109.17 L1498.04 1109.33 L1498.26 1109.34 L1498.49 1109.09 L1498.72 1108.32 L1498.95 1108.26 L1499.18 1108.34 L1499.4 1108.17 L1499.63 1108.51 L1499.86 1108.35 L1500.09 1108.86 L1500.31 1109.23 L1500.54 1108.31 L1500.77 1108.21 L1501 1107.26 L1501.23 1107.42 L1501.45 1107.82 L1501.68 1107.8 L1501.91 1107.13 L1502.14 1107.75 L1502.36 1107.76 L1502.59 1107.14 L1502.82 1107.58 L1503.05 1107.58 L1503.28 1106.88 L1503.5 1107.75 L1503.73 1107.08 L1503.96 1107.46 L1504.19 1106.95 L1504.41 1107.35 L1504.64 1108.01 L1504.87 1108.43 L1505.1 1107.98 L1505.33 1107.73 L1505.55 1107.96 L1505.78 1107.71 L1506.01 1107.48 L1506.24 1108.28 L1506.46 1108.28 L1506.69 1107.99 L1506.92 1107.33 L1507.15 1107.16 L1507.38 1107.1 L1507.6 1107.64 L1507.83 1107.38 L1508.06 1106.27 L1508.29 1106.97 L1508.51 1107.22 L1508.74 1107.61 L1508.97 1106.73 L1509.2 1107.48 L1509.43 1106.81 L1509.65 1107.27 L1509.88 1106.96 L1510.11 1106.65 L1510.34 1105.75 L1510.56 1106.29 L1510.79 1106.51 L1511.02 1107.27 L1511.25 1107.02 L1511.48 1107.2 L1511.7 1107.38 L1511.93 1107.84 L1512.16 1107.81 L1512.39 1108.05 L1512.61 1108.54 L1512.84 1108.78 L1513.07 1109.23 L1513.3 1110.06 L1513.52 1109.91 L1513.75 1108.92 L1513.98 1109.24 L1514.21 1109.55 L1514.44 1109.77 L1514.66 1108.9 L1514.89 1108.73 L1515.12 1109.09 L1515.35 1109.25 L1515.57 1108.55 L1515.8 1108.88 L1516.03 1109.51 L1516.26 1109.26 L1516.49 1109.34 L1516.71 1109.02 L1516.94 1109.57 L1517.17 1109.2 L1517.4 1109.21 L1517.62 1108.83 L1517.85 1108.35 L1518.08 1107.81 L1518.31 1108.26 L1518.54 1108.46 L1518.76 1108.52 L1518.99 1108.55 L1519.22 1108.52 L1519.45 1108.48 L1519.67 1108.59 L1519.9 1108.81 L1520.13 1108.6 L1520.36 1108.26 L1520.59 1108.2 L1520.81 1108.09 L1521.04 1109.1 L1521.27 1109.21 L1521.5 1109.1 L1521.72 1109.38 L1521.95 1109.3 L1522.18 1109.93 L1522.41 1109.88 L1522.64 1109.89 L1522.86 1110.2 L1523.09 1110.45 L1523.32 1111.09 L1523.55 1111.49 L1523.77 1110.98 L1524 1110.63 L1524.23 1110.73 L1524.46 1110.19 L1524.69 1110.9 L1524.91 1111.37 L1525.14 1111.01 L1525.37 1111.51 L1525.6 1110.72 L1525.82 1110.94 L1526.05 1110.02 L1526.28 1110.27 L1526.51 1110.41 L1526.74 1109.74 L1526.96 1108.99 L1527.19 1109.35 L1527.42 1109.59 L1527.65 1109.47 L1527.87 1109.59 L1528.1 1109.56 L1528.33 1109.16 L1528.56 1108.78 L1528.79 1107.69 L1529.01 1107.37 L1529.24 1108.08 L1529.47 1107.46 L1529.7 1107.62 L1529.92 1107.36 L1530.15 1106.12 L1530.38 1106.14 L1530.61 1106.9 L1530.84 1105.69 L1531.06 1105.2 L1531.29 1104.61 L1531.52 1104.65 L1531.75 1103.23 L1531.97 1103.44 L1532.2 1102.89 L1532.43 1102.67 L1532.66 1102.92 L1532.89 1103.87 L1533.11 1103.75 L1533.34 1103.55 L1533.57 1102.94 L1533.8 1103.91 L1534.02 1104.3 L1534.25 1103.01 L1534.48 1102.84 L1534.71 1103.06 L1534.94 1103.82 L1535.16 1103.49 L1535.39 1103.24 L1535.62 1103.65 L1535.85 1103.85 L1536.07 1103.61 L1536.3 1103.96 L1536.53 1103.96 L1536.76 1103.39 L1536.99 1103.06 L1537.21 1102.95 L1537.44 1103.59 L1537.67 1104.39 L1537.9 1104.37 L1538.12 1104.99 L1538.35 1105.78 L1538.58 1105.61 L1538.81 1106.47 L1539.04 1106.74 L1539.26 1106.92 L1539.49 1107.59 L1539.72 1107.75 L1539.95 1107.73 L1540.17 1107.86 L1540.4 1106.47 L1540.63 1106.14 L1540.86 1105.61 L1541.09 1106.07 L1541.31 1106.23 L1541.54 1106.69 L1541.77 1106.78 L1542 1107.18 L1542.22 1107.82 L1542.45 1107.75 L1542.68 1106.93 L1542.91 1107.36 L1543.14 1106.75 L1543.36 1107.33 L1543.59 1107.44 L1543.82 1107.06 L1544.05 1106.73 L1544.27 1106.34 L1544.5 1105.2 L1544.73 1106.33 L1544.96 1106.42 L1545.19 1105.69 L1545.41 1105.68 L1545.64 1105.94 L1545.87 1106.73 L1546.1 1106.09 L1546.32 1105.77 L1546.55 1105.93 L1546.78 1106.11 L1547.01 1105.9 L1547.24 1106.56 L1547.46 1107.14 L1547.69 1107.28 L1547.92 1107.85 L1548.15 1108.3 L1548.37 1108.6 L1548.6 1109.47 L1548.83 1109.6 L1549.06 1109.18 L1549.28 1109.12 L1549.51 1109.17 L1549.74 1108.52 L1549.97 1108.74 L1550.2 1108.26 L1550.42 1108.02 L1550.65 1108.36 L1550.88 1108.57 L1551.11 1108.58 L1551.33 1109.05 L1551.56 1109.99 L1551.79 1110.13 L1552.02 1109.96 L1552.25 1110.15 L1552.47 1109.82 L1552.7 1109.72 L1552.93 1110.29 L1553.16 1110.51 L1553.38 1110.3 L1553.61 1110.36 L1553.84 1110.61 L1554.07 1110.66 L1554.3 1111.18 L1554.52 1111.44 L1554.75 1110.85 L1554.98 1110.97 L1555.21 1110.67 L1555.43 1110.55 L1555.66 1110.57 L1555.89 1111.08 L1556.12 1111.59 L1556.35 1111.87 L1556.57 1111.83 L1556.8 1111.77 L1557.03 1111.43 L1557.26 1111.3 L1557.48 1111.14 L1557.71 1111.29 L1557.94 1111.36 L1558.17 1111.25 L1558.4 1110.62 L1558.62 1110.04 L1558.85 1110.77 L1559.08 1110.22 L1559.31 1110.24 L1559.53 1109.65 L1559.76 1109.93 L1559.99 1109.72 L1560.22 1110.32 L1560.45 1110.22 L1560.67 1109.29 L1560.9 1108.97 L1561.13 1108.73 L1561.36 1109.05 L1561.58 1109.39 L1561.81 1109.95 L1562.04 1109.31 L1562.27 1109.29 L1562.5 1109.11 L1562.72 1108.44 L1562.95 1108.93 L1563.18 1108.12 L1563.41 1107.69 L1563.63 1106.94 L1563.86 1106.7 L1564.09 1107.4 L1564.32 1107.26 L1564.55 1107.24 L1564.77 1106.69 L1565 1107.69 L1565.23 1107.31 L1565.46 1107.94 L1565.68 1108.08 L1565.91 1108.09 L1566.14 1108.18 L1566.37 1108.27 L1566.6 1108.16 L1566.82 1107.64 L1567.05 1107.46 L1567.28 1107.34 L1567.51 1107.17 L1567.73 1107.42 L1567.96 1107.2 L1568.19 1106.96 L1568.42 1106.33 L1568.65 1107.19 L1568.87 1107.49 L1569.1 1106.67 L1569.33 1106.47 L1569.56 1107.19 L1569.78 1107.24 L1570.01 1106.81 L1570.24 1106.55 L1570.47 1106.02 L1570.7 1106.28 L1570.92 1106.35 L1571.15 1105.56 L1571.38 1105.07 L1571.61 1104.78 L1571.83 1104.19 L1572.06 1104.14 L1572.29 1103.7 L1572.52 1104.75 L1572.75 1105.33 L1572.97 1105.59 L1573.2 1106.15 L1573.43 1106.81 L1573.66 1107.03 L1573.88 1106.66 L1574.11 1106.46 L1574.34 1106.42 L1574.57 1105.68 L1574.8 1104.76 L1575.02 1104.75 L1575.25 1104.13 L1575.48 1104.52 L1575.71 1104.87 L1575.93 1104.63 L1576.16 1104.98 L1576.39 1105.2 L1576.62 1105.23 L1576.85 1105.49 L1577.07 1106.05 L1577.3 1105.62 L1577.53 1105.37 L1577.76 1105.49 L1577.98 1106.43 L1578.21 1105.95 L1578.44 1105.78 L1578.67 1105.18 L1578.9 1104.88 L1579.12 1104.11 L1579.35 1103.98 L1579.58 1103.74 L1579.81 1103.82 L1580.03 1103.88 L1580.26 1103.64 L1580.49 1103.18 L1580.72 1103.27 L1580.95 1103.45 L1581.17 1104.12 L1581.4 1104.41 L1581.63 1103.9 L1581.86 1102.95 L1582.08 1103.57 L1582.31 1103.27 L1582.54 1103.16 L1582.77 1103.17 L1583 1103.34 L1583.22 1104.32 L1583.45 1104.85 L1583.68 1105.56 L1583.91 1104.61 L1584.13 1103.62 L1584.36 1104.3 L1584.59 1103.81 L1584.82 1103.74 L1585.04 1104 L1585.27 1103.97 L1585.5 1103.85 L1585.73 1103.31 L1585.96 1103.88 L1586.18 1104.74 L1586.41 1104.04 L1586.64 1104.27 L1586.87 1104.35 L1587.09 1104.47 L1587.32 1104.85 L1587.55 1105.12 L1587.78 1104.75 L1588.01 1104.32 L1588.23 1104.11 L1588.46 1103.69 L1588.69 1103.81 L1588.92 1104.66 L1589.14 1105.05 L1589.37 1105.2 L1589.6 1106.04 L1589.83 1106.14 L1590.06 1106.85 L1590.28 1107.3 L1590.51 1106.74 L1590.74 1106.22 L1590.97 1106.56 L1591.19 1106.88 L1591.42 1106.59 L1591.65 1107.21 L1591.88 1106.89 L1592.11 1106.81 L1592.33 1106.37 L1592.56 1106.23 L1592.79 1106.55 L1593.02 1107.2 L1593.24 1107.56 L1593.47 1107.35 L1593.7 1107.27 L1593.93 1107.25 L1594.16 1107.35 L1594.38 1107.03 L1594.61 1106.88 L1594.84 1106.51 L1595.07 1106.57 L1595.29 1106.88 L1595.52 1106.5 L1595.75 1105.93 L1595.98 1105.3 L1596.21 1105.7 L1596.43 1104.86 L1596.66 1104.9 L1596.89 1104.84 L1597.12 1105.02 L1597.34 1104.95 L1597.57 1104.69 L1597.8 1103.5 L1598.03 1103.77 L1598.26 1104.14 L1598.48 1103.75 L1598.71 1103.5 L1598.94 1103.47 L1599.17 1103.48 L1599.39 1103.1 L1599.62 1103.15 L1599.85 1102.73 L1600.08 1102.82 L1600.31 1102.15 L1600.53 1101.9 L1600.76 1100.84 L1600.99 1100.83 L1601.22 1100.61 L1601.44 1100.45 L1601.67 1099.89 L1601.9 1098.94 L1602.13 1099.16 L1602.36 1098.84 L1602.58 1099.21 L1602.81 1100.07 L1603.04 1100.62 L1603.27 1100.74 L1603.49 1101.54 L1603.72 1100.84 L1603.95 1101.33 L1604.18 1101.08 L1604.41 1100.85 L1604.63 1101.15 L1604.86 1101.87 L1605.09 1101.13 L1605.32 1101.46 L1605.54 1101.1 L1605.77 1101.07 L1606 1100.87 L1606.23 1101.46 L1606.46 1102.06 L1606.68 1101.71 L1606.91 1100.9 L1607.14 1100.93 L1607.37 1100 L1607.59 1100.12 L1607.82 1099.99 L1608.05 1100.31 L1608.28 1100.3 L1608.51 1098.78 L1608.73 1098.92 L1608.96 1099.59 L1609.19 1099.21 L1609.42 1099.1 L1609.64 1098.66 L1609.87 1099.31 L1610.1 1099.3 L1610.33 1099.46 L1610.56 1099.24 L1610.78 1099.54 L1611.01 1099.3 L1611.24 1098.71 L1611.47 1098.43 L1611.69 1098.59 L1611.92 1099.08 L1612.15 1099.48 L1612.38 1098.59 L1612.61 1098.6 L1612.83 1098.42 L1613.06 1098.47 L1613.29 1098.58 L1613.52 1098.76 L1613.74 1098.67 L1613.97 1098.57 L1614.2 1099.06 L1614.43 1098.07 L1614.66 1098.7 L1614.88 1098.95 L1615.11 1099.52 L1615.34 1099.84 L1615.57 1100.49 L1615.79 1100.86 L1616.02 1100.91 L1616.25 1101.03 L1616.48 1100.44 L1616.71 1100.93 L1616.93 1100.94 L1617.16 1100.6 L1617.39 1100.16 L1617.62 1100.53 L1617.84 1099.68 L1618.07 1099.59 L1618.3 1099.59 L1618.53 1099.44 L1618.76 1099.15 L1618.98 1098.66 L1619.21 1099.48 L1619.44 1098.98 L1619.67 1099.48 L1619.89 1099 L1620.12 1099.39 L1620.35 1099.63 L1620.58 1098.75 L1620.81 1097.89 L1621.03 1097.82 L1621.26 1097.87 L1621.49 1098.5 L1621.72 1098.91 L1621.94 1099.39 L1622.17 1099.61 L1622.4 1099.82 L1622.63 1099.85 L1622.85 1099.95 L1623.08 1100.73 L1623.31 1099.99 L1623.54 1100.36 L1623.77 1099.85 L1623.99 1099.55 L1624.22 1099.87 L1624.45 1099.44 L1624.68 1099.91 L1624.9 1100.58 L1625.13 1100.48 L1625.36 1100.75 L1625.59 1101.71 L1625.82 1102.33 L1626.04 1102.57 L1626.27 1102.65 L1626.5 1103.53 L1626.73 1103.61 L1626.95 1103.62 L1627.18 1104.09 L1627.41 1103.43 L1627.64 1103.99 L1627.87 1103.41 L1628.09 1102.51 L1628.32 1102.32 L1628.55 1102.56 L1628.78 1102.28 L1629 1102.21 L1629.23 1102.27 L1629.46 1100.97 L1629.69 1101.28 L1629.92 1101.03 L1630.14 1100.81 L1630.37 1099.85 L1630.6 1100.05 L1630.83 1100.25 L1631.05 1100.57 L1631.28 1100.91 L1631.51 1101.64 L1631.74 1101.53 L1631.97 1101.35 L1632.19 1100.94 L1632.42 1101.13 L1632.65 1100.58 L1632.88 1101.01 L1633.1 1101.2 L1633.33 1101.8 L1633.56 1101.14 L1633.79 1101.34 L1634.02 1101.11 L1634.24 1101.43 L1634.47 1101.11 L1634.7 1101.7 L1634.93 1102.31 L1635.15 1102.14 L1635.38 1102.07 L1635.61 1101.93 L1635.84 1101.66 L1636.07 1101.12 L1636.29 1101.95 L1636.52 1101.54 L1636.75 1100.98 L1636.98 1101.26 L1637.2 1100.83 L1637.43 1101.62 L1637.66 1101.82 L1637.89 1101.49 L1638.12 1101.86 L1638.34 1101.86 L1638.57 1101.97 L1638.8 1101.76 L1639.03 1101.88 L1639.25 1101.24 L1639.48 1100.76 L1639.71 1101.29 L1639.94 1101.69 L1640.17 1102 L1640.39 1101.92 L1640.62 1102.3 L1640.85 1102.24 L1641.08 1101.89 L1641.3 1102.28 L1641.53 1102.78 L1641.76 1102.87 L1641.99 1103.04 L1642.22 1103.51 L1642.44 1103.77 L1642.67 1103.58 L1642.9 1103.56 L1643.13 1104.03 L1643.35 1103.57 L1643.58 1103.62 L1643.81 1103.68 L1644.04 1103.83 L1644.27 1104.1 L1644.49 1104.05 L1644.72 1103.55 L1644.95 1104.06 L1645.18 1103.56 L1645.4 1103.32 L1645.63 1102.05 L1645.86 1102.13 L1646.09 1102.23 L1646.32 1102.84 L1646.54 1102.78 L1646.77 1103.38 L1647 1104.05 L1647.23 1104.67 L1647.45 1105.05 L1647.68 1104.77 L1647.91 1104.96 L1648.14 1105.08 L1648.37 1104.85 L1648.59 1105.04 L1648.82 1105.29 L1649.05 1105.54 L1649.28 1105.45 L1649.5 1106.11 L1649.73 1106.18 L1649.96 1106.77 L1650.19 1107.33 L1650.42 1107.24 L1650.64 1107.17 L1650.87 1106.89 L1651.1 1107.75 L1651.33 1107.7 L1651.55 1108.11 L1651.78 1107.35 L1652.01 1107.37 L1652.24 1107.59 L1652.47 1107.93 L1652.69 1107.45 L1652.92 1108.03 L1653.15 1107.77 L1653.38 1108.49 L1653.6 1107.84 L1653.83 1107.61 L1654.06 1107.05 L1654.29 1107.5 L1654.52 1107.85 L1654.74 1107.95 L1654.97 1107.67 L1655.2 1107.92 L1655.43 1108.65 L1655.65 1108.84 L1655.88 1108.94 L1656.11 1109.66 L1656.34 1109.28 L1656.57 1109.03 L1656.79 1108.97 L1657.02 1109.33 L1657.25 1109.26 L1657.48 1109.5 L1657.7 1110.09 L1657.93 1109.98 L1658.16 1109.96 L1658.39 1110.6 L1658.61 1110.45 L1658.84 1110.9 L1659.07 1111.64 L1659.3 1112.23 L1659.53 1112.74 L1659.75 1112.95 L1659.98 1113.21 L1660.21 1113.04 L1660.44 1112.93 L1660.66 1112.51 L1660.89 1112.72 L1661.12 1112.4 L1661.35 1112.96 L1661.58 1112.4 L1661.8 1113.07 L1662.03 1113.43 L1662.26 1113.87 L1662.49 1113.67 L1662.71 1114.4 L1662.94 1113.87 L1663.17 1114.29 L1663.4 1113.98 L1663.63 1114.09 L1663.85 1114.56 L1664.08 1114.66 L1664.31 1114.28 L1664.54 1114.57 L1664.76 1113.95 L1664.99 1114.07 L1665.22 1114.04 L1665.45 1113.46 L1665.68 1114.15 L1665.9 1114.91 L1666.13 1114.72 L1666.36 1114.24 L1666.59 1115.22 L1666.81 1115.45 L1667.04 1114.63 L1667.27 1115.54 L1667.5 1116.05 L1667.73 1115.87 L1667.95 1116.7 L1668.18 1116.73 L1668.41 1116.51 L1668.64 1116.08 L1668.86 1115.35 L1669.09 1115.62 L1669.32 1115.39 L1669.55 1115.85 L1669.78 1115.74 L1670 1116.03 L1670.23 1115.7 L1670.46 1116.4 L1670.69 1115.8 L1670.91 1115.94 L1671.14 1115.21 L1671.37 1115.43 L1671.6 1115.37 L1671.83 1115.67 L1672.05 1116.22 L1672.28 1115.76 L1672.51 1115.77 L1672.74 1114.54 L1672.96 1114.6 L1673.19 1114.66 L1673.42 1115.3 L1673.65 1115 L1673.88 1115.25 L1674.1 1115.17 L1674.33 1115.18 L1674.56 1115.58 L1674.79 1116.18 L1675.01 1116.33 L1675.24 1116.2 L1675.47 1115.81 L1675.7 1115.9 L1675.93 1115.69 L1676.15 1115.68 L1676.38 1115.06 L1676.61 1114.99 L1676.84 1115.06 L1677.06 1115.22 L1677.29 1115.13 L1677.52 1115.81 L1677.75 1115.84 L1677.98 1115.24 L1678.2 1115.33 L1678.43 1115.68 L1678.66 1115.11 L1678.89 1115.25 L1679.11 1115.09 L1679.34 1115.34 L1679.57 1114.73 L1679.8 1114.55 L1680.03 1114.3 L1680.25 1114.42 L1680.48 1114.05 L1680.71 1113.55 L1680.94 1112.71 L1681.16 1112.64 L1681.39 1112.47 L1681.62 1112.5 L1681.85 1112.05 L1682.08 1111.95 L1682.3 1111.67 L1682.53 1112.6 L1682.76 1112.69 L1682.99 1112.16 L1683.21 1111.99 L1683.44 1111.41 L1683.67 1110.41 L1683.9 1111.14 L1684.13 1110.53 L1684.35 1110.39 L1684.58 1110.33 L1684.81 1110.6 L1685.04 1109.74 L1685.26 1109.87 L1685.49 1109.84 L1685.72 1109.5 L1685.95 1109.64 L1686.18 1109.29 L1686.4 1109.17 L1686.63 1109.64 L1686.86 1109.94 L1687.09 1109.86 L1687.31 1109.02 L1687.54 1108.84 L1687.77 1108.55 L1688 1108.85 L1688.23 1109.23 L1688.45 1109.29 L1688.68 1109.34 L1688.91 1108.9 L1689.14 1108.82 L1689.36 1109.29 L1689.59 1109.51 L1689.82 1110.07 L1690.05 1110.18 L1690.28 1110.64 L1690.5 1110.91 L1690.73 1110.56 L1690.96 1111.16 L1691.19 1111.16 L1691.41 1110.72 L1691.64 1111.15 L1691.87 1111.08 L1692.1 1111.02 L1692.33 1110.76 L1692.55 1110.38 L1692.78 1110.38 L1693.01 1110.73 L1693.24 1111.14 L1693.46 1110.84 L1693.69 1111.15 L1693.92 1111 L1694.15 1111.48 L1694.37 1111.5 L1694.6 1112.17 L1694.83 1112.55 L1695.06 1112.74 L1695.29 1112.54 L1695.51 1112.57 L1695.74 1113.27 L1695.97 1113.88 L1696.2 1113.2 L1696.42 1113.44 L1696.65 1113.46 L1696.88 1112.66 L1697.11 1112.18 L1697.34 1112.29 L1697.56 1111.69 L1697.79 1110.94 L1698.02 1110.96 L1698.25 1111.14 L1698.47 1111.3 L1698.7 1111.54 L1698.93 1111.2 L1699.16 1111 L1699.39 1110.96 L1699.61 1110.05 L1699.84 1110.24 L1700.07 1110.46 L1700.3 1110.38 L1700.52 1109.75 L1700.75 1110.02 L1700.98 1109.88 L1701.21 1110.22 L1701.44 1110.13 L1701.66 1108.99 L1701.89 1108.73 L1702.12 1107.88 L1702.35 1108.43 L1702.57 1108.73 L1702.8 1108.61 L1703.03 1108.78 L1703.26 1108.66 L1703.49 1108.62 L1703.71 1108.41 L1703.94 1108.88 L1704.17 1109.53 L1704.4 1109.7 L1704.62 1109.54 L1704.85 1108.82 L1705.08 1108.65 L1705.31 1108.91 L1705.54 1109.42 L1705.76 1109.38 L1705.99 1108.54 L1706.22 1109.26 L1706.45 1108.9 L1706.67 1109.04 L1706.9 1108.66 L1707.13 1108.66 L1707.36 1108.62 L1707.59 1108.61 L1707.81 1109.3 L1708.04 1109.53 L1708.27 1110.35 L1708.5 1109.65 L1708.72 1109.78 L1708.95 1110.19 L1709.18 1110.38 L1709.41 1110.25 L1709.64 1110.13 L1709.86 1110.37 L1710.09 1110.31 L1710.32 1110.29 L1710.55 1110.79 L1710.77 1111.05 L1711 1110.1 L1711.23 1110.06 L1711.46 1109.49 L1711.69 1109.16 L1711.91 1109.78 L1712.14 1110.37 L1712.37 1110.41 L1712.6 1110.86 L1712.82 1110.25 L1713.05 1110.65 L1713.28 1111.14 L1713.51 1111.38 L1713.74 1111.45 L1713.96 1111.2 L1714.19 1110.06 L1714.42 1110.04 L1714.65 1110.39 L1714.87 1109.98 L1715.1 1109.94 L1715.33 1110.05 L1715.56 1110.13 L1715.79 1110.12 L1716.01 1110.38 L1716.24 1110.27 L1716.47 1110.57 L1716.7 1110.96 L1716.92 1110.62 L1717.15 1110.84 L1717.38 1111.15 L1717.61 1110.69 L1717.84 1110.15 L1718.06 1109.59 L1718.29 1109.74 L1718.52 1109.4 L1718.75 1109.51 L1718.97 1109.59 L1719.2 1109.3 L1719.43 1109.14 L1719.66 1108.99 L1719.89 1108.94 L1720.11 1108.95 L1720.34 1109.76 L1720.57 1109.99 L1720.8 1110.03 L1721.02 1110.35 L1721.25 1109.88 L1721.48 1109.63 L1721.71 1109.27 L1721.94 1109.82 L1722.16 1110.32 L1722.39 1110.11 L1722.62 1110.31 L1722.85 1110.47 L1723.07 1109.95 L1723.3 1109.79 L1723.53 1109.32 L1723.76 1110.03 L1723.99 1109.74 L1724.21 1109.93 L1724.44 1110.01 L1724.67 1110.35 L1724.9 1110.59 L1725.12 1110.42 L1725.35 1110.73 L1725.58 1111.13 L1725.81 1111.01 L1726.04 1111.21 L1726.26 1111.35 L1726.49 1111.11 L1726.72 1110.68 L1726.95 1110.2 L1727.17 1109.54 L1727.4 1109.48 L1727.63 1109.27 L1727.86 1109.27 L1728.09 1109.69 L1728.31 1110.2 L1728.54 1110.36 L1728.77 1110.63 L1729 1110.87 L1729.22 1110 L1729.45 1109.74 L1729.68 1110.22 L1729.91 1109.92 L1730.14 1110.44 L1730.36 1110.04 L1730.59 1110.09 L1730.82 1110.2 L1731.05 1109.85 L1731.27 1109.78 L1731.5 1109.68 L1731.73 1109.65 L1731.96 1109.64 L1732.18 1109.87 L1732.41 1110.13 L1732.64 1110.32 L1732.87 1110.17 L1733.1 1109.74 L1733.32 1110.29 L1733.55 1110.04 L1733.78 1109.64 L1734.01 1109.99 L1734.23 1109.94 L1734.46 1109.75 L1734.69 1109.95 L1734.92 1109.97 L1735.15 1109.12 L1735.37 1109.64 L1735.6 1109.84 L1735.83 1109.31 L1736.06 1109.66 L1736.28 1109.81 L1736.51 1109.45 L1736.74 1108.78 L1736.97 1108.08 L1737.2 1108.33 L1737.42 1107.95 L1737.65 1108.04 L1737.88 1108.05 L1738.11 1108.43 L1738.33 1108.12 L1738.56 1108.79 L1738.79 1108.38 L1739.02 1107.6 L1739.25 1107.72 L1739.47 1107.83 L1739.7 1107.94 L1739.93 1108.37 L1740.16 1108.2 L1740.38 1108.87 L1740.61 1109.2 L1740.84 1109.6 L1741.07 1109.95 L1741.3 1109.99 L1741.52 1109.91 L1741.75 1108.96 L1741.98 1109.5 L1742.21 1109.44 L1742.43 1109.6 L1742.66 1109.7 L1742.89 1109.72 L1743.12 1110.09 L1743.35 1109.03 L1743.57 1109.21 L1743.8 1109.46 L1744.03 1109.99 L1744.26 1110.11 L1744.48 1110.15 L1744.71 1110.05 L1744.94 1110.16 L1745.17 1110.86 L1745.4 1111.32 L1745.62 1111.82 L1745.85 1112.5 L1746.08 1111.96 L1746.31 1112.46 L1746.53 1112.91 L1746.76 1112.64 L1746.99 1112.7 L1747.22 1112.31 L1747.45 1112.06 L1747.67 1111.71 L1747.9 1111.41 L1748.13 1110.75 L1748.36 1110.96 L1748.58 1111.4 L1748.81 1111.97 L1749.04 1111.71 L1749.27 1111.39 L1749.5 1111.26 L1749.72 1110.13 L1749.95 1110.3 L1750.18 1110.53 L1750.41 1110.34 L1750.63 1110.86 L1750.86 1111.08 L1751.09 1111.46 L1751.32 1111.59 L1751.55 1111.24 L1751.77 1111.46 L1752 1111.36 L1752.23 1110.95 L1752.46 1110.86 L1752.68 1110.63 L1752.91 1111.01 L1753.14 1111.21 L1753.37 1111.61 L1753.6 1111.04 L1753.82 1110.47 L1754.05 1110.98 L1754.28 1110.15 L1754.51 1110.58 L1754.73 1110.75 L1754.96 1111.44 L1755.19 1111.41 L1755.42 1111.3 L1755.65 1111.17 L1755.87 1111.57 L1756.1 1111.84 L1756.33 1111.06 L1756.56 1111.09 L1756.78 1111.22 L1757.01 1111.53 L1757.24 1112.12 L1757.47 1112.1 L1757.7 1111.53 L1757.92 1111.49 L1758.15 1111.43 L1758.38 1110.83 L1758.61 1111.11 L1758.83 1110.62 L1759.06 1110.64 L1759.29 1111.32 L1759.52 1111.07 L1759.75 1111.29 L1759.97 1110.74 L1760.2 1111.49 L1760.43 1110.96 L1760.66 1111.15 L1760.88 1110.9 L1761.11 1110.44 L1761.34 1110.21 L1761.57 1110.53 L1761.8 1111.26 L1762.02 1111 L1762.25 1110.97 L1762.48 1111.1 L1762.71 1111.44 L1762.93 1111.57 L1763.16 1111.81 L1763.39 1112.02 L1763.62 1111.92 L1763.85 1111.49 L1764.07 1111.87 L1764.3 1111.97 L1764.53 1111.65 L1764.76 1111.8 L1764.98 1111.74 L1765.21 1111.62 L1765.44 1111.72 L1765.67 1111.86 L1765.9 1112.2 L1766.12 1112.54 L1766.35 1112.98 L1766.58 1112.91 L1766.81 1113.5 L1767.03 1113.55 L1767.26 1113.19 L1767.49 1113.54 L1767.72 1112.47 L1767.94 1112.48 L1768.17 1112.87 L1768.4 1112.57 L1768.63 1112.48 L1768.86 1112.59 L1769.08 1112.18 L1769.31 1112.45 L1769.54 1112.84 L1769.77 1112.6 L1769.99 1112.42 L1770.22 1112.83 L1770.45 1113.04 L1770.68 1113.05 L1770.91 1112.81 L1771.13 1112.6 L1771.36 1112.67 L1771.59 1112.48 L1771.82 1112.32 L1772.04 1112.22 L1772.27 1112.14 L1772.5 1111.97 L1772.73 1111.76 L1772.96 1111.71 L1773.18 1111.31 L1773.41 1110.84 L1773.64 1110.78 L1773.87 1111.01 L1774.09 1110.64 L1774.32 1110.31 L1774.55 1110.38 L1774.78 1110.49 L1775.01 1111.02 L1775.23 1111.27 L1775.46 1111.25 L1775.69 1111.32 L1775.92 1111.32 L1776.14 1111.3 L1776.37 1111.11 L1776.6 1111.28 L1776.83 1111.27 L1777.06 1110.47 L1777.28 1110.38 L1777.51 1110.51 L1777.74 1110.86 L1777.97 1111.21 L1778.19 1111.49 L1778.42 1111.62 L1778.65 1111.1 L1778.88 1110.93 L1779.11 1110.39 L1779.33 1110.32 L1779.56 1110.57 L1779.79 1110.59 L1780.02 1110.55 L1780.24 1111.01 L1780.47 1110.88 L1780.7 1110.97 L1780.93 1111.98 L1781.16 1112.7 L1781.38 1112.38 L1781.61 1112.26 L1781.84 1112.77 L1782.07 1112.37 L1782.29 1112.72 L1782.52 1112.35 L1782.75 1112.52 L1782.98 1112.32 L1783.21 1112.63 L1783.43 1113.3 L1783.66 1112.67 L1783.89 1112.8 L1784.12 1112.65 L1784.34 1112.05 L1784.57 1111.81 L1784.8 1112.02 L1785.03 1112.1 L1785.26 1112.01 L1785.48 1112.24 L1785.71 1111.96 L1785.94 1111.35 L1786.17 1111.86 L1786.39 1111.56 L1786.62 1112.42 L1786.85 1112.6 L1787.08 1113.42 L1787.31 1112.89 L1787.53 1112.59 L1787.76 1112.16 L1787.99 1112.57 L1788.22 1112.23 L1788.44 1112.35 L1788.67 1112.77 L1788.9 1112.94 L1789.13 1113.31 L1789.36 1113.32 L1789.58 1112.05 L1789.81 1112.17 L1790.04 1112.73 L1790.27 1111.67 L1790.49 1112.14 L1790.72 1112.36 L1790.95 1112.45 L1791.18 1112.83 L1791.41 1112.51 L1791.63 1113.2 L1791.86 1112.85 L1792.09 1112.6 L1792.32 1112.96 L1792.54 1112.62 L1792.77 1112.34 L1793 1112.55 L1793.23 1112.5 L1793.46 1112.38 L1793.68 1112.42 L1793.91 1112.55 L1794.14 1112.25 L1794.37 1111.71 L1794.59 1111.87 L1794.82 1111.98 L1795.05 1111.95 L1795.28 1112.41 L1795.51 1112.21 L1795.73 1111.88 L1795.96 1111.76 L1796.19 1111.82 L1796.42 1112.12 L1796.64 1111.59 L1796.87 1111.29 L1797.1 1111.18 L1797.33 1111.21 L1797.56 1111.37 L1797.78 1111.28 L1798.01 1111.14 L1798.24 1111.28 L1798.47 1111.42 L1798.69 1111.01 L1798.92 1111.2 L1799.15 1110.92 L1799.38 1111.51 L1799.61 1111.36 L1799.83 1111.5 L1800.06 1111.48 L1800.29 1111.45 L1800.52 1111.07 L1800.74 1111.73 L1800.97 1112.07 L1801.2 1112.44 L1801.43 1112.37 L1801.66 1112.51 L1801.88 1112.41 L1802.11 1112.54 L1802.34 1112.53 L1802.57 1112.48 L1802.79 1112.61 L1803.02 1113.26 L1803.25 1112.71 L1803.48 1113.05 L1803.7 1112.69 L1803.93 1112.76 L1804.16 1113.11 L1804.39 1113.3 L1804.62 1113.43 L1804.84 1113.15 L1805.07 1113.03 L1805.3 1112.94 L1805.53 1112.8 L1805.75 1113.25 L1805.98 1114.04 L1806.21 1114.21 L1806.44 1114.17 L1806.67 1113.84 L1806.89 1113.93 L1807.12 1113.13 L1807.35 1112.72 L1807.58 1113.57 L1807.8 1114.13 L1808.03 1113.99 L1808.26 1113.69 L1808.49 1113.55 L1808.72 1114.09 L1808.94 1114.56 L1809.17 1114.47 L1809.4 1114.1 L1809.63 1113.16 L1809.85 1113.06 L1810.08 1113.31 L1810.31 1112.8 L1810.54 1113.56 L1810.77 1113.59 L1810.99 1113.58 L1811.22 1114.05 L1811.45 1114.07 L1811.68 1114.08 L1811.9 1113.82 L1812.13 1113.81 L1812.36 1113.97 L1812.59 1113.38 L1812.82 1113.66 L1813.04 1113.8 L1813.27 1113.76 L1813.5 1113.9 L1813.73 1113.99 L1813.95 1114.4 L1814.18 1114.61 L1814.41 1115.32 L1814.64 1115.17 L1814.87 1115.27 L1815.09 1115.73 L1815.32 1116.64 L1815.55 1117.34 L1815.78 1117.6 L1816 1117.07 L1816.23 1117.32 L1816.46 1117.75 L1816.69 1117.01 L1816.92 1116.91 L1817.14 1116.84 L1817.37 1117.26 L1817.6 1117.35 L1817.83 1118.09 L1818.05 1117.86 L1818.28 1118.64 L1818.51 1118.44 L1818.74 1117.63 L1818.97 1116.96 L1819.19 1117.09 L1819.42 1117.26 L1819.65 1117.43 L1819.88 1116.9 L1820.1 1116.64 L1820.33 1116.53 L1820.56 1116.45 L1820.79 1116.38 L1821.02 1116.07 L1821.24 1115.35 L1821.47 1115.32 L1821.7 1115.53 L1821.93 1115.76 L1822.15 1115.61 L1822.38 1115.41 L1822.61 1115.19 L1822.84 1115.51 L1823.07 1115.86 L1823.29 1115.67 L1823.52 1116.25 L1823.75 1116.67 L1823.98 1115.59 L1824.2 1115.66 L1824.43 1115.11 L1824.66 1115.13 L1824.89 1114.99 L1825.12 1115.38 L1825.34 1114.99 L1825.57 1115.21 L1825.8 1115.65 L1826.03 1114.96 L1826.25 1115.1 L1826.48 1115.42 L1826.71 1115.97 L1826.94 1116.32 L1827.17 1116.42 L1827.39 1115.79 L1827.62 1116.12 L1827.85 1116.14 L1828.08 1116.01 L1828.3 1115.75 L1828.53 1115.87 L1828.76 1115.62 L1828.99 1115.88 L1829.22 1115.99 L1829.44 1116.03 L1829.67 1115.71 L1829.9 1115.67 L1830.13 1115.61 L1830.35 1115.53 L1830.58 1114.74 L1830.81 1114.77 L1831.04 1114.63 L1831.27 1114.8 L1831.49 1114.99 L1831.72 1114.91 L1831.95 1115.01 L1832.18 1114 L1832.4 1113.98 L1832.63 1113.7 L1832.86 1113.64 L1833.09 1113.61 L1833.32 1113.45 L1833.54 1112.76 L1833.77 1112.83 L1834 1113.07 L1834.23 1113.08 L1834.45 1112.75 L1834.68 1113.25 L1834.91 1113.58 L1835.14 1114.09 L1835.37 1114.35 L1835.59 1115.07 L1835.82 1115.61 L1836.05 1115.67 L1836.28 1115.08 L1836.5 1115.2 L1836.73 1115.34 L1836.96 1115.56 L1837.19 1115.6 L1837.42 1115.89 L1837.64 1116.53 L1837.87 1117.14 L1838.1 1117.04 L1838.33 1116.97 L1838.55 1116.71 L1838.78 1116.71 L1839.01 1116.92 L1839.24 1117.12 L1839.46 1116.6 L1839.69 1117.12 L1839.92 1116.3 L1840.15 1116.12 L1840.38 1116.48 L1840.6 1116.98 L1840.83 1117.35 L1841.06 1117.8 L1841.29 1117.33 L1841.51 1116.95 L1841.74 1117.5 L1841.97 1117.72 L1842.2 1117.56 L1842.43 1117.24 L1842.65 1116.93 L1842.88 1117.32 L1843.11 1117.2 L1843.34 1116.83 L1843.56 1116.89 L1843.79 1117.13 L1844.02 1117.61 L1844.25 1117.9 L1844.48 1118.16 L1844.7 1117.73 L1844.93 1117.74 L1845.16 1117.87 L1845.39 1118.19 L1845.61 1118.32 L1845.84 1118.16 L1846.07 1117.94 L1846.3 1117.02 L1846.53 1116.71 L1846.75 1117.07 L1846.98 1117.16 L1847.21 1117.2 L1847.44 1117.39 L1847.66 1117.16 L1847.89 1117.11 L1848.12 1117.01 L1848.35 1116.96 L1848.58 1116.35 L1848.8 1116.35 L1849.03 1116.06 L1849.26 1115.95 L1849.49 1116.23 L1849.71 1116.05 L1849.94 1115.82 L1850.17 1116.26 L1850.4 1116.5 L1850.63 1116.61 L1850.85 1116.22 L1851.08 1115.36 L1851.31 1115.39 L1851.54 1116.32 L1851.76 1116.59 L1851.99 1116.77 L1852.22 1117.02 L1852.45 1116.92 L1852.68 1116.93 L1852.9 1116.85 L1853.13 1116.32 L1853.36 1116.28 L1853.59 1116.76 L1853.81 1116.37 L1854.04 1116.53 L1854.27 1116.86 L1854.5 1116.79 L1854.73 1116.98 L1854.95 1116.03 L1855.18 1116.21 L1855.41 1116.69 L1855.64 1116.98 L1855.86 1116.65 L1856.09 1116.81 L1856.32 1116.05 L1856.55 1116.48 L1856.78 1116.43 L1857 1116.74 L1857.23 1116.27 L1857.46 1115.7 L1857.69 1115.65 L1857.91 1115.53 L1858.14 1115.26 L1858.37 1114.6 L1858.6 1114.25 L1858.83 1114.27 L1859.05 1114.69 L1859.28 1115.19 L1859.51 1115.5 L1859.74 1116.17 L1859.96 1116.01 L1860.19 1115.86 L1860.42 1116.03 L1860.65 1116.09 L1860.88 1115.93 L1861.1 1115.8 L1861.33 1115.44 L1861.56 1115.76 L1861.79 1115.22 L1862.01 1115.59 L1862.24 1115.93 L1862.47 1115.97 L1862.7 1115.65 L1862.93 1115.5 L1863.15 1115.46 L1863.38 1115.51 L1863.61 1115.48 L1863.84 1115.81 L1864.06 1115.57 L1864.29 1115.39 L1864.52 1115.73 L1864.75 1115.11 L1864.98 1115.07 L1865.2 1115.35 L1865.43 1115.05 L1865.66 1115.67 L1865.89 1115.89 L1866.11 1115.92 L1866.34 1116.46 L1866.57 1116.63 L1866.8 1117.46 L1867.03 1116.87 L1867.25 1116.69 L1867.48 1117.16 L1867.71 1116.54 L1867.94 1117.09 L1868.16 1115.8 L1868.39 1115.63 L1868.62 1115.74 L1868.85 1116.09 L1869.08 1116.38 L1869.3 1116.57 L1869.53 1116.77 L1869.76 1116.69 L1869.99 1116.61 L1870.21 1116.14 L1870.44 1116.8 L1870.67 1116.9 L1870.9 1116.87 L1871.13 1116.48 L1871.35 1116.93 L1871.58 1117.15 L1871.81 1117.21 L1872.04 1116.57 L1872.26 1117.23 L1872.49 1117.07 L1872.72 1117.23 L1872.95 1117.44 L1873.18 1117.78 L1873.4 1118.04 L1873.63 1117.27 L1873.86 1117.48 L1874.09 1117.65 L1874.31 1117.26 L1874.54 1117.65 L1874.77 1117.53 L1875 1117.63 L1875.23 1118.04 L1875.45 1117.45 L1875.68 1116.65 L1875.91 1116.61 L1876.14 1116.92 L1876.36 1117.56 L1876.59 1118.09 L1876.82 1118.41 L1877.05 1118.6 L1877.27 1118.8 L1877.5 1119.12 L1877.73 1119.24 L1877.96 1119.63 L1878.19 1119.94 L1878.41 1119.82 L1878.64 1119.52 L1878.87 1119.43 L1879.1 1119.8 L1879.32 1120.06 L1879.55 1120.41 L1879.78 1119.83 L1880.01 1120.62 L1880.24 1121.01 L1880.46 1120.48 L1880.69 1120.48 L1880.92 1120.61 L1881.15 1120.12 L1881.37 1120.35 L1881.6 1120.08 L1881.83 1120.22 L1882.06 1119.76 L1882.29 1118.89 L1882.51 1119.73 L1882.74 1119.19 L1882.97 1119.14 L1883.2 1118.52 L1883.42 1118.13 L1883.65 1118.72 L1883.88 1118.49 L1884.11 1118.49 L1884.34 1118.5 L1884.56 1118.31 L1884.79 1118.92 L1885.02 1118.51 L1885.25 1118.28 L1885.47 1117.83 L1885.7 1117.79 L1885.93 1118.09 L1886.16 1118.37 L1886.39 1117.94 L1886.61 1117.86 L1886.84 1118.11 L1887.07 1118 L1887.3 1118.21 L1887.52 1117.9 L1887.75 1118.25 L1887.98 1117.51 L1888.21 1117.24 L1888.44 1116.89 L1888.66 1117.94 L1888.89 1117.42 L1889.12 1118.02 L1889.35 1118.47 L1889.57 1117.45 L1889.8 1117.44 L1890.03 1117.11 L1890.26 1117.32 L1890.49 1117.37 L1890.71 1117.83 L1890.94 1117.81 L1891.17 1117.8 L1891.4 1118.17 L1891.62 1118.33 L1891.85 1118.24 L1892.08 1118.47 L1892.31 1118.86 L1892.54 1119.29 L1892.76 1119.56 L1892.99 1119.75 L1893.22 1118.89 L1893.45 1118.97 L1893.67 1119.08 L1893.9 1119.52 L1894.13 1119.66 L1894.36 1119.48 L1894.59 1118.88 L1894.81 1118.34 L1895.04 1118.03 L1895.27 1118.24 L1895.5 1118.57 L1895.72 1118.11 L1895.95 1118.13 L1896.18 1118.25 L1896.41 1118.57 L1896.64 1118.76 L1896.86 1118.9 L1897.09 1118.93 L1897.32 1119.25 L1897.55 1119.08 L1897.77 1118.79 L1898 1119.3 L1898.23 1119.04 L1898.46 1118.59 L1898.69 1118.98 L1898.91 1118.6 L1899.14 1118.08 L1899.37 1117.54 L1899.6 1117.46 L1899.82 1117.25 L1900.05 1116.95 L1900.28 1116.4 L1900.51 1115.96 L1900.74 1116.02 L1900.96 1116.35 L1901.19 1116.75 L1901.42 1116.66 L1901.65 1116.65 L1901.87 1117.04 L1902.1 1117.55 L1902.33 1117.95 L1902.56 1118.28 L1902.79 1117.95 L1903.01 1117.85 L1903.24 1117.63 L1903.47 1117.16 L1903.7 1116.93 L1903.92 1117 L1904.15 1117.31 L1904.38 1117.54 L1904.61 1117.64 L1904.84 1117.98 L1905.06 1117.89 L1905.29 1117.74 L1905.52 1117.19 L1905.75 1116.93 L1905.97 1117.23 L1906.2 1116.21 L1906.43 1116.56 L1906.66 1116.21 L1906.89 1116.44 L1907.11 1116.84 L1907.34 1116.46 L1907.57 1116.26 L1907.8 1116.23 L1908.02 1116.52 L1908.25 1116.67 L1908.48 1116.58 L1908.71 1116.91 L1908.94 1116.29 L1909.16 1116.53 L1909.39 1117 L1909.62 1117.15 L1909.85 1117.02 L1910.07 1117.61 L1910.3 1118.35 L1910.53 1117.42 L1910.76 1117.99 L1910.99 1118.07 L1911.21 1118.68 L1911.44 1117.95 L1911.67 1117.91 L1911.9 1117.89 L1912.12 1117.98 L1912.35 1117.08 L1912.58 1117.29 L1912.81 1117.56 L1913.03 1117.66 L1913.26 1117.35 L1913.49 1117.34 L1913.72 1117.71 L1913.95 1117.45 L1914.17 1117.68 L1914.4 1118.01 L1914.63 1118.24 L1914.86 1118.27 L1915.08 1117.93 L1915.31 1117.29 L1915.54 1117.28 L1915.77 1116.94 L1916 1117.04 L1916.22 1117.31 L1916.45 1117.75 L1916.68 1117.77 L1916.91 1117.32 L1917.13 1117.58 L1917.36 1117.83 L1917.59 1117.99 L1917.82 1117.58 L1918.05 1117.81 L1918.27 1117.91 L1918.5 1117.89 L1918.73 1118.06 L1918.96 1118.03 L1919.18 1118.22 L1919.41 1118.03 L1919.64 1117.97 L1919.87 1118.66 L1920.1 1118.26 L1920.32 1118.42 L1920.55 1118.39 L1920.78 1118.76 L1921.01 1118.26 L1921.23 1118.04 L1921.46 1118.88 L1921.69 1119.08 L1921.92 1118.76 L1922.15 1118.77 L1922.37 1118.57 L1922.6 1119.12 L1922.83 1119.14 L1923.06 1119.93 L1923.28 1120 L1923.51 1120.84 L1923.74 1120.73 L1923.97 1120.19 L1924.2 1120 L1924.42 1119.74 L1924.65 1119.9 L1924.88 1119.55 L1925.11 1119.85 L1925.33 1120.08 L1925.56 1119.66 L1925.79 1119.31 L1926.02 1119.31 L1926.25 1119.82 L1926.47 1120.38 L1926.7 1119.6 L1926.93 1119.16 L1927.16 1119.8 L1927.38 1119.55 L1927.61 1118.98 L1927.84 1119.58 L1928.07 1119.55 L1928.3 1119.11 L1928.52 1119.24 L1928.75 1119.28 L1928.98 1118.98 L1929.21 1119.51 L1929.43 1119.13 L1929.66 1119.42 L1929.89 1119.38 L1930.12 1119.57 L1930.35 1119.71 L1930.57 1120.07 L1930.8 1119.62 L1931.03 1119.81 L1931.26 1119.46 L1931.48 1119.95 L1931.71 1119.92 L1931.94 1119.98 L1932.17 1120.39 L1932.4 1120.51 L1932.62 1120.36 L1932.85 1120.68 L1933.08 1120.76 L1933.31 1120.39 L1933.53 1120.68 L1933.76 1120.56 L1933.99 1121.22 L1934.22 1121.36 L1934.45 1121.57 L1934.67 1121.41 L1934.9 1121.44 L1935.13 1121.41 L1935.36 1121.51 L1935.58 1121.83 L1935.81 1121.46 L1936.04 1121.24 L1936.27 1121.28 L1936.5 1121.13 L1936.72 1120.9 L1936.95 1121.17 L1937.18 1121.75 L1937.41 1121.5 L1937.63 1121.45 L1937.86 1121.11 L1938.09 1120.81 L1938.32 1120.74 L1938.55 1120.42 L1938.77 1120.16 L1939 1119.94 L1939.23 1119.81 L1939.46 1119.29 L1939.68 1118.94 L1939.91 1119.07 L1940.14 1119.3 L1940.37 1119.43 L1940.6 1119.25 L1940.82 1119.48 L1941.05 1119.98 L1941.28 1119.82 L1941.51 1118.93 L1941.73 1119.61 L1941.96 1119.38 L1942.19 1119.45 L1942.42 1119.57 L1942.65 1119.72 L1942.87 1119.69 L1943.1 1119.89 L1943.33 1119.78 L1943.56 1120.26 L1943.78 1119.85 L1944.01 1120.08 L1944.24 1120.73 L1944.47 1121.21 L1944.7 1121.17 L1944.92 1121.04 L1945.15 1121.27 L1945.38 1121.3 L1945.61 1121.77 L1945.83 1121.93 L1946.06 1121.44 L1946.29 1121.22 L1946.52 1121.75 L1946.75 1121.97 L1946.97 1121.35 L1947.2 1121.77 L1947.43 1122.05 L1947.66 1122.1 L1947.88 1122.3 L1948.11 1122.26 L1948.34 1121.86 L1948.57 1122.44 L1948.79 1122.79 L1949.02 1122.51 L1949.25 1122.47 L1949.48 1122.76 L1949.71 1122.56 L1949.93 1122.59 L1950.16 1122.68 L1950.39 1122.72 L1950.62 1122.23 L1950.84 1122.08 L1951.07 1121.64 L1951.3 1121.65 L1951.53 1122.37 L1951.76 1122.23 L1951.98 1122.22 L1952.21 1122.11 L1952.44 1122.28 L1952.67 1122.1 L1952.89 1122.48 L1953.12 1122.69 L1953.35 1121.84 L1953.58 1121.73 L1953.81 1122.11 L1954.03 1122.14 L1954.26 1122.56 L1954.49 1122.45 L1954.72 1121.88 L1954.94 1121.74 L1955.17 1122.1 L1955.4 1122.06 L1955.63 1122.33 L1955.86 1122.64 L1956.08 1122.27 L1956.31 1122.32 L1956.54 1122.34 L1956.77 1122.65 L1956.99 1122.55 L1957.22 1122.89 L1957.45 1123.11 L1957.68 1123.45 L1957.91 1123.84 L1958.13 1123.62 L1958.36 1123.46 L1958.59 1123.48 L1958.82 1123.44 L1959.04 1123.67 L1959.27 1123.38 L1959.5 1123.57 L1959.73 1123.74 L1959.96 1123.34 L1960.18 1123.56 L1960.41 1123.35 L1960.64 1122.82 L1960.87 1122.88 L1961.09 1122.3 L1961.32 1122.04 L1961.55 1122.72 L1961.78 1123.15 L1962.01 1122.79 L1962.23 1122.96 L1962.46 1123.65 L1962.69 1123.82 L1962.92 1123.55 L1963.14 1123.31 L1963.37 1123.19 L1963.6 1123.42 L1963.83 1122.97 L1964.06 1122.91 L1964.28 1123.32 L1964.51 1123.48 L1964.74 1123.71 L1964.97 1123.51 L1965.19 1123.28 L1965.42 1123.33 L1965.65 1123.43 L1965.88 1123.73 L1966.11 1124 L1966.33 1123.86 L1966.56 1123.84 L1966.79 1124.61 L1967.02 1124.67 L1967.24 1125.1 L1967.47 1125.15 L1967.7 1125.58 L1967.93 1124.84 L1968.16 1125.09 L1968.38 1125.37 L1968.61 1125.55 L1968.84 1125.55 L1969.07 1125.13 L1969.29 1125.13 L1969.52 1125.14 L1969.75 1124.73 L1969.98 1124.7 L1970.21 1124.5 L1970.43 1124.49 L1970.66 1124.64 L1970.89 1124.66 L1971.12 1124.76 L1971.34 1125.06 L1971.57 1125.18 L1971.8 1125.34 L1972.03 1124.85 L1972.26 1124.16 L1972.48 1124.06 L1972.71 1124.51 L1972.94 1124.7 L1973.17 1125 L1973.39 1124.52 L1973.62 1123.95 L1973.85 1123.94 L1974.08 1123.8 L1974.31 1123.86 L1974.53 1124.35 L1974.76 1124.45 L1974.99 1125.09 L1975.22 1125.02 L1975.44 1124.28 L1975.67 1124.01 L1975.9 1123.4 L1976.13 1123.49 L1976.36 1123.16 L1976.58 1123.22 L1976.81 1122.77 L1977.04 1122.49 L1977.27 1122.72 L1977.49 1122.56 L1977.72 1122.86 L1977.95 1123.19 L1978.18 1122.97 L1978.41 1122.64 L1978.63 1122.89 L1978.86 1122.56 L1979.09 1122.19 L1979.32 1122.12 L1979.54 1122.04 L1979.77 1121.3 L1980 1120.5 L1980.23 1120.77 L1980.46 1120.89 L1980.68 1120.39 L1980.91 1120.18 L1981.14 1120.5 L1981.37 1120.19 L1981.59 1120.06 L1981.82 1120.01 L1982.05 1119.99 L1982.28 1119.51 L1982.51 1119.85 L1982.73 1119.5 L1982.96 1119.65 L1983.19 1120.27 L1983.42 1119.96 L1983.64 1120.54 L1983.87 1120.45 L1984.1 1120.25 L1984.33 1119.91 L1984.56 1119.63 L1984.78 1119.75 L1985.01 1119.82 L1985.24 1120 L1985.47 1120.11 L1985.69 1120.06 L1985.92 1119.67 L1986.15 1119.5 L1986.38 1119.74 L1986.6 1119.37 L1986.83 1119.89 L1987.06 1119.31 L1987.29 1119.02 L1987.52 1119.39 L1987.74 1118.97 L1987.97 1119.39 L1988.2 1119.62 L1988.43 1119.56 L1988.65 1119.71 L1988.88 1119.49 L1989.11 1119.46 L1989.34 1118.73 L1989.57 1118.92 L1989.79 1118.66 L1990.02 1119.05 L1990.25 1118.96 L1990.48 1118.97 L1990.7 1119.21 L1990.93 1119.42 L1991.16 1119.8 L1991.39 1119.57 L1991.62 1119.77 L1991.84 1119.53 L1992.07 1119.73 L1992.3 1119.54 L1992.53 1119.85 L1992.75 1120.42 L1992.98 1119.98 L1993.21 1119.35 L1993.44 1119.49 L1993.67 1119.51 L1993.89 1120.09 L1994.12 1119.85 L1994.35 1119.34 L1994.58 1119.16 L1994.8 1119.66 L1995.03 1119.95 L1995.26 1119.51 L1995.49 1119.97 L1995.72 1119.32 L1995.94 1119.33 L1996.17 1119.88 L1996.4 1119.56 L1996.63 1120.08 L1996.85 1119.96 L1997.08 1120.02 L1997.31 1120.34 L1997.54 1120.14 L1997.77 1119.56 L1997.99 1119.97 L1998.22 1120.08 L1998.45 1120.42 L1998.68 1120.6 L1998.9 1120.1 L1999.13 1120.45 L1999.36 1119.59 L1999.59 1119.15 L1999.82 1119.73 L2000.04 1119.76 L2000.27 1119.9 L2000.5 1119.89 L2000.73 1119.32 L2000.95 1118.39 L2001.18 1118.58 L2001.41 1118.72 L2001.64 1118.6 L2001.87 1118.41 L2002.09 1119.07 L2002.32 1119.18 L2002.55 1119.06 L2002.78 1119.05 L2003 1119.55 L2003.23 1119.54 L2003.46 1119.31 L2003.69 1119.51 L2003.92 1119.48 L2004.14 1119.14 L2004.37 1119.59 L2004.6 1119.71 L2004.83 1120.01 L2005.05 1119.8 L2005.28 1119.57 L2005.51 1119.28 L2005.74 1118.68 L2005.97 1118.67 L2006.19 1119.29 L2006.42 1119.46 L2006.65 1119.49 L2006.88 1119.26 L2007.1 1119.78 L2007.33 1119.82 L2007.56 1119.28 L2007.79 1119.78 L2008.02 1119.69 L2008.24 1120.02 L2008.47 1120.02 L2008.7 1119.97 L2008.93 1119.17 L2009.15 1119.45 L2009.38 1119.37 L2009.61 1119.67 L2009.84 1119.4 L2010.07 1119.76 L2010.29 1118.6 L2010.52 1118.87 L2010.75 1118.41 L2010.98 1118.48 L2011.2 1118.56 L2011.43 1118.64 L2011.66 1118.93 L2011.89 1118.64 L2012.12 1118.02 L2012.34 1118.3 L2012.57 1118.4 L2012.8 1118.12 L2013.03 1118.29 L2013.25 1118.09 L2013.48 1118.04 L2013.71 1118.31 L2013.94 1118.9 L2014.17 1118.71 L2014.39 1118.48 L2014.62 1119.11 L2014.85 1118.92 L2015.08 1118.85 L2015.3 1119.03 L2015.53 1119.38 L2015.76 1120.1 L2015.99 1120.16 L2016.22 1120.91 L2016.44 1120.49 L2016.67 1120.75 L2016.9 1120.93 L2017.13 1120.87 L2017.35 1120.92 L2017.58 1120.68 L2017.81 1120.32 L2018.04 1120.43 L2018.27 1120.3 L2018.49 1120.14 L2018.72 1120.28 L2018.95 1120.59 L2019.18 1120.35 L2019.4 1120.74 L2019.63 1121.29 L2019.86 1121.7 L2020.09 1121.95 L2020.32 1122.3 L2020.54 1122.55 L2020.77 1122.34 L2021 1122.66 L2021.23 1122.38 L2021.45 1122.39 L2021.68 1122.13 L2021.91 1122.31 L2022.14 1121.76 L2022.36 1121.53 L2022.59 1121.22 L2022.82 1121.41 L2023.05 1121.32 L2023.28 1121.48 L2023.5 1121.18 L2023.73 1121.25 L2023.96 1120.79 L2024.19 1120.53 L2024.41 1120.66 L2024.64 1120.95 L2024.87 1120.34 L2025.1 1120.9 L2025.33 1120.77 L2025.55 1120.43 L2025.78 1120.46 L2026.01 1120.44 L2026.24 1120.5 L2026.46 1120.58 L2026.69 1120.53 L2026.92 1120.42 L2027.15 1120.88 L2027.38 1121.15 L2027.6 1121.19 L2027.83 1120.78 L2028.06 1121.03 L2028.29 1120.66 L2028.51 1120.16 L2028.74 1119.88 L2028.97 1119.52 L2029.2 1119.22 L2029.43 1119.47 L2029.65 1118.96 L2029.88 1119.52 L2030.11 1119.67 L2030.34 1119.98 L2030.56 1119.85 L2030.79 1119.8 L2031.02 1120.39 L2031.25 1120.3 L2031.48 1120.55 L2031.7 1119.72 L2031.93 1119.88 L2032.16 1119.58 L2032.39 1119.27 L2032.61 1119.25 L2032.84 1119.02 L2033.07 1119.15 L2033.3 1118.99 L2033.53 1119.57 L2033.75 1119.13 L2033.98 1119.22 L2034.21 1119.32 L2034.44 1119.43 L2034.66 1119.84 L2034.89 1120 L2035.12 1119.75 L2035.35 1120.15 L2035.58 1119.51 L2035.8 1119.56 L2036.03 1119.87 L2036.26 1119.83 L2036.49 1120.34 L2036.71 1119.78 L2036.94 1119.34 L2037.17 1119.17 L2037.4 1119.24 L2037.63 1119.25 L2037.85 1119.87 L2038.08 1119.66 L2038.31 1119.22 L2038.54 1119.15 L2038.76 1118.49 L2038.99 1117.67 L2039.22 1117.42 L2039.45 1117.38 L2039.68 1117.51 L2039.9 1117.24 L2040.13 1117.2 L2040.36 1118.08 L2040.59 1118.42 L2040.81 1118.49 L2041.04 1118.58 L2041.27 1118.69 L2041.5 1118.41 L2041.73 1118.16 L2041.95 1117.94 L2042.18 1117.68 L2042.41 1117.52 L2042.64 1117.83 L2042.86 1118.18 L2043.09 1117.59 L2043.32 1117.73 L2043.55 1118.15 L2043.78 1118.5 L2044 1118.69 L2044.23 1118.7 L2044.46 1119.04 L2044.69 1119.06 L2044.91 1118.67 L2045.14 1119.25 L2045.37 1119.64 L2045.6 1119.23 L2045.83 1119.59 L2046.05 1119.11 L2046.28 1119.14 L2046.51 1118.9 L2046.74 1119.51 L2046.96 1118.85 L2047.19 1118.76 L2047.42 1118.87 L2047.65 1119.19 L2047.88 1118.95 L2048.1 1119.01 L2048.33 1119 L2048.56 1119.07 L2048.79 1119.43 L2049.01 1119.58 L2049.24 1119.31 L2049.47 1119.33 L2049.7 1119 L2049.93 1119.22 L2050.15 1118.87 L2050.38 1118.83 L2050.61 1118.22 L2050.84 1118.13 L2051.06 1118.64 L2051.29 1118.33 L2051.52 1118.28 L2051.75 1118.18 L2051.98 1118.45 L2052.2 1118.83 L2052.43 1118.45 L2052.66 1118.55 L2052.89 1119.15 L2053.11 1118.66 L2053.34 1118.79 L2053.57 1118.68 L2053.8 1118.52 L2054.03 1118.71 L2054.25 1118.88 L2054.48 1119.4 L2054.71 1119.29 L2054.94 1119.3 L2055.16 1119.32 L2055.39 1119.41 L2055.62 1119.59 L2055.85 1119.46 L2056.08 1119.02 L2056.3 1119.16 L2056.53 1118.68 L2056.76 1118.75 L2056.99 1118.68 L2057.21 1118.67 L2057.44 1118.49 L2057.67 1118.1 L2057.9 1118.66 L2058.12 1118.36 L2058.35 1117.78 L2058.58 1118.12 L2058.81 1117.77 L2059.04 1117.99 L2059.26 1117.87 L2059.49 1117.81 L2059.72 1117.77 L2059.95 1118.07 L2060.17 1117.53 L2060.4 1117.91 L2060.63 1117.49 L2060.86 1117.7 L2061.09 1117.08 L2061.31 1117.31 L2061.54 1117.57 L2061.77 1117.67 L2062 1118.11 L2062.22 1117.77 L2062.45 1117.84 L2062.68 1118.17 L2062.91 1118.44 L2063.14 1118.13 L2063.36 1118.43 L2063.59 1118.93 L2063.82 1118.63 L2064.05 1119.06 L2064.27 1119.06 L2064.5 1118.86 L2064.73 1118.03 L2064.96 1117.84 L2065.19 1118.16 L2065.41 1118.36 L2065.64 1118.54 L2065.87 1118.18 L2066.1 1118.31 L2066.32 1117.83 L2066.55 1117.37 L2066.78 1117.22 L2067.01 1117 L2067.24 1116.96 L2067.46 1116.97 L2067.69 1116.95 L2067.92 1116.83 L2068.15 1117.14 L2068.37 1116.98 L2068.6 1117.06 L2068.83 1116.5 L2069.06 1116.75 L2069.29 1117.07 L2069.51 1117.6 L2069.74 1117.02 L2069.97 1117.36 L2070.2 1117.43 L2070.42 1117.76 L2070.65 1117.97 L2070.88 1118.14 L2071.11 1118.59 L2071.34 1118.2 L2071.56 1117.95 L2071.79 1117.52 L2072.02 1117.37 L2072.25 1117.29 L2072.47 1117.61 L2072.7 1117.56 L2072.93 1117.51 L2073.16 1117.29 L2073.39 1117.63 L2073.61 1117.38 L2073.84 1117.47 L2074.07 1117.49 L2074.3 1117.44 L2074.52 1118.01 L2074.75 1118.13 L2074.98 1118.39 L2075.21 1118.52 L2075.44 1118.52 L2075.66 1118.98 L2075.89 1119.37 L2076.12 1119.35 L2076.35 1119.03 L2076.57 1119.41 L2076.8 1119.56 L2077.03 1119.74 L2077.26 1119.36 L2077.49 1119.18 L2077.71 1118.75 L2077.94 1118.85 L2078.17 1118.51 L2078.4 1118.51 L2078.62 1117.93 L2078.85 1117.89 L2079.08 1117.27 L2079.31 1117.09 L2079.54 1117.17 L2079.76 1117.38 L2079.99 1117.19 L2080.22 1117.01 L2080.45 1117.46 L2080.67 1117.7 L2080.9 1117.61 L2081.13 1117.58 L2081.36 1117.48 L2081.59 1117.03 L2081.81 1116.74 L2082.04 1117.09 L2082.27 1116.71 L2082.5 1116.79 L2082.72 1116.5 L2082.95 1116.47 L2083.18 1116.4 L2083.41 1116.4 L2083.64 1115.63 L2083.86 1115.37 L2084.09 1115.38 L2084.32 1115.27 L2084.55 1115.74 L2084.77 1115.98 L2085 1116.17 L2085.23 1116.25 L2085.46 1115.45 L2085.69 1115.43 L2085.91 1115.39 L2086.14 1115.5 L2086.37 1115.41 L2086.6 1115.23 L2086.82 1115.49 L2087.05 1115.89 L2087.28 1116.09 L2087.51 1116.81 L2087.74 1116.79 L2087.96 1116.6 L2088.19 1116.25 L2088.42 1116.11 L2088.65 1116.36 L2088.87 1116.89 L2089.1 1117.14 L2089.33 1117.23 L2089.56 1117.46 L2089.79 1117.31 L2090.01 1117.52 L2090.24 1117.99 L2090.47 1117.51 L2090.7 1117.79 L2090.92 1117.78 L2091.15 1117.62 L2091.38 1117.48 L2091.61 1116.74 L2091.84 1116.68 L2092.06 1116.88 L2092.29 1116.19 L2092.52 1116.68 L2092.75 1116.71 L2092.97 1116.56 L2093.2 1116.54 L2093.43 1116.94 L2093.66 1117.15 L2093.89 1116.49 L2094.11 1116.94 L2094.34 1117.23 L2094.57 1117.22 L2094.8 1117.75 L2095.02 1118.14 L2095.25 1118.3 L2095.48 1118.27 L2095.71 1118.87 L2095.93 1119.1 L2096.16 1119.35 L2096.39 1119.88 L2096.62 1119.62 L2096.85 1120.05 L2097.07 1120.24 L2097.3 1120.29 L2097.53 1120.17 L2097.76 1120.04 L2097.98 1120.32 L2098.21 1120.4 L2098.44 1120.77 L2098.67 1120.62 L2098.9 1120.81 L2099.12 1120.61 L2099.35 1119.96 L2099.58 1119.5 L2099.81 1119.07 L2100.03 1119.31 L2100.26 1119.6 L2100.49 1119.35 L2100.72 1118.76 L2100.95 1118.76 L2101.17 1118.48 L2101.4 1118.71 L2101.63 1118.68 L2101.86 1118.23 L2102.08 1118.17 L2102.31 1118.27 L2102.54 1117.93 L2102.77 1117.81 L2103 1117.72 L2103.22 1117.79 L2103.45 1117.33 L2103.68 1117.08 L2103.91 1116.95 L2104.13 1116.82 L2104.36 1117.25 L2104.59 1117.33 L2104.82 1117.27 L2105.05 1117.44 L2105.27 1116.73 L2105.5 1117.05 L2105.73 1116.61 L2105.96 1115.99 L2106.18 1116.58 L2106.41 1116.94 L2106.64 1116.84 L2106.87 1116.35 L2107.1 1116.79 L2107.32 1116.52 L2107.55 1116.38 L2107.78 1116.59 L2108.01 1116.05 L2108.23 1116.3 L2108.46 1116.16 L2108.69 1116.04 L2108.92 1116.05 L2109.15 1116.2 L2109.37 1116.47 L2109.6 1116.65 L2109.83 1116.46 L2110.06 1116.07 L2110.28 1116.33 L2110.51 1116.26 L2110.74 1116.29 L2110.97 1116.66 L2111.2 1116.47 L2111.42 1116.4 L2111.65 1116.82 L2111.88 1116.38 L2112.11 1116.48 L2112.33 1115.97 L2112.56 1116.3 L2112.79 1116.65 L2113.02 1116.17 L2113.25 1116.17 L2113.47 1116.55 L2113.7 1116.67 L2113.93 1116.76 L2114.16 1117.46 L2114.38 1117.27 L2114.61 1117.63 L2114.84 1117.45 L2115.07 1117.53 L2115.3 1117.09 L2115.52 1116.94 L2115.75 1117.07 L2115.98 1117.37 L2116.21 1116.66 L2116.43 1116.82 L2116.66 1116.86 L2116.89 1116.95 L2117.12 1117.11 L2117.35 1117.51 L2117.57 1117.48 L2117.8 1117.73 L2118.03 1117.76 L2118.26 1117.88 L2118.48 1117.9 L2118.71 1118.09 L2118.94 1117.93 L2119.17 1117.62 L2119.4 1117.62 L2119.62 1117.8 L2119.85 1117.97 L2120.08 1117.96 L2120.31 1118.28 L2120.53 1117.93 L2120.76 1118.27 L2120.99 1118.5 L2121.22 1118.96 L2121.45 1118.56 L2121.67 1118.81 L2121.9 1118.61 L2122.13 1118.9 L2122.36 1119.31 L2122.58 1119.18 L2122.81 1118.85 L2123.04 1118.39 L2123.27 1118.41 L2123.5 1118.02 L2123.72 1118.44 L2123.95 1119.14 L2124.18 1119.51 L2124.41 1119.32 L2124.63 1118.64 L2124.86 1118.5 L2125.09 1118.07 L2125.32 1117.78 L2125.55 1117.83 L2125.77 1117.43 L2126 1117.49 L2126.23 1117.63 L2126.46 1117.66 L2126.68 1117.89 L2126.91 1118.1 L2127.14 1118.17 L2127.37 1118.31 L2127.6 1118.51 L2127.82 1118.19 L2128.05 1117.97 L2128.28 1117.72 L2128.51 1117.93 L2128.73 1118.18 L2128.96 1117.8 L2129.19 1117.84 L2129.42 1117.68 L2129.65 1118.09 L2129.87 1118.4 L2130.1 1118.52 L2130.33 1117.93 L2130.56 1118.22 L2130.78 1118.31 L2131.01 1118.37 L2131.24 1118.78 L2131.47 1118.89 L2131.69 1118.98 L2131.92 1118.73 L2132.15 1118.86 L2132.38 1119.25 L2132.61 1118.71 L2132.83 1119.44 L2133.06 1119.57 L2133.29 1118.99 L2133.52 1118.91 L2133.74 1119.08 L2133.97 1118.52 L2134.2 1118.51 L2134.43 1118.51 L2134.66 1119.06 L2134.88 1119.05 L2135.11 1118.89 L2135.34 1118.44 L2135.57 1118.36 L2135.79 1118.73 L2136.02 1118.34 L2136.25 1118.73 L2136.48 1118.79 L2136.71 1118.82 L2136.93 1118.97 L2137.16 1118.94 L2137.39 1118.9 L2137.62 1118.86 L2137.84 1119.54 L2138.07 1119.72 L2138.3 1119.73 L2138.53 1119.85 L2138.76 1119.71 L2138.98 1119.7 L2139.21 1119.43 L2139.44 1119.22 L2139.67 1119.12 L2139.89 1118.79 L2140.12 1118.59 L2140.35 1118.87 L2140.58 1118.8 L2140.81 1118.62 L2141.03 1118.68 L2141.26 1118.79 L2141.49 1118.94 L2141.72 1119.34 L2141.94 1119.63 L2142.17 1119.02 L2142.4 1119.17 L2142.63 1119.04 L2142.86 1119.35 L2143.08 1118.98 L2143.31 1119.01 L2143.54 1118.43 L2143.77 1118.25 L2143.99 1118.19 L2144.22 1118.19 L2144.45 1118.7 L2144.68 1118.65 L2144.91 1118.2 L2145.13 1118.58 L2145.36 1118.38 L2145.59 1118.51 L2145.82 1118.53 L2146.04 1119.07 L2146.27 1119.09 L2146.5 1118.88 L2146.73 1119.12 L2146.96 1118.87 L2147.18 1118.93 L2147.41 1118.96 L2147.64 1119.32 L2147.87 1119.52 L2148.09 1119.32 L2148.32 1118.43 L2148.55 1118.9 L2148.78 1119.25 L2149.01 1119.64 L2149.23 1118.98 L2149.46 1119.29 L2149.69 1119.42 L2149.92 1119.58 L2150.14 1119.08 L2150.37 1118.33 L2150.6 1118.54 L2150.83 1118.47 L2151.06 1118.81 L2151.28 1119.16 L2151.51 1119.26 L2151.74 1119.5 L2151.97 1119.52 L2152.19 1119.54 L2152.42 1119.65 L2152.65 1119.45 L2152.88 1119.42 L2153.11 1119.53 L2153.33 1119.66 L2153.56 1119.11 L2153.79 1119.31 L2154.02 1119.68 L2154.24 1119.9 L2154.47 1120.28 L2154.7 1119.77 L2154.93 1119.17 L2155.16 1119.19 L2155.38 1118.75 L2155.61 1118.91 L2155.84 1118.93 L2156.07 1119.05 L2156.29 1119.53 L2156.52 1119.56 L2156.75 1119.32 L2156.98 1119.36 L2157.21 1119.8 L2157.43 1120.08 L2157.66 1119.72 L2157.89 1118.61 L2158.12 1118.56 L2158.34 1118.43 L2158.57 1118.32 L2158.8 1117.83 L2159.03 1117.39 L2159.26 1116.79 L2159.48 1116.52 L2159.71 1116.94 L2159.94 1116.98 L2160.17 1116.57 L2160.39 1116.88 L2160.62 1115.85 L2160.85 1115.66 L2161.08 1114.98 L2161.31 1114.78 L2161.53 1114.81 L2161.76 1114.53 L2161.99 1114.53 L2162.22 1114.57 L2162.44 1114.44 L2162.67 1114.36 L2162.9 1114.32 L2163.13 1113.76 L2163.36 1113.3 L2163.58 1113.22 L2163.81 1113.42 L2164.04 1113.28 L2164.27 1113.65 L2164.49 1113.75 L2164.72 1113.79 L2164.95 1113.12 L2165.18 1112.86 L2165.41 1112.33 L2165.63 1112.4 L2165.86 1112.28 L2166.09 1112.36 L2166.32 1112.6 L2166.54 1113 L2166.77 1113.34 L2167 1113.06 L2167.23 1113.36 L2167.45 1113.14 L2167.68 1113.57 L2167.91 1113.71 L2168.14 1113.67 L2168.37 1113.59 L2168.59 1113.45 L2168.82 1112.8 L2169.05 1112.25 L2169.28 1112.79 L2169.5 1112.91 L2169.73 1112.84 L2169.96 1112.54 L2170.19 1112.4 L2170.42 1112.34 L2170.64 1112.66 L2170.87 1112.73 L2171.1 1112.6 L2171.33 1112.65 L2171.55 1112.7 L2171.78 1112.9 L2172.01 1112.61 L2172.24 1112.45 L2172.47 1112.73 L2172.69 1112.68 L2172.92 1112.52 L2173.15 1112.27 L2173.38 1112.03 L2173.6 1111.73 L2173.83 1111.72 L2174.06 1112.11 L2174.29 1112.34 L2174.52 1112.12 L2174.74 1112.37 L2174.97 1112.83 L2175.2 1112.59 L2175.43 1113.25 L2175.65 1112.96 L2175.88 1112.94 L2176.11 1113.33 L2176.34 1113.65 L2176.57 1113.53 L2176.79 1114.21 L2177.02 1114.02 L2177.25 1114.11 L2177.48 1114.73 L2177.7 1114.76 L2177.93 1114.91 L2178.16 1114.87 L2178.39 1114.53 L2178.62 1114.68 L2178.84 1114.68 L2179.07 1114.43 L2179.3 1114.88 L2179.53 1114.79 L2179.75 1114.64 L2179.98 1114.2 L2180.21 1113.81 L2180.44 1114.08 L2180.67 1114.36 L2180.89 1114.9 L2181.12 1114.15 L2181.35 1114.17 L2181.58 1114.17 L2181.8 1114.51 L2182.03 1114.07 L2182.26 1114.09 L2182.49 1113.31 L2182.72 1113.21 L2182.94 1113.06 L2183.17 1112.88 L2183.4 1113.01 L2183.63 1113.28 L2183.85 1113.54 L2184.08 1113.68 L2184.31 1114.02 L2184.54 1114.21 L2184.77 1114.34 L2184.99 1114.42 L2185.22 1114.39 L2185.45 1114.69 L2185.68 1115.36 L2185.9 1115.49 L2186.13 1115.07 L2186.36 1114.93 L2186.59 1115 L2186.82 1115.08 L2187.04 1115.34 L2187.27 1115.58 L2187.5 1115.97 L2187.73 1116.04 L2187.95 1116.09 L2188.18 1116.03 L2188.41 1115.45 L2188.64 1115.08 L2188.87 1114.84 L2189.09 1114.99 L2189.32 1114.92 L2189.55 1115.08 L2189.78 1114.89 L2190 1115.02 L2190.23 1114.84 L2190.46 1114.83 L2190.69 1114.76 L2190.92 1114.8 L2191.14 1114.99 L2191.37 1115.3 L2191.6 1115.23 L2191.83 1115.76 L2192.05 1115.59 L2192.28 1115.28 L2192.51 1115.27 L2192.74 1115.37 L2192.97 1115.09 L2193.19 1115.58 L2193.42 1114.85 L2193.65 1115.35 L2193.88 1115.63 L2194.1 1115.37 L2194.33 1115.6 L2194.56 1115.67 L2194.79 1115.15 L2195.02 1115.22 L2195.24 1114.81 L2195.47 1114.88 L2195.7 1114.83 L2195.93 1114.94 L2196.15 1114.46 L2196.38 1114.49 L2196.61 1114.49 L2196.84 1114.8 L2197.07 1114.05 L2197.29 1114.49 L2197.52 1114.19 L2197.75 1114.31 L2197.98 1114.21 L2198.2 1114.43 L2198.43 1114.7 L2198.66 1114.87 L2198.89 1114.89 L2199.12 1114.75 L2199.34 1114.54 L2199.57 1114.88 L2199.8 1114.69 L2200.03 1114.65 L2200.25 1114.52 L2200.48 1114.29 L2200.71 1114.32 L2200.94 1114.65 L2201.17 1115.23 L2201.39 1115.09 L2201.62 1115.11 L2201.85 1115.5 L2202.08 1115.73 L2202.3 1115.99 L2202.53 1115.74 L2202.76 1115.59 L2202.99 1115.74 L2203.21 1116.03 L2203.44 1116.64 L2203.67 1116.5 L2203.9 1116.02 L2204.13 1116.42 L2204.35 1115.74 L2204.58 1116.32 L2204.81 1116.64 L2205.04 1116.8 L2205.26 1116.88 L2205.49 1116.77 L2205.72 1116.83 L2205.95 1116.39 L2206.18 1117.08 L2206.4 1117.15 L2206.63 1116.8 L2206.86 1116.69 L2207.09 1116.52 L2207.31 1116.63 L2207.54 1116.82 L2207.77 1116.26 L2208 1116.75 L2208.23 1117.15 L2208.45 1117.18 L2208.68 1117.13 L2208.91 1117.31 L2209.14 1116.89 L2209.36 1116.62 L2209.59 1116.82 L2209.82 1116.75 L2210.05 1116.53 L2210.28 1116.05 L2210.5 1116.26 L2210.73 1116.42 L2210.96 1116.02 L2211.19 1115.89 L2211.41 1116.11 L2211.64 1115.99 L2211.87 1115.75 L2212.1 1115.42 L2212.33 1115.09 L2212.55 1115.19 L2212.78 1115.44 L2213.01 1115.64 L2213.24 1115.87 L2213.46 1115.76 L2213.69 1114.92 L2213.92 1114.15 L2214.15 1114.46 L2214.38 1114.54 L2214.6 1114.39 L2214.83 1114.77 L2215.06 1114.29 L2215.29 1114.54 L2215.51 1114.52 L2215.74 1114.45 L2215.97 1114.53 L2216.2 1114.54 L2216.43 1114.04 L2216.65 1114.1 L2216.88 1113.83 L2217.11 1113.8 L2217.34 1114.25 L2217.56 1114.29 L2217.79 1114.19 L2218.02 1113.99 L2218.25 1114.33 L2218.48 1114.11 L2218.7 1114.49 L2218.93 1114.44 L2219.16 1114.26 L2219.39 1114.27 L2219.61 1114.32 L2219.84 1114.87 L2220.07 1114.52 L2220.3 1114.47 L2220.53 1114.69 L2220.75 1114.06 L2220.98 1114.33 L2221.21 1113.9 L2221.44 1113.56 L2221.66 1113.56 L2221.89 1113.45 L2222.12 1113.39 L2222.35 1113.57 L2222.58 1113.09 L2222.8 1112.92 L2223.03 1113.1 L2223.26 1113.06 L2223.49 1113.31 L2223.71 1113.47 L2223.94 1113.39 L2224.17 1113.54 L2224.4 1113.67 L2224.63 1113.69 L2224.85 1113.79 L2225.08 1113.92 L2225.31 1113.33 L2225.54 1112.9 L2225.76 1112.86 L2225.99 1112.55 L2226.22 1112.72 L2226.45 1112.49 L2226.68 1112.62 L2226.9 1112.61 L2227.13 1112.82 L2227.36 1112.35 L2227.59 1112.43 L2227.81 1112.18 L2228.04 1111.86 L2228.27 1111.82 L2228.5 1111.43 L2228.73 1111.41 L2228.95 1111.63 L2229.18 1112.07 L2229.41 1112.39 L2229.64 1112.48 L2229.86 1112.72 L2230.09 1112.58 L2230.32 1112.95 L2230.55 1112.96 L2230.78 1112.83 L2231 1113.04 L2231.23 1113.1 L2231.46 1113.27 L2231.69 1113.34 L2231.91 1113.34 L2232.14 1113.8 L2232.37 1114.22 L2232.6 1113.52 L2232.83 1113.51 L2233.05 1113.48 L2233.28 1113.27 L2233.51 1113.25 L2233.74 1113.3 L2233.96 1113.09 L2234.19 1113.33 L2234.42 1113.46 L2234.65 1113.47 L2234.88 1113.44 L2235.1 1113.18 L2235.33 1113.27 L2235.56 1113.63 L2235.79 1113.73 L2236.01 1114.34 L2236.24 1114.34 L2236.47 1114.08 L2236.7 1113.77 L2236.93 1113.46 L2237.15 1113.92 L2237.38 1113.66 L2237.61 1113.85 L2237.84 1114.06 L2238.06 1114.19 L2238.29 1114.15 L2238.52 1114.36 L2238.75 1114.37 L2238.98 1113.95 L2239.2 1114.41 L2239.43 1114.53 L2239.66 1114.14 L2239.89 1114.12 L2240.11 1113.59 L2240.34 1113.89 L2240.57 1114.21 L2240.8 1114.06 L2241.02 1114.51 L2241.25 1114.39 L2241.48 1114.52 L2241.71 1113.94 L2241.94 1113.92 L2242.16 1113.3 L2242.39 1113.5 L2242.62 1114.05 L2242.85 1113.74 L2243.07 1113.56 L2243.3 1112.85 L2243.53 1112.59 L2243.76 1112.6 L2243.99 1112.5 L2244.21 1112.9 L2244.44 1113.13 L2244.67 1112.31 L2244.9 1112.63 L2245.12 1112.88 L2245.35 1112.2 L2245.58 1112.23 L2245.81 1112.12 L2246.04 1111.77 L2246.26 1111.72 L2246.49 1111.55 L2246.72 1111.59 L2246.95 1111.72 L2247.17 1112.24 L2247.4 1112.19 L2247.63 1112.61 L2247.86 1112.64 L2248.09 1112.15 L2248.31 1111.7 L2248.54 1111.3 L2248.77 1111.03 L2249 1111.22 L2249.22 1111.02 L2249.45 1111.44 L2249.68 1111.37 L2249.91 1111.29 L2250.14 1110.88 L2250.36 1110.98 L2250.59 1111.28 L2250.82 1111.31 L2251.05 1110.96 L2251.27 1111.14 L2251.5 1111.37 L2251.73 1111.95 L2251.96 1112.13 L2252.19 1112.18 L2252.41 1112.2 L2252.64 1112.37 L2252.87 1112.15 L2253.1 1112.45 L2253.32 1112.89 L2253.55 1112.82 L2253.78 1113.03 L2254.01 1113.05 L2254.24 1113.14 L2254.46 1113.41 L2254.69 1113.55 L2254.92 1113.46 L2255.15 1113.82 L2255.37 1113.5 L2255.6 1113.39 L2255.83 1113.82 L2256.06 1113.96 L2256.29 1113.84 L2256.51 1114.2 L2256.74 1114.06 L2256.97 1113.54 L2257.2 1113.86 L2257.42 1113.76 L2257.65 1113.9 L2257.88 1113.94 L2258.11 1113.13 L2258.34 1112.85 L2258.56 1112.77 L2258.79 1112.71 L2259.02 1112.96 L2259.25 1112.81 L2259.47 1113.07 L2259.7 1112.93 L2259.93 1113.09 L2260.16 1112.19 L2260.39 1112.29 L2260.61 1112.14 L2260.84 1112.2 L2261.07 1112.14 L2261.3 1111.28 L2261.52 1110.85 L2261.75 1110.57 L2261.98 1110.53 L2262.21 1110.46 L2262.44 1109.79 L2262.66 1109.81 L2262.89 1110.42 L2263.12 1110.15 L2263.35 1110.44 L2263.57 1110.33 L2263.8 1110.27 L2264.03 1110.34 L2264.26 1110.66 L2264.49 1110.69 L2264.71 1110.9 L2264.94 1111.04 L2265.17 1110.83 L2265.4 1110.84 L2265.62 1111.06 L2265.85 1111.39 L2266.08 1111.12 L2266.31 1111.22 L2266.54 1111.08 L2266.76 1111.2 L2266.99 1111.25 L2267.22 1111.75 L2267.45 1111.51 L2267.67 1111.3 L2267.9 1111.16 L2268.13 1111.06 L2268.36 1111.59 L2268.59 1111.6 L2268.81 1111.56 L2269.04 1111.33 L2269.27 1111.08 L2269.5 1111.13 L2269.72 1111.1 L2269.95 1110.76 L2270.18 1110.77 L2270.41 1110.87 L2270.64 1110.81 L2270.86 1110.43 L2271.09 1110.4 L2271.32 1110.75 L2271.55 1110.56 L2271.77 1110.9 L2272 1110.96 L2272.23 1111.39 L2272.46 1111.29 L2272.69 1111.21 L2272.91 1111.21 L2273.14 1111.13 L2273.37 1111.34 L2273.6 1111.36 L2273.82 1111.07 L2274.05 1111.23 L2274.28 1111.52 L2274.51 1111.16 L2274.74 1110.87 L2274.96 1110.67 L2275.19 1110.69 L2275.42 1110.24 L2275.65 1110.47 L2275.87 1110.68 L2276.1 1110.2 L2276.33 1110.28 L2276.56 1109.86 L2276.78 1109.78 L2277.01 1109.32 L2277.24 1109.67 L2277.47 1109.47 L2277.7 1109.3 L2277.92 1109.36 L2278.15 1109.04 L2278.38 1109.05 L2278.61 1109.63 L2278.83 1109.69 L2279.06 1109.99 L2279.29 1109.75 L2279.52 1109.72 L2279.75 1110.02 L2279.97 1109.9 L2280.2 1109.75 L2280.43 1109.92 L2280.66 1109.68 L2280.88 1109.84 L2281.11 1109.37 L2281.34 1109.54 L2281.57 1110.09 L2281.8 1109.99 L2282.02 1109.7 L2282.25 1109.28 L2282.48 1108.35 L2282.71 1108.36 L2282.93 1108.27 L2283.16 1108.42 L2283.39 1107.48 L2283.62 1107.4 L2283.85 1107.26 L2284.07 1107.28 L2284.3 1107.38 L2284.53 1106.94 L2284.76 1106.85 L2284.98 1106.5 L2285.21 1106.34 L2285.44 1105.89 L2285.67 1105.97 L2285.9 1105.85 L2286.12 1105.69 L2286.35 1105.5 L2286.58 1105.15 L2286.81 1105.18 L2287.03 1105.51 L2287.26 1105.52 L2287.49 1105.43 L2287.72 1105.31 L2287.95 1104.83 L2288.17 1104.65 L2288.4 1104.73 L2288.63 1105.31 L2288.86 1105.34 L2289.08 1105.59 L2289.31 1106.03 L2289.54 1106.39 L2289.77 1106.27 L2290 1106.31 L2290.22 1106.56 L2290.45 1106.53 L2290.68 1106.46 L2290.91 1106.26 L2291.13 1106.41 L2291.36 1106.32 L2291.59 1106.49 L2291.82 1106.27 L2292.05 1106.4 L2292.27 1106.93 L2292.5 1106.98 L2292.73 1106.64 L2292.96 1106.68 L2293.18 1106.73 L2293.41 1106.6 L2293.64 1106.16 L2293.87 1105.9 L2294.1 1105.85 L2294.32 1106.19 L2294.55 1105.98 L2294.78 1105.77 L2295.01 1105.38 L2295.23 1105.67 L2295.46 1105.27 L2295.69 1104.77 L2295.92 1104.6 L2296.15 1104.56 L2296.37 1104.87 L2296.6 1104.67 L2296.83 1104.22 L2297.06 1104.56 L2297.28 1104.67 L2297.51 1104.39 L2297.74 1104.32 L2297.97 1104.35 L2298.2 1104.69 L2298.42 1104.74 L2298.65 1104.93 L2298.88 1104.87 L2299.11 1105.15 L2299.33 1104.99 L2299.56 1104.96 L2299.79 1105.36 L2300.02 1104.99 L2300.25 1104.53 L2300.47 1104.98 L2300.7 1105.18 L2300.93 1105.24 L2301.16 1105.42 L2301.38 1105.5 L2301.61 1105.19 L2301.84 1104.73 L2302.07 1104.94 L2302.3 1104.26 L2302.52 1104.21 L2302.75 1104.11 L2302.98 1104.51 L2303.21 1104.48 L2303.43 1104.51 L2303.66 1104.07 L2303.89 1104.22 L2304.12 1104.17 L2304.35 1104.45 L2304.57 1104.3 L2304.8 1104.46 L2305.03 1104.98 L2305.26 1105.1 L2305.48 1105.25 L2305.71 1105.33 L2305.94 1104.74 L2306.17 1104.9 L2306.4 1105.32 L2306.62 1105.24 L2306.85 1105.37 L2307.08 1105.51 L2307.31 1105.12 L2307.53 1104.77 L2307.76 1104.45 L2307.99 1104.76 L2308.22 1104.29 L2308.45 1104.27 L2308.67 1104.2 L2308.9 1104.23 L2309.13 1104.6 L2309.36 1105.19 L2309.58 1105.09 L2309.81 1105.38 L2310.04 1105.13 L2310.27 1105.21 L2310.5 1105.18 L2310.72 1105 L2310.95 1104.57 L2311.18 1104.37 L2311.41 1104 L2311.63 1103.59 L2311.86 1104.09 L2312.09 1104.3 L2312.32 1104.38 L2312.54 1104.43 L2312.77 1104.4 L2313 1104.07 L2313.23 1104.44 L2313.46 1104.42 L2313.68 1104.59 L2313.91 1104.47 L2314.14 1104.16 L2314.37 1103.8 L2314.59 1103.69 L2314.82 1103.59 L2315.05 1103.34 L2315.28 1103.18 L2315.51 1103.2 L2315.73 1103.26 L2315.96 1103.07 L2316.19 1103.26 L2316.42 1103.3 L2316.64 1102.58 L2316.87 1102.76 L2317.1 1102.81 L2317.33 1103.2 L2317.56 1103.47 L2317.78 1102.86 L2318.01 1102.71 L2318.24 1102.8 L2318.47 1102.95 L2318.69 1103.11 L2318.92 1103.2 L2319.15 1102.98 L2319.38 1102.56 L2319.61 1102.78 L2319.83 1102.68 L2320.06 1102.87 L2320.29 1102.78 L2320.52 1102.71 L2320.74 1103.16 L2320.97 1103.23 L2321.2 1103.3 L2321.43 1102.86 L2321.66 1102.43 L2321.88 1102.7 L2322.11 1102.35 L2322.34 1101.99 L2322.57 1101.87 L2322.79 1101.77 L2323.02 1101.46 L2323.25 1101.7 L2323.48 1102.1 L2323.71 1102.34 L2323.93 1102.25 L2324.16 1102.83 L2324.39 1102.82 L2324.62 1102.91 L2324.84 1102.97 L2325.07 1103.24 L2325.3 1103.17 L2325.53 1103.43 L2325.76 1103.48 L2325.98 1103.7 L2326.21 1103.66 L2326.44 1103.76 L2326.67 1103.94 L2326.89 1103.98 L2327.12 1103.65 L2327.35 1103.91 L2327.58 1103.55 L2327.81 1103.18 L2328.03 1103.3 L2328.26 1103.59 L2328.49 1103.81 L2328.72 1103.9 L2328.94 1104.27 L2329.17 1103.81 L2329.4 1103.69 L2329.63 1104.19 L2329.86 1104.32 L2330.08 1104.51 L2330.31 1104.64 L2330.54 1104.87 L2330.77 1104.7 L2330.99 1104.72 L2331.22 1104.98 L2331.45 1105.12 L2331.68 1105.46 L2331.91 1105.54 L2332.13 1105.66 L2332.36 1105.48 L2332.59 1105.57 L2332.82 1105.21 L2333.04 1105.24 L2333.27 1105.22 L2333.5 1104.48 L2333.73 1104.52 L2333.96 1104.41 L2334.18 1104.26 L2334.41 1104.19 L2334.64 1104.27 L2334.87 1104.35 L2335.09 1104.12 L2335.32 1103.99 L2335.55 1104.23 L2335.78 1104.24 L2336.01 1103.93 L2336.23 1104.02 L2336.46 1104.17 L2336.69 1103.96 L2336.92 1104.05 L2337.14 1103.88 L2337.37 1104.18 L2337.6 1104.24 L2337.83 1104.37 L2338.06 1104.44 L2338.28 1104.48 L2338.51 1104.12 L2338.74 1104.01 L2338.97 1104.07 L2339.19 1103.58 L2339.42 1103.17 L2339.65 1103.34 L2339.88 1103.31 L2340.11 1103.16 L2340.33 1103.43 L2340.56 1102.91 L2340.79 1102.7 L2341.02 1102.68 L2341.24 1103.12 L2341.47 1102.98 L2341.7 1102.95 L2341.93 1102.7 L2342.16 1102.93 L2342.38 1103.03 L2342.61 1103.37 L2342.84 1103.39 L2343.07 1103.57 L2343.29 1103.49 L2343.52 1103.63 L2343.75 1103.16 L2343.98 1103.3 L2344.21 1103.18 L2344.43 1102.92 L2344.66 1102.71 L2344.89 1102.76 L2345.12 1102.94 L2345.34 1102.8 L2345.57 1102.84 L2345.8 1102.84 L2346.03 1102.88 L2346.26 1102.93 L2346.48 1102.71 L2346.71 1102.36 L2346.94 1102.45 L2347.17 1102.1 L2347.39 1102.31 L2347.62 1102.55 L2347.85 1102.46 L2348.08 1102.48 L2348.31 1102.44 L2348.53 1102.28 L2348.76 1102.39 L2348.99 1101.97 L2349.22 1102.43 L2349.44 1101.84 L2349.67 1102.03 L2349.9 1102.31 L2350.13 1102.24 L2350.35 1102.55 L2350.58 1102.96 L2350.81 1103 L2351.04 1102.97 L2351.27 1102.78 L2351.49 1103.15 L2351.72 1103.52 L2351.95 1103.46 L2352.18 1103.85 L2352.4 1104.4 L2352.63 1104.27 L2352.86 1104.19 L2353.09 1104.55 L2353.32 1104.25 L2353.54 1104.17 L2353.77 1104.37 L2354 1104.83 L2354.23 1104.88 L2354.45 1104.64 L2354.68 1104.38 L2354.91 1104.06 L2355.14 1103.9 L2355.37 1104.13 L2355.59 1103.84 L2355.82 1103.65 L2356.05 1103.54 L2356.28 1103.48 L2356.5 1103.9 L2356.73 1103.64 L2356.96 1103.52 L2357.19 1103.63 L2357.42 1103.09 L2357.64 1103.58 L2357.87 1103.51 L2358.1 1103.7 L2358.33 1103.81 L2358.55 1103.88 L2358.78 1104.01 L2359.01 1103.85 L2359.24 1103.89 L2359.47 1103.53 L2359.69 1103.55 L2359.92 1104.01 L2360.15 1103.8 L2360.38 1103.41 L2360.6 1103.45 L2360.83 1103.43 L2361.06 1103.89 L2361.29 1104.11 L2361.52 1104.29 L2361.74 1104.48 L2361.97 1104.19 L2362.2 1104.51 L2362.43 1104.68 L2362.65 1105.04 L2362.88 1105.16 L2363.11 1105.3 L2363.34 1105.02 L2363.57 1104.93 L2363.79 1104.96 L2364.02 1104.92 L2364.25 1104.95 L2364.48 1105.03 L2364.7 1105.18 L2364.93 1105.49 L2365.16 1105.55 L2365.39 1105.6 L2365.62 1105.9 L2365.84 1106.08 L2366.07 1105.69 L2366.3 1105.85 L2366.53 1106 L2366.75 1105.68 L2366.98 1105.82 L2367.21 1105.69 L2367.44 1105.71 L2367.67 1105.74 L2367.89 1105.82 L2368.12 1106.25 L2368.35 1105.81 L2368.58 1105.99 L2368.8 1105.58 L2369.03 1105.46 L2369.26 1105.83 L2369.49 1105.94 L2369.72 1106.12 L2369.94 1106.33 L2370.17 1106.53 L2370.4 1105.97 L2370.63 1104.73 L2370.85 1104.48 L2371.08 1104.88 L2371.31 1104.58 L2371.54 1104.72 L2371.77 1105.02 L2371.99 1105.36 L2372.22 1105.39 L2372.45 1105.88 L2372.68 1106.41 L2372.9 1106.94 L2373.13 1107.26 L2373.36 1107.11 L2373.59 1106.82 L2373.82 1107.23 L2374.04 1107.41 L2374.27 1107.43 L2374.5 1107.32 L2374.73 1107.23 L2374.95 1106.74 L2375.18 1107.05 L2375.41 1106.96 L2375.64 1107.14 L2375.87 1106.83 L2376.09 1106.87 L2376.32 1106.78 L2376.55 1106.93 L2376.78 1107.04 L2377 1106.96 L2377.23 1106.95 L2377.46 1107.08 L2377.69 1106.76 L2377.92 1106.78 L2378.14 1106.64 L2378.37 1106.36 L2378.6 1105.61 L2378.83 1105.6 L2379.05 1105.8 L2379.28 1105.74 L2379.51 1105.52 L2379.74 1105.31 L2379.97 1105.02 L2380.19 1105.19 L2380.42 1105.37 L2380.65 1105.47 L2380.88 1105.1 L2381.1 1104.91 L2381.33 1105.22 L2381.56 1104.85 L2381.79 1104.46 L2382.02 1104.32 L2382.24 1104.73 L2382.47 1104.38 L2382.7 1103.81 L2382.93 1103.31 L2383.15 1102.87 L2383.38 1103.23 L2383.61 1102.97 L2383.84 1103.23 L2384.07 1103.68 L2384.29 1103.5 L2384.52 1103.41 L2384.75 1103.33 L2384.98 1103.63 L2385.2 1103.66 L2385.43 1103.82 L2385.66 1103.9 L2385.89 1103.61 L2386.11 1102.89 L2386.34 1102.73 L2386.57 1102.76 L2386.8 1102.87 L2387.03 1103.21 L2387.25 1103.25 L2387.48 1103.39 L2387.71 1103.06 L2387.94 1103.31 L2388.16 1102.69 L2388.39 1102.75 L2388.62 1103.01 L2388.85 1102.58 L2389.08 1102.66 L2389.3 1102.32 L2389.53 1102.4 L2389.76 1102.39 L2389.99 1102.7 L2390.21 1102.88 L2390.44 1102.91 L2390.67 1103.2 L2390.9 1103.41 L2391.13 1103.67 L2391.35 1103.97 L2391.58 1103.88 L2391.81 1103.17 L2392.04 1103.05 L2392.26 1102.62 L2392.49 1102.86 L2392.72 1103.14 L2392.95 1103.3 L2393.18 1103.74 L2393.4 1103.73 L2393.63 1103.44 L2393.86 1103.03 L2394.09 1103.31 L2394.31 1103.2 L2394.54 1103.5 L2394.77 1103.39 L2395 1103.45 L2395.23 1102.83 L2395.45 1103.03 L2395.68 1103.47 L2395.91 1103.43 L2396.14 1103.88 L2396.36 1104.02 L2396.59 1104.15 L2396.82 1104.04 L2397.05 1104.33 L2397.28 1104.02 L2397.5 1104.14 L2397.73 1103.65 L2397.96 1103.79 L2398.19 1103.73 L2398.41 1103.77 L2398.64 1104.03 L2398.87 1104.33 L2399.1 1104.43 L2399.33 1104.29 L2399.55 1103.7 L2399.78 1103.93 L2400.01 1104.07 L2400.24 1103.67 L2400.46 1103.42 L2400.69 1103.72 L2400.92 1103.5 L2401.15 1103.69 L2401.38 1103.65 L2401.6 1104.2 L2401.83 1104.19 L2402.06 1104.58 L2402.29 1104.77 L2402.51 1104.54 L2402.74 1104.39 L2402.97 1104.19 L2403.2 1104.77 L2403.43 1104.78 L2403.65 1104.74 L2403.88 1105.19 L2404.11 1105.51 L2404.34 1105.33 L2404.56 1105.47 L2404.79 1105.34 L2405.02 1105.23 L2405.25 1105.42 L2405.48 1105.87 L2405.7 1105.71 L2405.93 1105.59 L2406.16 1105.85 L2406.39 1106.43 L2406.61 1106.37 L2406.84 1106.34 L2407.07 1106.76 L2407.3 1107.12 L2407.53 1107.25 L2407.75 1107.37 L2407.98 1107.28 L2408.21 1106.82 L2408.44 1107.06 L2408.66 1107.37 L2408.89 1107.31 L2409.12 1106.95 L2409.35 1106.72 L2409.58 1106.49 L2409.8 1106.71 L2410.03 1106.76 L2410.26 1106.8 L2410.49 1106.71 L2410.71 1106.57 L2410.94 1106.59 L2411.17 1106.58 L2411.4 1106.73 L2411.63 1106.9 L2411.85 1107.53 L2412.08 1107.82 L2412.31 1107.96 L2412.54 1107.97 L2412.76 1107.52 L2412.99 1107.62 L2413.22 1107.83 L2413.45 1108.02 L2413.68 1107.77 L2413.9 1107.55 L2414.13 1107.52 L2414.36 1107.9 L2414.59 1108.48 L2414.81 1108.14 L2415.04 1107.86 L2415.27 1107.95 L2415.5 1107.92 L2415.73 1108.08 L2415.95 1107.68 L2416.18 1107.81 L2416.41 1108.15 L2416.64 1108.38 L2416.86 1108.18 L2417.09 1108.27 L2417.32 1108.31 L2417.55 1108.32 L2417.78 1108.79 L2418 1108.61 L2418.23 1109.14 L2418.46 1109.16 L2418.69 1108.83 L2418.91 1109.11 L2419.14 1109.01 L2419.37 1109.14 L2419.6 1109.06 L2419.83 1109.41 L2420.05 1109.33 L2420.28 1109.4 L2420.51 1109.5 L2420.74 1108.92 L2420.96 1108.63 L2421.19 1108.77 L2421.42 1108.13 L2421.65 1108.2 L2421.87 1107.75 L2422.1 1107.37 L2422.33 1107.42 L2422.56 1107.41 L2422.79 1107.08 L2423.01 1107.24 L2423.24 1107.62 L2423.47 1107.56 L2423.7 1107.47 L2423.92 1107.97 L2424.15 1108.26 L2424.38 1108.21 L2424.61 1108.16 L2424.84 1108.25 L2425.06 1108.4 L2425.29 1107.98 L2425.52 1108.13 L2425.75 1108.32 L2425.97 1108.39 L2426.2 1108.73 L2426.43 1108.75 L2426.66 1108.9 L2426.89 1108.49 L2427.11 1108.68 L2427.34 1108.52 L2427.57 1108.45 L2427.8 1108.3 L2428.02 1108.28 L2428.25 1108.48 L2428.48 1108.79 L2428.71 1109.04 L2428.94 1108.57 L2429.16 1108.72 L2429.39 1108.5 L2429.62 1108.38 L2429.85 1108.67 L2430.07 1108.8 L2430.3 1108.42 L2430.53 1108.69 L2430.76 1108.95 L2430.99 1108.85 L2431.21 1108.69 L2431.44 1109.07 L2431.67 1108.83 L2431.9 1108.66 L2432.12 1108.84 L2432.35 1109.09 L2432.58 1108.99 L2432.81 1108.65 L2433.04 1108.83 L2433.26 1108.8 L2433.49 1109.37 L2433.72 1109.41 L2433.95 1109.52 L2434.17 1109.3 L2434.4 1109.11 L2434.63 1109.49 L2434.86 1109.32 L2435.09 1109.31 L2435.31 1108.69 L2435.54 1108.54 L2435.77 1108.63 L2436 1108.71 L2436.22 1109 L2436.45 1109.2 L2436.68 1108.53 L2436.91 1108.85 L2437.14 1109.07 L2437.36 1108.89 L2437.59 1108.8 L2437.82 1108.95 L2438.05 1108.59 L2438.27 1108.53 L2438.5 1108.44 L2438.73 1108.1 L2438.96 1108.17 L2439.19 1108.37 L2439.41 1108.39 L2439.64 1108.02 L2439.87 1107.92 L2440.1 1107.92 L2440.32 1107.69 L2440.55 1107.7 L2440.78 1107.52 L2441.01 1107.94 L2441.24 1108.25 L2441.46 1108.29 L2441.69 1108.38 L2441.92 1108.08 L2442.15 1107.73 L2442.37 1107.52 L2442.6 1106.92 L2442.83 1107.04 L2443.06 1107.16 L2443.29 1107.04 L2443.51 1107.01 L2443.74 1107.31 L2443.97 1107.39 L2444.2 1106.98 L2444.42 1107.1 L2444.65 1107.04 L2444.88 1106.97 L2445.11 1107.45 L2445.34 1107.17 L2445.56 1107.06 L2445.79 1106.93 L2446.02 1106.59 L2446.25 1106.33 L2446.47 1106.5 L2446.7 1106.83 L2446.93 1106.86 L2447.16 1107.04 L2447.39 1107.15 L2447.61 1107.02 L2447.84 1106.99 L2448.07 1107.39 L2448.3 1107.12 L2448.52 1107.04 L2448.75 1107.44 L2448.98 1107.56 L2449.21 1107.98 L2449.44 1108.02 L2449.66 1108.17 L2449.89 1108.17 L2450.12 1107.78 L2450.35 1108.25 L2450.57 1107.39 L2450.8 1107.4 L2451.03 1107 L2451.26 1106.97 L2451.49 1107 L2451.71 1107.03 L2451.94 1106.87 L2452.17 1107.15 L2452.4 1107.23 L2452.62 1106.59 L2452.85 1106.89 L2453.08 1106.64 L2453.31 1107.02 L2453.54 1106.96 L2453.76 1107.37 L2453.99 1107.23 L2454.22 1107.6 L2454.45 1107.46 L2454.67 1107 L2454.9 1106.49 L2455.13 1106.08 L2455.36 1105.94 L2455.59 1106.19 L2455.81 1106.04 L2456.04 1106.07 L2456.27 1106.45 L2456.5 1106.62 L2456.72 1106.86 L2456.95 1107.05 L2457.18 1106.66 L2457.41 1106.93 L2457.64 1106.66 L2457.86 1106.56 L2458.09 1106.93 L2458.32 1107.04 L2458.55 1106.82 L2458.77 1106.51 L2459 1106.89 L2459.23 1107.01 L2459.46 1107.13 L2459.68 1106.56 L2459.91 1106.55 L2460.14 1106.2 L2460.37 1105.63 L2460.6 1106.08 L2460.82 1106.41 L2461.05 1106.65 L2461.28 1106.18 L2461.51 1106.66 L2461.73 1106.84 L2461.96 1107.11 L2462.19 1107.07 L2462.42 1107.31 L2462.65 1107.06 L2462.87 1106.92 L2463.1 1106.88 L2463.33 1107.07 L2463.56 1106.89 L2463.78 1107.26 L2464.01 1106.64 L2464.24 1106.95 L2464.47 1106.68 L2464.7 1106.73 L2464.92 1107.04 L2465.15 1107.23 L2465.38 1107.28 L2465.61 1107.21 L2465.83 1107.18 L2466.06 1106.8 L2466.29 1107.29 L2466.52 1107.03 L2466.75 1106.91 L2466.97 1107.17 L2467.2 1107.3 L2467.43 1106.92 L2467.66 1106.97 L2467.88 1106.45 L2468.11 1106.68 L2468.34 1106.44 L2468.57 1106.48 L2468.8 1106.85 L2469.02 1106.73 L2469.25 1106.89 L2469.48 1107.21 L2469.71 1106.92 L2469.93 1106.55 L2470.16 1106.72 L2470.39 1106.16 L2470.62 1106.18 L2470.85 1105.93 L2471.07 1105.81 L2471.3 1105.98 L2471.53 1106.04 L2471.76 1106.11 L2471.98 1105.76 L2472.21 1105.5 L2472.44 1105.67 L2472.67 1105.75 L2472.9 1105.78 L2473.12 1106.05 L2473.35 1105.65 L2473.58 1106.15 L2473.81 1106.64 L2474.03 1106.81 L2474.26 1106.41 L2474.49 1106.37 L2474.72 1106.17 L2474.95 1106.42 L2475.17 1106.38 L2475.4 1106.28 L2475.63 1106.37 L2475.86 1106.34 L2476.08 1105.56 L2476.31 1105.64 L2476.54 1105.49 L2476.77 1105.64 L2477 1105.29 L2477.22 1105.54 L2477.45 1105.94 L2477.68 1105.88 L2477.91 1106.22 L2478.13 1106.55 L2478.36 1106.46 L2478.59 1106.71 L2478.82 1107.12 L2479.05 1107.09 L2479.27 1107.09 L2479.5 1107.2 L2479.73 1107.42 L2479.96 1107.11 L2480.18 1107.34 L2480.41 1107.47 L2480.64 1107.55 L2480.87 1107.33 L2481.1 1107.37 L2481.32 1107.07 L2481.55 1107.42 L2481.78 1107.54 L2482.01 1107.3 L2482.23 1106.92 L2482.46 1106.84 L2482.69 1106.18 L2482.92 1106.78 L2483.15 1106.36 L2483.37 1106.74 L2483.6 1107.18 L2483.83 1107.66 L2484.06 1107.36 L2484.28 1107.16 L2484.51 1106.63 L2484.74 1106.95 L2484.97 1106.91 L2485.2 1106.95 L2485.42 1107.23 L2485.65 1107.02 L2485.88 1106.98 L2486.11 1106.45 L2486.33 1106.74 L2486.56 1106.76 L2486.79 1106.9 L2487.02 1106.85 L2487.25 1106.78 L2487.47 1106.63 L2487.7 1106.75 L2487.93 1107.32 L2488.16 1107.46 L2488.38 1107.02 L2488.61 1106.93 L2488.84 1107.22 L2489.07 1107.88 L2489.3 1107.82 L2489.52 1107.52 L2489.75 1107.81 L2489.98 1107.95 L2490.21 1107.57 L2490.43 1107.8 L2490.66 1107.82 L2490.89 1107.24 L2491.12 1107.67 L2491.35 1107.63 L2491.57 1107.64 L2491.8 1107.45 L2492.03 1107.49 L2492.26 1107 L2492.48 1107.22 L2492.71 1107.57 L2492.94 1107.65 L2493.17 1108.12 L2493.4 1108.29 L2493.62 1107.99 L2493.85 1107.37 L2494.08 1107.42 L2494.31 1107.06 L2494.53 1106.69 L2494.76 1106.68 L2494.99 1106.82 L2495.22 1106.41 L2495.44 1106.62 L2495.67 1106.37 L2495.9 1106.5 L2496.13 1106.61 L2496.36 1106.55 L2496.58 1106.85 L2496.81 1107.14 L2497.04 1107.06 L2497.27 1107.32 L2497.49 1107.45 L2497.72 1107.85 L2497.95 1107.19 L2498.18 1106.52 L2498.41 1106.53 L2498.63 1106.52 L2498.86 1106.26 L2499.09 1106.15 L2499.32 1106 L2499.54 1106.03 L2499.77 1105.85 L2500 1106.05 L2500.23 1105.82 L2500.46 1106.09 L2500.68 1105.93 L2500.91 1105.83 L2501.14 1105.97 L2501.37 1105.52 L2501.59 1105.25 L2501.82 1105.22 L2502.05 1105.03 L2502.28 1105.2 L2502.51 1104.09 L2502.73 1103.91 L2502.96 1104.1 L2503.19 1104.57 L2503.42 1104.14 L2503.64 1104.37 L2503.87 1104 L2504.1 1104.37 L2504.33 1104.47 L2504.56 1104.34 L2504.78 1104.22 L2505.01 1104 L2505.24 1104.22 L2505.47 1104.36 L2505.69 1103.68 L2505.92 1103.7 L2506.15 1103.76 L2506.38 1103.64 L2506.61 1103.43 L2506.83 1103.59 L2507.06 1103.38 L2507.29 1103.42 L2507.52 1102.64 L2507.74 1102.62 L2507.97 1102.5 L2508.2 1102.34 L2508.43 1102.22 L2508.66 1102.43 L2508.88 1102.05 L2509.11 1101.93 L2509.34 1101.63 L2509.57 1101.66 L2509.79 1101.82 L2510.02 1101.87 L2510.25 1101.82 L2510.48 1101.25 L2510.71 1101.55 L2510.93 1101.39 L2511.16 1101.38 L2511.39 1101.29 L2511.62 1101.33 L2511.84 1101.26 L2512.07 1101.85 L2512.3 1101.71 L2512.53 1101.08 L2512.76 1100.97 L2512.98 1100.82 L2513.21 1100.18 L2513.44 1099.91 L2513.67 1099.95 L2513.89 1100.05 L2514.12 1100.48 L2514.35 1100.12 L2514.58 1099.74 L2514.81 1099.73 L2515.03 1099.71 L2515.26 1099.63 L2515.49 1100.07 L2515.72 1100.29 L2515.94 1100.71 L2516.17 1100.99 L2516.4 1101.04 L2516.63 1101.04 L2516.86 1100.9 L2517.08 1100.7 L2517.31 1100.83 L2517.54 1100.9 L2517.77 1100.94 L2517.99 1101.08 L2518.22 1101.08 L2518.45 1100.71 L2518.68 1100.73 L2518.91 1101.01 L2519.13 1100.91 L2519.36 1101.14 L2519.59 1101.19 L2519.82 1101.82 L2520.04 1102.01 L2520.27 1102.14 L2520.5 1102.19 L2520.73 1102.07 L2520.96 1102.35 L2521.18 1102.63 L2521.41 1103.04 L2521.64 1103.03 L2521.87 1102.79 L2522.09 1102.55 L2522.32 1102.1 L2522.55 1102.11 L2522.78 1101.36 L2523.01 1101.43 L2523.23 1101.61 L2523.46 1101.77 L2523.69 1101.43 L2523.92 1101.81 L2524.14 1101.99 L2524.37 1101.87 L2524.6 1101.79 L2524.83 1101.36 L2525.06 1100.98 L2525.28 1100.71 L2525.51 1100.67 L2525.74 1100.73 L2525.97 1101.21 L2526.19 1101.43 L2526.42 1101.49 L2526.65 1101.63 L2526.88 1101.46 L2527.11 1101.53 L2527.33 1101.8 L2527.56 1101.94 L2527.79 1102.16 L2528.02 1102.42 L2528.24 1102.49 L2528.47 1102.83 L2528.7 1102.84 L2528.93 1103.19 L2529.16 1103.53 L2529.38 1103.52 L2529.61 1104.05 L2529.84 1104.47 L2530.07 1104.15 L2530.29 1104.77 L2530.52 1105.05 L2530.75 1105.14 L2530.98 1105.48 L2531.2 1105.53 L2531.43 1104.89 L2531.66 1104.6 L2531.89 1104.4 L2532.12 1104.09 L2532.34 1104.18 L2532.57 1103.97 L2532.8 1103.94 L2533.03 1104.13 L2533.25 1103.92 L2533.48 1104.01 L2533.71 1103.91 L2533.94 1103.97 L2534.17 1104.15 L2534.39 1104.46 L2534.62 1104.82 L2534.85 1104.58 L2535.08 1104.93 L2535.3 1104.51 L2535.53 1104.88 L2535.76 1104.81 L2535.99 1104.5 L2536.22 1104.25 L2536.44 1104.58 L2536.67 1104.4 L2536.9 1104.28 L2537.13 1104.14 L2537.35 1103.99 L2537.58 1103.81 L2537.81 1103.75 L2538.04 1103.38 L2538.27 1103.5 L2538.49 1103.7 L2538.72 1104.15 L2538.95 1104.46 L2539.18 1104.23 L2539.4 1104.06 L2539.63 1103.9 L2539.86 1104.04 L2540.09 1103.71 L2540.32 1103.5 L2540.54 1103.71 L2540.77 1103.73 L2541 1103.64 L2541.23 1103.65 L2541.45 1103.37 L2541.68 1103.03 L2541.91 1103.14 L2542.14 1103.37 L2542.37 1103.56 L2542.59 1103.11 L2542.82 1103.07 L2543.05 1102.95 L2543.28 1102.77 L2543.5 1102.88 L2543.73 1102.78 L2543.96 1102.38 L2544.19 1102.54 L2544.42 1102.59 L2544.64 1103.01 L2544.87 1103.29 L2545.1 1103.02 L2545.33 1103.36 L2545.55 1103.1 L2545.78 1103.26 L2546.01 1103.34 L2546.24 1103.64 L2546.47 1103.86 L2546.69 1104.22 L2546.92 1103.88 L2547.15 1103.92 L2547.38 1104.09 L2547.6 1104 L2547.83 1103.92 L2548.06 1104.04 L2548.29 1104.28 L2548.52 1104.31 L2548.74 1104.49 L2548.97 1104.68 L2549.2 1104.79 L2549.43 1104.91 L2549.65 1104.8 L2549.88 1104.99 L2550.11 1105.46 L2550.34 1105.46 L2550.57 1105.59 L2550.79 1106.06 L2551.02 1105.78 L2551.25 1105.87 L2551.48 1105.99 L2551.7 1106.15 L2551.93 1105.68 L2552.16 1105.92 L2552.39 1106.01 L2552.62 1105.95 L2552.84 1105.85 L2553.07 1105.52 L2553.3 1105.67 L2553.53 1106.13 L2553.75 1106.34 L2553.98 1106.21 L2554.21 1106.24 L2554.44 1106.65 L2554.67 1106.56 L2554.89 1106.81 L2555.12 1106.4 L2555.35 1106.58 L2555.58 1107 L2555.8 1107.38 L2556.03 1107.42 L2556.26 1107.91 L2556.49 1107.95 L2556.72 1107.56 L2556.94 1107.64 L2557.17 1107.39 L2557.4 1107.68 L2557.63 1107.47 L2557.85 1107.76 L2558.08 1107.43 L2558.31 1107.5 L2558.54 1107.24 L2558.77 1107.17 L2558.99 1107.25 L2559.22 1107.24 L2559.45 1107.54 L2559.68 1107.99 L2559.9 1108.01 L2560.13 1108.39 L2560.36 1108.74 L2560.59 1108.37 L2560.82 1108.38 L2561.04 1108.35 L2561.27 1107.95 L2561.5 1108.29 L2561.73 1108.43 L2561.95 1108.4 L2562.18 1108.34 L2562.41 1108.31 L2562.64 1108.12 L2562.87 1108.03 L2563.09 1107.08 L2563.32 1107.19 L2563.55 1107.19 L2563.78 1107.09 L2564 1107.09 L2564.23 1106.57 L2564.46 1106.24 L2564.69 1106.19 L2564.92 1106.29 L2565.14 1106.55 L2565.37 1106.73 L2565.6 1107.21 L2565.83 1106.9 L2566.05 1106.72 L2566.28 1107.11 L2566.51 1107.11 L2566.74 1107.09 L2566.96 1107.07 L2567.19 1106.91 L2567.42 1106.43 L2567.65 1106.71 L2567.88 1106.57 L2568.1 1106.5 L2568.33 1106.13 L2568.56 1106.06 L2568.79 1106.18 L2569.01 1105.7 L2569.24 1105.78 L2569.47 1105.75 L2569.7 1105.5 L2569.93 1105.49 L2570.15 1105.88 L2570.38 1105.63 L2570.61 1105.6 L2570.84 1105.72 L2571.06 1105.29 L2571.29 1104.54 L2571.52 1104.7 L2571.75 1104.67 L2571.98 1104.66 L2572.2 1104.36 L2572.43 1104.76 L2572.43 1002.06 L2572.2 1001.66 L2571.98 1001.95 L2571.75 1001.95 L2571.52 1001.97 L2571.29 1001.8 L2571.06 1002.58 L2570.84 1003.02 L2570.61 1002.9 L2570.38 1002.91 L2570.15 1003.16 L2569.93 1002.77 L2569.7 1002.77 L2569.47 1003.01 L2569.24 1003.03 L2569.01 1002.94 L2568.79 1003.42 L2568.56 1003.3 L2568.33 1003.36 L2568.1 1003.73 L2567.88 1003.79 L2567.65 1003.92 L2567.42 1003.64 L2567.19 1004.12 L2566.96 1004.27 L2566.74 1004.28 L2566.51 1004.29 L2566.28 1004.28 L2566.05 1003.9 L2565.83 1004.07 L2565.6 1004.37 L2565.37 1003.9 L2565.14 1003.71 L2564.92 1003.44 L2564.69 1003.33 L2564.46 1003.37 L2564.23 1003.7 L2564 1004.23 L2563.78 1004.23 L2563.55 1004.31 L2563.32 1004.3 L2563.09 1004.18 L2562.87 1005.2 L2562.64 1005.27 L2562.41 1005.46 L2562.18 1005.48 L2561.95 1005.53 L2561.73 1005.55 L2561.5 1005.4 L2561.27 1005.06 L2561.04 1005.46 L2560.82 1005.48 L2560.59 1005.46 L2560.36 1005.83 L2560.13 1005.48 L2559.9 1005.1 L2559.68 1005.07 L2559.45 1004.62 L2559.22 1004.32 L2558.99 1004.32 L2558.77 1004.23 L2558.54 1004.29 L2558.31 1004.54 L2558.08 1004.46 L2557.85 1004.79 L2557.63 1004.49 L2557.4 1004.7 L2557.17 1004.41 L2556.94 1004.65 L2556.72 1004.56 L2556.49 1004.95 L2556.26 1004.91 L2556.03 1004.42 L2555.8 1004.37 L2555.58 1003.98 L2555.35 1003.57 L2555.12 1003.39 L2554.89 1003.8 L2554.67 1003.54 L2554.44 1003.62 L2554.21 1003.21 L2553.98 1003.17 L2553.75 1003.3 L2553.53 1003.08 L2553.3 1002.62 L2553.07 1002.47 L2552.84 1002.79 L2552.62 1002.89 L2552.39 1002.93 L2552.16 1002.83 L2551.93 1002.59 L2551.7 1003.06 L2551.48 1002.89 L2551.25 1002.77 L2551.02 1002.67 L2550.79 1002.94 L2550.57 1002.47 L2550.34 1002.34 L2550.11 1002.32 L2549.88 1001.86 L2549.65 1001.67 L2549.43 1001.76 L2549.2 1001.63 L2548.97 1001.52 L2548.74 1001.31 L2548.52 1001.13 L2548.29 1001.09 L2548.06 1000.84 L2547.83 1000.72 L2547.6 1000.78 L2547.38 1000.86 L2547.15 1000.69 L2546.92 1000.63 L2546.69 1000.97 L2546.47 1000.61 L2546.24 1000.39 L2546.01 1000.08 L2545.78 999.995 L2545.55 999.826 L2545.33 1000.08 L2545.1 999.735 L2544.87 999.998 L2544.64 999.711 L2544.42 999.296 L2544.19 999.241 L2543.96 999.074 L2543.73 999.474 L2543.5 999.56 L2543.28 999.437 L2543.05 999.617 L2542.82 999.726 L2542.59 999.752 L2542.37 1000.2 L2542.14 1000.01 L2541.91 999.769 L2541.68 999.652 L2541.45 999.997 L2541.23 1000.27 L2541 1000.25 L2540.77 1000.32 L2540.54 1000.3 L2540.32 1000.08 L2540.09 1000.28 L2539.86 1000.61 L2539.63 1000.46 L2539.4 1000.61 L2539.18 1000.77 L2538.95 1001 L2538.72 1000.68 L2538.49 1000.24 L2538.27 1000.03 L2538.04 999.898 L2537.81 1000.27 L2537.58 1000.32 L2537.35 1000.49 L2537.13 1000.64 L2536.9 1000.77 L2536.67 1000.88 L2536.44 1001.05 L2536.22 1000.71 L2535.99 1000.96 L2535.76 1001.27 L2535.53 1001.32 L2535.3 1000.95 L2535.08 1001.38 L2534.85 1001.03 L2534.62 1001.26 L2534.39 1000.9 L2534.17 1000.59 L2533.94 1000.4 L2533.71 1000.32 L2533.48 1000.42 L2533.25 1000.31 L2533.03 1000.53 L2532.8 1000.32 L2532.57 1000.35 L2532.34 1000.54 L2532.12 1000.45 L2531.89 1000.75 L2531.66 1000.95 L2531.43 1001.23 L2531.2 1001.89 L2530.98 1001.83 L2530.75 1001.49 L2530.52 1001.39 L2530.29 1001.11 L2530.07 1000.5 L2529.84 1000.82 L2529.61 1000.4 L2529.38 999.881 L2529.16 999.883 L2528.93 999.539 L2528.7 999.188 L2528.47 999.162 L2528.24 998.82 L2528.02 998.746 L2527.79 998.48 L2527.56 998.253 L2527.33 998.103 L2527.11 997.827 L2526.88 997.744 L2526.65 997.912 L2526.42 997.757 L2526.19 997.686 L2525.97 997.467 L2525.74 996.99 L2525.51 996.915 L2525.28 996.948 L2525.06 997.208 L2524.83 997.589 L2524.6 998.024 L2524.37 998.094 L2524.14 998.208 L2523.92 998.015 L2523.69 997.639 L2523.46 997.977 L2523.23 997.809 L2523.01 997.619 L2522.78 997.535 L2522.55 998.324 L2522.32 998.305 L2522.09 998.756 L2521.87 998.989 L2521.64 999.226 L2521.41 999.222 L2521.18 998.813 L2520.96 998.526 L2520.73 998.248 L2520.5 998.356 L2520.27 998.3 L2520.04 998.152 L2519.82 997.959 L2519.59 997.343 L2519.36 997.282 L2519.13 997.053 L2518.91 997.139 L2518.68 996.859 L2518.45 996.822 L2518.22 997.199 L2517.99 997.185 L2517.77 997.033 L2517.54 996.986 L2517.31 996.908 L2517.08 996.762 L2516.86 996.961 L2516.63 997.088 L2516.4 997.073 L2516.17 997.014 L2515.94 996.728 L2515.72 996.312 L2515.49 996.091 L2515.26 995.654 L2515.03 995.718 L2514.81 995.725 L2514.58 995.731 L2514.35 996.107 L2514.12 996.471 L2513.89 996.038 L2513.67 995.93 L2513.44 995.876 L2513.21 996.147 L2512.98 996.8 L2512.76 996.949 L2512.53 997.051 L2512.3 997.697 L2512.07 997.822 L2511.84 997.252 L2511.62 997.305 L2511.39 997.255 L2511.16 997.343 L2510.93 997.34 L2510.71 997.494 L2510.48 997.187 L2510.25 997.772 L2510.02 997.812 L2509.79 997.75 L2509.57 997.582 L2509.34 997.536 L2509.11 997.835 L2508.88 997.947 L2508.66 998.32 L2508.43 998.104 L2508.2 998.222 L2507.97 998.37 L2507.74 998.484 L2507.52 998.489 L2507.29 999.3 L2507.06 999.259 L2506.83 999.459 L2506.61 999.287 L2506.38 999.49 L2506.15 999.6 L2505.92 999.53 L2505.69 999.503 L2505.47 1000.21 L2505.24 1000.06 L2505.01 999.825 L2504.78 1000.04 L2504.56 1000.16 L2504.33 1000.27 L2504.1 1000.16 L2503.87 999.79 L2503.64 1000.16 L2503.42 999.921 L2503.19 1000.35 L2502.96 999.894 L2502.73 999.693 L2502.51 999.867 L2502.28 1001.07 L2502.05 1000.89 L2501.82 1001.06 L2501.59 1001.09 L2501.37 1001.35 L2501.14 1001.8 L2500.91 1001.65 L2500.68 1001.74 L2500.46 1001.9 L2500.23 1001.62 L2500 1001.84 L2499.77 1001.63 L2499.54 1001.81 L2499.32 1001.76 L2499.09 1001.91 L2498.86 1002.01 L2498.63 1002.26 L2498.41 1002.26 L2498.18 1002.24 L2497.95 1002.93 L2497.72 1003.61 L2497.49 1003.21 L2497.27 1003.08 L2497.04 1002.81 L2496.81 1002.88 L2496.58 1002.59 L2496.36 1002.28 L2496.13 1002.33 L2495.9 1002.21 L2495.67 1002.07 L2495.44 1002.32 L2495.22 1002.1 L2494.99 1002.51 L2494.76 1002.36 L2494.53 1002.36 L2494.31 1002.73 L2494.08 1003.09 L2493.85 1003.03 L2493.62 1003.66 L2493.4 1003.96 L2493.17 1003.78 L2492.94 1003.32 L2492.71 1003.23 L2492.48 1002.88 L2492.26 1002.65 L2492.03 1003.15 L2491.8 1003.09 L2491.57 1003.28 L2491.35 1003.25 L2491.12 1003.28 L2490.89 1002.86 L2490.66 1003.46 L2490.43 1003.43 L2490.21 1003.19 L2489.98 1003.57 L2489.75 1003.41 L2489.52 1003.12 L2489.3 1003.42 L2489.07 1003.46 L2488.84 1002.82 L2488.61 1002.53 L2488.38 1002.61 L2488.16 1003.05 L2487.93 1002.9 L2487.7 1002.35 L2487.47 1002.22 L2487.25 1002.36 L2487.02 1002.41 L2486.79 1002.46 L2486.56 1002.31 L2486.33 1002.28 L2486.11 1001.98 L2485.88 1002.53 L2485.65 1002.55 L2485.42 1002.76 L2485.2 1002.47 L2484.97 1002.42 L2484.74 1002.45 L2484.51 1002.12 L2484.28 1002.66 L2484.06 1002.85 L2483.83 1003.16 L2483.6 1002.68 L2483.37 1002.25 L2483.15 1001.86 L2482.92 1002.29 L2482.69 1001.7 L2482.46 1002.38 L2482.23 1002.46 L2482.01 1002.84 L2481.78 1003.07 L2481.55 1002.94 L2481.32 1002.58 L2481.1 1002.88 L2480.87 1002.83 L2480.64 1003.04 L2480.41 1002.95 L2480.18 1002.82 L2479.96 1002.58 L2479.73 1002.88 L2479.5 1002.65 L2479.27 1002.54 L2479.05 1002.52 L2478.82 1002.54 L2478.59 1002.13 L2478.36 1001.88 L2478.13 1001.95 L2477.91 1001.62 L2477.68 1001.28 L2477.45 1001.33 L2477.22 1000.93 L2477 1000.68 L2476.77 1001.02 L2476.54 1000.86 L2476.31 1001 L2476.08 1000.91 L2475.86 1001.73 L2475.63 1001.75 L2475.4 1001.65 L2475.17 1001.74 L2474.95 1001.77 L2474.72 1001.51 L2474.49 1001.7 L2474.26 1001.74 L2474.03 1002.13 L2473.81 1001.96 L2473.58 1001.48 L2473.35 1000.98 L2473.12 1001.38 L2472.9 1001.1 L2472.67 1001.07 L2472.44 1000.97 L2472.21 1000.79 L2471.98 1001.04 L2471.76 1001.4 L2471.53 1001.32 L2471.3 1001.24 L2471.07 1001.06 L2470.85 1001.17 L2470.62 1001.42 L2470.39 1001.39 L2470.16 1001.96 L2469.93 1001.78 L2469.71 1002.15 L2469.48 1002.44 L2469.25 1002.12 L2469.02 1001.94 L2468.8 1002.05 L2468.57 1001.68 L2468.34 1001.63 L2468.11 1001.87 L2467.88 1001.63 L2467.66 1002.16 L2467.43 1002.1 L2467.2 1002.48 L2466.97 1002.33 L2466.75 1002.07 L2466.52 1002.18 L2466.29 1002.43 L2466.06 1001.95 L2465.83 1002.33 L2465.61 1002.35 L2465.38 1002.4 L2465.15 1002.35 L2464.92 1002.15 L2464.7 1001.83 L2464.47 1001.78 L2464.24 1002.04 L2464.01 1001.73 L2463.78 1002.36 L2463.56 1001.99 L2463.33 1002.16 L2463.1 1001.97 L2462.87 1001.99 L2462.65 1002.12 L2462.42 1002.36 L2462.19 1002.12 L2461.96 1002.15 L2461.73 1001.87 L2461.51 1001.68 L2461.28 1001.21 L2461.05 1001.69 L2460.82 1001.44 L2460.6 1001.11 L2460.37 1000.65 L2460.14 1001.25 L2459.91 1001.59 L2459.68 1001.59 L2459.46 1002.17 L2459.23 1002.05 L2459 1001.91 L2458.77 1001.53 L2458.55 1001.84 L2458.32 1002.05 L2458.09 1001.93 L2457.86 1001.56 L2457.64 1001.65 L2457.41 1001.92 L2457.18 1001.63 L2456.95 1002.03 L2456.72 1001.83 L2456.5 1001.58 L2456.27 1001.41 L2456.04 1001.02 L2455.81 1000.98 L2455.59 1001.12 L2455.36 1000.87 L2455.13 1001 L2454.9 1001.41 L2454.67 1001.93 L2454.45 1002.39 L2454.22 1002.52 L2453.99 1002.15 L2453.76 1002.28 L2453.54 1001.88 L2453.31 1001.92 L2453.08 1001.54 L2452.85 1001.78 L2452.62 1001.48 L2452.4 1002.14 L2452.17 1002.05 L2451.94 1001.77 L2451.71 1001.92 L2451.49 1001.88 L2451.26 1001.83 L2451.03 1001.85 L2450.8 1002.25 L2450.57 1002.23 L2450.35 1003.13 L2450.12 1002.67 L2449.89 1003.06 L2449.66 1003.05 L2449.44 1002.88 L2449.21 1002.84 L2448.98 1002.42 L2448.75 1002.29 L2448.52 1001.89 L2448.3 1001.96 L2448.07 1002.22 L2447.84 1001.83 L2447.61 1001.85 L2447.39 1001.96 L2447.16 1001.84 L2446.93 1001.66 L2446.7 1001.61 L2446.47 1001.28 L2446.25 1001.1 L2446.02 1001.36 L2445.79 1001.69 L2445.56 1001.81 L2445.34 1001.92 L2445.11 1002.19 L2444.88 1001.71 L2444.65 1001.77 L2444.42 1001.82 L2444.2 1001.69 L2443.97 1002.1 L2443.74 1002.01 L2443.51 1001.71 L2443.29 1001.73 L2443.06 1001.84 L2442.83 1001.71 L2442.6 1001.58 L2442.37 1002.19 L2442.15 1002.39 L2441.92 1002.74 L2441.69 1003.04 L2441.46 1002.93 L2441.24 1002.88 L2441.01 1002.57 L2440.78 1002.15 L2440.55 1002.32 L2440.32 1002.3 L2440.1 1002.52 L2439.87 1002.51 L2439.64 1002.6 L2439.41 1002.97 L2439.19 1002.94 L2438.96 1002.74 L2438.73 1002.65 L2438.5 1002.98 L2438.27 1003.07 L2438.05 1003.12 L2437.82 1003.47 L2437.59 1003.32 L2437.36 1003.39 L2437.14 1003.56 L2436.91 1003.34 L2436.68 1003.02 L2436.45 1003.71 L2436.22 1003.5 L2436 1003.2 L2435.77 1003.11 L2435.54 1003.01 L2435.31 1003.15 L2435.09 1003.79 L2434.86 1003.78 L2434.63 1003.95 L2434.4 1003.57 L2434.17 1003.75 L2433.95 1003.97 L2433.72 1003.84 L2433.49 1003.79 L2433.26 1003.23 L2433.04 1003.26 L2432.81 1003.06 L2432.58 1003.4 L2432.35 1003.49 L2432.12 1003.23 L2431.9 1003.04 L2431.67 1003.2 L2431.44 1003.44 L2431.21 1003.05 L2430.99 1003.2 L2430.76 1003.29 L2430.53 1003.03 L2430.3 1002.75 L2430.07 1003.14 L2429.85 1002.99 L2429.62 1002.7 L2429.39 1002.81 L2429.16 1003.02 L2428.94 1002.86 L2428.71 1003.34 L2428.48 1003.08 L2428.25 1002.76 L2428.02 1002.55 L2427.8 1002.56 L2427.57 1002.71 L2427.34 1002.76 L2427.11 1002.92 L2426.89 1002.71 L2426.66 1003.13 L2426.43 1002.97 L2426.2 1002.94 L2425.97 1002.59 L2425.75 1002.52 L2425.52 1002.32 L2425.29 1002.15 L2425.06 1002.57 L2424.84 1002.42 L2424.61 1002.31 L2424.38 1002.36 L2424.15 1002.39 L2423.92 1002.1 L2423.7 1001.61 L2423.47 1001.68 L2423.24 1001.74 L2423.01 1001.35 L2422.79 1001.19 L2422.56 1001.51 L2422.33 1001.51 L2422.1 1001.45 L2421.87 1001.82 L2421.65 1002.28 L2421.42 1002.2 L2421.19 1002.86 L2420.96 1002.71 L2420.74 1003 L2420.51 1003.59 L2420.28 1003.48 L2420.05 1003.4 L2419.83 1003.47 L2419.6 1003.11 L2419.37 1003.18 L2419.14 1003.04 L2418.91 1003.13 L2418.69 1002.84 L2418.46 1003.18 L2418.23 1003.14 L2418 1002.62 L2417.78 1002.79 L2417.55 1002.33 L2417.32 1002.31 L2417.09 1002.25 L2416.86 1002.15 L2416.64 1002.35 L2416.41 1002.1 L2416.18 1001.76 L2415.95 1001.62 L2415.73 1002.02 L2415.5 1001.85 L2415.27 1001.87 L2415.04 1001.77 L2414.81 1002.04 L2414.59 1002.38 L2414.36 1001.81 L2414.13 1001.44 L2413.9 1001.45 L2413.68 1001.66 L2413.45 1001.9 L2413.22 1001.71 L2412.99 1001.49 L2412.76 1001.38 L2412.54 1001.84 L2412.31 1001.81 L2412.08 1001.66 L2411.85 1001.37 L2411.63 1000.75 L2411.4 1000.57 L2411.17 1000.41 L2410.94 1000.42 L2410.71 1000.38 L2410.49 1000.51 L2410.26 1000.59 L2410.03 1000.54 L2409.8 1000.48 L2409.58 1000.25 L2409.35 1000.47 L2409.12 1000.7 L2408.89 1001.06 L2408.66 1001.1 L2408.44 1000.79 L2408.21 1000.54 L2407.98 1001 L2407.75 1001.09 L2407.53 1000.96 L2407.3 1000.81 L2407.07 1000.45 L2406.84 1000.03 L2406.61 1000.05 L2406.39 1000.1 L2406.16 999.531 L2405.93 999.265 L2405.7 999.371 L2405.48 999.529 L2405.25 999.081 L2405.02 998.878 L2404.79 998.982 L2404.56 999.095 L2404.34 998.953 L2404.11 999.121 L2403.88 998.796 L2403.65 998.353 L2403.43 998.378 L2403.2 998.352 L2402.97 997.786 L2402.74 997.98 L2402.51 998.117 L2402.29 998.345 L2402.06 998.146 L2401.83 997.755 L2401.6 997.753 L2401.38 997.21 L2401.15 997.242 L2400.92 997.04 L2400.69 997.254 L2400.46 996.943 L2400.24 997.193 L2400.01 997.588 L2399.78 997.437 L2399.55 997.206 L2399.33 997.81 L2399.1 997.94 L2398.87 997.828 L2398.64 997.523 L2398.41 997.253 L2398.19 997.206 L2397.96 997.248 L2397.73 997.103 L2397.5 997.593 L2397.28 997.468 L2397.05 997.771 L2396.82 997.474 L2396.59 997.579 L2396.36 997.432 L2396.14 997.288 L2395.91 996.841 L2395.68 996.869 L2395.45 996.428 L2395.23 996.224 L2395 996.856 L2394.77 996.786 L2394.54 996.882 L2394.31 996.579 L2394.09 996.676 L2393.86 996.392 L2393.63 996.806 L2393.4 997.093 L2393.18 997.085 L2392.95 996.654 L2392.72 996.48 L2392.49 996.195 L2392.26 995.944 L2392.04 996.381 L2391.81 996.486 L2391.58 997.226 L2391.35 997.299 L2391.13 997 L2390.9 996.73 L2390.67 996.514 L2390.44 996.218 L2390.21 996.181 L2389.99 995.99 L2389.76 995.673 L2389.53 995.672 L2389.3 995.579 L2389.08 995.917 L2388.85 995.827 L2388.62 996.257 L2388.39 995.987 L2388.16 995.924 L2387.94 996.56 L2387.71 996.295 L2387.48 996.627 L2387.25 996.474 L2387.03 996.42 L2386.8 996.082 L2386.57 995.962 L2386.34 995.917 L2386.11 996.073 L2385.89 996.818 L2385.66 997.102 L2385.43 997.012 L2385.2 996.839 L2384.98 996.802 L2384.75 996.496 L2384.52 996.558 L2384.29 996.645 L2384.07 996.809 L2383.84 996.364 L2383.61 996.096 L2383.38 996.354 L2383.15 995.984 L2382.93 996.431 L2382.7 996.938 L2382.47 997.522 L2382.24 997.871 L2382.02 997.456 L2381.79 997.583 L2381.56 997.982 L2381.33 998.34 L2381.1 998.028 L2380.88 998.209 L2380.65 998.582 L2380.42 998.472 L2380.19 998.274 L2379.97 998.095 L2379.74 998.383 L2379.51 998.583 L2379.28 998.801 L2379.05 998.842 L2378.83 998.636 L2378.6 998.637 L2378.37 999.417 L2378.14 999.685 L2377.92 999.815 L2377.69 999.784 L2377.46 1000.1 L2377.23 999.959 L2377 999.958 L2376.78 1000.03 L2376.55 999.906 L2376.32 999.752 L2376.09 999.825 L2375.87 999.779 L2375.64 1000.08 L2375.41 999.892 L2375.18 999.968 L2374.95 999.654 L2374.73 1000.15 L2374.5 1000.23 L2374.27 1000.33 L2374.04 1000.3 L2373.82 1000.11 L2373.59 999.699 L2373.36 999.981 L2373.13 1000.13 L2372.9 999.802 L2372.68 999.273 L2372.45 998.752 L2372.22 998.272 L2371.99 998.232 L2371.77 997.883 L2371.54 997.583 L2371.31 997.431 L2371.08 997.727 L2370.85 997.325 L2370.63 997.568 L2370.4 998.908 L2370.17 999.489 L2369.94 999.279 L2369.72 999.056 L2369.49 998.866 L2369.26 998.747 L2369.03 998.378 L2368.8 998.48 L2368.58 998.894 L2368.35 998.707 L2368.12 999.143 L2367.89 998.716 L2367.67 998.633 L2367.44 998.582 L2367.21 998.555 L2366.98 998.674 L2366.75 998.527 L2366.53 998.839 L2366.3 998.676 L2366.07 998.511 L2365.84 998.893 L2365.62 998.704 L2365.39 998.401 L2365.16 998.345 L2364.93 998.267 L2364.7 997.953 L2364.48 997.791 L2364.25 997.7 L2364.02 997.658 L2363.79 997.69 L2363.57 997.648 L2363.34 997.722 L2363.11 998.004 L2362.88 997.852 L2362.65 997.715 L2362.43 997.356 L2362.2 997.178 L2361.97 996.855 L2361.74 997.14 L2361.52 996.939 L2361.29 996.751 L2361.06 996.524 L2360.83 996.062 L2360.6 996.073 L2360.38 996.025 L2360.15 996.408 L2359.92 996.612 L2359.69 996.152 L2359.47 996.12 L2359.24 996.483 L2359.01 996.423 L2358.78 996.583 L2358.55 996.436 L2358.33 996.354 L2358.1 996.238 L2357.87 996.037 L2357.64 996.099 L2357.42 995.611 L2357.19 996.158 L2356.96 996.036 L2356.73 996.146 L2356.5 996.4 L2356.28 995.982 L2356.05 996.031 L2355.82 996.13 L2355.59 996.314 L2355.37 996.59 L2355.14 996.356 L2354.91 996.506 L2354.68 996.818 L2354.45 997.076 L2354.23 997.309 L2354 997.244 L2353.77 996.789 L2353.54 996.575 L2353.32 996.642 L2353.09 996.938 L2352.86 996.573 L2352.63 996.643 L2352.4 996.761 L2352.18 996.226 L2351.95 995.836 L2351.72 995.885 L2351.49 995.506 L2351.27 995.133 L2351.04 995.322 L2350.81 995.332 L2350.58 995.282 L2350.35 994.877 L2350.13 994.563 L2349.9 994.619 L2349.67 994.334 L2349.44 994.131 L2349.22 994.739 L2348.99 994.279 L2348.76 994.704 L2348.53 994.583 L2348.31 994.733 L2348.08 994.757 L2347.85 994.721 L2347.62 994.804 L2347.39 994.557 L2347.17 994.34 L2346.94 994.683 L2346.71 994.583 L2346.48 994.931 L2346.26 995.143 L2346.03 995.078 L2345.8 995.031 L2345.57 995.013 L2345.34 994.967 L2345.12 995.097 L2344.89 994.901 L2344.66 994.839 L2344.43 995.043 L2344.21 995.292 L2343.98 995.407 L2343.75 995.258 L2343.52 995.724 L2343.29 995.575 L2343.07 995.65 L2342.84 995.455 L2342.61 995.426 L2342.38 995.083 L2342.16 994.972 L2341.93 994.734 L2341.7 994.969 L2341.47 994.992 L2341.24 995.122 L2341.02 994.685 L2340.79 994.689 L2340.56 994.891 L2340.33 995.419 L2340.11 995.138 L2339.88 995.285 L2339.65 995.295 L2339.42 995.122 L2339.19 995.531 L2338.97 996.029 L2338.74 995.951 L2338.51 996.053 L2338.28 996.408 L2338.06 996.361 L2337.83 996.273 L2337.6 996.13 L2337.37 996.058 L2337.14 995.753 L2336.92 995.912 L2336.69 995.82 L2336.46 996.012 L2336.23 995.854 L2336.01 995.75 L2335.78 996.06 L2335.55 996.04 L2335.32 995.791 L2335.09 995.913 L2334.87 996.13 L2334.64 996.035 L2334.41 995.943 L2334.18 996.009 L2333.96 996.146 L2333.73 996.243 L2333.5 996.195 L2333.27 996.96 L2333.04 996.966 L2332.82 996.928 L2332.59 997.283 L2332.36 997.176 L2332.13 997.355 L2331.91 997.22 L2331.68 997.133 L2331.45 996.783 L2331.22 996.636 L2330.99 996.371 L2330.77 996.333 L2330.54 996.499 L2330.31 996.255 L2330.08 996.118 L2329.86 995.916 L2329.63 995.775 L2329.4 995.284 L2329.17 995.388 L2328.94 995.853 L2328.72 995.481 L2328.49 995.382 L2328.26 995.154 L2328.03 994.852 L2327.81 994.723 L2327.58 995.09 L2327.35 995.445 L2327.12 995.176 L2326.89 995.506 L2326.67 995.457 L2326.44 995.265 L2326.21 995.154 L2325.98 995.177 L2325.76 994.956 L2325.53 994.886 L2325.3 994.623 L2325.07 994.677 L2324.84 994.406 L2324.62 994.331 L2324.39 994.229 L2324.16 994.229 L2323.93 993.655 L2323.71 993.741 L2323.48 993.492 L2323.25 993.085 L2323.02 992.84 L2322.79 993.149 L2322.57 993.237 L2322.34 993.347 L2322.11 993.696 L2321.88 994.05 L2321.66 993.77 L2321.43 994.199 L2321.2 994.639 L2320.97 994.56 L2320.74 994.482 L2320.52 994.027 L2320.29 994.09 L2320.06 994.169 L2319.83 993.966 L2319.61 994.058 L2319.38 993.828 L2319.15 994.246 L2318.92 994.458 L2318.69 994.356 L2318.47 994.19 L2318.24 994.027 L2318.01 993.92 L2317.78 994.063 L2317.56 994.688 L2317.33 994.408 L2317.1 994.019 L2316.87 993.958 L2316.64 993.764 L2316.42 994.511 L2316.19 994.463 L2315.96 994.262 L2315.73 994.445 L2315.51 994.372 L2315.28 994.337 L2315.05 994.483 L2314.82 994.732 L2314.59 994.813 L2314.37 994.916 L2314.14 995.277 L2313.91 995.574 L2313.68 995.685 L2313.46 995.508 L2313.23 995.511 L2313 995.139 L2312.77 995.465 L2312.54 995.484 L2312.32 995.425 L2312.09 995.326 L2311.86 995.11 L2311.63 994.62 L2311.41 995.03 L2311.18 995.397 L2310.95 995.588 L2310.72 996.011 L2310.5 996.179 L2310.27 996.2 L2310.04 996.108 L2309.81 996.347 L2309.58 996.053 L2309.36 996.14 L2309.13 995.563 L2308.9 995.196 L2308.67 995.152 L2308.45 995.204 L2308.22 995.212 L2307.99 995.684 L2307.76 995.369 L2307.53 995.686 L2307.31 996.035 L2307.08 996.422 L2306.85 996.273 L2306.62 996.135 L2306.4 996.201 L2306.17 995.783 L2305.94 995.609 L2305.71 996.211 L2305.48 996.123 L2305.26 995.955 L2305.03 995.824 L2304.8 995.316 L2304.57 995.142 L2304.35 995.283 L2304.12 994.994 L2303.89 995.032 L2303.66 994.868 L2303.43 995.314 L2303.21 995.268 L2302.98 995.292 L2302.75 994.89 L2302.52 994.979 L2302.3 995.014 L2302.07 995.719 L2301.84 995.497 L2301.61 995.961 L2301.38 996.266 L2301.16 996.168 L2300.93 995.978 L2300.7 995.909 L2300.47 995.699 L2300.25 995.252 L2300.02 995.709 L2299.79 996.078 L2299.56 995.681 L2299.33 995.703 L2299.11 995.85 L2298.88 995.564 L2298.65 995.609 L2298.42 995.412 L2298.2 995.35 L2297.97 994.997 L2297.74 994.96 L2297.51 995.02 L2297.28 995.291 L2297.06 995.169 L2296.83 994.82 L2296.6 995.282 L2296.37 995.468 L2296.15 995.148 L2295.92 995.179 L2295.69 995.343 L2295.46 995.846 L2295.23 996.241 L2295.01 995.943 L2294.78 996.331 L2294.55 996.531 L2294.32 996.735 L2294.1 996.389 L2293.87 996.433 L2293.64 996.679 L2293.41 997.123 L2293.18 997.241 L2292.96 997.176 L2292.73 997.127 L2292.5 997.467 L2292.27 997.396 L2292.05 996.881 L2291.82 996.734 L2291.59 996.944 L2291.36 996.762 L2291.13 996.847 L2290.91 996.68 L2290.68 996.869 L2290.45 996.931 L2290.22 996.948 L2290 996.694 L2289.77 996.64 L2289.54 996.744 L2289.31 996.383 L2289.08 995.948 L2288.86 995.685 L2288.63 995.645 L2288.4 995.079 L2288.17 994.981 L2287.95 995.149 L2287.72 995.633 L2287.49 995.749 L2287.26 995.826 L2287.03 995.798 L2286.81 995.46 L2286.58 995.426 L2286.35 995.767 L2286.12 995.949 L2285.9 996.096 L2285.67 996.201 L2285.44 996.118 L2285.21 996.563 L2284.98 996.717 L2284.76 997.063 L2284.53 997.142 L2284.3 997.577 L2284.07 997.464 L2283.85 997.431 L2283.62 997.559 L2283.39 997.626 L2283.16 998.62 L2282.93 998.461 L2282.71 998.539 L2282.48 998.518 L2282.25 999.498 L2282.02 999.913 L2281.8 1000.2 L2281.57 1000.29 L2281.34 999.752 L2281.11 999.564 L2280.88 1000.04 L2280.66 999.87 L2280.43 1000.1 L2280.2 999.919 L2279.97 1000.06 L2279.75 1000.17 L2279.52 999.858 L2279.29 999.874 L2279.06 1000.11 L2278.83 999.799 L2278.61 999.731 L2278.38 999.164 L2278.15 999.136 L2277.92 999.453 L2277.7 999.384 L2277.47 999.542 L2277.24 999.729 L2277.01 999.374 L2276.78 999.835 L2276.56 999.907 L2276.33 1000.32 L2276.1 1000.23 L2275.87 1000.72 L2275.65 1000.5 L2275.42 1000.26 L2275.19 1000.71 L2274.96 1000.68 L2274.74 1000.87 L2274.51 1001.15 L2274.28 1001.51 L2274.05 1001.21 L2273.82 1001.04 L2273.6 1001.33 L2273.37 1001.29 L2273.14 1001.07 L2272.91 1001.13 L2272.69 1001.12 L2272.46 1001.2 L2272.23 1001.29 L2272 1000.85 L2271.77 1000.78 L2271.55 1000.44 L2271.32 1000.61 L2271.09 1000.26 L2270.86 1000.28 L2270.64 1000.65 L2270.41 1000.7 L2270.18 1000.59 L2269.95 1000.57 L2269.72 1000.9 L2269.5 1000.92 L2269.27 1000.85 L2269.04 1001.1 L2268.81 1001.32 L2268.59 1001.35 L2268.36 1001.33 L2268.13 1000.8 L2267.9 1000.89 L2267.67 1001.01 L2267.45 1001.21 L2267.22 1001.45 L2266.99 1000.95 L2266.76 1000.89 L2266.54 1000.76 L2266.31 1000.89 L2266.08 1000.78 L2265.85 1001.04 L2265.62 1000.71 L2265.4 1000.48 L2265.17 1000.45 L2264.94 1000.65 L2264.71 1000.5 L2264.49 1000.28 L2264.26 1000.24 L2264.03 999.91 L2263.8 999.832 L2263.57 999.873 L2263.35 999.978 L2263.12 999.683 L2262.89 999.939 L2262.66 999.343 L2262.44 999.313 L2262.21 1000 L2261.98 1000.05 L2261.75 1000.08 L2261.52 1000.36 L2261.3 1000.78 L2261.07 1001.69 L2260.84 1001.74 L2260.61 1001.66 L2260.39 1001.8 L2260.16 1001.69 L2259.93 1002.64 L2259.7 1002.47 L2259.47 1002.6 L2259.25 1002.33 L2259.02 1002.47 L2258.79 1002.2 L2258.56 1002.25 L2258.34 1002.33 L2258.11 1002.6 L2257.88 1003.44 L2257.65 1003.39 L2257.42 1003.24 L2257.2 1003.32 L2256.97 1003 L2256.74 1003.52 L2256.51 1003.66 L2256.29 1003.29 L2256.06 1003.4 L2255.83 1003.24 L2255.6 1002.82 L2255.37 1002.91 L2255.15 1003.23 L2254.92 1002.86 L2254.69 1002.94 L2254.46 1002.79 L2254.24 1002.51 L2254.01 1002.41 L2253.78 1002.38 L2253.55 1002.16 L2253.32 1002.22 L2253.1 1001.78 L2252.87 1001.46 L2252.64 1001.68 L2252.41 1001.5 L2252.19 1001.47 L2251.96 1001.4 L2251.73 1001.21 L2251.5 1000.64 L2251.27 1000.4 L2251.05 1000.22 L2250.82 1000.56 L2250.59 1000.51 L2250.36 1000.21 L2250.14 1000.09 L2249.91 1000.5 L2249.68 1000.57 L2249.45 1000.63 L2249.22 1000.21 L2249 1000.4 L2248.77 1000.2 L2248.54 1000.46 L2248.31 1000.86 L2248.09 1001.31 L2247.86 1001.8 L2247.63 1001.76 L2247.4 1001.34 L2247.17 1001.38 L2246.95 1000.86 L2246.72 1000.72 L2246.49 1000.67 L2246.26 1000.83 L2246.04 1000.87 L2245.81 1001.21 L2245.58 1001.31 L2245.35 1001.27 L2245.12 1001.96 L2244.9 1001.71 L2244.67 1001.38 L2244.44 1002.24 L2244.21 1002 L2243.99 1001.6 L2243.76 1001.69 L2243.53 1001.66 L2243.3 1001.91 L2243.07 1002.64 L2242.85 1002.81 L2242.62 1003.12 L2242.39 1002.58 L2242.16 1002.36 L2241.94 1003 L2241.71 1003.01 L2241.48 1003.6 L2241.25 1003.45 L2241.02 1003.56 L2240.8 1003.12 L2240.57 1003.25 L2240.34 1002.93 L2240.11 1002.62 L2239.89 1003.16 L2239.66 1003.16 L2239.43 1003.55 L2239.2 1003.42 L2238.98 1002.96 L2238.75 1003.38 L2238.52 1003.35 L2238.29 1003.14 L2238.06 1003.16 L2237.84 1003.02 L2237.61 1002.8 L2237.38 1002.61 L2237.15 1002.85 L2236.93 1002.4 L2236.7 1002.7 L2236.47 1003.01 L2236.24 1003.25 L2236.01 1003.24 L2235.79 1002.65 L2235.56 1002.53 L2235.33 1002.16 L2235.1 1002.06 L2234.88 1002.31 L2234.65 1002.33 L2234.42 1002.31 L2234.19 1002.17 L2233.96 1001.92 L2233.74 1002.12 L2233.51 1002.06 L2233.28 1002.06 L2233.05 1002.26 L2232.83 1002.28 L2232.6 1002.27 L2232.37 1003 L2232.14 1002.58 L2231.91 1002.12 L2231.69 1002.11 L2231.46 1002.02 L2231.23 1001.85 L2231 1001.77 L2230.78 1001.55 L2230.55 1001.67 L2230.32 1001.65 L2230.09 1001.28 L2229.86 1001.4 L2229.64 1001.15 L2229.41 1001.05 L2229.18 1000.72 L2228.95 1000.28 L2228.73 1000.06 L2228.5 1000.06 L2228.27 1000.44 L2228.04 1000.47 L2227.81 1000.79 L2227.59 1001.03 L2227.36 1000.94 L2227.13 1001.41 L2226.9 1001.19 L2226.68 1001.19 L2226.45 1001.04 L2226.22 1001.27 L2225.99 1001.08 L2225.76 1001.39 L2225.54 1001.41 L2225.31 1001.84 L2225.08 1002.44 L2224.85 1002.3 L2224.63 1002.19 L2224.4 1002.15 L2224.17 1002.01 L2223.94 1001.86 L2223.71 1001.92 L2223.49 1001.75 L2223.26 1001.49 L2223.03 1001.52 L2222.8 1001.33 L2222.58 1001.49 L2222.35 1001.96 L2222.12 1001.78 L2221.89 1001.82 L2221.66 1001.93 L2221.44 1001.91 L2221.21 1002.24 L2220.98 1002.67 L2220.75 1002.39 L2220.53 1003.04 L2220.3 1002.81 L2220.07 1002.84 L2219.84 1003.19 L2219.61 1002.65 L2219.39 1002.58 L2219.16 1002.56 L2218.93 1002.73 L2218.7 1002.77 L2218.48 1002.39 L2218.25 1002.59 L2218.02 1002.25 L2217.79 1002.44 L2217.56 1002.52 L2217.34 1002.47 L2217.11 1002.03 L2216.88 1002.04 L2216.65 1002.31 L2216.43 1002.23 L2216.2 1002.74 L2215.97 1002.71 L2215.74 1002.62 L2215.51 1002.67 L2215.29 1002.68 L2215.06 1002.43 L2214.83 1002.9 L2214.6 1002.52 L2214.38 1002.66 L2214.15 1002.56 L2213.92 1002.25 L2213.69 1003.05 L2213.46 1003.92 L2213.24 1004.03 L2213.01 1003.78 L2212.78 1003.57 L2212.55 1003.32 L2212.33 1003.2 L2212.1 1003.53 L2211.87 1003.85 L2211.64 1004.08 L2211.41 1004.19 L2211.19 1003.95 L2210.96 1004.07 L2210.73 1004.48 L2210.5 1004.31 L2210.28 1004.08 L2210.05 1004.56 L2209.82 1004.78 L2209.59 1004.83 L2209.36 1004.63 L2209.14 1004.89 L2208.91 1005.3 L2208.68 1005.11 L2208.45 1005.15 L2208.23 1005.1 L2208 1004.7 L2207.77 1004.21 L2207.54 1004.78 L2207.31 1004.58 L2207.09 1004.45 L2206.86 1004.61 L2206.63 1004.71 L2206.4 1005.06 L2206.18 1004.97 L2205.95 1004.31 L2205.72 1004.75 L2205.49 1004.68 L2205.26 1004.77 L2205.04 1004.68 L2204.81 1004.51 L2204.58 1004.18 L2204.35 1003.61 L2204.13 1004.31 L2203.9 1003.9 L2203.67 1004.39 L2203.44 1004.51 L2203.21 1003.92 L2202.99 1003.62 L2202.76 1003.46 L2202.53 1003.6 L2202.3 1003.84 L2202.08 1003.57 L2201.85 1003.33 L2201.62 1002.94 L2201.39 1002.9 L2201.17 1003.03 L2200.94 1002.46 L2200.71 1002.12 L2200.48 1002.09 L2200.25 1002.31 L2200.03 1002.42 L2199.8 1002.45 L2199.57 1002.63 L2199.34 1002.28 L2199.12 1002.48 L2198.89 1002.61 L2198.66 1002.57 L2198.43 1002.39 L2198.2 1002.11 L2197.98 1001.88 L2197.75 1001.97 L2197.52 1001.84 L2197.29 1002.14 L2197.07 1001.69 L2196.84 1002.47 L2196.61 1002.15 L2196.38 1002.13 L2196.15 1002.09 L2195.93 1002.58 L2195.7 1002.45 L2195.47 1002.49 L2195.24 1002.4 L2195.02 1002.82 L2194.79 1002.74 L2194.56 1003.26 L2194.33 1003.18 L2194.1 1002.93 L2193.88 1003.18 L2193.65 1002.9 L2193.42 1002.4 L2193.19 1003.15 L2192.97 1002.67 L2192.74 1002.94 L2192.51 1002.82 L2192.28 1002.83 L2192.05 1003.13 L2191.83 1003.28 L2191.6 1002.76 L2191.37 1002.82 L2191.14 1002.5 L2190.92 1002.31 L2190.69 1002.25 L2190.46 1002.31 L2190.23 1002.3 L2190 1002.47 L2189.78 1002.33 L2189.55 1002.51 L2189.32 1002.33 L2189.09 1002.39 L2188.87 1002.23 L2188.64 1002.45 L2188.41 1002.83 L2188.18 1003.41 L2187.95 1003.46 L2187.73 1003.39 L2187.5 1003.32 L2187.27 1002.93 L2187.04 1002.67 L2186.82 1002.41 L2186.59 1002.31 L2186.36 1002.23 L2186.13 1002.35 L2185.9 1002.78 L2185.68 1002.63 L2185.45 1001.97 L2185.22 1001.67 L2184.99 1001.69 L2184.77 1001.59 L2184.54 1001.45 L2184.31 1001.25 L2184.08 1000.9 L2183.85 1000.75 L2183.63 1000.48 L2183.4 1000.21 L2183.17 1000.06 L2182.94 1000.23 L2182.72 1000.37 L2182.49 1000.45 L2182.26 1001.26 L2182.03 1001.23 L2181.8 1001.67 L2181.58 1001.32 L2181.35 1001.31 L2181.12 1001.28 L2180.89 1002.05 L2180.67 1001.52 L2180.44 1001.23 L2180.21 1000.96 L2179.98 1001.34 L2179.75 1001.78 L2179.53 1001.92 L2179.3 1001.99 L2179.07 1001.54 L2178.84 1001.78 L2178.62 1001.77 L2178.39 1001.61 L2178.16 1001.94 L2177.93 1001.97 L2177.7 1001.81 L2177.48 1001.77 L2177.25 1001.16 L2177.02 1001.05 L2176.79 1001.24 L2176.57 1000.57 L2176.34 1000.68 L2176.11 1000.35 L2175.88 999.952 L2175.65 999.959 L2175.43 1000.24 L2175.2 999.601 L2174.97 999.832 L2174.74 999.366 L2174.52 999.107 L2174.29 999.318 L2174.06 999.076 L2173.83 998.684 L2173.6 998.686 L2173.38 998.978 L2173.15 999.208 L2172.92 999.444 L2172.69 999.596 L2172.47 999.625 L2172.24 999.34 L2172.01 999.495 L2171.78 999.773 L2171.55 999.565 L2171.33 999.496 L2171.1 999.437 L2170.87 999.548 L2170.64 999.47 L2170.42 999.142 L2170.19 999.19 L2169.96 999.316 L2169.73 999.61 L2169.5 999.658 L2169.28 999.529 L2169.05 998.996 L2168.82 999.557 L2168.59 1000.22 L2168.37 1000.34 L2168.14 1000.41 L2167.91 1000.44 L2167.68 1000.29 L2167.45 999.86 L2167.23 1000.07 L2167 999.759 L2166.77 1000.03 L2166.54 999.683 L2166.32 999.276 L2166.09 999.026 L2165.86 998.938 L2165.63 999.043 L2165.41 998.961 L2165.18 999.493 L2164.95 999.742 L2164.72 1000.43 L2164.49 1000.38 L2164.27 1000.27 L2164.04 999.895 L2163.81 1000.02 L2163.58 999.812 L2163.36 999.873 L2163.13 1000.33 L2162.9 1000.9 L2162.67 1000.93 L2162.44 1000.99 L2162.22 1001.11 L2161.99 1001.06 L2161.76 1001.04 L2161.53 1001.32 L2161.31 1001.28 L2161.08 1001.46 L2160.85 1002.16 L2160.62 1002.34 L2160.39 1003.44 L2160.17 1003.12 L2159.94 1003.52 L2159.71 1003.47 L2159.48 1003.05 L2159.26 1003.31 L2159.03 1003.92 L2158.8 1004.35 L2158.57 1004.85 L2158.34 1004.95 L2158.12 1005.07 L2157.89 1005.1 L2157.66 1006.28 L2157.43 1006.64 L2157.21 1006.35 L2156.98 1005.91 L2156.75 1005.86 L2156.52 1006.09 L2156.29 1006.04 L2156.07 1005.57 L2155.84 1005.43 L2155.61 1005.39 L2155.38 1005.23 L2155.16 1005.66 L2154.93 1005.63 L2154.7 1006.25 L2154.47 1006.76 L2154.24 1006.37 L2154.02 1006.15 L2153.79 1005.77 L2153.56 1005.55 L2153.33 1006.11 L2153.11 1005.97 L2152.88 1005.85 L2152.65 1005.87 L2152.42 1006.05 L2152.19 1005.93 L2151.97 1005.9 L2151.74 1005.86 L2151.51 1005.61 L2151.28 1005.5 L2151.06 1005.14 L2150.83 1004.8 L2150.6 1004.85 L2150.37 1004.64 L2150.14 1005.41 L2149.92 1005.91 L2149.69 1005.74 L2149.46 1005.59 L2149.23 1005.28 L2149.01 1005.95 L2148.78 1005.56 L2148.55 1005.21 L2148.32 1004.74 L2148.09 1005.66 L2147.87 1005.86 L2147.64 1005.65 L2147.41 1005.28 L2147.18 1005.24 L2146.96 1005.16 L2146.73 1005.4 L2146.5 1005.15 L2146.27 1005.35 L2146.04 1005.32 L2145.82 1004.78 L2145.59 1004.75 L2145.36 1004.61 L2145.13 1004.79 L2144.91 1004.41 L2144.68 1004.86 L2144.45 1004.9 L2144.22 1004.39 L2143.99 1004.37 L2143.77 1004.42 L2143.54 1004.59 L2143.31 1005.18 L2143.08 1005.13 L2142.86 1005.5 L2142.63 1005.18 L2142.4 1005.3 L2142.17 1005.14 L2141.94 1005.76 L2141.72 1005.46 L2141.49 1005.05 L2141.26 1004.9 L2141.03 1004.77 L2140.81 1004.69 L2140.58 1004.87 L2140.35 1004.92 L2140.12 1004.64 L2139.89 1004.82 L2139.67 1005.14 L2139.44 1005.23 L2139.21 1005.42 L2138.98 1005.68 L2138.76 1005.68 L2138.53 1005.81 L2138.3 1005.68 L2138.07 1005.65 L2137.84 1005.47 L2137.62 1004.8 L2137.39 1004.82 L2137.16 1004.86 L2136.93 1004.87 L2136.71 1004.7 L2136.48 1004.66 L2136.25 1004.59 L2136.02 1004.19 L2135.79 1004.58 L2135.57 1004.21 L2135.34 1004.27 L2135.11 1004.72 L2134.88 1004.86 L2134.66 1004.87 L2134.43 1004.32 L2134.2 1004.31 L2133.97 1004.3 L2133.74 1004.86 L2133.52 1004.69 L2133.29 1004.75 L2133.06 1005.34 L2132.83 1005.2 L2132.61 1004.49 L2132.38 1005.04 L2132.15 1004.64 L2131.92 1004.5 L2131.69 1004.74 L2131.47 1004.64 L2131.24 1004.51 L2131.01 1004.1 L2130.78 1004.02 L2130.56 1003.92 L2130.33 1003.63 L2130.1 1004.22 L2129.87 1004.09 L2129.65 1003.77 L2129.42 1003.35 L2129.19 1003.51 L2128.96 1003.45 L2128.73 1003.83 L2128.51 1003.56 L2128.28 1003.35 L2128.05 1003.59 L2127.82 1003.79 L2127.6 1004.11 L2127.37 1003.9 L2127.14 1003.75 L2126.91 1003.66 L2126.68 1003.44 L2126.46 1003.19 L2126.23 1003.16 L2126 1003 L2125.77 1002.93 L2125.55 1003.32 L2125.32 1003.26 L2125.09 1003.54 L2124.86 1003.97 L2124.63 1004.09 L2124.41 1004.8 L2124.18 1004.97 L2123.95 1004.6 L2123.72 1003.92 L2123.5 1003.5 L2123.27 1003.88 L2123.04 1003.84 L2122.81 1004.3 L2122.58 1004.63 L2122.36 1004.74 L2122.13 1004.33 L2121.9 1004.03 L2121.67 1004.22 L2121.45 1003.96 L2121.22 1004.36 L2120.99 1003.9 L2120.76 1003.66 L2120.53 1003.31 L2120.31 1003.65 L2120.08 1003.33 L2119.85 1003.32 L2119.62 1003.14 L2119.4 1002.95 L2119.17 1002.94 L2118.94 1003.24 L2118.71 1003.38 L2118.48 1003.19 L2118.26 1003.14 L2118.03 1003.01 L2117.8 1002.97 L2117.57 1002.71 L2117.35 1002.73 L2117.12 1002.33 L2116.89 1002.15 L2116.66 1002.05 L2116.43 1001.99 L2116.21 1001.82 L2115.98 1002.55 L2115.75 1002.24 L2115.52 1002.09 L2115.3 1002.24 L2115.07 1002.68 L2114.84 1002.58 L2114.61 1002.74 L2114.38 1002.38 L2114.16 1002.55 L2113.93 1001.88 L2113.7 1001.78 L2113.47 1001.64 L2113.25 1001.25 L2113.02 1001.24 L2112.79 1001.72 L2112.56 1001.36 L2112.33 1001.03 L2112.11 1001.54 L2111.88 1001.43 L2111.65 1001.86 L2111.42 1001.44 L2111.2 1001.5 L2110.97 1001.68 L2110.74 1001.3 L2110.51 1001.26 L2110.28 1001.32 L2110.06 1001.05 L2109.83 1001.42 L2109.6 1001.61 L2109.37 1001.41 L2109.15 1001.13 L2108.92 1000.98 L2108.69 1000.95 L2108.46 1001.05 L2108.23 1001.18 L2108.01 1000.92 L2107.78 1001.47 L2107.55 1001.24 L2107.32 1001.37 L2107.1 1001.63 L2106.87 1001.19 L2106.64 1001.68 L2106.41 1001.77 L2106.18 1001.4 L2105.96 1000.82 L2105.73 1001.45 L2105.5 1001.89 L2105.27 1001.57 L2105.05 1002.29 L2104.82 1002.11 L2104.59 1002.16 L2104.36 1002.06 L2104.13 1001.63 L2103.91 1001.74 L2103.68 1001.86 L2103.45 1002.1 L2103.22 1002.56 L2103 1002.48 L2102.77 1002.56 L2102.54 1002.66 L2102.31 1003 L2102.08 1002.88 L2101.86 1002.92 L2101.63 1003.38 L2101.4 1003.39 L2101.17 1003.15 L2100.95 1003.42 L2100.72 1003.41 L2100.49 1004.01 L2100.26 1004.24 L2100.03 1003.95 L2099.81 1003.69 L2099.58 1004.12 L2099.35 1004.58 L2099.12 1005.24 L2098.9 1005.43 L2098.67 1005.23 L2098.44 1005.37 L2098.21 1004.99 L2097.98 1004.89 L2097.76 1004.61 L2097.53 1004.72 L2097.3 1004.83 L2097.07 1004.76 L2096.85 1004.56 L2096.62 1004.14 L2096.39 1004.38 L2096.16 1003.86 L2095.93 1003.6 L2095.71 1003.36 L2095.48 1002.76 L2095.25 1002.78 L2095.02 1002.61 L2094.8 1002.22 L2094.57 1001.68 L2094.34 1001.69 L2094.11 1001.39 L2093.89 1000.94 L2093.66 1001.61 L2093.43 1001.39 L2093.2 1000.98 L2092.97 1000.99 L2092.75 1001.12 L2092.52 1001.08 L2092.29 1000.59 L2092.06 1001.3 L2091.84 1001.09 L2091.61 1001.14 L2091.38 1001.9 L2091.15 1002.02 L2090.92 1002.17 L2090.7 1002.17 L2090.47 1001.88 L2090.24 1002.35 L2090.01 1001.89 L2089.79 1001.67 L2089.56 1001.8 L2089.33 1001.56 L2089.1 1001.45 L2088.87 1001.19 L2088.65 1000.67 L2088.42 1000.41 L2088.19 1000.53 L2087.96 1000.88 L2087.74 1001.06 L2087.51 1001.06 L2087.28 1000.36 L2087.05 1000.15 L2086.82 999.739 L2086.6 999.478 L2086.37 999.636 L2086.14 999.719 L2085.91 999.592 L2085.69 999.622 L2085.46 999.626 L2085.23 1000.46 L2085 1000.36 L2084.77 1000.16 L2084.55 999.902 L2084.32 999.434 L2084.09 999.53 L2083.86 999.502 L2083.64 999.757 L2083.41 1000.55 L2083.18 1000.54 L2082.95 1000.59 L2082.72 1000.6 L2082.5 1000.89 L2082.27 1000.79 L2082.04 1001.17 L2081.81 1000.81 L2081.59 1001.09 L2081.36 1001.54 L2081.13 1001.63 L2080.9 1001.65 L2080.67 1001.72 L2080.45 1001.46 L2080.22 1001.01 L2079.99 1001.19 L2079.76 1001.37 L2079.54 1001.14 L2079.31 1001.05 L2079.08 1001.21 L2078.85 1001.84 L2078.62 1001.87 L2078.4 1002.46 L2078.17 1002.45 L2077.94 1002.78 L2077.71 1002.66 L2077.49 1003.08 L2077.26 1003.26 L2077.03 1003.63 L2076.8 1003.44 L2076.57 1003.27 L2076.35 1002.89 L2076.12 1003.2 L2075.89 1003.2 L2075.66 1002.81 L2075.44 1002.35 L2075.21 1002.34 L2074.98 1002.2 L2074.75 1001.92 L2074.52 1001.79 L2074.3 1001.23 L2074.07 1001.26 L2073.84 1001.22 L2073.61 1001.12 L2073.39 1001.36 L2073.16 1001.02 L2072.93 1001.22 L2072.7 1001.26 L2072.47 1001.3 L2072.25 1000.96 L2072.02 1001.03 L2071.79 1001.16 L2071.56 1001.59 L2071.34 1001.83 L2071.11 1002.22 L2070.88 1001.77 L2070.65 1001.58 L2070.42 1001.36 L2070.2 1001.02 L2069.97 1000.94 L2069.74 1000.59 L2069.51 1001.17 L2069.29 1000.66 L2069.06 1000.32 L2068.83 1000.07 L2068.6 1000.63 L2068.37 1000.53 L2068.15 1000.68 L2067.92 1000.36 L2067.69 1000.47 L2067.46 1000.47 L2067.24 1000.45 L2067.01 1000.48 L2066.78 1000.69 L2066.55 1000.82 L2066.32 1001.28 L2066.1 1001.75 L2065.87 1001.61 L2065.64 1001.97 L2065.41 1001.77 L2065.19 1001.56 L2064.96 1001.24 L2064.73 1001.41 L2064.5 1002.27 L2064.27 1002.46 L2064.05 1002.44 L2063.82 1002.01 L2063.59 1002.3 L2063.36 1001.8 L2063.14 1001.49 L2062.91 1001.8 L2062.68 1001.51 L2062.45 1001.18 L2062.22 1001.09 L2062 1001.43 L2061.77 1000.98 L2061.54 1000.87 L2061.31 1000.61 L2061.09 1000.36 L2060.86 1000.99 L2060.63 1000.77 L2060.4 1001.19 L2060.17 1000.8 L2059.95 1001.34 L2059.72 1001.03 L2059.49 1001.06 L2059.26 1001.1 L2059.04 1001.21 L2058.81 1000.98 L2058.58 1001.33 L2058.35 1000.97 L2058.12 1001.57 L2057.9 1001.85 L2057.67 1001.3 L2057.44 1001.68 L2057.21 1001.84 L2056.99 1001.85 L2056.76 1001.9 L2056.53 1001.81 L2056.3 1002.29 L2056.08 1002.14 L2055.85 1002.58 L2055.62 1002.7 L2055.39 1002.5 L2055.16 1002.4 L2054.94 1002.37 L2054.71 1002.33 L2054.48 1002.44 L2054.25 1001.91 L2054.03 1001.73 L2053.8 1001.53 L2053.57 1001.68 L2053.34 1001.77 L2053.11 1001.63 L2052.89 1002.12 L2052.66 1001.53 L2052.43 1001.41 L2052.2 1001.79 L2051.98 1001.4 L2051.75 1001.12 L2051.52 1001.21 L2051.29 1001.24 L2051.06 1001.54 L2050.84 1001.04 L2050.61 1001.11 L2050.38 1001.73 L2050.15 1001.75 L2049.93 1002.1 L2049.7 1001.86 L2049.47 1002.19 L2049.24 1002.15 L2049.01 1002.41 L2048.79 1002.25 L2048.56 1001.88 L2048.33 1001.79 L2048.1 1001.79 L2047.88 1001.72 L2047.65 1001.95 L2047.42 1001.62 L2047.19 1001.49 L2046.96 1001.56 L2046.74 1002.24 L2046.51 1001.64 L2046.28 1001.87 L2046.05 1001.82 L2045.83 1002.3 L2045.6 1001.93 L2045.37 1002.34 L2045.14 1001.94 L2044.91 1001.37 L2044.69 1001.76 L2044.46 1001.72 L2044.23 1001.38 L2044 1001.34 L2043.78 1001.15 L2043.55 1000.78 L2043.32 1000.36 L2043.09 1000.21 L2042.86 1000.81 L2042.64 1000.45 L2042.41 1000.13 L2042.18 1000.28 L2041.95 1000.53 L2041.73 1000.73 L2041.5 1000.98 L2041.27 1001.24 L2041.04 1001.12 L2040.81 1001.01 L2040.59 1000.93 L2040.36 1000.58 L2040.13 999.74 L2039.9 999.758 L2039.68 1000.02 L2039.45 999.873 L2039.22 999.904 L2038.99 1000.14 L2038.76 1000.99 L2038.54 1001.66 L2038.31 1001.71 L2038.08 1002.15 L2037.85 1002.35 L2037.63 1001.74 L2037.4 1001.72 L2037.17 1001.63 L2036.94 1001.79 L2036.71 1002.22 L2036.49 1002.79 L2036.26 1002.29 L2036.03 1002.31 L2035.8 1001.99 L2035.58 1001.92 L2035.35 1002.57 L2035.12 1002.17 L2034.89 1002.41 L2034.66 1002.23 L2034.44 1001.82 L2034.21 1001.7 L2033.98 1001.58 L2033.75 1001.47 L2033.53 1001.91 L2033.3 1001.34 L2033.07 1001.49 L2032.84 1001.34 L2032.61 1001.56 L2032.39 1001.56 L2032.16 1001.86 L2031.93 1002.15 L2031.7 1001.98 L2031.48 1002.84 L2031.25 1002.58 L2031.02 1002.66 L2030.79 1002.08 L2030.56 1002.11 L2030.34 1002.22 L2030.11 1001.9 L2029.88 1001.74 L2029.65 1001.18 L2029.43 1001.69 L2029.2 1001.43 L2028.97 1001.73 L2028.74 1002.08 L2028.51 1002.35 L2028.29 1002.85 L2028.06 1003.21 L2027.83 1002.95 L2027.6 1003.36 L2027.38 1003.3 L2027.15 1003.02 L2026.92 1002.56 L2026.69 1002.65 L2026.46 1002.69 L2026.24 1002.59 L2026.01 1002.52 L2025.78 1002.53 L2025.55 1002.47 L2025.33 1002.81 L2025.1 1002.92 L2024.87 1002.37 L2024.64 1002.99 L2024.41 1002.69 L2024.19 1002.55 L2023.96 1002.79 L2023.73 1003.26 L2023.5 1003.17 L2023.28 1003.46 L2023.05 1003.28 L2022.82 1003.36 L2022.59 1003.15 L2022.36 1003.45 L2022.14 1003.67 L2021.91 1004.23 L2021.68 1004.03 L2021.45 1004.28 L2021.23 1004.26 L2021 1004.53 L2020.77 1004.2 L2020.54 1004.4 L2020.32 1004.14 L2020.09 1003.78 L2019.86 1003.51 L2019.63 1003.09 L2019.4 1002.55 L2019.18 1002.15 L2018.95 1002.38 L2018.72 1002.07 L2018.49 1001.91 L2018.27 1002.06 L2018.04 1002.18 L2017.81 1002.05 L2017.58 1002.4 L2017.35 1002.63 L2017.13 1002.56 L2016.9 1002.61 L2016.67 1002.41 L2016.44 1002.15 L2016.22 1002.56 L2015.99 1001.83 L2015.76 1001.76 L2015.53 1001.05 L2015.3 1000.69 L2015.08 1000.5 L2014.85 1000.56 L2014.62 1000.73 L2014.39 1000.12 L2014.17 1000.33 L2013.94 1000.51 L2013.71 999.921 L2013.48 999.638 L2013.25 999.676 L2013.03 999.859 L2012.8 999.682 L2012.57 999.946 L2012.34 999.829 L2012.12 999.546 L2011.89 1000.17 L2011.66 1000.45 L2011.43 1000.15 L2011.2 1000.06 L2010.98 999.962 L2010.75 999.87 L2010.52 1000.33 L2010.29 1000.05 L2010.07 1001.28 L2009.84 1000.92 L2009.61 1001.18 L2009.38 1000.87 L2009.15 1000.93 L2008.93 1000.65 L2008.7 1001.47 L2008.47 1001.51 L2008.24 1001.49 L2008.02 1001.15 L2007.79 1001.23 L2007.56 1000.73 L2007.33 1001.27 L2007.1 1001.22 L2006.88 1000.7 L2006.65 1000.92 L2006.42 1000.87 L2006.19 1000.68 L2005.97 1000.08 L2005.74 1000.06 L2005.51 1000.67 L2005.28 1000.96 L2005.05 1001.17 L2004.83 1001.37 L2004.6 1001.06 L2004.37 1000.93 L2004.14 1000.47 L2003.92 1000.8 L2003.69 1000.82 L2003.46 1000.6 L2003.23 1000.82 L2003 1000.82 L2002.78 1000.32 L2002.55 1000.31 L2002.32 1000.42 L2002.09 1000.29 L2001.87 999.643 L2001.64 999.821 L2001.41 999.927 L2001.18 999.774 L2000.95 999.567 L2000.73 1000.54 L2000.5 1001.12 L2000.27 1001.11 L2000.04 1000.96 L1999.82 1000.91 L1999.59 1000.33 L1999.36 1000.77 L1999.13 1001.66 L1998.9 1001.31 L1998.68 1001.8 L1998.45 1001.61 L1998.22 1001.27 L1997.99 1001.14 L1997.77 1000.72 L1997.54 1001.31 L1997.31 1001.5 L1997.08 1001.17 L1996.85 1001.09 L1996.63 1001.19 L1996.4 1000.68 L1996.17 1000.99 L1995.94 1000.44 L1995.72 1000.42 L1995.49 1001.08 L1995.26 1000.62 L1995.03 1001.05 L1994.8 1000.75 L1994.58 1000.26 L1994.35 1000.42 L1994.12 1000.93 L1993.89 1001.16 L1993.67 1000.59 L1993.44 1000.54 L1993.21 1000.39 L1992.98 1001.03 L1992.75 1001.47 L1992.53 1000.91 L1992.3 1000.58 L1992.07 1000.76 L1991.84 1000.55 L1991.62 1000.77 L1991.39 1000.56 L1991.16 1000.78 L1990.93 1000.39 L1990.7 1000.17 L1990.48 999.921 L1990.25 999.897 L1990.02 999.963 L1989.79 999.566 L1989.57 999.822 L1989.34 999.618 L1989.11 1000.36 L1988.88 1000.38 L1988.65 1000.59 L1988.43 1000.42 L1988.2 1000.46 L1987.97 1000.22 L1987.74 999.8 L1987.52 1000.21 L1987.29 999.833 L1987.06 1000.12 L1986.83 1000.7 L1986.6 1000.18 L1986.38 1000.55 L1986.15 1000.3 L1985.92 1000.45 L1985.69 1000.83 L1985.47 1000.86 L1985.24 1000.74 L1985.01 1000.55 L1984.78 1000.46 L1984.56 1000.33 L1984.33 1000.6 L1984.1 1000.93 L1983.87 1001.11 L1983.64 1001.19 L1983.42 1000.61 L1983.19 1000.91 L1982.96 1000.3 L1982.73 1000.13 L1982.51 1000.48 L1982.28 1000.13 L1982.05 1000.6 L1981.82 1000.61 L1981.59 1000.64 L1981.37 1000.77 L1981.14 1001.06 L1980.91 1000.73 L1980.68 1000.93 L1980.46 1001.42 L1980.23 1001.3 L1980 1001.01 L1979.77 1001.84 L1979.54 1002.59 L1979.32 1002.66 L1979.09 1002.72 L1978.86 1003.08 L1978.63 1003.4 L1978.41 1003.14 L1978.18 1003.46 L1977.95 1003.66 L1977.72 1003.32 L1977.49 1003.01 L1977.27 1003.16 L1977.04 1002.92 L1976.81 1003.19 L1976.58 1003.63 L1976.36 1003.56 L1976.13 1003.88 L1975.9 1003.77 L1975.67 1004.39 L1975.44 1004.64 L1975.22 1005.4 L1974.99 1005.47 L1974.76 1004.83 L1974.53 1004.72 L1974.31 1004.23 L1974.08 1004.15 L1973.85 1004.27 L1973.62 1004.27 L1973.39 1004.84 L1973.17 1005.32 L1972.94 1005.01 L1972.71 1004.81 L1972.48 1004.35 L1972.26 1004.43 L1972.03 1005.14 L1971.8 1005.64 L1971.57 1005.46 L1971.34 1005.32 L1971.12 1005.01 L1970.89 1004.9 L1970.66 1004.87 L1970.43 1004.7 L1970.21 1004.7 L1969.98 1004.88 L1969.75 1004.89 L1969.52 1005.29 L1969.29 1005.27 L1969.07 1005.25 L1968.84 1005.67 L1968.61 1005.66 L1968.38 1005.46 L1968.16 1005.17 L1967.93 1004.9 L1967.7 1005.67 L1967.47 1005.23 L1967.24 1005.16 L1967.02 1004.73 L1966.79 1004.65 L1966.56 1003.9 L1966.33 1003.91 L1966.11 1004.04 L1965.88 1003.75 L1965.65 1003.44 L1965.42 1003.32 L1965.19 1003.26 L1964.97 1003.48 L1964.74 1003.66 L1964.51 1003.42 L1964.28 1003.24 L1964.06 1002.83 L1963.83 1002.87 L1963.6 1003.32 L1963.37 1003.07 L1963.14 1003.18 L1962.92 1003.41 L1962.69 1003.66 L1962.46 1003.49 L1962.23 1002.8 L1962.01 1002.62 L1961.78 1002.97 L1961.55 1002.54 L1961.32 1001.87 L1961.09 1002.12 L1960.87 1002.7 L1960.64 1002.63 L1960.41 1003.16 L1960.18 1003.36 L1959.96 1003.12 L1959.73 1003.52 L1959.5 1003.34 L1959.27 1003.13 L1959.04 1003.4 L1958.82 1003.16 L1958.59 1003.18 L1958.36 1003.15 L1958.13 1003.3 L1957.91 1003.5 L1957.68 1003.11 L1957.45 1002.75 L1957.22 1002.52 L1956.99 1002.17 L1956.77 1002.26 L1956.54 1001.94 L1956.31 1001.9 L1956.08 1001.83 L1955.86 1002.2 L1955.63 1001.88 L1955.4 1001.6 L1955.17 1001.62 L1954.94 1001.25 L1954.72 1001.37 L1954.49 1001.95 L1954.26 1002.05 L1954.03 1001.62 L1953.81 1001.57 L1953.58 1001.19 L1953.35 1001.27 L1953.12 1002.16 L1952.89 1001.93 L1952.67 1001.55 L1952.44 1001.72 L1952.21 1001.53 L1951.98 1001.63 L1951.76 1001.62 L1951.53 1001.74 L1951.3 1001.04 L1951.07 1001.01 L1950.84 1001.44 L1950.62 1001.58 L1950.39 1002.07 L1950.16 1002.02 L1949.93 1001.91 L1949.71 1001.86 L1949.48 1002.05 L1949.25 1001.75 L1949.02 1001.77 L1948.79 1002.04 L1948.57 1001.68 L1948.34 1001.1 L1948.11 1001.5 L1947.88 1001.52 L1947.66 1001.31 L1947.43 1001.24 L1947.2 1000.95 L1946.97 1000.52 L1946.75 1001.15 L1946.52 1000.92 L1946.29 1000.39 L1946.06 1000.6 L1945.83 1001.08 L1945.61 1000.91 L1945.38 1000.44 L1945.15 1000.4 L1944.92 1000.15 L1944.7 1000.26 L1944.47 1000.28 L1944.24 999.801 L1944.01 999.167 L1943.78 998.919 L1943.56 999.32 L1943.33 998.838 L1943.1 998.93 L1942.87 998.726 L1942.65 998.734 L1942.42 998.566 L1942.19 998.434 L1941.96 998.35 L1941.73 998.568 L1941.51 997.894 L1941.28 998.825 L1941.05 998.969 L1940.82 998.464 L1940.6 998.224 L1940.37 998.388 L1940.14 998.245 L1939.91 998 L1939.68 997.859 L1939.46 998.196 L1939.23 998.712 L1939 998.83 L1938.77 999.034 L1938.55 999.282 L1938.32 999.597 L1938.09 999.644 L1937.86 999.936 L1937.63 1000.27 L1937.41 1000.3 L1937.18 1000.53 L1936.95 999.961 L1936.72 999.678 L1936.5 999.897 L1936.27 1000.03 L1936.04 999.967 L1935.81 1000.18 L1935.58 1000.54 L1935.36 1000.21 L1935.13 1000.1 L1934.9 1000.1 L1934.67 1000.05 L1934.45 1000.21 L1934.22 999.982 L1933.99 999.82 L1933.76 999.174 L1933.53 999.281 L1933.31 998.976 L1933.08 999.336 L1932.85 999.244 L1932.62 998.916 L1932.4 999.05 L1932.17 998.908 L1931.94 998.493 L1931.71 998.42 L1931.48 998.426 L1931.26 997.938 L1931.03 998.281 L1930.8 998.074 L1930.57 998.519 L1930.35 998.151 L1930.12 998.001 L1929.89 997.79 L1929.66 997.819 L1929.43 997.513 L1929.21 997.882 L1928.98 997.36 L1928.75 997.645 L1928.52 997.588 L1928.3 997.442 L1928.07 997.875 L1927.84 997.889 L1927.61 997.3 L1927.38 997.865 L1927.16 998.101 L1926.93 997.477 L1926.7 997.905 L1926.47 998.709 L1926.25 998.149 L1926.02 997.639 L1925.79 997.629 L1925.56 997.962 L1925.33 998.381 L1925.11 998.133 L1924.88 997.829 L1924.65 998.167 L1924.42 997.994 L1924.2 998.238 L1923.97 998.417 L1923.74 998.953 L1923.51 999.045 L1923.28 998.235 L1923.06 998.15 L1922.83 997.382 L1922.6 997.337 L1922.37 996.79 L1922.15 996.974 L1921.92 996.949 L1921.69 997.264 L1921.46 997.045 L1921.23 996.23 L1921.01 996.441 L1920.78 996.938 L1920.55 996.556 L1920.32 996.572 L1920.1 996.396 L1919.87 996.79 L1919.64 996.111 L1919.41 996.154 L1919.18 996.33 L1918.96 996.121 L1918.73 996.135 L1918.5 995.957 L1918.27 995.957 L1918.05 995.845 L1917.82 995.596 L1917.59 995.996 L1917.36 995.825 L1917.13 995.56 L1916.91 995.29 L1916.68 995.729 L1916.45 995.693 L1916.22 995.251 L1916 994.966 L1915.77 994.855 L1915.54 995.182 L1915.31 995.172 L1915.08 995.821 L1914.86 996.154 L1914.63 996.108 L1914.4 995.865 L1914.17 995.521 L1913.95 995.277 L1913.72 995.525 L1913.49 995.146 L1913.26 995.141 L1913.03 995.443 L1912.81 995.323 L1912.58 995.041 L1912.35 994.817 L1912.12 995.757 L1911.9 995.642 L1911.67 995.654 L1911.44 995.669 L1911.21 996.421 L1910.99 995.812 L1910.76 995.72 L1910.53 995.149 L1910.3 996.125 L1910.07 995.401 L1909.85 994.819 L1909.62 994.928 L1909.39 994.757 L1909.16 994.285 L1908.94 994.031 L1908.71 994.662 L1908.48 994.321 L1908.25 994.399 L1908.02 994.235 L1907.8 993.924 L1907.57 993.938 L1907.34 994.131 L1907.11 994.501 L1906.89 994.087 L1906.66 993.844 L1906.43 994.186 L1906.2 993.83 L1905.97 994.894 L1905.75 994.59 L1905.52 994.838 L1905.29 995.389 L1905.06 995.52 L1904.84 995.596 L1904.61 995.24 L1904.38 995.124 L1904.15 994.88 L1903.92 994.556 L1903.7 994.476 L1903.47 994.687 L1903.24 995.159 L1903.01 995.365 L1902.79 995.449 L1902.56 995.768 L1902.33 995.421 L1902.1 995.022 L1901.87 994.509 L1901.65 994.105 L1901.42 994.102 L1901.19 994.174 L1900.96 993.764 L1900.74 993.424 L1900.51 993.35 L1900.28 993.789 L1900.05 994.34 L1899.82 994.621 L1899.6 994.821 L1899.37 994.884 L1899.14 995.42 L1898.91 995.946 L1898.69 996.31 L1898.46 995.913 L1898.23 996.364 L1898 996.605 L1897.77 996.1 L1897.55 996.377 L1897.32 996.528 L1897.09 996.198 L1896.86 996.149 L1896.64 995.989 L1896.41 995.791 L1896.18 995.461 L1895.95 995.323 L1895.72 995.281 L1895.5 995.743 L1895.27 995.402 L1895.04 995.171 L1894.81 995.478 L1894.59 996.018 L1894.36 996.622 L1894.13 996.789 L1893.9 996.624 L1893.67 996.179 L1893.45 996.055 L1893.22 995.956 L1892.99 996.847 L1892.76 996.641 L1892.54 996.358 L1892.31 995.925 L1892.08 995.527 L1891.85 995.287 L1891.62 995.362 L1891.4 995.185 L1891.17 994.804 L1890.94 994.794 L1890.71 994.794 L1890.49 994.336 L1890.26 994.27 L1890.03 994.041 L1889.8 994.355 L1889.57 994.353 L1889.35 995.422 L1889.12 994.97 L1888.89 994.375 L1888.66 994.888 L1888.44 993.89 L1888.21 994.234 L1887.98 994.485 L1887.75 995.24 L1887.52 994.88 L1887.3 995.178 L1887.07 994.952 L1886.84 995.049 L1886.61 994.787 L1886.39 994.851 L1886.16 995.275 L1885.93 994.983 L1885.7 994.668 L1885.47 994.695 L1885.25 995.134 L1885.02 995.353 L1884.79 995.756 L1884.56 995.146 L1884.34 995.328 L1884.11 995.292 L1883.88 995.274 L1883.65 995.494 L1883.42 994.909 L1883.2 995.294 L1882.97 995.921 L1882.74 995.95 L1882.51 996.487 L1882.29 995.676 L1882.06 996.579 L1881.83 997.031 L1881.6 996.87 L1881.37 997.131 L1881.15 996.889 L1880.92 997.376 L1880.69 997.228 L1880.46 997.21 L1880.24 997.734 L1880.01 997.341 L1879.78 996.576 L1879.55 997.15 L1879.32 996.795 L1879.1 996.52 L1878.87 996.145 L1878.64 996.218 L1878.41 996.5 L1878.19 996.608 L1877.96 996.281 L1877.73 995.88 L1877.5 995.744 L1877.27 995.42 L1877.05 995.201 L1876.82 995 L1876.59 994.667 L1876.36 994.132 L1876.14 993.504 L1875.91 993.179 L1875.68 993.205 L1875.45 994.03 L1875.23 994.624 L1875 994.207 L1874.77 994.089 L1874.54 994.189 L1874.31 993.795 L1874.09 994.168 L1873.86 993.986 L1873.63 993.764 L1873.4 994.55 L1873.18 994.282 L1872.95 993.93 L1872.72 993.705 L1872.49 993.524 L1872.26 993.67 L1872.04 993.019 L1871.81 993.662 L1871.58 993.585 L1871.35 993.356 L1871.13 992.903 L1870.9 993.28 L1870.67 993.29 L1870.44 993.173 L1870.21 992.521 L1869.99 992.988 L1869.76 993.05 L1869.53 993.117 L1869.3 992.897 L1869.08 992.69 L1868.85 992.391 L1868.62 992.029 L1868.39 991.903 L1868.16 992.058 L1867.94 993.44 L1867.71 992.893 L1867.48 993.522 L1867.25 993.044 L1867.03 993.21 L1866.8 993.806 L1866.57 992.994 L1866.34 992.815 L1866.11 992.272 L1865.89 992.227 L1865.66 991.987 L1865.43 991.373 L1865.2 991.662 L1864.98 991.37 L1864.75 991.388 L1864.52 992.019 L1864.29 991.664 L1864.06 991.835 L1863.84 992.055 L1863.61 991.719 L1863.38 991.732 L1863.15 991.662 L1862.93 991.682 L1862.7 991.819 L1862.47 992.124 L1862.24 992.063 L1862.01 991.714 L1861.79 991.336 L1861.56 991.872 L1861.33 991.547 L1861.1 991.895 L1860.88 992.011 L1860.65 992.153 L1860.42 992.07 L1860.19 991.889 L1859.96 992.021 L1859.74 992.157 L1859.51 991.498 L1859.28 991.178 L1859.05 990.681 L1858.83 990.249 L1858.6 990.213 L1858.37 990.55 L1858.14 991.217 L1857.91 991.471 L1857.69 991.574 L1857.46 991.611 L1857.23 992.184 L1857 992.645 L1856.78 992.323 L1856.55 992.363 L1856.32 991.917 L1856.09 992.705 L1855.86 992.529 L1855.64 992.839 L1855.41 992.541 L1855.18 992.061 L1854.95 991.861 L1854.73 992.849 L1854.5 992.641 L1854.27 992.696 L1854.04 992.359 L1853.81 992.184 L1853.59 992.556 L1853.36 992.08 L1853.13 992.1 L1852.9 992.622 L1852.68 992.693 L1852.45 992.665 L1852.22 992.745 L1851.99 992.482 L1851.76 992.279 L1851.54 991.998 L1851.31 991.101 L1851.08 991.056 L1850.85 991.946 L1850.63 992.321 L1850.4 992.194 L1850.17 991.938 L1849.94 991.493 L1849.71 991.716 L1849.49 991.872 L1849.26 991.585 L1849.03 991.676 L1848.8 991.955 L1848.58 991.928 L1848.35 992.543 L1848.12 992.58 L1847.89 992.657 L1847.66 992.694 L1847.44 992.905 L1847.21 992.705 L1846.98 992.648 L1846.75 992.537 L1846.53 992.165 L1846.3 992.462 L1846.07 993.417 L1845.84 993.628 L1845.61 993.763 L1845.39 993.62 L1845.16 993.293 L1844.93 993.142 L1844.7 993.115 L1844.48 993.537 L1844.25 993.265 L1844.02 992.961 L1843.79 992.474 L1843.56 992.222 L1843.34 992.143 L1843.11 992.503 L1842.88 992.601 L1842.65 992.207 L1842.43 992.506 L1842.2 992.811 L1841.97 992.954 L1841.74 992.716 L1841.51 992.165 L1841.29 992.539 L1841.06 993.006 L1840.83 992.545 L1840.6 992.17 L1840.38 991.667 L1840.15 991.297 L1839.92 991.459 L1839.69 992.3 L1839.46 991.785 L1839.24 992.303 L1839.01 992.087 L1838.78 991.863 L1838.55 991.838 L1838.33 992.09 L1838.1 992.138 L1837.87 992.214 L1837.64 991.611 L1837.42 990.976 L1837.19 990.679 L1836.96 990.621 L1836.73 990.381 L1836.5 990.223 L1836.28 990.092 L1836.05 990.679 L1835.82 990.606 L1835.59 990.067 L1835.37 989.352 L1835.14 989.082 L1834.91 988.571 L1834.68 988.223 L1834.45 987.723 L1834.23 988.04 L1834 988.01 L1833.77 987.764 L1833.54 987.669 L1833.32 988.372 L1833.09 988.512 L1832.86 988.525 L1832.63 988.568 L1832.4 988.831 L1832.18 988.841 L1831.95 989.892 L1831.72 989.771 L1831.49 989.833 L1831.27 989.625 L1831.04 989.439 L1830.81 989.566 L1830.58 989.517 L1830.35 990.326 L1830.13 990.392 L1829.9 990.434 L1829.67 990.456 L1829.44 990.757 L1829.22 990.7 L1828.99 990.571 L1828.76 990.299 L1828.53 990.532 L1828.3 990.4 L1828.08 990.642 L1827.85 990.755 L1827.62 990.717 L1827.39 990.375 L1827.17 991.009 L1826.94 990.89 L1826.71 990.53 L1826.48 989.977 L1826.25 989.652 L1826.03 989.489 L1825.8 990.19 L1825.57 989.743 L1825.34 989.51 L1825.12 989.889 L1824.89 989.489 L1824.66 989.61 L1824.43 989.572 L1824.2 990.121 L1823.98 990.035 L1823.75 991.168 L1823.52 990.749 L1823.29 990.169 L1823.07 990.338 L1822.84 989.977 L1822.61 989.651 L1822.38 989.854 L1822.15 990.033 L1821.93 990.164 L1821.7 989.919 L1821.47 989.694 L1821.24 989.701 L1821.02 990.439 L1820.79 990.74 L1820.56 990.789 L1820.33 990.849 L1820.1 990.941 L1819.88 991.188 L1819.65 991.711 L1819.42 991.531 L1819.19 991.337 L1818.97 991.198 L1818.74 991.878 L1818.51 992.706 L1818.28 992.884 L1818.05 992.126 L1817.83 992.336 L1817.6 991.609 L1817.37 991.509 L1817.14 991.079 L1816.92 991.127 L1816.69 991.208 L1816.46 991.967 L1816.23 991.525 L1816 991.267 L1815.78 991.796 L1815.55 991.517 L1815.32 990.825 L1815.09 989.945 L1814.87 989.483 L1814.64 989.365 L1814.41 989.494 L1814.18 988.796 L1813.95 988.576 L1813.73 988.15 L1813.5 988.045 L1813.27 987.889 L1813.04 987.912 L1812.82 987.755 L1812.59 987.459 L1812.36 988.05 L1812.13 987.87 L1811.9 987.868 L1811.68 988.112 L1811.45 988.082 L1811.22 988.045 L1810.99 987.571 L1810.77 987.558 L1810.54 987.508 L1810.31 986.766 L1810.08 987.269 L1809.85 987.008 L1809.63 987.085 L1809.4 988.061 L1809.17 988.421 L1808.94 988.498 L1808.72 988.023 L1808.49 987.476 L1808.26 987.604 L1808.03 987.889 L1807.8 988.004 L1807.58 987.447 L1807.35 986.622 L1807.12 987.028 L1806.89 987.85 L1806.67 987.734 L1806.44 988.059 L1806.21 988.074 L1805.98 987.885 L1805.75 987.114 L1805.53 986.659 L1805.3 986.784 L1805.07 986.848 L1804.84 986.953 L1804.62 987.225 L1804.39 987.073 L1804.16 986.867 L1803.93 986.508 L1803.7 986.411 L1803.48 986.765 L1803.25 986.41 L1803.02 986.958 L1802.79 986.317 L1802.57 986.167 L1802.34 986.197 L1802.11 986.195 L1801.88 986.044 L1801.66 986.129 L1801.43 985.965 L1801.2 986.017 L1800.97 985.641 L1800.74 985.288 L1800.52 984.636 L1800.29 985.004 L1800.06 985.014 L1799.83 985.017 L1799.61 984.861 L1799.38 984.984 L1799.15 984.398 L1798.92 984.662 L1798.69 984.458 L1798.47 984.858 L1798.24 984.704 L1798.01 984.546 L1797.78 984.665 L1797.56 984.738 L1797.33 984.558 L1797.1 984.512 L1796.87 984.602 L1796.64 984.887 L1796.42 985.418 L1796.19 985.104 L1795.96 985.026 L1795.73 985.126 L1795.51 985.446 L1795.28 985.624 L1795.05 985.156 L1794.82 985.168 L1794.59 985.044 L1794.37 984.864 L1794.14 985.399 L1793.91 985.688 L1793.68 985.544 L1793.46 985.477 L1793.23 985.585 L1793 985.616 L1792.77 985.385 L1792.54 985.65 L1792.32 985.975 L1792.09 985.611 L1791.86 985.844 L1791.63 986.179 L1791.41 985.497 L1791.18 985.807 L1790.95 985.417 L1790.72 985.308 L1790.49 985.067 L1790.27 984.597 L1790.04 985.71 L1789.81 985.15 L1789.58 985.011 L1789.36 986.362 L1789.13 986.336 L1788.9 985.953 L1788.67 985.763 L1788.44 985.337 L1788.22 985.194 L1787.99 985.523 L1787.76 985.108 L1787.53 985.528 L1787.31 985.809 L1787.08 986.339 L1786.85 985.542 L1786.62 985.343 L1786.39 984.506 L1786.17 984.793 L1785.94 984.283 L1785.71 984.895 L1785.48 985.16 L1785.26 984.917 L1785.03 984.983 L1784.8 984.884 L1784.57 984.655 L1784.34 984.883 L1784.12 985.488 L1783.89 985.612 L1783.66 985.467 L1783.43 986.097 L1783.21 985.44 L1782.98 985.111 L1782.75 985.299 L1782.52 985.115 L1782.29 985.465 L1782.07 985.103 L1781.84 985.5 L1781.61 984.982 L1781.38 985.08 L1781.16 985.388 L1780.93 984.686 L1780.7 983.715 L1780.47 983.604 L1780.24 983.711 L1780.02 983.245 L1779.79 983.27 L1779.56 983.231 L1779.33 982.968 L1779.11 983.01 L1778.88 983.555 L1778.65 983.707 L1778.42 984.222 L1778.19 984.07 L1777.97 983.774 L1777.74 983.411 L1777.51 983.05 L1777.28 982.904 L1777.06 982.979 L1776.83 983.801 L1776.6 983.783 L1776.37 983.6 L1776.14 983.775 L1775.92 983.77 L1775.69 983.754 L1775.46 983.66 L1775.23 983.662 L1775.01 983.394 L1774.78 982.87 L1774.55 982.738 L1774.32 982.647 L1774.09 982.963 L1773.87 983.323 L1773.64 983.076 L1773.41 983.12 L1773.18 983.578 L1772.96 983.969 L1772.73 984.003 L1772.5 984.194 L1772.27 984.342 L1772.04 984.401 L1771.82 984.488 L1771.59 984.629 L1771.36 984.802 L1771.13 984.708 L1770.91 984.905 L1770.68 985.121 L1770.45 985.099 L1770.22 984.872 L1769.99 984.451 L1769.77 984.611 L1769.54 984.833 L1769.31 984.434 L1769.08 984.152 L1768.86 984.546 L1768.63 984.417 L1768.4 984.491 L1768.17 984.778 L1767.94 984.379 L1767.72 984.343 L1767.49 985.471 L1767.26 985.105 L1767.03 985.454 L1766.81 985.381 L1766.58 984.8 L1766.35 984.845 L1766.12 984.394 L1765.9 984.043 L1765.67 983.688 L1765.44 983.537 L1765.21 983.42 L1764.98 983.52 L1764.76 983.554 L1764.53 983.391 L1764.3 983.691 L1764.07 983.579 L1763.85 983.18 L1763.62 983.606 L1763.39 983.682 L1763.16 983.462 L1762.93 983.202 L1762.71 983.056 L1762.48 982.698 L1762.25 982.552 L1762.02 982.567 L1761.8 982.802 L1761.57 982.091 L1761.34 981.752 L1761.11 981.963 L1760.88 982.424 L1760.66 982.656 L1760.43 982.446 L1760.2 982.977 L1759.97 982.235 L1759.75 982.785 L1759.52 982.545 L1759.29 982.78 L1759.06 982.104 L1758.83 982.071 L1758.61 982.555 L1758.38 982.256 L1758.15 982.857 L1757.92 982.898 L1757.7 982.92 L1757.47 983.485 L1757.24 983.487 L1757.01 982.903 L1756.78 982.578 L1756.56 982.423 L1756.33 982.374 L1756.1 983.172 L1755.87 982.884 L1755.65 982.481 L1755.42 982.592 L1755.19 982.679 L1754.96 982.693 L1754.73 982.009 L1754.51 981.822 L1754.28 981.384 L1754.05 982.23 L1753.82 981.714 L1753.6 982.285 L1753.37 982.853 L1753.14 982.444 L1752.91 982.23 L1752.68 981.839 L1752.46 982.045 L1752.23 982.121 L1752 982.523 L1751.77 982.603 L1751.55 982.36 L1751.32 982.699 L1751.09 982.555 L1750.86 982.158 L1750.63 981.923 L1750.41 981.396 L1750.18 981.568 L1749.95 981.325 L1749.72 981.133 L1749.5 982.328 L1749.27 982.431 L1749.04 982.747 L1748.81 982.99 L1748.58 982.412 L1748.36 981.969 L1748.13 981.742 L1747.9 982.401 L1747.67 982.687 L1747.45 983.029 L1747.22 983.264 L1746.99 983.64 L1746.76 983.557 L1746.53 983.813 L1746.31 983.353 L1746.08 982.846 L1745.85 983.384 L1745.62 982.717 L1745.4 982.207 L1745.17 981.743 L1744.94 981.05 L1744.71 980.921 L1744.48 980.999 L1744.26 980.938 L1744.03 980.795 L1743.8 980.263 L1743.57 980.001 L1743.35 979.799 L1743.12 980.907 L1742.89 980.526 L1742.66 980.491 L1742.43 980.369 L1742.21 980.193 L1741.98 980.232 L1741.75 979.688 L1741.52 980.676 L1741.3 980.731 L1741.07 980.677 L1740.84 980.312 L1740.61 979.903 L1740.38 979.552 L1740.16 978.89 L1739.93 979.041 L1739.7 978.604 L1739.47 978.474 L1739.25 978.345 L1739.02 978.208 L1738.79 979.006 L1738.56 979.399 L1738.33 978.741 L1738.11 979.029 L1737.88 978.638 L1737.65 978.607 L1737.42 978.5 L1737.2 978.868 L1736.97 978.603 L1736.74 979.31 L1736.51 979.987 L1736.28 980.333 L1736.06 980.164 L1735.83 979.805 L1735.6 980.332 L1735.37 980.114 L1735.15 979.589 L1734.92 980.459 L1734.69 980.42 L1734.46 980.203 L1734.23 980.368 L1734.01 980.4 L1733.78 980.035 L1733.55 980.424 L1733.32 980.66 L1733.1 980.11 L1732.87 980.527 L1732.64 980.661 L1732.41 980.449 L1732.18 980.169 L1731.96 979.926 L1731.73 979.912 L1731.5 979.927 L1731.27 980.005 L1731.05 980.057 L1730.82 980.393 L1730.59 980.264 L1730.36 980.192 L1730.14 980.576 L1729.91 980.057 L1729.68 980.337 L1729.45 979.854 L1729.22 980.091 L1729 980.994 L1728.77 980.734 L1728.54 980.451 L1728.31 980.267 L1728.09 979.753 L1727.86 979.323 L1727.63 979.308 L1727.4 979.493 L1727.17 979.539 L1726.95 980.202 L1726.72 980.676 L1726.49 981.094 L1726.26 981.313 L1726.04 981.158 L1725.81 980.938 L1725.58 981.042 L1725.35 980.631 L1725.12 980.297 L1724.9 980.454 L1724.67 980.195 L1724.44 979.846 L1724.21 979.741 L1723.99 979.532 L1723.76 979.811 L1723.53 979.103 L1723.3 979.57 L1723.07 979.707 L1722.85 980.228 L1722.62 980.048 L1722.39 979.825 L1722.16 980.023 L1721.94 979.514 L1721.71 978.961 L1721.48 979.311 L1721.25 979.54 L1721.02 980.006 L1720.8 979.667 L1720.57 979.605 L1720.34 979.358 L1720.11 978.569 L1719.89 978.537 L1719.66 978.56 L1719.43 978.691 L1719.2 978.835 L1718.97 979.112 L1718.75 979.007 L1718.52 978.877 L1718.29 979.208 L1718.06 979.031 L1717.84 979.592 L1717.61 980.125 L1717.38 980.58 L1717.15 980.255 L1716.92 980.022 L1716.7 980.343 L1716.47 979.945 L1716.24 979.628 L1716.01 979.711 L1715.79 979.435 L1715.56 979.429 L1715.33 979.331 L1715.1 979.196 L1714.87 979.219 L1714.65 979.616 L1714.42 979.249 L1714.19 979.254 L1713.96 980.446 L1713.74 980.682 L1713.51 980.59 L1713.28 980.332 L1713.05 979.835 L1712.82 979.425 L1712.6 980.035 L1712.37 979.579 L1712.14 979.52 L1711.91 978.922 L1711.69 978.308 L1711.46 978.626 L1711.23 979.188 L1711 979.209 L1710.77 980.193 L1710.55 979.92 L1710.32 979.406 L1710.09 979.414 L1709.86 979.453 L1709.64 979.188 L1709.41 979.296 L1709.18 979.398 L1708.95 979.189 L1708.72 978.77 L1708.5 978.619 L1708.27 979.328 L1708.04 978.524 L1707.81 978.276 L1707.59 977.599 L1707.36 977.586 L1707.13 977.606 L1706.9 977.586 L1706.67 977.945 L1706.45 977.79 L1706.22 978.136 L1705.99 977.423 L1705.76 978.28 L1705.54 978.303 L1705.31 977.788 L1705.08 977.512 L1704.85 977.665 L1704.62 978.385 L1704.4 978.531 L1704.17 978.341 L1703.94 977.693 L1703.71 977.219 L1703.49 977.404 L1703.26 977.426 L1703.03 977.53 L1702.8 977.337 L1702.57 977.439 L1702.35 977.118 L1702.12 976.57 L1701.89 977.441 L1701.66 977.686 L1701.44 978.884 L1701.21 978.947 L1700.98 978.593 L1700.75 978.717 L1700.52 978.425 L1700.3 979.059 L1700.07 979.117 L1699.84 978.879 L1699.61 978.669 L1699.39 979.609 L1699.16 979.624 L1698.93 979.813 L1698.7 980.131 L1698.47 979.875 L1698.25 979.7 L1698.02 979.5 L1697.79 979.459 L1697.56 980.222 L1697.34 980.82 L1697.11 980.686 L1696.88 981.155 L1696.65 981.976 L1696.42 981.934 L1696.2 981.679 L1695.97 982.36 L1695.74 981.756 L1695.51 981.064 L1695.29 981.01 L1695.06 981.194 L1694.83 980.98 L1694.6 980.587 L1694.37 979.924 L1694.15 979.885 L1693.92 979.398 L1693.69 979.522 L1693.46 979.199 L1693.24 979.483 L1693.01 979.057 L1692.78 978.697 L1692.55 978.679 L1692.33 979.045 L1692.1 979.288 L1691.87 979.318 L1691.64 979.374 L1691.41 978.932 L1691.19 979.36 L1690.96 979.335 L1690.73 978.742 L1690.5 979.076 L1690.28 978.782 L1690.05 978.321 L1689.82 978.184 L1689.59 977.618 L1689.36 977.381 L1689.14 976.91 L1688.91 976.961 L1688.68 977.389 L1688.45 977.317 L1688.23 977.238 L1688 976.85 L1687.77 976.529 L1687.54 976.808 L1687.31 976.961 L1687.09 977.827 L1686.86 977.882 L1686.63 977.568 L1686.4 977.088 L1686.18 977.189 L1685.95 977.521 L1685.72 977.36 L1685.49 977.687 L1685.26 977.695 L1685.04 977.548 L1684.81 978.431 L1684.58 978.144 L1684.35 978.176 L1684.13 978.301 L1683.9 978.909 L1683.67 978.19 L1683.44 979.226 L1683.21 979.803 L1682.99 979.952 L1682.76 980.482 L1682.53 980.365 L1682.3 979.468 L1682.08 979.726 L1681.85 979.803 L1681.62 980.25 L1681.39 980.195 L1681.16 980.34 L1680.94 980.389 L1680.71 981.256 L1680.48 981.748 L1680.25 982.097 L1680.03 981.958 L1679.8 982.189 L1679.57 982.355 L1679.34 982.962 L1679.11 982.696 L1678.89 982.834 L1678.66 982.67 L1678.43 983.242 L1678.2 982.877 L1677.98 982.762 L1677.75 983.369 L1677.52 983.314 L1677.29 982.633 L1677.06 982.701 L1676.84 982.527 L1676.61 982.435 L1676.38 982.485 L1676.15 983.102 L1675.93 983.089 L1675.7 983.283 L1675.47 983.17 L1675.24 983.552 L1675.01 983.653 L1674.79 983.488 L1674.56 982.884 L1674.33 982.469 L1674.1 982.446 L1673.88 982.496 L1673.65 982.232 L1673.42 982.514 L1673.19 981.871 L1672.96 981.792 L1672.74 981.717 L1672.51 983.017 L1672.28 982.984 L1672.05 983.435 L1671.83 982.881 L1671.6 982.558 L1671.37 982.595 L1671.14 982.357 L1670.91 983.1 L1670.69 982.941 L1670.46 983.536 L1670.23 982.841 L1670 983.156 L1669.78 982.85 L1669.55 982.937 L1669.32 982.465 L1669.09 982.68 L1668.86 982.388 L1668.64 983.136 L1668.41 983.553 L1668.18 983.754 L1667.95 983.703 L1667.73 982.889 L1667.5 983.044 L1667.27 982.53 L1667.04 981.648 L1666.81 982.485 L1666.59 982.237 L1666.36 981.284 L1666.13 981.76 L1665.9 981.928 L1665.68 981.172 L1665.45 980.494 L1665.22 981.066 L1664.99 981.081 L1664.76 980.937 L1664.54 981.56 L1664.31 981.25 L1664.08 981.612 L1663.85 981.496 L1663.63 981.013 L1663.4 980.885 L1663.17 981.175 L1662.94 980.748 L1662.71 981.271 L1662.49 980.546 L1662.26 980.723 L1662.03 980.272 L1661.8 979.905 L1661.58 979.238 L1661.35 979.787 L1661.12 979.229 L1660.89 979.53 L1660.66 979.302 L1660.44 979.705 L1660.21 979.798 L1659.98 979.944 L1659.75 979.67 L1659.53 979.436 L1659.3 978.923 L1659.07 978.33 L1658.84 977.6 L1658.61 977.136 L1658.39 977.265 L1658.16 976.626 L1657.93 976.625 L1657.7 976.712 L1657.48 976.119 L1657.25 975.863 L1657.02 975.912 L1656.79 975.532 L1656.57 975.575 L1656.34 975.801 L1656.11 976.169 L1655.88 975.453 L1655.65 975.331 L1655.43 975.127 L1655.2 974.402 L1654.97 974.135 L1654.74 974.392 L1654.52 974.277 L1654.29 973.912 L1654.06 973.452 L1653.83 974.008 L1653.6 974.217 L1653.38 974.872 L1653.15 974.157 L1652.92 974.399 L1652.69 973.811 L1652.47 974.28 L1652.24 973.925 L1652.01 973.687 L1651.78 973.648 L1651.55 974.42 L1651.33 973.992 L1651.1 974.021 L1650.87 973.186 L1650.64 973.442 L1650.42 973.494 L1650.19 973.556 L1649.96 972.999 L1649.73 972.398 L1649.5 972.305 L1649.28 971.654 L1649.05 971.717 L1648.82 971.452 L1648.59 971.178 L1648.37 970.967 L1648.14 971.181 L1647.91 971.045 L1647.68 970.829 L1647.45 971.092 L1647.23 970.701 L1647 970.082 L1646.77 969.408 L1646.54 968.812 L1646.32 968.849 L1646.09 968.24 L1645.86 968.116 L1645.63 968.014 L1645.4 969.358 L1645.18 969.573 L1644.95 970.071 L1644.72 969.548 L1644.49 970.041 L1644.27 970.068 L1644.04 969.781 L1643.81 969.61 L1643.58 969.531 L1643.35 969.46 L1643.13 969.91 L1642.9 969.424 L1642.67 969.424 L1642.44 969.597 L1642.22 969.319 L1641.99 968.835 L1641.76 968.641 L1641.53 968.534 L1641.3 968.02 L1641.08 967.619 L1640.85 967.958 L1640.62 967.99 L1640.39 967.598 L1640.17 967.659 L1639.94 967.332 L1639.71 966.918 L1639.48 966.374 L1639.25 966.848 L1639.03 967.491 L1638.8 967.352 L1638.57 967.539 L1638.34 967.405 L1638.12 967.385 L1637.89 967.002 L1637.66 967.31 L1637.43 967.095 L1637.2 966.317 L1636.98 966.727 L1636.75 966.429 L1636.52 966.988 L1636.29 967.379 L1636.07 966.566 L1635.84 967.107 L1635.61 967.357 L1635.38 967.467 L1635.15 967.517 L1634.93 967.667 L1634.7 967.057 L1634.47 966.466 L1634.24 966.767 L1634.02 966.432 L1633.79 966.634 L1633.56 966.418 L1633.33 967.082 L1633.1 966.476 L1632.88 966.265 L1632.65 965.827 L1632.42 966.372 L1632.19 966.163 L1631.97 966.55 L1631.74 966.717 L1631.51 966.798 L1631.28 966.081 L1631.05 965.726 L1630.83 965.385 L1630.6 965.162 L1630.37 964.944 L1630.14 965.932 L1629.92 966.133 L1629.69 966.363 L1629.46 966.032 L1629.23 967.412 L1629 967.337 L1628.78 967.382 L1628.55 967.639 L1628.32 967.386 L1628.09 967.546 L1627.87 968.475 L1627.64 969.048 L1627.41 968.486 L1627.18 969.15 L1626.95 968.666 L1626.73 968.635 L1626.5 968.536 L1626.27 967.676 L1626.04 967.573 L1625.82 967.312 L1625.59 966.689 L1625.36 965.761 L1625.13 965.464 L1624.9 965.543 L1624.68 964.874 L1624.45 964.4 L1624.22 964.819 L1623.99 964.473 L1623.77 964.759 L1623.54 965.266 L1623.31 964.877 L1623.08 965.622 L1622.85 964.86 L1622.63 964.729 L1622.4 964.678 L1622.17 964.449 L1621.94 964.213 L1621.72 963.719 L1621.49 963.292 L1621.26 962.662 L1621.03 962.594 L1620.81 962.643 L1620.58 963.52 L1620.35 964.424 L1620.12 964.165 L1619.89 963.752 L1619.67 964.229 L1619.44 963.718 L1619.21 964.211 L1618.98 963.406 L1618.76 963.883 L1618.53 964.159 L1618.3 964.281 L1618.07 964.257 L1617.84 964.322 L1617.62 965.197 L1617.39 964.81 L1617.16 965.237 L1616.93 965.563 L1616.71 965.526 L1616.48 965.024 L1616.25 965.614 L1616.02 965.474 L1615.79 965.402 L1615.57 965.015 L1615.34 964.362 L1615.11 964.025 L1614.88 963.455 L1614.66 963.179 L1614.43 962.555 L1614.2 963.58 L1613.97 963.077 L1613.74 963.154 L1613.52 963.219 L1613.29 963.015 L1613.06 962.889 L1612.83 962.81 L1612.61 962.97 L1612.38 962.935 L1612.15 963.85 L1611.92 963.438 L1611.69 962.936 L1611.47 962.755 L1611.24 963.015 L1611.01 963.607 L1610.78 963.819 L1610.56 963.503 L1610.33 963.702 L1610.1 963.524 L1609.87 963.508 L1609.64 962.857 L1609.42 963.287 L1609.19 963.374 L1608.96 963.74 L1608.73 963.066 L1608.51 962.909 L1608.28 964.548 L1608.05 964.529 L1607.82 964.196 L1607.59 964.297 L1607.37 964.152 L1607.14 965.116 L1606.91 965.06 L1606.68 965.885 L1606.46 966.22 L1606.23 965.618 L1606 965.026 L1605.77 965.204 L1605.54 965.205 L1605.32 965.55 L1605.09 965.204 L1604.86 965.951 L1604.63 965.236 L1604.41 964.917 L1604.18 965.127 L1603.95 965.358 L1603.72 964.857 L1603.49 965.565 L1603.27 964.777 L1603.04 964.632 L1602.81 964.074 L1602.58 963.23 L1602.36 962.848 L1602.13 963.149 L1601.9 962.909 L1601.67 963.888 L1601.44 964.443 L1601.22 964.579 L1600.99 964.775 L1600.76 964.76 L1600.53 965.864 L1600.31 966.09 L1600.08 966.765 L1599.85 966.652 L1599.62 967.054 L1599.39 966.985 L1599.17 967.346 L1598.94 967.313 L1598.71 967.319 L1598.48 967.549 L1598.26 967.923 L1598.03 967.536 L1597.8 967.253 L1597.57 968.494 L1597.34 968.741 L1597.12 968.788 L1596.89 968.579 L1596.66 968.619 L1596.43 968.555 L1596.21 969.409 L1595.98 968.995 L1595.75 969.623 L1595.52 970.187 L1595.29 970.558 L1595.07 970.224 L1594.84 970.139 L1594.61 970.493 L1594.38 970.627 L1594.16 970.922 L1593.93 970.797 L1593.7 970.795 L1593.47 970.855 L1593.24 971.037 L1593.02 970.664 L1592.79 970.012 L1592.56 969.682 L1592.33 969.8 L1592.11 970.217 L1591.88 970.274 L1591.65 970.581 L1591.42 969.956 L1591.19 970.225 L1590.97 969.886 L1590.74 969.533 L1590.51 970.038 L1590.28 970.6 L1590.06 970.134 L1589.83 969.429 L1589.6 969.301 L1589.37 968.476 L1589.14 968.31 L1588.92 967.9 L1588.69 967.068 L1588.46 966.926 L1588.23 967.327 L1588.01 967.516 L1587.78 967.93 L1587.55 968.284 L1587.32 968.001 L1587.09 967.598 L1586.87 967.459 L1586.64 967.353 L1586.41 967.105 L1586.18 967.802 L1585.96 966.965 L1585.73 966.387 L1585.5 966.918 L1585.27 967.017 L1585.04 967.019 L1584.82 966.743 L1584.59 966.79 L1584.36 967.267 L1584.13 966.59 L1583.91 967.615 L1583.68 968.589 L1583.45 967.883 L1583.22 967.347 L1583 966.394 L1582.77 966.2 L1582.54 966.17 L1582.31 966.255 L1582.08 966.53 L1581.86 965.907 L1581.63 966.889 L1581.4 967.392 L1581.17 967.081 L1580.95 966.408 L1580.72 966.204 L1580.49 966.091 L1580.26 966.542 L1580.03 966.756 L1579.81 966.673 L1579.58 966.569 L1579.35 966.792 L1579.12 966.902 L1578.9 967.676 L1578.67 967.962 L1578.44 968.551 L1578.21 968.695 L1577.98 969.168 L1577.76 968.256 L1577.53 968.112 L1577.3 968.331 L1577.07 968.753 L1576.85 968.186 L1576.62 967.905 L1576.39 967.853 L1576.16 967.613 L1575.93 967.242 L1575.71 967.464 L1575.48 967.09 L1575.25 966.69 L1575.02 967.303 L1574.8 967.289 L1574.57 968.24 L1574.34 968.978 L1574.11 968.999 L1573.88 969.17 L1573.66 969.527 L1573.43 969.286 L1573.2 968.63 L1572.97 968.062 L1572.75 967.782 L1572.52 967.192 L1572.29 966.178 L1572.06 966.605 L1571.83 966.633 L1571.61 967.218 L1571.38 967.48 L1571.15 967.964 L1570.92 968.759 L1570.7 968.667 L1570.47 968.386 L1570.24 968.914 L1570.01 969.146 L1569.78 969.564 L1569.56 969.486 L1569.33 968.772 L1569.1 968.946 L1568.87 969.785 L1568.65 969.467 L1568.42 968.62 L1568.19 969.253 L1567.96 969.466 L1567.73 969.66 L1567.51 969.394 L1567.28 969.543 L1567.05 969.635 L1566.82 969.798 L1566.6 970.299 L1566.37 970.386 L1566.14 970.277 L1565.91 970.162 L1565.68 970.121 L1565.46 969.959 L1565.23 969.333 L1565 969.695 L1564.77 968.721 L1564.55 969.261 L1564.32 969.259 L1564.09 969.372 L1563.86 968.68 L1563.63 968.894 L1563.41 969.657 L1563.18 970.063 L1562.95 970.886 L1562.72 970.391 L1562.5 971.054 L1562.27 971.219 L1562.04 971.211 L1561.81 971.852 L1561.58 971.283 L1561.36 970.924 L1561.13 970.587 L1560.9 970.803 L1560.67 971.102 L1560.45 972.056 L1560.22 972.132 L1559.99 971.532 L1559.76 971.712 L1559.53 971.41 L1559.31 971.994 L1559.08 971.95 L1558.85 972.495 L1558.62 971.771 L1558.4 972.341 L1558.17 972.966 L1557.94 973.057 L1557.71 972.965 L1557.48 972.791 L1557.26 972.927 L1557.03 973.025 L1556.8 973.35 L1556.57 973.385 L1556.35 973.401 L1556.12 973.099 L1555.89 972.582 L1555.66 972.059 L1555.43 972.015 L1555.21 972.112 L1554.98 972.393 L1554.75 972.249 L1554.52 972.83 L1554.3 972.551 L1554.07 972.016 L1553.84 971.948 L1553.61 971.67 L1553.38 971.586 L1553.16 971.772 L1552.93 971.538 L1552.7 970.954 L1552.47 971.036 L1552.25 971.34 L1552.02 971.135 L1551.79 971.276 L1551.56 971.111 L1551.33 970.196 L1551.11 969.71 L1550.88 969.673 L1550.65 969.442 L1550.42 969.089 L1550.2 969.299 L1549.97 969.769 L1549.74 969.527 L1549.51 970.175 L1549.28 970.1 L1549.06 970.135 L1548.83 970.545 L1548.6 970.389 L1548.37 969.533 L1548.15 969.209 L1547.92 968.747 L1547.69 968.175 L1547.46 968.01 L1547.24 967.424 L1547.01 966.758 L1546.78 966.95 L1546.55 966.742 L1546.32 966.557 L1546.1 966.86 L1545.87 967.493 L1545.64 966.716 L1545.41 966.436 L1545.19 966.418 L1544.96 967.158 L1544.73 967.037 L1544.5 965.955 L1544.27 967.135 L1544.05 967.514 L1543.82 967.824 L1543.59 968.179 L1543.36 968.048 L1543.14 967.465 L1542.91 968.066 L1542.68 967.622 L1542.45 968.449 L1542.22 968.501 L1542 967.858 L1541.77 967.435 L1541.54 967.33 L1541.31 966.853 L1541.09 966.67 L1540.86 966.194 L1540.63 966.711 L1540.4 967.027 L1540.17 968.504 L1539.95 968.35 L1539.72 968.345 L1539.49 968.154 L1539.26 967.49 L1539.04 967.28 L1538.81 966.992 L1538.58 966.145 L1538.35 966.295 L1538.12 965.512 L1537.9 964.889 L1537.67 964.882 L1537.44 964.095 L1537.21 963.444 L1536.99 963.529 L1536.76 963.847 L1536.53 964.408 L1536.3 964.378 L1536.07 964.014 L1535.85 964.226 L1535.62 964.005 L1535.39 963.578 L1535.16 963.807 L1534.94 964.119 L1534.71 963.359 L1534.48 963.117 L1534.25 963.267 L1534.02 964.63 L1533.8 964.22 L1533.57 963.275 L1533.34 963.881 L1533.11 964.056 L1532.89 964.151 L1532.66 963.228 L1532.43 962.948 L1532.2 963.15 L1531.97 963.687 L1531.75 963.458 L1531.52 964.964 L1531.29 964.903 L1531.06 965.489 L1530.84 965.965 L1530.61 967.236 L1530.38 966.476 L1530.15 966.433 L1529.92 967.736 L1529.7 967.968 L1529.47 967.788 L1529.24 968.401 L1529.01 967.693 L1528.79 967.991 L1528.56 969.125 L1528.33 969.487 L1528.1 969.867 L1527.87 969.873 L1527.65 969.729 L1527.42 969.83 L1527.19 969.56 L1526.96 969.187 L1526.74 969.944 L1526.51 970.612 L1526.28 970.447 L1526.05 970.174 L1525.82 971.11 L1525.6 970.873 L1525.37 971.667 L1525.14 971.159 L1524.91 971.493 L1524.69 971.013 L1524.46 970.31 L1524.23 970.837 L1524 970.711 L1523.77 971.041 L1523.55 971.535 L1523.32 971.126 L1523.09 970.482 L1522.86 970.21 L1522.64 969.878 L1522.41 969.837 L1522.18 969.866 L1521.95 969.23 L1521.72 969.285 L1521.5 968.979 L1521.27 969.066 L1521.04 968.928 L1520.81 967.949 L1520.59 968.036 L1520.36 968.065 L1520.13 968.384 L1519.9 968.574 L1519.67 968.331 L1519.45 968.192 L1519.22 968.207 L1518.99 968.218 L1518.76 968.16 L1518.54 968.074 L1518.31 967.844 L1518.08 967.386 L1517.85 967.917 L1517.62 968.374 L1517.4 968.741 L1517.17 968.7 L1516.94 969.05 L1516.71 968.496 L1516.49 968.794 L1516.26 968.691 L1516.03 968.919 L1515.8 968.28 L1515.57 967.928 L1515.35 968.629 L1515.12 968.448 L1514.89 968.073 L1514.66 968.21 L1514.44 969.094 L1514.21 968.854 L1513.98 968.521 L1513.75 968.182 L1513.52 969.199 L1513.3 969.326 L1513.07 968.51 L1512.84 968.042 L1512.61 967.78 L1512.39 967.282 L1512.16 967.014 L1511.93 967.014 L1511.7 966.541 L1511.48 966.343 L1511.25 966.132 L1511.02 966.364 L1510.79 965.605 L1510.56 965.368 L1510.34 964.814 L1510.11 965.733 L1509.88 966.022 L1509.65 966.306 L1509.43 965.838 L1509.2 966.506 L1508.97 965.753 L1508.74 966.654 L1508.51 966.243 L1508.29 965.974 L1508.06 965.272 L1507.83 966.43 L1507.6 966.669 L1507.38 966.112 L1507.15 966.152 L1506.92 966.29 L1506.69 966.949 L1506.46 967.218 L1506.24 967.193 L1506.01 966.397 L1505.78 966.611 L1505.55 966.836 L1505.33 966.575 L1505.1 966.805 L1504.87 967.236 L1504.64 966.799 L1504.41 966.141 L1504.19 965.719 L1503.96 966.221 L1503.73 965.818 L1503.5 966.495 L1503.28 965.634 L1503.05 966.339 L1502.82 966.312 L1502.59 965.857 L1502.36 966.471 L1502.14 966.426 L1501.91 965.808 L1501.68 966.472 L1501.45 966.469 L1501.23 966.047 L1501 965.868 L1500.77 966.836 L1500.54 966.907 L1500.31 967.854 L1500.09 967.466 L1499.86 966.941 L1499.63 967.075 L1499.4 966.715 L1499.18 966.854 L1498.95 966.747 L1498.72 966.784 L1498.49 967.558 L1498.26 967.792 L1498.04 967.752 L1497.81 967.57 L1497.58 967.244 L1497.35 967.746 L1497.13 967.423 L1496.9 967.311 L1496.67 968.019 L1496.44 967.274 L1496.21 967.077 L1495.99 966.816 L1495.76 966.323 L1495.53 966.032 L1495.3 966.121 L1495.08 965.819 L1494.85 965.382 L1494.62 965.865 L1494.39 965.16 L1494.16 964.833 L1493.94 964.787 L1493.71 964.291 L1493.48 963.46 L1493.25 963.196 L1493.03 963.088 L1492.8 962.738 L1492.57 962.948 L1492.34 963.322 L1492.11 962.703 L1491.89 962.51 L1491.66 962.084 L1491.43 962.812 L1491.2 962.881 L1490.98 962.411 L1490.75 962.73 L1490.52 962.378 L1490.29 961.843 L1490.06 962.169 L1489.84 963.796 L1489.61 964.196 L1489.38 963.824 L1489.15 963.763 L1488.93 962.546 L1488.7 962.579 L1488.47 962.695 L1488.24 962.783 L1488.01 962.455 L1487.79 963.261 L1487.56 963.224 L1487.33 962.996 L1487.1 963.144 L1486.88 963.911 L1486.65 964.768 L1486.42 964.121 L1486.19 963.723 L1485.96 963.546 L1485.74 963.474 L1485.51 963.155 L1485.28 962.948 L1485.05 963.033 L1484.83 963.013 L1484.6 963.329 L1484.37 963.8 L1484.14 963.671 L1483.91 963.718 L1483.69 963.841 L1483.46 964.723 L1483.23 965.634 L1483 965.605 L1482.78 965.118 L1482.55 965.816 L1482.32 965.717 L1482.09 965.182 L1481.86 964.841 L1481.64 964.886 L1481.41 964.803 L1481.18 964.367 L1480.95 965.181 L1480.73 964.915 L1480.5 964.764 L1480.27 965.619 L1480.04 965.752 L1479.81 965.497 L1479.59 965.796 L1479.36 966.119 L1479.13 966.264 L1478.9 966.874 L1478.68 967.021 L1478.45 967.787 L1478.22 966.923 L1477.99 967.872 L1477.76 968.139 L1477.54 967.991 L1477.31 968.588 L1477.08 968.176 L1476.85 967.356 L1476.63 967.554 L1476.4 966.672 L1476.17 966.584 L1475.94 967.91 L1475.71 968.104 L1475.49 968.378 L1475.26 968.097 L1475.03 967.368 L1474.8 967.689 L1474.58 967.872 L1474.35 968.482 L1474.12 968.635 L1473.89 969.349 L1473.67 969.511 L1473.44 968.53 L1473.21 968.584 L1472.98 967.827 L1472.75 967.527 L1472.53 967.253 L1472.3 967.006 L1472.07 967.623 L1471.84 967.527 L1471.62 967.445 L1471.39 967.348 L1471.16 967.884 L1470.93 967.269 L1470.7 967.283 L1470.48 967.93 L1470.25 967.38 L1470.02 967.145 L1469.79 966.216 L1469.57 966.607 L1469.34 966.096 L1469.11 966.691 L1468.88 966.26 L1468.65 965.659 L1468.43 966.987 L1468.2 966.567 L1467.97 966.259 L1467.74 967.656 L1467.52 967.098 L1467.29 967.146 L1467.06 967.844 L1466.83 968.292 L1466.6 968.581 L1466.38 970.17 L1466.15 969.764 L1465.92 969.564 L1465.69 969.656 L1465.47 969.303 L1465.24 969.046 L1465.01 969.005 L1464.78 968.452 L1464.55 968.786 L1464.33 968.588 L1464.1 968.524 L1463.87 968.136 L1463.64 968.019 L1463.42 966.841 L1463.19 967.853 L1462.96 967.569 L1462.73 967.7 L1462.5 967.56 L1462.28 967.478 L1462.05 966.636 L1461.82 967.255 L1461.59 966.684 L1461.37 966.627 L1461.14 966.339 L1460.91 967.093 L1460.68 967.029 L1460.45 967.538 L1460.23 967.853 L1460 967.701 L1459.77 968.279 L1459.54 968.23 L1459.32 968.251 L1459.09 968.822 L1458.86 968.314 L1458.63 968.236 L1458.4 967.786 L1458.18 967.903 L1457.95 967.533 L1457.72 968.038 L1457.49 967.433 L1457.27 966.833 L1457.04 967.611 L1456.81 968.476 L1456.58 968.48 L1456.35 968.752 L1456.13 969.65 L1455.9 969.653 L1455.67 970.25 L1455.44 970.462 L1455.22 970.41 L1454.99 970.423 L1454.76 970.439 L1454.53 970.827 L1454.3 971.096 L1454.08 971.004 L1453.85 971.367 L1453.62 970.906 L1453.39 970.324 L1453.17 969.641 L1452.94 969.759 L1452.71 969.296 L1452.48 968.475 L1452.25 968.23 L1452.03 968.288 L1451.8 968.258 L1451.57 968.24 L1451.34 968.147 L1451.12 968.263 L1450.89 968.712 L1450.66 967.847 L1450.43 967.601 L1450.2 967.184 L1449.98 966.741 L1449.75 967.688 L1449.52 967.594 L1449.29 967.04 L1449.07 968.083 L1448.84 967.96 L1448.61 967.874 L1448.38 968.433 L1448.15 968.592 L1447.93 969.255 L1447.7 968.879 L1447.47 968.603 L1447.24 968.004 L1447.02 967.265 L1446.79 967.04 L1446.56 967.384 L1446.33 967.581 L1446.1 967.606 L1445.88 967.767 L1445.65 967.733 L1445.42 967.975 L1445.19 966.813 L1444.97 965.842 L1444.74 965.425 L1444.51 965.534 L1444.28 966.669 L1444.05 967.269 L1443.83 966.653 L1443.6 966.35 L1443.37 965.763 L1443.14 965.741 L1442.92 965.308 L1442.69 965.108 L1442.46 964.598 L1442.23 964.175 L1442 964.577 L1441.78 965.082 L1441.55 964.996 L1441.32 964.686 L1441.09 964.001 L1440.87 963.523 L1440.64 962.874 L1440.41 962.567 L1440.18 962.437 L1439.95 961.443 L1439.73 961.831 L1439.5 961.341 L1439.27 960.801 L1439.04 961.013 L1438.82 960.867 L1438.59 960.919 L1438.36 960.969 L1438.13 961.733 L1437.91 962.103 L1437.68 962.248 L1437.45 962.221 L1437.22 961.317 L1436.99 961.046 L1436.77 960.637 L1436.54 960.554 L1436.31 960.783 L1436.08 960.517 L1435.86 960.505 L1435.63 960.019 L1435.4 959.829 L1435.17 959.454 L1434.94 959.694 L1434.72 959.815 L1434.49 959.582 L1434.26 959.568 L1434.03 959.332 L1433.81 959.748 L1433.58 959.335 L1433.35 959.852 L1433.12 960.66 L1432.89 961.233 L1432.67 961.28 L1432.44 961.577 L1432.21 961.472 L1431.98 961.949 L1431.76 961.452 L1431.53 961.789 L1431.3 962.211 L1431.07 962.34 L1430.84 962.961 L1430.62 962.005 L1430.39 961.27 L1430.16 960.838 L1429.93 960.467 L1429.71 961.026 L1429.48 961.301 L1429.25 962.477 L1429.02 962.051 L1428.79 962.415 L1428.57 963.003 L1428.34 963.027 L1428.11 962.954 L1427.88 964.062 L1427.66 964.065 L1427.43 964.32 L1427.2 964.44 L1426.97 964.264 L1426.74 964.619 L1426.52 964.143 L1426.29 963.451 L1426.06 963.567 L1425.83 962.686 L1425.61 962.769 L1425.38 962.727 L1425.15 962.467 L1424.92 962.683 L1424.69 962.218 L1424.47 961.768 L1424.24 962.169 L1424.01 961.991 L1423.78 961.648 L1423.56 960.712 L1423.33 960.453 L1423.1 960.036 L1422.87 960.315 L1422.64 960.39 L1422.42 959.85 L1422.19 959.616 L1421.96 960.08 L1421.73 959.617 L1421.51 959.551 L1421.28 958.655 L1421.05 958.339 L1420.82 958.47 L1420.59 959.328 L1420.37 959.621 L1420.14 959.912 L1419.91 959.726 L1419.68 960.459 L1419.46 960.373 L1419.23 960.176 L1419 960.888 L1418.77 962.241 L1418.54 962.006 L1418.32 962.541 L1418.09 962.213 L1417.86 962.271 L1417.63 962.394 L1417.41 963.058 L1417.18 962.687 L1416.95 962.462 L1416.72 963.102 L1416.49 962.558 L1416.27 962.217 L1416.04 962.067 L1415.81 961.291 L1415.58 962.36 L1415.36 961.831 L1415.13 962.622 L1414.9 962.397 L1414.67 962.951 L1414.44 962.378 L1414.22 962.885 L1413.99 963.183 L1413.76 963.663 L1413.53 964.486 L1413.31 964.232 L1413.08 963.631 L1412.85 963.601 L1412.62 964.522 L1412.39 964.414 L1412.17 964.306 L1411.94 964.068 L1411.71 964.24 L1411.48 963.892 L1411.26 963.302 L1411.03 963.868 L1410.8 964.22 L1410.57 964.266 L1410.34 964.361 L1410.12 964.318 L1409.89 964.101 L1409.66 963.946 L1409.43 963.745 L1409.21 963.718 L1408.98 964.028 L1408.75 964.014 L1408.52 963.955 L1408.29 964.612 L1408.07 965.025 L1407.84 965.12 L1407.61 965.667 L1407.38 965.431 L1407.16 966.068 L1406.93 966.205 L1406.7 965.85 L1406.47 967.107 L1406.24 967.344 L1406.02 967.868 L1405.79 967.729 L1405.56 968.209 L1405.33 968.332 L1405.11 968.109 L1404.88 967.774 L1404.65 967.381 L1404.42 966.79 L1404.19 966.557 L1403.97 966.451 L1403.74 967.386 L1403.51 967.836 L1403.28 967.35 L1403.06 967.876 L1402.83 969.811 L1402.6 969.41 L1402.37 968.867 L1402.15 968.329 L1401.92 967.968 L1401.69 968.01 L1401.46 967.757 L1401.23 966.952 L1401.01 966.477 L1400.78 966.431 L1400.55 966.049 L1400.32 966.03 L1400.1 965.813 L1399.87 965.788 L1399.64 965.093 L1399.41 964.366 L1399.18 964.234 L1398.96 963.335 L1398.73 963.817 L1398.5 963.816 L1398.27 964.196 L1398.05 964.072 L1397.82 964.417 L1397.59 964.632 L1397.36 963.951 L1397.13 963.733 L1396.91 963.024 L1396.68 962.643 L1396.45 962.881 L1396.22 961.832 L1396 961.881 L1395.77 961.101 L1395.54 961.533 L1395.31 963.231 L1395.08 964.61 L1394.86 964.39 L1394.63 963.37 L1394.4 963.072 L1394.17 963.063 L1393.95 962.446 L1393.72 963.266 L1393.49 963.232 L1393.26 964.639 L1393.03 965.128 L1392.81 964.594 L1392.58 964.73 L1392.35 964.616 L1392.12 964.017 L1391.9 963.947 L1391.67 963.447 L1391.44 963.356 L1391.21 962.624 L1390.98 962.217 L1390.76 961.57 L1390.53 961.23 L1390.3 961.465 L1390.07 961.418 L1389.85 961.548 L1389.62 961.711 L1389.39 961.68 L1389.16 963.431 L1388.93 963.545 L1388.71 963.83 L1388.48 963.25 L1388.25 963.424 L1388.02 963.456 L1387.8 964.061 L1387.57 963.455 L1387.34 964.007 L1387.11 963.707 L1386.88 963.333 L1386.66 963.292 L1386.43 963.366 L1386.2 963.414 L1385.97 963.353 L1385.75 963.554 L1385.52 964.558 L1385.29 964.938 L1385.06 965.582 L1384.83 966.26 L1384.61 966.445 L1384.38 965.95 L1384.15 964.801 L1383.92 965.601 L1383.7 965.263 L1383.47 965.717 L1383.24 965.268 L1383.01 964.931 L1382.78 965.112 L1382.56 965.211 L1382.33 964.938 L1382.1 964.473 L1381.87 964.42 L1381.65 963.983 L1381.42 964.457 L1381.19 964.737 L1380.96 965.381 L1380.73 965.291 L1380.51 965.12 L1380.28 965.542 L1380.05 965.067 L1379.82 965.474 L1379.6 965.523 L1379.37 965.623 L1379.14 966.158 L1378.91 965.069 L1378.68 965.136 L1378.46 965.036 L1378.23 964.16 L1378 963.495 L1377.77 962.599 L1377.55 962.453 L1377.32 962.607 L1377.09 962.051 L1376.86 962.295 L1376.63 962.733 L1376.41 961.906 L1376.18 962.012 L1375.95 961.629 L1375.72 962.937 L1375.5 963.527 L1375.27 964.232 L1375.04 963.846 L1374.81 963.585 L1374.58 964.141 L1374.36 964.362 L1374.13 964.694 L1373.9 964.601 L1373.67 964.424 L1373.45 963.896 L1373.22 963.812 L1372.99 963.863 L1372.76 964.324 L1372.53 963.006 L1372.31 962.899 L1372.08 963.703 L1371.85 963.342 L1371.62 962.737 L1371.4 962.829 L1371.17 962.851 L1370.94 962.698 L1370.71 962.761 L1370.48 962.597 L1370.26 962.606 L1370.03 962.021 L1369.8 961.595 L1369.57 960.836 L1369.35 961.558 L1369.12 961.168 L1368.89 960.74 L1368.66 960.472 L1368.43 959.986 L1368.21 959.899 L1367.98 959.135 L1367.75 958.827 L1367.52 959.229 L1367.3 959.45 L1367.07 959.227 L1366.84 958.207 L1366.61 958.961 L1366.39 957.547 L1366.16 957.407 L1365.93 957.56 L1365.7 957.376 L1365.47 956.504 L1365.25 955.892 L1365.02 956.516 L1364.79 956.75 L1364.56 957.327 L1364.34 957.002 L1364.11 957.308 L1363.88 957.245 L1363.65 957.434 L1363.42 956.855 L1363.2 956.617 L1362.97 956.235 L1362.74 955.993 L1362.51 955.217 L1362.29 956.328 L1362.06 956.516 L1361.83 956.135 L1361.6 957.051 L1361.37 956.788 L1361.15 956.369 L1360.92 956.13 L1360.69 956.165 L1360.46 956.876 L1360.24 957.468 L1360.01 957.019 L1359.78 957.591 L1359.55 957.836 L1359.32 957.127 L1359.1 956.739 L1358.87 957.19 L1358.64 957.632 L1358.41 957.483 L1358.19 957.533 L1357.96 957.5 L1357.73 957.934 L1357.5 957.369 L1357.27 956.481 L1357.05 955.806 L1356.82 956.28 L1356.59 955.385 L1356.36 955.522 L1356.14 954.904 L1355.91 956.499 L1355.68 957.087 L1355.45 956.817 L1355.22 957.099 L1355 956.865 L1354.77 956.723 L1354.54 958.191 L1354.31 957.335 L1354.09 957.675 L1353.86 957.092 L1353.63 958.112 L1353.4 957.137 L1353.17 957.224 L1352.95 956.246 L1352.72 956.966 L1352.49 956.62 L1352.26 955.921 L1352.04 955.258 L1351.81 955.57 L1351.58 954.594 L1351.35 954.706 L1351.12 954.654 L1350.9 954.458 L1350.67 953.66 L1350.44 953.711 L1350.21 954.172 L1349.99 954.274 L1349.76 953.493 L1349.53 952.759 L1349.3 954.691 L1349.07 954.614 L1348.85 954.832 L1348.62 954.84 L1348.39 955.331 L1348.16 955.309 L1347.94 955.142 L1347.71 955.378 L1347.48 955.171 L1347.25 955.829 L1347.02 955.674 L1346.8 955.944 L1346.57 955.703 L1346.34 956.163 L1346.11 958.181 L1345.89 958.096 L1345.66 959.78 L1345.43 960.958 L1345.2 961.08 L1344.97 960.892 L1344.75 962.034 L1344.52 961.411 L1344.29 962.026 L1344.06 962.492 L1343.84 962.221 L1343.61 962.477 L1343.38 962.884 L1343.15 964.251 L1342.92 963.844 L1342.7 963.296 L1342.47 962.723 L1342.24 962.288 L1342.01 962.917 L1341.79 962.655 L1341.56 962.733 L1341.33 962.032 L1341.1 961.645 L1340.87 960.71 L1340.65 959.709 L1340.42 959.194 L1340.19 960.597 L1339.96 960.377 L1339.74 960.355 L1339.51 960.279 L1339.28 960.135 L1339.05 959.529 L1338.82 959.333 L1338.6 959.255 L1338.37 959.669 L1338.14 958.806 L1337.91 958.675 L1337.69 958.798 L1337.46 958.322 L1337.23 957.395 L1337 957.257 L1336.77 957.384 L1336.55 957.719 L1336.32 958.252 L1336.09 959.089 L1335.86 958.868 L1335.64 959.064 L1335.41 959.437 L1335.18 958.752 L1334.95 958.461 L1334.72 957.835 L1334.5 956.812 L1334.27 956.127 L1334.04 955.444 L1333.81 955.154 L1333.59 954.321 L1333.36 956.086 L1333.13 956.191 L1332.9 956.389 L1332.67 957.516 L1332.45 957.614 L1332.22 957.681 L1331.99 956.814 L1331.76 957.856 L1331.54 959.012 L1331.31 959.408 L1331.08 958.919 L1330.85 959.29 L1330.62 959.174 L1330.4 959.331 L1330.17 959.407 L1329.94 959.356 L1329.71 960.059 L1329.49 960.42 L1329.26 961.463 L1329.03 961.85 L1328.8 961.859 L1328.58 961.659 L1328.35 961.018 L1328.12 960.467 L1327.89 959.747 L1327.66 959.222 L1327.44 959.212 L1327.21 958.99 L1326.98 959.004 L1326.75 959.831 L1326.53 959.299 L1326.3 959.087 L1326.07 959.198 L1325.84 959.262 L1325.61 958.638 L1325.39 959.501 L1325.16 959.192 L1324.93 958.951 L1324.7 958.396 L1324.48 957.932 L1324.25 957.225 L1324.02 956.957 L1323.79 956.332 L1323.56 955.794 L1323.34 955.093 L1323.11 954.059 L1322.88 954.217 L1322.65 954.474 L1322.43 954.446 L1322.2 954.074 L1321.97 954.022 L1321.74 954.52 L1321.51 954.212 L1321.29 953.836 L1321.06 953.16 L1320.83 953.588 L1320.6 953.278 L1320.38 953.236 L1320.15 953.301 L1319.92 952.491 L1319.69 952.679 L1319.46 953.752 L1319.24 953.352 L1319.01 952.732 L1318.78 951.971 L1318.55 951.215 L1318.33 951.153 L1318.1 952.085 L1317.87 952.154 L1317.64 951.884 L1317.41 951.848 L1317.19 951.044 L1316.96 951.361 L1316.73 951.031 L1316.5 951.348 L1316.28 952.536 L1316.05 952.013 L1315.82 951.804 L1315.59 951.295 L1315.36 951.523 L1315.14 951.297 L1314.91 951.523 L1314.68 950.608 L1314.45 950.163 L1314.23 950.366 L1314 949.915 L1313.77 949.696 L1313.54 949.885 L1313.31 950.327 L1313.09 949.447 L1312.86 949.947 L1312.63 949.123 L1312.4 949.194 L1312.18 949.018 L1311.95 950.274 L1311.72 950.836 L1311.49 950.747 L1311.26 949.809 L1311.04 950.563 L1310.81 950.982 L1310.58 951.877 L1310.35 952.108 L1310.13 951.968 L1309.9 951.321 L1309.67 950.443 L1309.44 949.501 L1309.21 949.29 L1308.99 949.473 L1308.76 949.374 L1308.53 948.913 L1308.3 949.408 L1308.08 949.465 L1307.85 948.395 L1307.62 947.975 L1307.39 948.107 L1307.16 947.527 L1306.94 947.578 L1306.71 947.135 L1306.48 947.408 L1306.25 947.194 L1306.03 946.438 L1305.8 946.099 L1305.57 945.687 L1305.34 944.815 L1305.11 944.603 L1304.89 944.938 L1304.66 945.408 L1304.43 945.148 L1304.2 944.995 L1303.98 944.93 L1303.75 944.992 L1303.52 944.28 L1303.29 944.257 L1303.06 944.388 L1302.84 944.441 L1302.61 944.654 L1302.38 945.086 L1302.15 944.651 L1301.93 944.527 L1301.7 945.108 L1301.47 945.44 L1301.24 945.91 L1301.01 945.325 L1300.79 945.26 L1300.56 945.386 L1300.33 945.496 L1300.1 945.83 L1299.88 946.603 L1299.65 946.685 L1299.42 948.015 L1299.19 948.329 L1298.96 947.229 L1298.74 946.934 L1298.51 946.607 L1298.28 946.007 L1298.05 944.977 L1297.83 944.659 L1297.6 946.501 L1297.37 947.04 L1297.14 947.296 L1296.91 947.197 L1296.69 947.27 L1296.46 947.265 L1296.23 947.161 L1296 946.773 L1295.78 946.578 L1295.55 947.639 L1295.32 946.798 L1295.09 947.314 L1294.86 946.958 L1294.64 947.252 L1294.41 947.276 L1294.18 946.422 L1293.95 945.611 L1293.73 946.511 L1293.5 946.315 L1293.27 947.663 L1293.04 947.94 L1292.82 949.26 L1292.59 949.079 L1292.36 949.375 L1292.13 949.016 L1291.9 948.77 L1291.68 948.068 L1291.45 947.608 L1291.22 947.843 L1290.99 948.218 L1290.77 947.457 L1290.54 948.022 L1290.31 946.336 L1290.08 945.591 L1289.85 945.131 L1289.63 944.955 L1289.4 945.769 L1289.17 945.912 L1288.94 946.206 L1288.72 946.326 L1288.49 946.63 L1288.26 946.844 L1288.03 946.587 L1287.8 947.001 L1287.58 947.055 L1287.35 946.849 L1287.12 946.684 L1286.89 946.428 L1286.67 946.068 L1286.44 945.304 L1286.21 944.729 L1285.98 945.089 L1285.75 945.381 L1285.53 945.322 L1285.3 945.901 L1285.07 946.19 L1284.84 945.698 L1284.62 945.484 L1284.39 945.181 L1284.16 945.432 L1283.93 944.596 L1283.7 945.679 L1283.48 945.157 L1283.25 944.83 L1283.02 944.921 L1282.79 945.885 L1282.57 946.795 L1282.34 946.119 L1282.11 946.322 L1281.88 946.848 L1281.65 946.702 L1281.43 946.321 L1281.2 945.632 L1280.97 945.737 L1280.74 945.913 L1280.52 946.456 L1280.29 946.336 L1280.06 946.397 L1279.83 946.366 L1279.6 946.521 L1279.38 945.136 L1279.15 945.252 L1278.92 945.73 L1278.69 946.057 L1278.47 945.836 L1278.24 945.393 L1278.01 945.848 L1277.78 945.277 L1277.55 944.203 L1277.33 944.3 L1277.1 944.312 L1276.87 945.033 L1276.64 945.172 L1276.42 945.433 L1276.19 945.841 L1275.96 946.159 L1275.73 945.889 L1275.5 946.019 L1275.28 945.649 L1275.05 945.727 L1274.82 945.43 L1274.59 945.433 L1274.37 944.361 L1274.14 945.084 L1273.91 944.428 L1273.68 943.84 L1273.45 945.245 L1273.23 946.031 L1273 945.604 L1272.77 945.229 L1272.54 945.607 L1272.32 945.471 L1272.09 945.123 L1271.86 945.494 L1271.63 945.351 L1271.4 945.097 L1271.18 944.497 L1270.95 945.646 L1270.72 945.655 L1270.49 945.617 L1270.27 945.462 L1270.04 945.732 L1269.81 946.691 L1269.58 948.12 L1269.35 947.435 L1269.13 947.622 L1268.9 947.276 L1268.67 947.887 L1268.44 947.967 L1268.22 948.917 L1267.99 949.428 L1267.76 949.56 L1267.53 949.298 L1267.3 949.488 L1267.08 949.116 L1266.85 950.005 L1266.62 949.98 L1266.39 949.734 L1266.17 949.832 L1265.94 950.143 L1265.71 949.336 L1265.48 950.019 L1265.25 950.717 L1265.03 950.2 L1264.8 950.518 L1264.57 949.469 L1264.34 949.074 L1264.12 949.88 L1263.89 949.932 L1263.66 949.948 L1263.43 949.889 L1263.2 948.707 L1262.98 949.65 L1262.75 950.52 L1262.52 950.389 L1262.29 951.511 L1262.07 952.599 L1261.84 953.044 L1261.61 952.298 L1261.38 952.157 L1261.15 952.531 L1260.93 952.168 L1260.7 952.064 L1260.47 951.343 L1260.24 951.807 L1260.02 951.688 L1259.79 950.833 L1259.56 951.455 L1259.33 952.279 L1259.1 951.817 L1258.88 951.511 L1258.65 951.404 L1258.42 951.998 L1258.19 952.361 L1257.97 952.042 L1257.74 952.996 L1257.51 953.321 L1257.28 953.902 L1257.06 953.737 L1256.83 953.355 L1256.6 953.992 L1256.37 954.964 L1256.14 954.492 L1255.92 953.662 L1255.69 953.229 L1255.46 953.515 L1255.23 953.471 L1255.01 954.61 L1254.78 955.845 L1254.55 954.94 L1254.32 955.156 L1254.09 953.925 L1253.87 953.695 L1253.64 954.047 L1253.41 954.324 L1253.18 954.502 L1252.96 954.518 L1252.73 953.846 L1252.5 953.734 L1252.27 953.552 L1252.04 953.511 L1251.82 955.134 L1251.59 956.035 L1251.36 955.248 L1251.13 955.538 L1250.91 955.377 L1250.68 955.573 L1250.45 955.646 L1250.22 955.865 L1249.99 956.26 L1249.77 956.424 L1249.54 956.188 L1249.31 956.209 L1249.08 955.524 L1248.86 955.733 L1248.63 955.956 L1248.4 955.346 L1248.17 955.379 L1247.94 954.65 L1247.72 953.787 L1247.49 953.02 L1247.26 952.982 L1247.03 952.575 L1246.81 952.414 L1246.58 951.601 L1246.35 952.07 L1246.12 952.059 L1245.89 951.833 L1245.67 951.975 L1245.44 950.938 L1245.21 952.103 L1244.98 951.823 L1244.76 950.428 L1244.53 950.227 L1244.3 950.471 L1244.07 950.917 L1243.84 950.986 L1243.62 951.116 L1243.39 951.149 L1243.16 951.286 L1242.93 950.863 L1242.71 953.315 L1242.48 952.787 L1242.25 952.255 L1242.02 952.66 L1241.79 952.509 L1241.57 952.378 L1241.34 953.449 L1241.11 952.865 L1240.88 953.407 L1240.66 953.378 L1240.43 953.349 L1240.2 953.962 L1239.97 953.248 L1239.74 953.482 L1239.52 952.737 L1239.29 952.787 L1239.06 952.774 L1238.83 953.691 L1238.61 953.276 L1238.38 953.884 L1238.15 953.627 L1237.92 953.341 L1237.69 954.458 L1237.47 954.336 L1237.24 954.824 L1237.01 954.338 L1236.78 954.57 L1236.56 953.393 L1236.33 954.182 L1236.1 953.639 L1235.87 953.21 L1235.64 953.742 L1235.42 953.456 L1235.19 953.206 L1234.96 953.379 L1234.73 953.681 L1234.51 954.965 L1234.28 955.099 L1234.05 954.522 L1233.82 954.382 L1233.59 954.76 L1233.37 954.821 L1233.14 954.278 L1232.91 953.655 L1232.68 953.794 L1232.46 953.672 L1232.23 954.343 L1232 953.326 L1231.77 954.621 L1231.54 955.52 L1231.32 955.271 L1231.09 955.354 L1230.86 955.48 L1230.63 954.986 L1230.41 954.28 L1230.18 954.142 L1229.95 953.407 L1229.72 954.287 L1229.49 954.094 L1229.27 953.602 L1229.04 953.899 L1228.81 953.586 L1228.58 953.963 L1228.36 953.65 L1228.13 954.544 L1227.9 953.779 L1227.67 953.708 L1227.44 953.602 L1227.22 953.469 L1226.99 953.225 L1226.76 953.296 L1226.53 953.947 L1226.31 953.545 L1226.08 954.898 L1225.85 956.277 L1225.62 957.657 L1225.39 957.426 L1225.17 956.714 L1224.94 956.506 L1224.71 955.973 L1224.48 957.582 L1224.26 957.414 L1224.03 957.544 L1223.8 958.412 L1223.57 959.119 L1223.34 959.639 L1223.12 959.432 L1222.89 959.626 L1222.66 959.189 L1222.43 958.924 L1222.21 958.7 L1221.98 960.169 L1221.75 961.515 L1221.52 960.817 L1221.29 959.66 L1221.07 959.115 L1220.84 958.764 L1220.61 959.581 L1220.38 959.819 L1220.16 960.55 L1219.93 960.036 L1219.7 959.742 L1219.47 959.248 L1219.25 958.751 L1219.02 959.256 L1218.79 959.215 L1218.56 959.653 L1218.33 958.329 L1218.11 958.987 L1217.88 959.906 L1217.65 959.998 L1217.42 960.087 L1217.2 958.76 L1216.97 958.721 L1216.74 957.95 L1216.51 957.629 L1216.28 957.492 L1216.06 957.368 L1215.83 956.478 L1215.6 955.86 L1215.37 957.362 L1215.15 957.569 L1214.92 957.333 L1214.69 956.882 L1214.46 955.888 L1214.23 955.676 L1214.01 955.854 L1213.78 955.334 L1213.55 955.189 L1213.32 954.992 L1213.1 955.224 L1212.87 955.12 L1212.64 955.248 L1212.41 954.03 L1212.18 953.679 L1211.96 953.097 L1211.73 952.573 L1211.5 951.908 L1211.27 951.026 L1211.05 951.705 L1210.82 951.372 L1210.59 951.431 L1210.36 951.081 L1210.13 952.31 L1209.91 951.858 L1209.68 951.746 L1209.45 950.741 L1209.22 951.296 L1209 951.734 L1208.77 952.638 L1208.54 952.811 L1208.31 953.725 L1208.08 953.181 L1207.86 954.982 L1207.63 954.79 L1207.4 955.095 L1207.17 954.45 L1206.95 953.958 L1206.72 953.906 L1206.49 954.157 L1206.26 954.712 L1206.03 954.745 L1205.81 954.223 L1205.58 953.797 L1205.35 953.267 L1205.12 953.726 L1204.9 953.951 L1204.67 954.601 L1204.44 955.084 L1204.21 954.238 L1203.98 953.046 L1203.76 952.267 L1203.53 952.129 L1203.3 951.507 L1203.07 951.54 L1202.85 950.79 L1202.62 950.695 L1202.39 950.555 L1202.16 950.893 L1201.93 951.202 L1201.71 951.029 L1201.48 950.318 L1201.25 950.761 L1201.02 950.329 L1200.8 950.549 L1200.57 950.875 L1200.34 950.52 L1200.11 950.13 L1199.88 949.813 L1199.66 950.067 L1199.43 949.918 L1199.2 948.875 L1198.97 948.049 L1198.75 947.485 L1198.52 948.365 L1198.29 948.236 L1198.06 947.598 L1197.83 947.864 L1197.61 948.479 L1197.38 947.944 L1197.15 948.114 L1196.92 949.038 L1196.7 949.537 L1196.47 949.124 L1196.24 949.46 L1196.01 948.898 L1195.78 948.368 L1195.56 947.291 L1195.33 946.673 L1195.1 947.041 L1194.87 947.01 L1194.65 945.841 L1194.42 947.471 L1194.19 946.915 L1193.96 946.791 L1193.73 947.482 L1193.51 946.862 L1193.28 946.741 L1193.05 948.076 L1192.82 949.169 L1192.6 948.618 L1192.37 947.982 L1192.14 946.365 L1191.91 946.415 L1191.68 945.947 L1191.46 945.803 L1191.23 946.597 L1191 947.627 L1190.77 946.926 L1190.55 946.231 L1190.32 944.85 L1190.09 945.622 L1189.86 945.981 L1189.63 946.147 L1189.41 946.493 L1189.18 947.537 L1188.95 947.204 L1188.72 947.571 L1188.5 946.863 L1188.27 946.992 L1188.04 946.937 L1187.81 947.298 L1187.58 946.065 L1187.36 945.921 L1187.13 945.696 L1186.9 944.468 L1186.67 944.251 L1186.45 944.749 L1186.22 945.276 L1185.99 944.035 L1185.76 943.308 L1185.53 943.667 L1185.31 943.702 L1185.08 943.546 L1184.85 943.092 L1184.62 942.156 L1184.4 942.86 L1184.17 942.368 L1183.94 942.717 L1183.71 944.545 L1183.49 943.573 L1183.26 943.223 L1183.03 943.426 L1182.8 944.234 L1182.57 944.102 L1182.35 942.733 L1182.12 942.853 L1181.89 942.844 L1181.66 943.043 L1181.44 943.376 L1181.21 943.839 L1180.98 944.329 L1180.75 944.661 L1180.52 944.551 L1180.3 944.864 L1180.07 944.811 L1179.84 944.8 L1179.61 944.024 L1179.39 944.267 L1179.16 945.098 L1178.93 945.051 L1178.7 945.726 L1178.47 945.646 L1178.25 945.292 L1178.02 945.296 L1177.79 944.098 L1177.56 943.543 L1177.34 943.388 L1177.11 942.899 L1176.88 942.601 L1176.65 942.285 L1176.42 941.804 L1176.2 942.478 L1175.97 942.551 L1175.74 942.493 L1175.51 941.664 L1175.29 943.433 L1175.06 942.922 L1174.83 943.06 L1174.6 943.402 L1174.37 944.549 L1174.15 943.539 L1173.92 944.087 L1173.69 945.09 L1173.46 945.031 L1173.24 943.873 L1173.01 945.071 L1172.78 945.233 L1172.55 944.959 L1172.32 945.756 L1172.1 946.454 L1171.87 947.423 L1171.64 947.588 L1171.41 947.604 L1171.19 947.947 L1170.96 948.488 L1170.73 947.973 L1170.5 948.461 L1170.27 948.081 L1170.05 948.495 L1169.82 949.164 L1169.59 948.795 L1169.36 948.747 L1169.14 949.049 L1168.91 948.6 L1168.68 948.295 L1168.45 948.361 L1168.22 947.753 L1168 947.012 L1167.77 946.315 L1167.54 946.151 L1167.31 945.369 L1167.09 945.367 L1166.86 945.885 L1166.63 946.032 L1166.4 945.891 L1166.17 945.054 L1165.95 944.208 L1165.72 944.454 L1165.49 944.557 L1165.26 943.67 L1165.04 943.52 L1164.81 945.467 L1164.58 945.023 L1164.35 945.123 L1164.12 945.488 L1163.9 944.648 L1163.67 945.096 L1163.44 944.848 L1163.21 945.39 L1162.99 945.246 L1162.76 944.703 L1162.53 944.9 L1162.3 944.422 L1162.07 944.368 L1161.85 946.027 L1161.62 946.082 L1161.39 946.977 L1161.16 947.133 L1160.94 946.793 L1160.71 946.468 L1160.48 946.233 L1160.25 946.994 L1160.02 948.082 L1159.8 948.005 L1159.57 947.485 L1159.34 946.599 L1159.11 946.491 L1158.89 946.302 L1158.66 945.974 L1158.43 947.717 L1158.2 947.718 L1157.97 947.473 L1157.75 947.364 L1157.52 948.231 L1157.29 948.41 L1157.06 948.846 L1156.84 948.656 L1156.61 949.7 L1156.38 949.448 L1156.15 949.131 L1155.92 949.048 L1155.7 949.111 L1155.47 948.114 L1155.24 947.855 L1155.01 947.633 L1154.79 948.02 L1154.56 947.401 L1154.33 946.431 L1154.1 945.36 L1153.87 944.326 L1153.65 945.056 L1153.42 944.688 L1153.19 944.927 L1152.96 944.312 L1152.74 944.611 L1152.51 943.784 L1152.28 943.788 L1152.05 944.093 L1151.82 946.093 L1151.6 946.629 L1151.37 947.262 L1151.14 948.073 L1150.91 947.633 L1150.69 947.983 L1150.46 947.953 L1150.23 948.312 L1150 947.686 L1149.77 947.887 L1149.55 948.347 L1149.32 948.011 L1149.09 946.672 L1148.86 947.568 L1148.64 947.76 L1148.41 947.165 L1148.18 946.583 L1147.95 947.35 L1147.73 947.489 L1147.5 947.995 L1147.27 946.623 L1147.04 947.27 L1146.81 947.307 L1146.59 948.9 L1146.36 949.361 L1146.13 949.594 L1145.9 949.608 L1145.68 949.196 L1145.45 948.779 L1145.22 948.408 L1144.99 948.407 L1144.76 948.514 L1144.54 947.709 L1144.31 949.662 L1144.08 949.939 L1143.85 950.007 L1143.63 950.845 L1143.4 951.591 L1143.17 951.783 L1142.94 952.833 L1142.71 954.902 L1142.49 954.718 L1142.26 954.735 L1142.03 955.036 L1141.8 956.425 L1141.58 955.783 L1141.35 954.679 L1141.12 954.929 L1140.89 954.756 L1140.66 956.461 L1140.44 956.809 L1140.21 956.591 L1139.98 956.647 L1139.75 956.886 L1139.53 957.908 L1139.3 957.712 L1139.07 956.641 L1138.84 956.949 L1138.61 956.157 L1138.39 955.899 L1138.16 955.38 L1137.93 955.032 L1137.7 956.586 L1137.48 956.753 L1137.25 956.587 L1137.02 956.968 L1136.79 958.383 L1136.56 956.964 L1136.34 956.842 L1136.11 956.818 L1135.88 958.298 L1135.65 958.362 L1135.43 957.99 L1135.2 957.49 L1134.97 957.587 L1134.74 957.513 L1134.51 958.109 L1134.29 958.397 L1134.06 958.597 L1133.83 958.271 L1133.6 959.235 L1133.38 958.296 L1133.15 958.806 L1132.92 959.958 L1132.69 959.134 L1132.46 959.158 L1132.24 957.942 L1132.01 957.031 L1131.78 956.08 L1131.55 955.99 L1131.33 956.097 L1131.1 956.28 L1130.87 956.732 L1130.64 955.573 L1130.41 955.409 L1130.19 955.188 L1129.96 953.614 L1129.73 953.692 L1129.5 952.999 L1129.28 951.946 L1129.05 952.688 L1128.82 952.548 L1128.59 952.838 L1128.36 952.284 L1128.14 951.645 L1127.91 951.94 L1127.68 952.386 L1127.45 952.371 L1127.23 953.243 L1127 952.869 L1126.77 952.232 L1126.54 952.094 L1126.31 952.976 L1126.09 953.324 L1125.86 952.71 L1125.63 952.856 L1125.4 953.016 L1125.18 953.8 L1124.95 954.202 L1124.72 953.088 L1124.49 953.288 L1124.26 953.727 L1124.04 954.181 L1123.81 953.774 L1123.58 954.556 L1123.35 955.28 L1123.13 955.282 L1122.9 954.736 L1122.67 954.376 L1122.44 954.804 L1122.21 954.581 L1121.99 954.109 L1121.76 954.752 L1121.53 954.488 L1121.3 954.072 L1121.08 954.161 L1120.85 955.194 L1120.62 955.431 L1120.39 955.832 L1120.16 956.025 L1119.94 955.186 L1119.71 955.609 L1119.48 956.372 L1119.25 955.78 L1119.03 956.754 L1118.8 956.764 L1118.57 956.871 L1118.34 957.171 L1118.11 958.002 L1117.89 958.161 L1117.66 958.681 L1117.43 958.451 L1117.2 958.07 L1116.98 957.806 L1116.75 958.129 L1116.52 957.341 L1116.29 956.686 L1116.06 956.886 L1115.84 957.549 L1115.61 956.462 L1115.38 955.925 L1115.15 956.406 L1114.93 956.668 L1114.7 957.616 L1114.47 957.586 L1114.24 958.497 L1114.01 958.371 L1113.79 957.919 L1113.56 956.968 L1113.33 956.541 L1113.1 956.95 L1112.88 957.736 L1112.65 957.393 L1112.42 956.414 L1112.19 956.534 L1111.97 955.722 L1111.74 954.446 L1111.51 954.898 L1111.28 954.817 L1111.05 954.65 L1110.83 953.016 L1110.6 953.666 L1110.37 953.407 L1110.14 952.862 L1109.92 951.605 L1109.69 950.648 L1109.46 951.089 L1109.23 952.417 L1109 952.511 L1108.78 952.405 L1108.55 951.312 L1108.32 950.512 L1108.09 949.443 L1107.87 949.539 L1107.64 949.512 L1107.41 949.35 L1107.18 948.444 L1106.95 948.158 L1106.73 947.831 L1106.5 948.224 L1106.27 947.352 L1106.04 946.161 L1105.82 947.436 L1105.59 947.953 L1105.36 947.29 L1105.13 946.566 L1104.9 946.111 L1104.68 946.651 L1104.45 946.825 L1104.22 946.841 L1103.99 947.417 L1103.77 946.992 L1103.54 945.953 L1103.31 945.403 L1103.08 946.283 L1102.85 946.817 L1102.63 947.197 L1102.4 946.594 L1102.17 946.894 L1101.94 947.689 L1101.72 946.723 L1101.49 946.504 L1101.26 945.632 L1101.03 947.806 L1100.8 947.717 L1100.58 947.411 L1100.35 946.941 L1100.12 946.662 L1099.89 945.806 L1099.67 946.518 L1099.44 946.772 L1099.21 946.887 L1098.98 946.86 L1098.75 946.418 L1098.53 947.293 L1098.3 946.682 L1098.07 946.752 L1097.84 946.329 L1097.62 945.462 L1097.39 945.915 L1097.16 946.68 L1096.93 947.93 L1096.7 946.971 L1096.48 946.999 L1096.25 947.663 L1096.02 948.053 L1095.79 947.616 L1095.57 947.165 L1095.34 947.388 L1095.11 947.772 L1094.88 947.26 L1094.65 946.384 L1094.43 947.724 L1094.2 947.688 L1093.97 949.274 L1093.74 949.799 L1093.52 950.343 L1093.29 950.697 L1093.06 949.148 L1092.83 949.518 L1092.6 951.576 L1092.38 951.261 L1092.15 951.58 L1091.92 952.12 L1091.69 953.302 L1091.47 952.887 L1091.24 952.624 L1091.01 952.017 L1090.78 951.402 L1090.55 952.078 L1090.33 952.594 L1090.1 953.129 L1089.87 953.579 L1089.64 952.804 L1089.42 952.286 L1089.19 952.187 L1088.96 952.459 L1088.73 953.738 L1088.5 953.106 L1088.28 952.14 L1088.05 952.283 L1087.82 951.438 L1087.59 952.125 L1087.37 952.084 L1087.14 951.683 L1086.91 952.139 L1086.68 951.838 L1086.45 952.227 L1086.23 953.011 L1086 952.512 L1085.77 951.239 L1085.54 951.346 L1085.32 950.817 L1085.09 950.38 L1084.86 949.842 L1084.63 949.684 L1084.4 950.615 L1084.18 951.276 L1083.95 951.347 L1083.72 950.575 L1083.49 950.051 L1083.27 953.059 L1083.04 953.372 L1082.81 952.142 L1082.58 951.974 L1082.35 951 L1082.13 950.548 L1081.9 949.962 L1081.67 950.67 L1081.44 949.975 L1081.22 949.034 L1080.99 948.676 L1080.76 949.438 L1080.53 949.529 L1080.3 950.516 L1080.08 951.118 L1079.85 951.835 L1079.62 951.416 L1079.39 950.28 L1079.17 950.894 L1078.94 952.348 L1078.71 955.311 L1078.48 954.429 L1078.25 953.369 L1078.03 954.356 L1077.8 955.092 L1077.57 955.109 L1077.34 955.731 L1077.12 955.899 L1076.89 955.703 L1076.66 954.971 L1076.43 955.075 L1076.2 955.198 L1075.98 955.78 L1075.75 956.412 L1075.52 956.512 L1075.29 957.317 L1075.07 956.796 L1074.84 956.571 L1074.61 958.36 L1074.38 958.24 L1074.16 957.898 L1073.93 959.126 L1073.7 957.88 L1073.47 957.546 L1073.24 957.453 L1073.02 959.236 L1072.79 958.298 L1072.56 958.139 L1072.33 957.213 L1072.11 958.707 L1071.88 957.354 L1071.65 956.8 L1071.42 956.406 L1071.19 957.248 L1070.97 956.646 L1070.74 956.196 L1070.51 957.053 L1070.28 959.2 L1070.06 958.636 L1069.83 959.43 L1069.6 960.464 L1069.37 960.347 L1069.14 960.136 L1068.92 961.428 L1068.69 960.844 L1068.46 961.303 L1068.23 960.99 L1068.01 959.974 L1067.78 959.52 L1067.55 959.537 L1067.32 958.729 L1067.09 958.888 L1066.87 959.214 L1066.64 958.789 L1066.41 958.845 L1066.18 959.075 L1065.96 961.067 L1065.73 962.032 L1065.5 960.425 L1065.27 960.046 L1065.04 960.971 L1064.82 961.363 L1064.59 961.257 L1064.36 961.092 L1064.13 960.972 L1063.91 960.472 L1063.68 959.213 L1063.45 959.353 L1063.22 957.732 L1062.99 958.39 L1062.77 957.507 L1062.54 956.759 L1062.31 956.515 L1062.08 955.399 L1061.86 954.792 L1061.63 954.221 L1061.4 953.683 L1061.17 952.688 L1060.94 953.298 L1060.72 952.955 L1060.49 953.713 L1060.26 953.978 L1060.03 954.825 L1059.81 954.857 L1059.58 954.349 L1059.35 955.142 L1059.12 954.569 L1058.89 955.137 L1058.67 955.611 L1058.44 956.519 L1058.21 956.722 L1057.98 955.668 L1057.76 954.807 L1057.53 954.386 L1057.3 953.35 L1057.07 953.134 L1056.84 953.408 L1056.62 953.564 L1056.39 953.277 L1056.16 952.195 L1055.93 952.675 L1055.71 951.58 L1055.48 951.122 L1055.25 951.815 L1055.02 951.177 L1054.79 950.437 L1054.57 950.95 L1054.34 952.284 L1054.11 952.332 L1053.88 952.06 L1053.66 951.749 L1053.43 952.452 L1053.2 951.501 L1052.97 952.144 L1052.74 950.964 L1052.52 951.197 L1052.29 951.994 L1052.06 951.634 L1051.83 951.491 L1051.61 952.137 L1051.38 952.444 L1051.15 951.865 L1050.92 951.706 L1050.69 950.863 L1050.47 950.071 L1050.24 949.067 L1050.01 949.018 L1049.78 948.06 L1049.56 947.563 L1049.33 947.84 L1049.1 947.355 L1048.87 946.575 L1048.64 946.716 L1048.42 947.411 L1048.19 946.973 L1047.96 946.782 L1047.73 946.301 L1047.51 946.821 L1047.28 948.357 L1047.05 949.71 L1046.82 949.372 L1046.59 948.047 L1046.37 948.269 L1046.14 948.231 L1045.91 948.809 L1045.68 948.705 L1045.46 947.7 L1045.23 947.429 L1045 948.566 L1044.77 949.713 L1044.54 948.791 L1044.32 949.183 L1044.09 948.429 L1043.86 948.173 L1043.63 949.518 L1043.41 948.458 L1043.18 947.808 L1042.95 948.259 L1042.72 947.881 L1042.49 948.173 L1042.27 947.899 L1042.04 947.746 L1041.81 947.102 L1041.58 946.882 L1041.36 946.114 L1041.13 945.36 L1040.9 945.011 L1040.67 945.131 L1040.44 945.35 L1040.22 945.022 L1039.99 946.397 L1039.76 946.225 L1039.53 946.619 L1039.31 945.893 L1039.08 946.902 L1038.85 947.156 L1038.62 946.606 L1038.4 946.357 L1038.17 946.03 L1037.94 945.437 L1037.71 946.033 L1037.48 946.583 L1037.26 948.078 L1037.03 947.372 L1036.8 946.325 L1036.57 946.271 L1036.35 945.71 L1036.12 945.613 L1035.89 947.035 L1035.66 947.907 L1035.43 947.44 L1035.21 948.376 L1034.98 948.592 L1034.75 948.615 L1034.52 948.698 L1034.3 947.973 L1034.07 948.499 L1033.84 949.434 L1033.61 949.262 L1033.38 949.387 L1033.16 947.727 L1032.93 948.213 L1032.7 947.819 L1032.47 949.01 L1032.25 949.412 L1032.02 949.422 L1031.79 949.181 L1031.56 950.349 L1031.33 950.806 L1031.11 950.367 L1030.88 950.057 L1030.65 951.557 L1030.42 950.901 L1030.2 951.146 L1029.97 950.919 L1029.74 951.243 L1029.51 950.18 L1029.28 950.879 L1029.06 950.687 L1028.83 949.876 L1028.6 950.781 L1028.37 950.035 L1028.15 951.111 L1027.92 951.911 L1027.69 952.333 L1027.46 953.193 L1027.23 952.688 L1027.01 952.874 L1026.78 954.379 L1026.55 954.241 L1026.32 954.868 L1026.1 954.522 L1025.87 954.3 L1025.64 954.68 L1025.41 954.381 L1025.18 952.968 L1024.96 952.032 L1024.73 951.472 L1024.5 951.332 L1024.27 950.873 L1024.05 950.855 L1023.82 952.166 L1023.59 951.176 L1023.36 949.143 L1023.13 949.387 L1022.91 951.034 L1022.68 951.713 L1022.45 950.665 L1022.22 950.033 L1022 950.359 L1021.77 950.732 L1021.54 952.031 L1021.31 952.605 L1021.08 952.749 L1020.86 952.047 L1020.63 952.495 L1020.4 952.045 L1020.17 952.572 L1019.95 954.697 L1019.72 955.463 L1019.49 955.046 L1019.26 956.093 L1019.03 955.608 L1018.81 955.405 L1018.58 955.051 L1018.35 954.883 L1018.12 956.106 L1017.9 956.227 L1017.67 956.262 L1017.44 956.033 L1017.21 956.07 L1016.98 957.308 L1016.76 956.614 L1016.53 956.501 L1016.3 957.839 L1016.07 958.365 L1015.85 959.505 L1015.62 958.796 L1015.39 958.46 L1015.16 958.461 L1014.93 959.055 L1014.71 958.547 L1014.48 958.507 L1014.25 959.821 L1014.02 959.663 L1013.8 959.403 L1013.57 959.736 L1013.34 959.368 L1013.11 959.146 L1012.88 960.163 L1012.66 959.963 L1012.43 959.788 L1012.2 959.344 L1011.97 958.055 L1011.75 956.865 L1011.52 958.455 L1011.29 957.473 L1011.06 958.365 L1010.83 959.677 L1010.61 959.83 L1010.38 959.428 L1010.15 959.626 L1009.92 960.333 L1009.7 959.488 L1009.47 959.106 L1009.24 958.32 L1009.01 958.688 L1008.78 957.87 L1008.56 958.007 L1008.33 958.464 L1008.1 958.263 L1007.87 957.183 L1007.65 957.29 L1007.42 957.04 L1007.19 956.832 L1006.96 958.464 L1006.73 957.363 L1006.51 956.706 L1006.28 955.114 L1006.05 954.492 L1005.82 954.931 L1005.6 954.839 L1005.37 955.052 L1005.14 954.829 L1004.91 953.485 L1004.68 953.619 L1004.46 953.171 L1004.23 952.699 L1004 953.03 L1003.77 952.196 L1003.55 953.369 L1003.32 953.17 L1003.09 953.555 L1002.86 954.406 L1002.64 955.662 L1002.41 955.381 L1002.18 954.37 L1001.95 955.282 L1001.72 954.765 L1001.5 955.378 L1001.27 953.593 L1001.04 955.715 L1000.81 954.157 L1000.59 954.553 L1000.36 954.541 L1000.13 953.881 L999.902 953.914 L999.674 952.889 L999.446 953.344 L999.218 953.472 L998.991 953.024 L998.763 952.741 L998.535 952.431 L998.307 952.185 L998.08 953.008 L997.852 952.217 L997.624 952.817 L997.396 952.806 L997.169 952.423 L996.941 951.786 L996.713 951.7 L996.485 950.978 L996.257 950.712 L996.03 949.802 L995.802 950.19 L995.574 950.073 L995.346 950.286 L995.119 949.359 L994.891 949.549 L994.663 949.638 L994.435 949.994 L994.207 950.36 L993.98 951.702 L993.752 951.081 L993.524 950.563 L993.296 952.006 L993.069 950.981 L992.841 950.233 L992.613 949.659 L992.385 950.377 L992.158 953.231 L991.93 953.527 L991.702 953.51 L991.474 953.21 L991.246 952.856 L991.019 952.845 L990.791 953.336 L990.563 953.709 L990.335 952.321 L990.108 951.988 L989.88 951.658 L989.652 951.895 L989.424 952.934 L989.197 954.946 L988.969 954.925 L988.741 956.031 L988.513 956.694 L988.285 955.658 L988.058 955.838 L987.83 955.545 L987.602 955.393 L987.374 953.743 L987.147 952.256 L986.919 951.878 L986.691 950.372 L986.463 950.662 L986.236 949.134 L986.008 947.432 L985.78 946.542 L985.552 945.66 L985.324 945.882 L985.097 946.06 L984.869 946.996 L984.641 947.31 L984.413 945.325 L984.186 944.635 L983.958 944.447 L983.73 944.93 L983.502 943.885 L983.274 943.793 L983.047 942.351 L982.819 941.924 L982.591 943.115 L982.363 944.187 L982.136 942.392 L981.908 941.934 L981.68 941.975 L981.452 943.762 L981.225 942.609 L980.997 943.107 L980.769 944.601 L980.541 944.411 L980.313 945.355 L980.086 945.317 L979.858 945.537 L979.63 945.99 L979.402 945.734 L979.175 945.604 L978.947 944.798 L978.719 944.778 L978.491 944.234 L978.264 944.382 L978.036 944.148 L977.808 942.836 L977.58 942.988 L977.352 942.908 L977.125 942.331 L976.897 941.688 L976.669 941.675 L976.441 941.349 L976.214 940.68 L975.986 940.163 L975.758 939.953 L975.53 939.86 L975.303 940.855 L975.075 942.355 L974.847 942.426 L974.619 941.575 L974.391 943.059 L974.164 942.966 L973.936 942.641 L973.708 943.176 L973.48 944.516 L973.253 943.876 L973.025 945.405 L972.797 944.387 L972.569 944.675 L972.341 944.247 L972.114 944.966 L971.886 945.816 L971.658 945.832 L971.43 945.562 L971.203 945.949 L970.975 945.141 L970.747 945.44 L970.519 946.135 L970.292 947.699 L970.064 947.595 L969.836 948.931 L969.608 949.028 L969.38 947.642 L969.153 947.573 L968.925 948.31 L968.697 948.911 L968.469 949.429 L968.242 950.498 L968.014 950.105 L967.786 951.851 L967.558 950.72 L967.331 949.742 L967.103 948.405 L966.875 949.535 L966.647 951.002 L966.419 949.959 L966.192 951.441 L965.964 951.575 L965.736 951.517 L965.508 950.906 L965.281 950.998 L965.053 950.595 L964.825 950.278 L964.597 951.21 L964.37 950.443 L964.142 950.009 L963.914 950.442 L963.686 951.438 L963.458 950.303 L963.231 951.561 L963.003 950.725 L962.775 950.306 L962.547 949.592 L962.32 948.008 L962.092 947.163 L961.864 947.103 L961.636 948.485 L961.408 948.558 L961.181 949.113 L960.953 949.167 L960.725 948.64 L960.497 948.436 L960.27 948.381 L960.042 948.224 L959.814 947.6 L959.586 949.196 L959.359 947.927 L959.131 947.448 L958.903 946.544 L958.675 946.274 L958.447 945.816 L958.22 946.49 L957.992 949.432 L957.764 948.576 L957.536 948.282 L957.309 947.818 L957.081 947.834 L956.853 947.673 L956.625 950.113 L956.398 951.437 L956.17 950.852 L955.942 951.399 L955.714 950.806 L955.486 950.94 L955.259 951.464 L955.031 951.99 L954.803 953.677 L954.575 954.917 L954.348 955.316 L954.12 954.673 L953.892 953.869 L953.664 954.181 L953.437 954.815 L953.209 954.957 L952.981 954.844 L952.753 956.2 L952.525 957.687 L952.298 959.292 L952.07 959.456 L951.842 960.021 L951.614 959.875 L951.387 958.956 L951.159 959.701 L950.931 960.002 L950.703 960.928 L950.475 960.851 L950.248 962.935 L950.02 961.471 L949.792 961.177 L949.564 960.58 L949.337 960.702 L949.109 960.478 L948.881 960.52 L948.653 959.15 L948.426 960.483 L948.198 961.891 L947.97 960.576 L947.742 959.76 L947.514 959.713 L947.287 960.265 L947.059 960.097 L946.831 959.784 L946.603 958.736 L946.376 957.327 L946.148 959.14 L945.92 960.138 L945.692 959.572 L945.465 959.497 L945.237 959.902 L945.009 959.355 L944.781 958.946 L944.553 959.377 L944.326 958.694 L944.098 960.138 L943.87 959.701 L943.642 959.405 L943.415 958.808 L943.187 959.365 L942.959 958.867 L942.731 958.614 L942.504 957.446 L942.276 957.15 L942.048 956.879 L941.82 956.319 L941.592 955.983 L941.365 955.043 L941.137 954.664 L940.909 953.282 L940.681 952.75 L940.454 951.883 L940.226 950.137 L939.998 949.933 L939.77 949.56 L939.542 950.066 L939.315 951.22 L939.087 950.778 L938.859 951.094 L938.631 950.545 L938.404 950.984 L938.176 949.683 L937.948 949.221 L937.72 949.294 L937.493 948.213 L937.265 948.156 L937.037 947.172 L936.809 947.077 L936.581 947.818 L936.354 946.58 L936.126 945.694 L935.898 945.205 L935.67 945.218 L935.443 944.697 L935.215 944.672 L934.987 945.076 L934.759 943.495 L934.532 942.279 L934.304 940.837 L934.076 941.306 L933.848 940.504 L933.62 939.716 L933.393 939.518 L933.165 940.773 L932.937 940.878 L932.709 942.021 L932.482 942.618 L932.254 940.738 L932.026 940.626 L931.798 939.55 L931.571 939.662 L931.343 939.636 L931.115 940.485 L930.887 939.35 L930.659 937.388 L930.432 937.966 L930.204 936.637 L929.976 934.876 L929.748 936.549 L929.521 937.188 L929.293 937.353 L929.065 937.661 L928.837 936.696 L928.609 938.566 L928.382 941.512 L928.154 940.956 L927.926 941.463 L927.698 940.942 L927.471 941.86 L927.243 940.896 L927.015 940.428 L926.787 940.62 L926.56 940.399 L926.332 940.119 L926.104 940.828 L925.876 939.956 L925.648 941.304 L925.421 939.852 L925.193 939.291 L924.965 938.215 L924.737 937.59 L924.51 938.614 L924.282 940.111 L924.054 939.251 L923.826 938.458 L923.599 938.124 L923.371 939.865 L923.143 938.978 L922.915 938.96 L922.687 938.665 L922.46 938.612 L922.232 938.235 L922.004 937.528 L921.776 937.957 L921.549 936.649 L921.321 939.868 L921.093 939.286 L920.865 941.944 L920.638 941.489 L920.41 940.891 L920.182 941.561 L919.954 941.557 L919.726 940.395 L919.499 940.884 L919.271 940.939 L919.043 942.148 L918.815 940.836 L918.588 941.287 L918.36 940.344 L918.132 940.082 L917.904 941.896 L917.676 941.8 L917.449 940.84 L917.221 941.359 L916.993 941.622 L916.765 942.252 L916.538 943.442 L916.31 944.328 L916.082 945.804 L915.854 945.235 L915.627 945.916 L915.399 946.178 L915.171 946.13 L914.943 946.074 L914.715 946.617 L914.488 945.207 L914.26 944.333 L914.032 944.301 L913.804 943.359 L913.577 943.689 L913.349 944.119 L913.121 941.722 L912.893 940.561 L912.666 939.861 L912.438 939.302 L912.21 938.683 L911.982 938.48 L911.754 938.494 L911.527 939.509 L911.299 940.978 L911.071 940.449 L910.843 939.934 L910.616 938.832 L910.388 938.876 L910.16 939.229 L909.932 937.491 L909.705 938.331 L909.477 937.909 L909.249 938.107 L909.021 939.088 L908.793 938.807 L908.566 938.841 L908.338 938.588 L908.11 938.129 L907.882 937.43 L907.655 937.635 L907.427 938.276 L907.199 939.1 L906.971 937.884 L906.743 938.036 L906.516 937.85 L906.288 937.726 L906.06 937.443 L905.832 936.671 L905.605 936.591 L905.377 938.053 L905.149 939.343 L904.921 940.846 L904.694 939.881 L904.466 940.453 L904.238 941.958 L904.01 942.625 L903.782 942.869 L903.555 942.864 L903.327 945.112 L903.099 945.383 L902.871 944.284 L902.644 944.284 L902.416 943.103 L902.188 941.877 L901.96 940.43 L901.733 942.274 L901.505 941.921 L901.277 942.872 L901.049 942.717 L900.821 940.862 L900.594 938.966 L900.366 937.609 L900.138 937.493 L899.91 936.968 L899.683 937.3 L899.455 939.865 L899.227 940.476 L898.999 938.755 L898.772 938.009 L898.544 937.726 L898.316 938.553 L898.088 937.38 L897.86 937.848 L897.633 937.089 L897.405 935.569 L897.177 936.502 L896.949 935.896 L896.722 934.845 L896.494 933.943 L896.266 933.999 L896.038 933.293 L895.81 936.747 L895.583 936.893 L895.355 937.603 L895.127 936.875 L894.899 935.602 L894.672 934.995 L894.444 935.757 L894.216 934.946 L893.988 934.614 L893.761 935.436 L893.533 935.098 L893.305 933.992 L893.077 932.807 L892.849 933.641 L892.622 932.98 L892.394 933.571 L892.166 933.305 L891.938 933.13 L891.711 933.939 L891.483 933.814 L891.255 932.73 L891.027 931.893 L890.8 930.217 L890.572 931.605 L890.344 931.26 L890.116 930.227 L889.888 929.68 L889.661 929.554 L889.433 930.933 L889.205 931.708 L888.977 931.306 L888.75 932.413 L888.522 931.918 L888.294 930.776 L888.066 930.466 L887.839 928.73 L887.611 928.778 L887.383 928.782 L887.155 929.382 L886.927 928.068 L886.7 927.023 L886.472 926.904 L886.244 925.878 L886.016 925.692 L885.789 924.655 L885.561 923.223 L885.333 924.191 L885.105 923.316 L884.877 924.124 L884.65 923.295 L884.422 924.502 L884.194 923.543 L883.966 921.97 L883.739 922.212 L883.511 923.972 L883.283 923.554 L883.055 924.076 L882.828 923.918 L882.6 925.065 L882.372 924.492 L882.144 924.193 L881.916 923.133 L881.689 923.372 L881.461 924.16 L881.233 923.995 L881.005 925.656 L880.778 925.919 L880.55 925.252 L880.322 926.22 L880.094 924.841 L879.867 925.492 L879.639 924.976 L879.411 922.577 L879.183 922.231 L878.955 921.772 L878.728 921.756 L878.5 923.003 L878.272 923.53 L878.044 922.117 L877.817 921.343 L877.589 920.311 L877.361 921.034 L877.133 921.353 L876.906 924.09 L876.678 923.726 L876.45 923.619 L876.222 923.598 L875.994 923.412 L875.767 922.492 L875.539 923.107 L875.311 922.513 L875.083 923.864 L874.856 923.302 L874.628 923.475 L874.4 922.844 L874.172 923.455 L873.944 922.649 L873.717 924.524 L873.489 925.276 L873.261 925.346 L873.033 925.141 L872.806 924.788 L872.578 924.142 L872.35 923.905 L872.122 924.679 L871.895 925.292 L871.667 924.438 L871.439 925.529 L871.211 926.533 L870.983 926.327 L870.756 926.645 L870.528 925.795 L870.3 926.342 L870.072 927.319 L869.845 926.961 L869.617 928.362 L869.389 928.321 L869.161 929.143 L868.934 927.921 L868.706 928.55 L868.478 929.572 L868.25 930.617 L868.022 930.676 L867.795 930.088 L867.567 929.27 L867.339 930.63 L867.111 929.618 L866.884 931.018 L866.656 930.876 L866.428 929.815 L866.2 930.258 L865.973 929.804 L865.745 929.775 L865.517 929.989 L865.289 929.612 L865.061 928.559 L864.834 927.536 L864.606 925.986 L864.378 925.372 L864.15 924.238 L863.923 924.522 L863.695 925.35 L863.467 923.628 L863.239 922.885 L863.011 922.31 L862.784 921.207 L862.556 921.679 L862.328 921.389 L862.1 921.463 L861.873 921.902 L861.645 923.616 L861.417 923.819 L861.189 923.327 L860.962 922.938 L860.734 923.327 L860.506 923.381 L860.278 922.173 L860.05 920.767 L859.823 919.984 L859.595 920.295 L859.367 919.272 L859.139 919.039 L858.912 918.678 L858.684 921.167 L858.456 921.878 L858.228 924.016 L858.001 923.399 L857.773 922.316 L857.545 922.228 L857.317 921.73 L857.089 921.394 L856.862 921.95 L856.634 920.1 L856.406 919.792 L856.178 918.089 L855.951 919.465 L855.723 918.697 L855.495 917.962 L855.267 917.707 L855.04 916.84 L854.812 915.94 L854.584 917.46 L854.356 918.857 L854.128 918.172 L853.901 915.988 L853.673 915.879 L853.445 917.328 L853.217 915.265 L852.99 914.899 L852.762 914.443 L852.534 913.65 L852.306 914.224 L852.078 914.539 L851.851 915.147 L851.623 914.37 L851.395 914.191 L851.167 912.186 L850.94 912.348 L850.712 912.636 L850.484 912.568 L850.256 911.893 L850.029 911.514 L849.801 911.234 L849.573 910.656 L849.345 911.053 L849.117 909.172 L848.89 907.737 L848.662 908.097 L848.434 907.076 L848.206 906.968 L847.979 905.656 L847.751 905.439 L847.523 904.764 L847.295 903.545 L847.068 902.109 L846.84 899.643 L846.612 900.4 L846.384 900.568 L846.156 902.363 L845.929 902.42 L845.701 904.116 L845.473 904.183 L845.245 902.811 L845.018 902.196 L844.79 902.096 L844.562 904.416 L844.334 903.377 L844.107 903.422 L843.879 904.82 L843.651 905.141 L843.423 904.777 L843.195 904.197 L842.968 903.334 L842.74 903.416 L842.512 903.508 L842.284 902.782 L842.057 902.988 L841.829 902.947 L841.601 902.767 L841.373 903.638 L841.145 904.808 L840.918 902.73 L840.69 902.652 L840.462 900.918 L840.234 899.573 L840.007 899.017 L839.779 899.722 L839.551 899.073 L839.323 898.277 L839.096 898.157 L838.868 900.621 L838.64 899.887 L838.412 899.587 L838.184 899.065 L837.957 897.163 L837.729 895.802 L837.501 896.771 L837.273 895.594 L837.046 895.437 L836.818 894.614 L836.59 893.639 L836.362 893.468 L836.135 892.984 L835.907 893.183 L835.679 893.484 L835.451 894.223 L835.223 896.013 L834.996 897.239 L834.768 897.135 L834.54 898.457 L834.312 897.584 L834.085 897.197 L833.857 899.767 L833.629 899.019 L833.401 899.26 L833.174 898.467 L832.946 897.815 L832.718 898.035 L832.49 897.268 L832.262 897.707 L832.035 896.865 L831.807 896.252 L831.579 895.211 L831.351 893.905 L831.124 892.655 L830.896 891.627 L830.668 890.796 L830.44 893.934 L830.212 893.04 L829.985 894.163 L829.757 893.186 L829.529 892.726 L829.301 891.685 L829.074 890.833 L828.846 894.918 L828.618 894.626 L828.39 893.615 L828.163 892.36 L827.935 891.353 L827.707 891.418 L827.479 890.303 L827.251 891.548 L827.024 891.084 L826.796 890.023 L826.568 890.048 L826.34 889.977 L826.113 889.733 L825.885 888.773 L825.657 886.44 L825.429 887.111 L825.202 887.082 L824.974 886.6 L824.746 884.798 L824.518 885.055 L824.29 883.443 L824.063 882.605 L823.835 881.183 L823.607 882.878 L823.379 881.837 L823.152 881.418 L822.924 881.217 L822.696 879.981 L822.468 878.215 L822.241 876.701 L822.013 875.307 L821.785 876.569 L821.557 875.409 L821.329 876.762 L821.102 876.474 L820.874 875.484 L820.646 874.172 L820.418 874.338 L820.191 874.454 L819.963 873.491 L819.735 872.299 L819.507 872.213 L819.279 873.606 L819.052 872.825 L818.824 871.92 L818.596 872.654 L818.368 874.22 L818.141 876.535 L817.913 876.427 L817.685 875.584 L817.457 876.719 L817.23 876.087 L817.002 877.815 L816.774 878.888 L816.546 878.41 L816.318 877.229 L816.091 877.557 L815.863 877.574 L815.635 875.914 L815.407 876.331 L815.18 877.994 L814.952 879.044 L814.724 879.177 L814.496 879.392 L814.269 882 L814.041 880.198 L813.813 880.083 L813.585 878.639 L813.357 878.229 L813.13 877.733 L812.902 877.971 L812.674 878.31 L812.446 878.399 L812.219 877.675 L811.991 877.237 L811.763 878.321 L811.535 876.627 L811.308 876.236 L811.08 875.171 L810.852 875.422 L810.624 875.667 L810.396 875.269 L810.169 876.16 L809.941 875.638 L809.713 876.19 L809.485 874.953 L809.258 874.574 L809.03 874.75 L808.802 874.558 L808.574 873.908 L808.346 875.29 L808.119 874.91 L807.891 874.504 L807.663 873.956 L807.435 871.755 L807.208 870.025 L806.98 870.992 L806.752 871.036 L806.524 870.717 L806.297 871.059 L806.069 870.997 L805.841 871.799 L805.613 871.281 L805.385 871.273 L805.158 872.402 L804.93 871.869 L804.702 872.183 L804.474 871.563 L804.247 871.658 L804.019 875.163 L803.791 873.87 L803.563 873.817 L803.336 872.842 L803.108 871.043 L802.88 870.05 L802.652 870.681 L802.424 872.215 L802.197 872.07 L801.969 872.159 L801.741 870.749 L801.513 871.501 L801.286 869.584 L801.058 867.47 L800.83 868.081 L800.602 869.627 L800.375 869.8 L800.147 868.654 L799.919 869.017 L799.691 869.067 L799.463 867.705 L799.236 867.13 L799.008 869.989 L798.78 870.703 L798.552 870.346 L798.325 870.867 L798.097 870.667 L797.869 870.304 L797.641 869.998 L797.413 870.203 L797.186 869.45 L796.958 868.559 L796.73 865.516 L796.502 867.391 L796.275 867.208 L796.047 866.881 L795.819 865.636 L795.591 865.917 L795.364 865.426 L795.136 866.516 L794.908 869.788 L794.68 869.471 L794.452 870.402 L794.225 870.707 L793.997 870.408 L793.769 872.58 L793.541 871.015 L793.314 870.787 L793.086 869.192 L792.858 870.41 L792.63 869.776 L792.403 869.542 L792.175 869.063 L791.947 869.593 L791.719 870.719 L791.491 871.01 L791.264 869.229 L791.036 873.518 L790.808 872.452 L790.58 871.235 L790.353 871.865 L790.125 872.274 L789.897 871.483 L789.669 871.892 L789.442 869.849 L789.214 872.512 L788.986 871.201 L788.758 870.788 L788.53 870.272 L788.303 869.582 L788.075 869.636 L787.847 868.104 L787.619 866.841 L787.392 866.182 L787.164 866.826 L786.936 867.408 L786.708 869.903 L786.481 868.203 L786.253 866.329 L786.025 865.357 L785.797 866.983 L785.569 867.913 L785.342 866.309 L785.114 867.503 L784.886 868.056 L784.658 866.96 L784.431 865.596 L784.203 867.408 L783.975 867.121 L783.747 866.508 L783.519 866.561 L783.292 865.994 L783.064 864.447 L782.836 864.484 L782.608 865.795 L782.381 864.577 L782.153 863.043 L781.925 862.35 L781.697 861.484 L781.47 862.557 L781.242 862.529 L781.014 862.355 L780.786 861.519 L780.558 861.57 L780.331 859.272 L780.103 860.107 L779.875 861.873 L779.647 860.877 L779.42 858.787 L779.192 857.733 L778.964 858.241 L778.736 857.076 L778.509 856.642 L778.281 857.282 L778.053 858.009 L777.825 855.99 L777.597 854.801 L777.37 852.855 L777.142 854.237 L776.914 853.836 L776.686 854.102 L776.459 854.668 L776.231 854.757 L776.003 853.699 L775.775 852.594 L775.548 851.503 L775.32 853.948 L775.092 854.295 L774.864 854.018 L774.636 853.932 L774.409 853.87 L774.181 856.486 L773.953 856.364 L773.725 859.559 L773.498 860.061 L773.27 861.659 L773.042 862.668 L772.814 866.023 L772.586 866.57 L772.359 867.252 L772.131 866.659 L771.903 865.691 L771.675 863.537 L771.448 864.125 L771.22 864.151 L770.992 866.812 L770.764 867.519 L770.537 866.655 L770.309 868.113 L770.081 868.722 L769.853 873.64 L769.625 874.357 L769.398 873.697 L769.17 873.493 L768.942 872.729 L768.714 872.855 L768.487 872.606 L768.259 871.01 L768.031 869.448 L767.803 869.416 L767.576 868.244 L767.348 868.246 L767.12 866.952 L766.892 865.193 L766.664 865.213 L766.437 864.78 L766.209 862.398 L765.981 864.269 L765.753 864.941 L765.526 863.417 L765.298 863.37 L765.07 861.063 L764.842 860.886 L764.615 858.894 L764.387 859.396 L764.159 857.694 L763.931 859.464 L763.703 862.973 L763.476 861.224 L763.248 862.815 L763.02 865.547 L762.792 863.296 L762.565 862.22 L762.337 862.146 L762.109 864.172 L761.881 865.9 L761.653 866.813 L761.426 868.151 L761.198 866.256 L760.97 865.079 L760.742 865.292 L760.515 864.462 L760.287 863.783 L760.059 863.371 L759.831 862.641 L759.604 862.407 L759.376 860.846 L759.148 859.831 L758.92 859.917 L758.692 859.276 L758.465 859.783 L758.237 859.321 L758.009 858.348 L757.781 859.889 L757.554 860.245 L757.326 861.379 L757.098 863.262 L756.87 862.22 L756.643 865.846 L756.415 866.767 L756.187 865.802 L755.959 867.695 L755.731 867.246 L755.504 869.536 L755.276 868.407 L755.048 867.921 L754.82 867.231 L754.593 868.221 L754.365 868.49 L754.137 867.982 L753.909 872.316 L753.682 869.538 L753.454 872.626 L753.226 873.562 L752.998 872.87 L752.77 872.099 L752.543 869.617 L752.315 870.236 L752.087 869.944 L751.859 869.815 L751.632 869.446 L751.404 867.229 L751.176 867.124 L750.948 870.63 L750.72 871.509 L750.493 874.377 L750.265 875.287 L750.037 877.969 L749.809 876.953 L749.582 877.505 L749.354 877.698 L749.126 878.544 L748.898 877.979 L748.671 878.348 L748.443 877.231 L748.215 875.868 L747.987 874.372 L747.759 875.397 L747.532 874.154 L747.304 874.014 L747.076 872.755 L746.848 873.474 L746.621 873.135 L746.393 872.664 L746.165 873.574 L745.937 873.485 L745.71 872.343 L745.482 872.625 L745.254 872.899 L745.026 872.884 L744.798 874.934 L744.571 874.464 L744.343 873.132 L744.115 874.078 L743.887 873.424 L743.66 876.314 L743.432 879.306 L743.204 878.914 L742.976 878.653 L742.749 878.173 L742.521 881.637 L742.293 881.7 L742.065 880.205 L741.837 878.196 L741.61 877.408 L741.382 875.374 L741.154 875.826 L740.926 875.578 L740.699 875.063 L740.471 875.524 L740.243 873.946 L740.015 873.322 L739.787 872.977 L739.56 870.918 L739.332 869.736 L739.104 869.372 L738.876 868.558 L738.649 870.423 L738.421 868.786 L738.193 867.998 L737.965 867.842 L737.738 867.369 L737.51 868.947 L737.282 869.121 L737.054 867.943 L736.826 869.563 L736.599 868.781 L736.371 868.33 L736.143 867.28 L735.915 869.129 L735.688 871.63 L735.46 872.089 L735.232 872.032 L735.004 872.279 L734.777 869.748 L734.549 868.78 L734.321 869.789 L734.093 869.351 L733.865 867.278 L733.638 865.076 L733.41 866.071 L733.182 865.173 L732.954 864.172 L732.727 865.191 L732.499 866.116 L732.271 864.705 L732.043 866.279 L731.816 865.148 L731.588 865.599 L731.36 865.589 L731.132 864.177 L730.904 862.259 L730.677 863.632 L730.449 865.164 L730.221 864.304 L729.993 864.737 L729.766 864.294 L729.538 865.301 L729.31 865.193 L729.082 863.179 L728.854 864.742 L728.627 865.171 L728.399 866.174 L728.171 866.709 L727.943 868.489 L727.716 869.029 L727.488 867.412 L727.26 866.097 L727.032 863.609 L726.805 864.395 L726.577 862.932 L726.349 862.214 L726.121 860.345 L725.893 861.97 L725.666 865.657 L725.438 865.945 L725.21 865.254 L724.982 863.677 L724.755 860.862 L724.527 860.62 L724.299 859.762 L724.071 860.366 L723.844 863.178 L723.616 862.064 L723.388 861.537 L723.16 860.565 L722.932 861.678 L722.705 860.126 L722.477 859.391 L722.249 857.543 L722.021 859.074 L721.794 861.301 L721.566 860.194 L721.338 858.903 L721.11 858.936 L720.883 856.527 L720.655 854.8 L720.427 854.681 L720.199 854.154 L719.971 853.578 L719.744 854.062 L719.516 854.11 L719.288 855.101 L719.06 856.125 L718.833 858.632 L718.605 857.571 L718.377 857.121 L718.149 859.564 L717.921 857.946 L717.694 855.274 L717.466 857.278 L717.238 855.631 L717.01 854.195 L716.783 852.128 L716.555 851.582 L716.327 851.974 L716.099 854.276 L715.872 852.671 L715.644 849.741 L715.416 847.414 L715.188 846.719 L714.96 844.676 L714.733 842.437 L714.505 842.394 L714.277 840.902 L714.049 840.376 L713.822 839.897 L713.594 839.779 L713.366 838.084 L713.138 838.647 L712.911 840.111 L712.683 840.045 L712.455 838.297 L712.227 838.649 L711.999 838.327 L711.772 836.831 L711.544 837.366 L711.316 836.658 L711.088 836.36 L710.861 834.554 L710.633 834.82 L710.405 835.791 L710.177 836.251 L709.95 837.661 L709.722 834.107 L709.494 834.311 L709.266 836.698 L709.038 837.492 L708.811 838.87 L708.583 838.587 L708.355 837.343 L708.127 836.707 L707.9 837.628 L707.672 836.615 L707.444 837.632 L707.216 837.577 L706.988 837.558 L706.761 838.663 L706.533 839.298 L706.305 838.117 L706.077 836.9 L705.85 836.721 L705.622 835.408 L705.394 836.084 L705.166 835.459 L704.939 832.961 L704.711 830.764 L704.483 834.415 L704.255 833.055 L704.027 838.466 L703.8 838.022 L703.572 836.714 L703.344 836.378 L703.116 838.046 L702.889 835.678 L702.661 836.972 L702.433 836.896 L702.205 838.163 L701.978 837.672 L701.75 837.826 L701.522 836.067 L701.294 835.081 L701.066 836.34 L700.839 837.081 L700.611 836.992 L700.383 835.959 L700.155 833.897 L699.928 833.785 L699.7 833.193 L699.472 833.578 L699.244 832.068 L699.017 831.304 L698.789 829.43 L698.561 828.768 L698.333 828.753 L698.105 828.912 L697.878 827.945 L697.65 828.567 L697.422 830.445 L697.194 829.302 L696.967 827.405 L696.739 827.238 L696.511 826.619 L696.283 825.891 L696.055 827.587 L695.828 828.44 L695.6 826.625 L695.372 825.537 L695.144 823.968 L694.917 821.783 L694.689 820.642 L694.461 820.354 L694.233 817.418 L694.006 817.09 L693.778 819.366 L693.55 819.951 L693.322 816.968 L693.094 815.221 L692.867 813.767 L692.639 813.498 L692.411 814.152 L692.183 817.031 L691.956 815.565 L691.728 813.943 L691.5 812.911 L691.272 812.046 L691.045 815.405 L690.817 815.669 L690.589 815.407 L690.361 814.386 L690.133 812.957 L689.906 813.677 L689.678 814.129 L689.45 813.017 L689.222 812.36 L688.995 810.744 L688.767 812.31 L688.539 812.249 L688.311 811.5 L688.084 810.108 L687.856 808.567 L687.628 807.093 L687.4 806.496 L687.172 804.107 L686.945 804.087 L686.717 803.043 L686.489 800.665 L686.261 800.217 L686.034 799.59 L685.806 799.732 L685.578 799.77 L685.35 801.018 L685.122 798.643 L684.895 800.794 L684.667 800.137 L684.439 798.125 L684.211 797.442 L683.984 796.834 L683.756 799.909 L683.528 802.679 L683.3 805.456 L683.073 806.877 L682.845 808.861 L682.617 807.595 L682.389 810.218 L682.161 811.611 L681.934 813.199 L681.706 813.012 L681.478 812.86 L681.25 813.747 L681.023 813.321 L680.795 815.433 L680.567 813.494 L680.339 812.486 L680.112 812.382 L679.884 812.031 L679.656 810.305 L679.428 809.552 L679.2 807.102 L678.973 807.272 L678.745 808.615 L678.517 808.524 L678.289 808.795 L678.062 811.094 L677.834 808.913 L677.606 809.22 L677.378 810.975 L677.151 808.382 L676.923 807.676 L676.695 809.844 L676.467 807.393 L676.239 805.856 L676.012 806.49 L675.784 808.155 L675.556 807.331 L675.328 806.989 L675.101 806.296 L674.873 808.877 L674.645 807.114 L674.417 808.637 L674.189 807.614 L673.962 810.591 L673.734 809.216 L673.506 807.773 L673.278 809.205 L673.051 809.768 L672.823 809.384 L672.595 810.028 L672.367 808.162 L672.14 809.194 L671.912 809.541 L671.684 809.599 L671.456 807.921 L671.228 807.86 L671.001 805.08 L670.773 805.586 L670.545 804.652 L670.317 802.907 L670.09 801.902 L669.862 801.305 L669.634 802.458 L669.406 800.363 L669.179 801.321 L668.951 800.464 L668.723 801.908 L668.495 803.433 L668.267 802.095 L668.04 804.468 L667.812 805.808 L667.584 805.494 L667.356 809.425 L667.129 810.099 L666.901 810.649 L666.673 809.839 L666.445 809.393 L666.218 809.736 L665.99 807.842 L665.762 807.842 L665.534 806.741 L665.306 811.412 L665.079 809.504 L664.851 809.396 L664.623 809.662 L664.395 809.811 L664.168 807.858 L663.94 809.995 L663.712 813.922 L663.484 812.152 L663.256 811.072 L663.029 810.818 L662.801 810.468 L662.573 809.66 L662.345 809.62 L662.118 811.635 L661.89 809.498 L661.662 809.04 L661.434 810.156 L661.207 810.274 L660.979 809.5 L660.751 809.28 L660.523 809.738 L660.295 806.84 L660.068 806.495 L659.84 806.096 L659.612 806.533 L659.384 804.15 L659.157 802.429 L658.929 802.361 L658.701 800.582 L658.473 804.075 L658.246 803.733 L658.018 802.157 L657.79 800.406 L657.562 798.935 L657.334 801.603 L657.107 801.171 L656.879 802.004 L656.651 800.846 L656.423 801.246 L656.196 803.062 L655.968 804.398 L655.74 803.897 L655.512 803.677 L655.285 804.516 L655.057 803.743 L654.829 804.149 L654.601 807.727 L654.373 809.828 L654.146 811.947 L653.918 810.852 L653.69 809.944 L653.462 811.878 L653.235 813.177 L653.007 812.979 L652.779 813.158 L652.551 814.531 L652.323 814.043 L652.096 815.752 L651.868 815.245 L651.64 814.069 L651.412 813.088 L651.185 814.73 L650.957 814.658 L650.729 815.443 L650.501 813.881 L650.274 814.476 L650.046 813.685 L649.818 814.031 L649.59 811.68 L649.362 814.715 L649.135 818.063 L648.907 815.588 L648.679 813.876 L648.451 812.573 L648.224 814.004 L647.996 813.999 L647.768 813.561 L647.54 813.542 L647.313 811.874 L647.085 810.065 L646.857 809.095 L646.629 808.059 L646.401 808.473 L646.174 807.315 L645.946 805.89 L645.718 802.933 L645.49 801.064 L645.263 804.812 L645.035 803.91 L644.807 803.232 L644.579 800.424 L644.352 799.921 L644.124 801.412 L643.896 803.652 L643.668 801.148 L643.44 802.493 L643.213 801.604 L642.985 800.752 L642.757 802.171 L642.529 802.605 L642.302 801.801 L642.074 801.517 L641.846 803.149 L641.618 800.651 L641.39 803.214 L641.163 802.23 L640.935 805.806 L640.707 805.243 L640.479 808.906 L640.252 808.438 L640.024 805.65 L639.796 804.784 L639.568 803.083 L639.341 805.377 L639.113 804.022 L638.885 803.992 L638.657 807.238 L638.429 804.778 L638.202 804.245 L637.974 804.85 L637.746 806.22 L637.518 807.138 L637.291 805.694 L637.063 807.872 L636.835 808.549 L636.607 808.461 L636.38 808.164 L636.152 810.022 L635.924 808.053 L635.696 806.001 L635.468 804.887 L635.241 802.975 L635.013 801.183 L634.785 799.861 L634.557 802.585 L634.33 801.738 L634.102 802.495 L633.874 800.047 L633.646 800.205 L633.419 801.226 L633.191 800.11 L632.963 800.002 L632.735 802.343 L632.507 803.272 L632.28 799.733 L632.052 799.56 L631.824 799.405 L631.596 798.238 L631.369 800.69 L631.141 800.295 L630.913 800.641 L630.685 805.75 L630.457 807.669 L630.23 805.302 L630.002 804.372 L629.774 802.449 L629.546 803.072 L629.319 802.804 L629.091 804.125 L628.863 802.184 L628.635 803.91 L628.408 803.111 L628.18 803.645 L627.952 802.668 L627.724 800.471 L627.496 798.439 L627.269 798.746 L627.041 798.778 L626.813 798.567 L626.585 796.808 L626.358 798.64 L626.13 800.889 L625.902 800.853 L625.674 797.983 L625.447 798.968 L625.219 800.19 L624.991 803.375 L624.763 800.658 L624.535 797.761 L624.308 797.958 L624.08 798.629 L623.852 797.643 L623.624 797.465 L623.397 798.782 L623.169 799.104 L622.941 800.647 L622.713 799.604 L622.486 798.477 L622.258 797.161 L622.03 794.693 L621.802 792.769 L621.574 791.01 L621.347 791.844 L621.119 793.643 L620.891 796.248 L620.663 797.76 L620.436 801.355 L620.208 802.457 L619.98 801.582 L619.752 798.757 L619.524 798.317 L619.297 798.035 L619.069 795.631 L618.841 793.575 L618.613 790.034 L618.386 792.117 L618.158 793.566 L617.93 792.846 L617.702 794.261 L617.475 794.818 L617.247 796.729 L617.019 797.022 L616.791 796.989 L616.563 796.689 L616.336 794.95 L616.108 794.93 L615.88 794.582 L615.652 793.028 L615.425 794.088 L615.197 795.158 L614.969 794.827 L614.741 798.827 L614.514 799.092 L614.286 797.566 L614.058 796.395 L613.83 799.796 L613.602 802.795 L613.375 803.139 L613.147 805.201 L612.919 806.153 L612.691 805.632 L612.464 805.695 L612.236 803.606 L612.008 803.358 L611.78 805.36 L611.553 806.725 L611.325 806.711 L611.097 806.169 L610.869 810.654 L610.641 812.568 L610.414 813.293 L610.186 812.642 L609.958 810.691 L609.73 816.21 L609.503 813.056 L609.275 813.753 L609.047 811.955 L608.819 812.014 L608.591 813.561 L608.364 812.145 L608.136 811.891 L607.908 810.641 L607.68 809.157 L607.453 809.219 L607.225 808.686 L606.997 807.991 L606.769 807.218 L606.542 806.538 L606.314 808.583 L606.086 807.568 L605.858 812.215 L605.63 813.186 L605.403 808.555 L605.175 808.643 L604.947 810.23 L604.719 811.439 L604.492 811.169 L604.264 807.762 L604.036 807.215 L603.808 803.246 L603.581 801.436 L603.353 800.163 L603.125 796.833 L602.897 800.862 L602.669 799.349 L602.442 798.636 L602.214 797.144 L601.986 797.067 L601.758 797.082 L601.531 798.51 L601.303 795.1 L601.075 795.05 L600.847 796.034 L600.62 798.468 L600.392 796.766 L600.164 794.564 L599.936 797.342 L599.708 797.092 L599.481 798.205 L599.253 803.34 L599.025 802.243 L598.797 803.413 L598.57 801.125 L598.342 800.789 L598.114 802.936 L597.886 804.21 L597.658 803.709 L597.431 804.688 L597.203 802.212 L596.975 801.014 L596.747 804.109 L596.52 804.155 L596.292 806.173 L596.064 809.654 L595.836 809.993 L595.609 808.632 L595.381 808.04 L595.153 806.15 L594.925 804.42 L594.697 805.018 L594.47 802.728 L594.242 804.538 L594.014 803.498 L593.786 802.456 L593.559 801.935 L593.331 798.913 L593.103 797.451 L592.875 796.394 L592.648 796.792 L592.42 794.675 L592.192 793.977 L591.964 793.356 L591.736 789.517 L591.509 792.535 L591.281 792.773 L591.053 791.812 L590.825 791.433 L590.598 790.455 L590.37 789.505 L590.142 790.532 L589.914 789.898 L589.687 790.68 L589.459 789.94 L589.231 791.165 L589.003 791.011 L588.775 790.911 L588.548 790.091 L588.32 786.834 L588.092 788.125 L587.864 785.283 L587.637 785.695 L587.409 786.925 L587.181 789.029 L586.953 789.058 L586.725 794.606 L586.498 796.704 L586.27 797.197 L586.042 795.794 L585.814 798.971 L585.587 796.256 L585.359 795.139 L585.131 793.687 L584.903 792.859 L584.676 791.259 L584.448 790.26 L584.22 788.011 L583.992 784.911 L583.764 784.557 L583.537 783.016 L583.309 780.058 L583.081 777.739 L582.853 776.486 L582.626 778.275 L582.398 779.893 L582.17 781.485 L581.942 779.607 L581.715 779.842 L581.487 779.971 L581.259 785.053 L581.031 785.353 L580.803 784.687 L580.576 783.479 L580.348 785.522 L580.12 786.992 L579.892 786.422 L579.665 785.235 L579.437 785.617 L579.209 783.948 L578.981 785.744 L578.754 783.68 L578.526 781.803 L578.298 781.59 L578.07 778.288 L577.842 777.53 L577.615 779.534 L577.387 780.352 L577.159 780.657 L576.931 783.192 L576.704 785.319 L576.476 786.925 L576.248 787.203 L576.02 790.537 L575.792 790.938 L575.565 790.179 L575.337 790.876 L575.109 789.979 L574.881 794.044 L574.654 792.469 L574.426 792.694 L574.198 794.705 L573.97 795.831 L573.743 795.11 L573.515 795.575 L573.287 799.91 L573.059 798.043 L572.831 800.617 L572.604 803.818 L572.376 803.518 L572.148 806.839 L571.92 806.09 L571.693 806.567 L571.465 805.672 L571.237 805.679 L571.009 801.51 L570.782 800.158 L570.554 799.803 L570.326 800.003 L570.098 797.38 L569.87 796.192 L569.643 796.348 L569.415 791.728 L569.187 790.308 L568.959 789.885 L568.732 795.436 L568.504 793.506 L568.276 794.784 L568.048 792.809 L567.821 789.797 L567.593 789.841 L567.365 796.103 L567.137 797.41 L566.909 794.886 L566.682 792.1 L566.454 793.905 L566.226 797.288 L565.998 798.655 L565.771 795.227 L565.543 795.405 L565.315 795.313 L565.087 794.874 L564.859 792.377 L564.632 790.597 L564.404 792.107 L564.176 791.422 L563.948 791.144 L563.721 791.862 L563.493 791.135 L563.265 794.122 L563.037 792.722 L562.81 791.064 L562.582 794.05 L562.354 793.056 L562.126 795.012 L561.898 791.801 L561.671 791.027 L561.443 791.285 L561.215 790.878 L560.987 788.443 L560.76 788.205 L560.532 787.605 L560.304 784.386 L560.076 788.601 L559.849 788.213 L559.621 789.02 L559.393 789.724 L559.165 792.92 L558.937 794.481 L558.71 794.448 L558.482 795.202 L558.254 799.25 L558.026 795.302 L557.799 797.54 L557.571 795.564 L557.343 797.414 L557.115 797.927 L556.888 794.511 L556.66 790.929 L556.432 788.327 L556.204 791.422 L555.976 789.793 L555.749 788.362 L555.521 787.929 L555.293 785.947 L555.065 782.779 L554.838 779.948 L554.61 779.639 L554.382 778.746 L554.154 777.844 L553.926 777.214 L553.699 777.108 L553.471 775.049 L553.243 775.309 L553.015 778.726 L552.788 778.477 L552.56 776.17 L552.332 774.954 L552.104 773.243 L551.877 769.688 L551.649 777.556 L551.421 777.383 L551.193 776.356 L550.965 774.978 L550.738 772.699 L550.51 767.93 L550.282 771.775 L550.054 773.252 L549.827 774.557 L549.599 772.754 L549.371 771.645 L549.143 771.422 L548.916 767.993 L548.688 766.489 L548.46 766.33 L548.232 766.035 L548.004 765.629 L547.777 765.317 L547.549 766.597 L547.321 766.158 L547.093 762.921 L546.866 759.41 L546.638 761.258 L546.41 762.162 L546.182 761.4 L545.955 763.884 L545.727 767.357 L545.499 766.81 L545.271 770.637 L545.043 775.109 L544.816 776.495 L544.588 777.603 L544.36 774.952 L544.132 781.657 L543.905 782.871 L543.677 780.403 L543.449 776.999 L543.221 774.253 L542.993 772.156 L542.766 770.746 L542.538 773.419 L542.31 772.941 L542.082 778.786 L541.855 776.577 L541.627 776.777 L541.399 781.027 L541.171 780.619 L540.944 781.442 L540.716 779.294 L540.488 779.55 L540.26 779.295 L540.032 780.089 L539.805 777.121 L539.577 778.581 L539.349 777.98 L539.121 776.803 L538.894 774.985 L538.666 778.279 L538.438 776.478 L538.21 773.429 L537.983 769.095 L537.755 773.031 L537.527 769.789 L537.299 768.094 L537.071 764.777 L536.844 762.819 L536.616 761.194 L536.388 761.824 L536.16 759.902 L535.933 760.113 L535.705 758.871 L535.477 756.754 L535.249 754.95 L535.022 753.496 L534.794 748.994 L534.566 750.716 L534.338 751.917 L534.11 752.652 L533.883 751.657 L533.655 749.626 L533.427 752.474 L533.199 754.244 L532.972 752.134 L532.744 754.461 L532.516 756.759 L532.288 754.381 L532.06 752.116 L531.833 754.972 L531.605 758.958 L531.377 759.076 L531.149 756.141 L530.922 759.524 L530.694 757.923 L530.466 761.11 L530.238 765.202 L530.011 764.709 L529.783 763.344 L529.555 761.37 L529.327 763.154 L529.099 768.826 L528.872 772.604 L528.644 773.419 L528.416 776.356 L528.188 782.516 L527.961 782.58 L527.733 783.605 L527.505 782.238 L527.277 784.622 L527.05 783.725 L526.822 782.946 L526.594 784.099 L526.366 778.244 L526.138 777.082 L525.911 782.169 L525.683 784.357 L525.455 779.601 L525.227 783.584 L525 784.752 L524.772 781.72 L524.544 777.619 L524.316 777.501 L524.089 776.615 L523.861 779.527 L523.633 778.642 L523.405 780.788 L523.177 786.525 L522.95 787.892 L522.722 783.432 L522.494 781.194 L522.266 779.581 L522.039 778.926 L521.811 777.622 L521.583 779.429 L521.355 776.069 L521.127 775.532 L520.9 774.381 L520.672 776.202 L520.444 774.614 L520.216 774.919 L519.989 772.616 L519.761 769.829 L519.533 772.386 L519.305 773.422 L519.078 776.093 L518.85 779.931 L518.622 781.283 L518.394 782.702 L518.166 782.396 L517.939 779.406 L517.711 781.625 L517.483 784.005 L517.255 787.571 L517.028 786.687 L516.8 783.461 L516.572 780.285 L516.344 780.693 L516.117 782.994 L515.889 781.145 L515.661 778.067 L515.433 774.815 L515.205 774.761 L514.978 773.596 L514.75 768.899 L514.522 763.657 L514.294 763.415 L514.067 759.428 L513.839 761.045 L513.611 759.75 L513.383 765.336 L513.156 762.404 L512.928 765.145 L512.7 762.458 L512.472 761.481 L512.244 759.12 L512.017 757.49 L511.789 756.175 L511.561 751.644 L511.333 749.816 L511.106 748.349 L510.878 751.099 L510.65 751.566 L510.422 753.657 L510.194 750.243 L509.967 748.005 L509.739 746.433 L509.511 746.105 L509.283 747.681 L509.056 744.191 L508.828 747.663 L508.6 745.372 L508.372 750.415 L508.145 758.942 L507.917 758.184 L507.689 756.027 L507.461 756.389 L507.233 755.899 L507.006 754.322 L506.778 752.562 L506.55 753.791 L506.322 756.707 L506.095 756.85 L505.867 752.981 L505.639 749.841 L505.411 751.166 L505.184 746.722 L504.956 746.654 L504.728 752.963 L504.5 757.205 L504.272 756.886 L504.045 758.604 L503.817 752.712 L503.589 753.526 L503.361 756.072 L503.134 761.524 L502.906 759.615 L502.678 756.162 L502.45 756.289 L502.223 755.983 L501.995 751.776 L501.767 753.523 L501.539 751.379 L501.311 747.941 L501.084 747.769 L500.856 744.256 L500.628 744.066 L500.4 747.741 L500.173 743.932 L499.945 741.587 L499.717 740.605 L499.489 740.299 L499.261 740.161 L499.034 738.915 L498.806 733.393 L498.578 731.122 L498.35 724.952 L498.123 725.608 L497.895 728.039 L497.667 727.273 L497.439 727.008 L497.212 724.967 L496.984 722.9 L496.756 719.977 L496.528 721.311 L496.3 720.681 L496.073 715.829 L495.845 715.75 L495.617 714.553 L495.389 713.651 L495.162 710.934 L494.934 709.619 L494.706 707.902 L494.478 712.251 L494.251 712.307 L494.023 715.394 L493.795 713.375 L493.567 712.112 L493.339 711.029 L493.112 716.985 L492.884 714.523 L492.656 712.858 L492.428 713.326 L492.201 712.81 L491.973 711.245 L491.745 709.136 L491.517 712.199 L491.29 708.762 L491.062 718.538 L490.834 718.602 L490.606 716.134 L490.378 714.72 L490.151 712.298 L489.923 709.851 L489.695 709.751 L489.467 709.107 L489.24 705.469 L489.012 704.699 L488.784 711.257 L488.556 706.357 L488.328 710.352 L488.101 711.542 L487.873 712.648 L487.645 714.405 L487.417 712.385 L487.19 719.948 L486.962 720.57 L486.734 717.312 L486.506 713.617 L486.279 716.712 L486.051 715.669 L485.823 708.667 L485.595 704.948 L485.367 703.964 L485.14 709.975 L484.912 711.428 L484.684 715.248 L484.456 710.786 L484.229 708.168 L484.001 702.383 L483.773 708.323 L483.545 706.395 L483.318 706.871 L483.09 705.05 L482.862 703.342 L482.634 704.715 L482.406 697.15 L482.179 694.775 L481.951 691.599 L481.723 701.444 L481.495 697.519 L481.268 693.474 L481.04 691.96 L480.812 696.822 L480.584 694.334 L480.357 687.959 L480.129 687.908 L479.901 688.756 L479.673 691.03 L479.445 690.727 L479.218 692.826 L478.99 689.24 L478.762 691.742 L478.534 699.075 L478.307 698.432 L478.079 697.226 L477.851 699.788 L477.623 698.492 L477.395 700.87 L477.168 696.775 L476.94 695.404 L476.712 685.823 L476.484 682.621 L476.257 687.695 L476.029 686.655 L475.801 680.6 L475.573 675.833 L475.346 672.116 L475.118 665.397 L474.89 665.243 L474.662 663.017 L474.434 661.11 L474.207 661.252 L473.979 665.359 L473.751 666.718 L473.523 661.468 L473.296 662.599 L473.068 659.93 L472.84 657.881 L472.612 654.057 L472.385 651.771 L472.157 656.882 L471.929 664.288 L471.701 664.781 L471.473 663.692 L471.246 661.954 L471.018 654.376 L470.79 654.95 L470.562 650.498 L470.335 650.153 L470.107 653.912 L469.879 650.746 L469.651 662.082 L469.424 660.961 L469.196 659.712 L468.968 665.596 L468.74 668.783 L468.512 671.932 L468.285 667.012 L468.057 666.912 L467.829 664.486 L467.601 665.923 L467.374 663.7 L467.146 662.552 L466.918 664.308 L466.69 660.875 L466.462 657.552 L466.235 653.499 L466.007 650.748 L465.779 653.333 L465.551 650.79 L465.324 646.943 L465.096 643.987 L464.868 642.645 L464.64 642.17 L464.413 644.881 L464.185 642.362 L463.957 640.794 L463.729 635.332 L463.501 638.534 L463.274 636.577 L463.046 636.594 L462.818 640.162 L462.59 638.893 L462.363 640.188 L462.135 640.273 L461.907 637.128 L461.679 642.898 L461.452 636.652 L461.224 634.5 L460.996 631.96 L460.768 630.052 L460.54 625.896 L460.313 624.039 L460.085 624.986 L459.857 619.96 L459.629 624.504 L459.402 630.257 L459.174 626.858 L458.946 623.762 L458.718 622.524 L458.491 616.176 L458.263 624.68 L458.035 627.715 L457.807 626.03 L457.579 639.236 L457.352 641.647 L457.124 635.761 L456.896 642.077 L456.668 639.772 L456.441 631.965 L456.213 634.744 L455.985 637.545 L455.757 638.933 L455.53 637.955 L455.302 641.454 L455.074 645.266 L454.846 644.604 L454.618 637.71 L454.391 636.897 L454.163 638.865 L453.935 639.016 L453.707 639.382 L453.48 637.461 L453.252 639.977 L453.024 639.809 L452.796 638.611 L452.568 637.632 L452.341 640.694 L452.113 635.389 L451.885 637.658 L451.657 638.556 L451.43 640.614 L451.202 635.766 L450.974 631.725 L450.746 629.691 L450.519 635.734 L450.291 631.254 L450.063 630.489 L449.835 630.568 L449.607 627.643 L449.38 639.681 L449.152 640.03 L448.924 634.932 L448.696 634.867 L448.469 632.356 L448.241 636.848 L448.013 638.02 L447.785 635.146 L447.558 634.927 L447.33 633.631 L447.102 631.133 L446.874 636.568 L446.646 633.616 L446.419 627.718 L446.191 624.534 L445.963 623.498 L445.735 626.592 L445.508 621.839 L445.28 623.58 L445.052 621.896 L444.824 621.62 L444.597 623.433 L444.369 617.911 L444.141 619.139 L443.913 616.077 L443.685 614.373 L443.458 610.449 L443.23 610.733 L443.002 614.109 L442.774 612.373 L442.547 610.665 L442.319 614.057 L442.091 612.171 L441.863 614.98 L441.635 616.406 L441.408 625.058 L441.18 622.277 L440.952 622.429 L440.724 619.012 L440.497 621.598 L440.269 618.72 L440.041 620.303 L439.813 615.784 L439.586 621.432 L439.358 625.334 L439.13 623.679 L438.902 622.133 L438.674 620.317 L438.447 614.358 L438.219 612.837 L437.991 609.378 L437.763 601.535 L437.536 597.75 L437.308 595.539 L437.08 597.121 L436.852 595.114 L436.625 597.588 L436.397 597.368 L436.169 596.532 L435.941 595.139 L435.713 593.388 L435.486 594.561 L435.258 601.244 L435.03 603.788 L434.802 601.837 L434.575 601.137 L434.347 604.399 L434.119 604.67 L433.891 603.15 L433.664 594.632 L433.436 592.75 L433.208 592.332 L432.98 599.299 L432.752 596.147 L432.525 595.038 L432.297 604.236 L432.069 599.209 L431.841 593.722 L431.614 599.744 L431.386 598.568 L431.158 602.818 L430.93 602.447 L430.702 601.995 L430.475 598.647 L430.247 596.774 L430.019 592.299 L429.791 591.23 L429.564 588.199 L429.336 582.943 L429.108 580.694 L428.88 574.913 L428.653 576.59 L428.425 575.088 L428.197 573.347 L427.969 571.084 L427.741 572.667 L427.514 572.895 L427.286 569.638 L427.058 561.735 L426.83 558.694 L426.603 567.607 L426.375 561.74 L426.147 565.099 L425.919 564.265 L425.692 565.434 L425.464 560.833 L425.236 559.472 L425.008 553.237 L424.78 557.03 L424.553 553.971 L424.325 555.867 L424.097 550.733 L423.869 547.678 L423.642 547.47 L423.414 546.849 L423.186 547.481 L422.958 541.367 L422.731 537.58 L422.503 547.052 L422.275 544.768 L422.047 544.378 L421.819 546.357 L421.592 551.719 L421.364 555.88 L421.136 562.143 L420.908 561.896 L420.681 563.267 L420.453 565.461 L420.225 568.088 L419.997 563.949 L419.769 560.755 L419.542 564.869 L419.314 555.594 L419.086 557.527 L418.858 558.629 L418.631 563.599 L418.403 561.934 L418.175 557.222 L417.947 548.585 L417.72 540.795 L417.492 539.402 L417.264 535.267 L417.036 533.869 L416.808 525.477 L416.581 522.87 L416.353 526.309 L416.125 529.161 L415.897 533.099 L415.67 530.044 L415.442 534.009 L415.214 529.563 L414.986 529.224 L414.759 536.769 L414.531 531.634 L414.303 530.936 L414.075 530.166 L413.847 530.417 L413.62 534.639 L413.392 529.733 L413.164 525.528 L412.936 523.303 L412.709 521.624 L412.481 523.879 L412.253 517.247 L412.025 512.54 L411.798 506.817 L411.57 502.736 L411.342 504.911 L411.114 503.958 L410.886 506.16 L410.659 499.7 L410.431 493.361 L410.203 491.687 L409.975 500.745 L409.748 499.256 L409.52 506.025 L409.292 506.8 L409.064 502.289 L408.836 501.623 L408.609 495.85 L408.381 490.482 L408.153 485.888 L407.925 483.762 L407.698 482.759 L407.47 483.079 L407.242 484.422 L407.014 476.353 L406.787 474.826 L406.559 474.903 L406.331 472.857 L406.103 478.599 L405.875 476.091 L405.648 490.091 L405.42 481.059 L405.192 476.672 L404.964 475.67 L404.737 479.413 L404.509 479.118 L404.281 482.826 L404.053 484.915 L403.826 483.054 L403.598 490.384 L403.37 491.793 L403.142 481.594 L402.914 476.325 L402.687 474.842 L402.459 480.171 L402.231 481.186 L402.003 477.45 L401.776 485.752 L401.548 488.511 L401.32 479.932 L401.092 482.487 L400.865 479.14 L400.637 470.482 L400.409 480.881 L400.181 479.011 L399.953 479.598 L399.726 491.81 L399.498 483.489 L399.27 481.29 L399.042 487.201 L398.815 487.009 L398.587 482.282 L398.359 479.669 L398.131 483.244 L397.903 476.435 L397.676 490.044 L397.448 481.504 L397.22 478.489 L396.992 484.281 L396.765 484.225 L396.537 489.667 L396.309 486.382 L396.081 479.454 L395.854 476.36 L395.626 471.09 L395.398 472.685 L395.17 466.933 L394.942 457.77 L394.715 458.225 L394.487 448.936 L394.259 441.428 L394.031 437.019 L393.804 438.541 L393.576 435.5 L393.348 432.38 L393.12 428.911 L392.893 425.525 L392.665 429.127 L392.437 427.186 L392.209 424.676 L391.981 420.642 L391.754 415.215 L391.526 415.721 L391.298 423.512 L391.07 422.633 L390.843 414.64 L390.615 420.642 L390.387 413.931 L390.159 409.158 L389.932 404.868 L389.704 395.242 L389.476 394.987 L389.248 390.349 L389.02 386.405 L388.793 376.635 L388.565 379.728 L388.337 378.152 L388.109 381.792 L387.882 374.439 L387.654 376.445 L387.426 369.519 L387.198 373.349 L386.97 375.649 L386.743 374.556 L386.515 380.397 L386.287 374.068 L386.059 385.673 L385.832 380.503 L385.604 383.181 L385.376 411.29 L385.148 405.305 L384.921 405.367 L384.693 394.768 L384.465 386.282 L384.237 397.682 L384.009 389.692 L383.782 385.514 L383.554 388.879 L383.326 388.459 L383.098 397.275 L382.871 396.559 L382.643 391.411 L382.415 386.9 L382.187 379.37 L381.96 387.109 L381.732 390.261 L381.504 381.974 L381.276 380.245 L381.048 388.097 L380.821 375.995 L380.593 370.93 L380.365 359.989 L380.137 373.973 L379.91 370.211 L379.682 362.136 L379.454 361.322 L379.226 350.478 L378.999 346.143 L378.771 355.796 L378.543 345.799 L378.315 343.913 L378.087 350.024 L377.86 339.277 L377.632 346.395 L377.404 344.666 L377.176 338.509 L376.949 335.012 L376.721 340.164 L376.493 328.051 L376.265 329.823 L376.037 315.875 L375.81 324.278 L375.582 320.221 L375.354 320.81 L375.126 312.853 L374.899 318.138 L374.671 318.659 L374.443 317.031 L374.215 317.711 L373.988 313.112 L373.76 314.326 L373.532 319.798 L373.304 320.277 L373.076 327.463 L372.849 339.84 L372.621 338.757 L372.393 327.73 L372.165 335.009 L371.938 330.277 L371.71 320.605 L371.482 325.088 L371.254 314.089 L371.027 307.123 L370.799 296.452 L370.571 301.768 L370.343 304.76 L370.115 303.532 L369.888 300.531 L369.66 291.058 L369.432 293.758 L369.204 298.567 L368.977 286.284 L368.749 278.837 L368.521 264.336 L368.293 264.367 L368.066 282.847 L367.838 283.082 L367.61 274.18 L367.382 264.063 L367.154 249.405 L366.927 245.957 L366.699 260.45 L366.471 266.512 L366.243 258.591 L366.016 254.893 L365.788 273.179 L365.56 275.225 L365.332 268.617 L365.104 268.688 L364.877 271.343 L364.649 269.378 L364.421 273.96 L364.193 271.721 L363.966 276.128 L363.738 274.117 L363.51 276.722 L363.282 283.658 L363.055 275.456 L362.827 271.661 L362.599 257.796 L362.371 256.908 L362.143 250.432 L361.916 235.773 L361.688 227.859 L361.46 218.376 L361.232 213.652 L361.005 200.498 L360.777 212.921 L360.549 210.681 L360.321 213.219 L360.094 199.366 L359.866 212.463 L359.638 208.278 L359.41 219.994 L359.182 227.372 L358.955 215.548 L358.727 215.724 L358.499 223.371 L358.271 215.29 L358.044 214.543 L357.816 208.72 L357.588 205.641 L357.36 202.794 L357.133 202.431 L356.905 192.066 L356.677 193.616 L356.449 185.762 L356.221 189.117 L355.994 175.062 L355.766 171.929 L355.538 170.983 L355.31 167.296 L355.083 164.396 L354.855 151.595 L354.627 167.61 L354.399 173.099 L354.171 165.436 L353.944 163.716 L353.716 163.514 L353.488 152.003 L353.26 144.695 L353.033 144.899 L352.805 137.107 L352.577 122.966 L352.349 121.051 L352.122 133.624 L351.894 151.539 L351.666 152.622 L351.438 179.255 L351.21 165.959 L350.983 159.689 L350.755 155.68 L350.527 138.542 L350.299 125.155 L350.072 132.79 L349.844 120.961 L349.616 150.149 L349.388 153.65 L349.161 140.004 L348.933 142.919 L348.705 128.054 L348.477 127.135 L348.249 136.935 L348.022 138.83 L347.794 130.666 L347.566 157.094 L347.338 159.321 L347.111 181.938 L346.883 202.417 L346.655 204.488 L346.427 201.366 L346.2 185.485 L345.972 174.341 L345.744 179.619 L345.516 166.215 L345.288 150.887 L345.061 141.508 L344.833 123.353 L344.605 112.989 L344.377 119.695 L344.15 152.585 L343.922 131.396 L343.694 113.129 L343.466 138.522 L343.238 127.747 L343.011 150.512 L342.783 156.637 L342.555 138.291 L342.327 125.265 L342.1 135.154 L341.872 143.953 L341.644 138.055 L341.416 146.754 L341.189 145.826 L340.961 140.115 L340.733 136.774 L340.505 133.041 L340.277 153.291 L340.05 160.617 L339.822 160.016 L339.594 161.922 L339.366 154.305 L339.139 154.084 L338.911 171.872 L338.683 158.616 L338.455 141.018 L338.228 154.954 L338 182.834 L337.772 189.397 L337.544 209.21 L337.316 198.802 L337.089 193.899 L336.861 194.066 L336.633 199.666 L336.405 189.711 L336.178 193.304 L335.95 180.627 L335.722 165.732 L335.494 177.607 L335.267 201.665 L335.039 196.4 L334.811 276.413 L334.583 264.592 L334.355 304.91 L334.128 286.949 L333.9 269.879 L333.672 261.603 L333.444 269.903 L333.217 295.464 L332.989 339.405 L332.761 370.353 L332.533 367.348 L332.305 360.112 L332.078 363.084 L331.85 357.65 L331.622 397.646 L331.394 387.993 L331.167 373.762 L330.939 377.076 L330.711 360.2 L330.483 365.245 L330.256 353.197 L330.028 353.222 L329.8 396.969 L329.572 395.607 L329.344 367.656 L329.117 354.385 L328.889 357.025 L328.661 361.04 L328.433 354.832 L328.206 349.045 L327.978 330.716 L327.75 307.485 L327.522 287.26 L327.295 272.55 L327.067 253.207 L326.839 242.559 L326.611 227.874 L326.383 230.647 L326.156 214.416 L325.928 199.837 L325.7 173.384 L325.472 151.718 L325.245 159.796 L325.017 151.014 L324.789 134.889 L324.561 136.801 L324.334 112.307 L324.106 117.199 L323.878 119.49 L323.65 109.921 L323.422 79.5447 L323.195 53.2392 L322.967 54.4164 L322.739 36.9633 L322.511 39.6608 L322.284 71.0612 L322.056 71.4681 L321.828 49.8501 L321.6 76.4921 L321.372 50.8046 L321.145 82.7026 L320.917 95.1604 L320.689 79.1985 L320.461 50.8099 L320.234 43.7841 L320.006 86.7321 L319.778 85.3162 L319.55 56.6361 L319.323 24.9038 L319.095 16.603 L318.867 20.8578 L318.639 18.0522 L318.411 -1.0037 L318.184 -24.4921 L317.956 -17.4756 L317.728 49.9869 L317.5 160.429 L317.273 138.191 L317.045 127.544 L316.817 128.181 L316.589 157.158 L316.362 177.79 L316.134 175.807 L315.906 162.63 L315.678 133.034 L315.45 100.574 L315.223 56.9402 L314.995 22.2456 L314.767 43.053 L314.539 8.66762 L314.312 -12.5963 L314.084 -59.5565 L313.856 -52.3346 L313.628 -79.7793 L313.401 -98.138 L313.173 -161.432 L312.945 -65.9518 L312.717 21.5078 L312.489 23.1555 L312.262 70.9285 L312.034 59.5282 L311.806 46.2888 L311.578 56.7129 L311.351 76.5376 L311.123 47.9667 L310.895 45.387 L310.667 65.6593 L310.439 8.09604 L310.212 23.0407 L309.984 -23.6657 L309.756 -63.4874 L309.528 -72.6299 L309.301 -94.4704 L309.073 -0.174542 L308.845 -35.1687 L308.617 -18.8531 L308.39 -16.7039 L308.162 -14.3398 L307.934 -20.8773 L307.706 -62.1691 L307.478 -82.626 L307.251 -145.237 L307.023 -168.325 L306.795 -185.545 L306.567 -177.064 L306.34 -137.528 L306.112 -76.7955 L305.884 -118.118 L305.656 -153.255 L305.429 -197.471 L305.201 -259.811 L304.973 -251.556 L304.745 -298.071 L304.517 -310.122 L304.29 -262.707 L304.062 -309.974 L303.834 -268.666 L303.606 -205.904 L303.379 -180.559 L303.151 -149.733 L302.923 -132.92 L302.695 -141.158 L302.468 -35.3484 L302.24 -83.9515 L302.012 -137.696 L301.784 -251.322 L301.556 -259.201 L301.329 -147.664 L301.101 -187.876 L300.873 -301.075 L300.645 -182.136 L300.418 -186.465 L300.19 -350.653 L299.962 -322.892 L299.734 -274.919 L299.506 -344.708 L299.279 -351.036 L299.051 -387.793 L298.823 -541.995 L298.595 -402.804 L298.368 -380.164 L298.14 -484.193 L297.912 -320.271 L297.684 -456.715 L297.457 -109.216 L297.229 539.956 L297.001 563.91 L296.773 503.956 L296.545 1550.89 L296.318 1197.32 L296.09 1649.14 L295.862 1284.79 L295.634 854.729 L295.407 27.1769 L295.179 892.109 L294.951 4196.61  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="0.5"/>
<polyline clip-path="url(#clip002)" style="stroke:#009af9; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="294.951,4196.61 295.179,3080.22 295.407,2186.65 295.634,2502.22 295.862,2567.29 296.09,3013.57 296.318,2603.24 296.545,2901.59 296.773,2250.16 297.001,2140.49 297.229,1994.46 297.457,1516.16 297.684,1178.41 297.912,1193.86 298.14,984.116 298.368,993.435 298.595,900.312 298.823,730.751 299.051,832.515 299.279,807.673 299.506,761.201 299.734,780.18 299.962,698.058 300.19,634.692 300.418,843.28 300.645,809.433 300.873,684.153 301.101,779.872 301.329,786.2 301.556,670.88 301.784,649.254 302.012,761.708 302.24,789.468 302.468,813.277 302.695,708.208 302.923,693.05 303.151,656.748 303.379,609.79 303.606,568.214 303.834,498.221 304.062,445.168 304.29,476.847 304.517,420.442 304.745,415.755 304.973,449.322 305.201,427.05 305.429,483.74 305.656,516.742 305.884,539.59 306.112,570.458 306.34,508.445 306.567,462.684 306.795,443.179 307.023,448.742 307.251,460.982 307.478,526.266 307.706,536.266 307.934,571.099 308.162,567.558 308.39,555.872 308.617,544.685 308.845,521.125 309.073,550.171 309.301,469.702 309.528,483.448 309.756,484.102 309.984,520.347 310.212,566.61 310.439,545.348 310.667,610.243 310.895,584.594 311.123,579.718 311.351,602.922 311.578,578.193 311.806,561.774 312.034,568.349 312.262,573.187 312.489,527.096 312.717,519.266 312.945,446.06 313.173,367.273 313.401,446.906 313.628,459.196 313.856,482.069 314.084,469.132 314.312,518.955 314.539,535.016 314.767,567.194 314.995,542.686 315.223,575.591 315.45,621.238 315.678,651.386 315.906,678.031 316.134,685.931 316.362,682.558 316.589,658.806 316.817,628.233 317.045,622.591 317.273,628.342 317.5,647.035 317.728,560.642 317.956,501.338 318.184,489.76 318.411,509.791 318.639,524.815 318.867,522.832 319.095,514.155 319.323,517.881 319.55,548.819 319.778,575.88 320.006,572.892 320.234,532.155 320.461,534.869 320.689,561.911 320.917,574.27 321.145,558.65 321.372,526.782 321.6,550.751 321.828,523.198 322.056,542.354 322.284,538.113 322.511,506.893 322.739,500.548 322.967,515.135 323.195,510.354 323.422,535.738 323.65,566.509 323.878,572.657 324.106,566.977 324.334,558.893 324.561,582.342 324.789,577.158 325.017,590.754 325.245,596.381 325.472,585.574 325.7,605.872 325.928,632.261 326.156,644.348 326.383,658.358 326.611,652.664 326.839,665.021 327.067,672.973 327.295,690.838 327.522,703.357 327.75,722.382 327.978,745.176 328.206,761.994 328.433,765.024 328.661,768.528 328.889,761.99 329.117,756.788 329.344,768.007 329.572,797.517 329.8,796.268 330.028,757.243 330.256,754.652 330.483,764.612 330.711,757.291 330.939,772.836 331.167,767.196 331.394,779.749 331.622,787.254 331.85,751.241 332.078,754.314 332.305,749.099 332.533,754.109 332.761,754.793 332.989,725.763 333.217,686.979 333.444,662.142 333.672,652.042 333.9,658.232 334.128,674.339 334.355,691.54 334.583,655.458 334.811,665.555 335.039,602.559 335.267,605.579 335.494,581.925 335.722,568.64 335.95,582.18 336.178,593.233 336.405,587.601 336.633,595.706 336.861,588.233 337.089,585.965 337.316,588.807 337.544,597.509 337.772,577.602 338,569.33 338.228,542.897 338.455,528.033 338.683,545.065 338.911,557.04 339.139,538.915 339.366,537.182 339.594,543.023 339.822,539.258 340.05,537.958 340.277,529.138 340.505,509.075 340.733,510.96 340.961,512.465 341.189,516.428 341.416,515.547 341.644,505.574 341.872,509.786 342.1,499.754 342.327,488.756 342.555,500.819 342.783,519.245 343.011,511.709 343.238,489.799 343.466,499.366 343.694,475.36 343.922,493.742 344.15,515.851 344.377,485.994 344.605,477.952 344.833,487.096 345.061,505.368 345.288,513.45 345.516,528.324 345.744,540.957 345.972,534.306 346.2,544.4 346.427,560.003 346.655,561.561 346.883,557.996 347.111,538.107 347.338,516.471 347.566,512.776 347.794,488.104 348.022,494.993 348.249,491.64 348.477,480.955 348.705,480.379 348.933,494.916 349.161,490.625 349.388,503.754 349.616,498.926 349.844,472.217 350.072,483.267 350.299,474.605 350.527,487.477 350.755,504.86 350.983,507.49 351.21,512.494 351.438,525.308 351.666,500.658 351.894,498.218 352.122,480.639 352.349,467.631 352.577,468.18 352.805,482.036 353.033,488.728 353.26,487.197 353.488,493.39 353.716,504.245 353.944,503.141 354.171,503.559 354.399,510.18 354.627,503.65 354.855,487.799 355.083,500.21 355.31,501.862 355.538,504.328 355.766,504.028 355.994,505.946 356.221,519.907 356.449,515.445 356.677,522.365 356.905,519.651 357.133,529.374 357.36,528.55 357.588,530.232 357.816,532.159 358.044,536.953 358.271,536.544 358.499,543.79 358.727,535.422 358.955,534.12 359.182,545.616 359.41,537.512 359.638,525.548 359.866,528.681 360.094,515.533 360.321,529.468 360.549,525.911 360.777,527.07 361.005,514.529 361.232,527.662 361.46,531.399 361.688,540.303 361.916,547.473 362.143,562.463 362.371,568.082 362.599,567.924 362.827,581.974 363.055,584.783 363.282,592.307 363.51,584.708 363.738,581.163 363.966,582.17 364.193,576.93 364.421,578.18 364.649,572.79 364.877,573.773 365.104,570.219 365.332,569.181 365.56,575.04 365.788,572.088 366.016,554.773 366.243,557.564 366.471,564.879 366.699,558.162 366.927,544.02 367.154,546.572 367.382,561.795 367.61,571.602 367.838,580.037 368.066,578.887 368.293,561.459 368.521,560.515 368.749,575.575 368.977,582.415 369.204,594.785 369.432,589.269 369.66,585.75 369.888,594.873 370.115,597.013 370.343,597.356 370.571,593.578 370.799,587.621 371.027,598.155 371.254,604.522 371.482,615.449 371.71,610.283 371.938,619.685 372.165,623.671 372.393,615.942 372.621,626.932 372.849,627.174 373.076,614.959 373.304,607.329 373.532,606.03 373.76,599.978 373.988,597.969 374.215,601.853 374.443,600.375 374.671,601.199 374.899,599.886 375.126,594.035 375.354,601.588 375.582,600.22 375.81,603.566 376.037,594.897 376.265,609.542 376.493,607.033 376.721,619.428 376.949,613.725 377.176,616.505 377.404,622.113 377.632,623.086 377.86,615.599 378.087,626.401 378.315,619.837 378.543,620.981 378.771,630.922 378.999,621.187 379.226,624.875 379.454,635.819 379.682,635.897 379.91,643.663 380.137,646.763 380.365,633.324 380.593,644.403 380.821,648.884 381.048,661.362 381.276,653.252 381.504,654.272 381.732,662.301 381.96,658.537 382.187,650.544 382.415,657.735 382.643,661.653 382.871,666.252 383.098,666.272 383.326,657.335 383.554,657.066 383.782,653.126 384.009,656.712 384.237,664.453 384.465,653.276 384.693,661.582 384.921,672.341 385.148,671.61 385.376,677.146 385.604,652.673 385.832,649.398 386.059,654.051 386.287,642.7 386.515,648.612 386.743,642.385 386.97,642.817 387.198,639.919 387.426,635.57 387.654,642.156 387.882,639.55 388.109,646.617 388.337,642.458 388.565,643.398 388.793,639.763 389.02,649.604 389.248,652.997 389.476,657.128 389.704,656.755 389.932,666.444 390.159,670.214 390.387,674.501 390.615,680.894 390.843,674.57 391.07,682.402 391.298,682.668 391.526,674.733 391.754,673.628 391.981,678.636 392.209,682.159 392.437,684.096 392.665,685.452 392.893,681.379 393.12,684.234 393.348,687.178 393.576,689.761 393.804,692.263 394.031,690.191 394.259,694.14 394.487,701.48 394.715,710.863 394.942,709.842 395.17,719.083 395.398,724.487 395.626,722.358 395.854,727.244 396.081,729.825 396.309,736.538 396.537,739.322 396.765,733.576 396.992,733.078 397.22,727.017 397.448,729.529 397.676,738.087 397.903,725.213 398.131,731.809 398.359,727.812 398.587,729.914 398.815,734.245 399.042,733.898 399.27,727.743 399.498,729.428 399.726,737.757 399.953,726.078 400.181,724.968 400.409,726.321 400.637,716.197 400.865,724.922 401.092,727.808 401.32,724.799 401.548,733.442 401.776,730.242 402.003,721.959 402.231,725.263 402.459,723.749 402.687,718.153 402.914,719.131 403.142,724.084 403.37,734.66 403.598,732.77 403.826,725.369 404.053,726.74 404.281,724.199 404.509,720.122 404.737,719.918 404.964,715.812 405.192,716.321 405.42,720.344 405.648,729.567 405.875,716.454 406.103,718.503 406.331,712.555 406.559,714.133 406.787,713.572 407.014,714.622 407.242,722.74 407.47,720.942 407.698,720.146 407.925,720.674 408.153,722.346 408.381,726.614 408.609,731.722 408.836,737.275 409.064,737.471 409.292,741.658 409.52,740.429 409.748,733.58 409.975,734.615 410.203,725.742 410.431,726.967 410.659,733.162 410.886,739.494 411.114,736.891 411.342,737.391 411.57,734.817 411.798,738.562 412.025,744.088 412.253,748.509 412.481,755.048 412.709,752.405 412.936,753.652 413.164,755.461 413.392,759.35 413.62,763.997 413.847,759.505 414.075,758.82 414.303,759.155 414.531,759.42 414.759,764.325 414.986,756.824 415.214,756.733 415.442,760.897 415.67,756.656 415.897,759.346 416.125,755.135 416.353,751.945 416.581,748.204 416.808,750.433 417.036,759.038 417.264,760.025 417.492,763.869 417.72,764.854 417.947,772.767 418.175,781.67 418.403,786.139 418.631,787.405 418.858,782.252 419.086,780.759 419.314,778.463 419.542,788.138 419.769,783.781 419.997,786.639 420.225,790.502 420.453,787.548 420.681,785.008 420.908,783.262 421.136,783.11 421.364,776.797 421.592,772.406 421.819,766.911 422.047,764.585 422.275,764.581 422.503,766.506 422.731,757.372 422.958,760.876 423.186,766.921 423.414,765.909 423.642,766.143 423.869,765.965 424.097,768.702 424.325,773.67 424.553,771.436 424.78,774.18 425.008,770.152 425.236,776.348 425.464,777.339 425.692,781.735 425.919,780.211 426.147,780.672 426.375,777.057 426.603,782.851 426.83,774.23 427.058,776.967 427.286,785.088 427.514,788.054 427.741,787.46 427.969,785.543 428.197,787.473 428.425,788.866 428.653,790.017 428.88,788.013 429.108,793.727 429.336,795.649 429.564,800.784 429.791,803.523 430.019,804.239 430.247,808.524 430.475,810.064 430.702,813.144 430.93,813.242 431.158,813.261 431.386,808.838 431.614,809.671 431.841,803.634 432.069,809.04 432.297,813.939 432.525,805.109 432.752,805.876 432.98,808.759 433.208,801.881 433.436,801.954 433.664,803.515 433.891,812.421 434.119,813.612 434.347,813.003 434.575,809.513 434.802,809.875 435.03,811.513 435.258,808.705 435.486,802.091 435.713,800.604 435.941,802.038 436.169,803.109 436.397,803.615 436.625,803.503 436.852,800.77 437.08,802.474 437.308,800.597 437.536,802.516 437.763,806.094 437.991,814.234 438.219,817.465 438.447,818.676 438.674,824.647 438.902,826.161 439.13,827.399 439.358,828.751 439.586,824.685 439.813,819.021 440.041,823.402 440.269,821.534 440.497,824.16 440.724,821.334 440.952,824.533 441.18,824.069 441.408,826.597 441.635,818.291 441.863,816.58 442.091,813.546 442.319,815.145 442.547,811.566 442.774,812.984 443.002,814.431 443.23,810.87 443.458,810.283 443.685,814.036 443.913,815.455 444.141,818.29 444.369,816.781 444.597,822.294 444.824,820.22 445.052,820.195 445.28,821.599 445.508,819.597 445.735,824.263 445.963,820.976 446.191,821.721 446.419,824.695 446.646,830.642 446.874,833.372 447.102,827.931 447.33,830.185 447.558,831.198 447.785,831.125 448.013,833.776 448.241,832.334 448.469,827.758 448.696,830.031 448.924,829.808 449.152,834.872 449.38,834.239 449.607,823.131 449.835,825.84 450.063,825.476 450.291,825.958 450.519,830.343 450.746,824.368 450.974,826.15 451.202,830.059 451.43,834.852 451.657,832.563 451.885,831.397 452.113,828.909 452.341,834.214 452.568,830.977 452.796,831.682 453.024,832.61 453.252,832.501 453.48,829.783 453.707,831.456 453.935,830.819 454.163,830.395 454.391,828.202 454.618,828.745 454.846,835.879 455.074,836.271 455.302,832.343 455.53,828.706 455.757,829.419 455.985,827.789 456.213,824.81 456.441,821.852 456.668,830.074 456.896,832.156 457.124,825.961 457.352,831.942 457.579,829.333 457.807,817.317 458.035,818.757 458.263,815.558 458.491,807.45 458.718,813.957 458.946,814.939 459.174,817.856 459.402,821.096 459.629,815.406 459.857,810.81 460.085,815.827 460.313,814.632 460.54,816.253 460.768,820.315 460.996,821.99 461.224,824.325 461.452,826.256 461.679,832.658 461.907,826.958 462.135,829.934 462.363,829.593 462.59,828.065 462.818,829.087 463.046,825.402 463.274,825.13 463.501,826.863 463.729,823.522 463.957,829.041 464.185,830.374 464.413,832.695 464.64,829.817 464.868,830.041 465.096,831.145 465.324,833.931 465.551,837.672 465.779,840.023 466.007,837.268 466.235,839.839 466.462,843.807 466.69,846.988 466.918,850.287 467.146,848.325 467.374,849.236 467.601,851.257 467.829,849.605 468.057,851.839 468.285,851.696 468.512,856.627 468.74,853.348 468.968,850.034 469.196,844.257 469.424,845.276 469.651,846.165 469.879,835.725 470.107,838.745 470.335,834.9 470.562,835.006 470.79,839.422 471.018,838.616 471.246,846.625 471.473,848.149 471.701,849.008 471.929,848.283 472.157,841.169 472.385,836.091 472.612,838.187 472.84,841.92 473.068,843.77 473.296,846.27 473.523,844.923 473.751,850.236 473.979,848.67 474.207,844.511 474.434,844.138 474.662,845.844 474.89,847.882 475.118,847.805 475.346,854.811 475.573,858.436 475.801,863.213 476.029,869.448 476.257,870.265 476.484,865.229 476.712,868.302 476.94,878.816 477.168,879.973 477.395,884.009 477.623,881.471 477.851,882.551 478.079,879.84 478.307,880.829 478.534,881.248 478.762,874.212 478.99,871.558 479.218,875.047 479.445,872.778 479.673,872.856 479.901,870.421 480.129,869.362 480.357,869.19 480.584,875.808 480.812,878.13 481.04,873.294 481.268,874.605 481.495,878.598 481.723,882.46 481.951,873.287 482.179,876.34 482.406,878.547 482.634,886.574 482.862,885.007 483.09,886.519 483.318,888.148 483.545,887.459 483.773,889.201 484.001,883.401 484.229,889.342 484.456,891.807 484.684,896.263 484.912,892.385 485.14,890.745 485.367,884.882 485.595,885.657 485.823,889.302 486.051,896.664 486.279,897.5 486.506,894.299 486.734,897.919 486.962,901.068 487.19,900.24 487.417,893.019 487.645,894.861 487.873,892.932 488.101,891.631 488.328,890.25 488.556,886.217 488.784,891.167 489.012,884.827 489.24,885.39 489.467,888.953 489.695,889.389 489.923,889.279 490.151,891.573 490.378,893.841 490.606,895.063 490.834,897.381 491.062,897.11 491.29,888.015 491.517,891.365 491.745,888.203 491.973,890.145 492.201,891.524 492.428,891.835 492.656,891.166 492.884,892.65 493.112,894.966 493.339,889.168 493.567,890.055 493.795,891.127 494.023,892.98 494.251,889.799 494.478,889.542 494.706,885.195 494.934,886.736 495.162,887.864 495.389,890.453 495.617,891.159 495.845,892.167 496.073,892.047 496.3,896.963 496.528,897.396 496.756,895.889 496.984,898.7 497.212,900.608 497.439,902.49 497.667,902.559 497.895,903.132 498.123,900.576 498.35,899.733 498.578,906.16 498.806,908.286 499.034,913.967 499.261,915.031 499.489,914.974 499.717,915.086 499.945,915.881 500.173,918.086 500.4,921.859 500.628,918.144 500.856,918.142 501.084,921.595 501.311,921.575 501.539,924.948 501.767,926.945 501.995,925.047 502.223,929.261 502.45,929.377 502.678,929.061 502.906,932.453 503.134,934.207 503.361,928.882 503.589,926.226 503.817,925.234 504.045,931.356 504.272,929.489 504.5,929.62 504.728,925.39 504.956,919.307 505.184,919.188 505.411,923.67 505.639,922.183 505.867,925.241 506.095,929.089 506.322,928.762 506.55,925.762 506.778,924.37 507.006,925.974 507.233,927.389 507.461,927.697 507.689,927.155 507.917,929.175 508.145,929.753 508.372,921.762 508.6,916.811 508.828,918.972 509.056,915.457 509.283,918.898 509.511,917.173 509.739,917.32 509.967,918.734 510.194,920.841 510.422,924.202 510.65,921.987 510.878,921.345 511.106,918.506 511.333,919.814 511.561,921.495 511.789,926.086 512.017,927.238 512.244,928.715 512.472,930.955 512.7,931.762 512.928,934.348 513.156,931.521 513.383,934.37 513.611,928.942 513.839,930.076 514.067,928.319 514.294,932.312 514.522,932.379 514.75,937.778 514.978,942.56 515.205,943.562 515.433,943.442 515.661,946.636 515.889,949.644 516.117,951.353 516.344,948.944 516.572,948.367 516.8,951.482 517.028,954.651 517.255,955.368 517.483,951.778 517.711,949.297 517.939,946.969 518.166,949.887 518.394,950.022 518.622,948.461 518.85,946.964 519.078,943.126 519.305,940.373 519.533,939.184 519.761,936.538 519.989,939.241 520.216,941.431 520.444,940.96 520.672,942.405 520.9,940.46 521.127,941.456 521.355,941.827 521.583,945.149 521.811,943.219 522.039,944.372 522.266,944.864 522.494,946.337 522.722,948.462 522.95,952.997 523.177,951.492 523.405,945.948 523.633,943.697 523.861,944.424 524.089,941.451 524.316,942.179 524.544,942.135 524.772,946.273 525,949.245 525.227,947.934 525.455,943.972 525.683,948.844 525.911,946.554 526.138,941.594 526.366,942.606 526.594,948.739 526.822,947.444 527.05,948.066 527.277,948.808 527.505,946.335 527.733,947.559 527.961,946.389 528.188,946.166 528.416,940.257 528.644,937.265 528.872,936.302 529.099,932.531 529.327,927.053 529.555,925.151 529.783,927.007 530.011,928.231 530.238,928.568 530.466,924.511 530.694,921.288 530.922,922.757 531.149,919.353 531.377,922.229 531.605,921.955 531.833,917.997 532.06,915.084 532.288,917.248 532.516,919.531 532.744,917.145 532.972,914.731 533.199,916.733 533.427,914.85 533.655,911.947 533.883,913.867 534.11,914.716 534.338,913.836 534.566,912.502 534.794,910.667 535.022,915.267 535.249,916.589 535.477,918.273 535.705,920.285 535.933,921.39 536.16,921.027 536.388,922.837 536.616,922.062 536.844,923.563 537.071,925.411 537.299,928.706 537.527,930.28 537.755,933.496 537.983,929.592 538.21,934.009 538.438,937.017 538.666,938.702 538.894,935.39 539.121,937.093 539.349,938.134 539.577,938.588 539.805,937.009 540.032,939.931 540.26,938.999 540.488,939.106 540.716,938.704 540.944,940.756 541.171,939.796 541.399,940.059 541.627,935.873 541.855,935.528 542.082,937.645 542.31,932.034 542.538,932.367 542.766,929.638 542.993,930.922 543.221,932.921 543.449,935.61 543.677,939.009 543.905,941.401 544.132,940.064 544.36,933.704 544.588,936.291 544.816,935.058 545.043,933.555 545.271,929.171 545.499,925.375 545.727,925.78 545.955,922.309 546.182,919.761 546.41,920.384 546.638,919.35 546.866,917.405 547.093,920.924 547.321,924.145 547.549,924.443 547.777,923.045 548.004,923.215 548.232,923.48 548.46,923.634 548.688,923.651 548.916,925.038 549.143,928.47 549.371,928.553 549.599,929.534 549.827,931.232 550.054,929.812 550.282,928.226 550.51,924.42 550.738,929.348 550.965,931.546 551.193,932.805 551.421,933.703 551.649,933.737 551.877,926.393 552.104,929.966 552.332,931.571 552.56,932.663 552.788,934.893 553.015,935.004 553.243,931.592 553.471,931.196 553.699,933.166 553.926,933.134 554.154,933.631 554.382,934.403 554.61,935.167 554.838,935.34 555.065,938.132 555.293,941.286 555.521,943.177 555.749,943.476 555.976,944.792 556.204,946.315 556.432,943.205 556.66,945.754 556.888,949.363 557.115,952.789 557.343,952.148 557.571,950.21 557.799,952.096 558.026,949.79 558.254,953.805 558.482,949.821 558.71,948.943 558.937,948.843 559.165,947.183 559.393,943.983 559.621,943.155 559.849,942.225 560.076,942.482 560.304,938.349 560.532,941.564 560.76,942.035 560.987,942.142 561.215,944.516 561.443,944.793 561.671,944.406 561.898,945.054 562.126,948.264 562.354,946.229 562.582,947.102 562.81,944.099 563.037,945.658 563.265,946.949 563.493,943.946 563.721,944.548 563.948,943.709 564.176,943.859 564.404,944.419 564.632,942.813 564.859,944.501 565.087,946.944 565.315,947.257 565.543,947.221 565.771,946.916 565.998,950.368 566.226,948.901 566.454,945.535 566.682,943.647 566.909,946.403 567.137,948.878 567.365,947.47 567.593,941.529 567.821,941.359 568.048,944.358 568.276,946.253 568.504,944.873 568.732,946.72 568.959,941.402 569.187,941.701 569.415,943.017 569.643,947.802 569.87,947.522 570.098,948.6 570.326,951.182 570.554,950.859 570.782,951.09 571.009,952.337 571.237,956.615 571.465,956.483 571.693,957.263 571.92,956.666 572.148,957.296 572.376,953.992 572.604,954.169 572.831,950.975 573.059,948.365 573.287,950.15 573.515,945.92 573.743,945.337 573.97,945.94 574.198,944.711 574.426,942.633 574.654,942.288 574.881,943.769 575.109,939.786 575.337,940.569 575.565,939.759 575.792,940.402 576.02,939.883 576.248,936.57 576.476,936.173 576.704,934.483 576.931,932.298 577.159,929.728 577.387,929.305 577.615,928.377 577.842,926.309 578.07,926.953 578.298,930.279 578.526,930.372 578.754,932.171 578.981,934.169 579.209,932.298 579.437,933.882 579.665,933.384 579.892,934.468 580.12,934.923 580.348,933.366 580.576,931.263 580.803,932.369 581.031,932.921 581.259,932.506 581.487,927.618 581.715,927.373 581.942,927.022 582.17,928.826 582.398,927.154 582.626,925.457 582.853,923.596 583.081,924.75 583.309,927.021 583.537,929.977 583.764,931.43 583.992,931.669 584.22,934.78 584.448,936.977 584.676,937.871 584.903,939.386 585.131,940.106 585.359,941.467 585.587,942.484 585.814,945.18 586.042,942.019 586.27,943.331 586.498,942.728 586.725,940.578 586.953,935.284 587.181,935.142 587.409,932.985 587.637,931.664 587.864,931.143 588.092,933.976 588.32,932.598 588.548,935.884 588.775,936.598 589.003,936.585 589.231,936.626 589.459,935.312 589.687,935.944 589.914,935.06 590.142,935.587 590.37,934.464 590.598,935.312 590.825,936.189 591.053,936.458 591.281,937.318 591.509,936.97 591.736,933.962 591.964,937.896 592.192,938.41 592.42,939.002 592.648,941.065 592.875,940.56 593.103,941.519 593.331,942.896 593.559,945.931 593.786,946.345 594.014,947.29 594.242,948.232 594.47,946.36 594.697,948.608 594.925,947.908 595.153,949.566 595.381,951.392 595.609,951.878 595.836,953.152 596.064,952.708 596.292,949.277 596.52,947.209 596.747,947.055 596.975,943.979 597.203,945.087 597.431,947.535 597.658,946.464 597.886,946.86 598.114,945.504 598.342,943.315 598.57,943.544 598.797,945.794 599.025,944.539 599.253,945.543 599.481,940.627 599.708,939.428 599.936,939.571 600.164,936.792 600.392,938.95 600.62,940.583 600.847,938.125 601.075,937.051 601.303,936.996 601.531,940.463 601.758,938.96 601.986,938.84 602.214,938.812 602.442,940.227 602.669,940.84 602.897,942.276 603.125,938.352 603.353,941.732 603.581,942.92 603.808,944.667 604.036,948.759 604.264,949.205 604.492,952.671 604.719,952.838 604.947,951.547 605.175,949.893 605.403,949.702 605.63,954.549 605.858,953.49 606.086,949.012 606.314,949.936 606.542,947.847 606.769,948.428 606.997,949.104 607.225,949.701 607.453,950.135 607.68,949.97 607.908,951.378 608.136,952.545 608.364,952.697 608.591,954.036 608.819,952.423 609.047,952.262 609.275,953.999 609.503,953.209 609.73,956.401 609.958,951.158 610.186,953.057 610.414,953.611 610.641,952.793 610.869,950.831 611.097,946.502 611.325,946.945 611.553,946.859 611.78,945.422 612.008,943.377 612.236,943.525 612.464,945.571 612.691,945.407 612.919,945.831 613.147,944.794 613.375,942.693 613.602,942.252 613.83,939.275 614.058,935.929 614.286,937.017 614.514,938.473 614.741,938.11 614.969,934.22 615.197,934.452 615.425,933.302 615.652,932.161 615.88,933.646 616.108,933.897 616.336,933.819 616.563,935.499 616.791,935.701 617.019,935.636 617.247,935.247 617.475,933.291 617.702,932.643 617.93,931.161 618.158,931.788 618.386,930.274 618.613,928.156 618.841,931.783 619.069,933.798 619.297,936.184 619.524,936.37 619.752,936.715 619.98,939.555 620.208,940.342 620.436,939.164 620.663,935.645 620.891,934.072 621.119,931.464 621.347,929.617 621.574,928.699 621.802,930.402 622.03,932.28 622.258,934.736 622.486,935.978 622.713,937.025 622.941,937.986 623.169,936.383 623.397,935.969 623.624,934.583 623.852,934.667 624.08,935.569 624.308,934.812 624.535,934.521 624.763,937.443 624.991,940.171 625.219,937.03 625.447,935.737 625.674,934.674 625.902,937.567 626.13,937.51 626.358,935.239 626.585,933.363 626.813,935.07 627.041,935.187 627.269,935.062 627.496,934.663 627.724,936.659 627.952,938.829 628.18,939.724 628.408,939.103 628.635,939.816 628.863,938.042 629.091,939.941 629.319,938.555 629.546,938.732 629.774,938.023 630.002,939.904 630.23,940.752 630.457,943.106 630.685,941.15 630.913,936.284 631.141,935.849 631.369,936.154 631.596,933.695 631.824,934.789 632.052,934.852 632.28,934.934 632.507,938.569 632.735,937.564 632.963,935.21 633.191,935.226 633.419,936.267 633.646,935.173 633.874,934.925 634.102,937.368 634.33,936.53 634.557,937.294 634.785,934.585 635.013,935.84 635.241,937.585 635.468,939.456 635.696,940.496 635.924,942.517 636.152,944.45 636.38,942.553 636.607,942.761 636.835,942.76 637.063,942.002 637.291,939.804 637.518,941.186 637.746,940.194 637.974,938.764 638.202,938.077 638.429,938.524 638.657,940.982 638.885,937.793 639.113,937.735 639.341,939.026 639.568,936.719 639.796,938.371 640.024,939.158 640.252,941.972 640.479,942.354 640.707,938.786 640.935,939.265 641.163,935.777 641.39,936.684 641.618,934.128 641.846,936.628 642.074,934.95 642.302,935.146 642.529,935.87 642.757,935.353 642.985,933.878 643.213,934.652 643.44,935.463 643.668,934.06 643.896,936.568 644.124,934.315 644.352,932.772 644.579,933.191 644.807,936.03 645.035,936.626 645.263,937.452 645.49,933.81 645.718,935.641 645.946,938.642 646.174,940.008 646.401,941.097 646.629,940.601 646.857,941.564 647.085,942.461 647.313,944.23 647.54,945.85 647.768,945.784 647.996,946.139 648.224,946.058 648.451,944.574 648.679,945.815 648.907,947.482 649.135,949.962 649.362,946.686 649.59,943.696 649.818,946.042 650.046,945.615 650.274,946.329 650.501,945.655 650.729,947.167 650.957,946.308 651.185,946.296 651.412,944.611 651.64,945.521 651.868,946.63 652.096,947.057 652.323,945.308 652.551,945.715 652.779,944.289 653.007,944.028 653.235,944.142 653.462,942.787 653.69,940.826 653.918,941.661 654.146,942.689 654.373,940.555 654.601,938.437 654.829,934.954 655.057,934.469 655.285,935.166 655.512,934.257 655.74,934.395 655.968,934.817 656.196,933.426 656.423,931.579 656.651,931.1 656.879,932.194 657.107,931.29 657.334,931.642 657.562,928.997 657.79,930.415 658.018,932.128 658.246,933.658 658.473,933.92 658.701,930.517 658.929,932.259 659.157,932.245 659.384,933.927 659.612,936.313 659.84,935.8 660.068,936.119 660.295,936.385 660.523,939.331 660.751,938.796 660.979,938.936 661.207,939.637 661.434,939.439 661.662,938.263 661.89,938.643 662.118,940.768 662.345,938.734 662.573,938.694 662.801,939.43 663.029,939.701 663.256,939.876 663.484,940.892 663.712,942.628 663.94,938.835 664.168,936.688 664.395,938.617 664.623,938.39 664.851,938.046 665.079,938.075 665.306,939.957 665.534,935.503 665.762,936.541 665.99,936.462 666.218,938.329 666.445,937.91 666.673,938.28 666.901,939.019 667.129,938.397 667.356,937.652 667.584,933.858 667.812,934.095 668.04,932.705 668.267,930.338 668.495,931.623 668.723,930.056 668.951,928.568 669.179,929.355 669.406,928.335 669.634,930.418 669.862,929.209 670.09,929.733 670.317,930.673 670.545,932.385 670.773,933.253 671.001,932.675 671.228,935.498 671.456,935.482 671.684,937.123 671.912,936.988 672.14,936.566 672.367,935.476 672.595,937.316 672.823,936.604 673.051,936.912 673.278,936.278 673.506,934.803 673.734,936.2 673.962,937.524 674.189,934.601 674.417,935.561 674.645,933.999 674.873,935.731 675.101,933.174 675.328,933.796 675.556,934.064 675.784,934.821 676.012,933.124 676.239,932.422 676.467,933.917 676.695,936.385 676.923,934.213 677.151,934.85 677.378,937.472 677.606,935.69 677.834,935.31 678.062,937.488 678.289,935.194 678.517,934.85 678.745,934.866 678.973,933.477 679.2,933.233 679.428,935.701 679.656,936.387 679.884,938.082 680.112,938.36 680.339,938.39 680.567,939.337 680.795,941.257 681.023,939.14 681.25,939.494 681.478,938.547 681.706,938.625 681.934,938.738 682.161,937.117 682.389,935.681 682.617,933.088 682.845,934.302 683.073,932.306 683.3,930.844 683.528,928.108 683.756,925.379 683.984,922.369 684.211,922.908 684.439,923.524 684.667,925.524 684.895,926.112 685.122,923.959 685.35,926.349 685.578,925.054 685.806,924.943 686.034,924.729 686.261,925.288 686.489,925.665 686.717,928.058 686.945,929.044 687.172,928.992 687.4,931.398 687.628,931.927 687.856,933.359 688.084,934.864 688.311,936.211 688.539,936.896 688.767,936.884 688.995,935.286 689.222,936.868 689.45,937.458 689.678,938.516 689.906,937.997 690.133,937.215 690.361,938.602 690.589,939.566 690.817,939.757 691.045,939.423 691.272,936.158 691.5,936.962 691.728,937.938 691.956,939.527 692.183,940.954 692.411,938.127 692.639,937.411 692.867,937.609 693.094,939.024 693.322,940.746 693.55,943.803 693.778,943.154 694.006,940.887 694.233,941.146 694.461,944.152 694.689,944.369 694.917,945.459 695.144,947.647 695.372,949.182 695.6,950.217 695.828,952.012 696.055,951.102 696.283,949.382 696.511,950.047 696.739,950.6 696.967,950.698 697.194,952.58 697.422,953.672 697.65,951.78 697.878,951.095 698.105,952.006 698.333,951.778 698.561,951.724 698.789,952.323 699.017,954.181 699.244,954.884 699.472,956.359 699.7,955.908 699.928,956.435 700.155,956.478 700.383,958.538 700.611,959.517 700.839,959.537 701.066,958.737 701.294,957.436 701.522,958.368 701.75,960.106 701.978,959.884 702.205,960.309 702.433,959.002 702.661,959.009 702.889,957.675 703.116,960.065 703.344,958.374 703.572,958.643 703.8,959.909 704.027,960.287 704.255,955.213 704.483,956.533 704.711,953.009 704.939,955.215 705.166,957.746 705.394,958.307 705.622,957.572 705.85,958.844 706.077,958.955 706.305,960.126 706.533,961.26 706.761,960.564 706.988,959.413 707.216,959.365 707.444,959.352 707.672,958.286 707.9,959.247 708.127,958.274 708.355,958.848 708.583,960.049 708.811,960.265 709.038,958.852 709.266,958.002 709.494,955.638 709.722,955.368 709.95,959.073 710.177,957.629 710.405,957.107 710.633,956.086 710.861,955.756 711.088,957.546 711.316,957.779 711.544,958.427 711.772,957.832 711.999,959.296 712.227,959.552 712.455,959.138 712.683,960.867 712.911,960.867 713.138,959.373 713.366,958.751 713.594,960.424 713.822,960.477 714.049,960.893 714.277,961.357 714.505,962.817 714.733,962.795 714.96,965.05 715.188,967.095 715.416,967.732 715.644,970.081 715.872,973.09 716.099,974.67 716.327,972.386 716.555,971.933 716.783,972.418 717.01,974.488 717.238,975.891 717.466,977.514 717.694,975.51 717.921,978.236 718.149,979.83 718.377,977.416 718.605,977.803 718.833,978.816 719.06,976.342 719.288,975.272 719.516,974.234 719.744,974.122 719.971,973.578 720.199,974.094 720.427,974.561 720.655,974.616 720.883,976.325 721.11,978.765 721.338,978.669 721.566,979.921 721.794,980.982 722.021,978.77 722.249,977.214 722.477,979.053 722.705,979.731 722.932,981.257 723.16,980.102 723.388,981.024 723.616,981.492 723.844,982.56 724.071,979.807 724.299,979.146 724.527,979.952 724.755,980.131 724.982,983.017 725.21,984.57 725.438,985.204 725.666,984.854 725.893,981.307 726.121,979.663 726.349,981.525 726.577,982.187 726.805,983.62 727.032,982.782 727.26,985.311 727.488,986.59 727.716,988.184 727.943,987.588 728.171,985.797 728.399,985.205 728.627,984.158 728.854,983.669 729.082,982.085 729.31,984.103 729.538,984.148 729.766,983.097 729.993,983.48 730.221,982.989 730.449,983.797 730.677,982.243 730.904,980.839 731.132,982.755 731.36,984.136 731.588,984.084 731.816,983.576 732.043,984.664 732.271,983.069 732.499,984.45 732.727,983.479 732.954,982.417 733.182,983.371 733.41,984.219 733.638,983.181 733.865,985.401 734.093,987.484 734.321,987.862 734.549,986.81 734.777,987.731 735.004,990.309 735.232,990.003 735.46,989.999 735.688,989.483 735.915,987.02 736.143,985.166 736.371,986.173 736.599,986.565 736.826,987.295 737.054,985.658 737.282,986.797 737.51,986.563 737.738,984.966 737.965,985.382 738.193,985.477 738.421,986.214 738.649,987.833 738.876,985.965 739.104,986.729 739.332,987.034 739.56,988.178 739.787,990.247 740.015,990.533 740.243,991.103 740.471,992.661 740.699,992.144 740.926,992.602 741.154,992.791 741.382,992.284 741.61,994.327 741.837,995.065 742.065,997.081 742.293,998.551 742.521,998.43 742.749,995.09 742.976,995.513 743.204,995.716 743.432,996.05 743.66,993.139 743.887,990.319 744.115,990.92 744.343,989.931 744.571,991.231 744.798,991.646 745.026,989.605 745.254,989.561 745.482,989.23 745.71,988.891 745.937,989.995 746.165,990.024 746.393,989.071 746.621,989.486 746.848,989.767 747.076,989 747.304,990.225 747.532,990.307 747.759,991.515 747.987,990.45 748.215,991.925 748.443,993.258 748.671,994.336 748.898,993.912 749.126,994.423 749.354,993.532 749.582,993.282 749.809,992.679 750.037,993.652 750.265,991.027 750.493,990.075 750.72,987.278 750.948,986.356 751.176,982.98 751.404,983.028 751.632,985.271 751.859,985.584 752.087,985.655 752.315,985.89 752.543,985.221 752.77,987.752 752.998,988.474 753.226,989.116 753.454,988.138 753.682,985.142 753.909,987.998 754.137,983.888 754.365,984.341 754.593,984.016 754.82,982.987 755.048,983.627 755.276,984.058 755.504,985.15 755.731,982.887 755.959,983.282 756.187,981.391 756.415,982.314 756.643,981.351 756.87,977.87 757.098,978.871 757.326,976.99 757.554,975.822 757.781,975.412 758.009,973.854 758.237,974.785 758.465,975.193 758.692,974.635 758.92,975.224 759.148,975.082 759.376,976.057 759.604,977.601 759.831,977.779 760.059,978.461 760.287,978.819 760.515,979.448 760.742,980.231 760.97,979.964 761.198,981.107 761.426,983.006 761.653,981.643 761.881,980.689 762.109,978.956 762.337,976.942 762.565,976.96 762.792,977.998 763.02,980.28 763.248,977.612 763.476,976.007 763.703,977.752 763.931,974.378 764.159,972.605 764.387,974.298 764.615,973.747 764.842,975.75 765.07,975.871 765.298,978.215 765.526,978.206 765.753,979.713 765.981,978.993 766.209,977.126 766.437,979.551 766.664,979.93 766.892,979.855 767.12,981.61 767.348,982.877 767.576,982.819 767.803,983.958 768.031,983.935 768.259,985.482 768.487,987.065 768.714,987.259 768.942,987.079 769.17,987.796 769.398,987.946 769.625,988.557 769.853,987.795 770.081,983.181 770.309,982.524 770.537,981.048 770.764,981.868 770.992,981.116 771.22,978.514 771.448,978.434 771.675,977.798 771.903,979.977 772.131,980.905 772.359,981.449 772.586,980.721 772.814,980.126 773.042,976.892 773.27,975.847 773.498,974.238 773.725,973.687 773.953,970.599 774.181,970.666 774.409,968.106 774.636,968.114 774.864,968.146 775.092,968.37 775.32,967.971 775.548,965.57 775.775,966.625 776.003,967.695 776.231,968.717 776.459,968.575 776.686,967.961 776.914,967.643 777.142,967.992 777.37,966.59 777.597,968.547 777.825,969.705 778.053,971.74 778.281,970.97 778.509,970.284 778.736,970.667 778.964,971.801 779.192,971.244 779.42,972.263 779.647,974.375 779.875,975.333 780.103,973.566 780.331,972.691 780.558,975.029 780.786,974.925 781.014,975.719 781.242,975.839 781.47,975.814 781.697,974.709 781.925,975.533 782.153,976.181 782.381,977.702 782.608,978.891 782.836,977.557 783.064,977.467 783.292,979.001 783.519,979.52 783.747,979.415 783.975,979.981 784.203,980.216 784.431,978.407 784.658,979.75 784.886,980.813 785.114,980.213 785.342,978.993 785.569,980.587 785.797,979.621 786.025,977.989 786.253,978.923 786.481,980.806 786.708,982.502 786.936,980.057 787.164,979.43 787.392,978.742 787.619,979.355 787.847,980.592 788.075,982.112 788.303,982.006 788.53,982.651 788.758,983.119 788.986,983.483 789.214,984.77 789.442,982.171 789.669,984.236 789.897,983.778 790.125,984.527 790.353,984.07 790.58,983.397 790.808,984.586 791.036,985.619 791.264,981.561 791.491,983.344 791.719,983.004 791.947,981.849 792.175,981.273 792.403,981.704 792.63,981.887 792.858,982.477 793.086,981.234 793.314,982.82 793.541,982.998 793.769,984.553 793.997,982.41 794.225,982.658 794.452,982.304 794.68,981.339 794.908,981.605 795.136,978.453 795.364,977.334 795.591,977.777 795.819,977.448 796.047,978.667 796.275,978.945 796.502,979.077 796.73,977.212 796.958,980.376 797.186,981.229 797.413,981.94 797.641,981.685 797.869,981.942 798.097,982.256 798.325,982.406 798.552,981.84 798.78,982.148 799.008,981.394 799.236,978.618 799.463,979.146 799.691,980.489 799.919,980.39 800.147,979.979 800.375,981.096 800.602,980.875 800.83,979.319 801.058,978.666 801.286,980.809 801.513,982.742 801.741,981.95 801.969,983.343 802.197,983.205 802.424,983.301 802.652,981.758 802.88,981.085 803.108,982.044 803.336,983.849 803.563,984.79 803.791,984.794 804.019,986.065 804.247,982.705 804.474,982.561 804.702,983.138 804.93,982.776 805.158,983.264 805.385,982.109 805.613,982.067 805.841,982.539 806.069,981.7 806.297,981.713 806.524,981.324 806.752,981.595 806.98,981.503 807.208,980.504 807.435,982.236 807.663,984.476 807.891,984.979 808.119,985.338 808.346,985.672 808.574,984.275 808.802,984.883 809.03,985.026 809.258,984.802 809.485,985.134 809.713,986.347 809.941,985.753 810.169,986.231 810.396,985.306 810.624,985.657 810.852,985.365 811.08,985.067 811.308,986.103 811.535,986.447 811.763,988.143 811.991,987.032 812.219,987.425 812.446,988.108 812.674,987.972 812.902,987.587 813.13,987.302 813.357,987.754 813.585,988.118 813.813,989.55 814.041,989.617 814.269,991.429 814.496,988.886 814.724,988.624 814.952,988.444 815.18,987.366 815.407,985.704 815.635,985.243 815.863,986.903 816.091,986.839 816.318,986.466 816.546,987.623 816.774,988.056 817.002,986.956 817.23,985.234 817.457,985.825 817.685,984.666 817.913,985.472 818.141,985.533 818.368,983.261 818.596,981.692 818.824,980.92 819.052,981.791 819.279,982.535 819.507,981.13 819.735,981.169 819.963,982.337 820.191,983.269 820.418,983.106 820.646,982.893 820.874,984.188 821.102,985.147 821.329,985.389 821.557,984.022 821.785,985.158 822.013,983.878 822.241,985.259 822.468,986.766 822.696,988.541 822.924,989.756 823.152,989.911 823.379,990.286 823.607,991.299 823.835,989.608 824.063,991.018 824.29,991.821 824.518,993.432 824.746,993.13 824.974,994.944 825.202,995.382 825.429,995.365 825.657,994.657 825.885,997.045 826.113,997.974 826.34,998.172 826.568,998.197 826.796,998.125 827.024,999.159 827.251,999.58 827.479,998.317 827.707,999.407 827.935,999.296 828.163,1000.27 828.39,1001.51 828.618,1002.49 828.846,1002.74 829.074,998.875 829.301,999.692 829.529,1000.71 829.757,1001.12 829.985,1002.07 830.212,1000.92 830.44,1001.78 830.668,998.764 830.896,999.561 831.124,1000.56 831.351,1001.79 831.579,1003.08 831.807,1004.1 832.035,1004.67 832.262,1005.48 832.49,1005 832.718,1005.73 832.946,1005.46 833.174,1006.07 833.401,1006.83 833.629,1006.55 833.857,1007.26 834.085,1004.76 834.312,1005.1 834.54,1005.94 834.768,1004.6 834.996,1004.66 835.223,1003.42 835.451,1001.64 835.679,1000.87 835.907,1000.52 836.135,1000.28 836.362,1000.72 836.59,1000.85 836.818,1001.8 837.046,1002.58 837.273,1002.7 837.501,1003.85 837.729,1002.86 837.957,1004.2 838.184,1006.13 838.412,1006.61 838.64,1006.87 838.868,1007.56 839.096,1005.16 839.323,1005.23 839.551,1006 839.779,1006.61 840.007,1005.87 840.234,1006.38 840.462,1007.72 840.69,1009.46 840.918,1009.49 841.145,1011.61 841.373,1010.42 841.601,1009.52 841.829,1009.65 842.057,1009.65 842.284,1009.4 842.512,1010.09 842.74,1009.95 842.968,1009.83 843.195,1010.66 843.423,1011.2 843.651,1011.52 843.879,1011.16 844.107,1009.75 844.334,1009.67 844.562,1010.68 844.79,1008.41 845.018,1008.46 845.245,1009.04 845.473,1010.4 845.701,1010.29 845.929,1008.6 846.156,1008.5 846.384,1006.72 846.612,1006.51 846.84,1005.72 847.068,1008.26 847.295,1009.69 847.523,1010.89 847.751,1011.53 847.979,1011.7 848.206,1013 848.434,1013.07 848.662,1014.06 848.89,1013.66 849.117,1015.09 849.345,1016.99 849.573,1016.56 849.801,1017.1 850.029,1017.33 850.256,1017.67 850.484,1018.31 850.712,1018.33 850.94,1018.01 851.167,1017.8 851.395,1019.84 851.623,1019.97 851.851,1020.72 852.078,1020.07 852.306,1019.72 852.534,1019.11 852.762,1019.87 852.99,1020.28 853.217,1020.61 853.445,1022.71 853.673,1021.26 853.901,1021.32 854.128,1023.55 854.356,1024.2 854.584,1022.8 854.812,1021.28 855.04,1022.15 855.267,1022.99 855.495,1023.2 855.723,1023.9 855.951,1024.64 856.178,1023.25 856.406,1024.97 856.634,1025.23 856.862,1027.11 857.089,1026.51 857.317,1026.81 857.545,1027.27 857.773,1027.31 858.001,1028.37 858.228,1028.96 858.456,1026.86 858.684,1026.11 858.912,1023.69 859.139,1024.01 859.367,1024.2 859.595,1025.2 859.823,1024.85 860.05,1025.6 860.278,1027 860.506,1028.19 860.734,1028.09 860.962,1027.67 861.189,1028.02 861.417,1028.47 861.645,1028.23 861.873,1026.52 862.1,1026.05 862.328,1025.93 862.556,1026.18 862.784,1025.67 863.011,1026.75 863.239,1027.29 863.467,1028 863.695,1029.74 863.923,1028.88 864.15,1028.56 864.378,1029.67 864.606,1030.25 864.834,1031.8 865.061,1032.8 865.289,1033.83 865.517,1034.17 865.745,1033.92 865.973,1033.91 866.2,1034.32 866.428,1033.84 866.656,1034.88 866.884,1034.98 867.111,1033.58 867.339,1034.57 867.567,1033.2 867.795,1033.99 868.022,1034.54 868.25,1034.44 868.478,1033.37 868.706,1032.33 868.934,1031.67 869.161,1032.88 869.389,1032.03 869.617,1032.03 869.845,1030.62 870.072,1030.94 870.3,1029.94 870.528,1029.36 870.756,1030.18 870.983,1029.82 871.211,1029.99 871.439,1028.96 871.667,1027.85 871.895,1028.68 872.122,1028.03 872.35,1027.23 872.578,1027.43 872.806,1028.04 873.033,1028.36 873.261,1028.52 873.489,1028.41 873.717,1027.63 873.944,1025.78 874.172,1026.55 874.4,1025.91 874.628,1026.51 874.856,1026.29 875.083,1026.82 875.311,1025.46 875.539,1026.02 875.767,1025.38 875.994,1026.27 876.222,1026.42 876.45,1026.4 876.678,1026.46 876.906,1026.79 877.133,1024.14 877.361,1023.79 877.589,1023.03 877.817,1024.05 878.044,1024.79 878.272,1026.2 878.5,1025.64 878.728,1024.38 878.955,1024.36 879.183,1024.78 879.411,1025.09 879.639,1027.56 879.867,1028.04 880.094,1027.36 880.322,1028.73 880.55,1027.74 880.778,1028.38 881.005,1028.08 881.233,1026.43 881.461,1026.55 881.689,1025.74 881.916,1025.46 882.144,1026.5 882.372,1026.76 882.6,1027.3 882.828,1026.14 883.055,1026.26 883.283,1025.7 883.511,1026.08 883.739,1024.34 883.966,1024.06 884.194,1025.64 884.422,1026.58 884.65,1025.36 884.877,1026.16 885.105,1025.33 885.333,1026.18 885.561,1025.19 885.789,1026.62 886.016,1027.64 886.244,1027.78 886.472,1028.79 886.7,1028.87 886.927,1029.89 887.155,1031.2 887.383,1030.57 887.611,1030.53 887.839,1030.44 888.066,1032.2 888.294,1032.47 888.522,1033.59 888.75,1034.05 888.977,1032.93 889.205,1033.3 889.433,1032.5 889.661,1031.11 889.888,1031.2 890.116,1031.71 890.344,1032.73 890.572,1033.04 890.8,1031.65 891.027,1033.34 891.255,1034.15 891.483,1035.22 891.711,1035.3 891.938,1034.47 892.166,1034.6 892.394,1034.83 892.622,1034.21 892.849,1034.84 893.077,1033.98 893.305,1035.15 893.533,1036.24 893.761,1036.55 893.988,1035.7 894.216,1035.99 894.444,1036.78 894.672,1035.99 894.899,1036.56 895.127,1037.83 895.355,1038.53 895.583,1037.79 895.81,1037.61 896.038,1034.32 896.266,1035 896.494,1034.9 896.722,1035.78 896.949,1036.81 897.177,1037.39 897.405,1036.44 897.633,1037.96 897.86,1038.69 898.088,1038.19 898.316,1039.35 898.544,1038.5 898.772,1038.75 898.999,1039.46 899.227,1041.21 899.455,1040.56 899.683,1038.08 899.91,1037.71 900.138,1038.2 900.366,1038.28 900.594,1039.64 900.821,1041.56 901.049,1043.45 901.277,1043.57 901.505,1042.6 901.733,1042.91 901.96,1041.09 902.188,1042.54 902.416,1043.76 902.644,1044.93 902.871,1044.89 903.099,1045.98 903.327,1045.67 903.555,1043.48 903.782,1043.44 904.01,1043.16 904.238,1042.47 904.466,1040.97 904.694,1040.37 904.921,1041.31 905.149,1039.81 905.377,1038.52 905.605,1037.06 905.832,1037.1 906.06,1037.85 906.288,1038.09 906.516,1038.18 906.743,1038.33 906.971,1038.14 907.199,1039.35 907.427,1038.5 907.655,1037.83 907.882,1037.59 908.11,1038.26 908.338,1038.68 908.566,1038.9 908.793,1038.83 909.021,1039.08 909.249,1038.08 909.477,1037.84 909.705,1038.23 909.932,1037.37 910.16,1039.13 910.388,1038.74 910.616,1038.66 910.843,1039.75 911.071,1040.23 911.299,1040.73 911.527,1039.26 911.754,1038.23 911.982,1038.18 912.21,1038.35 912.438,1038.94 912.666,1039.46 912.893,1040.14 913.121,1041.29 913.349,1043.76 913.577,1043.3 913.804,1042.94 914.032,1043.86 914.26,1043.86 914.488,1044.71 914.715,1046.12 914.943,1045.55 915.171,1045.56 915.399,1045.58 915.627,1045.28 915.854,1044.57 916.082,1045.11 916.31,1043.64 916.538,1042.73 916.765,1041.53 916.993,1040.88 917.221,1040.58 917.449,1040.03 917.676,1040.97 917.904,1041.03 918.132,1039.24 918.36,1039.47 918.588,1040.39 918.815,1039.91 919.043,1041.22 919.271,1040 919.499,1039.91 919.726,1039.39 919.954,1040.54 920.182,1040.51 920.41,1039.81 920.638,1040.38 920.865,1040.8 921.093,1038.24 921.321,1038.79 921.549,1035.72 921.776,1037.02 922.004,1036.56 922.232,1037.24 922.46,1037.59 922.687,1037.6 922.915,1037.86 923.143,1037.85 923.371,1038.71 923.599,1036.99 923.826,1037.29 924.054,1038.06 924.282,1038.9 924.51,1037.41 924.737,1036.37 924.965,1036.97 925.193,1038.03 925.421,1038.56 925.648,1040.02 925.876,1038.67 926.104,1039.52 926.332,1038.79 926.56,1039.03 926.787,1039.22 927.015,1038.99 927.243,1039.43 927.471,1040.37 927.698,1039.44 927.926,1039.93 928.154,1039.39 928.382,1039.92 928.609,1037.09 928.837,1035.25 929.065,1036.2 929.293,1035.86 929.521,1035.66 929.748,1034.99 929.976,1033.34 930.204,1035.12 930.432,1036.45 930.659,1035.85 930.887,1037.85 931.115,1038.98 931.343,1038.11 931.571,1038.1 931.798,1037.95 932.026,1039.01 932.254,1039.09 932.482,1041.01 932.709,1040.38 932.937,1039.23 933.165,1039.09 933.393,1037.83 933.62,1038 933.848,1038.76 934.076,1039.54 934.304,1039.04 934.532,1040.49 934.759,1041.7 934.987,1043.29 935.215,1042.86 935.443,1042.85 935.67,1043.34 935.898,1043.29 936.126,1043.75 936.354,1044.62 936.581,1045.85 936.809,1045.08 937.037,1045.14 937.265,1046.11 937.493,1046.13 937.72,1047.2 937.948,1047.1 938.176,1047.53 938.404,1048.83 938.631,1048.36 938.859,1048.88 939.087,1048.53 939.315,1048.94 939.542,1047.78 939.77,1047.24 939.998,1047.58 940.226,1047.75 940.454,1049.53 940.681,1050.37 940.909,1050.88 941.137,1052.26 941.365,1052.61 941.592,1053.53 941.82,1053.83 942.048,1054.37 942.276,1054.6 942.504,1054.87 942.731,1056.03 942.959,1056.25 943.187,1056.71 943.415,1056.13 943.642,1056.7 943.87,1056.96 944.098,1057.37 944.326,1055.93 944.553,1056.59 944.781,1056.13 945.009,1056.51 945.237,1057.02 945.465,1056.59 945.692,1056.63 945.92,1057.17 946.148,1056.16 946.376,1054.37 946.603,1055.79 946.831,1056.82 947.059,1057.1 947.287,1057.24 947.514,1056.66 947.742,1056.67 947.97,1057.47 948.198,1058.78 948.426,1057.38 948.653,1056.05 948.881,1057.42 949.109,1057.35 949.337,1057.54 949.564,1057.38 949.792,1057.95 950.02,1058.21 950.248,1059.69 950.475,1057.65 950.703,1057.69 950.931,1056.75 951.159,1056.42 951.387,1055.65 951.614,1056.55 951.842,1056.67 952.07,1056.08 952.298,1055.88 952.525,1054.29 952.753,1052.81 952.981,1051.46 953.209,1051.54 953.437,1051.37 953.664,1050.71 953.892,1050.36 954.12,1051.15 954.348,1051.76 954.575,1051.34 954.803,1050.09 955.031,1048.43 955.259,1047.87 955.486,1047.32 955.714,1047.16 955.942,1047.72 956.17,1047.15 956.398,1047.71 956.625,1046.38 956.853,1044.02 957.081,1044.15 957.309,1044.1 957.536,1044.54 957.764,1044.8 957.992,1045.64 958.22,1042.82 958.447,1042.12 958.675,1042.55 958.903,1042.79 959.131,1043.68 959.359,1044.13 959.586,1045.39 959.814,1043.81 960.042,1044.41 960.27,1044.54 960.497,1044.56 960.725,1044.73 960.953,1045.23 961.181,1045.14 961.408,1044.56 961.636,1044.46 961.864,1043.08 962.092,1043.11 962.32,1043.93 962.547,1045.54 962.775,1046.23 963.003,1046.62 963.231,1047.43 963.458,1046.17 963.686,1047.3 963.914,1046.29 964.142,1045.83 964.37,1046.24 964.597,1046.98 964.825,1046.04 965.053,1046.32 965.281,1046.7 965.508,1046.57 965.736,1047.16 965.964,1047.18 966.192,1047.02 966.419,1045.54 966.647,1046.58 966.875,1045.12 967.103,1043.98 967.331,1045.32 967.558,1046.29 967.786,1047.41 968.014,1045.69 968.242,1046.06 968.469,1044.98 968.697,1044.43 968.925,1043.81 969.153,1043.05 969.38,1043.09 969.608,1044.48 969.836,1044.35 970.064,1043.02 970.292,1043.09 970.519,1041.54 970.747,1040.83 970.975,1040.5 971.203,1041.29 971.43,1040.87 971.658,1041.11 971.886,1041.06 972.114,1040.2 972.341,1039.46 972.569,1039.86 972.797,1039.54 973.025,1040.54 973.253,1039.03 973.48,1039.64 973.708,1038.31 973.936,1037.75 974.164,1038.04 974.391,1038.1 974.619,1036.63 974.847,1037.47 975.075,1037.36 975.303,1035.88 975.53,1034.87 975.758,1034.93 975.986,1035.11 976.214,1035.6 976.441,1036.25 976.669,1036.54 976.897,1036.52 977.125,1037.14 977.352,1037.69 977.58,1037.74 977.808,1037.56 978.036,1038.87 978.264,1039.08 978.491,1038.9 978.719,1039.42 978.947,1039.41 979.175,1040.19 979.402,1040.29 979.63,1040.52 979.858,1040.04 980.086,1039.79 980.313,1039.79 980.541,1038.84 980.769,1039 980.997,1037.51 981.225,1036.99 981.452,1038.14 981.68,1036.38 981.908,1036.31 982.136,1036.74 982.363,1038.57 982.591,1037.49 982.819,1036.3 983.047,1036.7 983.274,1038.15 983.502,1038.22 983.73,1039.25 983.958,1038.74 984.186,1038.9 984.413,1039.57 984.641,1041.61 984.869,1041.26 985.097,1040.31 985.324,1040.11 985.552,1039.85 985.78,1040.72 986.008,1041.6 986.236,1043.33 986.463,1044.87 986.691,1044.55 986.919,1046.08 987.147,1046.43 987.374,1047.93 987.602,1049.6 987.83,1049.73 988.058,1049.99 988.285,1049.78 988.513,1050.81 988.741,1050.12 988.969,1049.01 989.197,1049 989.424,1047.04 989.652,1045.99 989.88,1045.72 990.108,1046.02 990.335,1046.33 990.563,1047.72 990.791,1047.32 991.019,1046.81 991.246,1046.79 991.474,1047.11 991.702,1047.38 991.93,1047.37 992.158,1047.05 992.385,1044.31 992.613,1043.58 992.841,1044.13 993.069,1044.85 993.296,1045.87 993.524,1044.44 993.752,1044.93 993.98,1045.53 994.207,1044.19 994.435,1043.8 994.663,1043.41 994.891,1043.3 995.119,1043.08 995.346,1043.99 995.574,1043.75 995.802,1043.83 996.03,1043.42 996.257,1044.32 996.485,1044.55 996.713,1045.25 996.941,1045.31 997.169,1045.92 997.396,1046.28 997.624,1046.26 997.852,1045.64 998.08,1046.41 998.307,1045.57 998.535,1045.79 998.763,1046.07 998.991,1046.33 999.218,1046.75 999.446,1046.59 999.674,1046.11 999.902,1047.12 1000.13,1047.06 1000.36,1047.7 1000.59,1047.68 1000.81,1047.26 1001.04,1048.84 1001.27,1046.77 1001.5,1048.6 1001.72,1047.96 1001.95,1048.46 1002.18,1047.53 1002.41,1048.53 1002.64,1048.78 1002.86,1047.53 1003.09,1046.67 1003.32,1046.25 1003.55,1046.42 1003.77,1045.25 1004,1046.07 1004.23,1045.71 1004.46,1046.16 1004.68,1046.58 1004.91,1046.41 1005.14,1047.77 1005.37,1047.96 1005.6,1047.72 1005.82,1047.78 1006.05,1047.32 1006.28,1047.92 1006.51,1049.53 1006.73,1050.17 1006.96,1051.26 1007.19,1049.66 1007.42,1049.84 1007.65,1050.06 1007.87,1049.92 1008.1,1050.99 1008.33,1051.17 1008.56,1050.69 1008.78,1050.52 1009.01,1051.32 1009.24,1050.93 1009.47,1051.7 1009.7,1052.05 1009.92,1052.88 1010.15,1052.15 1010.38,1051.93 1010.61,1052.3 1010.83,1052.12 1011.06,1050.82 1011.29,1049.91 1011.52,1050.88 1011.75,1049.32 1011.97,1050.51 1012.2,1051.8 1012.43,1052.22 1012.66,1052.36 1012.88,1052.54 1013.11,1051.51 1013.34,1051.71 1013.57,1052.05 1013.8,1051.69 1014.02,1051.92 1014.25,1052.05 1014.48,1050.74 1014.71,1050.75 1014.93,1051.23 1015.16,1050.62 1015.39,1050.59 1015.62,1050.9 1015.85,1051.59 1016.07,1050.45 1016.3,1049.9 1016.53,1048.57 1016.76,1048.65 1016.98,1049.33 1017.21,1048.09 1017.44,1048.02 1017.67,1048.23 1017.9,1048.16 1018.12,1048.01 1018.35,1046.79 1018.58,1046.93 1018.81,1047.26 1019.03,1047.43 1019.26,1047.89 1019.49,1046.84 1019.72,1047.23 1019.95,1046.45 1020.17,1044.39 1020.4,1043.84 1020.63,1044.26 1020.86,1043.79 1021.08,1044.47 1021.31,1044.3 1021.54,1043.7 1021.77,1042.41 1022,1042.01 1022.22,1041.66 1022.45,1042.27 1022.68,1043.32 1022.91,1042.62 1023.13,1041 1023.36,1040.73 1023.59,1042.82 1023.82,1043.8 1024.05,1042.5 1024.27,1042.49 1024.5,1042.92 1024.73,1043.03 1024.96,1043.57 1025.18,1044.5 1025.41,1045.93 1025.64,1046.2 1025.87,1045.79 1026.1,1045.99 1026.32,1046.31 1026.55,1045.66 1026.78,1045.77 1027.01,1044.28 1027.23,1044.07 1027.46,1044.55 1027.69,1043.68 1027.92,1043.23 1028.15,1042.42 1028.37,1041.34 1028.6,1042.07 1028.83,1041.15 1029.06,1041.95 1029.28,1042.11 1029.51,1041.39 1029.74,1042.45 1029.97,1042.1 1030.2,1042.3 1030.42,1042.03 1030.65,1042.67 1030.88,1041.19 1031.11,1041.47 1031.33,1041.88 1031.56,1041.4 1031.79,1040.24 1032.02,1040.45 1032.25,1040.41 1032.47,1039.99 1032.7,1038.8 1032.93,1039.17 1033.16,1038.66 1033.38,1040.35 1033.61,1040.2 1033.84,1040.34 1034.07,1039.4 1034.3,1038.85 1034.52,1039.56 1034.75,1039.45 1034.98,1039.4 1035.21,1039.15 1035.43,1038.21 1035.66,1038.65 1035.89,1037.77 1036.12,1036.36 1036.35,1036.43 1036.57,1036.97 1036.8,1036.99 1037.03,1038.04 1037.26,1038.72 1037.48,1037.25 1037.71,1036.68 1037.94,1036.06 1038.17,1036.63 1038.4,1036.93 1038.62,1037.16 1038.85,1037.69 1039.08,1037.41 1039.31,1036.39 1039.53,1037.1 1039.76,1036.68 1039.99,1036.83 1040.22,1035.46 1040.44,1035.77 1040.67,1035.52 1040.9,1035.37 1041.13,1035.7 1041.36,1036.43 1041.58,1037.19 1041.81,1037.38 1042.04,1038.01 1042.27,1038.13 1042.49,1038.38 1042.72,1038.06 1042.95,1038.41 1043.18,1037.94 1043.41,1038.57 1043.63,1039.63 1043.86,1038.29 1044.09,1038.52 1044.32,1039.26 1044.54,1038.85 1044.77,1039.76 1045,1038.61 1045.23,1037.48 1045.46,1037.72 1045.68,1038.72 1045.91,1038.8 1046.14,1038.2 1046.37,1038.21 1046.59,1037.96 1046.82,1039.3 1047.05,1039.61 1047.28,1038.27 1047.51,1036.75 1047.73,1036.21 1047.96,1036.67 1048.19,1036.84 1048.42,1037.25 1048.64,1036.54 1048.87,1036.37 1049.1,1037.14 1049.33,1037.6 1049.56,1037.3 1049.78,1037.77 1050.01,1038.72 1050.24,1038.74 1050.47,1039.74 1050.69,1040.52 1050.92,1041.35 1051.15,1041.48 1051.38,1042.04 1051.61,1041.71 1051.83,1041.05 1052.06,1041.16 1052.29,1041.5 1052.52,1040.69 1052.74,1040.43 1052.97,1041.61 1053.2,1040.95 1053.43,1041.9 1053.66,1041.18 1053.88,1041.46 1054.11,1041.71 1054.34,1041.64 1054.57,1040.31 1054.79,1039.78 1055.02,1040.5 1055.25,1041.12 1055.48,1040.41 1055.71,1040.85 1055.93,1041.94 1056.16,1041.44 1056.39,1042.52 1056.62,1042.78 1056.84,1042.6 1057.07,1042.3 1057.3,1042.49 1057.53,1043.53 1057.76,1043.92 1057.98,1044.77 1058.21,1045.83 1058.44,1045.6 1058.67,1044.68 1058.89,1044.19 1059.12,1043.6 1059.35,1044.15 1059.58,1043.35 1059.81,1043.83 1060.03,1043.77 1060.26,1042.92 1060.49,1042.63 1060.72,1041.85 1060.94,1042.17 1061.17,1041.55 1061.4,1042.54 1061.63,1043.05 1061.86,1043.6 1062.08,1044.19 1062.31,1045.31 1062.54,1045.53 1062.77,1046.26 1062.99,1047.13 1063.22,1046.46 1063.45,1048.11 1063.68,1047.95 1063.91,1049.22 1064.13,1049.69 1064.36,1049.79 1064.59,1049.93 1064.82,1050.01 1065.04,1049.59 1065.27,1048.66 1065.5,1049.02 1065.73,1050.65 1065.96,1049.68 1066.18,1047.75 1066.41,1047.49 1066.64,1047.41 1066.87,1047.81 1067.09,1047.46 1067.32,1047.28 1067.55,1048.07 1067.78,1048.03 1068.01,1048.46 1068.23,1049.48 1068.46,1049.76 1068.69,1049.28 1068.92,1049.85 1069.14,1048.57 1069.37,1048.75 1069.6,1048.84 1069.83,1047.81 1070.06,1047 1070.28,1047.55 1070.51,1045.47 1070.74,1044.6 1070.97,1045.03 1071.19,1045.61 1071.42,1044.76 1071.65,1045.13 1071.88,1045.66 1072.11,1047.03 1072.33,1045.56 1072.56,1046.48 1072.79,1046.61 1073.02,1047.54 1073.24,1045.8 1073.47,1045.87 1073.7,1046.18 1073.93,1047.43 1074.16,1046.21 1074.38,1046.53 1074.61,1046.62 1074.84,1044.88 1075.07,1045.08 1075.29,1045.58 1075.52,1044.76 1075.75,1044.63 1075.98,1043.99 1076.2,1043.39 1076.43,1043.24 1076.66,1043.11 1076.89,1043.83 1077.12,1044 1077.34,1043.8 1077.57,1043.17 1077.8,1043.12 1078.03,1042.37 1078.25,1041.38 1078.48,1042.44 1078.71,1043.31 1078.94,1040.5 1079.17,1039.06 1079.39,1038.43 1079.62,1039.57 1079.85,1039.97 1080.08,1039.24 1080.3,1038.62 1080.53,1037.63 1080.76,1037.51 1080.99,1036.74 1081.22,1037.07 1081.44,1038.01 1081.67,1038.69 1081.9,1037.96 1082.13,1038.53 1082.35,1038.96 1082.58,1039.93 1082.81,1040.07 1083.04,1041.31 1083.27,1040.98 1083.49,1038.12 1083.72,1038.63 1083.95,1039.38 1084.18,1039.29 1084.4,1038.61 1084.63,1037.67 1084.86,1037.81 1085.09,1038.33 1085.32,1038.74 1085.54,1039.25 1085.77,1039.12 1086,1040.4 1086.23,1040.88 1086.45,1040.09 1086.68,1039.67 1086.91,1039.95 1087.14,1039.48 1087.37,1039.85 1087.59,1039.87 1087.82,1039.17 1088.05,1040 1088.28,1039.84 1088.5,1040.8 1088.73,1041.41 1088.96,1040.14 1089.19,1039.85 1089.42,1039.92 1089.64,1040.42 1089.87,1041.18 1090.1,1040.71 1090.33,1040.16 1090.55,1039.62 1090.78,1038.94 1091.01,1039.53 1091.24,1040.12 1091.47,1040.36 1091.69,1040.76 1091.92,1039.58 1092.15,1039.02 1092.38,1038.68 1092.6,1038.97 1092.83,1036.98 1093.06,1036.58 1093.29,1038.16 1093.52,1037.79 1093.74,1037.22 1093.97,1036.68 1094.2,1035.12 1094.43,1035.13 1094.65,1033.81 1094.88,1034.68 1095.11,1035.17 1095.34,1034.76 1095.57,1034.52 1095.79,1034.95 1096.02,1035.36 1096.25,1034.95 1096.48,1034.27 1096.7,1034.22 1096.93,1035.17 1097.16,1033.93 1097.39,1033.16 1097.62,1032.68 1097.84,1033.54 1098.07,1033.94 1098.3,1033.85 1098.53,1034.44 1098.75,1033.56 1098.98,1033.98 1099.21,1033.99 1099.44,1033.85 1099.67,1033.57 1099.89,1032.84 1100.12,1033.69 1100.35,1033.95 1100.58,1034.4 1100.8,1034.68 1101.03,1034.74 1101.26,1032.64 1101.49,1033.51 1101.72,1033.7 1101.94,1034.66 1102.17,1033.86 1102.4,1033.54 1102.63,1034.12 1102.85,1033.72 1103.08,1033.17 1103.31,1032.28 1103.54,1032.82 1103.77,1033.85 1103.99,1034.26 1104.22,1033.67 1104.45,1033.63 1104.68,1033.43 1104.9,1032.87 1105.13,1033.31 1105.36,1034.02 1105.59,1034.66 1105.82,1034.13 1106.04,1032.86 1106.27,1034.06 1106.5,1034.93 1106.73,1034.51 1106.95,1034.82 1107.18,1035.08 1107.41,1035.98 1107.64,1036.12 1107.87,1036.12 1108.09,1036 1108.32,1037.07 1108.55,1037.86 1108.78,1038.96 1109,1039.04 1109.23,1038.92 1109.46,1037.61 1109.69,1037.15 1109.92,1038.1 1110.14,1039.37 1110.37,1039.9 1110.6,1040.13 1110.83,1039.47 1111.05,1041.14 1111.28,1041.28 1111.51,1041.34 1111.74,1040.87 1111.97,1042.16 1112.19,1042.96 1112.42,1042.82 1112.65,1043.79 1112.88,1044.11 1113.1,1043.32 1113.33,1042.89 1113.56,1043.29 1113.79,1044.24 1114.01,1044.67 1114.24,1044.78 1114.47,1043.86 1114.7,1043.87 1114.93,1042.91 1115.15,1042.63 1115.38,1042.13 1115.61,1042.65 1115.84,1043.74 1116.06,1043.06 1116.29,1042.84 1116.52,1043.48 1116.75,1044.26 1116.98,1043.91 1117.2,1044.15 1117.43,1044.51 1117.66,1044.72 1117.89,1044.18 1118.11,1044 1118.34,1043.16 1118.57,1042.84 1118.8,1042.71 1119.03,1042.68 1119.25,1041.7 1119.48,1042.28 1119.71,1041.5 1119.94,1041.06 1120.16,1041.89 1120.39,1041.68 1120.62,1041.26 1120.85,1041 1121.08,1039.96 1121.3,1039.85 1121.53,1040.25 1121.76,1040.49 1121.99,1039.83 1122.21,1040.29 1122.44,1040.49 1122.67,1040.04 1122.9,1040.38 1123.13,1040.91 1123.35,1040.88 1123.58,1040.15 1123.81,1039.35 1124.04,1039.74 1124.26,1039.27 1124.49,1038.81 1124.72,1038.59 1124.95,1039.71 1125.18,1039.29 1125.4,1038.49 1125.63,1038.31 1125.86,1038.14 1126.09,1038.74 1126.31,1038.37 1126.54,1037.48 1126.77,1037.6 1127,1038.22 1127.23,1038.57 1127.45,1037.7 1127.68,1037.69 1127.91,1037.22 1128.14,1036.91 1128.36,1037.53 1128.59,1038.07 1128.82,1037.76 1129.05,1037.87 1129.28,1037.12 1129.5,1038.18 1129.73,1038.86 1129.96,1038.76 1130.19,1040.37 1130.41,1040.57 1130.64,1040.71 1130.87,1041.87 1131.1,1041.4 1131.33,1041.2 1131.55,1041.07 1131.78,1041.13 1132.01,1042.08 1132.24,1042.99 1132.46,1044.22 1132.69,1044.17 1132.92,1044.98 1133.15,1043.84 1133.38,1043.31 1133.6,1044.25 1133.83,1043.28 1134.06,1043.59 1134.29,1043.37 1134.51,1043.06 1134.74,1042.45 1134.97,1042.5 1135.2,1042.38 1135.43,1042.86 1135.65,1043.21 1135.88,1043.12 1136.11,1041.67 1136.34,1041.67 1136.56,1041.77 1136.79,1043.21 1137.02,1041.82 1137.25,1041.42 1137.48,1041.56 1137.7,1041.37 1137.93,1039.85 1138.16,1040.18 1138.39,1040.68 1138.61,1040.91 1138.84,1041.7 1139.07,1041.37 1139.3,1042.44 1139.53,1042.62 1139.75,1041.6 1139.98,1041.34 1140.21,1041.26 1140.44,1041.45 1140.66,1041.08 1140.89,1039.42 1141.12,1039.57 1141.35,1039.3 1141.58,1040.41 1141.8,1041.04 1142.03,1039.67 1142.26,1039.35 1142.49,1039.31 1142.71,1039.47 1142.94,1037.47 1143.17,1036.42 1143.4,1036.21 1143.63,1035.45 1143.85,1034.61 1144.08,1034.52 1144.31,1034.22 1144.54,1032.32 1144.76,1033.12 1144.99,1032.99 1145.22,1032.97 1145.45,1033.32 1145.68,1033.72 1145.9,1034.11 1146.13,1034.08 1146.36,1033.82 1146.59,1033.34 1146.81,1031.78 1147.04,1031.72 1147.27,1031.06 1147.5,1032.46 1147.73,1031.93 1147.95,1031.77 1148.18,1031 1148.41,1031.56 1148.64,1032.14 1148.86,1031.93 1149.09,1031.03 1149.32,1032.39 1149.55,1032.71 1149.77,1032.23 1150,1032.01 1150.23,1032.62 1150.46,1032.24 1150.69,1032.25 1150.91,1031.88 1151.14,1032.3 1151.37,1031.48 1151.6,1030.84 1151.82,1030.28 1152.05,1028.35 1152.28,1028.02 1152.51,1028 1152.74,1028.82 1152.96,1028.5 1153.19,1029.1 1153.42,1028.84 1153.65,1029.19 1153.87,1028.45 1154.1,1029.48 1154.33,1030.56 1154.56,1031.53 1154.79,1032.13 1155.01,1031.73 1155.24,1031.93 1155.47,1032.17 1155.7,1033.16 1155.92,1033.08 1156.15,1033.14 1156.38,1033.44 1156.61,1033.67 1156.84,1032.63 1157.06,1032.79 1157.29,1032.34 1157.52,1032.14 1157.75,1031.27 1157.97,1031.35 1158.2,1031.58 1158.43,1031.56 1158.66,1029.86 1158.89,1030.17 1159.11,1030.33 1159.34,1030.42 1159.57,1031.3 1159.8,1031.8 1160.02,1031.86 1160.25,1030.78 1160.48,1030.01 1160.71,1030.22 1160.94,1030.53 1161.16,1030.85 1161.39,1030.67 1161.62,1029.77 1161.85,1029.69 1162.07,1028.07 1162.3,1028.1 1162.53,1028.57 1162.76,1028.35 1162.99,1028.88 1163.21,1029 1163.44,1028.44 1163.67,1028.67 1163.9,1028.2 1164.12,1029.04 1164.35,1028.65 1164.58,1028.53 1164.81,1028.96 1165.04,1027.07 1165.26,1027.2 1165.49,1028.08 1165.72,1027.96 1165.95,1027.69 1166.17,1028.53 1166.4,1029.36 1166.63,1029.48 1166.86,1029.32 1167.09,1028.78 1167.31,1028.76 1167.54,1029.54 1167.77,1029.68 1168,1030.36 1168.22,1031.1 1168.45,1031.69 1168.68,1031.6 1168.91,1031.89 1169.14,1032.32 1169.36,1032 1169.59,1032.03 1169.82,1032.38 1170.05,1031.7 1170.27,1031.26 1170.5,1031.63 1170.73,1031.12 1170.96,1031.62 1171.19,1031.07 1171.41,1030.7 1171.64,1030.67 1171.87,1030.48 1172.1,1029.51 1172.32,1028.8 1172.55,1028 1172.78,1028.25 1173.01,1028.07 1173.24,1026.88 1173.46,1028.05 1173.69,1028.09 1173.92,1027.09 1174.15,1026.53 1174.37,1027.54 1174.6,1026.4 1174.83,1026.04 1175.06,1025.88 1175.29,1026.38 1175.51,1024.65 1175.74,1025.48 1175.97,1025.51 1176.2,1025.42 1176.42,1024.74 1176.65,1025.2 1176.88,1025.5 1177.11,1025.77 1177.34,1026.25 1177.56,1026.38 1177.79,1026.92 1178.02,1028.13 1178.25,1028.11 1178.47,1028.44 1178.7,1028.5 1178.93,1027.82 1179.16,1027.84 1179.39,1027.01 1179.61,1026.74 1179.84,1027.51 1180.07,1027.5 1180.3,1027.53 1180.52,1027.2 1180.75,1027.29 1180.98,1026.94 1181.21,1026.43 1181.44,1025.96 1181.66,1025.6 1181.89,1025.39 1182.12,1025.37 1182.35,1025.23 1182.57,1026.63 1182.8,1026.74 1183.03,1025.92 1183.26,1025.7 1183.49,1026.03 1183.71,1027 1183.94,1025.23 1184.17,1024.86 1184.4,1025.34 1184.62,1024.62 1184.85,1025.56 1185.08,1026 1185.31,1026.13 1185.53,1026.08 1185.76,1025.7 1185.99,1026.42 1186.22,1027.67 1186.45,1027.13 1186.67,1026.62 1186.9,1026.82 1187.13,1028.06 1187.36,1028.26 1187.58,1028.39 1187.81,1029.64 1188.04,1029.26 1188.27,1029.29 1188.5,1029.14 1188.72,1029.84 1188.95,1029.46 1189.18,1029.77 1189.41,1028.73 1189.63,1028.37 1189.86,1028.18 1190.09,1027.81 1190.32,1027.03 1190.55,1028.43 1190.77,1029.12 1191,1029.81 1191.23,1028.78 1191.46,1027.98 1191.68,1028.11 1191.91,1028.56 1192.14,1028.49 1192.37,1030.15 1192.6,1030.77 1192.82,1031.31 1193.05,1030.22 1193.28,1028.91 1193.51,1029.01 1193.73,1029.62 1193.96,1028.91 1194.19,1029.02 1194.42,1029.56 1194.65,1027.97 1194.87,1029.15 1195.1,1029.16 1195.33,1028.77 1195.56,1029.38 1195.78,1030.46 1196.01,1030.98 1196.24,1031.53 1196.47,1031.17 1196.7,1031.57 1196.92,1031.06 1197.15,1030.13 1197.38,1029.94 1197.61,1030.46 1197.83,1029.84 1198.06,1029.55 1198.29,1030.18 1198.52,1030.29 1198.75,1029.4 1198.97,1029.96 1199.2,1030.78 1199.43,1031.82 1199.66,1031.95 1199.88,1031.68 1200.11,1031.98 1200.34,1032.35 1200.57,1032.69 1200.8,1032.34 1201.02,1032.11 1201.25,1032.52 1201.48,1032.06 1201.71,1032.76 1201.93,1032.92 1202.16,1032.59 1202.39,1032.23 1202.62,1032.35 1202.85,1032.43 1203.07,1033.17 1203.3,1033.12 1203.53,1033.73 1203.76,1033.85 1203.98,1034.62 1204.21,1035.82 1204.44,1036.67 1204.67,1036.17 1204.9,1035.51 1205.12,1035.27 1205.35,1034.79 1205.58,1035.31 1205.81,1035.72 1206.03,1036.22 1206.26,1036.17 1206.49,1035.6 1206.72,1035.33 1206.95,1035.37 1207.17,1035.84 1207.4,1036.48 1207.63,1036.15 1207.86,1036.33 1208.08,1034.58 1208.31,1035.11 1208.54,1034.19 1208.77,1034 1209,1033.1 1209.22,1032.64 1209.45,1032.08 1209.68,1033.08 1209.91,1033.18 1210.13,1033.61 1210.36,1032.4 1210.59,1032.73 1210.82,1032.65 1211.05,1032.97 1211.27,1032.28 1211.5,1033.16 1211.73,1033.81 1211.96,1034.32 1212.18,1034.89 1212.41,1035.23 1212.64,1036.46 1212.87,1036.31 1213.1,1036.4 1213.32,1036.15 1213.55,1036.32 1213.78,1036.45 1214.01,1036.95 1214.23,1036.76 1214.46,1036.95 1214.69,1037.95 1214.92,1038.38 1215.15,1038.6 1215.37,1038.37 1215.6,1036.9 1215.83,1037.51 1216.06,1038.4 1216.28,1038.5 1216.51,1038.62 1216.74,1038.92 1216.97,1039.69 1217.2,1039.71 1217.42,1041.06 1217.65,1040.95 1217.88,1040.84 1218.11,1039.92 1218.33,1039.25 1218.56,1040.6 1218.79,1040.14 1219.02,1040.17 1219.25,1039.65 1219.47,1040.13 1219.7,1040.61 1219.93,1040.89 1220.16,1041.39 1220.38,1040.65 1220.61,1040.39 1220.84,1039.57 1221.07,1039.9 1221.29,1040.44 1221.52,1041.61 1221.75,1042.3 1221.98,1040.97 1222.21,1039.53 1222.43,1039.74 1222.66,1039.98 1222.89,1040.41 1223.12,1040.19 1223.34,1040.38 1223.57,1039.85 1223.8,1039.13 1224.03,1038.26 1224.26,1038.11 1224.48,1038.26 1224.71,1036.69 1224.94,1037.21 1225.17,1037.4 1225.39,1038.11 1225.62,1038.32 1225.85,1036.96 1226.08,1035.61 1226.31,1034.28 1226.53,1034.66 1226.76,1034 1226.99,1033.91 1227.22,1034.14 1227.44,1034.25 1227.67,1034.34 1227.9,1034.39 1228.13,1035.15 1228.36,1034.25 1228.58,1034.55 1228.81,1034.15 1229.04,1034.45 1229.27,1034.14 1229.49,1034.61 1229.72,1034.79 1229.95,1033.91 1230.18,1034.63 1230.41,1034.75 1230.63,1035.45 1230.86,1035.93 1231.09,1035.79 1231.32,1035.68 1231.54,1035.91 1231.77,1035.02 1232,1033.74 1232.23,1034.76 1232.46,1034.08 1232.68,1034.19 1232.91,1034.03 1233.14,1034.64 1233.37,1035.17 1233.59,1035.09 1233.82,1034.7 1234.05,1034.82 1234.28,1035.38 1234.51,1035.23 1234.73,1033.96 1234.96,1033.65 1235.19,1033.45 1235.42,1033.69 1235.64,1033.95 1235.87,1033.41 1236.1,1033.82 1236.33,1034.35 1236.56,1033.56 1236.78,1034.75 1237.01,1034.5 1237.24,1034.97 1237.47,1034.47 1237.69,1034.57 1237.92,1033.47 1238.15,1033.74 1238.38,1033.98 1238.61,1033.36 1238.83,1033.76 1239.06,1032.84 1239.29,1032.83 1239.52,1032.77 1239.74,1033.5 1239.97,1033.25 1240.2,1033.96 1240.43,1033.34 1240.66,1033.35 1240.88,1033.36 1241.11,1032.8 1241.34,1033.37 1241.57,1032.31 1241.79,1032.42 1242.02,1032.56 1242.25,1032.14 1242.48,1032.66 1242.71,1033.17 1242.93,1030.83 1243.16,1031.24 1243.39,1031.08 1243.62,1031.03 1243.84,1030.88 1244.07,1030.79 1244.3,1030.33 1244.53,1030.07 1244.76,1030.26 1244.98,1031.68 1245.21,1031.94 1245.44,1030.79 1245.67,1031.83 1245.89,1031.67 1246.12,1031.88 1246.35,1031.87 1246.58,1031.39 1246.81,1032.2 1247.03,1032.34 1247.26,1032.73 1247.49,1032.75 1247.72,1033.52 1247.94,1034.38 1248.17,1035.1 1248.4,1035.05 1248.63,1035.65 1248.86,1035.41 1249.08,1035.18 1249.31,1035.86 1249.54,1035.82 1249.77,1036.04 1249.99,1035.85 1250.22,1035.44 1250.45,1035.21 1250.68,1035.11 1250.91,1034.9 1251.13,1035.04 1251.36,1034.74 1251.59,1035.52 1251.82,1034.62 1252.04,1033.04 1252.27,1033.06 1252.5,1033.22 1252.73,1033.32 1252.96,1033.98 1253.18,1033.95 1253.41,1033.75 1253.64,1033.46 1253.87,1033.09 1254.09,1033.3 1254.32,1034.55 1254.55,1034.32 1254.78,1035.22 1255.01,1034.01 1255.23,1032.88 1255.46,1032.9 1255.69,1032.6 1255.92,1033.02 1256.14,1033.85 1256.37,1034.31 1256.6,1033.34 1256.83,1032.69 1257.06,1033.06 1257.28,1033.21 1257.51,1032.61 1257.74,1032.27 1257.97,1031.32 1258.19,1031.62 1258.42,1031.25 1258.65,1030.64 1258.88,1030.73 1259.1,1031.02 1259.33,1031.47 1259.56,1030.64 1259.79,1030.01 1260.02,1030.86 1260.24,1030.97 1260.47,1030.49 1260.7,1031.2 1260.93,1031.29 1261.15,1031.64 1261.38,1031.25 1261.61,1031.37 1261.84,1032.11 1262.07,1031.65 1262.29,1030.57 1262.52,1029.46 1262.75,1029.58 1262.98,1028.71 1263.2,1027.76 1263.43,1028.96 1263.66,1029 1263.89,1028.97 1264.12,1028.9 1264.34,1028.09 1264.57,1028.47 1264.8,1029.53 1265.03,1029.19 1265.25,1029.7 1265.48,1028.99 1265.71,1028.3 1265.94,1029.11 1266.17,1028.78 1266.39,1028.67 1266.62,1028.89 1266.85,1028.9 1267.08,1028.01 1267.3,1028.37 1267.53,1028.16 1267.76,1028.41 1267.99,1028.26 1268.22,1027.73 1268.44,1026.79 1268.67,1026.69 1268.9,1026.07 1269.13,1026.4 1269.35,1026.19 1269.58,1026.87 1269.81,1025.47 1270.04,1024.52 1270.27,1024.23 1270.49,1024.37 1270.72,1024.39 1270.95,1024.36 1271.18,1023.23 1271.4,1023.82 1271.63,1024.05 1271.86,1024.18 1272.09,1023.79 1272.32,1024.12 1272.54,1024.24 1272.77,1023.85 1273,1024.21 1273.23,1024.62 1273.45,1023.83 1273.68,1022.46 1273.91,1023.03 1274.14,1023.68 1274.37,1022.95 1274.59,1024.04 1274.82,1024.01 1275.05,1024.3 1275.28,1024.2 1275.5,1024.55 1275.73,1024.41 1275.96,1024.66 1276.19,1024.33 1276.42,1023.9 1276.64,1023.63 1276.87,1023.47 1277.1,1022.74 1277.33,1022.71 1277.55,1022.6 1277.78,1023.68 1278.01,1024.24 1278.24,1023.78 1278.47,1024.21 1278.69,1024.41 1278.92,1024.07 1279.15,1023.58 1279.38,1023.44 1279.6,1024.86 1279.83,1024.69 1280.06,1024.7 1280.29,1024.62 1280.52,1024.72 1280.74,1024.17 1280.97,1023.98 1281.2,1023.85 1281.43,1024.54 1281.65,1024.9 1281.88,1025.03 1282.11,1024.49 1282.34,1024.27 1282.57,1024.94 1282.79,1024.04 1283.02,1023.08 1283.25,1022.97 1283.48,1023.28 1283.7,1023.79 1283.93,1022.72 1284.16,1023.55 1284.39,1023.28 1284.62,1023.57 1284.84,1023.77 1285.07,1024.25 1285.3,1023.94 1285.53,1023.36 1285.75,1023.4 1285.98,1023.09 1286.21,1022.71 1286.44,1023.28 1286.67,1024.04 1286.89,1024.38 1287.12,1024.62 1287.35,1024.77 1287.58,1024.96 1287.8,1024.89 1288.03,1024.46 1288.26,1024.7 1288.49,1024.47 1288.72,1024.15 1288.94,1024.02 1289.17,1023.71 1289.4,1023.55 1289.63,1022.73 1289.85,1022.89 1290.08,1023.34 1290.31,1024.08 1290.54,1025.82 1290.77,1025.25 1290.99,1026 1291.22,1025.61 1291.45,1025.36 1291.68,1025.81 1291.9,1026.51 1292.13,1026.74 1292.36,1027.08 1292.59,1026.77 1292.82,1026.93 1293.04,1025.64 1293.27,1025.34 1293.5,1024.02 1293.73,1024.2 1293.95,1023.3 1294.18,1024.11 1294.41,1024.97 1294.64,1024.93 1294.86,1024.62 1295.09,1024.96 1295.32,1024.43 1295.55,1025.27 1295.78,1024.22 1296,1024.4 1296.23,1024.77 1296.46,1024.86 1296.69,1024.84 1296.91,1024.76 1297.14,1024.84 1297.37,1024.56 1297.6,1024.02 1297.83,1022.23 1298.05,1022.54 1298.28,1023.58 1298.51,1024.17 1298.74,1024.48 1298.96,1024.76 1299.19,1025.87 1299.42,1025.54 1299.65,1024.24 1299.88,1024.14 1300.1,1023.36 1300.33,1023.01 1300.56,1022.89 1300.79,1022.74 1301.01,1022.79 1301.24,1023.37 1301.47,1022.88 1301.7,1022.54 1301.93,1021.95 1302.15,1022.05 1302.38,1022.48 1302.61,1022.03 1302.84,1021.8 1303.06,1021.73 1303.29,1021.59 1303.52,1021.59 1303.75,1022.3 1303.98,1022.22 1304.2,1022.27 1304.43,1022.4 1304.66,1022.65 1304.89,1022.16 1305.11,1021.81 1305.34,1022.01 1305.57,1022.88 1305.8,1023.28 1306.03,1023.61 1306.25,1024.36 1306.48,1024.56 1306.71,1024.27 1306.94,1024.7 1307.16,1024.63 1307.39,1025.2 1307.62,1025.05 1307.85,1025.46 1308.08,1026.54 1308.3,1026.47 1308.53,1025.96 1308.76,1026.41 1308.99,1026.49 1309.21,1026.29 1309.44,1026.49 1309.67,1027.43 1309.9,1028.31 1310.13,1028.95 1310.35,1029.08 1310.58,1028.83 1310.81,1027.94 1311.04,1027.51 1311.26,1026.75 1311.49,1027.69 1311.72,1027.76 1311.95,1027.19 1312.18,1025.96 1312.4,1026.12 1312.63,1026.03 1312.86,1026.85 1313.09,1026.34 1313.31,1027.23 1313.54,1026.77 1313.77,1026.57 1314,1026.77 1314.23,1027.21 1314.45,1026.99 1314.68,1027.42 1314.91,1028.34 1315.14,1028.1 1315.36,1028.31 1315.59,1028.06 1315.82,1028.56 1316.05,1028.76 1316.28,1029.27 1316.5,1028.1 1316.73,1027.77 1316.96,1028.08 1317.19,1027.75 1317.41,1028.55 1317.64,1028.57 1317.87,1028.83 1318.1,1028.74 1318.33,1027.81 1318.55,1027.86 1318.78,1028.61 1319.01,1029.37 1319.24,1029.98 1319.46,1030.37 1319.69,1029.31 1319.92,1029.1 1320.15,1029.91 1320.38,1029.83 1320.6,1029.86 1320.83,1030.15 1321.06,1029.71 1321.29,1030.38 1321.51,1030.74 1321.74,1031.04 1321.97,1030.53 1322.2,1030.56 1322.43,1030.92 1322.65,1030.93 1322.88,1030.66 1323.11,1030.49 1323.34,1031.53 1323.56,1032.23 1323.79,1032.75 1324.02,1033.37 1324.25,1033.63 1324.48,1034.33 1324.7,1034.78 1324.93,1035.33 1325.16,1035.55 1325.39,1035.85 1325.61,1034.98 1325.84,1035.6 1326.07,1035.52 1326.3,1035.39 1326.53,1035.59 1326.75,1036.11 1326.98,1035.28 1327.21,1035.25 1327.44,1035.46 1327.66,1035.45 1327.89,1035.97 1328.12,1036.68 1328.35,1037.23 1328.58,1037.86 1328.8,1038.04 1329.03,1038.02 1329.26,1037.62 1329.49,1036.59 1329.71,1036.21 1329.94,1035.5 1330.17,1035.54 1330.4,1035.45 1330.62,1035.27 1330.85,1035.37 1331.08,1034.99 1331.31,1035.47 1331.54,1035.06 1331.76,1033.92 1331.99,1032.89 1332.22,1033.76 1332.45,1033.67 1332.67,1033.56 1332.9,1032.45 1333.13,1032.23 1333.36,1032.11 1333.59,1030.4 1333.81,1031.24 1334.04,1031.51 1334.27,1032.19 1334.5,1032.87 1334.72,1033.9 1334.95,1034.52 1335.18,1034.8 1335.41,1035.48 1335.64,1035.09 1335.86,1034.88 1336.09,1035.09 1336.32,1034.25 1336.55,1033.71 1336.77,1033.36 1337,1033.22 1337.23,1033.34 1337.46,1034.27 1337.69,1034.74 1337.91,1034.6 1338.14,1034.71 1338.37,1035.58 1338.6,1035.15 1338.82,1035.21 1339.05,1035.39 1339.28,1035.99 1339.51,1036.12 1339.74,1036.18 1339.96,1036.18 1340.19,1036.39 1340.42,1035.02 1340.65,1035.52 1340.87,1036.53 1341.1,1037.47 1341.33,1037.85 1341.56,1038.54 1341.79,1038.45 1342.01,1038.7 1342.24,1038.06 1342.47,1038.48 1342.7,1039.05 1342.92,1039.59 1343.15,1039.98 1343.38,1038.64 1343.61,1038.22 1343.84,1037.96 1344.06,1038.21 1344.29,1037.73 1344.52,1037.11 1344.75,1037.73 1344.97,1036.6 1345.2,1036.78 1345.43,1036.64 1345.66,1035.48 1345.89,1033.84 1346.11,1033.91 1346.34,1031.97 1346.57,1031.5 1346.8,1031.73 1347.02,1031.44 1347.25,1031.58 1347.48,1030.92 1347.71,1031.11 1347.94,1030.86 1348.16,1031.01 1348.39,1031.02 1348.62,1030.52 1348.85,1030.49 1349.07,1030.26 1349.3,1030.32 1349.53,1028.46 1349.76,1029.19 1349.99,1029.97 1350.21,1029.85 1350.44,1029.38 1350.67,1029.31 1350.9,1030.11 1351.12,1030.29 1351.35,1030.33 1351.58,1030.2 1351.81,1031.18 1352.04,1030.86 1352.26,1031.52 1352.49,1032.21 1352.72,1032.54 1352.95,1031.82 1353.17,1032.81 1353.4,1032.71 1353.63,1033.69 1353.86,1032.68 1354.09,1033.25 1354.31,1032.9 1354.54,1033.76 1354.77,1032.33 1355,1032.45 1355.22,1032.67 1355.45,1032.38 1355.68,1032.63 1355.91,1032.04 1356.14,1030.49 1356.36,1031.1 1356.59,1030.94 1356.82,1031.84 1357.05,1031.36 1357.27,1032.03 1357.5,1032.92 1357.73,1033.48 1357.96,1033.03 1358.19,1033.05 1358.41,1032.98 1358.64,1033.12 1358.87,1032.66 1359.1,1032.2 1359.32,1032.58 1359.55,1033.28 1359.78,1033.02 1360.01,1032.44 1360.24,1032.88 1360.46,1032.28 1360.69,1031.57 1360.92,1031.52 1361.15,1031.74 1361.37,1032.15 1361.6,1032.4 1361.83,1031.49 1362.06,1031.85 1362.29,1031.65 1362.51,1030.56 1362.74,1031.33 1362.97,1031.56 1363.2,1031.93 1363.42,1032.15 1363.65,1032.72 1363.88,1032.52 1364.11,1032.56 1364.34,1032.25 1364.56,1032.56 1364.79,1031.97 1365.02,1031.72 1365.25,1031.1 1365.47,1031.7 1365.7,1032.58 1365.93,1032.74 1366.16,1032.58 1366.39,1032.7 1366.61,1034.15 1366.84,1033.4 1367.07,1034.43 1367.3,1034.64 1367.52,1034.4 1367.75,1033.99 1367.98,1034.28 1368.21,1035.04 1368.43,1035.12 1368.66,1035.59 1368.89,1035.85 1369.12,1036.26 1369.35,1036.64 1369.57,1035.92 1369.8,1036.67 1370.03,1037.09 1370.26,1037.66 1370.48,1037.64 1370.71,1037.79 1370.94,1037.71 1371.17,1037.85 1371.4,1037.81 1371.62,1037.7 1371.85,1038.3 1372.08,1038.65 1372.31,1037.85 1372.53,1037.94 1372.76,1039.29 1372.99,1038.81 1373.22,1038.75 1373.45,1038.82 1373.67,1039.33 1373.9,1039.5 1374.13,1039.57 1374.36,1039.23 1374.58,1038.99 1374.81,1038.43 1375.04,1038.68 1375.27,1039.05 1375.5,1038.34 1375.72,1037.75 1375.95,1036.47 1376.18,1036.84 1376.41,1036.71 1376.63,1037.54 1376.86,1037.09 1377.09,1036.84 1377.32,1037.38 1377.55,1037.22 1377.77,1037.35 1378,1038.25 1378.23,1038.91 1378.46,1039.79 1378.68,1039.87 1378.91,1039.79 1379.14,1040.89 1379.37,1040.35 1379.6,1040.24 1379.82,1040.17 1380.05,1039.75 1380.28,1040.22 1380.51,1039.79 1380.73,1039.94 1380.96,1040.02 1381.19,1039.37 1381.42,1039.07 1381.65,1038.59 1381.87,1039.02 1382.1,1039.05 1382.33,1039.51 1382.56,1039.77 1382.78,1039.65 1383.01,1039.46 1383.24,1039.78 1383.47,1040.22 1383.7,1039.76 1383.92,1040.08 1384.15,1039.28 1384.38,1040.45 1384.61,1040.94 1384.83,1040.74 1385.06,1040.06 1385.29,1039.41 1385.52,1039.01 1385.75,1038.02 1385.97,1037.81 1386.2,1037.85 1386.43,1037.79 1386.66,1037.7 1386.88,1037.72 1387.11,1038.09 1387.34,1038.37 1387.57,1037.81 1387.8,1038.41 1388.02,1037.8 1388.25,1037.75 1388.48,1037.57 1388.71,1038.14 1388.93,1037.84 1389.16,1037.71 1389.39,1036.02 1389.62,1036.03 1389.85,1035.86 1390.07,1035.71 1390.3,1035.74 1390.53,1035.49 1390.76,1035.82 1390.98,1036.46 1391.21,1036.86 1391.44,1037.59 1391.67,1037.67 1391.9,1038.16 1392.12,1038.21 1392.35,1038.81 1392.58,1038.9 1392.81,1038.75 1393.03,1039.28 1393.26,1038.78 1393.49,1037.41 1393.72,1037.43 1393.95,1036.61 1394.17,1037.22 1394.4,1037.21 1394.63,1037.5 1394.86,1038.53 1395.08,1038.74 1395.31,1037.39 1395.54,1035.74 1395.77,1035.3 1396,1036.08 1396.22,1036.02 1396.45,1037.08 1396.68,1036.83 1396.91,1037.2 1397.13,1037.91 1397.36,1038.11 1397.59,1038.79 1397.82,1038.56 1398.05,1038.2 1398.27,1038.31 1398.5,1037.92 1398.73,1037.9 1398.96,1037.41 1399.18,1038.32 1399.41,1038.44 1399.64,1039.16 1399.87,1039.85 1400.1,1039.86 1400.32,1040.07 1400.55,1040.07 1400.78,1040.44 1401.01,1040.47 1401.23,1040.94 1401.46,1041.74 1401.69,1041.98 1401.92,1041.92 1402.15,1042.27 1402.37,1042.8 1402.6,1043.34 1402.83,1043.73 1403.06,1041.87 1403.28,1041.33 1403.51,1041.81 1403.74,1041.35 1403.97,1040.42 1404.19,1040.51 1404.42,1040.73 1404.65,1041.32 1404.88,1041.7 1405.11,1042.02 1405.33,1042.23 1405.56,1042.09 1405.79,1041.6 1406.02,1041.73 1406.24,1041.2 1406.47,1040.95 1406.7,1039.71 1406.93,1040.06 1407.16,1039.91 1407.38,1039.26 1407.61,1039.49 1407.84,1038.93 1408.07,1038.82 1408.29,1038.4 1408.52,1037.74 1408.75,1037.78 1408.98,1037.78 1409.21,1037.46 1409.43,1037.47 1409.66,1037.66 1409.89,1037.8 1410.12,1038 1410.34,1038.03 1410.57,1037.92 1410.8,1037.86 1411.03,1037.49 1411.26,1036.92 1411.48,1037.51 1411.71,1037.84 1411.94,1037.66 1412.17,1037.88 1412.39,1037.97 1412.62,1038.07 1412.85,1037.15 1413.08,1037.17 1413.31,1037.76 1413.53,1038 1413.76,1037.18 1413.99,1036.69 1414.22,1036.38 1414.44,1035.87 1414.67,1036.44 1414.9,1035.87 1415.13,1036.09 1415.36,1035.3 1415.58,1035.82 1415.81,1034.76 1416.04,1035.54 1416.27,1035.68 1416.49,1036 1416.72,1036.54 1416.95,1035.9 1417.18,1036.11 1417.41,1036.47 1417.63,1035.8 1417.86,1035.66 1418.09,1035.59 1418.32,1035.91 1418.54,1035.36 1418.77,1035.58 1419,1034.26 1419.23,1033.55 1419.46,1033.73 1419.68,1033.8 1419.91,1033.07 1420.14,1033.24 1420.37,1032.94 1420.59,1032.63 1420.82,1031.78 1421.05,1031.63 1421.28,1031.94 1421.51,1032.84 1421.73,1032.89 1421.96,1033.34 1422.19,1032.87 1422.42,1033.09 1422.64,1033.62 1422.87,1033.54 1423.1,1033.24 1423.33,1033.65 1423.56,1033.9 1423.78,1034.84 1424.01,1035.17 1424.24,1035.34 1424.47,1034.92 1424.69,1035.36 1424.92,1035.82 1425.15,1035.59 1425.38,1035.84 1425.61,1035.87 1425.83,1035.77 1426.06,1036.66 1426.29,1036.52 1426.52,1037.21 1426.74,1037.68 1426.97,1037.32 1427.2,1037.48 1427.43,1037.34 1427.66,1037.08 1427.88,1037.06 1428.11,1035.97 1428.34,1036.02 1428.57,1035.99 1428.79,1035.39 1429.02,1035.02 1429.25,1035.43 1429.48,1034.28 1429.71,1033.99 1429.93,1033.43 1430.16,1033.79 1430.39,1034.21 1430.62,1034.94 1430.84,1035.91 1431.07,1035.28 1431.3,1035.14 1431.53,1034.71 1431.76,1034.36 1431.98,1034.85 1432.21,1034.36 1432.44,1034.45 1432.67,1034.14 1432.89,1034.08 1433.12,1033.5 1433.35,1032.7 1433.58,1032.17 1433.81,1032.58 1434.03,1032.15 1434.26,1032.37 1434.49,1032.37 1434.72,1032.59 1434.94,1032.46 1435.17,1032.21 1435.4,1032.57 1435.63,1032.75 1435.86,1033.22 1436.08,1033.22 1436.31,1033.47 1436.54,1033.23 1436.77,1033.3 1436.99,1033.7 1437.22,1033.96 1437.45,1034.87 1437.68,1034.88 1437.91,1034.72 1438.13,1034.34 1438.36,1033.58 1438.59,1033.51 1438.82,1033.45 1439.04,1033.58 1439.27,1033.36 1439.5,1033.89 1439.73,1034.37 1439.95,1033.97 1440.18,1034.98 1440.41,1035.09 1440.64,1035.39 1440.87,1036.03 1441.09,1036.5 1441.32,1037.19 1441.55,1037.48 1441.78,1037.56 1442,1037.04 1442.23,1036.63 1442.46,1037.04 1442.69,1037.55 1442.92,1037.73 1443.14,1038.16 1443.37,1038.16 1443.6,1038.75 1443.83,1039.04 1444.05,1039.65 1444.28,1039.04 1444.51,1037.93 1444.74,1037.81 1444.97,1038.21 1445.19,1039.19 1445.42,1040.38 1445.65,1040.13 1445.88,1040.14 1446.1,1039.97 1446.33,1039.93 1446.56,1039.72 1446.79,1039.37 1447.02,1039.58 1447.24,1040.32 1447.47,1040.91 1447.7,1041.17 1447.93,1041.54 1448.15,1040.87 1448.38,1040.7 1448.61,1040.14 1448.84,1040.21 1449.07,1040.32 1449.29,1039.29 1449.52,1039.84 1449.75,1039.92 1449.98,1038.98 1450.2,1039.41 1450.43,1039.82 1450.66,1040.05 1450.89,1040.92 1451.12,1040.47 1451.34,1040.34 1451.57,1040.42 1451.8,1040.42 1452.03,1040.44 1452.25,1040.36 1452.48,1040.6 1452.71,1041.42 1452.94,1041.87 1453.17,1041.74 1453.39,1042.42 1453.62,1043 1453.85,1043.45 1454.08,1043.08 1454.3,1043.16 1454.53,1042.88 1454.76,1042.48 1454.99,1042.45 1455.22,1042.42 1455.44,1042.46 1455.67,1042.23 1455.9,1041.63 1456.13,1041.62 1456.35,1040.72 1456.58,1040.44 1456.81,1040.42 1457.04,1039.56 1457.27,1038.79 1457.49,1039.38 1457.72,1039.98 1457.95,1039.47 1458.18,1039.83 1458.4,1039.7 1458.63,1040.14 1458.86,1040.2 1459.09,1040.71 1459.32,1040.13 1459.54,1040.09 1459.77,1040.13 1460,1039.55 1460.23,1039.69 1460.45,1039.36 1460.68,1038.84 1460.91,1038.89 1461.14,1038.14 1461.37,1038.42 1461.59,1038.46 1461.82,1039.02 1462.05,1038.4 1462.28,1039.25 1462.5,1039.32 1462.73,1039.44 1462.96,1039.3 1463.19,1039.57 1463.42,1038.57 1463.64,1039.77 1463.87,1039.88 1464.1,1040.26 1464.33,1040.31 1464.55,1040.49 1464.78,1040.15 1465.01,1040.69 1465.24,1040.72 1465.47,1040.97 1465.69,1041.31 1465.92,1041.2 1466.15,1041.39 1466.38,1041.79 1466.6,1040.25 1466.83,1039.95 1467.06,1039.49 1467.29,1038.79 1467.52,1038.73 1467.74,1039.28 1467.97,1037.92 1468.2,1038.22 1468.43,1038.63 1468.65,1037.33 1468.88,1037.93 1469.11,1038.35 1469.34,1037.75 1469.57,1038.25 1469.79,1037.85 1470.02,1038.79 1470.25,1039.01 1470.48,1039.56 1470.7,1038.91 1470.93,1038.88 1471.16,1039.49 1471.39,1038.95 1471.62,1039.03 1471.84,1039.1 1472.07,1039.18 1472.3,1038.56 1472.53,1038.8 1472.75,1039.06 1472.98,1039.35 1473.21,1040.11 1473.44,1040.04 1473.67,1041.03 1473.89,1040.86 1474.12,1040.14 1474.35,1039.98 1474.58,1039.36 1474.8,1039.17 1475.03,1038.84 1475.26,1039.57 1475.49,1039.83 1475.71,1039.55 1475.94,1039.34 1476.17,1038.05 1476.4,1038.12 1476.63,1039.01 1476.85,1038.8 1477.08,1039.63 1477.31,1040.03 1477.54,1039.43 1477.76,1039.56 1477.99,1039.28 1478.22,1038.34 1478.45,1039.21 1478.68,1038.45 1478.9,1038.29 1479.13,1037.68 1479.36,1037.52 1479.59,1037.19 1479.81,1036.88 1480.04,1037.12 1480.27,1036.97 1480.5,1036.12 1480.73,1036.26 1480.95,1036.51 1481.18,1035.7 1481.41,1036.13 1481.64,1036.2 1481.86,1036.14 1482.09,1036.47 1482.32,1037 1482.55,1037.09 1482.78,1036.39 1483,1036.87 1483.23,1036.88 1483.46,1035.98 1483.69,1035.11 1483.91,1034.97 1484.14,1034.91 1484.37,1035.02 1484.6,1034.55 1484.83,1034.22 1485.05,1034.23 1485.28,1034.13 1485.51,1034.32 1485.74,1034.63 1485.96,1034.69 1486.19,1034.85 1486.42,1035.24 1486.65,1035.88 1486.88,1035.03 1487.1,1034.27 1487.33,1034.11 1487.56,1034.32 1487.79,1034.35 1488.01,1033.55 1488.24,1033.86 1488.47,1033.76 1488.7,1033.63 1488.93,1033.59 1489.15,1034.83 1489.38,1034.88 1489.61,1035.24 1489.84,1034.83 1490.06,1033.26 1490.29,1032.92 1490.52,1033.45 1490.75,1033.79 1490.98,1033.46 1491.2,1033.93 1491.43,1033.84 1491.66,1033.12 1491.89,1033.53 1492.11,1033.71 1492.34,1034.33 1492.57,1033.94 1492.8,1033.72 1493.03,1034.06 1493.25,1034.16 1493.48,1034.41 1493.71,1035.25 1493.94,1035.73 1494.16,1035.77 1494.39,1036.08 1494.62,1036.79 1494.85,1036.3 1495.08,1036.73 1495.3,1037.02 1495.53,1036.92 1495.76,1037.2 1495.99,1037.68 1496.21,1037.93 1496.44,1038.12 1496.67,1038.86 1496.9,1038.15 1497.13,1038.25 1497.35,1038.57 1497.58,1038.06 1497.81,1038.37 1498.04,1038.54 1498.26,1038.57 1498.49,1038.32 1498.72,1037.55 1498.95,1037.5 1499.18,1037.6 1499.4,1037.44 1499.63,1037.79 1499.86,1037.65 1500.09,1038.16 1500.31,1038.54 1500.54,1037.61 1500.77,1037.52 1501,1036.57 1501.23,1036.73 1501.45,1037.15 1501.68,1037.14 1501.91,1036.47 1502.14,1037.09 1502.36,1037.12 1502.59,1036.5 1502.82,1036.95 1503.05,1036.96 1503.28,1036.26 1503.5,1037.12 1503.73,1036.45 1503.96,1036.84 1504.19,1036.33 1504.41,1036.75 1504.64,1037.4 1504.87,1037.83 1505.1,1037.39 1505.33,1037.15 1505.55,1037.4 1505.78,1037.16 1506.01,1036.94 1506.24,1037.74 1506.46,1037.75 1506.69,1037.47 1506.92,1036.81 1507.15,1036.66 1507.38,1036.6 1507.6,1037.16 1507.83,1036.91 1508.06,1035.77 1508.29,1036.47 1508.51,1036.73 1508.74,1037.13 1508.97,1036.24 1509.2,1036.99 1509.43,1036.33 1509.65,1036.79 1509.88,1036.49 1510.11,1036.19 1510.34,1035.28 1510.56,1035.83 1510.79,1036.06 1511.02,1036.82 1511.25,1036.57 1511.48,1036.77 1511.7,1036.96 1511.93,1037.42 1512.16,1037.41 1512.39,1037.67 1512.61,1038.16 1512.84,1038.41 1513.07,1038.87 1513.3,1039.69 1513.52,1039.55 1513.75,1038.55 1513.98,1038.88 1514.21,1039.2 1514.44,1039.43 1514.66,1038.55 1514.89,1038.4 1515.12,1038.77 1515.35,1038.94 1515.57,1038.24 1515.8,1038.58 1516.03,1039.22 1516.26,1038.98 1516.49,1039.07 1516.71,1038.76 1516.94,1039.31 1517.17,1038.95 1517.4,1038.98 1517.62,1038.6 1517.85,1038.14 1518.08,1037.6 1518.31,1038.05 1518.54,1038.27 1518.76,1038.34 1518.99,1038.39 1519.22,1038.36 1519.45,1038.33 1519.67,1038.46 1519.9,1038.69 1520.13,1038.49 1520.36,1038.16 1520.59,1038.12 1520.81,1038.02 1521.04,1039.01 1521.27,1039.14 1521.5,1039.04 1521.72,1039.33 1521.95,1039.27 1522.18,1039.9 1522.41,1039.86 1522.64,1039.89 1522.86,1040.21 1523.09,1040.47 1523.32,1041.11 1523.55,1041.51 1523.77,1041.01 1524,1040.67 1524.23,1040.78 1524.46,1040.25 1524.69,1040.96 1524.91,1041.43 1525.14,1041.09 1525.37,1041.59 1525.6,1040.8 1525.82,1041.02 1526.05,1040.1 1526.28,1040.36 1526.51,1040.51 1526.74,1039.84 1526.96,1039.09 1527.19,1039.45 1527.42,1039.71 1527.65,1039.6 1527.87,1039.73 1528.1,1039.71 1528.33,1039.32 1528.56,1038.95 1528.79,1037.84 1529.01,1037.53 1529.24,1038.24 1529.47,1037.62 1529.7,1037.79 1529.92,1037.55 1530.15,1036.28 1530.38,1036.31 1530.61,1037.07 1530.84,1035.83 1531.06,1035.35 1531.29,1034.76 1531.52,1034.8 1531.75,1033.34 1531.97,1033.56 1532.2,1033.02 1532.43,1032.81 1532.66,1033.08 1532.89,1034.01 1533.11,1033.9 1533.34,1033.72 1533.57,1033.11 1533.8,1034.07 1534.02,1034.47 1534.25,1033.14 1534.48,1032.98 1534.71,1033.21 1534.94,1033.97 1535.16,1033.65 1535.39,1033.41 1535.62,1033.83 1535.85,1034.04 1536.07,1033.81 1536.3,1034.17 1536.53,1034.19 1536.76,1033.62 1536.99,1033.29 1537.21,1033.2 1537.44,1033.84 1537.67,1034.64 1537.9,1034.63 1538.12,1035.25 1538.35,1036.04 1538.58,1035.88 1538.81,1036.73 1539.04,1037.01 1539.26,1037.21 1539.49,1037.87 1539.72,1038.05 1539.95,1038.04 1540.17,1038.18 1540.4,1036.75 1540.63,1036.42 1540.86,1035.9 1541.09,1036.37 1541.31,1036.54 1541.54,1037.01 1541.77,1037.11 1542,1037.52 1542.22,1038.16 1542.45,1038.1 1542.68,1037.28 1542.91,1037.71 1543.14,1037.11 1543.36,1037.69 1543.59,1037.81 1543.82,1037.44 1544.05,1037.12 1544.27,1036.74 1544.5,1035.58 1544.73,1036.68 1544.96,1036.79 1545.19,1036.05 1545.41,1036.06 1545.64,1036.33 1545.87,1037.11 1546.1,1036.48 1546.32,1036.16 1546.55,1036.33 1546.78,1036.53 1547.01,1036.33 1547.24,1036.99 1547.46,1037.58 1547.69,1037.73 1547.92,1038.3 1548.15,1038.75 1548.37,1039.07 1548.6,1039.93 1548.83,1040.07 1549.06,1039.66 1549.28,1039.61 1549.51,1039.67 1549.74,1039.02 1549.97,1039.25 1550.2,1038.78 1550.42,1038.56 1550.65,1038.9 1550.88,1039.12 1551.11,1039.14 1551.33,1039.62 1551.56,1040.55 1551.79,1040.7 1552.02,1040.55 1552.25,1040.74 1552.47,1040.43 1552.7,1040.34 1552.93,1040.92 1553.16,1041.14 1553.38,1040.94 1553.61,1041.01 1553.84,1041.28 1554.07,1041.34 1554.3,1041.87 1554.52,1042.14 1554.75,1041.55 1554.98,1041.68 1555.21,1041.39 1555.43,1041.28 1555.66,1041.31 1555.89,1041.83 1556.12,1042.34 1556.35,1042.64 1556.57,1042.61 1556.8,1042.56 1557.03,1042.23 1557.26,1042.12 1557.48,1041.97 1557.71,1042.13 1557.94,1042.21 1558.17,1042.11 1558.4,1041.48 1558.62,1040.91 1558.85,1041.63 1559.08,1041.08 1559.31,1041.12 1559.53,1040.53 1559.76,1040.82 1559.99,1040.63 1560.22,1041.23 1560.45,1041.14 1560.67,1040.2 1560.9,1039.89 1561.13,1039.66 1561.36,1039.99 1561.58,1040.34 1561.81,1040.9 1562.04,1040.26 1562.27,1040.26 1562.5,1040.08 1562.72,1039.42 1562.95,1039.91 1563.18,1039.09 1563.41,1038.68 1563.63,1037.92 1563.86,1037.69 1564.09,1038.38 1564.32,1038.26 1564.55,1038.25 1564.77,1037.7 1565,1038.69 1565.23,1038.32 1565.46,1038.95 1565.68,1039.1 1565.91,1039.13 1566.14,1039.23 1566.37,1039.33 1566.6,1039.23 1566.82,1038.72 1567.05,1038.55 1567.28,1038.44 1567.51,1038.28 1567.73,1038.54 1567.96,1038.33 1568.19,1038.11 1568.42,1037.47 1568.65,1038.33 1568.87,1038.64 1569.1,1037.81 1569.33,1037.62 1569.56,1038.34 1569.78,1038.4 1570.01,1037.98 1570.24,1037.73 1570.47,1037.2 1570.7,1037.47 1570.92,1037.55 1571.15,1036.76 1571.38,1036.27 1571.61,1036 1571.83,1035.41 1572.06,1035.37 1572.29,1034.94 1572.52,1035.97 1572.75,1036.56 1572.97,1036.83 1573.2,1037.39 1573.43,1038.05 1573.66,1038.28 1573.88,1037.91 1574.11,1037.73 1574.34,1037.7 1574.57,1036.96 1574.8,1036.02 1575.02,1036.03 1575.25,1035.41 1575.48,1035.8 1575.71,1036.17 1575.93,1035.94 1576.16,1036.3 1576.39,1036.53 1576.62,1036.57 1576.85,1036.84 1577.07,1037.4 1577.3,1036.97 1577.53,1036.74 1577.76,1036.88 1577.98,1037.8 1578.21,1037.32 1578.44,1037.17 1578.67,1036.57 1578.9,1036.28 1579.12,1035.51 1579.35,1035.39 1579.58,1035.15 1579.81,1035.25 1580.03,1035.32 1580.26,1035.09 1580.49,1034.63 1580.72,1034.74 1580.95,1034.93 1581.17,1035.6 1581.4,1035.9 1581.63,1035.39 1581.86,1034.43 1582.08,1035.05 1582.31,1034.76 1582.54,1034.67 1582.77,1034.69 1583,1034.87 1583.22,1035.83 1583.45,1036.37 1583.68,1037.07 1583.91,1036.11 1584.13,1035.11 1584.36,1035.78 1584.59,1035.3 1584.82,1035.24 1585.04,1035.51 1585.27,1035.49 1585.5,1035.38 1585.73,1034.85 1585.96,1035.42 1586.18,1036.27 1586.41,1035.57 1586.64,1035.81 1586.87,1035.9 1587.09,1036.03 1587.32,1036.43 1587.55,1036.7 1587.78,1036.34 1588.01,1035.92 1588.23,1035.72 1588.46,1035.31 1588.69,1035.44 1588.92,1036.28 1589.14,1036.68 1589.37,1036.84 1589.6,1037.67 1589.83,1037.79 1590.06,1038.49 1590.28,1038.95 1590.51,1038.39 1590.74,1037.88 1590.97,1038.22 1591.19,1038.55 1591.42,1038.27 1591.65,1038.9 1591.88,1038.58 1592.11,1038.51 1592.33,1038.09 1592.56,1037.96 1592.79,1038.28 1593.02,1038.93 1593.24,1039.3 1593.47,1039.1 1593.7,1039.03 1593.93,1039.02 1594.16,1039.13 1594.38,1038.83 1594.61,1038.69 1594.84,1038.32 1595.07,1038.4 1595.29,1038.72 1595.52,1038.34 1595.75,1037.78 1595.98,1037.15 1596.21,1037.55 1596.43,1036.71 1596.66,1036.76 1596.89,1036.71 1597.12,1036.91 1597.34,1036.85 1597.57,1036.59 1597.8,1035.38 1598.03,1035.65 1598.26,1036.03 1598.48,1035.65 1598.71,1035.41 1598.94,1035.39 1599.17,1035.41 1599.39,1035.04 1599.62,1035.1 1599.85,1034.69 1600.08,1034.79 1600.31,1034.12 1600.53,1033.88 1600.76,1032.8 1600.99,1032.8 1601.22,1032.6 1601.44,1032.45 1601.67,1031.89 1601.9,1030.93 1602.13,1031.16 1602.36,1030.85 1602.58,1031.22 1602.81,1032.07 1603.04,1032.63 1603.27,1032.76 1603.49,1033.55 1603.72,1032.85 1603.95,1033.34 1604.18,1033.1 1604.41,1032.88 1604.63,1033.19 1604.86,1033.91 1605.09,1033.17 1605.32,1033.51 1605.54,1033.15 1605.77,1033.14 1606,1032.95 1606.23,1033.54 1606.46,1034.14 1606.68,1033.8 1606.91,1032.98 1607.14,1033.02 1607.37,1032.07 1607.59,1032.21 1607.82,1032.1 1608.05,1032.42 1608.28,1032.43 1608.51,1030.85 1608.73,1030.99 1608.96,1031.67 1609.19,1031.29 1609.42,1031.19 1609.64,1030.76 1609.87,1031.41 1610.1,1031.41 1610.33,1031.58 1610.56,1031.37 1610.78,1031.68 1611.01,1031.46 1611.24,1030.86 1611.47,1030.59 1611.69,1030.76 1611.92,1031.26 1612.15,1031.66 1612.38,1030.76 1612.61,1030.78 1612.83,1030.61 1613.06,1030.68 1613.29,1030.8 1613.52,1030.99 1613.74,1030.91 1613.97,1030.82 1614.2,1031.32 1614.43,1030.31 1614.66,1030.94 1614.88,1031.2 1615.11,1031.77 1615.34,1032.1 1615.57,1032.75 1615.79,1033.13 1616.02,1033.19 1616.25,1033.32 1616.48,1032.73 1616.71,1033.23 1616.93,1033.25 1617.16,1032.92 1617.39,1032.49 1617.62,1032.86 1617.84,1032 1618.07,1031.92 1618.3,1031.93 1618.53,1031.8 1618.76,1031.52 1618.98,1031.03 1619.21,1031.85 1619.44,1031.35 1619.67,1031.86 1619.89,1031.37 1620.12,1031.78 1620.35,1032.03 1620.58,1031.14 1620.81,1030.27 1621.03,1030.21 1621.26,1030.26 1621.49,1030.89 1621.72,1031.31 1621.94,1031.8 1622.17,1032.03 1622.4,1032.25 1622.63,1032.29 1622.85,1032.41 1623.08,1033.17 1623.31,1032.43 1623.54,1032.82 1623.77,1032.3 1623.99,1032.01 1624.22,1032.35 1624.45,1031.92 1624.68,1032.39 1624.9,1033.06 1625.13,1032.97 1625.36,1033.26 1625.59,1034.2 1625.82,1034.82 1626.04,1035.07 1626.27,1035.17 1626.5,1036.03 1626.73,1036.12 1626.95,1036.14 1627.18,1036.62 1627.41,1035.96 1627.64,1036.52 1627.87,1035.94 1628.09,1035.03 1628.32,1034.85 1628.55,1035.1 1628.78,1034.83 1629,1034.77 1629.23,1034.84 1629.46,1033.5 1629.69,1033.82 1629.92,1033.58 1630.14,1033.37 1630.37,1032.4 1630.6,1032.61 1630.83,1032.82 1631.05,1033.15 1631.28,1033.5 1631.51,1034.22 1631.74,1034.13 1631.97,1033.95 1632.19,1033.55 1632.42,1033.75 1632.65,1033.2 1632.88,1033.64 1633.1,1033.84 1633.33,1034.44 1633.56,1033.78 1633.79,1033.98 1634.02,1033.77 1634.24,1034.1 1634.47,1033.79 1634.7,1034.38 1634.93,1034.99 1635.15,1034.83 1635.38,1034.77 1635.61,1034.65 1635.84,1034.39 1636.07,1033.84 1636.29,1034.66 1636.52,1034.26 1636.75,1033.7 1636.98,1033.99 1637.2,1033.58 1637.43,1034.36 1637.66,1034.56 1637.89,1034.25 1638.12,1034.62 1638.34,1034.63 1638.57,1034.76 1638.8,1034.56 1639.03,1034.69 1639.25,1034.04 1639.48,1033.57 1639.71,1034.11 1639.94,1034.51 1640.17,1034.83 1640.39,1034.76 1640.62,1035.14 1640.85,1035.1 1641.08,1034.75 1641.3,1035.15 1641.53,1035.66 1641.76,1035.75 1641.99,1035.94 1642.22,1036.42 1642.44,1036.69 1642.67,1036.5 1642.9,1036.49 1643.13,1036.97 1643.35,1036.52 1643.58,1036.58 1643.81,1036.64 1644.04,1036.8 1644.27,1037.08 1644.49,1037.04 1644.72,1036.55 1644.95,1037.07 1645.18,1036.56 1645.4,1036.34 1645.63,1035.03 1645.86,1035.12 1646.09,1035.24 1646.32,1035.85 1646.54,1035.8 1646.77,1036.39 1647,1037.07 1647.23,1037.69 1647.45,1038.07 1647.68,1037.8 1647.91,1038 1648.14,1038.13 1648.37,1037.91 1648.59,1038.11 1648.82,1038.37 1649.05,1038.63 1649.28,1038.55 1649.5,1039.21 1649.73,1039.29 1649.96,1039.89 1650.19,1040.44 1650.42,1040.37 1650.64,1040.31 1650.87,1040.04 1651.1,1040.88 1651.33,1040.84 1651.55,1041.27 1651.78,1040.5 1652.01,1040.53 1652.24,1040.76 1652.47,1041.1 1652.69,1040.63 1652.92,1041.22 1653.15,1040.96 1653.38,1041.68 1653.6,1041.03 1653.83,1040.81 1654.06,1040.25 1654.29,1040.71 1654.52,1041.06 1654.74,1041.17 1654.97,1040.9 1655.2,1041.16 1655.43,1041.89 1655.65,1042.08 1655.88,1042.19 1656.11,1042.91 1656.34,1042.54 1656.57,1042.3 1656.79,1042.25 1657.02,1042.62 1657.25,1042.56 1657.48,1042.81 1657.7,1043.4 1657.93,1043.3 1658.16,1043.29 1658.39,1043.93 1658.61,1043.79 1658.84,1044.25 1659.07,1044.99 1659.3,1045.58 1659.53,1046.09 1659.75,1046.31 1659.98,1046.58 1660.21,1046.42 1660.44,1046.32 1660.66,1045.91 1660.89,1046.13 1661.12,1045.82 1661.35,1046.37 1661.58,1045.82 1661.8,1046.49 1662.03,1046.85 1662.26,1047.3 1662.49,1047.11 1662.71,1047.84 1662.94,1047.31 1663.17,1047.73 1663.4,1047.43 1663.63,1047.55 1663.85,1048.03 1664.08,1048.14 1664.31,1047.77 1664.54,1048.07 1664.76,1047.44 1664.99,1047.58 1665.22,1047.55 1665.45,1046.98 1665.68,1047.66 1665.9,1048.42 1666.13,1048.24 1666.36,1047.76 1666.59,1048.73 1666.81,1048.97 1667.04,1048.14 1667.27,1049.04 1667.5,1049.55 1667.73,1049.38 1667.95,1050.2 1668.18,1050.24 1668.41,1050.03 1668.64,1049.61 1668.86,1048.87 1669.09,1049.15 1669.32,1048.93 1669.55,1049.39 1669.78,1049.29 1670,1049.59 1670.23,1049.27 1670.46,1049.97 1670.69,1049.37 1670.91,1049.52 1671.14,1048.78 1671.37,1049.01 1671.6,1048.96 1671.83,1049.28 1672.05,1049.83 1672.28,1049.37 1672.51,1049.4 1672.74,1048.13 1672.96,1048.19 1673.19,1048.26 1673.42,1048.91 1673.65,1048.62 1673.88,1048.87 1674.1,1048.81 1674.33,1048.82 1674.56,1049.23 1674.79,1049.84 1675.01,1049.99 1675.24,1049.88 1675.47,1049.49 1675.7,1049.59 1675.93,1049.39 1676.15,1049.39 1676.38,1048.77 1676.61,1048.71 1676.84,1048.79 1677.06,1048.96 1677.29,1048.88 1677.52,1049.56 1677.75,1049.61 1677.98,1049 1678.2,1049.1 1678.43,1049.46 1678.66,1048.89 1678.89,1049.04 1679.11,1048.89 1679.34,1049.15 1679.57,1048.54 1679.8,1048.37 1680.03,1048.13 1680.25,1048.26 1680.48,1047.9 1680.71,1047.4 1680.94,1046.55 1681.16,1046.49 1681.39,1046.33 1681.62,1046.38 1681.85,1045.93 1682.08,1045.84 1682.3,1045.57 1682.53,1046.48 1682.76,1046.59 1682.99,1046.05 1683.21,1045.9 1683.44,1045.32 1683.67,1044.3 1683.9,1045.02 1684.13,1044.42 1684.35,1044.28 1684.58,1044.24 1684.81,1044.52 1685.04,1043.64 1685.26,1043.78 1685.49,1043.76 1685.72,1043.43 1685.95,1043.58 1686.18,1043.24 1686.4,1043.13 1686.63,1043.6 1686.86,1043.91 1687.09,1043.84 1687.31,1042.99 1687.54,1042.83 1687.77,1042.54 1688,1042.85 1688.23,1043.23 1688.45,1043.3 1688.68,1043.36 1688.91,1042.93 1689.14,1042.87 1689.36,1043.33 1689.59,1043.56 1689.82,1044.13 1690.05,1044.25 1690.28,1044.71 1690.5,1045 1690.73,1044.65 1690.96,1045.25 1691.19,1045.26 1691.41,1044.83 1691.64,1045.26 1691.87,1045.2 1692.1,1045.16 1692.33,1044.9 1692.55,1044.53 1692.78,1044.54 1693.01,1044.89 1693.24,1045.31 1693.46,1045.02 1693.69,1045.34 1693.92,1045.2 1694.15,1045.68 1694.37,1045.71 1694.6,1046.38 1694.83,1046.76 1695.06,1046.97 1695.29,1046.78 1695.51,1046.82 1695.74,1047.51 1695.97,1048.12 1696.2,1047.44 1696.42,1047.69 1696.65,1047.72 1696.88,1046.91 1697.11,1046.43 1697.34,1046.56 1697.56,1045.96 1697.79,1045.2 1698.02,1045.23 1698.25,1045.42 1698.47,1045.59 1698.7,1045.83 1698.93,1045.51 1699.16,1045.31 1699.39,1045.28 1699.61,1044.36 1699.84,1044.56 1700.07,1044.79 1700.3,1044.72 1700.52,1044.09 1700.75,1044.37 1700.98,1044.24 1701.21,1044.58 1701.44,1044.51 1701.66,1043.34 1701.89,1043.09 1702.12,1042.23 1702.35,1042.77 1702.57,1043.09 1702.8,1042.97 1703.03,1043.16 1703.26,1043.04 1703.49,1043.01 1703.71,1042.82 1703.94,1043.29 1704.17,1043.94 1704.4,1044.12 1704.62,1043.96 1704.85,1043.24 1705.08,1043.08 1705.31,1043.35 1705.54,1043.86 1705.76,1043.83 1705.99,1042.98 1706.22,1043.7 1706.45,1043.35 1706.67,1043.49 1706.9,1043.12 1707.13,1043.13 1707.36,1043.1 1707.59,1043.11 1707.81,1043.79 1708.04,1044.03 1708.27,1044.84 1708.5,1044.13 1708.72,1044.27 1708.95,1044.69 1709.18,1044.89 1709.41,1044.78 1709.64,1044.66 1709.86,1044.91 1710.09,1044.86 1710.32,1044.85 1710.55,1045.36 1710.77,1045.62 1711,1044.65 1711.23,1044.62 1711.46,1044.06 1711.69,1043.73 1711.91,1044.35 1712.14,1044.95 1712.37,1045 1712.6,1045.45 1712.82,1044.84 1713.05,1045.24 1713.28,1045.73 1713.51,1045.98 1713.74,1046.07 1713.96,1045.82 1714.19,1044.66 1714.42,1044.64 1714.65,1045 1714.87,1044.6 1715.1,1044.57 1715.33,1044.69 1715.56,1044.78 1715.79,1044.78 1716.01,1045.04 1716.24,1044.95 1716.47,1045.26 1716.7,1045.65 1716.92,1045.32 1717.15,1045.55 1717.38,1045.86 1717.61,1045.4 1717.84,1044.87 1718.06,1044.31 1718.29,1044.48 1718.52,1044.14 1718.75,1044.26 1718.97,1044.35 1719.2,1044.07 1719.43,1043.91 1719.66,1043.77 1719.89,1043.74 1720.11,1043.76 1720.34,1044.56 1720.57,1044.8 1720.8,1044.85 1721.02,1045.18 1721.25,1044.71 1721.48,1044.47 1721.71,1044.12 1721.94,1044.67 1722.16,1045.17 1722.39,1044.97 1722.62,1045.18 1722.85,1045.35 1723.07,1044.83 1723.3,1044.68 1723.53,1044.21 1723.76,1044.92 1723.99,1044.63 1724.21,1044.84 1724.44,1044.93 1724.67,1045.27 1724.9,1045.52 1725.12,1045.36 1725.35,1045.68 1725.58,1046.09 1725.81,1045.97 1726.04,1046.18 1726.26,1046.33 1726.49,1046.1 1726.72,1045.68 1726.95,1045.2 1727.17,1044.54 1727.4,1044.49 1727.63,1044.29 1727.86,1044.3 1728.09,1044.72 1728.31,1045.23 1728.54,1045.41 1728.77,1045.68 1729,1045.93 1729.22,1045.04 1729.45,1044.8 1729.68,1045.28 1729.91,1044.99 1730.14,1045.51 1730.36,1045.12 1730.59,1045.18 1730.82,1045.3 1731.05,1044.95 1731.27,1044.89 1731.5,1044.8 1731.73,1044.78 1731.96,1044.78 1732.18,1045.02 1732.41,1045.29 1732.64,1045.49 1732.87,1045.35 1733.1,1044.93 1733.32,1045.48 1733.55,1045.23 1733.78,1044.84 1734.01,1045.19 1734.23,1045.15 1734.46,1044.98 1734.69,1045.19 1734.92,1045.21 1735.15,1044.36 1735.37,1044.88 1735.6,1045.09 1735.83,1044.56 1736.06,1044.91 1736.28,1045.07 1736.51,1044.72 1736.74,1044.05 1736.97,1043.34 1737.2,1043.6 1737.42,1043.22 1737.65,1043.32 1737.88,1043.34 1738.11,1043.73 1738.33,1043.43 1738.56,1044.09 1738.79,1043.69 1739.02,1042.9 1739.25,1043.03 1739.47,1043.15 1739.7,1043.27 1739.93,1043.7 1740.16,1043.54 1740.38,1044.21 1740.61,1044.55 1740.84,1044.96 1741.07,1045.32 1741.3,1045.36 1741.52,1045.29 1741.75,1044.32 1741.98,1044.87 1742.21,1044.82 1742.43,1044.98 1742.66,1045.1 1742.89,1045.12 1743.12,1045.5 1743.35,1044.41 1743.57,1044.61 1743.8,1044.86 1744.03,1045.39 1744.26,1045.52 1744.48,1045.57 1744.71,1045.49 1744.94,1045.61 1745.17,1046.3 1745.4,1046.76 1745.62,1047.27 1745.85,1047.94 1746.08,1047.4 1746.31,1047.91 1746.53,1048.36 1746.76,1048.1 1746.99,1048.17 1747.22,1047.79 1747.45,1047.55 1747.67,1047.2 1747.9,1046.9 1748.13,1046.25 1748.36,1046.47 1748.58,1046.9 1748.81,1047.48 1749.04,1047.23 1749.27,1046.91 1749.5,1046.8 1749.72,1045.63 1749.95,1045.81 1750.18,1046.05 1750.41,1045.87 1750.63,1046.39 1750.86,1046.62 1751.09,1047.01 1751.32,1047.14 1751.55,1046.8 1751.77,1047.03 1752,1046.94 1752.23,1046.54 1752.46,1046.45 1752.68,1046.24 1752.91,1046.62 1753.14,1046.83 1753.37,1047.23 1753.6,1046.66 1753.82,1046.09 1754.05,1046.6 1754.28,1045.77 1754.51,1046.2 1754.73,1046.38 1754.96,1047.07 1755.19,1047.04 1755.42,1046.95 1755.65,1046.83 1755.87,1047.23 1756.1,1047.51 1756.33,1046.72 1756.56,1046.76 1756.78,1046.9 1757.01,1047.22 1757.24,1047.8 1757.47,1047.79 1757.7,1047.23 1757.92,1047.19 1758.15,1047.14 1758.38,1046.54 1758.61,1046.83 1758.83,1046.35 1759.06,1046.37 1759.29,1047.05 1759.52,1046.81 1759.75,1047.04 1759.97,1046.49 1760.2,1047.24 1760.43,1046.7 1760.66,1046.9 1760.88,1046.66 1761.11,1046.2 1761.34,1045.98 1761.57,1046.31 1761.8,1047.03 1762.02,1046.79 1762.25,1046.76 1762.48,1046.9 1762.71,1047.25 1762.93,1047.39 1763.16,1047.64 1763.39,1047.85 1763.62,1047.76 1763.85,1047.33 1764.07,1047.73 1764.3,1047.83 1764.53,1047.52 1764.76,1047.68 1764.98,1047.63 1765.21,1047.52 1765.44,1047.63 1765.67,1047.77 1765.9,1048.12 1766.12,1048.46 1766.35,1048.91 1766.58,1048.86 1766.81,1049.44 1767.03,1049.5 1767.26,1049.15 1767.49,1049.51 1767.72,1048.4 1767.94,1048.43 1768.17,1048.82 1768.4,1048.53 1768.63,1048.45 1768.86,1048.57 1769.08,1048.17 1769.31,1048.44 1769.54,1048.84 1769.77,1048.61 1769.99,1048.44 1770.22,1048.85 1770.45,1049.07 1770.68,1049.08 1770.91,1048.86 1771.13,1048.65 1771.36,1048.74 1771.59,1048.56 1771.82,1048.41 1772.04,1048.31 1772.27,1048.24 1772.5,1048.08 1772.73,1047.88 1772.96,1047.84 1773.18,1047.44 1773.41,1046.98 1773.64,1046.93 1773.87,1047.17 1774.09,1046.8 1774.32,1046.48 1774.55,1046.56 1774.78,1046.68 1775.01,1047.2 1775.23,1047.47 1775.46,1047.45 1775.69,1047.54 1775.92,1047.54 1776.14,1047.54 1776.37,1047.36 1776.6,1047.53 1776.83,1047.54 1777.06,1046.73 1777.28,1046.64 1777.51,1046.78 1777.74,1047.13 1777.97,1047.49 1778.19,1047.78 1778.42,1047.92 1778.65,1047.4 1778.88,1047.24 1779.11,1046.7 1779.33,1046.65 1779.56,1046.9 1779.79,1046.93 1780.02,1046.9 1780.24,1047.36 1780.47,1047.24 1780.7,1047.34 1780.93,1048.34 1781.16,1049.04 1781.38,1048.73 1781.61,1048.62 1781.84,1049.14 1782.07,1048.73 1782.29,1049.09 1782.52,1048.74 1782.75,1048.91 1782.98,1048.71 1783.21,1049.04 1783.43,1049.7 1783.66,1049.07 1783.89,1049.2 1784.12,1049.07 1784.34,1048.47 1784.57,1048.23 1784.8,1048.45 1785.03,1048.54 1785.26,1048.47 1785.48,1048.7 1785.71,1048.43 1785.94,1047.82 1786.17,1048.33 1786.39,1048.03 1786.62,1048.88 1786.85,1049.07 1787.08,1049.88 1787.31,1049.35 1787.53,1049.06 1787.76,1048.64 1787.99,1049.05 1788.22,1048.71 1788.44,1048.84 1788.67,1049.27 1788.9,1049.45 1789.13,1049.82 1789.36,1049.84 1789.58,1048.53 1789.81,1048.66 1790.04,1049.22 1790.27,1048.14 1790.49,1048.6 1790.72,1048.83 1790.95,1048.93 1791.18,1049.32 1791.41,1049 1791.63,1049.69 1791.86,1049.35 1792.09,1049.11 1792.32,1049.46 1792.54,1049.13 1792.77,1048.86 1793,1049.08 1793.23,1049.04 1793.46,1048.93 1793.68,1048.98 1793.91,1049.12 1794.14,1048.82 1794.37,1048.29 1794.59,1048.46 1794.82,1048.57 1795.05,1048.55 1795.28,1049.02 1795.51,1048.83 1795.73,1048.5 1795.96,1048.39 1796.19,1048.46 1796.42,1048.77 1796.64,1048.24 1796.87,1047.95 1797.1,1047.85 1797.33,1047.88 1797.56,1048.05 1797.78,1047.97 1798.01,1047.84 1798.24,1047.99 1798.47,1048.14 1798.69,1047.73 1798.92,1047.93 1799.15,1047.66 1799.38,1048.24 1799.61,1048.11 1799.83,1048.26 1800.06,1048.25 1800.29,1048.23 1800.52,1047.85 1800.74,1048.51 1800.97,1048.86 1801.2,1049.23 1801.43,1049.17 1801.66,1049.32 1801.88,1049.23 1802.11,1049.37 1802.34,1049.36 1802.57,1049.32 1802.79,1049.46 1803.02,1050.11 1803.25,1049.56 1803.48,1049.91 1803.7,1049.55 1803.93,1049.64 1804.16,1049.99 1804.39,1050.19 1804.62,1050.33 1804.84,1050.05 1805.07,1049.94 1805.3,1049.86 1805.53,1049.73 1805.75,1050.18 1805.98,1050.96 1806.21,1051.14 1806.44,1051.12 1806.67,1050.79 1806.89,1050.89 1807.12,1050.08 1807.35,1049.67 1807.58,1050.51 1807.8,1051.06 1808.03,1050.94 1808.26,1050.65 1808.49,1050.51 1808.72,1051.06 1808.94,1051.53 1809.17,1051.44 1809.4,1051.08 1809.63,1050.12 1809.85,1050.04 1810.08,1050.29 1810.31,1049.78 1810.54,1050.53 1810.77,1050.57 1810.99,1050.58 1811.22,1051.05 1811.45,1051.08 1811.68,1051.1 1811.9,1050.85 1812.13,1050.84 1812.36,1051.01 1812.59,1050.42 1812.82,1050.71 1813.04,1050.86 1813.27,1050.83 1813.5,1050.97 1813.73,1051.07 1813.95,1051.49 1814.18,1051.7 1814.41,1052.4 1814.64,1052.27 1814.87,1052.38 1815.09,1052.84 1815.32,1053.73 1815.55,1054.43 1815.78,1054.7 1816,1054.17 1816.23,1054.42 1816.46,1054.86 1816.69,1054.11 1816.92,1054.02 1817.14,1053.96 1817.37,1054.39 1817.6,1054.48 1817.83,1055.21 1818.05,1054.99 1818.28,1055.76 1818.51,1055.57 1818.74,1054.76 1818.97,1054.08 1819.19,1054.21 1819.42,1054.4 1819.65,1054.57 1819.88,1054.04 1820.1,1053.79 1820.33,1053.69 1820.56,1053.62 1820.79,1053.56 1821.02,1053.25 1821.24,1052.52 1821.47,1052.51 1821.7,1052.72 1821.93,1052.96 1822.15,1052.82 1822.38,1052.63 1822.61,1052.42 1822.84,1052.74 1823.07,1053.1 1823.29,1052.92 1823.52,1053.5 1823.75,1053.92 1823.98,1052.81 1824.2,1052.89 1824.43,1052.34 1824.66,1052.37 1824.89,1052.24 1825.12,1052.63 1825.34,1052.25 1825.57,1052.48 1825.8,1052.92 1826.03,1052.22 1826.25,1052.38 1826.48,1052.7 1826.71,1053.25 1826.94,1053.6 1827.17,1053.71 1827.39,1053.08 1827.62,1053.42 1827.85,1053.45 1828.08,1053.33 1828.3,1053.08 1828.53,1053.2 1828.76,1052.96 1828.99,1053.22 1829.22,1053.34 1829.44,1053.39 1829.67,1053.08 1829.9,1053.05 1830.13,1053 1830.35,1052.93 1830.58,1052.13 1830.81,1052.17 1831.04,1052.03 1831.27,1052.21 1831.49,1052.41 1831.72,1052.34 1831.95,1052.45 1832.18,1051.42 1832.4,1051.4 1832.63,1051.13 1832.86,1051.08 1833.09,1051.06 1833.32,1050.91 1833.54,1050.21 1833.77,1050.3 1834,1050.54 1834.23,1050.56 1834.45,1050.24 1834.68,1050.73 1834.91,1051.08 1835.14,1051.59 1835.37,1051.85 1835.59,1052.57 1835.82,1053.11 1836.05,1053.17 1836.28,1052.59 1836.5,1052.71 1836.73,1052.86 1836.96,1053.09 1837.19,1053.14 1837.42,1053.43 1837.64,1054.07 1837.87,1054.67 1838.1,1054.59 1838.33,1054.53 1838.55,1054.27 1838.78,1054.29 1839.01,1054.51 1839.24,1054.71 1839.46,1054.19 1839.69,1054.71 1839.92,1053.88 1840.15,1053.71 1840.38,1054.07 1840.6,1054.58 1840.83,1054.95 1841.06,1055.4 1841.29,1054.93 1841.51,1054.56 1841.74,1055.11 1841.97,1055.34 1842.2,1055.19 1842.43,1054.87 1842.65,1054.57 1842.88,1054.96 1843.11,1054.85 1843.34,1054.49 1843.56,1054.56 1843.79,1054.8 1844.02,1055.29 1844.25,1055.58 1844.48,1055.85 1844.7,1055.42 1844.93,1055.44 1845.16,1055.58 1845.39,1055.9 1845.61,1056.04 1845.84,1055.9 1846.07,1055.68 1846.3,1054.74 1846.53,1054.44 1846.75,1054.8 1846.98,1054.91 1847.21,1054.95 1847.44,1055.15 1847.66,1054.93 1847.89,1054.88 1848.12,1054.8 1848.35,1054.75 1848.58,1054.14 1848.8,1054.16 1849.03,1053.87 1849.26,1053.77 1849.49,1054.05 1849.71,1053.89 1849.94,1053.65 1850.17,1054.1 1850.4,1054.35 1850.63,1054.46 1850.85,1054.08 1851.08,1053.21 1851.31,1053.25 1851.54,1054.16 1851.76,1054.43 1851.99,1054.63 1852.22,1054.88 1852.45,1054.79 1852.68,1054.81 1852.9,1054.73 1853.13,1054.21 1853.36,1054.18 1853.59,1054.66 1853.81,1054.28 1854.04,1054.45 1854.27,1054.78 1854.5,1054.71 1854.73,1054.91 1854.95,1053.95 1855.18,1054.14 1855.41,1054.62 1855.64,1054.91 1855.86,1054.59 1856.09,1054.76 1856.32,1053.98 1856.55,1054.42 1856.78,1054.38 1857,1054.69 1857.23,1054.23 1857.46,1053.66 1857.69,1053.61 1857.91,1053.5 1858.14,1053.24 1858.37,1052.57 1858.6,1052.23 1858.83,1052.26 1859.05,1052.69 1859.28,1053.18 1859.51,1053.5 1859.74,1054.16 1859.96,1054.02 1860.19,1053.88 1860.42,1054.05 1860.65,1054.12 1860.88,1053.97 1861.1,1053.85 1861.33,1053.5 1861.56,1053.82 1861.79,1053.28 1862.01,1053.65 1862.24,1054 1862.47,1054.05 1862.7,1053.74 1862.93,1053.59 1863.15,1053.56 1863.38,1053.62 1863.61,1053.6 1863.84,1053.93 1864.06,1053.7 1864.29,1053.53 1864.52,1053.88 1864.75,1053.25 1864.98,1053.22 1865.2,1053.51 1865.43,1053.21 1865.66,1053.83 1865.89,1054.06 1866.11,1054.1 1866.34,1054.64 1866.57,1054.81 1866.8,1055.63 1867.03,1055.04 1867.25,1054.87 1867.48,1055.34 1867.71,1054.72 1867.94,1055.26 1868.16,1053.93 1868.39,1053.77 1868.62,1053.88 1868.85,1054.24 1869.08,1054.53 1869.3,1054.73 1869.53,1054.95 1869.76,1054.87 1869.99,1054.8 1870.21,1054.33 1870.44,1054.99 1870.67,1055.09 1870.9,1055.07 1871.13,1054.69 1871.35,1055.14 1871.58,1055.37 1871.81,1055.43 1872.04,1054.79 1872.26,1055.45 1872.49,1055.3 1872.72,1055.47 1872.95,1055.69 1873.18,1056.03 1873.4,1056.29 1873.63,1055.52 1873.86,1055.73 1874.09,1055.91 1874.31,1055.53 1874.54,1055.92 1874.77,1055.81 1875,1055.92 1875.23,1056.33 1875.45,1055.74 1875.68,1054.93 1875.91,1054.89 1876.14,1055.21 1876.36,1055.84 1876.59,1056.38 1876.82,1056.71 1877.05,1056.9 1877.27,1057.11 1877.5,1057.43 1877.73,1057.56 1877.96,1057.95 1878.19,1058.28 1878.41,1058.16 1878.64,1057.87 1878.87,1057.79 1879.1,1058.16 1879.32,1058.43 1879.55,1058.78 1879.78,1058.21 1880.01,1058.98 1880.24,1059.37 1880.46,1058.85 1880.69,1058.85 1880.92,1058.99 1881.15,1058.51 1881.37,1058.74 1881.6,1058.47 1881.83,1058.63 1882.06,1058.17 1882.29,1057.28 1882.51,1058.11 1882.74,1057.57 1882.97,1057.53 1883.2,1056.91 1883.42,1056.52 1883.65,1057.11 1883.88,1056.88 1884.11,1056.89 1884.34,1056.92 1884.56,1056.73 1884.79,1057.34 1885.02,1056.93 1885.25,1056.71 1885.47,1056.27 1885.7,1056.23 1885.93,1056.54 1886.16,1056.82 1886.39,1056.4 1886.61,1056.32 1886.84,1056.58 1887.07,1056.47 1887.3,1056.69 1887.52,1056.39 1887.75,1056.74 1887.98,1056 1888.21,1055.74 1888.44,1055.39 1888.66,1056.41 1888.89,1055.9 1889.12,1056.5 1889.35,1056.95 1889.57,1055.9 1889.8,1055.9 1890.03,1055.58 1890.26,1055.8 1890.49,1055.86 1890.71,1056.31 1890.94,1056.3 1891.17,1056.3 1891.4,1056.68 1891.62,1056.85 1891.85,1056.77 1892.08,1057 1892.31,1057.39 1892.54,1057.82 1892.76,1058.1 1892.99,1058.3 1893.22,1057.42 1893.45,1057.51 1893.67,1057.63 1893.9,1058.07 1894.13,1058.23 1894.36,1058.05 1894.59,1057.45 1894.81,1056.91 1895.04,1056.6 1895.27,1056.82 1895.5,1057.16 1895.72,1056.69 1895.95,1056.73 1896.18,1056.86 1896.41,1057.18 1896.64,1057.37 1896.86,1057.52 1897.09,1057.56 1897.32,1057.89 1897.55,1057.73 1897.77,1057.45 1898,1057.95 1898.23,1057.7 1898.46,1057.25 1898.69,1057.64 1898.91,1057.27 1899.14,1056.75 1899.37,1056.21 1899.6,1056.14 1899.82,1055.93 1900.05,1055.65 1900.28,1055.1 1900.51,1054.65 1900.74,1054.72 1900.96,1055.06 1901.19,1055.46 1901.42,1055.38 1901.65,1055.38 1901.87,1055.78 1902.1,1056.29 1902.33,1056.68 1902.56,1057.03 1902.79,1056.7 1903.01,1056.61 1903.24,1056.4 1903.47,1055.92 1903.7,1055.7 1903.92,1055.78 1904.15,1056.09 1904.38,1056.33 1904.61,1056.44 1904.84,1056.79 1905.06,1056.71 1905.29,1056.57 1905.52,1056.02 1905.75,1055.76 1905.97,1056.06 1906.2,1055.02 1906.43,1055.37 1906.66,1055.03 1906.89,1055.26 1907.11,1055.67 1907.34,1055.3 1907.57,1055.1 1907.8,1055.07 1908.02,1055.38 1908.25,1055.54 1908.48,1055.45 1908.71,1055.79 1908.94,1055.16 1909.16,1055.41 1909.39,1055.88 1909.62,1056.04 1909.85,1055.92 1910.07,1056.51 1910.3,1057.24 1910.53,1056.28 1910.76,1056.86 1910.99,1056.94 1911.21,1057.55 1911.44,1056.81 1911.67,1056.78 1911.9,1056.76 1912.12,1056.87 1912.35,1055.95 1912.58,1056.17 1912.81,1056.44 1913.03,1056.55 1913.26,1056.25 1913.49,1056.24 1913.72,1056.62 1913.95,1056.36 1914.17,1056.6 1914.4,1056.94 1914.63,1057.17 1914.86,1057.21 1915.08,1056.87 1915.31,1056.23 1915.54,1056.23 1915.77,1055.9 1916,1056 1916.22,1056.28 1916.45,1056.72 1916.68,1056.75 1916.91,1056.31 1917.13,1056.57 1917.36,1056.83 1917.59,1056.99 1917.82,1056.59 1918.05,1056.83 1918.27,1056.93 1918.5,1056.92 1918.73,1057.1 1918.96,1057.07 1919.18,1057.28 1919.41,1057.09 1919.64,1057.04 1919.87,1057.72 1920.1,1057.33 1920.32,1057.5 1920.55,1057.47 1920.78,1057.85 1921.01,1057.35 1921.23,1057.13 1921.46,1057.96 1921.69,1058.17 1921.92,1057.85 1922.15,1057.87 1922.37,1057.68 1922.6,1058.23 1922.83,1058.26 1923.06,1059.04 1923.28,1059.12 1923.51,1059.94 1923.74,1059.84 1923.97,1059.31 1924.2,1059.12 1924.42,1058.87 1924.65,1059.03 1924.88,1058.69 1925.11,1058.99 1925.33,1059.23 1925.56,1058.81 1925.79,1058.47 1926.02,1058.47 1926.25,1058.98 1926.47,1059.54 1926.7,1058.75 1926.93,1058.32 1927.16,1058.95 1927.38,1058.71 1927.61,1058.14 1927.84,1058.73 1928.07,1058.71 1928.3,1058.28 1928.52,1058.41 1928.75,1058.46 1928.98,1058.17 1929.21,1058.69 1929.43,1058.32 1929.66,1058.62 1929.89,1058.58 1930.12,1058.79 1930.35,1058.93 1930.57,1059.29 1930.8,1058.85 1931.03,1059.05 1931.26,1058.7 1931.48,1059.19 1931.71,1059.17 1931.94,1059.24 1932.17,1059.65 1932.4,1059.78 1932.62,1059.64 1932.85,1059.96 1933.08,1060.05 1933.31,1059.68 1933.53,1059.98 1933.76,1059.87 1933.99,1060.52 1934.22,1060.67 1934.45,1060.89 1934.67,1060.73 1934.9,1060.77 1935.13,1060.75 1935.36,1060.86 1935.58,1061.18 1935.81,1060.82 1936.04,1060.6 1936.27,1060.66 1936.5,1060.52 1936.72,1060.29 1936.95,1060.57 1937.18,1061.14 1937.41,1060.9 1937.63,1060.86 1937.86,1060.52 1938.09,1060.23 1938.32,1060.17 1938.55,1059.85 1938.77,1059.6 1939,1059.39 1939.23,1059.26 1939.46,1058.74 1939.68,1058.4 1939.91,1058.54 1940.14,1058.77 1940.37,1058.91 1940.6,1058.74 1940.82,1058.97 1941.05,1059.48 1941.28,1059.32 1941.51,1058.41 1941.73,1059.09 1941.96,1058.87 1942.19,1058.94 1942.42,1059.07 1942.65,1059.23 1942.87,1059.21 1943.1,1059.41 1943.33,1059.31 1943.56,1059.79 1943.78,1059.38 1944.01,1059.63 1944.24,1060.26 1944.47,1060.74 1944.7,1060.72 1944.92,1060.6 1945.15,1060.83 1945.38,1060.87 1945.61,1061.34 1945.83,1061.51 1946.06,1061.02 1946.29,1060.81 1946.52,1061.34 1946.75,1061.56 1946.97,1060.94 1947.2,1061.36 1947.43,1061.65 1947.66,1061.7 1947.88,1061.91 1948.11,1061.88 1948.34,1061.48 1948.57,1062.06 1948.79,1062.41 1949.02,1062.14 1949.25,1062.11 1949.48,1062.4 1949.71,1062.21 1949.93,1062.25 1950.16,1062.35 1950.39,1062.39 1950.62,1061.91 1950.84,1061.76 1951.07,1061.33 1951.3,1061.34 1951.53,1062.05 1951.76,1061.92 1951.98,1061.93 1952.21,1061.82 1952.44,1062 1952.67,1061.82 1952.89,1062.21 1953.12,1062.43 1953.35,1061.55 1953.58,1061.46 1953.81,1061.84 1954.03,1061.88 1954.26,1062.3 1954.49,1062.2 1954.72,1061.62 1954.94,1061.5 1955.17,1061.86 1955.4,1061.83 1955.63,1062.11 1955.86,1062.42 1956.08,1062.05 1956.31,1062.11 1956.54,1062.14 1956.77,1062.46 1956.99,1062.36 1957.22,1062.7 1957.45,1062.93 1957.68,1063.28 1957.91,1063.67 1958.13,1063.46 1958.36,1063.31 1958.59,1063.33 1958.82,1063.3 1959.04,1063.54 1959.27,1063.26 1959.5,1063.45 1959.73,1063.63 1959.96,1063.23 1960.18,1063.46 1960.41,1063.26 1960.64,1062.73 1960.87,1062.79 1961.09,1062.21 1961.32,1061.96 1961.55,1062.63 1961.78,1063.06 1962.01,1062.7 1962.23,1062.88 1962.46,1063.57 1962.69,1063.74 1962.92,1063.48 1963.14,1063.25 1963.37,1063.13 1963.6,1063.37 1963.83,1062.92 1964.06,1062.87 1964.28,1063.28 1964.51,1063.45 1964.74,1063.68 1964.97,1063.5 1965.19,1063.27 1965.42,1063.32 1965.65,1063.44 1965.88,1063.74 1966.11,1064.02 1966.33,1063.89 1966.56,1063.87 1966.79,1064.63 1967.02,1064.7 1967.24,1065.13 1967.47,1065.19 1967.7,1065.63 1967.93,1064.87 1968.16,1065.13 1968.38,1065.41 1968.61,1065.61 1968.84,1065.61 1969.07,1065.19 1969.29,1065.2 1969.52,1065.21 1969.75,1064.81 1969.98,1064.79 1970.21,1064.6 1970.43,1064.6 1970.66,1064.76 1970.89,1064.78 1971.12,1064.88 1971.34,1065.19 1971.57,1065.32 1971.8,1065.49 1972.03,1064.99 1972.26,1064.29 1972.48,1064.2 1972.71,1064.66 1972.94,1064.85 1973.17,1065.16 1973.39,1064.68 1973.62,1064.11 1973.85,1064.11 1974.08,1063.97 1974.31,1064.05 1974.53,1064.54 1974.76,1064.64 1974.99,1065.28 1975.22,1065.21 1975.44,1064.46 1975.67,1064.2 1975.9,1063.58 1976.13,1063.68 1976.36,1063.36 1976.58,1063.42 1976.81,1062.98 1977.04,1062.7 1977.27,1062.94 1977.49,1062.78 1977.72,1063.09 1977.95,1063.43 1978.18,1063.21 1978.41,1062.89 1978.63,1063.14 1978.86,1062.82 1979.09,1062.46 1979.32,1062.39 1979.54,1062.32 1979.77,1061.57 1980,1060.75 1980.23,1061.03 1980.46,1061.15 1980.68,1060.66 1980.91,1060.46 1981.14,1060.78 1981.37,1060.48 1981.59,1060.35 1981.82,1060.31 1982.05,1060.29 1982.28,1059.82 1982.51,1060.16 1982.73,1059.81 1982.96,1059.97 1983.19,1060.59 1983.42,1060.28 1983.64,1060.87 1983.87,1060.78 1984.1,1060.59 1984.33,1060.26 1984.56,1059.98 1984.78,1060.11 1985.01,1060.18 1985.24,1060.37 1985.47,1060.49 1985.69,1060.45 1985.92,1060.06 1986.15,1059.9 1986.38,1060.14 1986.6,1059.77 1986.83,1060.3 1987.06,1059.72 1987.29,1059.43 1987.52,1059.8 1987.74,1059.39 1987.97,1059.81 1988.2,1060.04 1988.43,1059.99 1988.65,1060.15 1988.88,1059.94 1989.11,1059.91 1989.34,1059.18 1989.57,1059.37 1989.79,1059.11 1990.02,1059.51 1990.25,1059.43 1990.48,1059.45 1990.7,1059.69 1990.93,1059.91 1991.16,1060.29 1991.39,1060.07 1991.62,1060.27 1991.84,1060.04 1992.07,1060.24 1992.3,1060.06 1992.53,1060.38 1992.75,1060.95 1992.98,1060.51 1993.21,1059.87 1993.44,1060.02 1993.67,1060.05 1993.89,1060.63 1994.12,1060.39 1994.35,1059.88 1994.58,1059.71 1994.8,1060.2 1995.03,1060.5 1995.26,1060.06 1995.49,1060.52 1995.72,1059.87 1995.94,1059.88 1996.17,1060.44 1996.4,1060.12 1996.63,1060.63 1996.85,1060.52 1997.08,1060.6 1997.31,1060.92 1997.54,1060.73 1997.77,1060.14 1997.99,1060.55 1998.22,1060.68 1998.45,1061.02 1998.68,1061.2 1998.9,1060.7 1999.13,1061.05 1999.36,1060.18 1999.59,1059.74 1999.82,1060.32 2000.04,1060.36 2000.27,1060.5 2000.5,1060.51 2000.73,1059.93 2000.95,1058.98 2001.18,1059.18 2001.41,1059.32 2001.64,1059.21 2001.87,1059.03 2002.09,1059.68 2002.32,1059.8 2002.55,1059.68 2002.78,1059.69 2003,1060.19 2003.23,1060.18 2003.46,1059.96 2003.69,1060.16 2003.92,1060.14 2004.14,1059.81 2004.37,1060.26 2004.6,1060.39 2004.83,1060.69 2005.05,1060.49 2005.28,1060.27 2005.51,1059.98 2005.74,1059.37 2005.97,1059.37 2006.19,1059.99 2006.42,1060.17 2006.65,1060.21 2006.88,1059.98 2007.1,1060.5 2007.33,1060.55 2007.56,1060 2007.79,1060.51 2008.02,1060.42 2008.24,1060.76 2008.47,1060.76 2008.7,1060.72 2008.93,1059.91 2009.15,1060.19 2009.38,1060.12 2009.61,1060.42 2009.84,1060.16 2010.07,1060.52 2010.29,1059.32 2010.52,1059.6 2010.75,1059.14 2010.98,1059.22 2011.2,1059.31 2011.43,1059.4 2011.66,1059.69 2011.89,1059.4 2012.12,1058.78 2012.34,1059.06 2012.57,1059.17 2012.8,1058.9 2013.03,1059.07 2013.25,1058.88 2013.48,1058.84 2013.71,1059.11 2013.94,1059.7 2014.17,1059.52 2014.39,1059.3 2014.62,1059.92 2014.85,1059.74 2015.08,1059.68 2015.3,1059.86 2015.53,1060.22 2015.76,1060.93 2015.99,1060.99 2016.22,1061.74 2016.44,1061.32 2016.67,1061.58 2016.9,1061.77 2017.13,1061.72 2017.35,1061.77 2017.58,1061.54 2017.81,1061.18 2018.04,1061.31 2018.27,1061.18 2018.49,1061.03 2018.72,1061.17 2018.95,1061.48 2019.18,1061.25 2019.4,1061.65 2019.63,1062.19 2019.86,1062.61 2020.09,1062.87 2020.32,1063.22 2020.54,1063.47 2020.77,1063.27 2021,1063.59 2021.23,1063.32 2021.45,1063.34 2021.68,1063.08 2021.91,1063.27 2022.14,1062.72 2022.36,1062.49 2022.59,1062.19 2022.82,1062.38 2023.05,1062.3 2023.28,1062.47 2023.5,1062.17 2023.73,1062.26 2023.96,1061.79 2024.19,1061.54 2024.41,1061.68 2024.64,1061.97 2024.87,1061.36 2025.1,1061.91 2025.33,1061.79 2025.55,1061.45 2025.78,1061.5 2026.01,1061.48 2026.24,1061.55 2026.46,1061.63 2026.69,1061.59 2026.92,1061.49 2027.15,1061.95 2027.38,1062.23 2027.6,1062.27 2027.83,1061.86 2028.06,1062.12 2028.29,1061.76 2028.51,1061.26 2028.74,1060.98 2028.97,1060.63 2029.2,1060.32 2029.43,1060.58 2029.65,1060.07 2029.88,1060.63 2030.11,1060.79 2030.34,1061.1 2030.56,1060.98 2030.79,1060.94 2031.02,1061.52 2031.25,1061.44 2031.48,1061.69 2031.7,1060.85 2031.93,1061.01 2032.16,1060.72 2032.39,1060.41 2032.61,1060.41 2032.84,1060.18 2033.07,1060.32 2033.3,1060.17 2033.53,1060.74 2033.75,1060.3 2033.98,1060.4 2034.21,1060.51 2034.44,1060.63 2034.66,1061.03 2034.89,1061.21 2035.12,1060.96 2035.35,1061.36 2035.58,1060.71 2035.8,1060.78 2036.03,1061.09 2036.26,1061.06 2036.49,1061.57 2036.71,1061 2036.94,1060.56 2037.17,1060.4 2037.4,1060.48 2037.63,1060.5 2037.85,1061.11 2038.08,1060.9 2038.31,1060.46 2038.54,1060.41 2038.76,1059.74 2038.99,1058.9 2039.22,1058.66 2039.45,1058.63 2039.68,1058.76 2039.9,1058.5 2040.13,1058.47 2040.36,1059.33 2040.59,1059.68 2040.81,1059.75 2041.04,1059.85 2041.27,1059.97 2041.5,1059.7 2041.73,1059.44 2041.95,1059.23 2042.18,1058.98 2042.41,1058.82 2042.64,1059.14 2042.86,1059.49 2043.09,1058.9 2043.32,1059.04 2043.55,1059.47 2043.78,1059.83 2044,1060.02 2044.23,1060.04 2044.46,1060.38 2044.69,1060.41 2044.91,1060.02 2045.14,1060.6 2045.37,1060.99 2045.6,1060.58 2045.83,1060.94 2046.05,1060.47 2046.28,1060.51 2046.51,1060.27 2046.74,1060.87 2046.96,1060.21 2047.19,1060.13 2047.42,1060.24 2047.65,1060.57 2047.88,1060.33 2048.1,1060.4 2048.33,1060.39 2048.56,1060.48 2048.79,1060.84 2049.01,1060.99 2049.24,1060.73 2049.47,1060.76 2049.7,1060.43 2049.93,1060.66 2050.15,1060.31 2050.38,1060.28 2050.61,1059.66 2050.84,1059.59 2051.06,1060.09 2051.29,1059.79 2051.52,1059.75 2051.75,1059.65 2051.98,1059.93 2052.2,1060.31 2052.43,1059.93 2052.66,1060.04 2052.89,1060.64 2053.11,1060.14 2053.34,1060.28 2053.57,1060.18 2053.8,1060.03 2054.03,1060.22 2054.25,1060.4 2054.48,1060.92 2054.71,1060.81 2054.94,1060.83 2055.16,1060.86 2055.39,1060.96 2055.62,1061.15 2055.85,1061.02 2056.08,1060.58 2056.3,1060.72 2056.53,1060.24 2056.76,1060.32 2056.99,1060.27 2057.21,1060.25 2057.44,1060.09 2057.67,1059.7 2057.9,1060.25 2058.12,1059.96 2058.35,1059.37 2058.58,1059.73 2058.81,1059.37 2059.04,1059.6 2059.26,1059.49 2059.49,1059.44 2059.72,1059.4 2059.95,1059.71 2060.17,1059.16 2060.4,1059.55 2060.63,1059.13 2060.86,1059.34 2061.09,1058.72 2061.31,1058.96 2061.54,1059.22 2061.77,1059.32 2062,1059.77 2062.22,1059.43 2062.45,1059.51 2062.68,1059.84 2062.91,1060.12 2063.14,1059.81 2063.36,1060.12 2063.59,1060.62 2063.82,1060.32 2064.05,1060.75 2064.27,1060.76 2064.5,1060.57 2064.73,1059.72 2064.96,1059.54 2065.19,1059.86 2065.41,1060.07 2065.64,1060.25 2065.87,1059.9 2066.1,1060.03 2066.32,1059.56 2066.55,1059.09 2066.78,1058.96 2067.01,1058.74 2067.24,1058.71 2067.46,1058.72 2067.69,1058.71 2067.92,1058.6 2068.15,1058.91 2068.37,1058.75 2068.6,1058.85 2068.83,1058.28 2069.06,1058.54 2069.29,1058.86 2069.51,1059.38 2069.74,1058.81 2069.97,1059.15 2070.2,1059.22 2070.42,1059.56 2070.65,1059.77 2070.88,1059.96 2071.11,1060.41 2071.34,1060.02 2071.56,1059.77 2071.79,1059.34 2072.02,1059.2 2072.25,1059.12 2072.47,1059.46 2072.7,1059.41 2072.93,1059.36 2073.16,1059.15 2073.39,1059.5 2073.61,1059.25 2073.84,1059.35 2074.07,1059.38 2074.3,1059.33 2074.52,1059.9 2074.75,1060.02 2074.98,1060.3 2075.21,1060.43 2075.44,1060.44 2075.66,1060.89 2075.89,1061.29 2076.12,1061.28 2076.35,1060.96 2076.57,1061.34 2076.8,1061.5 2077.03,1061.68 2077.26,1061.31 2077.49,1061.13 2077.71,1060.71 2077.94,1060.82 2078.17,1060.48 2078.4,1060.49 2078.62,1059.9 2078.85,1059.86 2079.08,1059.24 2079.31,1059.07 2079.54,1059.15 2079.76,1059.37 2079.99,1059.19 2080.22,1059.01 2080.45,1059.46 2080.67,1059.71 2080.9,1059.63 2081.13,1059.61 2081.36,1059.51 2081.59,1059.06 2081.81,1058.78 2082.04,1059.13 2082.27,1058.75 2082.5,1058.84 2082.72,1058.55 2082.95,1058.53 2083.18,1058.47 2083.41,1058.48 2083.64,1057.69 2083.86,1057.43 2084.09,1057.45 2084.32,1057.35 2084.55,1057.82 2084.77,1058.07 2085,1058.26 2085.23,1058.36 2085.46,1057.54 2085.69,1057.53 2085.91,1057.49 2086.14,1057.61 2086.37,1057.52 2086.6,1057.36 2086.82,1057.61 2087.05,1058.02 2087.28,1058.22 2087.51,1058.93 2087.74,1058.92 2087.96,1058.74 2088.19,1058.39 2088.42,1058.26 2088.65,1058.52 2088.87,1059.04 2089.1,1059.3 2089.33,1059.4 2089.56,1059.63 2089.79,1059.49 2090.01,1059.71 2090.24,1060.17 2090.47,1059.7 2090.7,1059.98 2090.92,1059.97 2091.15,1059.82 2091.38,1059.69 2091.61,1058.94 2091.84,1058.88 2092.06,1059.09 2092.29,1058.39 2092.52,1058.88 2092.75,1058.91 2092.97,1058.78 2093.2,1058.76 2093.43,1059.16 2093.66,1059.38 2093.89,1058.72 2094.11,1059.16 2094.34,1059.46 2094.57,1059.45 2094.8,1059.98 2095.02,1060.37 2095.25,1060.54 2095.48,1060.52 2095.71,1061.12 2095.93,1061.35 2096.16,1061.61 2096.39,1062.13 2096.62,1061.88 2096.85,1062.31 2097.07,1062.5 2097.3,1062.56 2097.53,1062.44 2097.76,1062.33 2097.98,1062.6 2098.21,1062.7 2098.44,1063.07 2098.67,1062.92 2098.9,1063.12 2099.12,1062.93 2099.35,1062.27 2099.58,1061.81 2099.81,1061.38 2100.03,1061.63 2100.26,1061.92 2100.49,1061.68 2100.72,1061.09 2100.95,1061.09 2101.17,1060.82 2101.4,1061.05 2101.63,1061.03 2101.86,1060.58 2102.08,1060.53 2102.31,1060.63 2102.54,1060.29 2102.77,1060.19 2103,1060.1 2103.22,1060.17 2103.45,1059.71 2103.68,1059.47 2103.91,1059.35 2104.13,1059.22 2104.36,1059.66 2104.59,1059.74 2104.82,1059.69 2105.05,1059.87 2105.27,1059.15 2105.5,1059.47 2105.73,1059.03 2105.96,1058.4 2106.18,1058.99 2106.41,1059.36 2106.64,1059.26 2106.87,1058.77 2107.1,1059.21 2107.32,1058.95 2107.55,1058.81 2107.78,1059.03 2108.01,1058.49 2108.23,1058.74 2108.46,1058.61 2108.69,1058.49 2108.92,1058.51 2109.15,1058.66 2109.37,1058.94 2109.6,1059.13 2109.83,1058.94 2110.06,1058.56 2110.28,1058.82 2110.51,1058.76 2110.74,1058.8 2110.97,1059.17 2111.2,1058.98 2111.42,1058.92 2111.65,1059.34 2111.88,1058.9 2112.11,1059.01 2112.33,1058.5 2112.56,1058.83 2112.79,1059.19 2113.02,1058.71 2113.25,1058.71 2113.47,1059.09 2113.7,1059.22 2113.93,1059.32 2114.16,1060 2114.38,1059.83 2114.61,1060.18 2114.84,1060.02 2115.07,1060.11 2115.3,1059.66 2115.52,1059.52 2115.75,1059.65 2115.98,1059.96 2116.21,1059.24 2116.43,1059.41 2116.66,1059.46 2116.89,1059.55 2117.12,1059.72 2117.35,1060.12 2117.57,1060.09 2117.8,1060.35 2118.03,1060.39 2118.26,1060.51 2118.48,1060.54 2118.71,1060.73 2118.94,1060.59 2119.17,1060.28 2119.4,1060.28 2119.62,1060.47 2119.85,1060.64 2120.08,1060.64 2120.31,1060.97 2120.53,1060.62 2120.76,1060.96 2120.99,1061.2 2121.22,1061.66 2121.45,1061.26 2121.67,1061.52 2121.9,1061.32 2122.13,1061.62 2122.36,1062.02 2122.58,1061.9 2122.81,1061.57 2123.04,1061.11 2123.27,1061.14 2123.5,1060.76 2123.72,1061.18 2123.95,1061.87 2124.18,1062.24 2124.41,1062.06 2124.63,1061.36 2124.86,1061.24 2125.09,1060.8 2125.32,1060.52 2125.55,1060.57 2125.77,1060.18 2126,1060.25 2126.23,1060.4 2126.46,1060.43 2126.68,1060.66 2126.91,1060.88 2127.14,1060.96 2127.37,1061.11 2127.6,1061.31 2127.82,1060.99 2128.05,1060.78 2128.28,1060.53 2128.51,1060.74 2128.73,1061.01 2128.96,1060.63 2129.19,1060.68 2129.42,1060.52 2129.65,1060.93 2129.87,1061.25 2130.1,1061.37 2130.33,1060.78 2130.56,1061.07 2130.78,1061.17 2131.01,1061.24 2131.24,1061.65 2131.47,1061.76 2131.69,1061.86 2131.92,1061.62 2132.15,1061.75 2132.38,1062.14 2132.61,1061.6 2132.83,1062.32 2133.06,1062.46 2133.29,1061.87 2133.52,1061.8 2133.74,1061.97 2133.97,1061.41 2134.2,1061.41 2134.43,1061.42 2134.66,1061.96 2134.88,1061.96 2135.11,1061.8 2135.34,1061.36 2135.57,1061.29 2135.79,1061.66 2136.02,1061.26 2136.25,1061.66 2136.48,1061.72 2136.71,1061.76 2136.93,1061.92 2137.16,1061.9 2137.39,1061.86 2137.62,1061.83 2137.84,1062.5 2138.07,1062.68 2138.3,1062.7 2138.53,1062.83 2138.76,1062.69 2138.98,1062.69 2139.21,1062.42 2139.44,1062.23 2139.67,1062.13 2139.89,1061.8 2140.12,1061.61 2140.35,1061.9 2140.58,1061.84 2140.81,1061.66 2141.03,1061.73 2141.26,1061.85 2141.49,1062 2141.72,1062.4 2141.94,1062.69 2142.17,1062.08 2142.4,1062.24 2142.63,1062.11 2142.86,1062.42 2143.08,1062.05 2143.31,1062.09 2143.54,1061.51 2143.77,1061.33 2143.99,1061.28 2144.22,1061.29 2144.45,1061.8 2144.68,1061.76 2144.91,1061.31 2145.13,1061.68 2145.36,1061.5 2145.59,1061.63 2145.82,1061.66 2146.04,1062.2 2146.27,1062.22 2146.5,1062.02 2146.73,1062.26 2146.96,1062.02 2147.18,1062.09 2147.41,1062.12 2147.64,1062.49 2147.87,1062.69 2148.09,1062.49 2148.32,1061.58 2148.55,1062.06 2148.78,1062.41 2149.01,1062.8 2149.23,1062.13 2149.46,1062.44 2149.69,1062.58 2149.92,1062.74 2150.14,1062.25 2150.37,1061.48 2150.6,1061.7 2150.83,1061.63 2151.06,1061.98 2151.28,1062.33 2151.51,1062.44 2151.74,1062.68 2151.97,1062.71 2152.19,1062.74 2152.42,1062.85 2152.65,1062.66 2152.88,1062.63 2153.11,1062.75 2153.33,1062.89 2153.56,1062.33 2153.79,1062.54 2154.02,1062.91 2154.24,1063.14 2154.47,1063.52 2154.7,1063.01 2154.93,1062.4 2155.16,1062.42 2155.38,1061.99 2155.61,1062.15 2155.84,1062.18 2156.07,1062.31 2156.29,1062.78 2156.52,1062.82 2156.75,1062.59 2156.98,1062.64 2157.21,1063.07 2157.43,1063.36 2157.66,1063 2157.89,1061.86 2158.12,1061.82 2158.34,1061.69 2158.57,1061.58 2158.8,1061.09 2159.03,1060.66 2159.26,1060.05 2159.48,1059.79 2159.71,1060.21 2159.94,1060.25 2160.17,1059.85 2160.39,1060.16 2160.62,1059.09 2160.85,1058.91 2161.08,1058.22 2161.31,1058.03 2161.53,1058.06 2161.76,1057.79 2161.99,1057.79 2162.22,1057.84 2162.44,1057.71 2162.67,1057.65 2162.9,1057.61 2163.13,1057.05 2163.36,1056.59 2163.58,1056.52 2163.81,1056.72 2164.04,1056.59 2164.27,1056.96 2164.49,1057.06 2164.72,1057.11 2164.95,1056.43 2165.18,1056.17 2165.41,1055.65 2165.63,1055.72 2165.86,1055.61 2166.09,1055.69 2166.32,1055.94 2166.54,1056.34 2166.77,1056.68 2167,1056.41 2167.23,1056.72 2167.45,1056.5 2167.68,1056.93 2167.91,1057.07 2168.14,1057.04 2168.37,1056.97 2168.59,1056.83 2168.82,1056.18 2169.05,1055.62 2169.28,1056.16 2169.5,1056.28 2169.73,1056.23 2169.96,1055.93 2170.19,1055.8 2170.42,1055.74 2170.64,1056.07 2170.87,1056.14 2171.1,1056.02 2171.33,1056.07 2171.55,1056.13 2171.78,1056.34 2172.01,1056.05 2172.24,1055.89 2172.47,1056.18 2172.69,1056.14 2172.92,1055.98 2173.15,1055.74 2173.38,1055.5 2173.6,1055.21 2173.83,1055.2 2174.06,1055.59 2174.29,1055.83 2174.52,1055.61 2174.74,1055.87 2174.97,1056.33 2175.2,1056.1 2175.43,1056.74 2175.65,1056.46 2175.88,1056.44 2176.11,1056.84 2176.34,1057.17 2176.57,1057.05 2176.79,1057.72 2177.02,1057.54 2177.25,1057.64 2177.48,1058.25 2177.7,1058.29 2177.93,1058.44 2178.16,1058.41 2178.39,1058.07 2178.62,1058.23 2178.84,1058.23 2179.07,1057.98 2179.3,1058.43 2179.53,1058.36 2179.75,1058.21 2179.98,1057.77 2180.21,1057.38 2180.44,1057.65 2180.67,1057.94 2180.89,1058.47 2181.12,1057.72 2181.35,1057.74 2181.58,1057.75 2181.8,1058.09 2182.03,1057.65 2182.26,1057.67 2182.49,1056.88 2182.72,1056.79 2182.94,1056.64 2183.17,1056.47 2183.4,1056.61 2183.63,1056.88 2183.85,1057.14 2184.08,1057.29 2184.31,1057.63 2184.54,1057.83 2184.77,1057.97 2184.99,1058.06 2185.22,1058.03 2185.45,1058.33 2185.68,1058.99 2185.9,1059.14 2186.13,1058.71 2186.36,1058.58 2186.59,1058.65 2186.82,1058.75 2187.04,1059.01 2187.27,1059.25 2187.5,1059.64 2187.73,1059.72 2187.95,1059.78 2188.18,1059.72 2188.41,1059.14 2188.64,1058.76 2188.87,1058.53 2189.09,1058.69 2189.32,1058.63 2189.55,1058.79 2189.78,1058.61 2190,1058.75 2190.23,1058.57 2190.46,1058.57 2190.69,1058.51 2190.92,1058.56 2191.14,1058.75 2191.37,1059.06 2191.6,1059 2191.83,1059.52 2192.05,1059.36 2192.28,1059.06 2192.51,1059.05 2192.74,1059.16 2192.97,1058.88 2193.19,1059.36 2193.42,1058.63 2193.65,1059.13 2193.88,1059.41 2194.1,1059.15 2194.33,1059.39 2194.56,1059.47 2194.79,1058.94 2195.02,1059.02 2195.24,1058.61 2195.47,1058.68 2195.7,1058.64 2195.93,1058.76 2196.15,1058.27 2196.38,1058.31 2196.61,1058.32 2196.84,1058.64 2197.07,1057.87 2197.29,1058.32 2197.52,1058.01 2197.75,1058.14 2197.98,1058.05 2198.2,1058.27 2198.43,1058.55 2198.66,1058.72 2198.89,1058.75 2199.12,1058.61 2199.34,1058.41 2199.57,1058.76 2199.8,1058.57 2200.03,1058.54 2200.25,1058.42 2200.48,1058.19 2200.71,1058.22 2200.94,1058.55 2201.17,1059.13 2201.39,1059 2201.62,1059.03 2201.85,1059.41 2202.08,1059.65 2202.3,1059.91 2202.53,1059.67 2202.76,1059.52 2202.99,1059.68 2203.21,1059.98 2203.44,1060.57 2203.67,1060.44 2203.9,1059.96 2204.13,1060.36 2204.35,1059.67 2204.58,1060.25 2204.81,1060.58 2205.04,1060.74 2205.26,1060.83 2205.49,1060.73 2205.72,1060.79 2205.95,1060.35 2206.18,1061.03 2206.4,1061.11 2206.63,1060.75 2206.86,1060.65 2207.09,1060.49 2207.31,1060.61 2207.54,1060.8 2207.77,1060.24 2208,1060.72 2208.23,1061.12 2208.45,1061.17 2208.68,1061.12 2208.91,1061.3 2209.14,1060.89 2209.36,1060.63 2209.59,1060.83 2209.82,1060.76 2210.05,1060.55 2210.28,1060.07 2210.5,1060.28 2210.73,1060.45 2210.96,1060.04 2211.19,1059.92 2211.41,1060.15 2211.64,1060.04 2211.87,1059.8 2212.1,1059.48 2212.33,1059.15 2212.55,1059.26 2212.78,1059.51 2213.01,1059.71 2213.24,1059.95 2213.46,1059.84 2213.69,1058.98 2213.92,1058.2 2214.15,1058.51 2214.38,1058.6 2214.6,1058.46 2214.83,1058.84 2215.06,1058.36 2215.29,1058.61 2215.51,1058.59 2215.74,1058.54 2215.97,1058.62 2216.2,1058.64 2216.43,1058.13 2216.65,1058.21 2216.88,1057.94 2217.11,1057.92 2217.34,1058.36 2217.56,1058.41 2217.79,1058.31 2218.02,1058.12 2218.25,1058.46 2218.48,1058.25 2218.7,1058.63 2218.93,1058.58 2219.16,1058.41 2219.39,1058.42 2219.61,1058.48 2219.84,1059.03 2220.07,1058.68 2220.3,1058.64 2220.53,1058.87 2220.75,1058.23 2220.98,1058.5 2221.21,1058.07 2221.44,1057.73 2221.66,1057.74 2221.89,1057.63 2222.12,1057.59 2222.35,1057.76 2222.58,1057.29 2222.8,1057.12 2223.03,1057.31 2223.26,1057.27 2223.49,1057.53 2223.71,1057.69 2223.94,1057.63 2224.17,1057.77 2224.4,1057.91 2224.63,1057.94 2224.85,1058.05 2225.08,1058.18 2225.31,1057.59 2225.54,1057.16 2225.76,1057.12 2225.99,1056.82 2226.22,1056.99 2226.45,1056.76 2226.68,1056.91 2226.9,1056.9 2227.13,1057.12 2227.36,1056.64 2227.59,1056.73 2227.81,1056.48 2228.04,1056.17 2228.27,1056.13 2228.5,1055.75 2228.73,1055.74 2228.95,1055.95 2229.18,1056.39 2229.41,1056.72 2229.64,1056.81 2229.86,1057.06 2230.09,1056.93 2230.32,1057.3 2230.55,1057.31 2230.78,1057.19 2231,1057.4 2231.23,1057.47 2231.46,1057.65 2231.69,1057.72 2231.91,1057.73 2232.14,1058.19 2232.37,1058.61 2232.6,1057.89 2232.83,1057.9 2233.05,1057.87 2233.28,1057.66 2233.51,1057.65 2233.74,1057.71 2233.96,1057.51 2234.19,1057.75 2234.42,1057.88 2234.65,1057.9 2234.88,1057.87 2235.1,1057.62 2235.33,1057.71 2235.56,1058.08 2235.79,1058.19 2236.01,1058.79 2236.24,1058.79 2236.47,1058.54 2236.7,1058.24 2236.93,1057.93 2237.15,1058.38 2237.38,1058.13 2237.61,1058.32 2237.84,1058.54 2238.06,1058.68 2238.29,1058.65 2238.52,1058.85 2238.75,1058.88 2238.98,1058.46 2239.2,1058.91 2239.43,1059.04 2239.66,1058.65 2239.89,1058.64 2240.11,1058.1 2240.34,1058.41 2240.57,1058.73 2240.8,1058.59 2241.02,1059.04 2241.25,1058.92 2241.48,1059.06 2241.71,1058.48 2241.94,1058.46 2242.16,1057.83 2242.39,1058.04 2242.62,1058.58 2242.85,1058.28 2243.07,1058.1 2243.3,1057.38 2243.53,1057.13 2243.76,1057.15 2243.99,1057.05 2244.21,1057.45 2244.44,1057.69 2244.67,1056.85 2244.9,1057.17 2245.12,1057.42 2245.35,1056.74 2245.58,1056.77 2245.81,1056.66 2246.04,1056.32 2246.26,1056.28 2246.49,1056.11 2246.72,1056.15 2246.95,1056.29 2247.17,1056.81 2247.4,1056.76 2247.63,1057.18 2247.86,1057.22 2248.09,1056.73 2248.31,1056.28 2248.54,1055.88 2248.77,1055.62 2249,1055.81 2249.22,1055.62 2249.45,1056.03 2249.68,1055.97 2249.91,1055.9 2250.14,1055.49 2250.36,1055.59 2250.59,1055.9 2250.82,1055.94 2251.05,1055.59 2251.27,1055.77 2251.5,1056.01 2251.73,1056.58 2251.96,1056.77 2252.19,1056.82 2252.41,1056.85 2252.64,1057.02 2252.87,1056.81 2253.1,1057.12 2253.32,1057.55 2253.55,1057.49 2253.78,1057.7 2254.01,1057.73 2254.24,1057.83 2254.46,1058.1 2254.69,1058.24 2254.92,1058.16 2255.15,1058.53 2255.37,1058.21 2255.6,1058.11 2255.83,1058.53 2256.06,1058.68 2256.29,1058.57 2256.51,1058.93 2256.74,1058.79 2256.97,1058.27 2257.2,1058.59 2257.42,1058.5 2257.65,1058.65 2257.88,1058.69 2258.11,1057.87 2258.34,1057.59 2258.56,1057.51 2258.79,1057.46 2259.02,1057.72 2259.25,1057.57 2259.47,1057.83 2259.7,1057.7 2259.93,1057.86 2260.16,1056.94 2260.39,1057.05 2260.61,1056.9 2260.84,1056.97 2261.07,1056.92 2261.3,1056.03 2261.52,1055.6 2261.75,1055.33 2261.98,1055.29 2262.21,1055.23 2262.44,1054.55 2262.66,1054.58 2262.89,1055.18 2263.12,1054.92 2263.35,1055.21 2263.57,1055.1 2263.8,1055.05 2264.03,1055.12 2264.26,1055.45 2264.49,1055.49 2264.71,1055.7 2264.94,1055.84 2265.17,1055.64 2265.4,1055.66 2265.62,1055.89 2265.85,1056.22 2266.08,1055.95 2266.31,1056.06 2266.54,1055.92 2266.76,1056.05 2266.99,1056.1 2267.22,1056.6 2267.45,1056.36 2267.67,1056.16 2267.9,1056.02 2268.13,1055.93 2268.36,1056.46 2268.59,1056.48 2268.81,1056.44 2269.04,1056.22 2269.27,1055.96 2269.5,1056.02 2269.72,1056 2269.95,1055.67 2270.18,1055.68 2270.41,1055.79 2270.64,1055.73 2270.86,1055.36 2271.09,1055.33 2271.32,1055.68 2271.55,1055.5 2271.77,1055.84 2272,1055.91 2272.23,1056.34 2272.46,1056.25 2272.69,1056.16 2272.91,1056.17 2273.14,1056.1 2273.37,1056.31 2273.6,1056.34 2273.82,1056.06 2274.05,1056.22 2274.28,1056.51 2274.51,1056.15 2274.74,1055.87 2274.96,1055.67 2275.19,1055.7 2275.42,1055.25 2275.65,1055.48 2275.87,1055.7 2276.1,1055.22 2276.33,1055.3 2276.56,1054.88 2276.78,1054.81 2277.01,1054.35 2277.24,1054.7 2277.47,1054.51 2277.7,1054.34 2277.92,1054.41 2278.15,1054.09 2278.38,1054.11 2278.61,1054.68 2278.83,1054.74 2279.06,1055.05 2279.29,1054.81 2279.52,1054.79 2279.75,1055.09 2279.97,1054.98 2280.2,1054.83 2280.43,1055.01 2280.66,1054.78 2280.88,1054.94 2281.11,1054.47 2281.34,1054.65 2281.57,1055.19 2281.8,1055.09 2282.02,1054.8 2282.25,1054.39 2282.48,1053.43 2282.71,1053.45 2282.93,1053.37 2283.16,1053.52 2283.39,1052.55 2283.62,1052.48 2283.85,1052.34 2284.07,1052.37 2284.3,1052.48 2284.53,1052.04 2284.76,1051.96 2284.98,1051.61 2285.21,1051.45 2285.44,1051.01 2285.67,1051.08 2285.9,1050.97 2286.12,1050.82 2286.35,1050.63 2286.58,1050.29 2286.81,1050.32 2287.03,1050.65 2287.26,1050.67 2287.49,1050.59 2287.72,1050.47 2287.95,1049.99 2288.17,1049.81 2288.4,1049.91 2288.63,1050.48 2288.86,1050.51 2289.08,1050.77 2289.31,1051.21 2289.54,1051.57 2289.77,1051.46 2290,1051.5 2290.22,1051.75 2290.45,1051.73 2290.68,1051.66 2290.91,1051.47 2291.13,1051.63 2291.36,1051.54 2291.59,1051.72 2291.82,1051.5 2292.05,1051.64 2292.27,1052.16 2292.5,1052.23 2292.73,1051.88 2292.96,1051.93 2293.18,1051.99 2293.41,1051.86 2293.64,1051.42 2293.87,1051.17 2294.1,1051.12 2294.32,1051.46 2294.55,1051.25 2294.78,1051.05 2295.01,1050.66 2295.23,1050.95 2295.46,1050.56 2295.69,1050.06 2295.92,1049.89 2296.15,1049.85 2296.37,1050.17 2296.6,1049.98 2296.83,1049.52 2297.06,1049.86 2297.28,1049.98 2297.51,1049.71 2297.74,1049.64 2297.97,1049.67 2298.2,1050.02 2298.42,1050.08 2298.65,1050.27 2298.88,1050.22 2299.11,1050.5 2299.33,1050.35 2299.56,1050.32 2299.79,1050.72 2300.02,1050.35 2300.25,1049.89 2300.47,1050.34 2300.7,1050.54 2300.93,1050.61 2301.16,1050.79 2301.38,1050.88 2301.61,1050.58 2301.84,1050.11 2302.07,1050.33 2302.3,1049.64 2302.52,1049.6 2302.75,1049.5 2302.98,1049.9 2303.21,1049.87 2303.43,1049.91 2303.66,1049.47 2303.89,1049.62 2304.12,1049.58 2304.35,1049.87 2304.57,1049.72 2304.8,1049.89 2305.03,1050.4 2305.26,1050.53 2305.48,1050.69 2305.71,1050.77 2305.94,1050.18 2306.17,1050.34 2306.4,1050.76 2306.62,1050.69 2306.85,1050.82 2307.08,1050.97 2307.31,1050.58 2307.53,1050.23 2307.76,1049.91 2307.99,1050.22 2308.22,1049.75 2308.45,1049.74 2308.67,1049.68 2308.9,1049.71 2309.13,1050.08 2309.36,1050.66 2309.58,1050.57 2309.81,1050.86 2310.04,1050.62 2310.27,1050.7 2310.5,1050.68 2310.72,1050.5 2310.95,1050.08 2311.18,1049.89 2311.41,1049.52 2311.63,1049.11 2311.86,1049.6 2312.09,1049.81 2312.32,1049.9 2312.54,1049.96 2312.77,1049.93 2313,1049.6 2313.23,1049.97 2313.46,1049.96 2313.68,1050.14 2313.91,1050.02 2314.14,1049.72 2314.37,1049.36 2314.59,1049.25 2314.82,1049.16 2315.05,1048.91 2315.28,1048.76 2315.51,1048.79 2315.73,1048.85 2315.96,1048.67 2316.19,1048.86 2316.42,1048.9 2316.64,1048.17 2316.87,1048.36 2317.1,1048.41 2317.33,1048.8 2317.56,1049.08 2317.78,1048.46 2318.01,1048.31 2318.24,1048.41 2318.47,1048.57 2318.69,1048.73 2318.92,1048.83 2319.15,1048.61 2319.38,1048.19 2319.61,1048.42 2319.83,1048.32 2320.06,1048.52 2320.29,1048.44 2320.52,1048.37 2320.74,1048.82 2320.97,1048.89 2321.2,1048.97 2321.43,1048.53 2321.66,1048.1 2321.88,1048.38 2322.11,1048.02 2322.34,1047.67 2322.57,1047.56 2322.79,1047.46 2323.02,1047.15 2323.25,1047.39 2323.48,1047.8 2323.71,1048.04 2323.93,1047.95 2324.16,1048.53 2324.39,1048.52 2324.62,1048.62 2324.84,1048.69 2325.07,1048.96 2325.3,1048.9 2325.53,1049.16 2325.76,1049.22 2325.98,1049.44 2326.21,1049.41 2326.44,1049.51 2326.67,1049.7 2326.89,1049.74 2327.12,1049.41 2327.35,1049.68 2327.58,1049.32 2327.81,1048.95 2328.03,1049.08 2328.26,1049.37 2328.49,1049.6 2328.72,1049.69 2328.94,1050.06 2329.17,1049.6 2329.4,1049.49 2329.63,1049.98 2329.86,1050.12 2330.08,1050.32 2330.31,1050.45 2330.54,1050.69 2330.77,1050.52 2330.99,1050.55 2331.22,1050.81 2331.45,1050.95 2331.68,1051.3 2331.91,1051.38 2332.13,1051.51 2332.36,1051.33 2332.59,1051.43 2332.82,1051.07 2333.04,1051.1 2333.27,1051.09 2333.5,1050.34 2333.73,1050.38 2333.96,1050.28 2334.18,1050.14 2334.41,1050.06 2334.64,1050.15 2334.87,1050.24 2335.09,1050.02 2335.32,1049.89 2335.55,1050.14 2335.78,1050.15 2336.01,1049.84 2336.23,1049.94 2336.46,1050.09 2336.69,1049.89 2336.92,1049.98 2337.14,1049.81 2337.37,1050.12 2337.6,1050.18 2337.83,1050.32 2338.06,1050.4 2338.28,1050.44 2338.51,1050.09 2338.74,1049.98 2338.97,1050.05 2339.19,1049.56 2339.42,1049.15 2339.65,1049.32 2339.88,1049.3 2340.11,1049.15 2340.33,1049.42 2340.56,1048.9 2340.79,1048.69 2341.02,1048.68 2341.24,1049.12 2341.47,1048.99 2341.7,1048.96 2341.93,1048.72 2342.16,1048.95 2342.38,1049.06 2342.61,1049.4 2342.84,1049.42 2343.07,1049.61 2343.29,1049.53 2343.52,1049.68 2343.75,1049.21 2343.98,1049.35 2344.21,1049.23 2344.43,1048.98 2344.66,1048.77 2344.89,1048.83 2345.12,1049.02 2345.34,1048.89 2345.57,1048.93 2345.8,1048.94 2346.03,1048.98 2346.26,1049.04 2346.48,1048.82 2346.71,1048.47 2346.94,1048.57 2347.17,1048.22 2347.39,1048.43 2347.62,1048.68 2347.85,1048.59 2348.08,1048.62 2348.31,1048.59 2348.53,1048.43 2348.76,1048.55 2348.99,1048.12 2349.22,1048.59 2349.44,1047.98 2349.67,1048.18 2349.9,1048.47 2350.13,1048.4 2350.35,1048.71 2350.58,1049.12 2350.81,1049.16 2351.04,1049.15 2351.27,1048.95 2351.49,1049.33 2351.72,1049.7 2351.95,1049.65 2352.18,1050.04 2352.4,1050.58 2352.63,1050.46 2352.86,1050.38 2353.09,1050.74 2353.32,1050.44 2353.54,1050.37 2353.77,1050.58 2354,1051.04 2354.23,1051.1 2354.45,1050.86 2354.68,1050.6 2354.91,1050.28 2355.14,1050.13 2355.37,1050.36 2355.59,1050.08 2355.82,1049.89 2356.05,1049.79 2356.28,1049.73 2356.5,1050.15 2356.73,1049.89 2356.96,1049.78 2357.19,1049.89 2357.42,1049.35 2357.64,1049.84 2357.87,1049.77 2358.1,1049.97 2358.33,1050.08 2358.55,1050.16 2358.78,1050.3 2359.01,1050.13 2359.24,1050.19 2359.47,1049.82 2359.69,1049.85 2359.92,1050.31 2360.15,1050.1 2360.38,1049.72 2360.6,1049.76 2360.83,1049.74 2361.06,1050.21 2361.29,1050.43 2361.52,1050.61 2361.74,1050.81 2361.97,1050.52 2362.2,1050.84 2362.43,1051.02 2362.65,1051.38 2362.88,1051.51 2363.11,1051.65 2363.34,1051.37 2363.57,1051.29 2363.79,1051.32 2364.02,1051.29 2364.25,1051.32 2364.48,1051.41 2364.7,1051.57 2364.93,1051.88 2365.16,1051.95 2365.39,1052 2365.62,1052.3 2365.84,1052.48 2366.07,1052.1 2366.3,1052.26 2366.53,1052.42 2366.75,1052.11 2366.98,1052.25 2367.21,1052.12 2367.44,1052.14 2367.67,1052.19 2367.89,1052.27 2368.12,1052.69 2368.35,1052.26 2368.58,1052.44 2368.8,1052.03 2369.03,1051.92 2369.26,1052.29 2369.49,1052.4 2369.72,1052.59 2369.94,1052.81 2370.17,1053.01 2370.4,1052.44 2370.63,1051.15 2370.85,1050.9 2371.08,1051.3 2371.31,1051.01 2371.54,1051.15 2371.77,1051.45 2371.99,1051.8 2372.22,1051.83 2372.45,1052.32 2372.68,1052.84 2372.9,1053.37 2373.13,1053.7 2373.36,1053.55 2373.59,1053.26 2373.82,1053.67 2374.04,1053.85 2374.27,1053.88 2374.5,1053.78 2374.73,1053.69 2374.95,1053.2 2375.18,1053.51 2375.41,1053.43 2375.64,1053.61 2375.87,1053.31 2376.09,1053.35 2376.32,1053.27 2376.55,1053.42 2376.78,1053.54 2377,1053.46 2377.23,1053.45 2377.46,1053.59 2377.69,1053.27 2377.92,1053.3 2378.14,1053.16 2378.37,1052.89 2378.6,1052.12 2378.83,1052.12 2379.05,1052.32 2379.28,1052.27 2379.51,1052.05 2379.74,1051.85 2379.97,1051.55 2380.19,1051.73 2380.42,1051.92 2380.65,1052.03 2380.88,1051.65 2381.1,1051.47 2381.33,1051.78 2381.56,1051.42 2381.79,1051.02 2382.02,1050.89 2382.24,1051.3 2382.47,1050.95 2382.7,1050.37 2382.93,1049.87 2383.15,1049.42 2383.38,1049.79 2383.61,1049.53 2383.84,1049.8 2384.07,1050.24 2384.29,1050.07 2384.52,1049.98 2384.75,1049.91 2384.98,1050.22 2385.2,1050.25 2385.43,1050.42 2385.66,1050.5 2385.89,1050.21 2386.11,1049.48 2386.34,1049.32 2386.57,1049.36 2386.8,1049.48 2387.03,1049.81 2387.25,1049.86 2387.48,1050.01 2387.71,1049.68 2387.94,1049.94 2388.16,1049.31 2388.39,1049.37 2388.62,1049.63 2388.85,1049.2 2389.08,1049.29 2389.3,1048.95 2389.53,1049.04 2389.76,1049.03 2389.99,1049.35 2390.21,1049.53 2390.44,1049.56 2390.67,1049.86 2390.9,1050.07 2391.13,1050.34 2391.35,1050.63 2391.58,1050.55 2391.81,1049.83 2392.04,1049.72 2392.26,1049.28 2392.49,1049.53 2392.72,1049.81 2392.95,1049.98 2393.18,1050.41 2393.4,1050.41 2393.63,1050.12 2393.86,1049.71 2394.09,1049.99 2394.31,1049.89 2394.54,1050.19 2394.77,1050.09 2395,1050.15 2395.23,1049.53 2395.45,1049.73 2395.68,1050.17 2395.91,1050.14 2396.14,1050.59 2396.36,1050.72 2396.59,1050.87 2396.82,1050.76 2397.05,1051.05 2397.28,1050.74 2397.5,1050.86 2397.73,1050.38 2397.96,1050.52 2398.19,1050.47 2398.41,1050.51 2398.64,1050.78 2398.87,1051.08 2399.1,1051.19 2399.33,1051.05 2399.55,1050.46 2399.78,1050.68 2400.01,1050.83 2400.24,1050.43 2400.46,1050.18 2400.69,1050.49 2400.92,1050.27 2401.15,1050.47 2401.38,1050.43 2401.6,1050.98 2401.83,1050.97 2402.06,1051.36 2402.29,1051.56 2402.51,1051.33 2402.74,1051.19 2402.97,1050.99 2403.2,1051.56 2403.43,1051.58 2403.65,1051.55 2403.88,1051.99 2404.11,1052.32 2404.34,1052.14 2404.56,1052.28 2404.79,1052.16 2405.02,1052.05 2405.25,1052.25 2405.48,1052.7 2405.7,1052.54 2405.93,1052.43 2406.16,1052.69 2406.39,1053.26 2406.61,1053.21 2406.84,1053.19 2407.07,1053.61 2407.3,1053.97 2407.53,1054.1 2407.75,1054.23 2407.98,1054.14 2408.21,1053.68 2408.44,1053.92 2408.66,1054.23 2408.89,1054.18 2409.12,1053.83 2409.35,1053.6 2409.58,1053.37 2409.8,1053.6 2410.03,1053.65 2410.26,1053.69 2410.49,1053.61 2410.71,1053.47 2410.94,1053.5 2411.17,1053.49 2411.4,1053.65 2411.63,1053.83 2411.85,1054.45 2412.08,1054.74 2412.31,1054.89 2412.54,1054.91 2412.76,1054.45 2412.99,1054.56 2413.22,1054.77 2413.45,1054.96 2413.68,1054.71 2413.9,1054.5 2414.13,1054.48 2414.36,1054.85 2414.59,1055.43 2414.81,1055.09 2415.04,1054.81 2415.27,1054.91 2415.5,1054.88 2415.73,1055.05 2415.95,1054.65 2416.18,1054.78 2416.41,1055.13 2416.64,1055.37 2416.86,1055.16 2417.09,1055.26 2417.32,1055.31 2417.55,1055.33 2417.78,1055.79 2418,1055.62 2418.23,1056.14 2418.46,1056.17 2418.69,1055.84 2418.91,1056.12 2419.14,1056.03 2419.37,1056.16 2419.6,1056.08 2419.83,1056.44 2420.05,1056.37 2420.28,1056.44 2420.51,1056.54 2420.74,1055.96 2420.96,1055.67 2421.19,1055.82 2421.42,1055.16 2421.65,1055.24 2421.87,1054.79 2422.1,1054.41 2422.33,1054.47 2422.56,1054.46 2422.79,1054.13 2423.01,1054.29 2423.24,1054.68 2423.47,1054.62 2423.7,1054.54 2423.92,1055.03 2424.15,1055.32 2424.38,1055.28 2424.61,1055.23 2424.84,1055.33 2425.06,1055.49 2425.29,1055.07 2425.52,1055.22 2425.75,1055.42 2425.97,1055.49 2426.2,1055.83 2426.43,1055.86 2426.66,1056.01 2426.89,1055.6 2427.11,1055.8 2427.34,1055.64 2427.57,1055.58 2427.8,1055.43 2428.02,1055.41 2428.25,1055.62 2428.48,1055.93 2428.71,1056.19 2428.94,1055.72 2429.16,1055.87 2429.39,1055.66 2429.62,1055.54 2429.85,1055.83 2430.07,1055.97 2430.3,1055.58 2430.53,1055.86 2430.76,1056.12 2430.99,1056.03 2431.21,1055.87 2431.44,1056.26 2431.67,1056.01 2431.9,1055.85 2432.12,1056.04 2432.35,1056.29 2432.58,1056.19 2432.81,1055.86 2433.04,1056.04 2433.26,1056.01 2433.49,1056.58 2433.72,1056.62 2433.95,1056.75 2434.17,1056.53 2434.4,1056.34 2434.63,1056.72 2434.86,1056.55 2435.09,1056.55 2435.31,1055.92 2435.54,1055.77 2435.77,1055.87 2436,1055.96 2436.22,1056.25 2436.45,1056.45 2436.68,1055.77 2436.91,1056.1 2437.14,1056.32 2437.36,1056.14 2437.59,1056.06 2437.82,1056.21 2438.05,1055.85 2438.27,1055.8 2438.5,1055.71 2438.73,1055.38 2438.96,1055.45 2439.19,1055.65 2439.41,1055.68 2439.64,1055.31 2439.87,1055.21 2440.1,1055.22 2440.32,1054.99 2440.55,1055.01 2440.78,1054.83 2441.01,1055.25 2441.24,1055.56 2441.46,1055.61 2441.69,1055.71 2441.92,1055.41 2442.15,1055.06 2442.37,1054.86 2442.6,1054.25 2442.83,1054.37 2443.06,1054.5 2443.29,1054.38 2443.51,1054.36 2443.74,1054.66 2443.97,1054.74 2444.2,1054.34 2444.42,1054.46 2444.65,1054.41 2444.88,1054.34 2445.11,1054.82 2445.34,1054.54 2445.56,1054.43 2445.79,1054.31 2446.02,1053.98 2446.25,1053.71 2446.47,1053.89 2446.7,1054.22 2446.93,1054.26 2447.16,1054.44 2447.39,1054.55 2447.61,1054.44 2447.84,1054.41 2448.07,1054.81 2448.3,1054.54 2448.52,1054.47 2448.75,1054.87 2448.98,1054.99 2449.21,1055.41 2449.44,1055.45 2449.66,1055.61 2449.89,1055.61 2450.12,1055.22 2450.35,1055.69 2450.57,1054.81 2450.8,1054.82 2451.03,1054.43 2451.26,1054.4 2451.49,1054.44 2451.71,1054.47 2451.94,1054.32 2452.17,1054.6 2452.4,1054.69 2452.62,1054.04 2452.85,1054.33 2453.08,1054.09 2453.31,1054.47 2453.54,1054.42 2453.76,1054.83 2453.99,1054.69 2454.22,1055.06 2454.45,1054.93 2454.67,1054.47 2454.9,1053.95 2455.13,1053.54 2455.36,1053.41 2455.59,1053.66 2455.81,1053.51 2456.04,1053.55 2456.27,1053.93 2456.5,1054.1 2456.72,1054.35 2456.95,1054.54 2457.18,1054.15 2457.41,1054.43 2457.64,1054.16 2457.86,1054.06 2458.09,1054.43 2458.32,1054.55 2458.55,1054.33 2458.77,1054.02 2459,1054.4 2459.23,1054.53 2459.46,1054.65 2459.68,1054.08 2459.91,1054.07 2460.14,1053.73 2460.37,1053.14 2460.6,1053.6 2460.82,1053.92 2461.05,1054.17 2461.28,1053.7 2461.51,1054.17 2461.73,1054.35 2461.96,1054.63 2462.19,1054.6 2462.42,1054.84 2462.65,1054.59 2462.87,1054.46 2463.1,1054.43 2463.33,1054.62 2463.56,1054.44 2463.78,1054.81 2464.01,1054.18 2464.24,1054.5 2464.47,1054.23 2464.7,1054.28 2464.92,1054.59 2465.15,1054.79 2465.38,1054.84 2465.61,1054.78 2465.83,1054.75 2466.06,1054.38 2466.29,1054.86 2466.52,1054.6 2466.75,1054.49 2466.97,1054.75 2467.2,1054.89 2467.43,1054.51 2467.66,1054.56 2467.88,1054.04 2468.11,1054.28 2468.34,1054.04 2468.57,1054.08 2468.8,1054.45 2469.02,1054.33 2469.25,1054.51 2469.48,1054.83 2469.71,1054.54 2469.93,1054.16 2470.16,1054.34 2470.39,1053.77 2470.62,1053.8 2470.85,1053.55 2471.07,1053.44 2471.3,1053.61 2471.53,1053.68 2471.76,1053.76 2471.98,1053.4 2472.21,1053.15 2472.44,1053.32 2472.67,1053.41 2472.9,1053.44 2473.12,1053.71 2473.35,1053.32 2473.58,1053.82 2473.81,1054.3 2474.03,1054.47 2474.26,1054.07 2474.49,1054.03 2474.72,1053.84 2474.95,1054.09 2475.17,1054.06 2475.4,1053.96 2475.63,1054.06 2475.86,1054.03 2476.08,1053.24 2476.31,1053.32 2476.54,1053.18 2476.77,1053.33 2477,1052.99 2477.22,1053.23 2477.45,1053.64 2477.68,1053.58 2477.91,1053.92 2478.13,1054.25 2478.36,1054.17 2478.59,1054.42 2478.82,1054.83 2479.05,1054.8 2479.27,1054.82 2479.5,1054.93 2479.73,1055.15 2479.96,1054.85 2480.18,1055.08 2480.41,1055.21 2480.64,1055.29 2480.87,1055.08 2481.1,1055.13 2481.32,1054.83 2481.55,1055.18 2481.78,1055.31 2482.01,1055.07 2482.23,1054.69 2482.46,1054.61 2482.69,1053.94 2482.92,1054.54 2483.15,1054.11 2483.37,1054.49 2483.6,1054.93 2483.83,1055.41 2484.06,1055.11 2484.28,1054.91 2484.51,1054.37 2484.74,1054.7 2484.97,1054.67 2485.2,1054.71 2485.42,1054.99 2485.65,1054.78 2485.88,1054.75 2486.11,1054.22 2486.33,1054.51 2486.56,1054.53 2486.79,1054.68 2487.02,1054.63 2487.25,1054.57 2487.47,1054.42 2487.7,1054.55 2487.93,1055.11 2488.16,1055.26 2488.38,1054.82 2488.61,1054.73 2488.84,1055.02 2489.07,1055.67 2489.3,1055.62 2489.52,1055.32 2489.75,1055.61 2489.98,1055.76 2490.21,1055.38 2490.43,1055.62 2490.66,1055.64 2490.89,1055.05 2491.12,1055.47 2491.35,1055.44 2491.57,1055.46 2491.8,1055.27 2492.03,1055.32 2492.26,1054.82 2492.48,1055.05 2492.71,1055.4 2492.94,1055.49 2493.17,1055.95 2493.4,1056.13 2493.62,1055.83 2493.85,1055.2 2494.08,1055.25 2494.31,1054.9 2494.53,1054.53 2494.76,1054.52 2494.99,1054.66 2495.22,1054.25 2495.44,1054.47 2495.67,1054.22 2495.9,1054.36 2496.13,1054.47 2496.36,1054.42 2496.58,1054.72 2496.81,1055.01 2497.04,1054.93 2497.27,1055.2 2497.49,1055.33 2497.72,1055.73 2497.95,1055.06 2498.18,1054.38 2498.41,1054.4 2498.63,1054.39 2498.86,1054.14 2499.09,1054.03 2499.32,1053.88 2499.54,1053.92 2499.77,1053.74 2500,1053.94 2500.23,1053.72 2500.46,1054 2500.68,1053.84 2500.91,1053.74 2501.14,1053.89 2501.37,1053.43 2501.59,1053.17 2501.82,1053.14 2502.05,1052.96 2502.28,1053.13 2502.51,1051.98 2502.73,1051.8 2502.96,1052 2503.19,1052.46 2503.42,1052.03 2503.64,1052.26 2503.87,1051.9 2504.1,1052.26 2504.33,1052.37 2504.56,1052.25 2504.78,1052.13 2505.01,1051.91 2505.24,1052.14 2505.47,1052.29 2505.69,1051.59 2505.92,1051.62 2506.15,1051.68 2506.38,1051.57 2506.61,1051.36 2506.83,1051.53 2507.06,1051.32 2507.29,1051.36 2507.52,1050.56 2507.74,1050.55 2507.97,1050.44 2508.2,1050.28 2508.43,1050.16 2508.66,1050.37 2508.88,1050 2509.11,1049.88 2509.34,1049.58 2509.57,1049.62 2509.79,1049.79 2510.02,1049.84 2510.25,1049.8 2510.48,1049.22 2510.71,1049.52 2510.93,1049.37 2511.16,1049.36 2511.39,1049.27 2511.62,1049.32 2511.84,1049.26 2512.07,1049.83 2512.3,1049.7 2512.53,1049.07 2512.76,1048.96 2512.98,1048.81 2513.21,1048.16 2513.44,1047.89 2513.67,1047.94 2513.89,1048.04 2514.12,1048.48 2514.35,1048.11 2514.58,1047.74 2514.81,1047.73 2515.03,1047.71 2515.26,1047.64 2515.49,1048.08 2515.72,1048.3 2515.94,1048.72 2516.17,1049 2516.4,1049.06 2516.63,1049.06 2516.86,1048.93 2517.08,1048.73 2517.31,1048.87 2517.54,1048.94 2517.77,1048.99 2517.99,1049.13 2518.22,1049.14 2518.45,1048.76 2518.68,1048.8 2518.91,1049.07 2519.13,1048.98 2519.36,1049.21 2519.59,1049.26 2519.82,1049.89 2520.04,1050.08 2520.27,1050.22 2520.5,1050.27 2520.73,1050.16 2520.96,1050.44 2521.18,1050.72 2521.41,1051.13 2521.64,1051.13 2521.87,1050.89 2522.09,1050.65 2522.32,1050.2 2522.55,1050.22 2522.78,1049.45 2523.01,1049.52 2523.23,1049.71 2523.46,1049.87 2523.69,1049.53 2523.92,1049.91 2524.14,1050.1 2524.37,1049.98 2524.6,1049.91 2524.83,1049.47 2525.06,1049.09 2525.28,1048.83 2525.51,1048.79 2525.74,1048.86 2525.97,1049.34 2526.19,1049.56 2526.42,1049.62 2526.65,1049.77 2526.88,1049.6 2527.11,1049.68 2527.33,1049.95 2527.56,1050.1 2527.79,1050.32 2528.02,1050.58 2528.24,1050.65 2528.47,1050.99 2528.7,1051.01 2528.93,1051.36 2529.16,1051.71 2529.38,1051.7 2529.61,1052.22 2529.84,1052.64 2530.07,1052.33 2530.29,1052.94 2530.52,1053.22 2530.75,1053.32 2530.98,1053.65 2531.2,1053.71 2531.43,1053.06 2531.66,1052.78 2531.89,1052.57 2532.12,1052.27 2532.34,1052.36 2532.57,1052.16 2532.8,1052.13 2533.03,1052.33 2533.25,1052.12 2533.48,1052.22 2533.71,1052.12 2533.94,1052.19 2534.17,1052.37 2534.39,1052.68 2534.62,1053.04 2534.85,1052.8 2535.08,1053.16 2535.3,1052.73 2535.53,1053.1 2535.76,1053.04 2535.99,1052.73 2536.22,1052.48 2536.44,1052.82 2536.67,1052.64 2536.9,1052.53 2537.13,1052.39 2537.35,1052.24 2537.58,1052.07 2537.81,1052.01 2538.04,1051.64 2538.27,1051.77 2538.49,1051.97 2538.72,1052.41 2538.95,1052.73 2539.18,1052.5 2539.4,1052.33 2539.63,1052.18 2539.86,1052.33 2540.09,1051.99 2540.32,1051.79 2540.54,1052 2540.77,1052.02 2541,1051.94 2541.23,1051.96 2541.45,1051.69 2541.68,1051.34 2541.91,1051.45 2542.14,1051.69 2542.37,1051.88 2542.59,1051.43 2542.82,1051.4 2543.05,1051.29 2543.28,1051.1 2543.5,1051.22 2543.73,1051.13 2543.96,1050.73 2544.19,1050.89 2544.42,1050.94 2544.64,1051.36 2544.87,1051.64 2545.1,1051.38 2545.33,1051.72 2545.55,1051.46 2545.78,1051.63 2546.01,1051.71 2546.24,1052.02 2546.47,1052.24 2546.69,1052.59 2546.92,1052.26 2547.15,1052.3 2547.38,1052.48 2547.6,1052.39 2547.83,1052.32 2548.06,1052.44 2548.29,1052.68 2548.52,1052.72 2548.74,1052.9 2548.97,1053.1 2549.2,1053.21 2549.43,1053.34 2549.65,1053.24 2549.88,1053.43 2550.11,1053.89 2550.34,1053.9 2550.57,1054.03 2550.79,1054.5 2551.02,1054.23 2551.25,1054.32 2551.48,1054.44 2551.7,1054.6 2551.93,1054.14 2552.16,1054.37 2552.39,1054.47 2552.62,1054.42 2552.84,1054.32 2553.07,1054 2553.3,1054.15 2553.53,1054.6 2553.75,1054.82 2553.98,1054.69 2554.21,1054.73 2554.44,1055.13 2554.67,1055.05 2554.89,1055.31 2555.12,1054.89 2555.35,1055.07 2555.58,1055.49 2555.8,1055.87 2556.03,1055.92 2556.26,1056.41 2556.49,1056.45 2556.72,1056.06 2556.94,1056.15 2557.17,1055.9 2557.4,1056.19 2557.63,1055.98 2557.85,1056.28 2558.08,1055.94 2558.31,1056.02 2558.54,1055.76 2558.77,1055.7 2558.99,1055.79 2559.22,1055.78 2559.45,1056.08 2559.68,1056.53 2559.9,1056.56 2560.13,1056.94 2560.36,1057.29 2560.59,1056.91 2560.82,1056.93 2561.04,1056.9 2561.27,1056.51 2561.5,1056.84 2561.73,1056.99 2561.95,1056.97 2562.18,1056.91 2562.41,1056.88 2562.64,1056.69 2562.87,1056.61 2563.09,1055.63 2563.32,1055.74 2563.55,1055.75 2563.78,1055.66 2564,1055.66 2564.23,1055.13 2564.46,1054.81 2564.69,1054.76 2564.92,1054.86 2565.14,1055.13 2565.37,1055.31 2565.6,1055.79 2565.83,1055.48 2566.05,1055.31 2566.28,1055.69 2566.51,1055.7 2566.74,1055.68 2566.96,1055.67 2567.19,1055.52 2567.42,1055.04 2567.65,1055.32 2567.88,1055.18 2568.1,1055.12 2568.33,1054.75 2568.56,1054.68 2568.79,1054.8 2569.01,1054.32 2569.24,1054.41 2569.47,1054.38 2569.7,1054.14 2569.93,1054.13 2570.15,1054.52 2570.38,1054.27 2570.61,1054.25 2570.84,1054.37 2571.06,1053.94 2571.29,1053.17 2571.52,1053.33 2571.75,1053.31 2571.98,1053.31 2572.2,1053.01 2572.43,1053.41 "/>
</svg>

Figure 4: Monte Carlo mean and 95% confidence interval of the simulated
temperature in 2100 as the number of samples increases.

1.  Here we can see the benefits of setting the variable `n_samples`
    above; if we had hard-coded the value and needed to change it, we
    would also have needed to change it in all three of these lines.
2.  There are more efficient ways to do this updating, for example
    `T_est[i] = (T_est[i-1] * (i-1) + T_est[i]) / i`, which avoids
    re-computing the average for the first `i-1` simulations, but in
    this case it’s fast enough.
3.  We can’t compute the standard deviation for a single value, so we
    just start with `i == 2`.

If we wanted tighter confidence intervals, we could run the simulation
longer (though recall the $1/\sqrt{n}$ error law, which means to get an
order of magnitude reduction, we’d need approximately 100,000 samples),
but it looks from <a href="#fig-mc-est" class="quarto-xref">Figure 4</a>
as though the estimate had stabilized around 8,000-9,000 samples, though
ultimately it depends on the desired level of precision in both the
estimate and the confidence interval. In this case, regardless, we’re
looking at an estimated temperature anomaly of around $4.7^\circ C$ once
we exceed 5,000 samples, and it isn’t clear what more precision
necessarily would get us.

### Problem 3 (10 points)

**This problem is only required for students in BEE 5750**.

A factory is planning a third wastewater discharge into the river
downstream of the second plant. This discharge would consist of 5
m<sup>3</sup>/day of wastewater with a dissolved oxygen content of 4.5
mg/L and CBOD and NBOD levels of 50 and 45 mg/L, respectively.

**In this problem**:

-   Assume that the treatment plan for waste stream 2 that you
    identified in Problem 1 is still in place for the existing
    discharges. If the third discharge will not be treated, under the
    original inflow conditions (7.5 mg/L DO), how far downstream from
    the second discharge does this third discharge need to be placed to
    keep the river concentration from dropping below 4 mg/L?

#### Solution

Based on how we solved Problem 1, we need to modify our solution to
include a third discharge with an unknown distance from the second
discharge.

> **Tip**
>
> We could have solved Problem 1 in a more modular way, which included
> multiple discharges (using a vector of tuples instead of explicitly
> specifying `waste1` and `waste2`, which would have made this solution
> simpler).

``` julia
# do_simulate: function to simulate dissolved oxygen concentrations over the entire river
# inputs:
#   - d: distance between waste stream 2 and waste stream 3
#   - inflow: tuple with inflow properties: (Volume, DO, CBOD, NBOD)
#   - waste1: tuple with waste stream 1 properties: (Volume, DO, CBOD, NBOD)
#   - waste2: tuple with waste stream 2 properties: (Volume, DO, CBOD, NBOD)
#   - waste3: tuple with waste stream 3 properties: (Volume, DO, CBOD, NBOD)
#   - C₀, B₀, N₀: initial conditions for DO, CBOD, and NBOD, respectively (mg/L)
#   - U: velocity of river (km/d)
#   - Cₛ = saturation oxygen concentration (mg/L)
#   - ka, kc, kn: reaeration, CBOD decay, and NBOD decay rates, respectively (d^{-1})
function do_simulate2(d, inflow, waste1, waste2, waste3, U, Cₛ, ka, kc, kn)
    # set up ranges for each box/segment
    x₁ = 0:1:15
    x₂ = (15:1:15+d) .- 15 
    x₃ = (15+d:1:50) .- (15 + d)


    V_inflow, C_inflow, B_inflow, N_inflow = inflow
    V_ws1, C_ws1, B_ws1, N_ws1 = waste1
    V_ws2, C_ws2, B_ws2, N_ws2 = waste2
    V_ws3, C_ws3, B_ws3, N_ws3 = waste2

    # initialize storage for final C, B, and N
    # need to store d=1 so the total length should be d+1
    C = zeros(51)
    B = zeros(51)
    N = zeros(51)

    # compute initial conditions for first segment
    C₀ = mix_concentration(V_inflow, C_inflow, V_ws1, C_ws1)
    B₀ = mix_concentration(V_inflow, B_inflow, V_ws1, B_ws1)
    N₀ = mix_concentration(V_inflow, N_inflow, V_ws1, N_ws1)

    # conduct first segment simulation
    (C₁, B₁, N₁) = dissolved_oxygen(x₁, C₀, B₀, N₀, U, Cₛ, ka, kc, kn)
    C[1:15] = C₁[1:end-1] 
    B[1:15] = B₁[1:end-1]
    N[1:15] = N₁[1:end-1]

    # compute initial conditions for second segment
    C₀₁ = mix_concentration(V_inflow + V_ws1, C₁[end], V_ws2, C_ws2)
    B₀₁ = mix_concentration(V_inflow + V_ws1, B₁[end], V_ws2, B_ws2)
    N₀₁ = mix_concentration(V_inflow + V_ws1, N₁[end], V_ws2, N_ws2)

    # conduct second segment simulation
    (C₂, B₂, N₂) = dissolved_oxygen(x₂, C₀₁, B₀₁, N₀₁, U, Cₛ, ka, kc, kn)
    C[16:16+d] = C₂ 
    B[16:16+d] = B₂
    N[16:16+d] = N₂

    # compute initial conditions for third segment
    C₀₂ = mix_concentration(V_inflow + V_ws1 + V_ws2, C₂[end], V_ws3, C_ws3)
    B₀₂ = mix_concentration(V_inflow + V_ws1 + V_ws2, B₂[end], V_ws3, B_ws3)
    N₀₂ = mix_concentration(V_inflow + V_ws1 + V_ws2, N₂[end], V_ws3, N_ws3)

    # conduct second segment simulation
    (C₃, B₃, N₃) = dissolved_oxygen(x₃, C₀₂, B₀₂, N₀₂, U, Cₛ, ka, kc, kn)
    C[16+d:end] = C₃ 
    B[16+d:end] = B₃
    N[16+d:end] = N₃


    return (C, B, N)
end

# set up treatment level for waste stream 2 based on solution for Problem 1
B_ws2 = (1 - 0.15) * 45.0
N_ws2 = (1 - 0.15) * 35.0
waste2 = (V_ws2, C_ws2, B_ws2, N_ws2)

# set parameters for waste stream 3
C_ws3 = 4.5
B_ws3 = 50.0
N_ws3 = 45.0
V_ws3 = 5.0 * 1_000 # convert from m^3/day to L/day
waste3 = (V_ws3, C_ws3, B_ws3, N_ws3)
```

    (5000.0, 4.5, 50.0, 45.0)

Now we can evaluate `do_simulate2` over the different values of `d` to
find the minimum value of `C`.

``` julia
# three_discharge_minC: function to find the minimum DO concentration based on the three discharge model
# inputs:
#   - d: distance between waste stream 2 and waste stream 3
#   - inflow: tuple with inflow properties: (Volume, DO, CBOD, NBOD)
#   - waste1: tuple with waste stream 1 properties: (Volume, DO, CBOD, NBOD)
#   - waste2: tuple with waste stream 2 properties: (Volume, DO, CBOD, NBOD)
#   - waste3: tuple with waste stream 3 properties: (Volume, DO, CBOD, NBOD)
#   - C₀, B₀, N₀: initial conditions for DO, CBOD, and NBOD, respectively (mg/L)
#   - U: velocity of river (km/d)
#   - Cₛ = saturation oxygen concentration (mg/L)
#   - ka, kc, kn: reaeration, CBOD decay, and NBOD decay rates, respectively (d^{-1})
function three_discharge_minC(d, inflow, waste1, waste2, waste3, U, Cₛ, ka, kc, kn)
    C, B, N = do_simulate2(d, inflow, waste1, waste2, waste3, U, Cₛ, ka, kc, kn)
    return minimum(C)
end

# evaluate over all distances from 1 to 35 m downstream using broadcasting and anonymous functions
d = 1:1:35
minC_d = (dist -> three_discharge_minC(dist, inflow, waste1, waste2, waste3, U, Cₛ, ka, kc, kn)).(d)
idx = findfirst(minC_d .>= 4.0)
min_dist = d[idx]
@show min_dist;
```

    min_dist = 14

So, without treatment, the third discharge should be placed 14 m
downstream from the second discharge.

## References

List any external references consulted, including classmates.